In [ ]:
import math
import numpy as np
import cv2 as cv
import random as rd
import os
import time

def image_none(img_path):
    image=cv.imread(img_path)
    return image

def show_image(image):
    cv.imshow('image', image) 
    cv.waitKey(0)
    cv.destroyAllWindows()

def euclidean_distance(node1, node2):
    return math.sqrt((node1[0] - node2[0])**2 + (node1[1] - node2[1])**2)

def manhattan_distance(node1, node2):
    return abs(node1[0] - node2[0]) + abs(node1[1] - node2[1])
  

def octile_distance(node1,node2):
    dx= node2[0] - node1[0]
    dy= node2[1] - node1[1]
    f = math.sqrt(2) - 1
    if dx < dy:
        return f * dx + dy
    else:
        return f * dy + dx

def coloring_pixel(image,color,pixel):
    if image is  not None:
        image[pixel[0],pixel[1]]=color
        return image

def define_start_goal(image,start,goal):
    start_color = [255, 0, 102] #pink 
    goal_color =  [0, 0, 204] #blue
    if image is not None:
        # should not be obstacle 
        if np.array_equal(image[start[0],start[1]],[229, 229, 229]) and np.array_equal(image[goal[0],goal[1]],[229, 229, 229]):
            image = coloring_pixel(image,start_color,start)
            image = coloring_pixel(image,goal_color,goal) 
            return image
        else: 
            print("Goal or Start is an obstacle, Redefine it!")
            return image

def draw_path(img_path,path):    
    image=cv.imread(img_path)
    if image is not None:
        for tuple in path:
            print(tuple)
            image[tuple[1],tuple[0]]=[0, 0, 255] #red
    return image

def save_image(img_path,path,image_name):
    image=cv.imread(img_path)
    new_image = draw_path(image,path)
    saving_path = '/home/aisha/PFE/implementations/UAV-Path-Planning/src/tests/output/'
    cv.imwrite(os.path.join(saving_path , image_name), new_image)    

def isValid(img_path,node):
    image=cv.imread(img_path,0)
    if image is not None:
        return  image[node[0],node[1]] == 229 



def imageToMatrix(img_path):
    image=cv.imread(img_path,0)
    rows,cols=(1073,1073)    
    # image (grey scale) to a matrix of 0 and 1 
    for i in range(rows):
        for j in range(cols):
            if image[i,j] == 0 or image[i,j] == 99: 
                image[i,j] = 0 #obstacle
            elif image[i,j] == 229:
                image[i,j] = 1
    return image                
    


In [ ]:

""" output structure: (room_name,method used, type de test, path, path length, operations, time)"""
class room_path:
    def __init__(self,room_name,method,test,path,path_length,operations,time):
        self.room_name=room_name
        self.method=method
        self.test=test
        self.path=path
        self.path_length=path_length
        self.operations=operations
        self.time=time
    def path_info(self):
        path_info = "room name : " + str(self.room_name) + ", method used : " +  self.method + ", test : " + self.test + ", path : " + str(self.path) + ", path length : " + str(self.path_length) + ", number of operations : " + str(self.operations) + ", time : " + str(self.time)
        return path_info
        


def findNeighbors(matrix,node):
    #check validity and return neighbors to choose randomly one of them
    neighbors=[]

    if matrix[node[0]+1,node[1]] == 1:
        neighbor = (node[0]+1,node[1])
        neighbors.append(neighbor)

    if matrix[node[0],node[1]+1]==1:
        neighbor = (node[0],node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]+1,node[1]+1]==1:
        neighbor = (node[0]+1,node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]]==1:
        neighbor = (node[0]-1,node[1])
        neighbors.append(neighbor)
    
    if matrix[node[0],node[1]-1]==1:
        neighbor = (node[0],node[1]-1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]-1]==1:
        neighbor = (node[0]-1,node[1]-1)
        neighbors.append(neighbor)

    if matrix[node[0]-1,node[1]+1]==1:
        neighbor = (node[0]-1,node[1]+1)
        neighbors.append(neighbor)

    if matrix[node[0]+1,node[1]-1]==1:
        neighbor = (node[0]+1,node[1]-1)
        neighbors.append(neighbor) 

    return neighbors


def generatePath(matrix,start,end): 
    path=[]
    path.append(start)
    node=start 
    while(node!=end):
        neighbors = findNeighbors(matrix,node)
        node=neighbors[rd.randint(0,len(neighbors)-1)]
        path.append(node)
    return path




def generateNeighboringPath(matrix,path): 
    x=rd.randint(0,len(path)-2)
    neighbors=findNeighbors(matrix,path[x])
    output=path.copy()
    output[x]=neighbors[rd.randint(0,len(neighbors)-2)] 
    output[x+1:]=generatePath(matrix,output[x+1],output[-1])
    return output


def cost(path):
    distance=0
    for i in range(len(path)-1):
        distance+=octile_distance(path[i],path[i+1])
    return distance

# Initializing parameters


def sa(matrix,start,end):  
    CT = TI   # Current temperature
    CP = generatePath(matrix,start,end) # current path (random)
    CL = cost(CP) 
    OP = CP # optimal path
    OL = CL
    for i in range(N):
        print("i= ",i)
        for j in range(NT):
            RP = generateNeighboringPath(matrix,CP)
            RL = cost(RP)  
            if RL < CL:
                CP = RP
                CL = RL 
            else: 
                p = math.exp(-(RL-CL)/CT) # Transition Probability 
                r=rd.randint(0,1)
                if r<p:
                    CP = RP
                    CL = RL 
            if CL < OL:
                OP = CP
                OL = CL
        CT *= alpha 
        if CT <= FT:
            break 
    return OP , OL 


# Save image
def SA(img_path,start,end,image_name):
    image=imageToMatrix(img_path)
    op , ol = sa(image,start,end) 
    print("optimal path " , op , "path length " , ol)     
    #Draw path in the image and save it
    image=cv.imread(img_path)
    new_image = draw_path(img_path,op) 
    saving_path = '/home/aisha/PFE/implementations/UAV-Path-Planning/src/tests/output/output_sa'
    cv.imwrite(os.path.join(saving_path ,image_name), new_image)
    # FIXME : Make start and end with different colors 
    return op, ol 

#TEST
IT = [ 2,6,9]
FT = [ 1,2,3 ]
N =[ 5,8] 
NT = [ 5,8] 
alpha = [ 0.9 , 0.3 ]
images=["8room_000.png","16room_000.png","32room_000.png","64room_000.png"]
folder_dir="/content/"

start = (133, 30)
end = (21, 201)
easy_output =[]

for it in IT:
    for ft in FT:
        for n in N:
            for nt in NT:
                for a in alpha:
                #test the 4 images and save results    
                    for image in images:
                       # Initializing parameters
                        parameters = {}
                        parameters['IT'] = it     # Initial temperature
                        parameters['FT'] = ft     # Final temperature 
                        parameters['N']  = n      # Number of iterations
                        parameters['NT']= nt       # Number of iterations per Temperature 
                        parameters['alpha'] = a  # Geometric Coefficient alpha

                        img_path=folder_dir+"/"+str(image)
                        img=cv.imread(img_path,0)
                        if img is not None:
                            if isValid(img_path,start):
                                if isValid(img_path,end):
                                    #print(str(image))
                                    start_time = time.time()
                                    print(img)
                                    path , length = SA(img_path,start,end,"new_"+str(image),parameters)
                                    #print("--- %s seconds ---" % (time.time() - start_time))
                                    easy_output.append(room_path("new_"+str(image),"SA","easy",path , length,time.time() - start_time))
                                else:
                                    print(str(image) ,"invalid end")
                            else:
                                print(str(image),"invalid start")
                        else:
                            print("image is None")

print("length (easy output) : ",len(easy_output))        
for i in range(len(easy_output)):
    pathInfo = easy_output[i].path_info()
    print(pathInfo)


(21, 201)
([(133, 30), (134, 31), (133, 31), (134, 30), (133, 30), (132, 31), (132, 32), (131, 31), (131, 32), (130, 30), (131, 29), (130, 29), (129, 30), (129, 31), (128, 30), (128, 29), (130, 32), (129, 32), (133, 32), (134, 33), (133, 33), (132, 30), (131, 30), (129, 29), (130, 28), (130, 27), (131, 26), (130, 26), (129, 25), (128, 25), (128, 24), (127, 24), (127, 25), (127, 26), (128, 27), (128, 26), (129, 26), (129, 27), (131, 27), (132, 27), (131, 25), (130, 25), (127, 23), (128, 23), (127, 22), (126, 23), (126, 24), (126, 26), (125, 27), (124, 28), (125, 28), (124, 29), (125, 30), (125, 29), (126, 28), (127, 28), (123, 30), (122, 30), (121, 31), (122, 31), (122, 32), (122, 33), (122, 34), (123, 34), (123, 35), (122, 36), (124, 34), (125, 33), (126, 34), (125, 35), (123, 32), (123, 31), (124, 30), (125, 31), (125, 32), (124, 32), (123, 33), (121, 33), (121, 32), (120, 32), (119, 31), (118, 31), (117, 31), (117, 30), (118, 29), (118, 30), (119, 30), (119, 29), (117, 32), (116, 31), (116, 30), (115, 32), (115, 31), (114, 30), (114, 31), (113, 31), (113, 30), (112, 31), (111, 30), (111, 29), (111, 31), (110, 31), (112, 29), (113, 28), (114, 28), (115, 29), (115, 28), (116, 27), (117, 28), (117, 29), (116, 29), (116, 26), (117, 26), (118, 26), (118, 25), (117, 25), (117, 27), (116, 28), (118, 28), (119, 27), (118, 27), (119, 26), (120, 25), (121, 25), (122, 24), (122, 23), (123, 23), (122, 22), (122, 21), (123, 20), (122, 20), (123, 21), (123, 22), (124, 21), (124, 22), (124, 23), (123, 24), (124, 24), (121, 24), (120, 24), (123, 25), (125, 21), (124, 20), (125, 19), (124, 18), (124, 19), (125, 20), (123, 19), (123, 18), (126, 20), (126, 21), (127, 21), (125, 22), (121, 22), (120, 23), (121, 23), (119, 24), (118, 24), (117, 24), (116, 24), (117, 23), (116, 23), (116, 22), (117, 22), (117, 21), (115, 22), (114, 23), (114, 22), (115, 21), (116, 21), (115, 20), (114, 20), (115, 23), (113, 22), (116, 20), (116, 19), (116, 18), (115, 17), (114, 16), (114, 15), (113, 16), (115, 16), (115, 18), (114, 19), (113, 20), (113, 21), (115, 19), (117, 18), (117, 19), (118, 18), (119, 19), (119, 20), (120, 21), (121, 20), (121, 21), (120, 20), (121, 19), (122, 19), (122, 18), (124, 17), (123, 16), (123, 17), (124, 16), (125, 18), (125, 17), (122, 16), (121, 17), (121, 16), (122, 15), (123, 15), (123, 14), (122, 14), (122, 13), (121, 13), (120, 12), (120, 13), (121, 12), (122, 11), (123, 12), (124, 11), (125, 11), (126, 11), (126, 10), (127, 10), (128, 9), (129, 10), (127, 11), (126, 12), (127, 13), (128, 13), (127, 12), (126, 13), (127, 14), (126, 14), (125, 14), (125, 15), (128, 15), (129, 16), (129, 17), (128, 17), (127, 16), (126, 17), (126, 19), (126, 18), (125, 23), (122, 25), (119, 25), (120, 27), (120, 26), (118, 23), (118, 22), (119, 21), (118, 21), (120, 19), (121, 18), (122, 17), (120, 17), (120, 18), (127, 20), (127, 19), (128, 20), (129, 21), (130, 21), (129, 20), (128, 22), (128, 19), (130, 19), (130, 18), (131, 18), (132, 19), (131, 20), (132, 21), (133, 22), (133, 21), (132, 22), (131, 21), (130, 20), (131, 22), (132, 23), (134, 20), (133, 20), (134, 21), (133, 19), (132, 20), (130, 22), (130, 23), (133, 23), (134, 23), (134, 22), (132, 18), (132, 17), (132, 16), (133, 17), (133, 18), (131, 19), (130, 17), (131, 17), (134, 18), (134, 17), (134, 16), (134, 19), (135, 21), (133, 15), (134, 15), (133, 16), (133, 14), (134, 13), (133, 12), (132, 13), (132, 14), (133, 13), (134, 12), (134, 11), (133, 11), (132, 11), (131, 12), (130, 11), (129, 11), (130, 10), (131, 9), (131, 8), (132, 7), (131, 7), (132, 6), (133, 7), (132, 5), (133, 6), (134, 7), (134, 8), (134, 9), (133, 8), (132, 8), (130, 7), (130, 8), (129, 6), (128, 5), (127, 5), (128, 6), (127, 6), (126, 6), (127, 7), (126, 7), (125, 7), (125, 6), (130, 5), (131, 5), (131, 6), (130, 6), (129, 8), (128, 8), (129, 7), (128, 7), (125, 8), (133, 5), (133, 10), (134, 10), (133, 9), (132, 10), (132, 9), (131, 10), (130, 9), (131, 11), (132, 12), (131, 13), (132, 15), (131, 14), (131, 15), (134, 14), (130, 14), (130, 13), (129, 13), (129, 14), (129, 15), (128, 16), (126, 15), (125, 16), (124, 14), (127, 15), (127, 17), (127, 18), (126, 16), (130, 15), (125, 13), (128, 14), (128, 12), (130, 12), (129, 12), (125, 12), (130, 16), (124, 12), (123, 11), (122, 12), (121, 11), (120, 10), (119, 10), (118, 11), (118, 10), (117, 10), (116, 11), (117, 11), (116, 12), (115, 11), (115, 10), (114, 9), (113, 8), (112, 9), (113, 7), (112, 8), (111, 7), (112, 7), (111, 6), (111, 5), (112, 5), (113, 5), (113, 6), (113, 9), (114, 8), (114, 6), (115, 6), (116, 6), (116, 5), (117, 7), (116, 8), (117, 8), (116, 7), (115, 8), (116, 9), (116, 10), (115, 13), (115, 12), (114, 13), (114, 12), (113, 11), (113, 12), (113, 13), (112, 12), (114, 11), (114, 10), (115, 9), (117, 9), (117, 12), (117, 13), (118, 12), (119, 11), (119, 12), (118, 13), (118, 14), (119, 15), (119, 14), (119, 13), (118, 9), (119, 8), (118, 7), (114, 7), (115, 7), (112, 6), (110, 6), (110, 5), (109, 6), (108, 5), (107, 5), (109, 5), (110, 7), (109, 8), (108, 9), (109, 10), (110, 11), (109, 12), (110, 13), (109, 14), (108, 15), (108, 14), (109, 13), (110, 12), (108, 13), (109, 11), (108, 10), (107, 10), (107, 11), (109, 9), (110, 8), (108, 7), (107, 8), (106, 7), (105, 6), (106, 6), (106, 5), (107, 6), (107, 7), (104, 7), (104, 8), (104, 9), (105, 9), (105, 10), (105, 11), (104, 12), (103, 13), (103, 14), (104, 15), (105, 14), (105, 15), (105, 16), (105, 17), (104, 18), (103, 19), (103, 18), (105, 18), (104, 19), (104, 17), (106, 16), (106, 14), (107, 13), (106, 13), (105, 13), (107, 15), (107, 16), (108, 16), (108, 17), (109, 17), (107, 17), (106, 17), (107, 18), (108, 19), (106, 18), (105, 19), (106, 19), (106, 15), (104, 16), (103, 16), (104, 14), (102, 12), (101, 11), (100, 12), (101, 10), (101, 12), (100, 13), (101, 14), (101, 13), (103, 11), (104, 11), (104, 10), (103, 10), (103, 9), (105, 8), (105, 7), (106, 8), (103, 6), (103, 7), (104, 6), (102, 6), (101, 6), (100, 6), (101, 5), (100, 7), (99, 6), (98, 5), (98, 6), (97, 5), (97, 6), (96, 6), (95, 6), (95, 7), (94, 6), (94, 7), (95, 8), (95, 9), (95, 10), (96, 11), (97, 11), (97, 12), (98, 11), (99, 12), (99, 13), (100, 14), (101, 15), (102, 16), (103, 15), (102, 20), (102, 21), (101, 22), (100, 22), (100, 23), (100, 24), (99, 24), (98, 25), (98, 24), (97, 24), (96, 25), (98, 23), (97, 22), (96, 21), (95, 20), (94, 19), (94, 20), (95, 18), (94, 18), (93, 17), (92, 16), (92, 15), (91, 14), (90, 13), (90, 14), (89, 14), (90, 15), (91, 13), (92, 12), (92, 11), (91, 12), (90, 12), (91, 11), (91, 10), (92, 9), (93, 10), (93, 8), (95, 5), (96, 7), (96, 8), (97, 9), (97, 8), (98, 8), (99, 9), (98, 10), (98, 13), (99, 14), (99, 15), (98, 15), (98, 14), (102, 11), (103, 12), (104, 13), (102, 13), (102, 14), (100, 15), (99, 16), (97, 15), (96, 14), (97, 16), (96, 15), (96, 16), (95, 16), (94, 17), (93, 18), (92, 18), (91, 17), (91, 18), (92, 19), (92, 17), (93, 16), (93, 15), (93, 14), (94, 14), (94, 13), (93, 13), (93, 12), (94, 12), (94, 11), (93, 11), (95, 11), (94, 10), (92, 10), (95, 13), (95, 12), (96, 12), (99, 10), (98, 9), (97, 7), (99, 8), (100, 8), (101, 9), (101, 8), (100, 9), (102, 7), (105, 5), (104, 5), (106, 9), (103, 8), (101, 7), (105, 12), (106, 12), (107, 12), (108, 12), (106, 10), (110, 14), (110, 10), (111, 12), (111, 13), (112, 11), (111, 11), (112, 10), (111, 8), (114, 14), (113, 14), (115, 14), (116, 14), (115, 15), (116, 16), (116, 15), (116, 17), (117, 17), (117, 16), (118, 15), (120, 16), (119, 17), (120, 15), (119, 16), (118, 16), (118, 17), (120, 14), (114, 21), (112, 21), (111, 22), (110, 22), (111, 21), (111, 23), (110, 23), (109, 22), (108, 21), (109, 23), (110, 24), (111, 24), (111, 25), (111, 26), (110, 26), (112, 27), (112, 26), (113, 26), (113, 27), (114, 27), (115, 27), (115, 26), (114, 26), (116, 25), (115, 25), (119, 28), (120, 28), (121, 28), (121, 27), (120, 29), (119, 32), (119, 33), (120, 33), (120, 34), (119, 34), (118, 32), (120, 31), (118, 34), (118, 33), (116, 32), (116, 33), (116, 34), (115, 35), (117, 34), (118, 35), (119, 36), (119, 37), (118, 38), (119, 39), (118, 40), (117, 39), (117, 38), (118, 37), (117, 37), (117, 36), (116, 37), (116, 36), (115, 36), (115, 37), (114, 36), (117, 35), (118, 36), (120, 36), (120, 37), (118, 39), (117, 40), (118, 41), (117, 42), (117, 41), (118, 42), (117, 43), (118, 43), (119, 44), (119, 43), (118, 44), (116, 40), (116, 41), (115, 42), (116, 42), (117, 44), (119, 45), (120, 45), (120, 44), (118, 45), (121, 45), (122, 46), (123, 45), (124, 46), (123, 46), (123, 47), (122, 47), (121, 47), (122, 48), (122, 49), (123, 50), (124, 49), (125, 50), (126, 51), (127, 52), (128, 51), (128, 52), (127, 51), (129, 51), (128, 50), (127, 49), (126, 48), (126, 49), (125, 49), (125, 48), (126, 50), (127, 48), (128, 47), (128, 48), (129, 47), (130, 46), (130, 45), (129, 46), (130, 44), (131, 43), (132, 44), (133, 45), (134, 44), (134, 45), (134, 46), (133, 46), (134, 47), (133, 47), (133, 44), (132, 45), (132, 43), (131, 44), (133, 43), (133, 42), (133, 41), (134, 40), (134, 41), (133, 40), (134, 39), (132, 40), (131, 41), (131, 40), (131, 39), (130, 39), (129, 39), (128, 40), (129, 41), (129, 42), (130, 43), (130, 42), (129, 43), (130, 41), (128, 42), (129, 44), (129, 45), (128, 44), (127, 44), (126, 45), (127, 46), (127, 47), (128, 49), (129, 48), (130, 48), (130, 49), (126, 52), (125, 51), (127, 50), (129, 50), (129, 49), (130, 50), (125, 53), (126, 53), (125, 52), (124, 51), (123, 52), (122, 51), (123, 51), (122, 50), (121, 51), (120, 50), (119, 49), (121, 49), (121, 48), (120, 49), (120, 48), (119, 50), (120, 51), (121, 52), (122, 52), (119, 52), (118, 53), (119, 53), (120, 54), (119, 55), (120, 55), (118, 55), (118, 56), (117, 56), (118, 57), (119, 57), (120, 56), (119, 56), (117, 57), (121, 57), (120, 58), (122, 56), (121, 56), (121, 55), (118, 58), (117, 59), (116, 58), (115, 59), (114, 60), (115, 60), (115, 61), (114, 61), (115, 62), (114, 63), (116, 63), (117, 63), (118, 62), (119, 61), (118, 60), (117, 60), (116, 59), (115, 58), (114, 57), (114, 56), (113, 56), (114, 55), (113, 54), (114, 54), (113, 53), (112, 54), (111, 53), (110, 54), (109, 54), (110, 55), (108, 54), (109, 55), (107, 54), (108, 53), (107, 53), (106, 53), (106, 52), (105, 52), (104, 53), (103, 52), (102, 51), (101, 50), (101, 49), (102, 48), (101, 47), (101, 48), (102, 47), (103, 49), (104, 50), (103, 50), (102, 49), (104, 51), (103, 51), (102, 52), (101, 51), (101, 52), (102, 53), (102, 54), (103, 53), (104, 54), (103, 55), (105, 55), (105, 56), (104, 55), (104, 56), (105, 53), (104, 52), (101, 54), (101, 55), (100, 56), (100, 57), (101, 56), (101, 57), (102, 57), (102, 56), (103, 57), (103, 56), (104, 57), (105, 58), (106, 58), (105, 59), (104, 58), (104, 60), (103, 60), (102, 59), (103, 58), (104, 59), (103, 59), (102, 58), (104, 61), (105, 60), (102, 60), (102, 61), (103, 61), (101, 60), (100, 59), (99, 58), (98, 58), (98, 57), (99, 56), (98, 55), (97, 56), (96, 55), (98, 56), (99, 57), (97, 57), (97, 55), (99, 54), (99, 55), (98, 54), (96, 56), (95, 56), (94, 57), (94, 58), (95, 57), (95, 58), (93, 56), (93, 55), (92, 55), (92, 54), (91, 55), (92, 56), (94, 55), (93, 54), (94, 54), (95, 53), (96, 52), (96, 53), (96, 54), (97, 53), (97, 54), (95, 55), (97, 59), (100, 58), (100, 55), (99, 53), (99, 52), (100, 51), (99, 50), (98, 51), (97, 51), (95, 52), (94, 51), (94, 53), (93, 52), (93, 53), (95, 54), (102, 55), (100, 54), (100, 53), (98, 53), (101, 58), (101, 59), (101, 61), (97, 58), (96, 58), (96, 57), (98, 59), (95, 59), (96, 59), (99, 59), (105, 57), (105, 54), (107, 52), (106, 51), (105, 51), (106, 50), (105, 49), (104, 49), (104, 48), (103, 48), (102, 46), (101, 46), (100, 46), (100, 47), (100, 48), (99, 46), (98, 46), (97, 46), (97, 45), (98, 44), (97, 44), (96, 45), (96, 44), (95, 45), (95, 46), (96, 46), (96, 47), (97, 47), (97, 48), (97, 49), (98, 48), (98, 47), (95, 44), (94, 43), (99, 48), (100, 49), (102, 50), (98, 49), (97, 50), (96, 50), (96, 51), (95, 51), (97, 52), (98, 52), (95, 50), (94, 49), (93, 48), (92, 49), (93, 49), (95, 48), (94, 47), (93, 47), (94, 46), (96, 48), (96, 49), (98, 50), (99, 51), (99, 49), (100, 52), (101, 53), (97, 60), (95, 60), (94, 60), (93, 59), (93, 60), (92, 60), (93, 61), (91, 59), (92, 59), (91, 58), (90, 60), (91, 60), (92, 61), (91, 61), (92, 62), (91, 63), (92, 63), (93, 63), (93, 62), (91, 62), (89, 61), (88, 62), (88, 61), (87, 60), (88, 59), (87, 58), (87, 59), (88, 57), (88, 58), (89, 59), (90, 59), (89, 60), (88, 60), (89, 62), (88, 63), (89, 63), (90, 64), (86, 60), (86, 59), (85, 59), (84, 60), (83, 59), (83, 60), (82, 60), (84, 59), (83, 61), (83, 58), (82, 59), (81, 60), (80, 61), (81, 61), (81, 62), (80, 63), (80, 62), (79, 62), (78, 61), (79, 61), (82, 61), (82, 58), (81, 57), (80, 56), (81, 55), (79, 56), (78, 55), (81, 56), (79, 55), (80, 55), (80, 57), (79, 54), (77, 56), (78, 57), (82, 56), (83, 56), (82, 57), (83, 55), (84, 54), (83, 54), (82, 55), (82, 54), (83, 53), (83, 57), (84, 56), (84, 55), (85, 56), (86, 56), (86, 57), (84, 57), (85, 58), (86, 58), (87, 63), (87, 64), (86, 65), (85, 64), (86, 64), (86, 66), (86, 67), (87, 67), (88, 67), (89, 67), (88, 66), (87, 65), (85, 66), (87, 68), (87, 69), (87, 70), (86, 70), (85, 69), (86, 68), (89, 66), (86, 63), (86, 62), (85, 62), (85, 61), (84, 61), (84, 62), (86, 61), (87, 61), (89, 58), (89, 57), (89, 56), (89, 55), (88, 56), (87, 56), (86, 55), (87, 54), (88, 55), (89, 54), (90, 53), (91, 54), (91, 56), (92, 57), (90, 54), (90, 55), (92, 53), (91, 53), (92, 52), (95, 49), (99, 47), (100, 45), (99, 45), (99, 43), (98, 43), (97, 42), (98, 42), (99, 41), (99, 40), (100, 41), (100, 42), (100, 40), (101, 39), (101, 40), (99, 42), (98, 41), (99, 39), (101, 41), (102, 41), (101, 42), (101, 43), (102, 42), (103, 41), (103, 40), (104, 39), (103, 39), (102, 40), (97, 43), (100, 43), (99, 44), (98, 45), (98, 40), (97, 41), (96, 41), (96, 40), (95, 41), (95, 40), (94, 39), (94, 38), (93, 39), (93, 40), (94, 41), (93, 41), (94, 40), (94, 42), (93, 42), (92, 42), (92, 43), (92, 41), (91, 42), (90, 41), (89, 41), (88, 41), (88, 40), (87, 42), (86, 42), (85, 43), (85, 42), (86, 41), (87, 41), (86, 40), (84, 42), (83, 43), (82, 43), (81, 44), (80, 43), (82, 45), (81, 46), (80, 46), (80, 45), (80, 44), (80, 42), (81, 43), (82, 44), (83, 46), (84, 47), (85, 47), (84, 46), (86, 46), (85, 45), (86, 48), (86, 47), (87, 48), (88, 48), (89, 49), (89, 47), (88, 46), (88, 47), (87, 45), (87, 47), (87, 49), (86, 50), (85, 51), (86, 49), (87, 50), (88, 49), (88, 50), (89, 50), (88, 51), (87, 52), (88, 53), (88, 52), (86, 51), (86, 52), (86, 53), (87, 53), (87, 51), (89, 48), (90, 49), (90, 50), (90, 51), (89, 51), (90, 52), (91, 49), (92, 50), (89, 52), (87, 55), (85, 57), (85, 54), (85, 55), (84, 58), (84, 63), (85, 63), (87, 62), (90, 61), (94, 61), (94, 59), (92, 58), (90, 62), (92, 64), (95, 61), (96, 61), (95, 62), (96, 62), (97, 61), (98, 62), (99, 61), (100, 60), (98, 60), (99, 60), (98, 61), (97, 62), (97, 63), (96, 63), (94, 63), (94, 64), (94, 65), (95, 65), (95, 66), (94, 66), (95, 67), (94, 67), (95, 68), (95, 69), (95, 70), (94, 71), (95, 72), (95, 73), (94, 74), (93, 74), (92, 75), (92, 76), (94, 75), (95, 76), (94, 77), (95, 78), (96, 77), (97, 78), (98, 79), (99, 79), (99, 78), (100, 79), (100, 80), (101, 81), (102, 82), (101, 82), (100, 83), (100, 81), (101, 80), (101, 79), (100, 78), (100, 77), (99, 77), (101, 76), (100, 76), (99, 75), (99, 74), (98, 75), (99, 76), (97, 74), (96, 74), (95, 75), (94, 73), (93, 73), (94, 72), (92, 72), (93, 72), (92, 74), (92, 73), (91, 75), (92, 77), (91, 78), (91, 77), (90, 78), (92, 78), (92, 79), (91, 79), (90, 79), (90, 80), (89, 80), (89, 79), (88, 78), (88, 77), (87, 78), (86, 77), (85, 76), (85, 77), (84, 77), (86, 76), (86, 75), (84, 76), (83, 76), (84, 78), (87, 77), (88, 76), (89, 76), (89, 75), (90, 76), (93, 76), (93, 75), (94, 76), (93, 77), (93, 78), (91, 76), (90, 77), (92, 80), (89, 78), (91, 80), (88, 79), (88, 80), (87, 79), (89, 77), (90, 75), (89, 74), (88, 75), (87, 74), (87, 73), (88, 73), (89, 73), (88, 74), (86, 72), (90, 81), (92, 81), (91, 81), (91, 82), (90, 82), (89, 82), (90, 83), (91, 84), (90, 84), (91, 85), (92, 85), (92, 84), (93, 85), (92, 86), (90, 85), (89, 85), (91, 86), (89, 83), (89, 84), (88, 85), (87, 84), (86, 83), (86, 82), (87, 82), (85, 84), (84, 84), (84, 83), (83, 84), (83, 85), (83, 86), (84, 86), (84, 87), (83, 88), (84, 89), (83, 90), (82, 91), (82, 92), (83, 93), (84, 93), (83, 94), (84, 94), (85, 94), (86, 93), (87, 93), (88, 93), (87, 92), (86, 92), (87, 91), (86, 90), (87, 90), (87, 89), (88, 88), (89, 88), (90, 87), (90, 88), (91, 87), (93, 86), (92, 87), (89, 87), (88, 89), (89, 89), (88, 87), (87, 86), (87, 85), (86, 86), (85, 87), (85, 88), (84, 88), (85, 89), (86, 88), (86, 87), (85, 86), (85, 85), (84, 85), (83, 83), (82, 82), (83, 81), (83, 82), (82, 81), (82, 80), (83, 80), (84, 81), (84, 80), (84, 82), (85, 81), (86, 80), (86, 78), (85, 79), (85, 78), (87, 76), (93, 81), (94, 80), (94, 81), (95, 82), (96, 81), (95, 80), (96, 82), (93, 80), (94, 79), (93, 79), (94, 78), (95, 81), (94, 82), (93, 82), (95, 79), (95, 77), (91, 74), (90, 74), (88, 81), (87, 80), (89, 81), (88, 83), (86, 84), (88, 84), (87, 83), (82, 87), (81, 88), (81, 89), (80, 89), (81, 90), (82, 89), (84, 90), (83, 91), (84, 91), (84, 92), (83, 92), (81, 91), (82, 90), (83, 89), (82, 88), (83, 87), (82, 86), (81, 85), (80, 85), (80, 86), (79, 85), (79, 86), (81, 86), (86, 85), (88, 86), (89, 86), (87, 87), (85, 82), (86, 81), (87, 81), (85, 83), (82, 85), (82, 84), (81, 83), (80, 83), (79, 84), (79, 82), (78, 81), (78, 82), (77, 81), (76, 81), (78, 80), (77, 80), (78, 79), (79, 78), (80, 79), (81, 79), (82, 78), (81, 77), (81, 76), (82, 76), (82, 83), (81, 82), (81, 81), (80, 80), (78, 78), (78, 77), (79, 76), (79, 77), (80, 76), (80, 75), (81, 74), (81, 75), (80, 74), (80, 73), (79, 73), (78, 73), (79, 74), (78, 75), (78, 74), (81, 72), (81, 71), (82, 71), (81, 70), (83, 70), (83, 71), (84, 71), (85, 70), (84, 69), (85, 68), (88, 65), (89, 65), (88, 64), (90, 63), (93, 65), (93, 66), (93, 68), (94, 69), (94, 70), (93, 70), (93, 69), (92, 68), (91, 69), (90, 70), (91, 71), (91, 70), (92, 71), (92, 70), (91, 72), (93, 71), (95, 71), (96, 69), (96, 70), (96, 73), (96, 72), (97, 72), (96, 75), (87, 75), (86, 74), (86, 73), (87, 71), (87, 72), (85, 72), (84, 72), (85, 73), (84, 73), (82, 72), (81, 73), (82, 70), (83, 69), (84, 67), (85, 67), (84, 68), (83, 68), (82, 67), (81, 67), (80, 68), (80, 69), (81, 69), (81, 68), (82, 68), (79, 68), (78, 68), (77, 68), (76, 69), (77, 70), (78, 71), (79, 70), (78, 70), (78, 69), (77, 69), (76, 68), (76, 67), (75, 66), (76, 65), (75, 64), (74, 64), (73, 63), (73, 64), (74, 65), (74, 66), (73, 66), (74, 67), (72, 65), (71, 64), (72, 64), (74, 63), (75, 62), (75, 63), (75, 65), (76, 64), (77, 65), (77, 64), (78, 65), (78, 66), (79, 66), (79, 65), (80, 64), (79, 63), (79, 64), (81, 64), (82, 64), (81, 63), (78, 62), (78, 63), (78, 64), (77, 63), (77, 61), (76, 61), (77, 60), (75, 61), (75, 60), (76, 60), (76, 59), (75, 58), (74, 59), (73, 60), (73, 59), (73, 58), (74, 58), (76, 57), (76, 56), (75, 57), (75, 56), (74, 57), (74, 56), (77, 57), (77, 55), (76, 55), (77, 54), (78, 53), (79, 53), (78, 54), (77, 52), (77, 51), (76, 52), (75, 52), (76, 53), (76, 51), (76, 50), (77, 50), (77, 53), (79, 52), (80, 51), (80, 50), (81, 51), (80, 52), (78, 52), (79, 51), (78, 51), (80, 53), (81, 54), (81, 53), (82, 53), (81, 58), (80, 58), (79, 58), (79, 59), (78, 58), (77, 58), (78, 59), (77, 59), (78, 60), (76, 58), (74, 60), (72, 60), (75, 59), (74, 61), (74, 62), (72, 62), (73, 57), (72, 61), (73, 61), (73, 62), (72, 63), (71, 62), (71, 61), (76, 62), (76, 63), (73, 65), (75, 67), (76, 66), (75, 68), (75, 69), (76, 70), (77, 66), (73, 67), (70, 62), (71, 63), (70, 63), (70, 64), (69, 63), (69, 62), (72, 66), (72, 68), (72, 67), (74, 68), (71, 68), (70, 69), (71, 69), (71, 70), (72, 71), (72, 72), (71, 72), (70, 71), (70, 70), (70, 68), (70, 67), (71, 67), (69, 70), (68, 69), (68, 68), (67, 68), (67, 69), (68, 70), (67, 70), (67, 71), (66, 69), (65, 68), (66, 68), (65, 67), (65, 66), (66, 67), (65, 70), (66, 70), (66, 71), (65, 71), (64, 71), (63, 71), (64, 70), (63, 69), (64, 69), (64, 68), (63, 70), (62, 69), (62, 70), (64, 72), (63, 72), (62, 72), (61, 73), (64, 73), (65, 72), (64, 67), (63, 67), (62, 68), (62, 67), (61, 67), (62, 66), (65, 69), (69, 67), (69, 68), (68, 67), (67, 66), (66, 65), (65, 64), (64, 63), (65, 62), (66, 62), (65, 63), (64, 64), (63, 65), (62, 65), (61, 66), (61, 65), (60, 66), (59, 66), (58, 65), (59, 64), (60, 65), (62, 64), (60, 64), (61, 68), (60, 67), (63, 68), (64, 66), (64, 65), (65, 65), (66, 64), (63, 66), (62, 71), (61, 72), (62, 73), (63, 74), (63, 75), (62, 75), (62, 74), (63, 76), (63, 77), (62, 76), (62, 77), (61, 76), (64, 76), (64, 75), (61, 77), (61, 78), (60, 77), (59, 76), (58, 76), (59, 77), (59, 78), (58, 77), (58, 78), (58, 79), (58, 80), (59, 80), (58, 81), (57, 81), (57, 80), (59, 79), (60, 78), (61, 75), (60, 75), (56, 79), (55, 79), (55, 78), (54, 79), (54, 78), (55, 77), (56, 76), (54, 77), (55, 76), (56, 77), (57, 77), (57, 76), (58, 75), (57, 74), (58, 74), (59, 73), (59, 72), (58, 73), (59, 75), (59, 74), (57, 75), (57, 73), (57, 72), (58, 71), (56, 72), (56, 71), (56, 73), (55, 74), (54, 74), (55, 75), (53, 78), (52, 77), (53, 77), (52, 76), (53, 75), (51, 76), (51, 75), (50, 76), (50, 77), (49, 76), (50, 75), (49, 74), (49, 75), (48, 76), (48, 77), (49, 73), (48, 72), (48, 73), (50, 72), (51, 72), (51, 73), (52, 72), (51, 71), (52, 71), (50, 74), (50, 73), (48, 74), (47, 74), (46, 74), (45, 75), (46, 76), (45, 77), (44, 77), (45, 78), (44, 78), (45, 76), (46, 77), (46, 78), (47, 75), (47, 73), (47, 72), (47, 71), (46, 71), (47, 70), (48, 71), (49, 72), (49, 71), (48, 70), (49, 69), (48, 69), (47, 69), (47, 68), (46, 67), (45, 67), (46, 68), (44, 67), (44, 66), (45, 66), (46, 65), (47, 66), (48, 67), (48, 68), (49, 68), (50, 68), (49, 67), (49, 66), (48, 66), (50, 69), (51, 69), (50, 67), (51, 67), (52, 68), (52, 69), (51, 70), (50, 70), (50, 71), (49, 70), (52, 70), (52, 67), (53, 67), (54, 66), (55, 65), (56, 65), (57, 66), (58, 66), (57, 65), (59, 65), (58, 64), (57, 64), (58, 63), (57, 63), (57, 62), (57, 61), (56, 60), (57, 59), (56, 58), (56, 59), (55, 58), (54, 57), (57, 58), (56, 57), (56, 56), (55, 56), (57, 57), (56, 55), (57, 54), (58, 55), (58, 54), (59, 54), (59, 55), (57, 53), (58, 53), (57, 52), (58, 51), (58, 52), (56, 52), (57, 51), (56, 50), (57, 50), (58, 49), (59, 50), (60, 51), (61, 52), (62, 51), (63, 52), (64, 52), (65, 52), (66, 53), (65, 53), (66, 52), (63, 51), (63, 53), (61, 50), (62, 50), (61, 49), (60, 49), (61, 48), (61, 47), (60, 46), (61, 46), (62, 45), (61, 45), (63, 45), (63, 46), (63, 44), (62, 46), (62, 47), (63, 48), (64, 47), (65, 47), (66, 47), (67, 47), (66, 46), (67, 45), (68, 44), (68, 45), (69, 45), (67, 46), (68, 47), (69, 46), (70, 45), (69, 44), (70, 46), (69, 43), (68, 46), (65, 45), (64, 46), (65, 48), (66, 49), (66, 48), (67, 49), (67, 48), (65, 49), (66, 50), (67, 50), (68, 48), (69, 47), (70, 48), (70, 47), (71, 48), (71, 49), (70, 50), (70, 51), (69, 51), (68, 50), (67, 51), (68, 52), (69, 53), (70, 54), (70, 53), (71, 54), (70, 52), (69, 52), (69, 54), (68, 55), (69, 56), (69, 55), (71, 55), (70, 56), (70, 55), (71, 56), (72, 54), (71, 53), (72, 53), (73, 54), (74, 54), (75, 55), (75, 54), (74, 55), (73, 55), (73, 52), (74, 51), (73, 50), (72, 49), (72, 50), (70, 49), (71, 50), (71, 51), (72, 52), (72, 51), (74, 52), (74, 53), (75, 53), (75, 51), (75, 50), (74, 49), (73, 53), (73, 56), (76, 54), (78, 56), (79, 57), (79, 60), (80, 59), (80, 60), (82, 62), (82, 63), (81, 52), (82, 52), (83, 52), (83, 51), (84, 51), (82, 51), (81, 50), (82, 50), (82, 49), (80, 49), (83, 50), (84, 50), (83, 49), (84, 48), (85, 48), (85, 49), (85, 50), (85, 52), (85, 53), (84, 53), (86, 54), (88, 54), (90, 56), (91, 57), (90, 57), (90, 58), (96, 60), (95, 63), (101, 62), (102, 62), (103, 63), (104, 63), (104, 64), (103, 64), (103, 65), (102, 64), (103, 62), (104, 62), (105, 63), (105, 62), (106, 62), (106, 63), (104, 65), (105, 64), (106, 65), (107, 66), (107, 67), (106, 66), (106, 67), (105, 67), (104, 67), (103, 68), (102, 68), (101, 67), (100, 68), (99, 69), (101, 69), (100, 69), (101, 68), (100, 67), (100, 66), (101, 65), (101, 66), (99, 68), (98, 67), (99, 66), (99, 67), (100, 65), (99, 65), (100, 64), (101, 63), (100, 62), (99, 62), (99, 63), (99, 64), (98, 66), (97, 65), (96, 64), (97, 64), (98, 63), (98, 64), (96, 65), (96, 66), (97, 67), (97, 66), (98, 65), (102, 66), (102, 65), (101, 64), (100, 63), (98, 68), (95, 64), (93, 67), (92, 66), (91, 67), (91, 68), (90, 68), (92, 69), (92, 67), (90, 69), (89, 68), (88, 68), (89, 69), (88, 70), (88, 69), (86, 71), (85, 71), (84, 70), (88, 71), (89, 72), (88, 72), (90, 73), (89, 71), (85, 74), (84, 75), (84, 74), (83, 75), (85, 75), (90, 71), (90, 67), (91, 66), (89, 64), (84, 64), (83, 64), (84, 65), (85, 65), (84, 66), (83, 63), (83, 62), (81, 59), (97, 68), (96, 67), (96, 68), (97, 69), (97, 70), (97, 71), (96, 71), (98, 71), (98, 72), (97, 73), (95, 74), (98, 73), (100, 74), (101, 74), (101, 75), (100, 75), (98, 74), (99, 73), (99, 71), (99, 72), (100, 71), (101, 72), (102, 71), (102, 72), (103, 72), (103, 71), (104, 71), (105, 70), (106, 70), (105, 69), (104, 68), (104, 70), (105, 71), (102, 73), (102, 75), (101, 73), (100, 72), (98, 76), (98, 77), (98, 78), (97, 77), (97, 76), (96, 78), (96, 80), (96, 79), (97, 75), (96, 76), (102, 77), (103, 76), (103, 77), (104, 78), (104, 77), (104, 76), (103, 75), (105, 76), (106, 76), (107, 75), (106, 74), (106, 73), (105, 72), (106, 71), (106, 72), (107, 71), (108, 72), (108, 71), (108, 70), (109, 70), (110, 69), (110, 71), (109, 71), (110, 70), (111, 70), (111, 69), (112, 69), (113, 69), (114, 69), (115, 69), (114, 70), (115, 68), (115, 67), (116, 67), (117, 67), (118, 68), (117, 69), (116, 69), (115, 70), (112, 70), (113, 71), (112, 71), (111, 72), (111, 73), (111, 74), (110, 74), (109, 73), (108, 74), (107, 74), (108, 73), (107, 73), (107, 72), (109, 74), (107, 70), (107, 69), (107, 68), (108, 66), (108, 67), (106, 68), (108, 69), (106, 69), (108, 68), (109, 69), (110, 68), (109, 68), (110, 67), (109, 67), (113, 70), (104, 69), (105, 68), (103, 70), (103, 69), (102, 70), (101, 70), (102, 69), (98, 69), (100, 70), (98, 70), (99, 70), (103, 67), (104, 72), (103, 73), (104, 73), (102, 74), (101, 71), (100, 73), (102, 76), (105, 74), (105, 73), (105, 75), (106, 75), (105, 77), (110, 73), (110, 75), (111, 75), (109, 76), (109, 77), (110, 76), (109, 75), (108, 75), (107, 65), (110, 66), (111, 67), (112, 66), (113, 67), (112, 68), (111, 68), (114, 71), (115, 72), (114, 72), (113, 72), (112, 73), (112, 72), (113, 73), (112, 74), (111, 76), (112, 75), (112, 76), (112, 77), (111, 78), (111, 79), (110, 78), (110, 80), (110, 79), (109, 78), (109, 81), (109, 80), (110, 81), (108, 82), (107, 82), (106, 83), (105, 84), (104, 83), (104, 84), (105, 83), (103, 82), (104, 82), (103, 81), (104, 81), (105, 82), (107, 83), (108, 83), (108, 84), (108, 85), (109, 86), (109, 87), (108, 87), (108, 88), (109, 89), (108, 89), (108, 90), (108, 91), (109, 91), (110, 88), (110, 89), (109, 88), (107, 87), (107, 86), (106, 85), (106, 86), (105, 87), (106, 88), (108, 86), (106, 87), (105, 86), (107, 85), (109, 85), (110, 85), (110, 86), (111, 85), (112, 86), (112, 85), (111, 84), (110, 84), (109, 83), (110, 83), (106, 84), (104, 85), (103, 84), (103, 83), (102, 81), (102, 79), (100, 82), (99, 82), (99, 84), (98, 85), (99, 86), (99, 87), (99, 88), (100, 88), (101, 87), (101, 86), (101, 88), (100, 89), (101, 90), (101, 89), (102, 90), (101, 91), (100, 92), (101, 93), (100, 94), (99, 95), (99, 96), (100, 96), (99, 97), (98, 96), (97, 95), (97, 94), (96, 95), (95, 94), (94, 95), (94, 96), (94, 97), (95, 96), (95, 95), (96, 96), (97, 97), (100, 95), (99, 94), (98, 95), (101, 96), (101, 95), (102, 96), (103, 96), (104, 96), (105, 97), (105, 96), (104, 95), (104, 97), (105, 98), (106, 99), (107, 99), (107, 100), (108, 99), (107, 98), (108, 98), (108, 97), (109, 98), (109, 96), (110, 96), (111, 96), (112, 97), (112, 96), (111, 97), (110, 98), (110, 97), (111, 98), (112, 99), (112, 100), (113, 99), (112, 98), (113, 98), (114, 99), (111, 101), (113, 100), (114, 100), (113, 101), (112, 101), (111, 100), (110, 100), (112, 102), (113, 103), (109, 99), (108, 100), (109, 101), (110, 102), (110, 99), (109, 95), (108, 96), (109, 94), (109, 93), (108, 92), (110, 93), (111, 92), (112, 92), (113, 91), (112, 91), (111, 90), (111, 91), (110, 91), (110, 92), (107, 93), (107, 91), (106, 90), (106, 91), (106, 92), (106, 93), (105, 93), (106, 94), (105, 95), (104, 94), (103, 94), (103, 95), (102, 94), (102, 95), (101, 94), (102, 97), (103, 98), (102, 98), (101, 98), (100, 99), (99, 100), (98, 99), (99, 98), (100, 98), (101, 97), (101, 99), (100, 100), (101, 101), (101, 100), (100, 102), (99, 102), (98, 101), (97, 102), (98, 102), (99, 103), (99, 104), (100, 104), (99, 105), (100, 105), (98, 105), (98, 104), (97, 104), (96, 104), (96, 103), (97, 103), (97, 105), (96, 106), (95, 105), (96, 105), (95, 106), (97, 106), (98, 106), (100, 103), (95, 103), (96, 102), (95, 102), (96, 101), (97, 100), (99, 101), (100, 101), (99, 99), (98, 103), (102, 100), (102, 99), (100, 97), (100, 93), (104, 98), (104, 99), (103, 99), (97, 99), (98, 100), (97, 98), (96, 100), (95, 101), (95, 104), (94, 106), (94, 105), (93, 105), (92, 106), (93, 106), (93, 107), (92, 108), (92, 109), (92, 110), (93, 110), (93, 109), (92, 111), (91, 108), (90, 107), (91, 106), (90, 105), (91, 104), (90, 103), (89, 106), (89, 107), (90, 106), (91, 107), (93, 104), (92, 103), (94, 104), (94, 103), (94, 102), (93, 103), (92, 104), (92, 105), (89, 108), (88, 109), (89, 110), (90, 110), (90, 111), (89, 111), (88, 110), (89, 109), (88, 108), (90, 108), (91, 105), (94, 107), (91, 102), (92, 101), (93, 100), (92, 100), (93, 101), (92, 102), (89, 102), (90, 102), (89, 101), (88, 100), (88, 101), (87, 100), (86, 100), (85, 100), (86, 101), (87, 102), (90, 104), (89, 105), (90, 109), (88, 107), (91, 103), (91, 101), (87, 108), (86, 107), (85, 106), (84, 106), (85, 107), (84, 105), (83, 105), (85, 104), (84, 104), (84, 107), (83, 108), (84, 108), (83, 109), (84, 110), (84, 111), (85, 111), (86, 112), (85, 112), (85, 110), (86, 110), (87, 111), (91, 110), (91, 111), (92, 112), (91, 109), (92, 107), (89, 104), (88, 104), (87, 105), (87, 106), (88, 105), (88, 106), (87, 104), (88, 103), (87, 103), (86, 102), (86, 103), (86, 104), (85, 102), (85, 101), (84, 101), (84, 99), (85, 99), (84, 98), (84, 97), (83, 98), (83, 99), (84, 100), (86, 98), (87, 98), (87, 99), (86, 99), (85, 98), (85, 97), (86, 96), (85, 96), (86, 97), (87, 101), (89, 99), (84, 103), (85, 103), (90, 101), (89, 103), (91, 100), (93, 99), (92, 99), (92, 98), (91, 97), (92, 97), (93, 96), (93, 95), (93, 94), (94, 94), (93, 93), (94, 93), (95, 92), (96, 92), (95, 93), (94, 92), (94, 91), (93, 92), (92, 93), (93, 97), (92, 95), (91, 96), (90, 95), (89, 96), (88, 95), (88, 96), (87, 97), (88, 98), (90, 99), (89, 100), (88, 99), (88, 102), (90, 100), (84, 102), (83, 102), (83, 104), (82, 104), (82, 105), (82, 103), (81, 104), (83, 106), (83, 107), (84, 109), (83, 110), (86, 113), (85, 114), (84, 113), (83, 113), (83, 112), (83, 111), (84, 112), (82, 114), (85, 113), (84, 114), (83, 114), (82, 115), (82, 116), (81, 115), (80, 115), (79, 114), (79, 115), (80, 114), (78, 115), (77, 116), (77, 115), (76, 116), (76, 115), (77, 114), (76, 113), (77, 113), (76, 112), (77, 112), (78, 113), (78, 114), (79, 113), (79, 112), (81, 116), (80, 116), (79, 116), (79, 117), (78, 116), (80, 117), (81, 117), (82, 118), (83, 118), (84, 117), (84, 116), (83, 116), (82, 119), (81, 119), (81, 118), (80, 119), (79, 120), (79, 119), (78, 120), (77, 121), (78, 122), (78, 121), (79, 121), (79, 122), (80, 123), (80, 122), (81, 121), (82, 120), (82, 121), (82, 122), (81, 122), (81, 124), (81, 123), (82, 123), (80, 120), (80, 121), (78, 119), (77, 118), (77, 120), (82, 124), (82, 125), (83, 126), (82, 126), (83, 125), (84, 124), (83, 124), (79, 124), (78, 123), (81, 125), (80, 124), (79, 125), (79, 126), (78, 126), (78, 127), (80, 127), (81, 126), (83, 123), (84, 123), (85, 124), (86, 123), (87, 122), (88, 121), (89, 121), (86, 122), (87, 123), (88, 124), (89, 125), (89, 126), (88, 126), (88, 127), (87, 128), (87, 127), (86, 127), (85, 126), (84, 127), (84, 128), (83, 127), (83, 128), (82, 129), (81, 129), (82, 130), (83, 131), (82, 132), (82, 133), (83, 133), (82, 134), (83, 134), (81, 133), (81, 132), (80, 132), (81, 131), (83, 132), (82, 131), (79, 132), (78, 132), (78, 133), (78, 131), (77, 132), (77, 133), (76, 132), (75, 131), (75, 132), (74, 133), (75, 133), (75, 134), (76, 133), (76, 134), (73, 134), (73, 133), (72, 133), (74, 132), (76, 130), (76, 129), (75, 129), (77, 129), (77, 130), (75, 128), (76, 128), (78, 128), (77, 127), (80, 125), (77, 125), (78, 124), (78, 125), (76, 126), (76, 125), (76, 124), (75, 124), (74, 124), (76, 123), (77, 124), (77, 126), (77, 128), (78, 130), (79, 129), (78, 129), (79, 128), (80, 129), (80, 130), (82, 128), (84, 129), (84, 125), (86, 124), (87, 124), (88, 123), (88, 122), (89, 120), (90, 121), (90, 122), (90, 119), (91, 118), (92, 117), (92, 118), (91, 117), (91, 116), (93, 118), (94, 118), (93, 119), (92, 120), (91, 120), (89, 122), (89, 124), (87, 126), (87, 125), (85, 125), (86, 125), (84, 126), (82, 127), (81, 128), (80, 126), (81, 127), (85, 122), (86, 121), (89, 123), (90, 123), (90, 124), (91, 124), (91, 125), (92, 124), (93, 125), (94, 125), (95, 124), (95, 125), (94, 126), (93, 124), (96, 125), (95, 126), (96, 124), (96, 123), (95, 122), (94, 122), (95, 121), (96, 122), (95, 123), (94, 123), (97, 125), (96, 126), (97, 127), (95, 127), (96, 128), (96, 127), (95, 128), (94, 129), (93, 130), (93, 131), (93, 132), (92, 131), (94, 130), (95, 130), (94, 131), (95, 129), (96, 130), (95, 131), (93, 129), (93, 128), (93, 127), (94, 127), (94, 128), (92, 130), (91, 131), (92, 132), (93, 133), (93, 134), (92, 133), (92, 134), (91, 134), (91, 133), (94, 134), (94, 133), (95, 132), (96, 132), (96, 131), (97, 131), (98, 131), (99, 131), (100, 131), (100, 130), (99, 130), (99, 129), (101, 130), (102, 131), (103, 130), (104, 129), (105, 130), (104, 131), (104, 130), (103, 129), (102, 128), (102, 130), (102, 129), (99, 132), (99, 133), (97, 132), (98, 132), (98, 134), (97, 134), (96, 133), (99, 134), (100, 133), (101, 134), (100, 132), (101, 131), (103, 132), (102, 132), (103, 133), (102, 133), (101, 133), (101, 132), (98, 129), (99, 128), (98, 128), (98, 127), (97, 126), (97, 128), (97, 129), (98, 130), (101, 129), (100, 129), (101, 128), (102, 127), (101, 127), (103, 128), (104, 127), (105, 126), (105, 125), (105, 124), (104, 123), (103, 124), (103, 125), (104, 125), (103, 126), (102, 126), (101, 126), (100, 127), (99, 126), (98, 126), (99, 127), (100, 126), (99, 125), (102, 125), (104, 124), (106, 123), (107, 122), (107, 123), (108, 124), (109, 125), (108, 125), (108, 126), (107, 126), (107, 127), (106, 126), (108, 127), (106, 128), (106, 127), (107, 125), (109, 126), (106, 125), (104, 126), (106, 129), (107, 128), (107, 129), (107, 130), (108, 130), (109, 131), (109, 130), (109, 129), (109, 128), (106, 130), (105, 129), (108, 129), (108, 131), (109, 132), (108, 132), (107, 131), (109, 133), (110, 132), (111, 131), (110, 130), (110, 133), (109, 134), (111, 133), (111, 132), (110, 131), (112, 130), (112, 131), (113, 130), (114, 130), (115, 130), (115, 129), (113, 131), (110, 134), (111, 134), (112, 134), (112, 133), (113, 134), (114, 134), (115, 134), (115, 133), (113, 133), (112, 132), (107, 132), (106, 133), (105, 132), (105, 131), (104, 132), (106, 132), (106, 131), (104, 128), (110, 128), (111, 129), (110, 129), (111, 130), (112, 129), (115, 131), (114, 132), (114, 133), (113, 129), (112, 128), (111, 127), (111, 126), (112, 125), (112, 124), (111, 125), (110, 125), (107, 124), (109, 124), (110, 123), (109, 123), (108, 123), (108, 122), (106, 124), (105, 123), (103, 123), (104, 122), (103, 121), (105, 121), (106, 120), (105, 119), (104, 119), (103, 119), (102, 120), (103, 120), (104, 120), (105, 120), (104, 121), (103, 122), (105, 122), (106, 121), (107, 120), (106, 122), (109, 122), (109, 121), (109, 120), (108, 121), (110, 122), (103, 118), (102, 119), (102, 118), (103, 117), (104, 116), (104, 117), (105, 118), (105, 117), (106, 116), (102, 116), (101, 116), (102, 115), (103, 115), (105, 115), (104, 118), (104, 114), (105, 113), (105, 114), (103, 113), (103, 112), (102, 111), (103, 110), (102, 110), (103, 109), (103, 111), (102, 112), (101, 113), (100, 114), (100, 112), (101, 111), (104, 112), (101, 110), (102, 109), (103, 108), (102, 108), (101, 109), (100, 108), (100, 107), (101, 107), (101, 106), (102, 105), (102, 104), (101, 103), (101, 104), (102, 103), (102, 102), (103, 103), (104, 102), (104, 103), (103, 104), (103, 101), (101, 102), (103, 105), (101, 105), (102, 101), (103, 100), (104, 100), (105, 100), (104, 101), (105, 101), (106, 100), (107, 101), (105, 99), (106, 102), (109, 100), (111, 99), (114, 101), (113, 102), (114, 103), (115, 102), (116, 101), (116, 102), (115, 101), (111, 103), (112, 104), (113, 105), (114, 104), (115, 104), (115, 105), (116, 105), (116, 106), (116, 107), (116, 108), (115, 107), (115, 108), (115, 109), (117, 108), (118, 109), (119, 109), (118, 108), (119, 110), (120, 111), (119, 111), (120, 110), (118, 112), (118, 113), (117, 113), (116, 113), (117, 114), (116, 115), (115, 116), (114, 115), (115, 115), (116, 116), (116, 117), (115, 118), (114, 119), (115, 120), (115, 121), (114, 121), (113, 122), (114, 118), (114, 117), (113, 117), (114, 116), (113, 119), (112, 118), (117, 115), (116, 114), (118, 116), (118, 115), (118, 114), (118, 117), (118, 118), (117, 118), (116, 119), (116, 120), (117, 120), (117, 117), (119, 116), (117, 119), (116, 118), (117, 116), (118, 119), (118, 120), (119, 121), (120, 122), (121, 123), (120, 124), (119, 124), (118, 123), (117, 124), (116, 123), (115, 123), (114, 123), (112, 121), (112, 122), (111, 123), (111, 121), (110, 121), (110, 124), (111, 124), (110, 126), (107, 121), (108, 120), (107, 119), (106, 118), (106, 119), (107, 118), (107, 117), (104, 115), (101, 117), (101, 120), (101, 119), (101, 118), (102, 117), (100, 116), (103, 114), (100, 117), (99, 117), (100, 118), (100, 120), (101, 121), (102, 122), (101, 123), (102, 123), (102, 124), (111, 122), (108, 119), (109, 118), (108, 118), (108, 117), (108, 116), (109, 117), (109, 116), (109, 115), (110, 114), (110, 113), (109, 114), (110, 115), (111, 114), (111, 115), (112, 115), (111, 116), (110, 117), (111, 117), (110, 116), (110, 119), (110, 120), (109, 119), (111, 120), (112, 120), (106, 117), (107, 115), (106, 115), (105, 116), (104, 113), (105, 112), (100, 110), (99, 111), (99, 113), (100, 113), (101, 112), (103, 116), (102, 114), (101, 115), (106, 113), (106, 114), (107, 113), (108, 114), (108, 115), (112, 117), (113, 118), (115, 117), (113, 116), (119, 118), (114, 120), (115, 119), (116, 121), (119, 119), (120, 120), (119, 120), (118, 121), (117, 122), (116, 122), (115, 124), (116, 124), (116, 125), (115, 122), (113, 121), (112, 123), (113, 124), (113, 120), (112, 119), (113, 123), (114, 124), (113, 125), (112, 126), (114, 125), (113, 126), (113, 127), (114, 128), (111, 128), (114, 129), (114, 127), (115, 127), (115, 128), (116, 131), (117, 131), (117, 132), (117, 133), (116, 134), (113, 132), (116, 129), (117, 130), (118, 130), (118, 129), (119, 130), (118, 131), (119, 132), (120, 132), (119, 133), (121, 131), (121, 130), (122, 130), (120, 133), (120, 134), (121, 132), (122, 132), (122, 131), (123, 132), (124, 131), (123, 131), (124, 130), (123, 129), (123, 130), (120, 129), (120, 130), (120, 131), (121, 129), (122, 129), (121, 128), (121, 127), (121, 126), (120, 126), (120, 127), (120, 125), (121, 124), (121, 125), (119, 126), (118, 125), (119, 123), (119, 122), (118, 124), (117, 123), (117, 125), (119, 125), (122, 124), (122, 125), (123, 124), (123, 123), (122, 123), (123, 122), (124, 123), (122, 122), (122, 121), (124, 122), (123, 121), (123, 125), (124, 124), (124, 125), (125, 125), (125, 124), (126, 125), (126, 124), (127, 125), (127, 124), (126, 123), (125, 123), (124, 121), (124, 120), (125, 120), (126, 120), (127, 120), (128, 121), (129, 120), (129, 119), (128, 120), (129, 121), (128, 119), (128, 118), (128, 117), (129, 116), (129, 117), (130, 117), (131, 117), (132, 116), (132, 117), (131, 116), (130, 116), (128, 116), (129, 115), (129, 118), (130, 122), (129, 123), (128, 122), (129, 122), (127, 119), (126, 118), (125, 117), (126, 117), (125, 118), (127, 116), (130, 115), (131, 114), (131, 115), (129, 114), (128, 114), (128, 115), (132, 115), (131, 118), (130, 119), (127, 118), (126, 119), (127, 117), (126, 116), (127, 115), (126, 115), (126, 114), (130, 118), (132, 118), (130, 114), (131, 113), (132, 114), (132, 113), (133, 115), (133, 116), (131, 119), (132, 120), (133, 120), (134, 120), (133, 119), (132, 119), (131, 120), (130, 121), (130, 123), (129, 124), (128, 125), (128, 123), (127, 123), (128, 124), (128, 126), (129, 126), (130, 127), (129, 125), (130, 125), (130, 124), (131, 122), (131, 121), (130, 120), (133, 121), (134, 122), (133, 122), (132, 123), (131, 124), (124, 117), (125, 115), (124, 114), (127, 113), (127, 114), (128, 113), (129, 113), (133, 112), (134, 112), (133, 113), (132, 112), (132, 111), (131, 111), (131, 112), (130, 113), (134, 116), (133, 117), (133, 118), (134, 117), (134, 114), (134, 115), (133, 114), (134, 111), (133, 111), (132, 110), (133, 110), (134, 110), (134, 109), (133, 109), (133, 108), (134, 108), (134, 107), (133, 106), (134, 106), (133, 107), (132, 106), (132, 105), (131, 105), (130, 105), (131, 104), (129, 105), (130, 104), (132, 103), (133, 103), (134, 104), (134, 103), (134, 105), (133, 105), (133, 104), (131, 102), (132, 102), (133, 102), (132, 104), (130, 103), (129, 104), (131, 103), (133, 101), (132, 100), (131, 100), (130, 101), (132, 101), (133, 100), (132, 99), (132, 98), (132, 97), (133, 99), (134, 98), (133, 98), (134, 99), (133, 97), (131, 99), (130, 99), (131, 98), (134, 101), (134, 102), (131, 101), (130, 100), (134, 100), (134, 97), (134, 96), (134, 95), (134, 94), (133, 93), (133, 94), (132, 95), (131, 94), (130, 94), (130, 95), (129, 96), (128, 97), (129, 97), (128, 96), (128, 98), (127, 99), (126, 99), (125, 98), (125, 99), (126, 98), (127, 97), (127, 95), (126, 95), (126, 94), (125, 93), (124, 94), (123, 93), (124, 93), (123, 92), (124, 92), (125, 92), (126, 93), (125, 94), (125, 95), (126, 92), (126, 91), (127, 91), (127, 90), (128, 91), (129, 90), (130, 91), (130, 92), (130, 93), (131, 93), (129, 94), (128, 93), (127, 94), (127, 93), (128, 94), (129, 95), (130, 96), (131, 96), (132, 96), (133, 96), (132, 94), (131, 97), (129, 99), (130, 98), (129, 98), (130, 97), (129, 93), (131, 95), (129, 101), (129, 100), (128, 100), (127, 101), (128, 101), (127, 102), (127, 103), (128, 104), (127, 104), (126, 103), (125, 104), (126, 105), (127, 105), (128, 105), (126, 104), (125, 105), (124, 106), (123, 105), (123, 104), (124, 104), (124, 105), (125, 107), (125, 108), (124, 107), (123, 106), (122, 106), (122, 105), (121, 106), (123, 107), (125, 103), (125, 106), (126, 106), (123, 103), (122, 103), (123, 102), (124, 101), (125, 102), (124, 103), (124, 102), (124, 100), (125, 101), (126, 101), (128, 102), (129, 102), (128, 103), (129, 106), (128, 106), (127, 106), (128, 107), (127, 108), (126, 108), (125, 109), (124, 108), (124, 109), (125, 110), (123, 108), (122, 107), (120, 106), (119, 106), (120, 107), (119, 108), (119, 107), (118, 107), (117, 107), (120, 109), (118, 110), (121, 109), (120, 108), (117, 110), (117, 109), (116, 111), (115, 111), (115, 110), (114, 111), (113, 112), (112, 111), (113, 111), (112, 112), (111, 111), (113, 110), (112, 110), (114, 112), (112, 109), (112, 108), (113, 107), (114, 107), (118, 106), (117, 106), (118, 105), (117, 105), (118, 104), (119, 104), (119, 103), (117, 104), (116, 104), (116, 103), (114, 105), (113, 104), (112, 103), (117, 103), (118, 102), (117, 101), (118, 101), (119, 101), (118, 100), (119, 100), (120, 100), (120, 101), (119, 99), (118, 99), (117, 99), (116, 99), (115, 100), (115, 99), (114, 98), (115, 97), (116, 98), (116, 100), (115, 98), (114, 96), (113, 95), (109, 97), (107, 96), (107, 95), (108, 95), (108, 94), (108, 93), (109, 90), (107, 89), (107, 88), (110, 90), (105, 88), (104, 88), (103, 88), (104, 89), (105, 89), (104, 90), (103, 89), (104, 87), (105, 85), (107, 84), (101, 83), (99, 81), (98, 80), (97, 80), (99, 80), (98, 81), (97, 81), (92, 83), (93, 84), (91, 83), (92, 82), (94, 83), (98, 82), (99, 83), (97, 79), (102, 80), (103, 80), (103, 79), (102, 83), (102, 84), (102, 85), (102, 86), (102, 87), (101, 85), (103, 85), (103, 87), (103, 86), (104, 86), (100, 86), (99, 85), (100, 84), (98, 83), (97, 84), (96, 85), (97, 85), (96, 84), (96, 83), (97, 82), (105, 81), (106, 82), (106, 80), (107, 80), (108, 80), (109, 79), (108, 81), (107, 81), (111, 88), (102, 88), (102, 91), (101, 92), (103, 90), (105, 90), (105, 91), (107, 90), (110, 87), (111, 87), (111, 89), (112, 90), (112, 89), (112, 88), (111, 93), (111, 94), (112, 93), (113, 93), (114, 94), (113, 94), (114, 93), (115, 94), (115, 93), (116, 92), (117, 91), (116, 91), (115, 92), (115, 91), (117, 92), (118, 91), (119, 91), (119, 92), (120, 92), (119, 93), (120, 94), (120, 93), (121, 91), (122, 92), (122, 91), (123, 91), (124, 91), (125, 90), (122, 93), (122, 94), (121, 93), (119, 94), (118, 95), (119, 95), (120, 95), (118, 94), (118, 93), (117, 94), (116, 94), (117, 95), (121, 92), (117, 93), (117, 90), (116, 93), (116, 90), (116, 89), (117, 88), (118, 89), (119, 88), (119, 87), (120, 88), (118, 86), (119, 86), (120, 86), (121, 85), (122, 85), (122, 86), (121, 86), (120, 85), (120, 87), (118, 88), (117, 87), (116, 88), (118, 87), (121, 88), (121, 87), (122, 87), (123, 87), (122, 88), (123, 85), (123, 86), (124, 84), (125, 84), (125, 85), (124, 86), (125, 86), (124, 87), (125, 87), (124, 85), (126, 86), (126, 87), (127, 87), (126, 88), (127, 88), (128, 87), (126, 89), (126, 90), (125, 91), (124, 90), (120, 91), (119, 90), (118, 92), (117, 89), (118, 85), (117, 86), (117, 85), (117, 84), (116, 83), (117, 82), (118, 83), (119, 82), (119, 84), (118, 84), (119, 83), (118, 82), (120, 82), (119, 81), (120, 81), (121, 80), (120, 83), (120, 84), (117, 83), (116, 84), (115, 84), (115, 83), (115, 82), (114, 82), (114, 81), (113, 81), (113, 80), (114, 79), (113, 79), (114, 80), (115, 80), (115, 81), (114, 83), (115, 85), (116, 86), (115, 87), (120, 89), (121, 89), (121, 90), (122, 90), (123, 89), (123, 88), (124, 89), (123, 90), (120, 90), (118, 90), (123, 94), (122, 95), (121, 95), (124, 95), (123, 96), (124, 96), (124, 97), (125, 97), (125, 96), (126, 96), (123, 95), (128, 95), (128, 90), (129, 91), (131, 90), (132, 89), (132, 88), (133, 87), (132, 86), (131, 85), (130, 85), (131, 86), (132, 85), (131, 84), (133, 85), (134, 86), (134, 85), (133, 84), (132, 84), (132, 83), (131, 82), (130, 81), (132, 81), (132, 80), (133, 79), (133, 78), (134, 77), (133, 77), (134, 76), (134, 78), (133, 80), (132, 79), (131, 80), (130, 80), (131, 81), (129, 81), (130, 82), (129, 80), (128, 80), (128, 79), (129, 78), (128, 78), (127, 77), (128, 77), (127, 76), (127, 78), (127, 79), (126, 80), (126, 79), (126, 78), (128, 76), (127, 75), (126, 77), (125, 77), (125, 78), (126, 76), (125, 76), (124, 76), (124, 75), (125, 75), (123, 74), (122, 75), (123, 75), (122, 76), (122, 77), (123, 76), (124, 77), (124, 78), (123, 78), (124, 79), (126, 74), (127, 73), (128, 72), (129, 73), (129, 72), (129, 71), (128, 71), (127, 72), (128, 73), (127, 74), (126, 72), (130, 73), (129, 70), (130, 69), (130, 68), (129, 68), (128, 68), (127, 67), (127, 66), (128, 65), (129, 66), (128, 66), (128, 67), (126, 65), (127, 65), (126, 66), (125, 65), (127, 64), (125, 64), (124, 63), (124, 62), (125, 62), (126, 61), (125, 60), (125, 61), (124, 60), (124, 61), (123, 62), (123, 63), (122, 62), (123, 61), (122, 61), (121, 60), (120, 59), (119, 60), (118, 59), (116, 57), (117, 58), (115, 56), (116, 60), (113, 55), (110, 52), (111, 51), (112, 51), (112, 50), (113, 49), (114, 49), (114, 48), (112, 49), (113, 50), (111, 50), (110, 50), (109, 50), (108, 49), (109, 48), (108, 47), (107, 47), (107, 48), (106, 48), (105, 48), (105, 47), (104, 46), (103, 45), (104, 45), (105, 44), (105, 45), (103, 44), (104, 43), (103, 42), (106, 44), (107, 44), (107, 43), (108, 44), (107, 45), (108, 46), (108, 45), (109, 45), (107, 42), (106, 42), (105, 43), (106, 45), (106, 46), (107, 46), (109, 47), (108, 48), (109, 49), (110, 48), (110, 49), (108, 50), (107, 51), (108, 51), (105, 50), (106, 49), (107, 49), (93, 50), (94, 52), (94, 56), (93, 58), (90, 65), (85, 60), (93, 57), (92, 51), (93, 51), (91, 50), (94, 50), (91, 51), (87, 57), (80, 54), (71, 52), (76, 49), (77, 49), (78, 50), (81, 49), (77, 62), (80, 65), (82, 65), (81, 66), (80, 67), (80, 66), (81, 65), (83, 65), (94, 62), (78, 67), (77, 67), (69, 65), (70, 65), (70, 61), (71, 60), (70, 60), (69, 59), (68, 58), (68, 59), (68, 60), (68, 61), (69, 64), (68, 62), (67, 60), (67, 59), (67, 57), (67, 56), (66, 57), (65, 58), (64, 57), (65, 56), (65, 57), (64, 58), (63, 57), (62, 57), (61, 57), (62, 58), (63, 58), (64, 59), (64, 60), (64, 61), (63, 62), (63, 61), (65, 60), (66, 59), (66, 58), (67, 58), (69, 57), (70, 57), (71, 57), (69, 58), (70, 59), (66, 56), (64, 56), (64, 55), (64, 54), (65, 54), (63, 54), (64, 53), (65, 55), (65, 59), (63, 59), (65, 61), (67, 63), (67, 62), (66, 61), (66, 60), (62, 59), (63, 60), (62, 60), (61, 59), (61, 58), (60, 58), (59, 59), (59, 58), (59, 60), (60, 59), (61, 60), (59, 61), (58, 62), (56, 61), (56, 62), (59, 63), (56, 67), (55, 67), (54, 67), (53, 66), (54, 68), (55, 66), (56, 66), (56, 64), (56, 63), (55, 64), (54, 64), (53, 65), (54, 65), (51, 66), (50, 66), (53, 69), (54, 70), (53, 71), (53, 72), (53, 73), (52, 73), (53, 74), (52, 74), (54, 76), (52, 75), (51, 78), (50, 78), (49, 79), (48, 79), (49, 78), (48, 78), (49, 77), (47, 76), (50, 79), (51, 80), (52, 80), (51, 81), (50, 81), (51, 82), (52, 81), (53, 80), (53, 79), (53, 76), (52, 78), (51, 79), (47, 79), (47, 77), (49, 80), (48, 80), (49, 81), (48, 82), (47, 83), (48, 84), (48, 83), (47, 84), (47, 82), (48, 81), (50, 82), (47, 81), (46, 83), (45, 84), (44, 84), (44, 85), (44, 86), (45, 87), (46, 86), (47, 87), (46, 88), (45, 89), (44, 88), (44, 89), (45, 88), (46, 87), (45, 90), (46, 90), (47, 88), (48, 87), (48, 88), (49, 89), (50, 90), (49, 88), (49, 87), (47, 89), (46, 89), (47, 86), (45, 86), (46, 85), (46, 84), (47, 85), (48, 85), (48, 86), (49, 85), (49, 86), (50, 84), (49, 84), (50, 88), (51, 89), (52, 88), (51, 88), (52, 87), (53, 86), (52, 86), (52, 85), (51, 86), (50, 86), (50, 87), (51, 87), (50, 89), (49, 90), (48, 90), (51, 90), (50, 91), (45, 85), (43, 87), (42, 88), (43, 85), (49, 83), (49, 82), (44, 76), (43, 76), (46, 75), (46, 73), (46, 72), (46, 70), (45, 69), (45, 68), (44, 68), (43, 69), (42, 70), (44, 70), (43, 70), (44, 69), (41, 70), (43, 71), (42, 68), (41, 68), (40, 68), (39, 68), (38, 69), (40, 67), (41, 67), (42, 69), (44, 71), (43, 72), (42, 73), (41, 72), (41, 71), (42, 71), (40, 72), (40, 73), (39, 72), (38, 72), (37, 72), (38, 71), (38, 70), (37, 70), (36, 69), (36, 68), (35, 69), (37, 69), (36, 70), (35, 67), (36, 67), (34, 68), (33, 69), (34, 69), (33, 70), (33, 71), (33, 72), (33, 73), (34, 72), (34, 74), (33, 74), (34, 73), (34, 71), (32, 71), (31, 70), (32, 73), (32, 72), (35, 75), (33, 75), (34, 75), (32, 74), (31, 74), (30, 74), (30, 75), (32, 75), (31, 72), (30, 72), (30, 71), (31, 71), (30, 73), (31, 73), (29, 73), (32, 70), (29, 75), (30, 76), (31, 76), (31, 75), (33, 76), (32, 76), (31, 77), (31, 78), (31, 79), (30, 78), (30, 77), (29, 77), (28, 76), (29, 76), (29, 78), (30, 79), (31, 80), (32, 79), (33, 78), (34, 78), (33, 77), (35, 74), (36, 73), (35, 73), (36, 72), (35, 72), (35, 70), (37, 71), (37, 68), (37, 67), (35, 68), (34, 67), (33, 67), (33, 66), (33, 68), (35, 66), (34, 66), (36, 65), (37, 65), (36, 64), (37, 66), (36, 66), (32, 66), (32, 67), (31, 68), (32, 69), (30, 70), (31, 69), (34, 70), (29, 70), (32, 68), (31, 67), (30, 67), (31, 66), (30, 66), (29, 65), (30, 65), (30, 64), (30, 63), (29, 64), (28, 63), (27, 64), (28, 64), (29, 63), (31, 64), (31, 63), (32, 62), (31, 62), (32, 63), (32, 64), (33, 65), (34, 65), (34, 64), (33, 63), (33, 64), (34, 63), (35, 64), (35, 63), (35, 65), (32, 65), (35, 62), (37, 63), (37, 62), (36, 62), (37, 61), (38, 60), (39, 61), (39, 62), (38, 62), (39, 63), (38, 63), (39, 64), (39, 65), (40, 66), (39, 66), (39, 67), (38, 67), (38, 68), (31, 65), (30, 68), (36, 63), (35, 61), (34, 62), (33, 62), (30, 62), (30, 61), (29, 62), (27, 62), (27, 63), (26, 64), (25, 63), (26, 63), (25, 65), (25, 66), (24, 65), (23, 65), (22, 65), (21, 66), (22, 66), (23, 67), (23, 66), (22, 67), (21, 67), (20, 67), (21, 68), (25, 64), (25, 62), (24, 61), (26, 61), (25, 60), (25, 61), (24, 62), (23, 60), (23, 62), (23, 61), (24, 60), (22, 62), (22, 61), (21, 62), (20, 63), (20, 64), (21, 63), (19, 64), (19, 63), (21, 65), (22, 64), (20, 62), (19, 62), (21, 64), (20, 65), (19, 65), (19, 66), (20, 66), (22, 69), (22, 70), (23, 71), (23, 70), (22, 63), (18, 62), (17, 62), (18, 63), (17, 64), (18, 64), (16, 63), (15, 64), (16, 65), (16, 64), (14, 64), (13, 63), (14, 65), (14, 66), (13, 67), (13, 66), (14, 67), (15, 66), (15, 65), (16, 66), (16, 67), (17, 67), (17, 66), (17, 65), (18, 66), (23, 63), (24, 64), (26, 65), (26, 66), (26, 67), (26, 68), (25, 67), (27, 67), (27, 68), (27, 69), (26, 69), (27, 70), (28, 69), (29, 69), (29, 68), (29, 67), (28, 67), (28, 68), (28, 66), (28, 65), (27, 66), (27, 65), (28, 62), (29, 66), (36, 71), (35, 71), (30, 69), (28, 70), (28, 71), (27, 71), (26, 71), (27, 72), (28, 73), (29, 74), (28, 75), (32, 78), (34, 76), (38, 73), (37, 74), (36, 74), (27, 73), (26, 73), (25, 73), (25, 72), (24, 73), (23, 72), (23, 73), (24, 72), (24, 71), (24, 70), (25, 69), (25, 68), (24, 68), (26, 70), (25, 71), (25, 70), (24, 66), (23, 64), (24, 67), (19, 67), (18, 68), (19, 68), (19, 69), (18, 70), (17, 69), (17, 68), (16, 68), (16, 69), (15, 68), (14, 69), (14, 68), (13, 69), (13, 68), (15, 67), (13, 64), (12, 62), (13, 62), (12, 61), (13, 60), (14, 61), (15, 60), (15, 61), (16, 61), (17, 61), (16, 62), (18, 65), (15, 69), (16, 70), (18, 67), (19, 71), (18, 72), (18, 71), (17, 72), (18, 73), (17, 73), (16, 74), (15, 74), (14, 73), (14, 75), (13, 73), (13, 72), (14, 71), (13, 70), (12, 69), (11, 68), (11, 69), (12, 68), (12, 67), (11, 66), (10, 66), (9, 67), (10, 67), (11, 67), (8, 66), (9, 65), (8, 64), (7, 63), (6, 62), (8, 63), (9, 63), (10, 62), (9, 62), (10, 61), (9, 60), (10, 60), (10, 59), (9, 59), (10, 58), (11, 59), (11, 61), (8, 61), (9, 61), (8, 62), (6, 61), (5, 61), (5, 60), (6, 60), (7, 61), (8, 60), (7, 59), (8, 58), (7, 57), (6, 56), (5, 56), (5, 55), (5, 54), (6, 53), (7, 53), (8, 52), (9, 52), (8, 51), (7, 52), (7, 51), (6, 52), (5, 53), (5, 52), (5, 51), (6, 51), (7, 50), (5, 50), (6, 50), (7, 49), (8, 48), (5, 49), (8, 50), (9, 51), (9, 53), (9, 54), (8, 55), (7, 54), (7, 48), (8, 47), (8, 46), (9, 46), (7, 47), (6, 46), (7, 45), (6, 44), (5, 44), (5, 43), (6, 45), (7, 46), (6, 49), (8, 53), (6, 54), (7, 55), (6, 58), (9, 57), (9, 58), (7, 58), (8, 56), (5, 57), (5, 58), (6, 59), (5, 59), (8, 59), (7, 60), (5, 62), (11, 58), (12, 58), (13, 57), (12, 57), (13, 58), (14, 58), (15, 59), (14, 59), (11, 57), (12, 56), (13, 55), (14, 55), (14, 56), (15, 57), (15, 58), (14, 57), (15, 56), (16, 55), (16, 56), (15, 54), (16, 54), (15, 53), (14, 54), (16, 53), (14, 53), (17, 55), (17, 56), (15, 55), (13, 59), (14, 60), (16, 60), (17, 60), (18, 60), (16, 59), (12, 59), (11, 55), (12, 54), (11, 56), (12, 55), (11, 54), (10, 54), (10, 55), (9, 55), (10, 56), (10, 57), (9, 56), (6, 57), (6, 55), (9, 50), (8, 49), (7, 56), (8, 57), (10, 63), (10, 64), (10, 65), (9, 64), (7, 64), (8, 65), (7, 65), (6, 63), (7, 62), (8, 54), (11, 60), (5, 63), (11, 63), (12, 64), (12, 63), (11, 62), (6, 66), (7, 67), (7, 66), (5, 64), (12, 60), (18, 57), (16, 57), (16, 58), (13, 56), (17, 59), (18, 61), (17, 63), (11, 64), (6, 64), (6, 67), (7, 68), (5, 67), (6, 68), (8, 68), (9, 68), (5, 68), (5, 66), (6, 69), (5, 69), (7, 69), (8, 69), (9, 69), (10, 68), (8, 70), (7, 71), (9, 71), (10, 70), (11, 71), (12, 72), (11, 72), (10, 72), (11, 73), (11, 74), (10, 74), (9, 73), (9, 72), (8, 73), (8, 72), (7, 72), (6, 73), (7, 74), (5, 73), (6, 74), (5, 74), (7, 73), (6, 75), (7, 75), (8, 75), (8, 74), (9, 74), (10, 75), (10, 73), (9, 70), (8, 71), (6, 70), (7, 70), (5, 72), (6, 71), (5, 70), (8, 67), (6, 65), (10, 69), (11, 70), (10, 71), (13, 65), (15, 63), (14, 62), (14, 63), (13, 61), (15, 62), (11, 53), (10, 53), (13, 53), (15, 52), (14, 52), (15, 51), (13, 52), (12, 52), (12, 51), (12, 53), (10, 52), (6, 48), (5, 48), (10, 51), (11, 52), (11, 50), (10, 49), (10, 48), (9, 47), (9, 48), (10, 46), (10, 47), (11, 45), (11, 44), (12, 44), (13, 45), (12, 45), (11, 46), (12, 46), (11, 47), (12, 48), (13, 48), (13, 47), (14, 47), (14, 46), (14, 45), (10, 45), (9, 45), (9, 49), (9, 66), (12, 65), (13, 54), (16, 52), (17, 52), (18, 53), (19, 53), (20, 52), (20, 53), (19, 52), (18, 51), (17, 50), (18, 50), (19, 51), (18, 49), (19, 48), (19, 49), (20, 48), (21, 48), (22, 48), (22, 49), (22, 50), (21, 51), (21, 52), (22, 53), (21, 54), (22, 55), (21, 56), (20, 56), (19, 56), (20, 55), (19, 55), (18, 56), (18, 58), (17, 57), (19, 57), (17, 54), (18, 54), (18, 55), (17, 53), (16, 51), (16, 49), (15, 50), (14, 51), (14, 49), (13, 50), (13, 51), (12, 50), (11, 51), (12, 49), (11, 49), (14, 50), (15, 49), (16, 50), (15, 48), (14, 48), (16, 48), (15, 47), (17, 58), (19, 58), (20, 57), (20, 58), (21, 57), (22, 56), (21, 55), (20, 54), (22, 54), (23, 53), (24, 52), (24, 51), (23, 52), (22, 52), (21, 53), (23, 54), (23, 55), (22, 57), (22, 58), (21, 58), (19, 54), (20, 51), (22, 51), (21, 49), (21, 50), (21, 47), (22, 46), (23, 47), (22, 47), (21, 45), (20, 45), (21, 46), (22, 45), (22, 44), (23, 45), (24, 46), (24, 47), (25, 47), (26, 48), (25, 48), (25, 49), (26, 49), (27, 50), (27, 51), (26, 51), (27, 52), (28, 51), (26, 50), (25, 50), (24, 50), (24, 48), (23, 46), (21, 44), (23, 44), (23, 43), (23, 42), (24, 42), (22, 43), (21, 42), (23, 41), (22, 41), (21, 41), (20, 42), (20, 41), (21, 40), (20, 40), (20, 39), (19, 38), (21, 38), (22, 39), (22, 40), (21, 39), (22, 38), (23, 39), (23, 38), (23, 37), (22, 36), (22, 35), (23, 36), (21, 37), (21, 36), (22, 37), (24, 38), (24, 37), (25, 36), (26, 37), (24, 35), (24, 36), (24, 34), (23, 33), (23, 32), (24, 33), (24, 32), (22, 33), (22, 34), (21, 33), (20, 32), (19, 32), (20, 33), (23, 35), (23, 34), (21, 35), (21, 34), (20, 34), (20, 35), (19, 36), (19, 37), (20, 36), (20, 37), (23, 31), (23, 30), (24, 31), (25, 30), (24, 30), (22, 30), (21, 31), (22, 32), (25, 32), (26, 33), (27, 32), (25, 33), (27, 33), (27, 34), (27, 35), (28, 34), (28, 33), (26, 34), (25, 35), (25, 34), (26, 35), (26, 36), (27, 36), (27, 37), (28, 37), (27, 38), (26, 39), (25, 40), (24, 40), (25, 38), (25, 39), (26, 40), (27, 39), (27, 40), (28, 40), (28, 41), (27, 42), (27, 43), (26, 43), (25, 44), (26, 44), (26, 42), (25, 41), (25, 42), (26, 41), (28, 42), (29, 39), (30, 40), (31, 41), (30, 41), (31, 42), (30, 43), (30, 42), (31, 40), (32, 40), (32, 39), (33, 38), (31, 39), (32, 41), (29, 42), (29, 43), (29, 41), (26, 45), (27, 45), (24, 45), (24, 44), (25, 45), (26, 46), (26, 47), (24, 49), (24, 43), (25, 46), (23, 48), (23, 50), (23, 51), (27, 49), (27, 48), (27, 47), (28, 48), (28, 49), (29, 47), (29, 48), (30, 47), (29, 46), (31, 47), (31, 48), (30, 48), (32, 47), (31, 46), (32, 45), (32, 46), (32, 48), (33, 48), (34, 48), (35, 49), (36, 50), (37, 51), (38, 50), (38, 51), (38, 49), (39, 48), (39, 49), (40, 50), (39, 51), (40, 52), (37, 50), (37, 49), (36, 49), (35, 51), (34, 50), (33, 49), (31, 45), (33, 47), (34, 49), (35, 50), (36, 51), (36, 52), (37, 52), (36, 53), (37, 53), (40, 48), (40, 49), (39, 50), (38, 48), (38, 47), (37, 46), (38, 45), (39, 46), (39, 47), (36, 48), (37, 47), (36, 47), (38, 46), (37, 48), (41, 49), (42, 49), (43, 49), (43, 50), (42, 50), (44, 49), (45, 50), (44, 51), (46, 51), (44, 48), (45, 49), (45, 48), (46, 49), (47, 49), (46, 48), (43, 47), (47, 47), (46, 46), (47, 48), (48, 48), (48, 49), (46, 47), (48, 47), (49, 46), (50, 46), (48, 46), (49, 45), (48, 45), (47, 44), (46, 44), (47, 45), (46, 45), (48, 44), (49, 44), (48, 43), (45, 44), (45, 43), (45, 42), (45, 41), (44, 40), (45, 39), (44, 38), (45, 38), (46, 39), (46, 37), (46, 38), (47, 39), (47, 40), (46, 40), (47, 41), (48, 42), (48, 41), (49, 40), (48, 39), (48, 38), (49, 37), (48, 36), (47, 35), (47, 36), (47, 37), (48, 35), (46, 34), (46, 35), (48, 37), (49, 38), (50, 38), (50, 37), (51, 37), (51, 38), (52, 37), (51, 36), (50, 36), (47, 38), (46, 36), (45, 37), (49, 41), (49, 43), (47, 43), (50, 43), (50, 42), (51, 42), (51, 41), (52, 41), (50, 44), (47, 42), (50, 40), (50, 41), (51, 40), (49, 39), (51, 39), (52, 40), (53, 40), (53, 41), (52, 42), (51, 43), (49, 42), (46, 41), (50, 39), (50, 45), (49, 47), (48, 50), (46, 50), (43, 48), (42, 51), (43, 52), (43, 53), (42, 53), (41, 54), (40, 53), (40, 51), (41, 52), (41, 55), (40, 54), (40, 55), (39, 55), (39, 56), (38, 55), (39, 57), (39, 58), (38, 59), (38, 58), (39, 59), (39, 60), (38, 61), (40, 62), (40, 60), (40, 59), (40, 63), (40, 61), (41, 60), (37, 60), (37, 59), (37, 58), (36, 60), (36, 61), (35, 59), (34, 59), (33, 58), (35, 58), (35, 57), (36, 56), (37, 55), (38, 54), (37, 54), (38, 53), (39, 53), (41, 51), (41, 53), (42, 54), (42, 55), (39, 52), (38, 52), (35, 52), (35, 53), (34, 52), (34, 51), (34, 53), (33, 53), (33, 54), (33, 55), (34, 54), (35, 54), (35, 55), (36, 55), (35, 56), (34, 55), (34, 56), (34, 57), (37, 56), (38, 57), (38, 56), (40, 56), (41, 56), (42, 56), (41, 57), (42, 58), (42, 57), (43, 58), (42, 59), (41, 59), (40, 58), (40, 57), (37, 57), (38, 64), (40, 65), (41, 65), (42, 65), (43, 66), (43, 68), (45, 70), (44, 72), (45, 73), (45, 72), (44, 74), (43, 73), (44, 79), (43, 78), (43, 77), (42, 77), (41, 78), (41, 77), (44, 75), (45, 74), (48, 75), (47, 78), (47, 80), (46, 79), (45, 80), (45, 79), (42, 78), (42, 79), (41, 80), (40, 80), (41, 81), (41, 82), (42, 81), (42, 80), (43, 80), (43, 81), (41, 79), (40, 79), (39, 79), (39, 80), (38, 79), (38, 80), (40, 81), (39, 81), (42, 82), (43, 82), (44, 81), (44, 80), (41, 83), (41, 84), (42, 83), (40, 84), (40, 83), (40, 82), (39, 83), (39, 84), (39, 82), (40, 85), (38, 83), (37, 83), (36, 82), (37, 82), (35, 82), (36, 81), (37, 81), (36, 80), (37, 80), (38, 81), (37, 79), (38, 78), (43, 75), (43, 74), (44, 73), (42, 74), (42, 75), (42, 76), (41, 76), (43, 79), (45, 81), (45, 82), (44, 83), (46, 81), (46, 80), (45, 83), (44, 87), (45, 91), (44, 91), (46, 91), (45, 92), (45, 93), (46, 93), (44, 93), (43, 93), (44, 92), (43, 92), (44, 90), (47, 90), (48, 89), (43, 89), (48, 91), (49, 91), (52, 89), (53, 89), (53, 88), (54, 89), (55, 90), (53, 90), (54, 90), (53, 91), (54, 91), (55, 88), (56, 88), (56, 89), (55, 87), (55, 86), (56, 87), (54, 86), (54, 85), (53, 84), (54, 83), (54, 82), (53, 81), (52, 82), (52, 83), (53, 83), (53, 82), (53, 85), (54, 84), (56, 86), (57, 88), (58, 89), (57, 90), (56, 91), (55, 92), (56, 92), (57, 91), (56, 90), (58, 91), (58, 90), (59, 91), (58, 92), (59, 93), (60, 93), (59, 94), (58, 95), (59, 95), (58, 93), (58, 94), (57, 93), (57, 92), (56, 93), (57, 94), (56, 94), (55, 94), (55, 95), (56, 95), (57, 96), (57, 97), (58, 97), (59, 96), (58, 98), (57, 95), (56, 96), (56, 97), (55, 97), (57, 99), (56, 100), (55, 101), (55, 100), (56, 101), (57, 100), (58, 100), (58, 99), (59, 99), (60, 98), (61, 99), (60, 100), (62, 99), (62, 100), (63, 98), (64, 99), (64, 97), (63, 97), (62, 96), (61, 97), (62, 98), (62, 97), (61, 96), (63, 96), (63, 95), (64, 95), (65, 95), (63, 94), (63, 93), (63, 92), (62, 91), (61, 91), (63, 90), (64, 89), (64, 90), (63, 91), (62, 90), (61, 90), (62, 89), (61, 89), (60, 88), (59, 89), (59, 90), (60, 89), (63, 88), (63, 87), (64, 86), (63, 86), (64, 85), (63, 85), (62, 85), (63, 84), (63, 83), (62, 83), (61, 84), (60, 83), (60, 82), (60, 81), (60, 80), (61, 81), (61, 80), (62, 79), (62, 78), (63, 79), (62, 80), (63, 80), (64, 81), (64, 82), (63, 82), (62, 84), (61, 83), (60, 84), (59, 83), (59, 82), (60, 85), (61, 86), (59, 86), (59, 87), (60, 86), (61, 87), (61, 88), (60, 90), (63, 89), (62, 92), (64, 92), (65, 91), (65, 92), (66, 92), (65, 93), (66, 93), (67, 93), (67, 92), (68, 93), (69, 93), (69, 94), (69, 95), (70, 94), (70, 93), (69, 92), (68, 92), (66, 91), (67, 91), (67, 90), (68, 89), (67, 89), (68, 88), (69, 88), (68, 87), (67, 88), (69, 89), (68, 91), (68, 90), (64, 93), (65, 94), (66, 94), (66, 95), (66, 90), (65, 89), (64, 88), (64, 87), (65, 86), (66, 86), (65, 87), (65, 88), (65, 90), (62, 87), (62, 88), (60, 87), (61, 85), (59, 85), (58, 84), (57, 84), (56, 83), (56, 82), (57, 82), (58, 82), (59, 81), (58, 83), (57, 83), (56, 85), (57, 87), (57, 89), (54, 88), (54, 87), (53, 87), (49, 92), (50, 93), (49, 93), (48, 93), (48, 92), (47, 93), (50, 92), (51, 93), (51, 92), (52, 92), (54, 92), (52, 91), (51, 91), (52, 90), (55, 89), (51, 85), (52, 84), (51, 84), (50, 85), (51, 83), (43, 94), (42, 95), (42, 96), (42, 97), (43, 98), (42, 98), (41, 96), (41, 95), (40, 97), (40, 98), (39, 99), (39, 100), (38, 101), (39, 101), (38, 100), (39, 98), (39, 97), (38, 99), (37, 101), (38, 102), (39, 102), (38, 98), (37, 99), (36, 100), (36, 99), (35, 100), (34, 100), (33, 100), (33, 99), (32, 99), (32, 100), (32, 101), (31, 100), (31, 99), (30, 100), (29, 100), (30, 99), (30, 98), (29, 99), (31, 98), (32, 98), (31, 97), (33, 97), (34, 98), (34, 97), (33, 96), (32, 95), (31, 94), (31, 95), (30, 96), (29, 97), (31, 96), (32, 97), (32, 96), (29, 96), (30, 95), (29, 95), (30, 94), (29, 94), (32, 94), (33, 93), (29, 93), (30, 92), (29, 91), (28, 92), (29, 92), (28, 93), (27, 92), (27, 93), (28, 94), (27, 95), (26, 96), (27, 97), (26, 98), (25, 98), (24, 99), (23, 100), (24, 100), (25, 101), (26, 102), (25, 102), (26, 103), (24, 102), (26, 101), (24, 103), (23, 103), (25, 103), (25, 104), (23, 104), (24, 104), (25, 105), (26, 104), (25, 106), (24, 106), (25, 107), (26, 108), (27, 107), (28, 106), (29, 105), (30, 106), (31, 107), (30, 107), (29, 108), (30, 108), (28, 107), (27, 106), (27, 105), (28, 104), (28, 103), (29, 103), (29, 104), (30, 105), (29, 106), (31, 104), (31, 105), (30, 104), (30, 103), (31, 106), (32, 106), (33, 106), (32, 107), (32, 105), (31, 108), (29, 107), (28, 108), (29, 109), (28, 110), (28, 111), (27, 112), (28, 113), (27, 114), (29, 112), (30, 111), (31, 111), (31, 110), (32, 111), (32, 110), (32, 109), (30, 109), (32, 108), (33, 108), (34, 107), (35, 107), (36, 106), (35, 106), (34, 106), (34, 108), (33, 107), (33, 109), (33, 110), (33, 111), (31, 112), (34, 111), (34, 112), (33, 112), (34, 113), (34, 114), (33, 113), (35, 111), (35, 113), (36, 112), (35, 112), (36, 113), (37, 112), (36, 111), (37, 111), (36, 110), (38, 112), (37, 113), (36, 114), (36, 115), (36, 116), (36, 117), (37, 117), (36, 118), (35, 117), (35, 116), (34, 115), (35, 115), (37, 114), (39, 111), (38, 111), (40, 111), (38, 110), (37, 110), (40, 110), (40, 112), (39, 112), (39, 113), (38, 114), (38, 113), (35, 114), (37, 115), (33, 114), (32, 115), (32, 116), (33, 115), (33, 116), (30, 113), (31, 114), (31, 115), (30, 116), (29, 117), (30, 117), (29, 116), (31, 117), (32, 118), (31, 119), (30, 119), (30, 118), (30, 120), (29, 120), (29, 118), (28, 118), (27, 117), (27, 118), (26, 118), (26, 117), (25, 119), (26, 120), (25, 120), (27, 121), (26, 121), (27, 120), (26, 122), (25, 121), (24, 119), (23, 118), (22, 117), (21, 118), (21, 117), (20, 116), (20, 117), (19, 118), (18, 119), (19, 119), (19, 120), (18, 121), (18, 120), (17, 119), (19, 121), (20, 120), (21, 120), (22, 120), (23, 120), (23, 121), (23, 122), (22, 123), (23, 123), (22, 124), (21, 125), (22, 125), (23, 124), (21, 124), (22, 122), (21, 123), (24, 122), (24, 121), (24, 120), (26, 119), (27, 116), (28, 115), (28, 114), (27, 115), (26, 114), (25, 114), (25, 115), (26, 116), (25, 116), (25, 117), (24, 116), (25, 118), (24, 118), (23, 119), (22, 121), (21, 122), (27, 119), (28, 116), (28, 117), (26, 115), (24, 114), (27, 113), (29, 114), (30, 115), (29, 115), (28, 120), (28, 119), (27, 122), (28, 122), (29, 123), (29, 122), (25, 123), (26, 123), (26, 124), (27, 125), (28, 126), (29, 127), (30, 126), (31, 127), (31, 126), (32, 126), (33, 127), (33, 128), (33, 129), (32, 130), (31, 129), (31, 128), (30, 129), (29, 128), (28, 128), (28, 127), (27, 127), (26, 126), (25, 127), (25, 126), (24, 126), (24, 127), (25, 125), (24, 124), (24, 123), (20, 125), (22, 126), (20, 124), (19, 125), (18, 125), (17, 124), (18, 126), (19, 127), (19, 128), (20, 129), (21, 129), (22, 128), (23, 128), (23, 127), (22, 127), (23, 129), (24, 129), (22, 129), (25, 130), (24, 131), (24, 130), (25, 129), (26, 128), (26, 127), (25, 128), (27, 128), (29, 126), (29, 125), (30, 127), (30, 128), (28, 124), (30, 125), (31, 125), (30, 124), (29, 124), (28, 125), (27, 126), (26, 125), (28, 123), (27, 123), (25, 124), (23, 126), (21, 127), (21, 128), (21, 126), (23, 125), (29, 129), (27, 129), (26, 129), (28, 130), (28, 129), (29, 130), (28, 131), (27, 132), (27, 131), (26, 132), (25, 131), (26, 130), (27, 130), (30, 123), (28, 121), (29, 119), (31, 116), (33, 117), (34, 117), (35, 118), (34, 119), (34, 118), (38, 116), (38, 115), (39, 116), (37, 116), (39, 114), (40, 115), (40, 114), (39, 115), (41, 111), (42, 111), (41, 110), (42, 109), (43, 110), (42, 110), (43, 109), (42, 108), (43, 108), (43, 107), (44, 108), (44, 109), (45, 109), (45, 110), (45, 111), (46, 111), (47, 110), (47, 111), (48, 112), (49, 111), (50, 110), (49, 110), (49, 109), (48, 110), (47, 109), (47, 108), (48, 107), (49, 108), (48, 109), (48, 111), (50, 112), (50, 111), (51, 112), (50, 113), (49, 112), (46, 110), (46, 112), (44, 112), (45, 113), (45, 112), (44, 111), (43, 111), (46, 114), (46, 115), (45, 115), (46, 116), (47, 117), (46, 117), (46, 118), (48, 116), (48, 117), (48, 118), (48, 119), (48, 120), (49, 119), (49, 118), (47, 118), (46, 119), (45, 118), (47, 120), (48, 121), (49, 122), (49, 123), (48, 122), (47, 123), (47, 122), (48, 123), (47, 121), (47, 119), (46, 120), (46, 123), (46, 122), (46, 121), (45, 122), (46, 124), (45, 120), (45, 116), (45, 117), (44, 119), (43, 119), (44, 120), (43, 120), (44, 121), (45, 121), (44, 122), (43, 121), (44, 123), (43, 124), (44, 124), (43, 123), (42, 122), (42, 123), (42, 124), (41, 124), (41, 125), (42, 125), (42, 126), (41, 127), (40, 126), (40, 125), (41, 126), (40, 127), (39, 127), (38, 128), (37, 128), (37, 129), (38, 130), (38, 131), (37, 131), (36, 132), (36, 131), (35, 131), (37, 132), (35, 132), (36, 130), (35, 129), (34, 130), (34, 131), (35, 130), (37, 130), (38, 129), (39, 128), (40, 129), (41, 129), (41, 128), (40, 128), (38, 127), (37, 126), (38, 125), (39, 126), (39, 125), (39, 124), (38, 124), (40, 124), (40, 123), (40, 122), (39, 123), (41, 122), (37, 125), (36, 126), (35, 125), (35, 126), (36, 125), (35, 127), (34, 126), (33, 126), (32, 127), (32, 129), (31, 130), (29, 132), (30, 132), (30, 133), (31, 133), (30, 134), (31, 132), (30, 131), (29, 131), (24, 128), (23, 131), (23, 130), (20, 123), (20, 122), (18, 122), (17, 121), (17, 122), (18, 123), (18, 124), (16, 123), (15, 123), (15, 124), (16, 124), (17, 125), (16, 125), (16, 126), (17, 123), (16, 122), (14, 123), (15, 122), (14, 122), (13, 122), (14, 121), (14, 120), (13, 120), (12, 119), (13, 118), (13, 117), (14, 117), (15, 117), (15, 116), (16, 115), (17, 115), (18, 116), (19, 115), (18, 115), (19, 114), (20, 113), (21, 112), (20, 111), (20, 112), (21, 113), (19, 113), (18, 114), (17, 114), (16, 114), (14, 116), (13, 116), (12, 115), (13, 114), (13, 115), (14, 114), (11, 115), (10, 116), (9, 115), (9, 116), (9, 117), (8, 116), (8, 115), (7, 115), (6, 116), (5, 117), (6, 118), (5, 119), (5, 120), (6, 120), (6, 119), (5, 118), (6, 117), (7, 117), (8, 117), (11, 117), (10, 117), (10, 118), (9, 119), (8, 118), (7, 118), (7, 119), (8, 119), (9, 118), (8, 120), (8, 121), (9, 122), (9, 121), (10, 122), (10, 121), (9, 120), (10, 120), (10, 119), (11, 119), (8, 122), (9, 123), (9, 124), (10, 124), (11, 124), (11, 125), (10, 126), (11, 127), (12, 128), (9, 127), (10, 127), (9, 126), (10, 128), (9, 129), (10, 129), (11, 128), (12, 129), (11, 130), (11, 129), (9, 128), (8, 128), (9, 130), (10, 130), (9, 131), (10, 131), (8, 131), (9, 132), (8, 129), (11, 126), (12, 125), (13, 124), (14, 124), (16, 121), (15, 120), (13, 119), (14, 118), (14, 119), (11, 118), (12, 118), (11, 120), (12, 121), (13, 121), (12, 120), (11, 121), (12, 117), (15, 121), (16, 120), (16, 119), (19, 122), (17, 120), (18, 118), (18, 117), (17, 116), (19, 116), (20, 115), (18, 113), (20, 114), (21, 114), (21, 115), (21, 116), (21, 119), (21, 121), (25, 122), (30, 121), (31, 122), (32, 122), (32, 123), (32, 121), (33, 120), (34, 120), (32, 120), (33, 121), (33, 122), (34, 121), (35, 122), (36, 121), (37, 122), (36, 123), (35, 124), (34, 124), (33, 124), (33, 125), (32, 125), (27, 124), (24, 117), (19, 124), (15, 125), (16, 127), (16, 128), (15, 127), (17, 127), (17, 128), (17, 126), (18, 128), (18, 127), (19, 126), (15, 126), (14, 126), (14, 125), (13, 123), (12, 123), (12, 124), (13, 125), (12, 122), (11, 122), (12, 116), (15, 119), (19, 123), (22, 119), (20, 118), (20, 121), (29, 121), (24, 125), (20, 127), (13, 127), (12, 126), (12, 127), (14, 127), (14, 128), (15, 128), (15, 129), (15, 130), (14, 129), (13, 130), (13, 131), (12, 130), (12, 131), (13, 132), (14, 132), (14, 131), (14, 133), (13, 134), (13, 133), (14, 134), (12, 133), (12, 134), (12, 132), (15, 131), (13, 129), (13, 128), (11, 131), (13, 126), (16, 129), (16, 130), (17, 131), (18, 131), (17, 132), (17, 133), (16, 132), (18, 133), (16, 134), (15, 133), (16, 133), (17, 134), (18, 134), (18, 132), (19, 132), (19, 131), (18, 130), (17, 130), (14, 130), (16, 131), (17, 129), (7, 116), (7, 120), (15, 118), (16, 118), (17, 117), (19, 117), (22, 115), (22, 116), (23, 116), (24, 115), (23, 115), (23, 114), (22, 114), (19, 112), (18, 111), (17, 110), (19, 110), (18, 109), (17, 109), (16, 108), (15, 107), (14, 108), (15, 109), (16, 109), (15, 110), (17, 111), (16, 111), (17, 112), (18, 112), (17, 113), (16, 112), (15, 111), (14, 111), (14, 110), (13, 111), (13, 112), (14, 112), (15, 113), (15, 114), (22, 112), (21, 111), (20, 110), (19, 109), (18, 108), (17, 107), (18, 107), (18, 106), (19, 107), (20, 108), (17, 108), (16, 107), (17, 106), (19, 106), (20, 105), (21, 105), (22, 104), (23, 105), (24, 105), (27, 104), (28, 105), (26, 105), (26, 106), (24, 108), (24, 109), (24, 110), (24, 111), (23, 111), (23, 112), (25, 110), (26, 111), (27, 111), (28, 112), (29, 113), (30, 114), (32, 112), (32, 113), (32, 114), (31, 113), (33, 118), (33, 119), (31, 120), (31, 121), (30, 122), (23, 117), (22, 118), (17, 118), (20, 119), (23, 113), (24, 113), (26, 131), (26, 133), (27, 133), (28, 132), (29, 133), (29, 134), (28, 133), (32, 128), (34, 128), (34, 127), (35, 123), (34, 123), (33, 123), (32, 124), (31, 124), (31, 123), (32, 119), (35, 119), (31, 118), (32, 117), (34, 116), (38, 118), (38, 119), (38, 120), (39, 120), (40, 120), (40, 119), (39, 119), (37, 120), (41, 120), (41, 121), (40, 121), (39, 122), (38, 122), (37, 121), (36, 122), (39, 121), (35, 121), (34, 122), (36, 120), (35, 120), (36, 119), (37, 118), (38, 117), (39, 118), (37, 119), (34, 125), (37, 123), (36, 124), (37, 127), (36, 127), (37, 124), (39, 117), (30, 112), (29, 111), (30, 110), (31, 109), (34, 109), (34, 110), (35, 109), (35, 110), (36, 109), (36, 108), (35, 108), (37, 107), (38, 107), (39, 108), (38, 108), (37, 108), (37, 109), (39, 107), (40, 108), (41, 108), (41, 107), (42, 112), (40, 109), (41, 106), (42, 106), (41, 105), (43, 105), (42, 105), (42, 104), (40, 105), (39, 106), (39, 105), (40, 106), (41, 109), (39, 109), (38, 106), (40, 107), (43, 112), (43, 113), (42, 114), (42, 115), (43, 115), (43, 114), (42, 116), (43, 116), (44, 117), (44, 118), (43, 117), (43, 118), (42, 119), (43, 122), (42, 121), (41, 118), (41, 119), (42, 120), (50, 122), (50, 123), (49, 124), (49, 125), (49, 126), (48, 125), (50, 127), (50, 128), (49, 128), (50, 129), (51, 128), (49, 127), (50, 126), (51, 126), (51, 125), (50, 125), (50, 124), (51, 123), (52, 122), (52, 123), (53, 123), (52, 124), (52, 125), (53, 125), (54, 125), (53, 124), (54, 126), (55, 127), (56, 127), (57, 127), (58, 126), (59, 126), (59, 125), (59, 127), (60, 125), (60, 124), (60, 126), (60, 127), (59, 124), (58, 125), (61, 125), (62, 126), (61, 126), (61, 127), (62, 127), (61, 128), (60, 129), (60, 130), (61, 130), (60, 131), (59, 129), (58, 130), (58, 131), (58, 132), (58, 133), (57, 133), (57, 134), (59, 132), (60, 133), (61, 132), (61, 131), (62, 131), (62, 130), (61, 129), (63, 131), (64, 130), (63, 130), (64, 131), (64, 132), (65, 131), (65, 132), (65, 130), (63, 129), (63, 132), (64, 133), (65, 133), (66, 132), (67, 133), (66, 133), (66, 134), (67, 134), (68, 134), (69, 134), (69, 133), (68, 133), (69, 132), (70, 133), (70, 132), (71, 132), (70, 131), (69, 131), (68, 130), (68, 131), (67, 130), (68, 132), (67, 131), (66, 129), (66, 128), (67, 128), (67, 129), (65, 128), (64, 128), (63, 127), (64, 126), (64, 125), (63, 126), (63, 128), (64, 127), (65, 126), (65, 127), (65, 129), (64, 129), (62, 128), (60, 128), (59, 130), (59, 131), (58, 129), (57, 129), (56, 130), (55, 129), (54, 130), (54, 129), (54, 128), (53, 128), (52, 129), (53, 129), (52, 130), (52, 127), (53, 127), (52, 126), (51, 124), (48, 124), (48, 126), (49, 129), (49, 130), (48, 129), (47, 129), (46, 129), (47, 128), (48, 127), (51, 127), (52, 128), (51, 129), (48, 128), (47, 130), (48, 130), (46, 130), (45, 130), (46, 128), (51, 130), (50, 130), (50, 131), (49, 131), (51, 131), (50, 132), (51, 133), (50, 133), (51, 132), (53, 131), (54, 131), (53, 132), (52, 131), (53, 130), (52, 133), (52, 134), (51, 134), (49, 132), (50, 134), (49, 133), (48, 133), (47, 134), (46, 133), (47, 132), (46, 132), (47, 131), (47, 133), (46, 134), (49, 134), (48, 134), (48, 132), (48, 131), (53, 126), (51, 122), (54, 122), (53, 121), (52, 120), (51, 121), (50, 121), (50, 120), (51, 119), (52, 121), (53, 120), (53, 122), (54, 123), (55, 122), (54, 121), (51, 120), (50, 119), (49, 120), (47, 115), (49, 117), (50, 116), (50, 117), (51, 117), (52, 117), (52, 118), (53, 119), (53, 118), (54, 118), (55, 118), (56, 117), (56, 118), (57, 119), (57, 120), (56, 120), (58, 121), (57, 122), (56, 121), (55, 120), (54, 120), (54, 119), (55, 119), (56, 119), (57, 118), (57, 117), (57, 116), (58, 116), (56, 116), (58, 117), (59, 116), (58, 118), (55, 117), (56, 115), (55, 115), (55, 116), (54, 116), (54, 115), (55, 114), (53, 116), (52, 116), (54, 117), (53, 115), (53, 114), (52, 114), (51, 113), (51, 114), (52, 115), (54, 113), (53, 113), (53, 112), (55, 113), (54, 114), (55, 112), (54, 111), (54, 110), (53, 110), (52, 110), (51, 110), (51, 109), (52, 109), (52, 108), (53, 109), (53, 108), (51, 108), (50, 107), (49, 107), (48, 108), (51, 107), (52, 106), (52, 107), (51, 106), (53, 107), (54, 109), (50, 108), (54, 107), (54, 106), (53, 106), (53, 111), (52, 111), (52, 112), (55, 110), (54, 112), (56, 112), (57, 112), (56, 113), (55, 111), (50, 109), (52, 105), (53, 104), (54, 103), (53, 103), (53, 102), (53, 101), (52, 101), (52, 102), (54, 104), (53, 105), (50, 106), (49, 106), (50, 105), (51, 105), (51, 104), (50, 104), (49, 103), (48, 102), (49, 102), (48, 103), (47, 103), (47, 102), (46, 103), (45, 102), (46, 101), (46, 100), (47, 99), (46, 99), (46, 98), (45, 97), (44, 96), (44, 95), (42, 94), (43, 95), (45, 96), (44, 97), (43, 96), (46, 97), (44, 98), (43, 97), (41, 97), (44, 94), (43, 91), (42, 92), (41, 93), (40, 94), (40, 93), (40, 92), (39, 93), (38, 92), (37, 92), (37, 93), (36, 92), (36, 91), (37, 90), (36, 90), (35, 89), (35, 88), (34, 89), (37, 91), (38, 90), (39, 89), (40, 90), (40, 89), (39, 90), (40, 91), (39, 91), (41, 90), (42, 90), (41, 91), (39, 92), (41, 92), (38, 93), (38, 94), (36, 93), (38, 91), (38, 89), (38, 88), (37, 88), (36, 88), (37, 89), (36, 89), (35, 92), (35, 91), (34, 93), (33, 92), (32, 91), (33, 91), (31, 90), (31, 91), (31, 92), (30, 93), (31, 93), (27, 91), (26, 91), (25, 91), (25, 90), (25, 89), (24, 90), (24, 91), (23, 90), (22, 91), (22, 90), (23, 89), (24, 89), (22, 89), (21, 88), (21, 87), (20, 88), (19, 89), (19, 88), (18, 89), (19, 90), (18, 88), (20, 89), (20, 90), (21, 91), (20, 91), (20, 92), (19, 92), (18, 91), (19, 91), (17, 92), (18, 93), (18, 94), (17, 95), (18, 95), (19, 94), (19, 95), (18, 96), (19, 97), (18, 97), (19, 96), (20, 95), (20, 94), (21, 96), (22, 96), (21, 95), (21, 94), (21, 93), (21, 90), (18, 87), (17, 88), (17, 87), (17, 86), (16, 85), (16, 86), (19, 87), (19, 86), (18, 85), (15, 85), (17, 85), (15, 86), (15, 87), (18, 86), (20, 85), (16, 89), (17, 90), (15, 89), (14, 88), (14, 89), (15, 90), (16, 91), (14, 91), (14, 90), (16, 88), (17, 89), (20, 87), (20, 86), (14, 87), (15, 88), (16, 87), (14, 86), (13, 86), (12, 86), (14, 85), (14, 84), (12, 85), (11, 86), (13, 87), (11, 87), (12, 87), (10, 87), (10, 86), (10, 85), (9, 87), (8, 87), (8, 86), (7, 85), (7, 84), (6, 84), (6, 85), (8, 83), (7, 82), (6, 81), (5, 80), (5, 81), (6, 80), (5, 79), (6, 78), (6, 79), (5, 82), (6, 82), (6, 83), (5, 83), (7, 83), (7, 80), (8, 81), (9, 80), (9, 79), (10, 79), (11, 79), (10, 78), (11, 77), (11, 78), (11, 80), (12, 81), (11, 81), (10, 80), (13, 80), (14, 80), (14, 81), (13, 81), (14, 79), (14, 78), (15, 78), (16, 78), (17, 78), (18, 78), (17, 79), (18, 79), (16, 79), (17, 80), (16, 80), (15, 81), (15, 82), (16, 82), (16, 83), (16, 84), (15, 84), (18, 90), (16, 92), (16, 90), (17, 91), (18, 92), (15, 92), (14, 92), (13, 92), (13, 93), (12, 93), (14, 93), (15, 94), (15, 91), (16, 93), (16, 95), (16, 96), (15, 97), (16, 97), (17, 97), (18, 98), (19, 98), (20, 97), (21, 98), (20, 99), (21, 97), (20, 98), (21, 99), (22, 97), (22, 98), (22, 99), (21, 100), (22, 100), (23, 99), (24, 98), (23, 97), (23, 98), (23, 96), (23, 95), (24, 96), (24, 97), (25, 96), (24, 95), (23, 94), (22, 95), (22, 94), (22, 93), (22, 92), (21, 92), (17, 84), (18, 84), (18, 83), (19, 83), (19, 82), (20, 83), (20, 82), (21, 82), (22, 83), (21, 83), (21, 81), (22, 80), (23, 79), (24, 80), (23, 81), (24, 81), (25, 82), (25, 83), (24, 82), (24, 83), (23, 82), (23, 83), (22, 82), (21, 79), (20, 80), (20, 79), (19, 80), (19, 79), (19, 78), (18, 81), (19, 81), (18, 80), (20, 77), (20, 76), (19, 77), (17, 77), (16, 77), (16, 76), (15, 75), (14, 74), (13, 74), (12, 71), (12, 73), (14, 72), (13, 71), (14, 70), (17, 70), (15, 70), (16, 71), (5, 65), (11, 65), (18, 59), (19, 59), (20, 60), (21, 61), (38, 65), (37, 64), (38, 74), (39, 73), (39, 69), (29, 72), (26, 72), (24, 69), (23, 69), (29, 71), (38, 66), (27, 74), (27, 76), (28, 77), (30, 80), (29, 80), (29, 81), (32, 81), (31, 82), (32, 82), (33, 81), (34, 80), (35, 81), (36, 83), (36, 84), (38, 82), (39, 86), (40, 87), (38, 87), (37, 87), (37, 86), (36, 86), (36, 87), (35, 85), (34, 86), (33, 87), (32, 86), (32, 85), (31, 84), (31, 83), (32, 83), (33, 83), (33, 82), (31, 81), (30, 81), (30, 82), (29, 83), (28, 84), (28, 85), (29, 84), (28, 83), (27, 83), (28, 82), (27, 81), (26, 80), (27, 80), (26, 81), (26, 79), (25, 80), (22, 78), (22, 77), (21, 76), (21, 78), (21, 80), (20, 84), (18, 82), (19, 84), (19, 85), (23, 88), (24, 88), (25, 87), (25, 88), (26, 86), (27, 86), (26, 87), (24, 87), (25, 86), (26, 88), (27, 87), (28, 86), (29, 86), (30, 85), (29, 85), (30, 86), (29, 87), (28, 87), (28, 88), (27, 88), (26, 89), (26, 90), (27, 89), (25, 92), (28, 89), (29, 88), (30, 88), (23, 91), (24, 92), (24, 93), (23, 92), (23, 93), (25, 93), (25, 94), (26, 95), (26, 94), (25, 95), (27, 96), (28, 96), (28, 95), (28, 97), (26, 93), (20, 93), (19, 93), (20, 96), (17, 96), (17, 94), (13, 89), (13, 88), (12, 89), (12, 88), (11, 85), (9, 86), (10, 84), (9, 85), (8, 85), (8, 88), (9, 89), (10, 88), (9, 88), (8, 89), (7, 90), (6, 91), (5, 91), (5, 90), (6, 92), (7, 92), (7, 91), (8, 91), (9, 91), (7, 88), (6, 88), (7, 87), (6, 87), (5, 88), (5, 87), (5, 86), (6, 86), (7, 86), (5, 84), (5, 85), (9, 82), (10, 83), (9, 84), (8, 84), (8, 82), (9, 81), (10, 81), (12, 82), (13, 82), (13, 83), (14, 83), (13, 84), (12, 84), (11, 83), (11, 84), (11, 88), (10, 89), (8, 80), (8, 79), (9, 78), (10, 77), (10, 76), (9, 75), (11, 75), (12, 76), (13, 75), (14, 76), (13, 76), (12, 77), (12, 78), (12, 79), (15, 79), (15, 80), (14, 82), (12, 83), (10, 82), (9, 83), (11, 82), (7, 79), (7, 81), (9, 77), (11, 76), (9, 76), (12, 74), (5, 75), (7, 76), (7, 77), (6, 76), (6, 77), (7, 78), (5, 77), (12, 80), (13, 79), (5, 76), (8, 78), (13, 77), (13, 78), (14, 77), (12, 75), (12, 66), (18, 69), (19, 70), (19, 72), (19, 73), (20, 71), (17, 71), (15, 72), (16, 72), (16, 73), (15, 77), (18, 77), (17, 76), (17, 75), (18, 76), (19, 76), (19, 75), (19, 74), (20, 72), (21, 71), (22, 72), (22, 73), (21, 73), (21, 74), (20, 73), (18, 74), (18, 75), (20, 78), (22, 75), (23, 74), (24, 75), (25, 74), (26, 74), (28, 74), (28, 72), (27, 77), (32, 77), (33, 79), (34, 79), (35, 78), (36, 77), (35, 77), (35, 79), (34, 77), (36, 78), (36, 76), (35, 76), (37, 73), (37, 77), (37, 78), (37, 75), (36, 79), (37, 76), (38, 76), (38, 77), (39, 77), (40, 78), (39, 85), (40, 86), (39, 87), (38, 85), (38, 86), (37, 85), (35, 84), (34, 84), (35, 83), (34, 82), (34, 83), (42, 84), (41, 85), (42, 86), (41, 87), (40, 88), (39, 88), (34, 88), (33, 88), (32, 87), (32, 88), (31, 89), (32, 89), (30, 89), (32, 90), (32, 92), (30, 91), (30, 90), (31, 88), (30, 87), (31, 86), (31, 87), (30, 84), (29, 82), (26, 84), (25, 84), (26, 83), (27, 85), (25, 85), (24, 86), (30, 83), (28, 81), (33, 80), (31, 85), (33, 86), (33, 85), (34, 85), (35, 86), (34, 87), (33, 84), (32, 80), (32, 84), (28, 79), (28, 78), (35, 80), (39, 78), (40, 77), (39, 76), (40, 75), (41, 74), (42, 72), (41, 73), (41, 75), (51, 77), (50, 80), (55, 82), (56, 81), (56, 80), (56, 78), (64, 74), (61, 74), (60, 79), (61, 82), (59, 84), (61, 79), (57, 71), (55, 73), (54, 72), (54, 73), (55, 72), (56, 74), (55, 71), (56, 70), (55, 69), (56, 68), (57, 69), (57, 70), (56, 69), (55, 68), (54, 69), (55, 70), (57, 67), (55, 63), (58, 67), (60, 63), (61, 63), (61, 64), (62, 63), (59, 62), (58, 60), (58, 61), (60, 61), (58, 57), (59, 57), (58, 56), (57, 56), (57, 55), (60, 55), (59, 56), (60, 56), (61, 55), (62, 56), (63, 56), (60, 57), (60, 54), (61, 54), (62, 53), (62, 52), (61, 53), (61, 51), (60, 50), (59, 51), (56, 53), (55, 52), (54, 51), (53, 52), (53, 53), (52, 54), (53, 55), (54, 54), (54, 55), (53, 54), (55, 55), (55, 54), (58, 50), (57, 49), (55, 50), (54, 49), (53, 50), (52, 51), (52, 53), (54, 53), (54, 52), (53, 51), (52, 52), (51, 52), (51, 51), (52, 50), (56, 54), (55, 53), (55, 57), (59, 53), (55, 51), (54, 50), (56, 51), (54, 48), (53, 47), (52, 47), (52, 48), (53, 49), (55, 48), (55, 49), (54, 47), (54, 46), (55, 46), (55, 47), (53, 48), (59, 48), (58, 47), (60, 48), (59, 49), (59, 52), (60, 52), (60, 53), (62, 49), (63, 50), (50, 50), (51, 53), (50, 54), (50, 53), (51, 50), (50, 51), (51, 49), (50, 49), (49, 48), (45, 45), (44, 44), (45, 40), (44, 39), (43, 39), (42, 39), (42, 40), (41, 41), (41, 40), (40, 40), (40, 39), (39, 40), (39, 39), (41, 39), (41, 38), (42, 38), (43, 38), (40, 38), (39, 38), (38, 37), (37, 36), (37, 37), (36, 36), (37, 35), (36, 37), (37, 38), (38, 36), (39, 35), (40, 35), (39, 34), (38, 34), (36, 35), (37, 34), (35, 35), (34, 34), (34, 35), (34, 36), (35, 37), (34, 37), (33, 36), (32, 36), (33, 35), (33, 33), (34, 32), (35, 31), (34, 31), (34, 30), (33, 31), (33, 30), (33, 29), (32, 29), (31, 28), (32, 27), (30, 29), (30, 28), (30, 30), (29, 30), (28, 29), (29, 29), (28, 30), (27, 30), (27, 31), (28, 31), (29, 31), (29, 32), (30, 32), (30, 33), (29, 34), (28, 36), (29, 37), (29, 36), (30, 36), (31, 37), (32, 38), (33, 39), (35, 38), (33, 37), (32, 37), (31, 36), (30, 37), (29, 35), (30, 35), (30, 34), (29, 33), (28, 32), (28, 35), (25, 37), (19, 33), (18, 33), (18, 37), (18, 35), (19, 34), (18, 34), (19, 35), (17, 37), (18, 36), (17, 38), (16, 38), (16, 39), (18, 38), (18, 39), (19, 40), (18, 40), (18, 41), (20, 38), (17, 36), (16, 35), (16, 34), (17, 35), (21, 32), (22, 31), (19, 39), (17, 39), (15, 38), (15, 37), (16, 37), (15, 36), (14, 36), (13, 35), (13, 37), (14, 38), (14, 39), (14, 40), (15, 41), (15, 42), (15, 39), (15, 40), (14, 41), (13, 41), (12, 40), (11, 41), (11, 40), (12, 41), (12, 39), (13, 39), (13, 40), (11, 39), (16, 41), (16, 40), (15, 43), (16, 43), (17, 42), (18, 43), (19, 42), (19, 41), (23, 40), (24, 39), (25, 43), (28, 44), (29, 45), (30, 44), (31, 43), (32, 43), (32, 42), (33, 42), (34, 41), (33, 43), (33, 44), (28, 38), (28, 39), (22, 42), (24, 41), (18, 32), (25, 31), (26, 31), (27, 29), (27, 28), (26, 29), (26, 28), (31, 29), (31, 31), (32, 32), (31, 33), (31, 32), (30, 31), (31, 30), (32, 30), (34, 33), (33, 34), (36, 34), (35, 33), (33, 32), (35, 32), (35, 34), (36, 38), (35, 36), (38, 39), (40, 37), (40, 36), (41, 36), (41, 35), (39, 37), (40, 41), (41, 42), (41, 43), (42, 42), (42, 41), (38, 40), (37, 41), (38, 41), (39, 41), (41, 37), (38, 38), (37, 39), (39, 42), (40, 42), (42, 37), (43, 40), (44, 41), (43, 41), (36, 32), (34, 29), (35, 29), (36, 30), (37, 31), (38, 31), (37, 30), (36, 29), (35, 30), (32, 31), (36, 31), (38, 30), (36, 28), (37, 29), (37, 33), (31, 38), (30, 38), (29, 38), (31, 34), (32, 35), (32, 33), (17, 41), (18, 42), (19, 43), (17, 44), (17, 43), (18, 44), (19, 44), (19, 45), (19, 46), (20, 47), (20, 46), (27, 46), (28, 45), (27, 44), (28, 43), (31, 44), (32, 44), (34, 44), (35, 45), (35, 44), (36, 46), (36, 45), (35, 43), (36, 42), (35, 41), (35, 40), (36, 39), (43, 42), (43, 43), (44, 43), (45, 46), (43, 44), (42, 45), (43, 45), (42, 44), (41, 45), (41, 44), (42, 43), (44, 42), (40, 45), (41, 46), (40, 47), (41, 48), (42, 47), (42, 48), (42, 46), (41, 47), (40, 46), (35, 47), (35, 48), (34, 47), (32, 52), (32, 51), (31, 51), (30, 50), (29, 50), (29, 49), (30, 46), (28, 46), (28, 47), (30, 51), (29, 52), (29, 53), (29, 54), (28, 53), (28, 54), (27, 53), (27, 54), (27, 55), (28, 56), (29, 57), (29, 58), (29, 59), (28, 59), (29, 60), (31, 60), (32, 60), (33, 59), (33, 61), (32, 61), (31, 61), (30, 60), (34, 61), (35, 60), (34, 60), (36, 58), (36, 57), (36, 54), (39, 54), (33, 52), (32, 53), (33, 51), (42, 52), (44, 54), (44, 53), (45, 53), (46, 53), (45, 54), (44, 55), (45, 55), (46, 55), (45, 56), (46, 57), (47, 56), (48, 55), (47, 54), (47, 53), (47, 52), (48, 52), (47, 51), (46, 52), (48, 53), (49, 53), (50, 52), (48, 51), (49, 52), (45, 51), (44, 52), (43, 51), (44, 50), (44, 47), (47, 46), (50, 47), (51, 48), (51, 47), (52, 46), (51, 46), (51, 45), (50, 48), (52, 49), (56, 49), (56, 48), (57, 47), (58, 46), (57, 46), (57, 45), (58, 44), (59, 43), (58, 43), (57, 44), (57, 43), (58, 45), (56, 44), (55, 43), (54, 43), (55, 42), (54, 44), (53, 44), (54, 45), (53, 46), (53, 45), (55, 44), (56, 45), (55, 45), (53, 43), (53, 42), (46, 42), (44, 37), (43, 37), (42, 36), (39, 36), (37, 42), (38, 42), (37, 40), (36, 40), (36, 41), (34, 38), (36, 33), (37, 32), (38, 35), (38, 33), (40, 34), (34, 39), (33, 40), (33, 41), (34, 40), (35, 39), (31, 35), (32, 34), (32, 28), (33, 28), (26, 30), (21, 30), (22, 29), (23, 29), (24, 28), (22, 28), (23, 28), (24, 29), (25, 29), (25, 27), (23, 27), (24, 26), (23, 25), (24, 25), (25, 25), (26, 25), (27, 26), (26, 26), (25, 26), (26, 27), (27, 27), (28, 27), (29, 27), (30, 27), (30, 26), (31, 27), (31, 26), (30, 25), (32, 26), (29, 28), (29, 26), (29, 25), (33, 27), (34, 28), (38, 32), (39, 33), (30, 39), (43, 36), (36, 43), (37, 44), (37, 45), (36, 44), (34, 45), (33, 46), (32, 49), (31, 50), (30, 52), (28, 52), (29, 51), (26, 52), (25, 52), (26, 53), (26, 54), (29, 55), (29, 56), (28, 57), (27, 58), (27, 59), (27, 60), (26, 60), (20, 61), (22, 68), (23, 68), (21, 69), (20, 68), (19, 60), (12, 70), (20, 70), (19, 61), (26, 75), (25, 76), (26, 77), (26, 78), (27, 79), (27, 78), (25, 79), (22, 79), (21, 85), (21, 84), (22, 81), (22, 84), (24, 79), (23, 80), (21, 77), (23, 78), (23, 77), (24, 77), (24, 78), (26, 82), (28, 80), (34, 81), (38, 84), (41, 89), (42, 89), (42, 91), (41, 94), (40, 95), (37, 94), (35, 90), (36, 94), (35, 94), (34, 94), (33, 94), (34, 95), (34, 96), (35, 97), (36, 96), (36, 97), (35, 96), (35, 95), (33, 95), (37, 95), (38, 95), (35, 87), (33, 89), (29, 90), (29, 89), (28, 90), (28, 91), (26, 92), (32, 93), (35, 93), (34, 92), (33, 98), (30, 101), (29, 101), (28, 100), (27, 101), (26, 100), (26, 99), (25, 99), (27, 100), (28, 99), (27, 102), (25, 100), (24, 101), (23, 102), (22, 102), (21, 102), (22, 101), (21, 104), (21, 103), (29, 98), (27, 98), (26, 97), (25, 97), (28, 102), (27, 103), (23, 106), (23, 107), (22, 105), (22, 106), (21, 107), (21, 108), (22, 107), (23, 108), (25, 109), (26, 109), (21, 101), (24, 94), (27, 90), (30, 97), (28, 98), (27, 99), (34, 90), (33, 90), (26, 85), (24, 84), (23, 85), (22, 88), (21, 86), (17, 82), (20, 81), (24, 76), (23, 76), (25, 78), (25, 77), (26, 76), (23, 75), (20, 75), (20, 74), (22, 74), (21, 75), (22, 76), (25, 75), (24, 74), (29, 79), (43, 83), (43, 84), (42, 85), (41, 86), (41, 88), (27, 84), (25, 81), (17, 83), (13, 85), (13, 91), (13, 90), (20, 100), (19, 99), (19, 101), (20, 101), (20, 102), (22, 103), (20, 103), (20, 104), (19, 103), (19, 102), (18, 101), (18, 100), (19, 100), (22, 108), (20, 107), (20, 109), (27, 108), (27, 109), (27, 110), (29, 110), (28, 109), (26, 113), (25, 113), (36, 107), (34, 105), (36, 105), (35, 104), (34, 104), (35, 105), (37, 106), (38, 105), (38, 109), (33, 105), (33, 104), (33, 103), (32, 104), (32, 103), (31, 102), (31, 103), (32, 102), (31, 101), (30, 102), (17, 101), (17, 100), (16, 101), (17, 99), (16, 100), (15, 101), (16, 102), (15, 102), (14, 101), (14, 102), (13, 103), (12, 103), (11, 102), (12, 102), (10, 102), (9, 103), (9, 102), (10, 101), (9, 100), (10, 100), (9, 101), (9, 104), (8, 105), (9, 105), (9, 106), (10, 105), (10, 104), (11, 105), (12, 105), (11, 104), (11, 103), (13, 102), (12, 101), (8, 102), (7, 102), (6, 103), (7, 103), (8, 101), (6, 102), (5, 103), (6, 104), (5, 102), (5, 101), (6, 100), (5, 99), (5, 100), (7, 100), (8, 100), (9, 99), (10, 99), (9, 98), (10, 97), (9, 96), (8, 95), (7, 96), (6, 96), (7, 97), (7, 98), (6, 98), (5, 97), (6, 97), (5, 98), (8, 98), (8, 97), (11, 98), (11, 97), (12, 97), (13, 98), (12, 98), (10, 98), (13, 96), (14, 95), (15, 96), (15, 95), (17, 98), (16, 98), (18, 102), (18, 103), (17, 103), (18, 104), (23, 101), (21, 89), (22, 86), (23, 84), (27, 82), (24, 85), (15, 83), (23, 86), (23, 87), (22, 87), (22, 85), (16, 81), (8, 76), (15, 76), (6, 72), (39, 71), (39, 74), (40, 74), (40, 71), (39, 75), (38, 75), (37, 84), (36, 85), (17, 74), (21, 72), (22, 71), (21, 70), (20, 69), (15, 93), (16, 94), (12, 92), (12, 94), (11, 94), (13, 95), (14, 96), (14, 97), (16, 99), (15, 98), (14, 99), (13, 94), (14, 94), (12, 95), (11, 95), (10, 95), (10, 94), (9, 93), (10, 92), (11, 91), (12, 91), (12, 90), (8, 77), (5, 78), (5, 71), (24, 63), (26, 62), (28, 61), (29, 61), (32, 59), (33, 60), (31, 59), (31, 58), (32, 57), (32, 56), (31, 57), (30, 57), (27, 56), (28, 55), (30, 55), (31, 54), (31, 53), (30, 53), (30, 54), (31, 52), (32, 50), (31, 49), (33, 50), (41, 50), (39, 45), (39, 44), (38, 44), (40, 44), (44, 36), (44, 35), (43, 35), (45, 36), (45, 35), (44, 34), (51, 44), (52, 43), (52, 45), (52, 44), (54, 41), (54, 42), (58, 48), (64, 51), (64, 50), (64, 49), (66, 51), (65, 51), (62, 54), (62, 55), (66, 54), (67, 54), (65, 50), (67, 53), (67, 55), (68, 54), (72, 57), (72, 56), (72, 55), (71, 58), (72, 59), (71, 59), (69, 66), (68, 64), (68, 63), (64, 62), (63, 63), (61, 62), (60, 62), (61, 61), (60, 60), (62, 61), (69, 60), (69, 61), (71, 65), (71, 73), (73, 71), (74, 70), (74, 71), (73, 72), (74, 72), (75, 71), (75, 73), (76, 72), (76, 71), (77, 72), (77, 73), (77, 75), (77, 76), (76, 75), (76, 76), (75, 74), (74, 75), (75, 76), (74, 77), (75, 77), (76, 78), (75, 79), (74, 78), (74, 79), (75, 80), (74, 80), (73, 79), (72, 80), (73, 80), (71, 80), (72, 81), (72, 82), (73, 81), (71, 81), (73, 82), (73, 83), (74, 82), (75, 83), (75, 84), (76, 83), (76, 84), (77, 82), (79, 81), (81, 78), (82, 79), (83, 78), (82, 77), (93, 83), (94, 86), (94, 87), (93, 87), (95, 85), (94, 84), (97, 83), (95, 83), (95, 84), (94, 85), (95, 86), (96, 87), (96, 86), (97, 86), (98, 86), (98, 87), (100, 87), (103, 91), (105, 92), (104, 92), (103, 93), (102, 92), (102, 93), (104, 93), (106, 97), (103, 97), (103, 102), (105, 102), (105, 103), (106, 101), (110, 95), (108, 101), (106, 98), (107, 97), (114, 97), (113, 96), (116, 96), (115, 96), (115, 95), (114, 95), (112, 87), (111, 86), (113, 86), (114, 85), (114, 84), (114, 86), (115, 86), (116, 85), (116, 87), (113, 87), (102, 89), (110, 82), (109, 82), (108, 79), (107, 79), (106, 81), (105, 80), (104, 80), (109, 84), (105, 79), (103, 78), (104, 79), (102, 78), (101, 77), (101, 78), (106, 79), (105, 78), (106, 78), (106, 77), (104, 74), (104, 75), (103, 74), (91, 73), (90, 72), (94, 68), (107, 76), (107, 78), (108, 78), (107, 77), (80, 78), (80, 77), (82, 75), (83, 79), (83, 77), (86, 79), (85, 80), (84, 79), (81, 80), (80, 81), (80, 82), (80, 84), (78, 84), (77, 84), (78, 83), (79, 83), (81, 84), (81, 87), (80, 88), (79, 89), (78, 90), (79, 91), (79, 92), (80, 93), (81, 93), (81, 92), (80, 90), (77, 83), (78, 85), (79, 87), (80, 87), (78, 86), (77, 86), (78, 87), (79, 80), (79, 79), (77, 77), (76, 79), (73, 78), (72, 78), (72, 77), (72, 76), (73, 75), (71, 75), (71, 74), (70, 74), (69, 73), (70, 73), (69, 72), (69, 71), (68, 72), (68, 73), (67, 74), (68, 75), (69, 76), (70, 77), (71, 78), (71, 77), (72, 79), (74, 81), (76, 80), (79, 88), (77, 88), (76, 88), (77, 87), (76, 87), (75, 87), (75, 88), (74, 88), (75, 89), (74, 89), (74, 90), (73, 89), (72, 89), (73, 88), (73, 87), (72, 90), (73, 90), (71, 91), (72, 92), (73, 93), (73, 94), (74, 94), (74, 95), (73, 96), (74, 96), (75, 96), (75, 95), (76, 96), (76, 97), (77, 96), (76, 95), (75, 97), (77, 95), (77, 94), (78, 95), (78, 96), (79, 95), (78, 94), (78, 93), (77, 92), (78, 91), (77, 90), (77, 91), (76, 93), (75, 94), (75, 93), (72, 95), (72, 94), (71, 95), (70, 96), (70, 97), (68, 96), (67, 97), (68, 97), (69, 96), (69, 97), (70, 98), (71, 99), (71, 97), (72, 98), (72, 99), (73, 99), (74, 100), (75, 101), (75, 100), (76, 100), (76, 101), (75, 99), (76, 98), (76, 99), (77, 98), (77, 97), (75, 98), (74, 98), (74, 99), (73, 97), (72, 97), (71, 96), (72, 96), (71, 94), (70, 95), (68, 95), (68, 98), (67, 95), (68, 94), (73, 98), (71, 98), (73, 100), (70, 100), (71, 101), (72, 100), (69, 101), (68, 101), (68, 102), (68, 103), (68, 104), (68, 105), (68, 106), (69, 105), (70, 104), (70, 103), (69, 102), (71, 100), (72, 101), (70, 101), (73, 101), (74, 101), (74, 102), (76, 102), (76, 103), (76, 104), (77, 103), (75, 103), (75, 104), (75, 105), (74, 105), (75, 106), (74, 106), (78, 103), (79, 104), (78, 104), (78, 105), (80, 104), (79, 105), (79, 106), (78, 106), (77, 106), (76, 106), (73, 105), (72, 105), (73, 104), (73, 103), (72, 104), (74, 104), (74, 103), (78, 102), (78, 101), (79, 101), (80, 100), (79, 99), (78, 99), (80, 99), (81, 100), (81, 101), (82, 99), (81, 98), (82, 97), (81, 97), (80, 96), (74, 97), (74, 92), (75, 91), (76, 89), (75, 90), (74, 87), (75, 86), (76, 86), (76, 85), (74, 83), (72, 83), (73, 84), (73, 85), (74, 84), (74, 85), (73, 86), (72, 87), (72, 86), (71, 88), (71, 89), (70, 90), (69, 90), (67, 87), (67, 86), (67, 85), (66, 85), (65, 84), (66, 84), (67, 83), (67, 82), (68, 84), (68, 83), (69, 82), (70, 83), (69, 83), (65, 85), (64, 84), (65, 83), (66, 83), (66, 82), (65, 81), (65, 82), (68, 86), (67, 84), (68, 85), (69, 84), (68, 82), (67, 81), (67, 80), (67, 79), (68, 80), (66, 80), (66, 81), (68, 78), (66, 78), (65, 80), (64, 79), (63, 78), (63, 81), (62, 81), (57, 78), (62, 82), (64, 80), (62, 86), (59, 88), (58, 88), (57, 86), (57, 85), (58, 85), (55, 83), (55, 84), (55, 85), (56, 84), (65, 79), (66, 79), (67, 78), (69, 77), (70, 78), (71, 76), (70, 75), (70, 76), (72, 73), (72, 74), (73, 73), (76, 74), (75, 75), (74, 74), (76, 73), (77, 74), (78, 76), (79, 75), (77, 78), (75, 72), (78, 72), (79, 71), (80, 72), (82, 73), (83, 72), (83, 67), (82, 66), (83, 73), (83, 74), (82, 69), (80, 70), (80, 71), (79, 69), (74, 69), (75, 70), (73, 70), (73, 69), (71, 66), (68, 71), (66, 72), (67, 72), (66, 73), (65, 74), (65, 75), (66, 76), (65, 76), (66, 75), (66, 74), (67, 73), (61, 69), (60, 69), (60, 68), (59, 68), (58, 69), (54, 71), (53, 70), (53, 68), (58, 70), (58, 68), (57, 68), (57, 60), (55, 60), (55, 59), (54, 58), (54, 59), (54, 56), (67, 52), (68, 51), (68, 49), (66, 45), (66, 44), (67, 44), (67, 43), (68, 43), (66, 43), (67, 42), (66, 41), (66, 40), (66, 39), (67, 38), (65, 40), (65, 39), (67, 40), (68, 39), (68, 40), (68, 41), (69, 42), (69, 41), (70, 44), (70, 43), (65, 46), (64, 45), (64, 44), (64, 48), (63, 43), (63, 47), (57, 48), (56, 47), (56, 46), (53, 56), (52, 57), (53, 57), (54, 60), (55, 61), (58, 58), (58, 59), (62, 48), (61, 56), (53, 58), (52, 58), (51, 59), (50, 58), (51, 58), (50, 59), (50, 60), (49, 60), (49, 58), (50, 57), (50, 56), (51, 57), (52, 55), (51, 55), (52, 56), (53, 59), (52, 59), (53, 60), (53, 61), (52, 61), (51, 60), (51, 56), (49, 49), (41, 66), (42, 67), (43, 67), (41, 69), (40, 69), (42, 66), (45, 65), (44, 65), (44, 64), (43, 64), (42, 63), (42, 64), (43, 65), (41, 64), (43, 63), (41, 63), (44, 62), (44, 61), (43, 60), (43, 59), (42, 60), (44, 60), (45, 61), (45, 60), (44, 59), (41, 58), (43, 61), (45, 62), (45, 63), (43, 62), (44, 63), (42, 61), (41, 62), (43, 55), (43, 56), (44, 56), (47, 57), (46, 56), (48, 57), (49, 57), (51, 54), (49, 51), (56, 41), (56, 42), (56, 43), (52, 39), (48, 40), (45, 47), (49, 50), (48, 34), (49, 35), (50, 34), (51, 33), (52, 33), (51, 34), (53, 33), (54, 33), (54, 32), (53, 32), (51, 32), (50, 33), (52, 34), (53, 34), (53, 35), (54, 34), (52, 35), (52, 31), (51, 30), (52, 30), (51, 31), (51, 35), (53, 39), (52, 38), (53, 38), (54, 38), (54, 39), (55, 40), (56, 39), (56, 38), (57, 39), (56, 40), (55, 41), (58, 42), (59, 42), (57, 42), (57, 41), (57, 40), (58, 40), (59, 40), (59, 41), (60, 40), (61, 40), (61, 41), (61, 39), (59, 39), (60, 38), (60, 39), (61, 38), (61, 37), (60, 37), (61, 36), (60, 36), (62, 36), (63, 35), (64, 36), (63, 36), (63, 34), (64, 35), (62, 33), (63, 32), (62, 32), (61, 31), (62, 31), (62, 30), (63, 30), (63, 31), (61, 30), (61, 32), (63, 33), (62, 34), (61, 34), (62, 35), (61, 35), (60, 34), (59, 34), (58, 34), (57, 33), (58, 33), (59, 32), (58, 31), (58, 30), (57, 30), (58, 29), (58, 28), (59, 28), (59, 27), (60, 27), (60, 28), (60, 29), (61, 27), (60, 26), (61, 29), (62, 28), (61, 28), (62, 29), (60, 30), (59, 30), (59, 29), (59, 31), (60, 32), (60, 31), (59, 38), (59, 37), (58, 36), (59, 36), (57, 35), (56, 36), (57, 37), (57, 36), (58, 35), (58, 37), (58, 38), (58, 39), (57, 38), (60, 35), (60, 33), (57, 34), (58, 32), (57, 32), (56, 34), (56, 35), (59, 35), (63, 37), (62, 38), (63, 39), (63, 40), (62, 40), (62, 42), (61, 42), (61, 43), (60, 42), (60, 43), (58, 41), (62, 43), (60, 41), (62, 44), (64, 43), (64, 42), (65, 43), (65, 44), (71, 46), (66, 42), (65, 42), (64, 41), (63, 41), (62, 41), (62, 37), (65, 35), (66, 34), (67, 35), (68, 36), (67, 37), (66, 38), (67, 39), (67, 41), (69, 40), (70, 41), (68, 42), (65, 41), (68, 38), (69, 39), (71, 47), (72, 48), (72, 46), (73, 45), (74, 45), (73, 44), (72, 45), (73, 46), (74, 46), (75, 47), (75, 48), (74, 47), (74, 48), (76, 48), (77, 48), (78, 47), (79, 47), (80, 47), (81, 45), (79, 46), (79, 45), (78, 46), (81, 47), (81, 48), (82, 47), (82, 46), (83, 47), (83, 45), (84, 45), (84, 44), (80, 48), (79, 48), (79, 44), (78, 43), (77, 44), (78, 44), (76, 45), (75, 46), (72, 47), (69, 48), (69, 49), (69, 50), (60, 47), (63, 49), (55, 62), (54, 63), (53, 62), (52, 60), (51, 61), (52, 62), (51, 63), (52, 64), (51, 65), (51, 64), (50, 62), (49, 63), (48, 62), (47, 63), (47, 62), (47, 61), (46, 60), (46, 59), (47, 60), (46, 61), (46, 62), (46, 63), (48, 61), (49, 61), (49, 62), (50, 63), (49, 64), (49, 65), (50, 65), (50, 64), (48, 65), (51, 62), (52, 63), (49, 59), (49, 56), (49, 55), (49, 54), (48, 54), (46, 54), (47, 55), (50, 61), (52, 66), (51, 68), (63, 64), (66, 63), (70, 72), (71, 71), (72, 69), (73, 68), (89, 53), (91, 52), (94, 48), (92, 48), (91, 48), (92, 47), (92, 46), (93, 45), (93, 44), (92, 44), (92, 45), (91, 43), (91, 44), (91, 45), (94, 44), (93, 43), (95, 42), (96, 43), (96, 42), (97, 40), (92, 39), (92, 40), (95, 39), (96, 38), (97, 38), (97, 39), (98, 39), (95, 38), (94, 37), (93, 38), (92, 38), (91, 37), (91, 36), (90, 36), (90, 35), (89, 34), (90, 34), (90, 33), (89, 33), (88, 33), (89, 32), (88, 31), (89, 30), (89, 29), (90, 28), (91, 27), (92, 28), (92, 27), (93, 27), (94, 28), (94, 29), (95, 30), (95, 29), (96, 28), (95, 27), (94, 26), (93, 28), (94, 27), (93, 26), (92, 26), (91, 25), (90, 26), (89, 25), (89, 26), (91, 26), (91, 28), (90, 25), (92, 24), (93, 24), (94, 23), (95, 24), (95, 23), (95, 22), (94, 24), (96, 23), (96, 24), (96, 22), (97, 21), (97, 20), (98, 20), (98, 19), (98, 18), (97, 18), (97, 19), (96, 20), (95, 19), (93, 19), (95, 17), (96, 19), (96, 18), (91, 15), (89, 15), (89, 16), (88, 15), (87, 15), (87, 16), (86, 16), (86, 15), (87, 14), (87, 13), (88, 14), (85, 15), (86, 14), (85, 13), (86, 12), (88, 12), (88, 13), (87, 12), (88, 11), (88, 10), (87, 11), (89, 10), (89, 9), (90, 10), (92, 13), (89, 11), (90, 11), (89, 12), (89, 13), (88, 16), (87, 17), (88, 17), (86, 11), (85, 10), (84, 9), (83, 9), (82, 8), (81, 7), (81, 8), (81, 9), (80, 8), (80, 10), (81, 11), (81, 12), (80, 12), (80, 11), (81, 13), (82, 14), (83, 15), (82, 16), (81, 15), (80, 16), (79, 16), (80, 17), (80, 15), (81, 16), (82, 15), (83, 13), (84, 13), (83, 12), (82, 13), (81, 14), (83, 14), (84, 16), (83, 16), (84, 15), (85, 16), (85, 17), (84, 17), (83, 18), (84, 19), (85, 19), (84, 18), (83, 19), (84, 20), (84, 21), (85, 22), (83, 21), (84, 22), (86, 23), (85, 23), (86, 22), (85, 21), (86, 20), (83, 17), (81, 17), (84, 14), (85, 14), (80, 13), (79, 12), (79, 10), (79, 9), (82, 11), (82, 10), (83, 8), (82, 9), (81, 10), (82, 12), (84, 10), (85, 9), (85, 11), (85, 8), (86, 8), (87, 7), (88, 6), (89, 5), (88, 5), (89, 6), (90, 5), (91, 6), (92, 5), (93, 5), (93, 6), (92, 7), (93, 7), (91, 7), (91, 5), (90, 6), (87, 6), (86, 6), (85, 7), (86, 7), (88, 7), (89, 7), (90, 7), (92, 8), (94, 8), (93, 9), (98, 7), (96, 9), (96, 10), (97, 10), (94, 15), (95, 15), (96, 13), (94, 9), (88, 9), (88, 8), (87, 5), (85, 6), (84, 6), (83, 6), (82, 7), (84, 7), (81, 6), (82, 5), (83, 5), (84, 5), (85, 5), (86, 5), (83, 7), (84, 8), (84, 11), (83, 10), (83, 11), (86, 9), (82, 6), (80, 6), (80, 7), (79, 8), (80, 9), (79, 11), (79, 14), (80, 14), (83, 20), (82, 17), (83, 22), (82, 23), (82, 24), (83, 25), (83, 26), (84, 25), (84, 26), (82, 26), (81, 26), (81, 25), (80, 25), (79, 26), (79, 27), (80, 27), (80, 28), (79, 24), (78, 23), (77, 22), (78, 22), (77, 21), (76, 22), (76, 21), (77, 20), (78, 19), (78, 20), (78, 21), (79, 18), (78, 17), (79, 17), (79, 19), (80, 19), (81, 20), (80, 20), (79, 20), (79, 21), (80, 22), (80, 23), (79, 23), (79, 22), (80, 21), (81, 22), (77, 19), (77, 18), (78, 18), (77, 17), (78, 16), (79, 15), (79, 13), (78, 12), (78, 13), (78, 11), (77, 10), (77, 9), (76, 10), (75, 9), (76, 9), (77, 8), (76, 8), (77, 13), (76, 13), (76, 12), (75, 12), (74, 12), (74, 11), (75, 10), (74, 9), (74, 8), (73, 8), (72, 8), (71, 8), (70, 9), (69, 9), (69, 10), (69, 8), (68, 8), (70, 8), (71, 9), (70, 10), (71, 11), (72, 11), (71, 12), (72, 12), (72, 13), (72, 14), (73, 15), (72, 15), (71, 16), (70, 16), (69, 17), (70, 18), (71, 18), (71, 17), (72, 17), (72, 18), (70, 17), (71, 19), (72, 19), (73, 20), (72, 20), (73, 21), (74, 22), (73, 22), (72, 21), (74, 20), (73, 19), (71, 21), (70, 22), (69, 21), (68, 20), (67, 19), (67, 20), (66, 21), (66, 20), (67, 21), (68, 22), (68, 21), (67, 22), (69, 23), (69, 24), (68, 25), (69, 26), (68, 27), (68, 26), (67, 26), (66, 27), (65, 27), (64, 27), (65, 26), (66, 26), (67, 27), (67, 24), (68, 24), (69, 25), (67, 23), (66, 24), (65, 25), (65, 24), (66, 25), (64, 26), (63, 25), (63, 27), (63, 28), (62, 27), (62, 26), (64, 24), (64, 25), (63, 26), (59, 26), (61, 33), (57, 31), (56, 33), (56, 32), (56, 31), (55, 32), (59, 33), (64, 34), (65, 34), (65, 36), (66, 36), (66, 35), (65, 33), (66, 33), (67, 32), (68, 32), (69, 31), (67, 31), (66, 31), (67, 30), (68, 29), (67, 28), (68, 28), (67, 29), (66, 29), (68, 31), (68, 30), (69, 32), (69, 30), (70, 32), (69, 33), (70, 33), (69, 34), (70, 34), (68, 34), (67, 33), (66, 32), (65, 32), (64, 31), (64, 32), (64, 33), (68, 33), (64, 23), (63, 23), (62, 24), (63, 22), (62, 23), (61, 24), (61, 23), (62, 22), (61, 21), (60, 22), (59, 22), (59, 23), (59, 24), (60, 25), (58, 24), (57, 23), (56, 24), (55, 23), (56, 22), (56, 23), (57, 22), (55, 24), (54, 25), (55, 26), (55, 25), (53, 25), (53, 26), (53, 27), (52, 26), (51, 27), (51, 28), (52, 29), (53, 29), (54, 30), (53, 30), (53, 31), (54, 28), (54, 29), (55, 28), (56, 27), (57, 28), (56, 25), (57, 24), (58, 22), (58, 23), (56, 21), (55, 22), (54, 22), (53, 22), (55, 21), (55, 20), (55, 19), (56, 19), (57, 19), (56, 20), (57, 20), (58, 20), (58, 19), (57, 21), (58, 21), (59, 19), (60, 18), (59, 18), (59, 20), (60, 20), (61, 20), (62, 20), (62, 21), (63, 20), (61, 19), (62, 18), (63, 19), (61, 18), (62, 17), (63, 17), (64, 17), (65, 17), (65, 16), (65, 15), (64, 15), (64, 16), (63, 16), (62, 16), (62, 15), (61, 15), (61, 14), (62, 14), (63, 15), (61, 13), (60, 13), (59, 12), (59, 11), (58, 10), (57, 9), (56, 8), (55, 8), (54, 7), (53, 6), (53, 5), (54, 5), (55, 5), (56, 5), (57, 6), (57, 5), (58, 7), (59, 6), (58, 6), (60, 6), (61, 5), (61, 6), (61, 7), (62, 6), (62, 5), (63, 7), (63, 6), (62, 7), (64, 6), (65, 6), (66, 5), (67, 6), (68, 7), (68, 9), (68, 10), (69, 11), (68, 11), (69, 12), (68, 12), (67, 13), (66, 13), (67, 12), (66, 12), (68, 13), (67, 11), (70, 11), (73, 12), (73, 11), (74, 10), (76, 11), (77, 11), (77, 14), (77, 15), (76, 14), (78, 15), (78, 14), (75, 15), (76, 15), (77, 16), (76, 17), (75, 14), (75, 13), (75, 8), (73, 9), (75, 11), (78, 10), (78, 9), (79, 6), (78, 6), (78, 7), (78, 8), (79, 7), (81, 5), (79, 5), (80, 5), (87, 9), (86, 10), (84, 12), (85, 12), (87, 8), (90, 9), (91, 8), (89, 8), (91, 9), (92, 6), (94, 5), (95, 14), (94, 16), (91, 16), (97, 13), (92, 14), (90, 16), (90, 17), (89, 17), (90, 18), (89, 18), (89, 19), (90, 20), (91, 21), (92, 21), (91, 22), (92, 22), (93, 21), (93, 22), (94, 22), (92, 23), (91, 23), (90, 22), (89, 21), (90, 21), (92, 25), (95, 21), (97, 23), (98, 21), (99, 20), (100, 19), (93, 23), (95, 25), (94, 25), (95, 26), (94, 21), (98, 22), (90, 23), (91, 24), (89, 23), (90, 24), (90, 27), (89, 27), (91, 29), (90, 30), (91, 31), (90, 32), (91, 32), (92, 33), (91, 34), (92, 36), (93, 37), (94, 36), (95, 36), (94, 35), (93, 35), (92, 37), (91, 38), (90, 37), (90, 39), (91, 40), (96, 37), (97, 36), (98, 36), (98, 37), (98, 38), (96, 39), (97, 37), (96, 36), (97, 35), (99, 38), (100, 39), (102, 39), (103, 38), (102, 38), (101, 38), (103, 37), (102, 37), (102, 36), (101, 37), (104, 38), (104, 40), (101, 45), (102, 45), (100, 44), (95, 43), (101, 44), (103, 46), (103, 47), (91, 41), (91, 39), (89, 38), (90, 38), (89, 39), (88, 39), (87, 40), (89, 40), (88, 42), (86, 39), (87, 39), (86, 38), (85, 39), (85, 38), (85, 40), (84, 39), (83, 38), (84, 38), (83, 39), (82, 39), (82, 37), (81, 38), (82, 38), (83, 40), (87, 38), (86, 37), (86, 43), (87, 43), (84, 40), (84, 41), (83, 41), (83, 42), (82, 42), (81, 42), (80, 41), (79, 40), (78, 41), (79, 41), (80, 40), (78, 39), (79, 39), (78, 38), (79, 37), (80, 37), (79, 38), (80, 39), (81, 40), (81, 39), (80, 38), (78, 40), (80, 36), (81, 37), (81, 36), (79, 36), (79, 35), (80, 35), (81, 34), (81, 33), (82, 32), (81, 32), (80, 32), (82, 31), (83, 31), (84, 30), (84, 31), (79, 32), (78, 33), (78, 32), (79, 34), (80, 33), (79, 33), (80, 34), (82, 33), (83, 33), (84, 32), (85, 32), (85, 31), (84, 33), (85, 34), (86, 34), (87, 34), (88, 34), (87, 35), (88, 35), (87, 36), (87, 37), (88, 37), (90, 40), (93, 36), (96, 35), (95, 34), (95, 37), (89, 37), (88, 36), (88, 38), (88, 43), (89, 43), (90, 43), (89, 42), (95, 47), (90, 45), (91, 46), (90, 47), (89, 46), (89, 45), (88, 44), (90, 42), (102, 44), (104, 47), (105, 46), (106, 47), (103, 54), (94, 45), (93, 46), (90, 44), (87, 46), (88, 45), (87, 44), (82, 40), (83, 37), (83, 44), (84, 43), (85, 41), (94, 34), (96, 34), (95, 35), (93, 34), (94, 33), (93, 33), (91, 33), (92, 32), (92, 35), (98, 35), (98, 34), (97, 34), (96, 33), (95, 33), (94, 32), (91, 35), (92, 34), (94, 31), (93, 30), (92, 31), (93, 31), (92, 30), (92, 29), (93, 29), (94, 30), (96, 30), (95, 31), (91, 30), (90, 29), (90, 31), (89, 31), (87, 31), (86, 30), (86, 29), (87, 29), (87, 28), (86, 27), (87, 26), (87, 25), (88, 26), (87, 27), (88, 25), (89, 24), (96, 31), (97, 31), (97, 30), (98, 30), (99, 30), (98, 31), (93, 32), (95, 32), (96, 32), (97, 32), (97, 33), (95, 28), (96, 29), (97, 28), (89, 35), (86, 36), (86, 35), (89, 36), (84, 35), (83, 36), (84, 36), (85, 35), (85, 33), (86, 32), (87, 30), (85, 29), (85, 30), (86, 31), (88, 30), (88, 29), (89, 28), (86, 28), (88, 27), (87, 33), (85, 36), (84, 37), (85, 37), (81, 41), (79, 42), (76, 43), (77, 43), (79, 43), (82, 48), (84, 52), (83, 66), (86, 69), (89, 70), (91, 65), (92, 65), (91, 64), (87, 66), (68, 57), (70, 58), (68, 56), (63, 55), (72, 58), (74, 50), (73, 47), (73, 48), (73, 49), (73, 51), (67, 61), (62, 62), (48, 63), (46, 64), (47, 64), (48, 64), (47, 58), (46, 58), (48, 58), (48, 59), (48, 60), (54, 61), (52, 65), (53, 64), (53, 63), (54, 62), (59, 69), (71, 41), (70, 40), (63, 42), (71, 45), (71, 44), (72, 44), (72, 43), (72, 42), (71, 43), (70, 42), (61, 44), (64, 40), (74, 44), (75, 43), (75, 42), (76, 42), (75, 44), (75, 45), (76, 46), (75, 49), (79, 49), (85, 46), (84, 49), (79, 50), (78, 45), (77, 45), (77, 47), (77, 46), (77, 42), (76, 44), (73, 43), (74, 42), (74, 41), (73, 42), (74, 43), (75, 41), (75, 40), (73, 41), (74, 40), (72, 41), (71, 42), (71, 40), (71, 39), (70, 39), (72, 40), (72, 38), (72, 37), (73, 36), (72, 35), (71, 35), (70, 35), (69, 35), (70, 36), (71, 37), (72, 36), (73, 37), (71, 38), (73, 35), (74, 34), (73, 34), (72, 34), (69, 37), (70, 38), (72, 39), (73, 38), (73, 39), (74, 38), (75, 39), (76, 41), (76, 40), (76, 39), (75, 38), (74, 37), (75, 37), (75, 36), (74, 35), (75, 34), (76, 34), (75, 35), (76, 35), (76, 36), (77, 35), (77, 36), (77, 37), (78, 36), (78, 35), (77, 34), (77, 32), (77, 33), (77, 31), (78, 30), (79, 29), (80, 29), (79, 28), (78, 28), (78, 29), (77, 30), (77, 29), (76, 30), (75, 29), (74, 30), (75, 30), (76, 29), (74, 28), (75, 27), (75, 26), (76, 25), (75, 25), (74, 26), (73, 26), (73, 27), (72, 26), (72, 27), (74, 25), (75, 24), (74, 23), (73, 23), (73, 24), (74, 24), (76, 27), (77, 26), (77, 25), (78, 26), (78, 27), (77, 27), (76, 24), (77, 24), (78, 24), (79, 25), (76, 23), (77, 23), (78, 25), (80, 18), (82, 19), (81, 21), (81, 19), (81, 18), (76, 16), (75, 17), (75, 18), (85, 18), (85, 20), (86, 21), (87, 22), (87, 21), (88, 20), (88, 19), (88, 18), (82, 20), (82, 18), (82, 21), (82, 22), (83, 23), (83, 24), (84, 27), (85, 28), (84, 28), (84, 29), (83, 30), (81, 30), (81, 31), (80, 30), (80, 31), (81, 29), (81, 27), (82, 27), (80, 26), (75, 22), (75, 20), (76, 19), (75, 16), (74, 17), (74, 15), (73, 14), (74, 14), (74, 13), (77, 12), (73, 13), (73, 10), (72, 9), (70, 7), (69, 6), (69, 7), (70, 6), (71, 6), (72, 7), (73, 7), (74, 7), (67, 9), (67, 8), (71, 7), (70, 5), (68, 6), (69, 5), (70, 12), (71, 10), (69, 13), (70, 13), (69, 14), (68, 15), (67, 15), (66, 16), (67, 17), (68, 16), (69, 15), (68, 14), (66, 11), (66, 10), (67, 10), (66, 9), (66, 8), (67, 7), (66, 6), (65, 7), (65, 8), (65, 9), (64, 9), (63, 8), (62, 8), (60, 7), (59, 8), (57, 8), (58, 9), (56, 10), (55, 11), (56, 11), (55, 12), (54, 13), (53, 14), (52, 15), (52, 14), (51, 14), (50, 14), (51, 13), (52, 13), (50, 12), (51, 12), (51, 11), (52, 10), (51, 9), (51, 8), (52, 8), (53, 9), (54, 8), (53, 10), (52, 9), (53, 8), (52, 11), (53, 11), (53, 12), (52, 12), (54, 14), (53, 15), (55, 13), (54, 11), (56, 13), (54, 10), (55, 9), (52, 7), (51, 7), (50, 6), (49, 5), (50, 5), (51, 6), (49, 7), (48, 6), (48, 7), (47, 7), (47, 6), (47, 5), (46, 6), (45, 5), (44, 5), (43, 5), (43, 6), (42, 5), (42, 7), (44, 7), (45, 7), (45, 6), (44, 6), (46, 5), (46, 8), (47, 9), (46, 9), (47, 8), (46, 7), (45, 8), (45, 9), (44, 9), (45, 10), (44, 8), (44, 10), (45, 11), (46, 11), (47, 11), (48, 12), (46, 12), (43, 11), (43, 10), (42, 11), (42, 12), (41, 13), (40, 12), (39, 13), (40, 13), (41, 12), (42, 13), (43, 13), (44, 12), (45, 13), (45, 12), (44, 13), (43, 12), (44, 11), (43, 14), (44, 14), (45, 15), (44, 16), (43, 17), (43, 18), (42, 19), (43, 20), (44, 19), (42, 18), (41, 18), (40, 19), (39, 18), (40, 18), (42, 17), (44, 18), (43, 19), (44, 20), (44, 21), (44, 22), (43, 21), (42, 21), (43, 22), (42, 22), (41, 21), (42, 20), (41, 22), (41, 20), (38, 17), (37, 18), (38, 16), (39, 15), (38, 14), (39, 14), (39, 12), (38, 13), (37, 14), (36, 14), (36, 13), (35, 14), (36, 12), (35, 11), (35, 12), (35, 13), (34, 14), (34, 13), (33, 12), (32, 11), (32, 10), (31, 10), (32, 9), (33, 8), (31, 8), (30, 8), (30, 9), (29, 8), (28, 8), (27, 8), (26, 9), (25, 10), (24, 9), (25, 8), (25, 9), (26, 10), (27, 11), (27, 10), (24, 11), (23, 10), (22, 10), (23, 9), (23, 8), (24, 8), (23, 7), (22, 7), (24, 6), (23, 5), (22, 6), (21, 5), (22, 5), (20, 6), (19, 6), (18, 6), (21, 6), (20, 5), (21, 7), (20, 7), (23, 6), (24, 5), (25, 6), (25, 5), (26, 6), (26, 5), (27, 6), (27, 5), (28, 5), (18, 5), (19, 5), (17, 6), (19, 7), (19, 8), (18, 9), (17, 8), (18, 8), (17, 7), (16, 5), (17, 5), (16, 6), (18, 7), (20, 9), (19, 10), (20, 10), (21, 10), (20, 8), (19, 9), (21, 8), (24, 7), (22, 8), (22, 9), (21, 11), (21, 9), (22, 12), (21, 12), (20, 11), (19, 12), (19, 11), (18, 10), (22, 11), (22, 13), (23, 14), (22, 15), (22, 16), (23, 16), (23, 17), (24, 18), (25, 17), (26, 17), (26, 16), (26, 15), (26, 14), (27, 13), (28, 14), (27, 15), (27, 14), (28, 13), (28, 12), (29, 13), (30, 12), (30, 13), (31, 12), (32, 12), (33, 11), (30, 11), (29, 11), (28, 10), (29, 10), (30, 10), (28, 11), (27, 12), (26, 13), (26, 12), (26, 11), (27, 9), (29, 9), (31, 11), (34, 11), (32, 13), (33, 13), (34, 12), (35, 15), (33, 10), (32, 8), (32, 7), (33, 7), (34, 8), (35, 7), (34, 6), (35, 6), (36, 7), (34, 7), (35, 8), (36, 9), (35, 10), (35, 9), (36, 8), (34, 5), (35, 5), (33, 9), (34, 9), (34, 10), (32, 14), (33, 14), (33, 15), (32, 15), (34, 15), (37, 15), (31, 13), (31, 9), (31, 14), (30, 15), (29, 15), (30, 14), (29, 14), (29, 16), (30, 17), (29, 18), (29, 17), (30, 18), (29, 19), (30, 19), (29, 20), (28, 19), (27, 19), (27, 18), (26, 19), (27, 20), (28, 21), (29, 21), (28, 22), (29, 22), (27, 21), (30, 22), (31, 21), (32, 22), (32, 21), (33, 20), (32, 19), (31, 20), (31, 19), (30, 20), (26, 21), (27, 22), (26, 22), (25, 21), (25, 20), (28, 20), (25, 22), (24, 23), (25, 23), (24, 22), (24, 24), (23, 23), (22, 23), (23, 24), (22, 25), (23, 26), (26, 24), (22, 26), (21, 25), (21, 24), (22, 22), (21, 23), (20, 24), (19, 25), (18, 24), (18, 25), (20, 25), (21, 26), (20, 22), (21, 22), (23, 22), (20, 23), (19, 23), (19, 24), (18, 23), (17, 24), (16, 23), (16, 22), (17, 21), (16, 21), (17, 22), (17, 23), (15, 20), (16, 20), (18, 21), (18, 20), (17, 19), (17, 20), (18, 22), (20, 26), (19, 27), (18, 28), (17, 27), (18, 27), (19, 28), (16, 26), (17, 28), (18, 29), (17, 30), (16, 28), (17, 29), (16, 29), (15, 29), (15, 30), (15, 28), (14, 29), (16, 30), (19, 29), (20, 28), (18, 30), (17, 31), (14, 28), (15, 27), (19, 30), (19, 31), (18, 26), (19, 26), (20, 27), (20, 29), (20, 31), (20, 30), (24, 27), (22, 27), (21, 28), (21, 27), (35, 28), (35, 27), (37, 28), (36, 27), (38, 28), (38, 29), (37, 27), (38, 27), (37, 26), (39, 31), (40, 31), (39, 32), (40, 33), (41, 34), (38, 25), (37, 25), (38, 24), (39, 24), (39, 23), (39, 25), (38, 26), (36, 25), (37, 24), (38, 23), (36, 24), (35, 25), (34, 26), (33, 25), (34, 25), (34, 27), (27, 25), (25, 28), (22, 24), (24, 21), (26, 20), (25, 19), (24, 19), (25, 18), (24, 20), (23, 21), (19, 22), (19, 21), (14, 21), (14, 20), (15, 21), (14, 22), (13, 22), (13, 23), (14, 23), (14, 24), (13, 21), (12, 21), (12, 22), (11, 23), (11, 24), (12, 25), (12, 26), (13, 25), (14, 26), (15, 25), (17, 25), (16, 24), (16, 25), (20, 21), (21, 21), (21, 20), (22, 21), (23, 20), (26, 32), (21, 29), (17, 26), (16, 27), (15, 24), (14, 25), (13, 24), (15, 23), (10, 23), (9, 23), (9, 22), (9, 21), (8, 20), (9, 19), (8, 19), (9, 18), (10, 17), (11, 18), (12, 19), (13, 18), (12, 18), (11, 17), (10, 16), (9, 16), (9, 17), (8, 17), (7, 16), (7, 17), (9, 15), (10, 15), (10, 14), (11, 14), (11, 15), (12, 16), (12, 17), (13, 17), (14, 17), (13, 16), (13, 15), (14, 15), (15, 16), (16, 15), (16, 16), (15, 17), (15, 18), (14, 18), (15, 19), (16, 19), (14, 16), (17, 16), (16, 14), (15, 13), (15, 12), (15, 14), (16, 13), (17, 13), (18, 12), (17, 12), (17, 14), (17, 11), (17, 10), (18, 11), (23, 11), (20, 12), (19, 13), (20, 13), (19, 14), (23, 12), (24, 12), (25, 11), (28, 15), (30, 16), (28, 16), (28, 17), (28, 18), (31, 18), (31, 17), (32, 16), (33, 17), (32, 17), (33, 18), (33, 19), (32, 20), (34, 16), (33, 16), (34, 17), (35, 18), (34, 19), (34, 20), (35, 19), (34, 18), (35, 17), (35, 16), (36, 15), (38, 15), (37, 13), (38, 12), (39, 11), (40, 11), (41, 11), (47, 12), (47, 13), (47, 14), (46, 15), (46, 14), (48, 13), (49, 12), (50, 13), (49, 14), (49, 13), (48, 14), (48, 15), (42, 14), (43, 15), (43, 16), (44, 15), (45, 14), (46, 10), (47, 10), (48, 9), (48, 10), (43, 9), (46, 13), (47, 15), (48, 16), (49, 16), (49, 17), (48, 17), (47, 18), (46, 19), (46, 20), (47, 21), (46, 21), (45, 22), (45, 21), (45, 23), (46, 24), (46, 25), (45, 25), (46, 26), (45, 26), (45, 24), (44, 24), (43, 25), (42, 26), (42, 25), (42, 24), (43, 26), (42, 27), (44, 26), (46, 27), (46, 28), (45, 29), (44, 29), (45, 28), (45, 27), (43, 28), (43, 29), (44, 28), (44, 27), (44, 25), (43, 24), (43, 23), (44, 23), (42, 23), (45, 20), (45, 18), (45, 19), (41, 23), (40, 21), (40, 23), (40, 22), (39, 22), (36, 26), (39, 27), (39, 28), (40, 27), (39, 26), (40, 28), (35, 26), (34, 24), (33, 24), (35, 24), (36, 23), (37, 23), (35, 23), (36, 22), (37, 21), (37, 22), (38, 22), (39, 21), (39, 20), (38, 20), (40, 20), (38, 21), (39, 19), (42, 16), (44, 17), (45, 16), (46, 16), (47, 16), (49, 15), (50, 15), (47, 17), (46, 18), (45, 17), (48, 18), (49, 19), (50, 18), (50, 19), (51, 18), (52, 17), (52, 18), (50, 17), (50, 16), (51, 16), (53, 17), (54, 17), (53, 16), (54, 15), (55, 15), (56, 14), (55, 14), (56, 12), (57, 13), (57, 12), (57, 14), (56, 15), (53, 13), (57, 15), (56, 16), (56, 17), (55, 17), (55, 16), (51, 15), (55, 18), (54, 18), (53, 19), (52, 19), (53, 18), (51, 19), (52, 16), (59, 21), (60, 23), (60, 24), (62, 25), (58, 26), (57, 26), (56, 26), (54, 26), (54, 27), (55, 29), (55, 30), (55, 27), (56, 28), (57, 29), (56, 30), (55, 31), (56, 29), (54, 31), (50, 32), (52, 32), (51, 29), (52, 28), (50, 27), (50, 28), (49, 29), (50, 29), (49, 28), (50, 30), (50, 31), (49, 30), (48, 30), (47, 31), (46, 30), (46, 29), (47, 28), (47, 27), (48, 28), (49, 27), (47, 26), (42, 30), (42, 29), (43, 30), (42, 31), (41, 32), (40, 32), (41, 31), (42, 32), (43, 33), (43, 34), (42, 35), (41, 33), (49, 32), (49, 31), (48, 31), (46, 31), (47, 30), (45, 30), (44, 30), (46, 32), (45, 31), (44, 32), (42, 33), (45, 34), (46, 43), (55, 38), (55, 39), (53, 37), (54, 36), (54, 37), (55, 36), (55, 35), (55, 34), (53, 36), (52, 36), (49, 33), (50, 26), (48, 27), (48, 26), (48, 29), (46, 33), (44, 33), (45, 32), (47, 32), (49, 26), (48, 25), (49, 34), (45, 33), (43, 32), (42, 34), (47, 33), (47, 34), (41, 26), (41, 28), (42, 28), (41, 27), (40, 26), (44, 31), (48, 32), (50, 35), (44, 45), (43, 46), (40, 43), (44, 46), (45, 52), (59, 47), (60, 44), (59, 44), (60, 45), (77, 40), (77, 41), (78, 42), (86, 44), (86, 45), (85, 44), (76, 47), (74, 39), (74, 36), (76, 37), (78, 34), (76, 33), (76, 32), (73, 33), (72, 33), (79, 30), (78, 31), (79, 31), (76, 31), (83, 32), (86, 33), (84, 34), (83, 35), (82, 35), (83, 34), (82, 36), (81, 35), (82, 34), (82, 30), (83, 29), (82, 28), (81, 28), (82, 29), (83, 28), (87, 32), (88, 32), (72, 22), (71, 20), (75, 19), (74, 21), (73, 18), (74, 18), (74, 19), (70, 19), (75, 23), (75, 21), (76, 20), (76, 18), (74, 16), (73, 16), (73, 17), (81, 23), (84, 24), (85, 25), (85, 26), (85, 27), (83, 27), (81, 24), (80, 24), (82, 25), (86, 18), (86, 19), (85, 24), (86, 25), (86, 26), (87, 23), (88, 23), (88, 24), (93, 20), (92, 20), (89, 20), (91, 19), (90, 19), (90, 8), (87, 10), (86, 13), (71, 22), (72, 23), (67, 14), (67, 16), (66, 15), (66, 14), (65, 13), (64, 13), (63, 12), (64, 11), (64, 10), (65, 10), (63, 9), (62, 10), (61, 10), (62, 11), (62, 12), (63, 13), (63, 11), (65, 11), (65, 12), (64, 12), (64, 14), (63, 14), (61, 11), (60, 10), (59, 9), (59, 10), (58, 11), (60, 12), (60, 14), (63, 18), (65, 18), (62, 13), (65, 14), (60, 11), (61, 16), (60, 17), (60, 21), (60, 19), (60, 16), (60, 15), (61, 17), (61, 12), (59, 13), (59, 14), (58, 13), (54, 12), (57, 10), (60, 9), (60, 8), (62, 9), (63, 10), (65, 5), (61, 8), (59, 7), (58, 8), (57, 7), (56, 7), (61, 9), (64, 8), (66, 7), (72, 5), (73, 6), (72, 6), (72, 10), (87, 20), (84, 23), (86, 24), (73, 25), (72, 25), (72, 24), (71, 25), (71, 24), (70, 24), (70, 26), (69, 27), (70, 28), (70, 27), (71, 28), (72, 29), (70, 29), (69, 28), (70, 25), (88, 22), (89, 22), (88, 21), (98, 17), (100, 11), (99, 11), (91, 20), (96, 17), (98, 16), (100, 16), (100, 17), (99, 17), (99, 18), (99, 19), (100, 20), (99, 21), (99, 23), (101, 23), (102, 23), (101, 24), (99, 22), (103, 22), (104, 23), (105, 22), (106, 21), (107, 20), (109, 20), (110, 19), (110, 18), (110, 17), (111, 18), (112, 18), (113, 18), (114, 17), (112, 13), (111, 14), (112, 15), (113, 15), (112, 16), (111, 15), (111, 16), (113, 17), (112, 14), (110, 15), (107, 9), (108, 11), (111, 9), (113, 10), (119, 18), (124, 15), (124, 13), (123, 10), (129, 18), (129, 19), (129, 22), (126, 22), (125, 24), (126, 25), (125, 25), (124, 25), (124, 26), (125, 26), (123, 26), (123, 27), (123, 28), (122, 27), (121, 26), (119, 23), (119, 22), (120, 22), (126, 27), (127, 27), (124, 27), (128, 18), (128, 21), (122, 28), (123, 29), (122, 29), (121, 30), (121, 29), (130, 24), (129, 24), (131, 23), (132, 25), (132, 26), (133, 27), (134, 27), (134, 28), (134, 26), (134, 29), (133, 29), (132, 29), (132, 33), (130, 31), (131, 33), (131, 34), (130, 35), (132, 34), (133, 34), (133, 35), (132, 35), (131, 35), (132, 36), (134, 34), (128, 32), (127, 33), (127, 35), (127, 34), (128, 35), (127, 36), (126, 35), (124, 36), (125, 36), (125, 34), (126, 36), (126, 37), (127, 37), (128, 38), (129, 37), (129, 36), (128, 37), (129, 38), (128, 39), (127, 39), (127, 40), (126, 41), (125, 41), (124, 41), (124, 42), (125, 42), (123, 40), (122, 41), (121, 42), (121, 41), (121, 40), (120, 40), (120, 39), (119, 38), (120, 35), (121, 35), (121, 34), (119, 35), (121, 36), (121, 37), (122, 35), (123, 36), (124, 37), (124, 38), (123, 39), (122, 38), (123, 37), (125, 38), (125, 37), (124, 35), (126, 33), (127, 32), (126, 31), (126, 30), (128, 28), (134, 32), (133, 36), (134, 37), (134, 36), (133, 37), (133, 38), (134, 38), (132, 39), (132, 41), (133, 39), (130, 40), (129, 40), (129, 35), (128, 34), (127, 31), (128, 31), (124, 33), (127, 30), (126, 29), (129, 23), (127, 29), (124, 31), (120, 30), (115, 33), (114, 34), (114, 33), (113, 33), (112, 33), (114, 32), (112, 32), (115, 30), (114, 29), (117, 33), (126, 32), (131, 28), (133, 28), (132, 28), (133, 26), (134, 25), (134, 24), (131, 24), (129, 28), (128, 33), (129, 33), (130, 33), (131, 36), (130, 37), (130, 36), (131, 37), (129, 34), (130, 34), (115, 24), (113, 24), (114, 25), (113, 25), (110, 27), (109, 28), (109, 27), (108, 27), (108, 28), (109, 29), (108, 29), (107, 28), (106, 29), (105, 28), (104, 28), (104, 29), (105, 29), (106, 30), (103, 28), (103, 29), (103, 27), (104, 26), (103, 26), (103, 25), (104, 25), (104, 24), (105, 24), (106, 23), (105, 23), (106, 24), (107, 23), (107, 22), (108, 20), (106, 20), (103, 20), (101, 20), (100, 18), (101, 21), (102, 22), (103, 24), (100, 21), (104, 20), (103, 21), (103, 23), (102, 24), (102, 25), (104, 21), (104, 22), (105, 21), (102, 19), (101, 25), (101, 26), (102, 27), (102, 26), (100, 27), (99, 27), (99, 26), (99, 25), (104, 27), (101, 27), (100, 28), (101, 29), (98, 28), (99, 28), (98, 27), (97, 26), (97, 25), (96, 26), (97, 27), (102, 43), (103, 36), (104, 36), (104, 37), (105, 36), (106, 35), (107, 36), (108, 35), (109, 35), (109, 34), (108, 34), (108, 33), (107, 32), (107, 31), (108, 31), (107, 30), (107, 29), (108, 30), (106, 31), (106, 28), (105, 27), (106, 27), (107, 26), (107, 25), (108, 25), (109, 24), (107, 24), (108, 24), (106, 25), (108, 26), (109, 25), (108, 23), (106, 22), (102, 18), (101, 17), (101, 18), (99, 7), (102, 10), (100, 10), (102, 9), (102, 8), (102, 5), (103, 5), (108, 6), (108, 8), (110, 9), (109, 7), (111, 10), (118, 8), (119, 9), (119, 7), (120, 9), (120, 8), (121, 7), (121, 6), (122, 7), (123, 6), (124, 6), (124, 7), (124, 8), (126, 9), (124, 9), (122, 10), (125, 10), (125, 9), (126, 8), (127, 8), (129, 9), (129, 5), (127, 9), (128, 10), (134, 6), (126, 5), (128, 11), (131, 16), (121, 14), (123, 13), (123, 9), (123, 8), (122, 6), (122, 5), (123, 5), (124, 5), (120, 7), (121, 8), (121, 9), (121, 10), (122, 9), (124, 10), (136, 21), (117, 14), (116, 13), (121, 15), (120, 11), (132, 24), (133, 24), (123, 7), (120, 6), (122, 8), (125, 5), (114, 18), (117, 15), (127, 38), (131, 42), (132, 38), (134, 42), (134, 43), (132, 42), (128, 43), (127, 45), (127, 43), (128, 41), (130, 38), (127, 41), (126, 42), (126, 40), (125, 40), (124, 39), (125, 39), (124, 40), (115, 34), (116, 35), (113, 34), (112, 34), (111, 35), (111, 36), (111, 34), (112, 35), (110, 34), (109, 33), (108, 32), (109, 32), (107, 34), (106, 34), (107, 33), (106, 32), (105, 30), (105, 31), (105, 32), (104, 31), (105, 33), (104, 33), (109, 31), (109, 30), (110, 29), (111, 28), (110, 28), (106, 26), (105, 26), (105, 25), (107, 27), (98, 26), (96, 27), (93, 25), (97, 29), (98, 29), (87, 19), (87, 18), (98, 12), (99, 5), (100, 5), (96, 5), (77, 6), (76, 7), (97, 17), (97, 14), (101, 19), (100, 25), (100, 26), (87, 24), (88, 28), (76, 26), (77, 28), (75, 28), (74, 29), (73, 30), (73, 29), (71, 30), (70, 31), (65, 31), (66, 30), (65, 30), (64, 30), (63, 29), (64, 29), (64, 28), (65, 28), (59, 25), (61, 25), (61, 26), (63, 24), (64, 22), (65, 21), (65, 20), (64, 20), (63, 21), (64, 21), (65, 22), (66, 22), (64, 19), (61, 22), (54, 21), (53, 21), (52, 21), (51, 20), (51, 17), (54, 19), (53, 20), (52, 23), (51, 22), (52, 22), (51, 23), (53, 23), (52, 24), (52, 25), (51, 26), (51, 25), (49, 25), (50, 25), (50, 24), (51, 24), (53, 24), (54, 24), (50, 23), (49, 23), (48, 22), (50, 22), (49, 24), (47, 25), (47, 24), (46, 23), (47, 23), (43, 31), (41, 30), (47, 29), (48, 24), (48, 23), (49, 21), (47, 22), (46, 22), (47, 20), (47, 19), (48, 20), (49, 20), (48, 21), (43, 27), (41, 25), (40, 24), (40, 25), (41, 24), (34, 23), (33, 23), (32, 25), (32, 23), (31, 22), (31, 23), (32, 24), (31, 25), (39, 29), (39, 30), (40, 30), (39, 43), (38, 43), (50, 55), (78, 49), (78, 48), (68, 53), (66, 55), (65, 38), (64, 39), (63, 38), (62, 39), (67, 34), (65, 29), (52, 20), (50, 11), (51, 10), (50, 10), (53, 7), (56, 9), (56, 6), (55, 6), (52, 5), (49, 6), (48, 5), (48, 8), (49, 9), (49, 8), (50, 8), (50, 9), (51, 5), (52, 6), (50, 7), (49, 11), (48, 11), (49, 10), (55, 10), (60, 5), (63, 5), (59, 5), (58, 5), (64, 7), (64, 5), (58, 15), (58, 14), (58, 16), (56, 18), (50, 21), (50, 20), (49, 22), (46, 17), (42, 15), (41, 14), (43, 7), (43, 8), (41, 10), (40, 10), (39, 9), (40, 9), (39, 10), (40, 14), (40, 15), (41, 16), (40, 16), (40, 17), (39, 16), (38, 19), (39, 17), (41, 15), (41, 19), (41, 17), (38, 10), (38, 11), (41, 8), (42, 9), (42, 10), (42, 8), (41, 9), (41, 6), (42, 6), (41, 5), (40, 5), (40, 6), (39, 6), (40, 7), (41, 7), (40, 8), (39, 7), (38, 6), (39, 8), (38, 7), (37, 5), (36, 6), (36, 10), (36, 11), (37, 11), (31, 15), (27, 23), (26, 23), (27, 24), (25, 24), (28, 24), (28, 23), (28, 25), (28, 26), (28, 28), (49, 36), (53, 28), (52, 27), (48, 33), (40, 29), (54, 35), (55, 37), (56, 37), (64, 37), (66, 37), (70, 30), (71, 29), (72, 30), (71, 32), (71, 31), (72, 31), (73, 31), (72, 32), (73, 32), (71, 33), (71, 34), (73, 28), (75, 32), (75, 31), (86, 17), (103, 17), (102, 15), (107, 21), (105, 20), (107, 19), (107, 14), (109, 16), (109, 15), (111, 17), (112, 17), (111, 19), (111, 20), (110, 20), (112, 20), (110, 25), (112, 25), (111, 27), (109, 21), (110, 21), (114, 5), (106, 11), (110, 16), (112, 19), (112, 22), (113, 19), (113, 23), (117, 20), (118, 19), (118, 20), (101, 16), (108, 18), (109, 18), (109, 19), (117, 6), (117, 5), (118, 6), (115, 5), (119, 6), (99, 31), (100, 30), (100, 29), (99, 29), (98, 32), (98, 33), (71, 23), (70, 23), (68, 23), (65, 19), (65, 23), (66, 23), (69, 22), (71, 27), (72, 28), (68, 35), (69, 36), (68, 37), (69, 29), (67, 25), (70, 21), (70, 20), (69, 20), (69, 19), (74, 27), (76, 28), (74, 6), (75, 7), (74, 5), (75, 5), (76, 5), (68, 5), (67, 5), (59, 16), (59, 15), (57, 11), (59, 17), (58, 12), (64, 18), (66, 19), (66, 18), (69, 16), (68, 18), (68, 19), (71, 15), (71, 14), (71, 26), (67, 18), (69, 18), (68, 17), (54, 16), (54, 20), (57, 18), (57, 25), (57, 27), (54, 23), (58, 25), (51, 21), (57, 17), (58, 18), (58, 17), (57, 16), (55, 7), (54, 9), (71, 5), (73, 5), (76, 6), (77, 5), (75, 6), (77, 7), (101, 28), (102, 29), (103, 30), (102, 28), (101, 30), (102, 31), (101, 31), (101, 32), (100, 32), (102, 33), (101, 34), (100, 35), (100, 34), (99, 35), (99, 34), (77, 39), (78, 37), (82, 41), (77, 38), (71, 36), (70, 37), (67, 36), (62, 19), (66, 17), (38, 18), (37, 19), (36, 18), (38, 8), (37, 8), (37, 7), (37, 6), (38, 9), (37, 9), (35, 22), (34, 21), (35, 21), (36, 21), (34, 22), (33, 21), (32, 18), (36, 19), (36, 20), (37, 20), (35, 20), (33, 22), (54, 6), (71, 13), (70, 15), (70, 14), (74, 33), (75, 33), (76, 38), (83, 48), (73, 40), (74, 31), (74, 32), (69, 38), (100, 37), (101, 36), (101, 35), (100, 36), (99, 33), (101, 33), (100, 31), (100, 33), (99, 32), (99, 36), (102, 34), (103, 33), (102, 35), (104, 35), (105, 35), (110, 32), (112, 30), (111, 32), (111, 33), (110, 33), (107, 35), (105, 34), (104, 34), (103, 34), (102, 32), (102, 30), (104, 30), (103, 31), (103, 35), (99, 37), (112, 23), (108, 22), (118, 5), (121, 5), (119, 5), (120, 5), (128, 36), (131, 38), (134, 35), (120, 38), (122, 37), (123, 38), (122, 39), (121, 39), (122, 40), (119, 40), (120, 41), (119, 42), (117, 45), (118, 46), (117, 46), (117, 47), (116, 46), (115, 47), (115, 48), (116, 48), (117, 49), (116, 50), (116, 49), (115, 50), (116, 51), (116, 52), (117, 51), (118, 52), (117, 53), (115, 52), (114, 51), (113, 51), (114, 52), (113, 52), (117, 50), (118, 49), (121, 50), (118, 51), (118, 50), (117, 48), (115, 46), (114, 47), (114, 46), (113, 47), (112, 46), (112, 45), (113, 46), (113, 45), (114, 45), (111, 45), (110, 46), (110, 45), (109, 46), (111, 48), (111, 47), (112, 48), (111, 49), (110, 47), (106, 41), (105, 42), (104, 42), (104, 41), (103, 32), (102, 17), (104, 32), (110, 30), (113, 29), (113, 32), (112, 28), (105, 38), (133, 25), (122, 26), (112, 24), (78, 5), (66, 28), (72, 16), (89, 44), (90, 48), (90, 46), (91, 47), (65, 37), (64, 38), (55, 33), (59, 45), (59, 46), (67, 64), (70, 66), (68, 66), (90, 66), (79, 67), (73, 76), (69, 74), (69, 75), (68, 74), (63, 73), (67, 67), (66, 66), (68, 65), (77, 71), (73, 74), (72, 75), (74, 73), (79, 72), (82, 74), (90, 86), (91, 88), (92, 89), (93, 90), (93, 91), (94, 90), (94, 89), (95, 88), (95, 87), (94, 88), (93, 89), (93, 88), (110, 72), (111, 71), (109, 72), (113, 68), (114, 68), (115, 73), (116, 74), (115, 74), (114, 75), (113, 74), (113, 75), (114, 74), (113, 76), (114, 76), (115, 76), (115, 75), (116, 76), (116, 73), (117, 72), (116, 71), (116, 72), (115, 71), (114, 73), (117, 74), (118, 73), (119, 74), (120, 73), (120, 72), (119, 72), (121, 72), (122, 71), (121, 71), (119, 73), (120, 74), (121, 74), (121, 73), (122, 72), (123, 71), (123, 70), (124, 70), (124, 69), (123, 68), (122, 68), (122, 69), (121, 69), (121, 68), (120, 69), (119, 69), (117, 68), (116, 68), (118, 69), (118, 70), (117, 70), (119, 70), (118, 71), (119, 71), (117, 71), (116, 70), (110, 65), (111, 66), (109, 65), (109, 66), (110, 64), (111, 64), (111, 65), (112, 64), (112, 63), (113, 63), (111, 62), (110, 62), (109, 63), (108, 63), (107, 64), (108, 65), (108, 64), (109, 64), (112, 67), (113, 65), (114, 64), (115, 64), (116, 65), (115, 66), (114, 66), (114, 67), (113, 66), (105, 66), (104, 66), (103, 66), (102, 67), (105, 65), (106, 64), (105, 61), (100, 61), (103, 43), (104, 44), (110, 51), (109, 53), (109, 52), (108, 52), (107, 50), (100, 50), (106, 43), (107, 41), (107, 40), (108, 41), (108, 43), (109, 43), (110, 42), (111, 43), (111, 44), (114, 44), (115, 43), (115, 41), (116, 43), (115, 40), (114, 39), (113, 40), (114, 40), (116, 44), (115, 44), (115, 45), (118, 47), (119, 47), (119, 46), (121, 44), (122, 44), (123, 44), (124, 45), (125, 45), (125, 44), (124, 43), (123, 43), (125, 43), (106, 33), (58, 27), (49, 18), (48, 19), (61, 71), (59, 67), (45, 64), (47, 59), (42, 62), (44, 58), (45, 57), (45, 58), (45, 59), (47, 67), (46, 69), (47, 65), (44, 57), (59, 71), (59, 70), (51, 74), (64, 83), (64, 78), (68, 79), (60, 76), (64, 77), (65, 78), (65, 77), (65, 73), (67, 75), (67, 77), (66, 77), (68, 76), (76, 77), (88, 82), (92, 88), (95, 91), (96, 91), (97, 92), (96, 93), (97, 93), (98, 92), (97, 91), (96, 90), (97, 90), (97, 89), (96, 88), (98, 84), (101, 84), (100, 85), (106, 89), (113, 89), (98, 88), (97, 87), (95, 89), (95, 90), (96, 94), (98, 94), (98, 93), (95, 97), (95, 98), (95, 99), (94, 98), (91, 98), (91, 99), (93, 102), (89, 98), (90, 97), (90, 98), (92, 96), (96, 99), (96, 98), (94, 99), (95, 100), (94, 100), (93, 98), (89, 97), (88, 97), (85, 95), (84, 95), (83, 96), (83, 95), (82, 94), (80, 92), (80, 91), (86, 89), (87, 88), (82, 93), (81, 94), (82, 95), (81, 96), (80, 97), (82, 98), (83, 97), (90, 96), (94, 101), (86, 111), (82, 112), (84, 115), (85, 115), (85, 116), (86, 115), (86, 116), (87, 117), (87, 116), (85, 117), (86, 118), (88, 117), (89, 116), (89, 117), (90, 118), (90, 117), (89, 118), (90, 116), (94, 119), (94, 120), (93, 120), (95, 120), (95, 118), (96, 118), (97, 118), (97, 119), (98, 118), (99, 119), (99, 118), (98, 119), (97, 120), (96, 120), (96, 121), (93, 122), (92, 121), (92, 122), (91, 123), (87, 120), (85, 121), (85, 120), (84, 119), (84, 118), (83, 117), (83, 115), (82, 117), (83, 120), (84, 120), (81, 120), (83, 121), (84, 121), (90, 125), (92, 123), (93, 123), (94, 121), (91, 122), (93, 121), (94, 124), (98, 120), (96, 119), (94, 117), (95, 117), (94, 116), (95, 115), (95, 116), (94, 115), (93, 115), (92, 116), (93, 114), (92, 113), (93, 112), (94, 111), (89, 112), (91, 114), (92, 115), (92, 114), (91, 113), (90, 112), (89, 113), (88, 113), (90, 113), (91, 112), (89, 114), (90, 115), (88, 115), (87, 114), (87, 115), (88, 114), (86, 114), (87, 109), (87, 107), (93, 108), (94, 109), (94, 108), (96, 107), (95, 107), (95, 108), (95, 110), (96, 109), (97, 108), (95, 109), (97, 107), (98, 107), (99, 108), (99, 109), (99, 112), (98, 111), (97, 110), (96, 110), (96, 108), (96, 111), (96, 112), (96, 113), (95, 112), (97, 111), (95, 111), (96, 114), (95, 114), (96, 115), (94, 114), (95, 113), (97, 114), (97, 113), (97, 112), (98, 112), (99, 114), (102, 113), (107, 114), (108, 113), (106, 112), (107, 112), (108, 111), (109, 110), (108, 110), (107, 109), (108, 109), (107, 110), (107, 108), (106, 109), (106, 108), (107, 107), (108, 107), (108, 106), (109, 107), (109, 106), (110, 106), (110, 105), (111, 106), (110, 104), (109, 104), (109, 105), (109, 108), (110, 109), (106, 111), (105, 111), (105, 110), (104, 110), (104, 111), (107, 111), (109, 112), (109, 111), (109, 113), (110, 111), (111, 110), (110, 110), (111, 109), (110, 108), (110, 107), (111, 108), (108, 108), (109, 109), (106, 107), (106, 106), (107, 106), (108, 105), (107, 104), (108, 103), (108, 104), (107, 103), (109, 103), (108, 102), (110, 94), (112, 94), (112, 95), (111, 95), (110, 101), (99, 93), (98, 91), (98, 90), (92, 91), (92, 92), (91, 91), (92, 90), (91, 89), (91, 90), (90, 91), (97, 88), (96, 89), (92, 94), (91, 94), (91, 95), (90, 94), (90, 93), (90, 92), (89, 94), (89, 95), (87, 95), (86, 94), (86, 95), (85, 93), (84, 96), (87, 96), (87, 94), (85, 92), (86, 91), (76, 82), (75, 81), (75, 82), (74, 76), (75, 85), (77, 85), (72, 70), (67, 65), (54, 75), (52, 79), (50, 83), (44, 82), (43, 86), (62, 93), (64, 94), (62, 95), (62, 94), (61, 95), (70, 92), (71, 93), (71, 92), (70, 91), (69, 86), (70, 87), (69, 87), (69, 85), (70, 85), (70, 84), (70, 86), (71, 85), (71, 83), (71, 84), (72, 85), (75, 78), (73, 77), (70, 79), (70, 80), (69, 80), (71, 79), (67, 76), (69, 78), (68, 81), (72, 84), (74, 86), (66, 87), (66, 88), (66, 89), (64, 91), (61, 92), (61, 93), (60, 92), (60, 97), (59, 98), (59, 97), (55, 102), (55, 99), (54, 100), (54, 99), (53, 99), (54, 101), (55, 103), (56, 103), (57, 103), (57, 102), (56, 102), (57, 101), (58, 103), (58, 102), (59, 103), (60, 102), (61, 101), (60, 101), (59, 101), (61, 102), (62, 103), (62, 104), (62, 105), (62, 106), (61, 105), (63, 103), (64, 103), (65, 104), (66, 104), (67, 103), (66, 103), (65, 102), (65, 101), (64, 102), (65, 100), (64, 100), (63, 99), (64, 98), (65, 97), (61, 98), (58, 96), (55, 93), (55, 91), (60, 72), (61, 70), (93, 64), (37, 43), (35, 46), (30, 49), (26, 56), (25, 57), (24, 56), (23, 56), (24, 55), (25, 55), (25, 56), (25, 51), (19, 50), (18, 48), (18, 45), (18, 46), (17, 45), (16, 45), (15, 46), (16, 47), (17, 48), (17, 49), (17, 47), (16, 46), (17, 46), (20, 44), (25, 53), (24, 54), (25, 54), (26, 55), (24, 57), (23, 57), (47, 50), (36, 59), (34, 58), (43, 57), (46, 66), (45, 71), (46, 82), (40, 70), (58, 72), (69, 81), (69, 79), (68, 77), (57, 79), (55, 80), (54, 80), (54, 81), (55, 81), (58, 87), (58, 86), (60, 91), (71, 87), (70, 88), (70, 89), (72, 91), (73, 92), (73, 91), (75, 92), (76, 91), (76, 92), (74, 91), (76, 90), (78, 89), (78, 88), (88, 94), (82, 96), (85, 90), (85, 91), (79, 94), (79, 93), (78, 92), (77, 89), (69, 91), (72, 93), (77, 102), (78, 100), (77, 99), (79, 102), (77, 100), (72, 102), (71, 102), (70, 102), (69, 103), (67, 102), (66, 102), (66, 101), (63, 100), (63, 101), (66, 99), (65, 98), (65, 96), (66, 98), (64, 96), (65, 99), (62, 101), (62, 102), (64, 101), (61, 100), (59, 100), (58, 101), (56, 99), (56, 98), (54, 98), (53, 100), (51, 102), (52, 104), (56, 104), (53, 98), (54, 97), (54, 96), (53, 95), (53, 96), (52, 95), (51, 94), (49, 94), (49, 95), (50, 94), (50, 95), (50, 96), (49, 96), (49, 97), (48, 97), (48, 98), (47, 97), (52, 94), (51, 95), (55, 98), (53, 97), (52, 98), (52, 97), (51, 98), (50, 99), (49, 100), (49, 99), (49, 98), (50, 98), (50, 100), (50, 101), (50, 102), (50, 103), (51, 103), (52, 103), (54, 102), (48, 106), (47, 106), (48, 105), (47, 107), (46, 106), (46, 107), (45, 107), (46, 108), (45, 106), (44, 107), (43, 106), (39, 104), (38, 104), (42, 107), (41, 104), (40, 103), (37, 100), (37, 98), (37, 97), (36, 98), (37, 96), (39, 95), (38, 96), (39, 96), (41, 98), (40, 96), (39, 94), (42, 93), (45, 95), (45, 94), (48, 94), (50, 97), (51, 96), (48, 95), (48, 96), (48, 99), (47, 98), (45, 98), (44, 99), (45, 100), (44, 100), (43, 99), (45, 99), (45, 101), (46, 102), (44, 101), (43, 101), (43, 102), (42, 102), (42, 103), (41, 103), (43, 104), (44, 104), (44, 105), (44, 106), (40, 104), (41, 102), (40, 101), (40, 100), (41, 99), (40, 99), (46, 96), (47, 96), (47, 95), (47, 91), (47, 92), (42, 99), (41, 100), (41, 101), (42, 100), (43, 100), (43, 90), (46, 92), (52, 93), (53, 94), (54, 93), (54, 95), (51, 100), (51, 101), (58, 104), (57, 105), (58, 106), (58, 107), (58, 108), (57, 107), (57, 109), (57, 110), (56, 109), (55, 108), (54, 108), (55, 107), (55, 105), (55, 104), (56, 105), (57, 104), (59, 104), (60, 103), (59, 102), (61, 103), (61, 104), (60, 105), (60, 104), (62, 107), (63, 106), (64, 107), (63, 108), (64, 109), (65, 108), (63, 105), (64, 104), (60, 99), (60, 94), (59, 92), (56, 75), (60, 73), (60, 71), (60, 70), (60, 74), (61, 94), (60, 95), (60, 96), (66, 96), (67, 100), (68, 100), (67, 101), (67, 99), (69, 100), (68, 99), (69, 98), (67, 94), (69, 99), (67, 98), (66, 97), (65, 103), (66, 100), (67, 104), (67, 105), (69, 107), (68, 108), (70, 106), (70, 105), (71, 105), (71, 106), (73, 106), (73, 107), (73, 108), (72, 107), (72, 108), (73, 109), (72, 109), (71, 109), (72, 110), (72, 111), (73, 110), (74, 107), (67, 106), (66, 107), (67, 107), (67, 108), (68, 107), (68, 109), (67, 109), (65, 106), (65, 107), (66, 108), (66, 106), (69, 110), (69, 111), (68, 111), (69, 112), (68, 112), (67, 112), (67, 113), (66, 112), (65, 112), (64, 113), (65, 113), (64, 114), (65, 114), (66, 113), (66, 114), (67, 114), (67, 111), (67, 110), (66, 110), (65, 111), (65, 110), (66, 111), (68, 110), (69, 109), (70, 109), (71, 110), (70, 108), (70, 107), (71, 107), (71, 103), (69, 104), (72, 103), (71, 104), (69, 106), (66, 105), (65, 105), (64, 108), (63, 109), (62, 108), (64, 110), (63, 110), (64, 111), (63, 112), (62, 112), (61, 113), (60, 114), (60, 113), (62, 113), (61, 114), (59, 114), (58, 113), (58, 114), (58, 115), (57, 115), (56, 114), (57, 114), (58, 112), (57, 113), (58, 111), (58, 110), (59, 112), (59, 111), (56, 111), (59, 118), (59, 117), (60, 116), (60, 115), (62, 111), (63, 111), (62, 110), (61, 109), (61, 108), (60, 108), (60, 109), (60, 110), (60, 111), (59, 110), (57, 111), (51, 111), (49, 113), (49, 114), (48, 114), (52, 113), (51, 115), (49, 116), (50, 118), (51, 118), (49, 121), (45, 123), (46, 125), (45, 124), (44, 125), (45, 125), (44, 126), (45, 127), (45, 126), (46, 127), (47, 127), (44, 127), (43, 127), (43, 126), (43, 125), (36, 129), (36, 128), (35, 128), (38, 123), (36, 133), (34, 132), (33, 133), (34, 133), (32, 132), (31, 131), (28, 134), (31, 134), (32, 133), (32, 131), (33, 130), (33, 134), (32, 134), (33, 132), (30, 130), (27, 134), (26, 134), (25, 133), (24, 133), (24, 134), (23, 134), (23, 132), (24, 132), (22, 134), (23, 133), (25, 134), (25, 132), (22, 131), (21, 130), (21, 131), (20, 132), (20, 131), (19, 133), (20, 133), (21, 132), (21, 133), (20, 134), (19, 134), (19, 130), (11, 123), (10, 123), (8, 123), (7, 123), (6, 123), (7, 122), (6, 124), (5, 124), (5, 125), (5, 123), (5, 122), (7, 124), (8, 124), (22, 113), (38, 121), (40, 116), (41, 117), (41, 112), (42, 113), (44, 114), (41, 113), (47, 105), (47, 104), (48, 104), (49, 104), (47, 100), (47, 101), (45, 103), (45, 104), (45, 105), (45, 108), (44, 103), (43, 103), (44, 110), (44, 113), (47, 112), (48, 113), (47, 113), (48, 115), (46, 113), (45, 114), (44, 116), (45, 119), (50, 114), (59, 113), (55, 109), (56, 110), (56, 108), (55, 106), (56, 106), (57, 106), (59, 106), (58, 105), (59, 105), (60, 106), (59, 107), (60, 107), (63, 102), (63, 104), (54, 105), (56, 107), (64, 106), (64, 105), (70, 99), (73, 95), (67, 96), (71, 108), (69, 108), (57, 98), (61, 106), (63, 107), (61, 107), (65, 109), (66, 109), (72, 106), (74, 108), (75, 107), (77, 101), (75, 102), (80, 102), (80, 103), (77, 105), (77, 104), (80, 105), (77, 107), (78, 108), (77, 109), (76, 108), (75, 108), (75, 109), (74, 110), (75, 110), (74, 109), (75, 111), (74, 112), (76, 111), (77, 111), (78, 111), (78, 110), (77, 110), (80, 111), (80, 110), (81, 109), (82, 110), (85, 108), (85, 105), (87, 110), (86, 109), (86, 105), (86, 108), (93, 111), (93, 113), (94, 113), (90, 114), (91, 115), (94, 110), (101, 114), (106, 110), (105, 108), (105, 107), (105, 109), (105, 106), (105, 105), (104, 105), (103, 106), (103, 107), (104, 107), (106, 105), (107, 105), (112, 105), (111, 105), (106, 104), (106, 103), (105, 104), (104, 106), (102, 107), (101, 108), (99, 106), (99, 107), (100, 106), (98, 108), (97, 109), (93, 116), (93, 117), (89, 115), (95, 119), (97, 121), (98, 121), (99, 120), (99, 121), (100, 122), (100, 121), (100, 119), (97, 122), (98, 122), (99, 123), (100, 124), (101, 124), (101, 122), (100, 123), (99, 124), (98, 124), (98, 125), (100, 125), (103, 127), (97, 124), (97, 123), (92, 127), (91, 127), (91, 128), (92, 128), (92, 129), (91, 130), (92, 126), (90, 128), (93, 126), (88, 125), (90, 126), (86, 126), (88, 128), (85, 127), (88, 129), (89, 127), (91, 126), (88, 120), (88, 119), (89, 119), (87, 121), (90, 120), (91, 121), (91, 119), (88, 118), (88, 116), (85, 118), (84, 122), (85, 123), (83, 122), (89, 128), (90, 127), (92, 125), (87, 113), (87, 112), (86, 117), (86, 119), (81, 130), (79, 130), (83, 129), (80, 128), (79, 127), (76, 127), (75, 123), (73, 124), (74, 125), (75, 125), (75, 126), (79, 123), (76, 120), (75, 119), (75, 118), (76, 117), (75, 115), (75, 116), (74, 115), (75, 114), (75, 113), (78, 112), (79, 110), (80, 109), (79, 109), (78, 109), (79, 108), (81, 108), (80, 107), (81, 106), (82, 106), (80, 108), (80, 118), (79, 107), (78, 107), (77, 108), (76, 107), (76, 110), (74, 111), (75, 112), (81, 110), (81, 111), (79, 111), (76, 109), (80, 106), (81, 107), (82, 108), (88, 111), (88, 112), (96, 117), (96, 116), (97, 117), (98, 116), (98, 117), (98, 123), (99, 122), (89, 129), (90, 130), (89, 130), (87, 130), (94, 132), (95, 133), (87, 119), (90, 131), (89, 131), (88, 130), (87, 129), (86, 129), (85, 129), (86, 128), (85, 128), (87, 131), (88, 132), (88, 131), (89, 132), (87, 132), (86, 131), (86, 130), (85, 131), (85, 130), (84, 130), (84, 131), (83, 130), (80, 131), (79, 131), (79, 133), (76, 131), (77, 131), (75, 127), (74, 131), (74, 134), (77, 134), (78, 134), (79, 134), (78, 135), (73, 131), (72, 132), (72, 131), (73, 130), (72, 129), (73, 129), (74, 128), (73, 128), (74, 127), (73, 127), (74, 126), (73, 126), (72, 127), (74, 129), (73, 125), (72, 124), (72, 125), (71, 125), (72, 126), (71, 126), (70, 126), (69, 126), (68, 127), (69, 128), (68, 129), (68, 128), (67, 126), (67, 127), (62, 129), (58, 124), (59, 123), (61, 124), (61, 123), (60, 123), (63, 125), (62, 124), (63, 124), (59, 128), (60, 132), (57, 126), (58, 128), (57, 128), (58, 127), (62, 125), (62, 122), (62, 121), (61, 122), (62, 123), (61, 121), (60, 120), (60, 121), (60, 122), (59, 121), (63, 122), (59, 122), (58, 120), (58, 119), (59, 120), (58, 122), (60, 119), (57, 121), (55, 121), (56, 122), (56, 123), (57, 123), (57, 124), (57, 125), (56, 126), (55, 126), (54, 127), (47, 116), (51, 116), (59, 115), (58, 109), (61, 112), (59, 109), (60, 112), (61, 111), (60, 118), (59, 119), (58, 123), (63, 121), (63, 120), (64, 121), (65, 121), (65, 120), (66, 119), (66, 120), (66, 121), (67, 120), (67, 121), (68, 120), (68, 122), (68, 121), (69, 121), (69, 120), (70, 121), (70, 122), (71, 122), (72, 121), (73, 122), (73, 123), (74, 122), (72, 123), (71, 123), (71, 124), (70, 124), (70, 125), (69, 124), (69, 125), (70, 123), (69, 122), (69, 123), (68, 124), (67, 125), (68, 125), (67, 122), (66, 122), (64, 120), (63, 119), (62, 120), (66, 125), (68, 126), (68, 123), (67, 124), (65, 124), (66, 127), (66, 126), (66, 130), (66, 131), (65, 134), (70, 134), (71, 134), (72, 134), (71, 133), (69, 130), (67, 132), (71, 131), (70, 130), (71, 129), (71, 130), (70, 129), (69, 127), (72, 130), (74, 130), (71, 128), (70, 127), (64, 134), (80, 134), (80, 133), (78, 117), (79, 118), (86, 120), (92, 119), (83, 103), (98, 98), (96, 97), (91, 93), (97, 96), (99, 91), (98, 89), (99, 92), (99, 89), (100, 90), (100, 91), (99, 90), (103, 92), (104, 104), (104, 108), (100, 109), (98, 110), (98, 109), (99, 110), (100, 111), (111, 113), (113, 115), (114, 114), (113, 114), (113, 113), (112, 114), (112, 113), (114, 113), (115, 112), (115, 113), (114, 110), (111, 112), (110, 112), (116, 110), (116, 112), (117, 112), (119, 114), (119, 113), (120, 113), (121, 114), (121, 113), (120, 112), (119, 112), (119, 115), (120, 116), (120, 117), (120, 118), (121, 118), (122, 119), (122, 120), (121, 120), (121, 122), (121, 121), (123, 120), (123, 119), (121, 119), (120, 119), (111, 118), (107, 116), (111, 119), (105, 128), (96, 129), (97, 130), (100, 128), (88, 91), (88, 90), (90, 89), (89, 92), (81, 95), (74, 93), (72, 88), (71, 86), (70, 81), (71, 82), (70, 82), (71, 90), (55, 96), (54, 94), (53, 93), (53, 92), (43, 88), (52, 99), (52, 96), (51, 97), (52, 100), (77, 79), (48, 101), (61, 110), (64, 112), (66, 115), (67, 115), (68, 114), (69, 113), (68, 113), (65, 115), (65, 116), (64, 115), (63, 116), (64, 116), (63, 117), (64, 117), (63, 118), (62, 119), (62, 118), (62, 117), (61, 119), (61, 120), (55, 124), (55, 125), (56, 125), (55, 123), (60, 117), (61, 118), (61, 117), (62, 116), (63, 123), (64, 124), (65, 123), (64, 123), (65, 122), (64, 122), (63, 115), (63, 114), (62, 115), (61, 116), (61, 115), (64, 119), (65, 119), (67, 123), (63, 133), (70, 128), (71, 127), (72, 128), (84, 132), (85, 132), (86, 133), (86, 134), (87, 133), (87, 134), (88, 133), (86, 132), (88, 134), (85, 133), (89, 133), (89, 134), (90, 132), (91, 132), (90, 133), (101, 125), (91, 129), (96, 134), (97, 133), (98, 133), (95, 134), (100, 134), (103, 131), (104, 134), (105, 134), (105, 133), (106, 134), (104, 133), (102, 134), (103, 134), (108, 128), (107, 133), (105, 127), (113, 108), (114, 109), (112, 116), (115, 114), (122, 110), (119, 105), (120, 103), (120, 102), (121, 103), (122, 102), (121, 101), (119, 102), (121, 100), (117, 98), (111, 102), (107, 102), (110, 103), (111, 104), (114, 102), (115, 103), (117, 102), (118, 103), (115, 106), (114, 106), (113, 106), (112, 107), (112, 106), (111, 107), (116, 109), (113, 109), (114, 108), (109, 102), (111, 80), (108, 77), (111, 77), (107, 62), (106, 56), (107, 56), (106, 57), (102, 63), (108, 76), (112, 79), (112, 65), (77, 93), (79, 98), (80, 98), (79, 100), (80, 101), (81, 102), (79, 103), (76, 105), (71, 112), (70, 112), (70, 111), (71, 111), (70, 113), (72, 112), (71, 113), (72, 113), (73, 113), (73, 114), (75, 117), (74, 114), (74, 113), (74, 116), (74, 117), (73, 116), (73, 115), (71, 114), (72, 115), (72, 114), (70, 110), (62, 109), (62, 114), (64, 118), (63, 113), (65, 117), (66, 116), (67, 116), (66, 118), (67, 117), (68, 117), (68, 118), (69, 119), (69, 118), (70, 118), (71, 117), (72, 117), (72, 116), (71, 115), (70, 114), (69, 115), (68, 116), (69, 117), (70, 117), (69, 116), (68, 115), (66, 117), (67, 118), (67, 119), (66, 123), (66, 124), (75, 130), (81, 134), (84, 133), (84, 134), (63, 134), (62, 132), (61, 133), (62, 133), (61, 134), (59, 133), (59, 134), (58, 134), (56, 134), (56, 133), (56, 132), (55, 132), (55, 133), (55, 131), (54, 132), (54, 133), (53, 134), (56, 131), (57, 132), (45, 132), (45, 131), (45, 128), (46, 126), (44, 115), (42, 118), (41, 116), (42, 117), (40, 117), (40, 113), (39, 110), (26, 107), (24, 107), (21, 109), (23, 109), (22, 109), (23, 110), (21, 106), (19, 108), (18, 110), (16, 110), (16, 113), (15, 112), (12, 112), (12, 111), (11, 110), (13, 110), (12, 113), (15, 108), (12, 110), (13, 109), (12, 108), (11, 109), (12, 109), (14, 109), (14, 107), (13, 108), (10, 109), (11, 108), (10, 110), (9, 110), (10, 108), (11, 107), (12, 107), (13, 107), (15, 106), (29, 102), (19, 104), (20, 106), (19, 105), (17, 102), (14, 100), (13, 99), (15, 100), (15, 99), (11, 93), (8, 94), (9, 95), (10, 93), (9, 92), (8, 93), (9, 94), (7, 93), (6, 90), (5, 89), (7, 89), (9, 90), (10, 91), (10, 90), (11, 92), (6, 93), (8, 92), (5, 92), (8, 90), (6, 89), (11, 89), (17, 81), (16, 75), (28, 60), (30, 58), (30, 56), (28, 58), (27, 57), (26, 57), (24, 53), (23, 58), (28, 50), (31, 56), (31, 55), (32, 58), (32, 55), (33, 56), (33, 57), (15, 73), (15, 71), (17, 93), (14, 98), (36, 95), (35, 98), (36, 101), (37, 102), (35, 101), (34, 101), (34, 99), (34, 91), (47, 94), (48, 56), (32, 54), (34, 42), (34, 43), (35, 42), (43, 54), (33, 45), (34, 46), (54, 40), (69, 69), (114, 77), (48, 100), (51, 99), (49, 105), (49, 101), (57, 108), (46, 95), (38, 97), (37, 103), (36, 103), (36, 102), (35, 102), (35, 103), (34, 102), (33, 101), (33, 102), (28, 101), (17, 105), (16, 105), (19, 111), (24, 112), (22, 111), (26, 112), (34, 103), (36, 104), (37, 104), (39, 103), (40, 102), (42, 101), (38, 103), (35, 99), (46, 94), (44, 102), (46, 104), (46, 105), (59, 108), (53, 117), (50, 115), (70, 115), (73, 117), (72, 118), (72, 119), (71, 118), (71, 116), (81, 114), (80, 113), (76, 118), (77, 119), (78, 118), (77, 117), (76, 114), (74, 119), (74, 118), (76, 119), (75, 120), (75, 121), (74, 120), (73, 121), (74, 121), (75, 122), (76, 122), (74, 123), (72, 122), (73, 119), (77, 122), (76, 121), (71, 121), (73, 120), (77, 123), (80, 112), (71, 119), (68, 119), (65, 125), (62, 134), (57, 130), (57, 131), (60, 134), (55, 134), (55, 130), (44, 129), (45, 129), (43, 130), (43, 131), (44, 131), (44, 132), (43, 133), (42, 133), (42, 132), (43, 132), (44, 133), (41, 133), (41, 134), (42, 134), (40, 134), (42, 131), (45, 133), (45, 134), (44, 134), (44, 130), (43, 129), (42, 128), (42, 129), (41, 130), (41, 132), (40, 132), (41, 131), (42, 130), (40, 130), (39, 130), (41, 123), (41, 114), (41, 115), (40, 118), (27, 94), (39, 70), (27, 61), (24, 59), (24, 58), (25, 58), (26, 58), (23, 49), (26, 59), (25, 59), (23, 59), (22, 60), (21, 59), (22, 59), (17, 104), (16, 103), (15, 103), (16, 104), (15, 104), (15, 105), (14, 105), (13, 104), (13, 105), (14, 106), (13, 106), (12, 106), (22, 110), (21, 110), (16, 116), (15, 115), (14, 113), (11, 113), (10, 112), (10, 113), (11, 114), (10, 115), (11, 116), (14, 115), (16, 117), (25, 111), (25, 112), (26, 110), (16, 106), (18, 105), (37, 105), (42, 87), (52, 119), (69, 114), (70, 116), (73, 111), (70, 119), (73, 118), (47, 114), (49, 115), (54, 124), (55, 128), (56, 128), (71, 120), (70, 120), (72, 120), (73, 102), (87, 118), (94, 112), (108, 112), (104, 109), (97, 101), (112, 84), (111, 83), (113, 84), (113, 85), (114, 88), (113, 88), (114, 87), (107, 92), (113, 90), (114, 90), (106, 95), (105, 94), (104, 91), (91, 92), (90, 90), (89, 91), (88, 92), (89, 90), (89, 93), (76, 94), (40, 64), (40, 76), (81, 113), (81, 112), (82, 113), (85, 109), (86, 106), (83, 119), (65, 118), (69, 129), (73, 112), (78, 98), (79, 97), (78, 97), (79, 96), (81, 99), (82, 101), (81, 105), (82, 107), (97, 115), (97, 116), (98, 97), (29, 24), (30, 24), (29, 23), (30, 21), (27, 17), (26, 18), (28, 9), (27, 7), (28, 7), (28, 6), (26, 7), (25, 7), (24, 10), (26, 8), (25, 12), (36, 5), (37, 10), (33, 5), (33, 6), (32, 5), (37, 16), (36, 16), (23, 19), (22, 20), (23, 18), (22, 18), (22, 19), (21, 19), (20, 18), (21, 18), (20, 19), (21, 17), (19, 17), (18, 16), (18, 15), (18, 13), (17, 15), (19, 16), (20, 15), (20, 16), (20, 17), (19, 18), (18, 17), (24, 17), (23, 15), (24, 16), (22, 17), (13, 27), (12, 28), (14, 27), (15, 26), (29, 40), (27, 41), (30, 45), (41, 61), (30, 59), (29, 44), (26, 38), (30, 23), (18, 18), (17, 18), (16, 18), (18, 19), (17, 17), (16, 17), (13, 19), (8, 16), (12, 15), (12, 14), (9, 13), (8, 14), (8, 13), (7, 14), (6, 13), (6, 12), (7, 13), (6, 14), (7, 12), (11, 16), (10, 18), (10, 20), (10, 21), (11, 20), (11, 21), (12, 20), (11, 19), (13, 20), (14, 19), (8, 18), (10, 19), (6, 16), (5, 17), (6, 15), (5, 14), (5, 13), (8, 15), (7, 15), (9, 12), (9, 11), (9, 10), (10, 10), (10, 11), (11, 9), (11, 10), (11, 11), (12, 12), (11, 13), (10, 13), (12, 11), (11, 12), (10, 12), (10, 9), (9, 9), (8, 10), (8, 9), (7, 9), (6, 10), (6, 11), (7, 11), (8, 11), (12, 9), (8, 12), (5, 11), (7, 10), (5, 10), (5, 9), (6, 9), (6, 8), (7, 7), (8, 7), (7, 6), (7, 5), (6, 7), (5, 8), (5, 7), (5, 6), (5, 5), (6, 5), (6, 6), (8, 5), (9, 6), (8, 6), (10, 7), (10, 8), (11, 7), (11, 6), (12, 8), (11, 8), (13, 7), (13, 6), (12, 6), (12, 7), (13, 8), (14, 9), (13, 10), (12, 10), (13, 9), (14, 6), (15, 5), (15, 6), (14, 8), (10, 6), (12, 5), (11, 5), (14, 7), (15, 7), (15, 8), (9, 14), (12, 13), (13, 14), (14, 14), (14, 13), (14, 12), (13, 13), (16, 11), (16, 12), (15, 11), (16, 10), (15, 9), (17, 9), (16, 8), (18, 14), (19, 15), (15, 15), (15, 22), (19, 19), (19, 20), (11, 26), (10, 27), (9, 27), (10, 26), (10, 25), (11, 25), (11, 27), (12, 27), (9, 26), (8, 26), (7, 25), (7, 24), (8, 24), (7, 23), (7, 22), (6, 24), (9, 25), (10, 24), (8, 25), (9, 24), (8, 23), (10, 22), (7, 26), (7, 27), (8, 28), (9, 28), (10, 29), (9, 30), (10, 31), (9, 32), (10, 32), (10, 30), (11, 30), (11, 32), (10, 33), (11, 31), (12, 32), (12, 31), (13, 31), (13, 32), (13, 33), (14, 31), (15, 31), (16, 31), (17, 32), (16, 33), (15, 34), (15, 35), (14, 33), (14, 35), (14, 42), (16, 44), (14, 43), (14, 44), (13, 44), (13, 43), (12, 42), (11, 43), (10, 44), (10, 43), (10, 42), (9, 42), (9, 41), (13, 42), (13, 38), (14, 37), (13, 36), (13, 34), (12, 36), (11, 35), (10, 34), (10, 35), (11, 34), (9, 33), (9, 34), (8, 33), (7, 34), (6, 34), (6, 35), (8, 34), (8, 35), (9, 35), (5, 35), (6, 36), (5, 36), (6, 37), (5, 37), (7, 36), (8, 37), (7, 38), (8, 38), (7, 39), (6, 39), (5, 38), (8, 39), (6, 38), (8, 40), (7, 40), (9, 39), (5, 39), (5, 40), (5, 41), (5, 42), (6, 43), (7, 43), (7, 44), (5, 46), (6, 47), (5, 45), (5, 47), (6, 42), (8, 44), (9, 44), (11, 42), (10, 41), (10, 40), (10, 39), (11, 38), (11, 37), (11, 36), (10, 38), (9, 37), (12, 37), (8, 36), (9, 36), (10, 36), (12, 35), (12, 34), (12, 33), (14, 34), (15, 33), (15, 32), (16, 32), (13, 28), (13, 26), (13, 29), (12, 29), (8, 27), (11, 28), (10, 28), (8, 29), (9, 31), (8, 31), (7, 30), (7, 31), (6, 30), (5, 29), (5, 30), (6, 29), (7, 29), (7, 28), (6, 28), (5, 28), (9, 29), (12, 24), (12, 23), (11, 22), (17, 34), (38, 5), (37, 12), (29, 12), (25, 14), (25, 13), (29, 6), (29, 5), (29, 7), (30, 7), (30, 6), (31, 7), (39, 5), (32, 6), (31, 5), (15, 10), (14, 11), (13, 12), (13, 11), (5, 15), (5, 16), (5, 18), (6, 17), (6, 18), (5, 19), (5, 20), (6, 20), (7, 21), (6, 23), (6, 22), (5, 22), (6, 21), (5, 21), (9, 20), (9, 8), (9, 7), (8, 8), (7, 8), (10, 5), (9, 5), (17, 33), (18, 31), (20, 20), (21, 15), (21, 14), (20, 14), (22, 14), (25, 15), (24, 15), (24, 14), (24, 13), (25, 16), (27, 16), (16, 36), (9, 38), (7, 37), (11, 33), (12, 43), (8, 45), (7, 42), (8, 43), (7, 41), (8, 41), (12, 38), (14, 32), (14, 30), (13, 30), (12, 30), (11, 29), (8, 30), (6, 26), (6, 25), (8, 22), (6, 27), (5, 26), (5, 25), (5, 27), (5, 24), (10, 37), (15, 44), (16, 42), (21, 43), (17, 51), (13, 49), (11, 48), (18, 47), (19, 47), (6, 41), (8, 42), (9, 40), (13, 46), (12, 47), (20, 49), (21, 60), (18, 52), (20, 59), (31, 24), (33, 26), (17, 40), (15, 45), (20, 43), (36, 17), (31, 16), (37, 17), (41, 29), (8, 21), (7, 19), (7, 18), (7, 20), (5, 23), (6, 19), (14, 10), (5, 12), (21, 16), (30, 5), (31, 6), (23, 13), (21, 13), (20, 50), (27, 75), (10, 50), (9, 43), (6, 40), (7, 35), (7, 33), (7, 32), (8, 32), (5, 34), (6, 33), (13, 5), (14, 5), (16, 9), (16, 7), (111, 81), (112, 81), (116, 81), (116, 80), (117, 79), (116, 79), (115, 79), (112, 78), (112, 82), (114, 89), (115, 88), (113, 82), (111, 82), (117, 75), (116, 75), (117, 76), (117, 77), (118, 76), (119, 75), (121, 75), (122, 74), (120, 76), (119, 76), (120, 77), (121, 78), (120, 79), (119, 78), (119, 77), (118, 78), (120, 78), (121, 79), (122, 80), (122, 78), (122, 79), (123, 80), (123, 79), (123, 77), (121, 77), (125, 80), (124, 80), (125, 79), (129, 77), (130, 76), (131, 75), (132, 76), (132, 77), (131, 78), (130, 78), (132, 78), (134, 80), (134, 81), (134, 82), (133, 81), (131, 79), (131, 83), (132, 82), (133, 83), (134, 84), (130, 87), (131, 88), (132, 87), (133, 90), (133, 86), (129, 87), (123, 84), (125, 88), (122, 84), (119, 85), (122, 89), (114, 92), (119, 89), (115, 89), (115, 90), (114, 91), (109, 92), (107, 94), (106, 96), (113, 97), (113, 78), (113, 77), (114, 78), (112, 80), (110, 77), (118, 72), (120, 71), (121, 70), (120, 70), (117, 73), (123, 72), (122, 73), (124, 72), (125, 69), (125, 68), (124, 68), (124, 67), (123, 67), (122, 67), (121, 66), (120, 65), (119, 65), (121, 64), (122, 64), (121, 65), (122, 66), (122, 65), (123, 65), (121, 67), (120, 66), (120, 67), (123, 69), (120, 68), (123, 66), (124, 65), (124, 66), (119, 68), (118, 67), (119, 66), (118, 66), (117, 65), (116, 66), (115, 63), (116, 64), (115, 65), (117, 66), (119, 67), (114, 62), (113, 62), (113, 61), (114, 65), (108, 62), (107, 63), (114, 24), (108, 36), (106, 36), (106, 37), (105, 39), (106, 40), (105, 40), (100, 38), (105, 41), (108, 40), (109, 40), (109, 41), (110, 40), (111, 40), (112, 41), (113, 42), (112, 43), (112, 42), (113, 43), (114, 43), (116, 39), (115, 38), (119, 41), (120, 42), (121, 43), (120, 43), (122, 43), (123, 42), (126, 44), (126, 47), (125, 46), (126, 46), (125, 47), (124, 44), (121, 46), (120, 47), (119, 48), (118, 48), (116, 47), (113, 48), (112, 47), (111, 46), (110, 44), (109, 42), (108, 42), (110, 43), (109, 44), (111, 42), (116, 45), (113, 44), (114, 42), (113, 41), (114, 41), (110, 41), (111, 41), (110, 39), (109, 39), (108, 39), (107, 38), (106, 39), (38, 126), (39, 129), (39, 131), (38, 132), (37, 133), (38, 133), (37, 134), (38, 134), (39, 134), (40, 133), (46, 131), (44, 128), (47, 126), (34, 129), (14, 103), (14, 104), (18, 99), (12, 96), (13, 100), (13, 101), (12, 100), (11, 101), (8, 99), (11, 100), (12, 99), (11, 99), (13, 97), (10, 96), (9, 97), (11, 96), (11, 90), (5, 93), (6, 94), (7, 94), (5, 94), (7, 95), (6, 95), (8, 96), (5, 95), (7, 99), (7, 101), (6, 101), (6, 99), (36, 75), (47, 124), (47, 125), (56, 124), (73, 132), (79, 90), (80, 94), (85, 119), (113, 92), (117, 100), (119, 98), (118, 98), (118, 97), (120, 98), (122, 100), (122, 101), (121, 102), (123, 101), (124, 98), (123, 98), (123, 97), (127, 92), (127, 96), (128, 89), (129, 92), (129, 89), (129, 88), (130, 89), (128, 88), (129, 86), (128, 86), (127, 85), (127, 86), (128, 85), (130, 86), (134, 87), (134, 88), (134, 89), (134, 90), (129, 84), (130, 84), (129, 85), (131, 87), (131, 89), (130, 90), (125, 89), (127, 89), (130, 88), (133, 88), (133, 89), (129, 83), (130, 83), (127, 80), (127, 81), (128, 82), (129, 82), (128, 81), (128, 83), (128, 84), (130, 79), (129, 79), (132, 90), (134, 83), (128, 92), (126, 97), (124, 99), (127, 98), (132, 107), (130, 111), (131, 110), (131, 109), (130, 109), (131, 108), (132, 108), (130, 108), (129, 108), (130, 107), (131, 107), (129, 111), (130, 110), (129, 110), (130, 112), (129, 112), (132, 109), (125, 113), (127, 112), (126, 113), (125, 119), (124, 116), (123, 115), (122, 116), (122, 117), (122, 118), (121, 117), (119, 117), (120, 115), (121, 115), (122, 115), (120, 114), (122, 113), (121, 116), (122, 114), (123, 113), (123, 112), (122, 112), (121, 111), (122, 111), (123, 110), (123, 111), (124, 113), (123, 114), (123, 116), (124, 115), (125, 114), (128, 112), (128, 111), (127, 111), (127, 110), (128, 109), (129, 109), (128, 108), (126, 107), (123, 99), (122, 99), (121, 98), (121, 97), (120, 96), (122, 97), (121, 96), (121, 94), (124, 88), (122, 96), (120, 97), (121, 99), (127, 100), (126, 100), (128, 99), (133, 95), (130, 102), (132, 92), (131, 92), (132, 91), (134, 91), (133, 92), (132, 93), (131, 91), (133, 91), (131, 106), (129, 103), (127, 107), (126, 112), (126, 111), (126, 110), (127, 109), (128, 110), (125, 112), (125, 111), (124, 110), (123, 109), (122, 108), (121, 107), (122, 109), (126, 109), (134, 113), (121, 110), (121, 112), (118, 111), (120, 121), (124, 119), (123, 118), (123, 117), (124, 118), (126, 121), (126, 122), (125, 126), (124, 126), (123, 126), (122, 126), (122, 128), (119, 131), (119, 128), (119, 127), (119, 129), (123, 128), (124, 128), (124, 127), (123, 127), (125, 122), (125, 121), (127, 122), (126, 126), (125, 127), (125, 129), (126, 129), (127, 129), (128, 130), (127, 131), (126, 132), (125, 132), (124, 132), (125, 133), (124, 133), (124, 134), (123, 133), (126, 131), (125, 130), (126, 130), (127, 128), (128, 129), (128, 131), (128, 132), (128, 133), (127, 132), (126, 133), (125, 134), (123, 134), (122, 133), (124, 129), (125, 128), (125, 131), (121, 133), (118, 128), (117, 129), (116, 128), (117, 128), (118, 127), (117, 127), (116, 127), (116, 126), (115, 126), (115, 125), (117, 121), (117, 111), (124, 111), (124, 112), (132, 122), (133, 123), (134, 121), (134, 123), (125, 116), (121, 108), (99, 115), (99, 116), (100, 115), (82, 109), (81, 103), (82, 102), (83, 101), (83, 100), (116, 82), (113, 83), (117, 80), (117, 81), (115, 78), (116, 77), (118, 81), (119, 80), (120, 80), (122, 81), (118, 77), (118, 75), (118, 74), (119, 79), (121, 81), (122, 82), (121, 82), (122, 83), (123, 82), (123, 81), (124, 81), (124, 82), (125, 82), (126, 82), (126, 81), (124, 83), (112, 83), (121, 84), (121, 83), (119, 97), (119, 96), (125, 100), (126, 102), (130, 106), (129, 107), (134, 119), (134, 118), (132, 121), (120, 123), (118, 122), (114, 122), (90, 129), (114, 126), (113, 128), (112, 127), (120, 105), (121, 104), (120, 104), (122, 104), (123, 100), (120, 99), (121, 105), (82, 100), (107, 61), (107, 60), (106, 61), (108, 61), (109, 61), (109, 60), (110, 60), (110, 59), (109, 59), (109, 58), (108, 60), (108, 59), (109, 57), (108, 56), (108, 57), (107, 57), (106, 55), (109, 51), (110, 53), (111, 52), (106, 38), (109, 26), (110, 35), (110, 36), (109, 37), (109, 36), (107, 37), (110, 37), (46, 109), (12, 114), (13, 113), (9, 109), (10, 111), (9, 112), (9, 111), (8, 111), (7, 111), (8, 113), (9, 113), (11, 112), (10, 114), (9, 114), (8, 114), (6, 115), (9, 125), (10, 125), (7, 125), (8, 126), (8, 125), (10, 103), (8, 103), (8, 104), (5, 104), (6, 105), (6, 106), (5, 107), (6, 107), (5, 108), (7, 104), (7, 105), (5, 105), (7, 106), (7, 108), (8, 108), (8, 109), (7, 109), (8, 110), (7, 110), (6, 111), (5, 111), (5, 110), (6, 110), (6, 109), (7, 107), (6, 108), (8, 107), (5, 112), (6, 112), (7, 112), (8, 112), (9, 107), (8, 106), (10, 106), (9, 108), (11, 106), (12, 104), (10, 107), (11, 111), (5, 109), (7, 113), (7, 114), (6, 113), (5, 114), (6, 114), (5, 113), (5, 106), (6, 126), (5, 127), (5, 128), (5, 129), (6, 130), (5, 130), (5, 131), (6, 129), (6, 128), (7, 128), (7, 129), (7, 127), (5, 121), (6, 121), (6, 122), (7, 121), (7, 126), (6, 125), (5, 126), (6, 127), (8, 127), (18, 129), (19, 129), (20, 128), (22, 130), (21, 134), (20, 130), (22, 132), (39, 133), (43, 134), (39, 132), (40, 131), (43, 128), (42, 127), (33, 131), (35, 133), (25, 108), (15, 132), (15, 134), (22, 133), (20, 126), (36, 134), (52, 132), (53, 133), (34, 134), (35, 134), (8, 130), (7, 130), (6, 131), (5, 132), (7, 131), (7, 132), (6, 133), (7, 134), (7, 133), (8, 133), (5, 133), (6, 132), (6, 134), (8, 134), (8, 132), (5, 134), (11, 132), (5, 96), (5, 116), (5, 115), (4, 107), (3, 107), (102, 121), (90, 134), (98, 113), (102, 106), (121, 76), (120, 64), (121, 63), (122, 63), (121, 62), (120, 62), (118, 61), (119, 62), (118, 63), (117, 64), (117, 62), (120, 63), (122, 70), (125, 67), (120, 61), (117, 61), (116, 61), (119, 58), (120, 57), (118, 54), (119, 54), (120, 53), (121, 54), (117, 52), (119, 51), (120, 46), (122, 45), (124, 47), (123, 48), (105, 37), (107, 39), (108, 38), (112, 44), (116, 95), (118, 96), (117, 97), (117, 96), (116, 97), (134, 92), (134, 93), (126, 85), (126, 84), (125, 83), (126, 83), (123, 83), (117, 78), (118, 80), (116, 78), (115, 77), (120, 75), (123, 73), (124, 71), (124, 73), (125, 74), (126, 73), (125, 73), (125, 72), (113, 64), (116, 62), (119, 59), (117, 55), (117, 54), (116, 53), (114, 53), (107, 58), (108, 55), (112, 52), (112, 53), (111, 54), (111, 55), (111, 56), (110, 57), (111, 57), (112, 58), (113, 57), (112, 56), (112, 57), (113, 58), (113, 59), (112, 59), (111, 60), (111, 61), (110, 63), (111, 63), (112, 61), (113, 60), (114, 59), (118, 65), (118, 64), (120, 60), (114, 58), (111, 58), (110, 61), (109, 62), (111, 59), (110, 58), (112, 60), (112, 62), (115, 57), (119, 64), (119, 63), (121, 61), (122, 60), (123, 60), (124, 59), (125, 59), (125, 71), (126, 70), (127, 69), (127, 70), (126, 69), (127, 68), (128, 69), (129, 69), (129, 67), (126, 68), (125, 70), (125, 66), (126, 67), (124, 64), (125, 63), (121, 59), (116, 56), (115, 55), (116, 55), (123, 59), (126, 59), (127, 60), (126, 60), (125, 58), (126, 58), (127, 59), (128, 58), (128, 59), (125, 57), (126, 56), (127, 55), (128, 56), (127, 56), (128, 57), (127, 54), (128, 53), (127, 53), (126, 54), (125, 54), (124, 53), (123, 54), (122, 53), (123, 53), (124, 54), (126, 55), (126, 57), (127, 63), (126, 64), (126, 63), (127, 62), (126, 62), (128, 64), (129, 65), (128, 63), (128, 62), (127, 61), (124, 58), (123, 57), (123, 58), (122, 59), (121, 58), (122, 58), (112, 55), (110, 56), (109, 56), (115, 53), (116, 54), (5, 32), (5, 33), (6, 32), (6, 31), (5, 31), (56, 129), (54, 134), (108, 37), (112, 40), (112, 39), (109, 38), (110, 38), (111, 38), (112, 38), (113, 38), (112, 37), (111, 37), (112, 36), (114, 35), (113, 39), (113, 37), (113, 36), (113, 35), (80, 95), (108, 58), (106, 60), (106, 59), (122, 57), (122, 55), (122, 54), (123, 55), (120, 52), (123, 49), (121, 53), (124, 57), (124, 56), (124, 55), (124, 50), (115, 54), (115, 39), (111, 39), (114, 37), (116, 38), (114, 38), (107, 55), (114, 50), (115, 49), (115, 51), (129, 64), (130, 65), (131, 64), (130, 64), (130, 63), (131, 62), (132, 62), (131, 61), (132, 61), (133, 62), (132, 63), (134, 62), (133, 63), (133, 64), (132, 64), (132, 65), (131, 63), (134, 64), (134, 63), (133, 65), (133, 60), (132, 59), (131, 60), (133, 61), (134, 61), (132, 66), (133, 66), (134, 66), (133, 67), (132, 68), (132, 69), (131, 70), (131, 71), (130, 71), (127, 71), (126, 75), (126, 71), (131, 72), (132, 71), (130, 70), (128, 70), (127, 83), (127, 84), (127, 82), (121, 38), (126, 38), (122, 42), (123, 64), (128, 74), (130, 72), (132, 73), (133, 73), (134, 74), (134, 75), (133, 74), (131, 73), (132, 72), (129, 74), (131, 74), (132, 75), (134, 73), (133, 72), (131, 69), (131, 68), (132, 67), (131, 67), (131, 66), (130, 66), (130, 67), (129, 63), (129, 62), (130, 62), (131, 65), (133, 71), (134, 70), (134, 69), (133, 69), (134, 68), (132, 60), (134, 60), (133, 59), (133, 58), (132, 58), (131, 57), (131, 56), (130, 57), (130, 56), (130, 55), (130, 54), (131, 54), (130, 53), (132, 55), (132, 56), (131, 55), (133, 55), (134, 54), (134, 55), (134, 56), (133, 54), (132, 53), (132, 54), (131, 53), (134, 57), (133, 57), (134, 58), (130, 59), (130, 60), (130, 58), (131, 58), (133, 56), (132, 57), (128, 61), (123, 56), (124, 48), (124, 52), (130, 47), (127, 42), (126, 43), (137, 21), (138, 22), (138, 21), (127, 57), (125, 55), (129, 53), (130, 51), (130, 52), (131, 52), (132, 52), (132, 51), (131, 51), (131, 46), (131, 45), (132, 46), (132, 47), (131, 47), (128, 45), (128, 46), (129, 52), (131, 50), (132, 50), (131, 49), (98, 115), (98, 114), (10, 132), (9, 133), (9, 134), (10, 133), (11, 134), (11, 133), (10, 134), (125, 56), (107, 59), (118, 79), (110, 127), (109, 127), (110, 118), (107, 134), (82, 111), (122, 98), (132, 70), (134, 67), (134, 65), (131, 59), (129, 58), (127, 58), (106, 54), (126, 39), (132, 37), (132, 48), (133, 48), (133, 49), (134, 49), (134, 50), (133, 50), (132, 49), (131, 48), (133, 51), (134, 52), (133, 52), (129, 54), (128, 55), (129, 55), (128, 54), (129, 60), (129, 61), (130, 61), (133, 68), (133, 70), (134, 71), (133, 75), (133, 76), (131, 76), (132, 74), (130, 74), (129, 75), (128, 75), (129, 76), (130, 77), (131, 77), (134, 79), (125, 81), (134, 5), (124, 74), (123, 41), (128, 60), (134, 59), (129, 59), (134, 53), (129, 56), (129, 57), (133, 53), (134, 48), (134, 51), (130, 75), (134, 72), (108, 133), (108, 134), (114, 131), (85, 134), (122, 127), (120, 128), (118, 126), (133, 124), (132, 124), (131, 123), (127, 126), (127, 127), (128, 127), (126, 128), (128, 128), (129, 127), (126, 127), (127, 130), (129, 129), (129, 128), (130, 128), (116, 130), (115, 132), (118, 133), (118, 134), (116, 132), (116, 133), (117, 134), (127, 121), (122, 134), (129, 133), (130, 133), (131, 132), (130, 132), (131, 131), (132, 131), (132, 130), (132, 129), (131, 128), (131, 126), (132, 127), (131, 127), (130, 126), (130, 129), (130, 130), (131, 130), (130, 131), (129, 131), (129, 130), (127, 133), (126, 134), (127, 134), (129, 134), (128, 134), (119, 134), (129, 132), (117, 126), (118, 132), (132, 125), (131, 125), (131, 129), (132, 126), (121, 134), (133, 82), (133, 126), (134, 125), (134, 124), (133, 125), (134, 126), (133, 127), (134, 128), (133, 129), (132, 128), (133, 128), (133, 130), (133, 131), (132, 132), (131, 133), (132, 134), (132, 133), (133, 133), (133, 134), (134, 134), (134, 133), (133, 132), (131, 134), (134, 131), (134, 132), (134, 130), (130, 134), (134, 129), (134, 127), (139, 21), (140, 21), (139, 22), (140, 23), (139, 24), (139, 23), (140, 22), (138, 23), (141, 21), (142, 22), (142, 21), (143, 20), (144, 19), (145, 18), (144, 17), (143, 16), (144, 15), (142, 17), (141, 16), (141, 17), (140, 15), (139, 14), (139, 13), (140, 12), (140, 11), (139, 10), (141, 11), (140, 10), (139, 9), (140, 13), (138, 14), (139, 15), (138, 15), (139, 16), (140, 16), (140, 17), (138, 17), (139, 18), (138, 19), (139, 19), (139, 17), (142, 18), (143, 17), (144, 18), (143, 18), (145, 17), (144, 16), (145, 16), (145, 14), (145, 15), (146, 14), (146, 15), (147, 15), (146, 16), (146, 18), (143, 19), (142, 19), (141, 18), (141, 19), (142, 20), (145, 19), (146, 19), (147, 18), (147, 20), (147, 19), (146, 20), (147, 21), (148, 20), (148, 19), (148, 17), (147, 16), (148, 16), (149, 16), (149, 15), (148, 15), (147, 14), (147, 13), (146, 12), (148, 14), (146, 13), (146, 11), (145, 13), (147, 12), (145, 12), (149, 13), (150, 14), (151, 15), (152, 14), (152, 13), (151, 12), (152, 12), (151, 13), (153, 11), (151, 11), (151, 10), (150, 9), (151, 9), (152, 9), (152, 10), (153, 10), (154, 9), (155, 8), (155, 7), (156, 7), (156, 6), (156, 5), (155, 6), (154, 5), (155, 5), (157, 6), (158, 5), (159, 5), (158, 6), (160, 5), (161, 6), (162, 6), (161, 7), (163, 7), (162, 8), (163, 8), (164, 7), (164, 8), (165, 8), (166, 8), (165, 9), (164, 10), (164, 11), (163, 12), (164, 12), (165, 11), (165, 12), (164, 13), (164, 14), (165, 13), (165, 14), (164, 15), (163, 15), (163, 16), (163, 17), (163, 18), (162, 19), (161, 20), (160, 21), (160, 20), (160, 19), (159, 19), (158, 19), (159, 20), (158, 20), (158, 21), (158, 22), (159, 21), (160, 18), (159, 18), (157, 19), (156, 18), (156, 19), (157, 18), (158, 18), (156, 17), (157, 16), (158, 17), (157, 17), (158, 16), (159, 16), (159, 15), (160, 16), (160, 15), (159, 14), (158, 15), (159, 17), (157, 20), (156, 20), (157, 21), (161, 19), (161, 18), (162, 17), (162, 18), (163, 19), (161, 17), (164, 19), (163, 20), (162, 20), (162, 21), (163, 22), (163, 21), (162, 22), (162, 23), (162, 24), (163, 24), (161, 23), (161, 24), (161, 22), (162, 25), (160, 23), (164, 21), (165, 21), (166, 21), (167, 20), (168, 19), (169, 19), (169, 20), (170, 19), (171, 19), (171, 20), (171, 21), (172, 21), (171, 22), (170, 22), (169, 22), (170, 23), (169, 24), (170, 25), (169, 25), (168, 25), (167, 25), (169, 26), (168, 26), (169, 27), (170, 26), (170, 27), (171, 28), (171, 29), (170, 29), (172, 29), (173, 28), (174, 28), (175, 27), (176, 27), (175, 28), (176, 29), (176, 30), (175, 29), (175, 30), (174, 29), (174, 30), (175, 31), (176, 31), (173, 27), (174, 27), (172, 28), (173, 26), (172, 25), (173, 25), (172, 24), (173, 23), (173, 24), (174, 25), (172, 23), (172, 22), (171, 23), (173, 22), (174, 22), (175, 23), (174, 24), (175, 26), (175, 25), (171, 25), (170, 24), (174, 26), (174, 23), (175, 24), (176, 24), (177, 25), (178, 24), (178, 23), (179, 25), (179, 26), (178, 26), (178, 25), (180, 27), (181, 28), (182, 29), (181, 29), (180, 30), (180, 31), (181, 30), (179, 31), (179, 32), (180, 32), (178, 33), (177, 33), (176, 33), (175, 32), (174, 31), (173, 30), (173, 29), (177, 28), (176, 28), (173, 31), (172, 32), (173, 32), (174, 33), (174, 34), (173, 35), (174, 35), (175, 34), (176, 34), (177, 34), (177, 35), (178, 35), (179, 34), (180, 33), (181, 34), (181, 35), (182, 34), (183, 33), (184, 34), (185, 34), (185, 35), (184, 36), (185, 36), (186, 35), (187, 34), (187, 33), (187, 32), (188, 34), (188, 35), (189, 36), (188, 36), (188, 37), (189, 38), (188, 38), (187, 36), (186, 37), (187, 37), (190, 36), (190, 35), (190, 34), (191, 36), (191, 35), (190, 33), (189, 33), (188, 33), (187, 35), (189, 35), (190, 37), (188, 39), (191, 34), (191, 33), (192, 32), (191, 32), (191, 37), (187, 40), (186, 39), (185, 40), (184, 39), (183, 39), (183, 38), (182, 37), (183, 36), (182, 35), (181, 33), (180, 34), (181, 36), (180, 36), (181, 32), (182, 33), (183, 34), (179, 35), (180, 35), (179, 36), (179, 37), (179, 38), (180, 39), (179, 39), (178, 38), (177, 37), (176, 37), (176, 36), (175, 37), (175, 38), (174, 39), (176, 38), (177, 36), (178, 36), (179, 33), (182, 36), (181, 37), (183, 35), (184, 33), (176, 32), (175, 33), (173, 34), (172, 35), (172, 34), (171, 34), (171, 33), (171, 32), (170, 33), (172, 33), (173, 33), (174, 32), (172, 30), (171, 30), (170, 30), (170, 31), (169, 31), (169, 30), (168, 29), (169, 29), (168, 28), (168, 27), (169, 28), (171, 27), (170, 28), (171, 26), (172, 26), (178, 28), (179, 29), (179, 28), (179, 27), (180, 28), (180, 29), (179, 30), (178, 31), (177, 30), (177, 29), (177, 27), (178, 29), (178, 30), (181, 31), (182, 30), (183, 30), (184, 29), (185, 29), (184, 28), (169, 34), (168, 35), (169, 36), (168, 36), (169, 35), (170, 34), (169, 33), (169, 32), (168, 33), (167, 33), (166, 33), (167, 32), (166, 32), (167, 34), (168, 34), (167, 31), (166, 31), (168, 32), (168, 30), (167, 30), (167, 29), (167, 26), (166, 27), (166, 28), (166, 29), (165, 28), (167, 27), (167, 28), (166, 26), (166, 25), (165, 26), (165, 27), (165, 25), (165, 24), (164, 27), (163, 26), (162, 26), (162, 27), (161, 27), (163, 25), (164, 26), (164, 28), (163, 29), (162, 29), (163, 30), (162, 28), (163, 28), (161, 28), (164, 30), (163, 31), (165, 29), (165, 30), (165, 31), (164, 29), (164, 31), (164, 32), (163, 33), (162, 32), (161, 32), (160, 33), (160, 32), (159, 32), (158, 32), (158, 33), (157, 32), (157, 31), (158, 31), (157, 33), (158, 34), (158, 35), (157, 34), (156, 33), (155, 34), (154, 35), (153, 35), (154, 36), (153, 36), (152, 36), (152, 35), (151, 35), (151, 36), (150, 36), (153, 37), (152, 37), (154, 38), (155, 38), (155, 39), (156, 40), (156, 41), (155, 40), (157, 40), (155, 42), (155, 43), (156, 43), (156, 44), (157, 45), (155, 44), (155, 45), (157, 44), (158, 44), (158, 45), (158, 46), (159, 46), (156, 45), (159, 47), (160, 46), (158, 47), (158, 48), (158, 49), (159, 49), (159, 48), (158, 50), (159, 50), (160, 51), (160, 50), (160, 48), (160, 49), (161, 50), (162, 50), (162, 51), (161, 51), (162, 52), (161, 53), (162, 53), (163, 52), (161, 49), (162, 48), (163, 47), (164, 47), (163, 46), (162, 46), (162, 47), (163, 48), (161, 47), (161, 48), (164, 48), (163, 49), (162, 49), (164, 49), (163, 50), (161, 46), (160, 47), (164, 50), (163, 51), (161, 52), (163, 53), (164, 54), (165, 54), (166, 55), (166, 56), (166, 57), (165, 55), (164, 56), (164, 57), (164, 58), (164, 59), (163, 58), (163, 59), (163, 60), (162, 59), (165, 59), (164, 60), (164, 61), (165, 62), (164, 62), (163, 61), (161, 58), (160, 57), (159, 56), (158, 55), (158, 56), (157, 55), (157, 56), (156, 54), (157, 54), (156, 53), (155, 54), (156, 55), (155, 55), (154, 56), (153, 55), (152, 56), (151, 55), (151, 56), (150, 55), (149, 54), (150, 54), (148, 53), (149, 53), (150, 52), (149, 51), (151, 52), (150, 51), (149, 52), (150, 53), (150, 56), (152, 54), (153, 56), (152, 55), (152, 57), (151, 58), (151, 57), (153, 57), (159, 54), (159, 55), (158, 57), (156, 56), (158, 54), (157, 53), (160, 54), (154, 54), (154, 55), (160, 53), (162, 54), (160, 52), (159, 52), (159, 51), (158, 51), (157, 52), (158, 53), (159, 53), (160, 56), (158, 52), (157, 50), (157, 48), (157, 47), (156, 48), (155, 48), (154, 47), (153, 47), (152, 47), (151, 48), (151, 49), (152, 49), (153, 48), (152, 48), (151, 47), (150, 46), (149, 45), (149, 46), (148, 46), (147, 45), (147, 46), (146, 45), (147, 44), (148, 43), (149, 44), (150, 43), (150, 44), (148, 45), (148, 44), (148, 47), (149, 43), (147, 43), (147, 42), (147, 41), (146, 40), (147, 40), (148, 41), (147, 39), (148, 39), (148, 40), (148, 42), (149, 41), (149, 40), (149, 39), (150, 40), (150, 41), (146, 41), (145, 41), (146, 42), (146, 43), (145, 42), (145, 43), (144, 44), (144, 43), (143, 44), (143, 45), (142, 46), (141, 47), (140, 46), (139, 45), (140, 44), (139, 44), (140, 45), (141, 43), (141, 44), (140, 43), (139, 43), (140, 42), (141, 41), (141, 40), (141, 39), (142, 38), (142, 39), (143, 40), (144, 40), (145, 40), (144, 39), (144, 41), (143, 41), (142, 42), (143, 43), (142, 44), (142, 47), (140, 48), (141, 49), (140, 50), (140, 51), (139, 52), (139, 53), (140, 54), (141, 54), (141, 53), (142, 53), (142, 52), (141, 51), (140, 52), (139, 51), (142, 50), (142, 51), (143, 52), (144, 51), (143, 51), (143, 50), (144, 50), (145, 51), (141, 52), (139, 50), (141, 50), (142, 49), (143, 49), (145, 52), (144, 52), (143, 53), (140, 53), (140, 55), (141, 48), (142, 48), (143, 47), (143, 46), (144, 47), (143, 48), (144, 49), (145, 49), (144, 48), (145, 48), (146, 48), (141, 46), (142, 54), (141, 55), (139, 54), (139, 55), (138, 56), (139, 57), (139, 58), (139, 56), (138, 55), (138, 54), (138, 53), (138, 52), (138, 51), (138, 50), (139, 49), (138, 48), (140, 49), (138, 49), (139, 47), (142, 45), (138, 43), (140, 41), (140, 40), (139, 39), (141, 37), (142, 36), (142, 37), (142, 35), (143, 36), (141, 34), (141, 35), (143, 38), (143, 39), (142, 41), (142, 43), (139, 40), (138, 41), (139, 41), (140, 39), (142, 40), (139, 38), (141, 38), (141, 36), (140, 33), (140, 34), (141, 33), (140, 32), (140, 31), (141, 32), (141, 31), (142, 30), (139, 31), (139, 30), (140, 30), (141, 29), (141, 30), (142, 31), (143, 31), (143, 30), (143, 29), (144, 30), (145, 30), (146, 29), (145, 29), (144, 31), (144, 29), (146, 30), (147, 28), (148, 27), (148, 26), (147, 25), (147, 26), (148, 25), (149, 25), (150, 26), (150, 25), (149, 26), (151, 26), (152, 27), (151, 27), (152, 26), (153, 25), (154, 26), (153, 27), (152, 28), (151, 28), (152, 29), (151, 30), (152, 30), (153, 31), (153, 32), (152, 32), (151, 31), (152, 31), (153, 30), (154, 29), (155, 30), (155, 29), (154, 28), (155, 28), (156, 29), (156, 30), (157, 30), (158, 29), (159, 28), (158, 28), (160, 28), (160, 29), (161, 30), (159, 29), (159, 27), (158, 26), (157, 27), (158, 27), (160, 27), (159, 26), (159, 25), (159, 24), (158, 25), (157, 24), (157, 25), (156, 25), (156, 24), (155, 23), (155, 24), (156, 23), (156, 22), (157, 22), (157, 26), (156, 26), (155, 26), (155, 27), (157, 28), (159, 30), (160, 30), (160, 25), (161, 26), (164, 23), (165, 22), (166, 23), (166, 24), (167, 24), (167, 23), (166, 22), (168, 24), (168, 23), (167, 22), (165, 20), (163, 23), (164, 24), (161, 25), (164, 25), (165, 23), (163, 27), (168, 21), (167, 21), (168, 22), (169, 23), (169, 21), (170, 20), (170, 18), (172, 18), (173, 17), (172, 17), (171, 18), (172, 19), (168, 20), (170, 21), (173, 20), (174, 19), (174, 20), (173, 21), (174, 21), (172, 20), (175, 22), (172, 31), (171, 31), (170, 32), (170, 35), (169, 37), (168, 37), (167, 36), (166, 35), (165, 35), (166, 36), (166, 37), (167, 37), (166, 34), (166, 30), (162, 30), (162, 31), (161, 29), (156, 27), (157, 29), (156, 28), (160, 31), (163, 32), (165, 32), (161, 33), (161, 31), (161, 34), (160, 34), (159, 34), (159, 33), (156, 31), (156, 32), (158, 30), (160, 26), (162, 34), (163, 35), (164, 36), (164, 35), (163, 36), (165, 34), (164, 33), (163, 34), (162, 33), (164, 34), (165, 33), (163, 37), (162, 35), (161, 35), (161, 36), (160, 36), (159, 37), (159, 36), (160, 35), (162, 36), (167, 35), (167, 38), (167, 39), (168, 38), (169, 38), (170, 37), (170, 36), (169, 39), (168, 39), (169, 40), (170, 40), (171, 39), (171, 38), (172, 39), (173, 40), (174, 40), (175, 40), (175, 39), (175, 35), (175, 36), (174, 37), (176, 35), (172, 38), (173, 38), (173, 37), (172, 37), (170, 38), (170, 39), (172, 36), (174, 38), (173, 41), (174, 42), (173, 42), (172, 43), (172, 42), (171, 44), (172, 45), (172, 44), (171, 45), (170, 43), (171, 43), (170, 44), (169, 42), (169, 43), (168, 42), (169, 41), (165, 36), (164, 37), (162, 37), (161, 37), (160, 38), (161, 39), (162, 39), (161, 40), (162, 40), (163, 41), (164, 41), (163, 42), (162, 42), (163, 43), (162, 43), (164, 44), (163, 44), (164, 43), (165, 45), (164, 46), (165, 47), (166, 46), (167, 45), (166, 45), (165, 46), (166, 47), (167, 46), (168, 47), (169, 47), (169, 48), (168, 46), (168, 45), (167, 47), (166, 48), (167, 49), (168, 50), (168, 49), (169, 50), (167, 50), (166, 50), (166, 51), (165, 51), (165, 52), (166, 52), (165, 53), (164, 53), (165, 49), (165, 50), (164, 51), (163, 54), (163, 55), (162, 55), (162, 56), (162, 57), (161, 60), (160, 59), (159, 60), (158, 59), (159, 59), (160, 60), (159, 61), (158, 62), (159, 62), (158, 61), (157, 61), (156, 62), (156, 61), (157, 63), (158, 63), (156, 64), (155, 64), (155, 65), (154, 66), (153, 66), (155, 66), (154, 67), (153, 67), (155, 68), (154, 68), (156, 67), (157, 68), (158, 67), (159, 66), (158, 65), (158, 64), (159, 65), (160, 64), (161, 63), (161, 65), (162, 64), (159, 64), (157, 65), (156, 65), (157, 64), (157, 62), (155, 61), (155, 60), (156, 59), (155, 59), (155, 58), (156, 58), (155, 57), (154, 57), (155, 56), (156, 57), (156, 60), (154, 60), (153, 60), (152, 59), (153, 58), (152, 60), (153, 59), (157, 57), (154, 59), (153, 61), (154, 61), (155, 62), (154, 63), (154, 62), (153, 62), (152, 62), (152, 61), (151, 60), (152, 58), (151, 54), (151, 53), (152, 51), (153, 52), (154, 51), (152, 50), (153, 51), (153, 53), (149, 55), (149, 56), (148, 56), (149, 57), (148, 55), (147, 55), (146, 54), (147, 53), (147, 54), (146, 53), (147, 52), (148, 51), (150, 50), (151, 51), (149, 50), (149, 49), (150, 49), (150, 48), (151, 50), (149, 47), (147, 47), (146, 47), (147, 48), (145, 47), (144, 46), (141, 45), (140, 47), (139, 48), (144, 53), (143, 54), (143, 55), (144, 54), (144, 55), (145, 54), (146, 55), (148, 54), (148, 52), (147, 51), (146, 50), (147, 49), (148, 48), (146, 46), (146, 49), (146, 51), (147, 50), (145, 50), (148, 50), (146, 52), (142, 56), (141, 56), (141, 57), (142, 58), (142, 59), (143, 60), (143, 61), (144, 61), (145, 62), (145, 61), (144, 62), (143, 63), (143, 64), (142, 65), (143, 66), (144, 65), (144, 66), (145, 67), (144, 67), (143, 67), (144, 68), (144, 69), (143, 70), (144, 71), (145, 70), (144, 72), (145, 72), (144, 73), (145, 71), (146, 72), (147, 71), (148, 70), (149, 70), (149, 69), (148, 71), (148, 69), (148, 72), (149, 73), (148, 73), (148, 74), (147, 73), (147, 72), (146, 71), (144, 70), (145, 69), (146, 68), (146, 69), (143, 71), (143, 72), (143, 73), (142, 70), (142, 71), (143, 69), (143, 68), (142, 69), (145, 68), (141, 68), (142, 67), (142, 68), (141, 69), (141, 70), (141, 71), (141, 67), (140, 67), (141, 66), (141, 65), (140, 65), (139, 64), (138, 65), (139, 65), (138, 66), (138, 67), (138, 68), (138, 69), (139, 69), (139, 70), (140, 70), (140, 69), (139, 68), (138, 70), (140, 68), (139, 71), (144, 64), (145, 64), (146, 65), (147, 66), (147, 67), (147, 68), (148, 67), (146, 67), (147, 65), (146, 66), (145, 65), (145, 66), (140, 66), (139, 66), (139, 67), (138, 72), (139, 72), (139, 73), (140, 74), (141, 75), (142, 76), (143, 77), (144, 76), (144, 77), (145, 76), (145, 77), (146, 77), (147, 78), (148, 79), (149, 78), (150, 79), (151, 78), (152, 78), (152, 77), (153, 79), (153, 80), (154, 81), (155, 81), (154, 80), (152, 79), (152, 80), (154, 79), (155, 79), (153, 78), (154, 78), (155, 77), (155, 76), (154, 75), (156, 77), (155, 78), (154, 77), (156, 79), (156, 80), (157, 81), (156, 82), (156, 83), (157, 84), (155, 84), (155, 83), (156, 84), (157, 83), (157, 82), (154, 82), (156, 78), (157, 79), (157, 80), (156, 81), (155, 80), (157, 78), (156, 76), (156, 75), (155, 75), (156, 74), (154, 76), (153, 76), (152, 76), (151, 75), (151, 76), (153, 77), (157, 75), (155, 74), (153, 81), (153, 82), (152, 83), (153, 83), (152, 84), (151, 84), (152, 85), (151, 85), (150, 85), (150, 86), (151, 83), (152, 82), (152, 81), (151, 81), (151, 80), (150, 77), (150, 78), (149, 79), (149, 77), (149, 76), (148, 77), (148, 78), (147, 77), (146, 78), (146, 76), (144, 78), (146, 75), (147, 75), (150, 72), (151, 73), (151, 72), (152, 73), (152, 72), (151, 71), (150, 73), (150, 71), (151, 70), (152, 70), (153, 70), (153, 71), (152, 69), (153, 68), (152, 68), (151, 67), (152, 66), (151, 66), (152, 65), (151, 65), (151, 64), (152, 64), (152, 63), (153, 64), (151, 63), (150, 63), (150, 64), (151, 62), (150, 62), (151, 61), (155, 63), (154, 64), (153, 65), (154, 58), (153, 63), (149, 63), (150, 61), (149, 60), (149, 61), (148, 64), (149, 62), (148, 62), (148, 60), (147, 60), (146, 59), (145, 60), (144, 59), (144, 60), (145, 59), (144, 58), (145, 58), (144, 57), (143, 57), (143, 56), (142, 57), (144, 56), (145, 57), (140, 56), (140, 57), (142, 55), (145, 53), (145, 46), (144, 45), (145, 45), (145, 44), (143, 42), (141, 42), (139, 42), (144, 42), (146, 44), (150, 45), (151, 45), (149, 42), (150, 42), (149, 38), (150, 38), (150, 37), (151, 37), (152, 38), (151, 38), (153, 38), (154, 39), (154, 40), (153, 40), (152, 41), (152, 42), (151, 43), (151, 42), (151, 41), (151, 40), (149, 48), (150, 47), (148, 49), (152, 52), (153, 49), (154, 49), (153, 50), (154, 50), (155, 50), (156, 50), (155, 51), (155, 52), (156, 52), (155, 53), (154, 52), (156, 49), (157, 49), (160, 45), (159, 44), (162, 45), (161, 44), (160, 43), (159, 43), (160, 42), (159, 45), (157, 51), (156, 51), (154, 53), (152, 53), (153, 54), (152, 46), (153, 46), (154, 46), (153, 45), (152, 45), (151, 46), (139, 46), (138, 47), (138, 46), (138, 44), (138, 42), (140, 38), (140, 36), (139, 35), (138, 34), (138, 36), (139, 34), (138, 33), (139, 32), (142, 33), (142, 32), (145, 31), (143, 32), (143, 33), (142, 34), (138, 32), (139, 33), (140, 35), (140, 29), (139, 28), (138, 29), (138, 28), (139, 29), (138, 30), (143, 28), (142, 29), (144, 28), (143, 27), (142, 26), (141, 27), (140, 27), (138, 27), (139, 27), (140, 28), (141, 28), (141, 26), (142, 27), (142, 28), (140, 26), (139, 26), (138, 31), (144, 33), (143, 34), (144, 32), (144, 34), (145, 35), (146, 34), (147, 35), (148, 36), (149, 35), (148, 34), (144, 35), (144, 36), (146, 36), (147, 36), (147, 37), (145, 37), (145, 38), (144, 38), (144, 37), (145, 36), (146, 37), (147, 38), (145, 39), (143, 37), (139, 37), (140, 37), (139, 36), (138, 37), (138, 35), (138, 26), (138, 25), (144, 27), (145, 27), (146, 28), (146, 27), (147, 27), (146, 26), (144, 26), (147, 29), (147, 31), (148, 31), (149, 31), (150, 30), (153, 29), (151, 29), (150, 29), (150, 27), (147, 24), (146, 24), (145, 24), (145, 23), (144, 25), (145, 26), (141, 25), (142, 25), (143, 26), (142, 24), (143, 24), (144, 24), (143, 23), (142, 23), (141, 23), (143, 25), (139, 25), (140, 25), (141, 24), (145, 28), (146, 25), (146, 23), (147, 22), (148, 22), (148, 23), (147, 23), (146, 22), (145, 21), (146, 21), (145, 22), (144, 23), (143, 22), (144, 21), (143, 21), (144, 20), (148, 24), (141, 22), (145, 20), (140, 24), (146, 17), (148, 13), (147, 17), (150, 15), (151, 14), (150, 13), (149, 12), (148, 12), (147, 11), (145, 10), (146, 10), (148, 10), (147, 9), (146, 8), (146, 7), (147, 7), (146, 6), (147, 8), (148, 8), (145, 6), (146, 5), (145, 5), (144, 6), (145, 7), (145, 8), (144, 8), (148, 9), (146, 9), (144, 7), (143, 5), (143, 6), (144, 5), (143, 7), (143, 8), (147, 6), (148, 6), (148, 7), (144, 9), (149, 11), (145, 11), (144, 12), (144, 11), (143, 11), (145, 9), (147, 10), (144, 10), (150, 11), (148, 11), (149, 9), (151, 8), (150, 10), (152, 11), (149, 10), (150, 12), (152, 15), (153, 15), (152, 16), (153, 17), (152, 18), (153, 19), (154, 20), (154, 21), (155, 22), (155, 21), (156, 21), (157, 23), (154, 22), (154, 23), (153, 24), (154, 24), (155, 25), (159, 23), (160, 24), (161, 21), (155, 20), (155, 19), (154, 19), (153, 18), (152, 19), (152, 20), (152, 21), (153, 21), (151, 21), (153, 20), (153, 22), (154, 25), (153, 26), (151, 20), (150, 21), (158, 23), (159, 22), (160, 22), (158, 24), (171, 24), (175, 21), (175, 20), (175, 19), (175, 18), (176, 19), (176, 20), (176, 18), (177, 18), (176, 17), (177, 16), (178, 16), (179, 17), (180, 18), (181, 19), (180, 19), (179, 20), (180, 20), (181, 21), (181, 22), (180, 22), (179, 23), (179, 22), (181, 20), (182, 20), (183, 19), (183, 20), (182, 21), (183, 21), (184, 21), (184, 22), (183, 22), (182, 23), (181, 23), (181, 24), (182, 22), (185, 21), (184, 23), (185, 24), (184, 24), (185, 25), (184, 26), (185, 27), (185, 26), (184, 25), (186, 24), (186, 23), (185, 23), (183, 23), (183, 24), (182, 25), (181, 25), (180, 26), (183, 28), (183, 27), (184, 27), (182, 27), (182, 26), (183, 26), (183, 25), (182, 24), (180, 21), (179, 19), (178, 20), (177, 19), (178, 19), (179, 21), (178, 21), (177, 20), (177, 21), (177, 22), (181, 18), (182, 17), (182, 18), (182, 16), (181, 15), (182, 15), (181, 14), (180, 14), (179, 13), (179, 14), (178, 15), (179, 15), (180, 15), (180, 16), (181, 16), (183, 17), (184, 16), (185, 17), (185, 18), (184, 17), (184, 18), (185, 16), (184, 15), (185, 14), (186, 14), (186, 13), (185, 12), (187, 14), (188, 14), (187, 13), (188, 12), (188, 11), (189, 10), (187, 10), (187, 9), (188, 8), (189, 7), (188, 6), (189, 6), (188, 7), (187, 6), (187, 5), (188, 5), (189, 8), (186, 6), (187, 7), (189, 9), (189, 11), (190, 11), (190, 12), (191, 12), (190, 13), (189, 13), (188, 13), (189, 12), (187, 12), (186, 8), (185, 7), (186, 7), (185, 6), (185, 9), (186, 10), (185, 10), (186, 11), (186, 9), (187, 11), (186, 12), (187, 15), (188, 15), (186, 15), (187, 16), (188, 17), (187, 18), (186, 19), (186, 18), (185, 19), (184, 19), (183, 18), (183, 15), (178, 18), (180, 24), (177, 23), (176, 22), (176, 23), (177, 26), (180, 25), (179, 24), (178, 27), (177, 31), (171, 35), (171, 36), (171, 37), (173, 39), (168, 31), (159, 31), (152, 24), (151, 24), (150, 24), (151, 25), (152, 23), (152, 25), (151, 23), (150, 22), (149, 21), (148, 21), (149, 22), (150, 23), (149, 27), (150, 28), (149, 30), (150, 32), (150, 31), (153, 33), (152, 33), (154, 34), (155, 36), (153, 39), (154, 37), (155, 37), (156, 35), (157, 36), (159, 35), (155, 31), (154, 27), (164, 17), (164, 18), (165, 18), (160, 17), (161, 16), (161, 15), (164, 22), (180, 37), (181, 38), (180, 38), (178, 37), (183, 37), (184, 35), (185, 37), (184, 37), (185, 38), (186, 38), (187, 38), (178, 34), (182, 28), (181, 26), (181, 27), (177, 24), (176, 21), (173, 19), (174, 18), (175, 17), (174, 16), (174, 15), (175, 16), (175, 15), (176, 15), (177, 15), (177, 17), (175, 14), (176, 16), (173, 15), (172, 16), (174, 17), (173, 18), (178, 22), (172, 27), (176, 25), (180, 23), (177, 32), (183, 29), (185, 28), (186, 27), (186, 26), (187, 27), (186, 29), (186, 28), (187, 28), (185, 30), (186, 30), (186, 25), (187, 25), (188, 25), (189, 25), (188, 24), (187, 23), (186, 22), (186, 21), (187, 22), (188, 21), (188, 23), (185, 15), (186, 16), (183, 14), (184, 14), (186, 17), (187, 19), (186, 20), (188, 18), (189, 18), (190, 19), (191, 18), (192, 19), (190, 17), (189, 17), (190, 18), (191, 17), (189, 16), (188, 16), (187, 17), (185, 13), (184, 12), (183, 12), (183, 11), (182, 12), (184, 11), (185, 11), (184, 10), (182, 13), (181, 12), (181, 13), (183, 13), (180, 12), (180, 13), (183, 16), (181, 17), (180, 17), (179, 16), (182, 14), (190, 16), (188, 19), (189, 19), (191, 16), (190, 15), (191, 14), (191, 13), (188, 10), (190, 8), (191, 7), (191, 6), (190, 6), (190, 5), (191, 5), (192, 5), (193, 5), (192, 6), (193, 6), (193, 7), (192, 8), (192, 9), (193, 9), (194, 9), (195, 9), (194, 10), (193, 11), (195, 8), (196, 8), (197, 8), (196, 9), (196, 10), (197, 9), (198, 9), (199, 8), (200, 7), (199, 6), (198, 6), (198, 7), (198, 8), (199, 9), (200, 10), (201, 9), (201, 8), (201, 7), (202, 7), (202, 9), (203, 8), (203, 7), (203, 6), (204, 7), (204, 9), (205, 10), (204, 10), (206, 9), (207, 9), (207, 10), (208, 9), (207, 8), (207, 7), (206, 6), (207, 5), (205, 6), (205, 5), (204, 5), (204, 6), (206, 7), (206, 5), (208, 6), (206, 8), (208, 8), (209, 7), (210, 7), (209, 8), (208, 7), (209, 9), (207, 11), (208, 12), (207, 13), (208, 13), (208, 14), (207, 14), (206, 14), (207, 15), (208, 15), (208, 16), (209, 15), (210, 15), (211, 14), (212, 14), (213, 13), (213, 14), (212, 13), (211, 12), (211, 13), (211, 11), (212, 11), (212, 12), (213, 11), (213, 12), (214, 12), (211, 10), (211, 9), (212, 10), (213, 9), (214, 8), (214, 7), (213, 7), (214, 6), (215, 7), (216, 6), (215, 5), (213, 5), (212, 6), (213, 6), (214, 5), (212, 5), (211, 5), (211, 6), (210, 6), (211, 7), (210, 5), (212, 7), (212, 8), (210, 8), (211, 8), (212, 9), (210, 9), (209, 10), (208, 10), (209, 11), (208, 11), (209, 12), (210, 12), (210, 13), (210, 14), (209, 16), (209, 17), (208, 18), (207, 16), (206, 16), (207, 17), (206, 17), (207, 18), (206, 18), (207, 19), (208, 20), (209, 20), (208, 21), (209, 22), (209, 21), (210, 21), (211, 21), (210, 20), (210, 19), (211, 18), (211, 17), (212, 18), (212, 19), (212, 17), (210, 17), (213, 20), (213, 17), (214, 16), (215, 17), (216, 16), (216, 17), (217, 17), (216, 18), (212, 16), (213, 15), (214, 17), (213, 16), (211, 15), (210, 16), (209, 13), (209, 14), (208, 17), (209, 18), (209, 19), (208, 19), (205, 15), (206, 15), (205, 16), (204, 16), (205, 14), (205, 17), (204, 14), (203, 14), (203, 15), (202, 15), (202, 16), (202, 17), (203, 17), (204, 18), (205, 19), (206, 20), (207, 20), (210, 18), (211, 19), (211, 20), (207, 21), (207, 22), (206, 22), (207, 23), (206, 21), (205, 22), (204, 23), (204, 22), (203, 21), (203, 22), (205, 21), (205, 23), (204, 24), (205, 24), (206, 23), (206, 24), (206, 25), (206, 26), (206, 27), (207, 28), (208, 27), (207, 27), (207, 29), (206, 28), (205, 28), (204, 29), (204, 28), (203, 27), (202, 27), (201, 28), (202, 29), (203, 29), (204, 27), (205, 26), (205, 25), (204, 25), (203, 26), (202, 28), (201, 29), (203, 28), (205, 27), (204, 26), (203, 25), (207, 24), (207, 25), (208, 24), (208, 26), (209, 26), (210, 27), (210, 26), (209, 25), (208, 22), (206, 19), (205, 18), (204, 17), (203, 18), (204, 19), (202, 18), (201, 19), (202, 19), (202, 20), (201, 21), (201, 22), (202, 21), (201, 20), (202, 23), (203, 23), (202, 24), (201, 24), (202, 25), (200, 25), (199, 26), (200, 26), (198, 26), (197, 26), (198, 27), (199, 27), (200, 27), (201, 27), (200, 28), (200, 30), (200, 29), (201, 26), (201, 25), (203, 24), (202, 22), (204, 21), (206, 29), (202, 26), (205, 29), (204, 30), (203, 31), (203, 32), (202, 32), (202, 31), (201, 30), (201, 31), (199, 28), (199, 29), (198, 29), (197, 28), (197, 29), (197, 30), (198, 28), (202, 30), (203, 30), (205, 30), (208, 28), (209, 29), (210, 30), (211, 30), (211, 29), (210, 29), (211, 31), (210, 32), (209, 33), (208, 34), (207, 33), (207, 32), (208, 33), (209, 32), (210, 34), (211, 35), (210, 36), (209, 36), (210, 37), (211, 37), (212, 38), (211, 38), (212, 39), (211, 40), (211, 39), (210, 40), (209, 41), (210, 41), (211, 42), (212, 41), (209, 42), (210, 42), (211, 43), (211, 41), (210, 39), (208, 43), (209, 43), (208, 44), (207, 43), (206, 43), (206, 42), (206, 41), (205, 42), (207, 44), (206, 44), (205, 43), (205, 44), (205, 45), (205, 46), (204, 44), (203, 43), (203, 44), (202, 45), (204, 43), (204, 42), (203, 41), (203, 42), (202, 41), (203, 40), (204, 41), (202, 44), (204, 45), (206, 46), (204, 47), (205, 47), (206, 48), (207, 47), (206, 47), (205, 49), (206, 50), (206, 49), (207, 50), (208, 51), (208, 50), (209, 50), (209, 49), (208, 49), (210, 51), (211, 52), (211, 53), (211, 54), (212, 53), (213, 54), (210, 52), (210, 54), (209, 55), (210, 55), (212, 54), (211, 56), (212, 57), (213, 58), (213, 59), (212, 60), (213, 60), (212, 59), (212, 58), (212, 56), (213, 55), (214, 54), (214, 55), (215, 54), (214, 56), (215, 55), (215, 56), (214, 57), (216, 55), (217, 54), (218, 53), (216, 53), (217, 53), (218, 52), (218, 51), (217, 51), (217, 52), (216, 51), (216, 52), (216, 50), (216, 49), (217, 48), (218, 47), (217, 47), (216, 46), (217, 46), (215, 45), (216, 45), (216, 44), (215, 43), (214, 42), (214, 41), (215, 41), (215, 40), (214, 40), (215, 39), (215, 38), (216, 38), (216, 39), (214, 39), (214, 38), (216, 40), (217, 40), (216, 37), (215, 36), (216, 36), (217, 35), (217, 34), (216, 35), (217, 36), (217, 37), (216, 41), (213, 40), (213, 39), (213, 42), (213, 43), (212, 43), (214, 44), (213, 44), (212, 42), (212, 44), (211, 44), (213, 41), (212, 40), (210, 38), (209, 37), (211, 36), (212, 36), (209, 39), (209, 40), (208, 42), (207, 42), (208, 41), (207, 41), (207, 40), (208, 39), (208, 40), (209, 38), (213, 37), (214, 36), (213, 36), (213, 35), (212, 35), (212, 34), (211, 34), (212, 37), (213, 38), (215, 42), (216, 42), (216, 43), (215, 44), (214, 45), (214, 43), (213, 45), (215, 37), (214, 37), (215, 35), (214, 34), (213, 33), (210, 33), (209, 34), (208, 35), (208, 37), (208, 36), (205, 40), (206, 40), (205, 41), (210, 43), (208, 38), (207, 38), (206, 39), (205, 39), (204, 40), (204, 38), (204, 37), (203, 38), (202, 37), (203, 39), (202, 39), (202, 40), (201, 39), (200, 38), (199, 39), (199, 38), (204, 39), (205, 37), (204, 36), (204, 35), (205, 34), (206, 35), (206, 33), (208, 32), (207, 31), (208, 31), (207, 30), (208, 30), (209, 30), (209, 31), (206, 31), (206, 30), (198, 30), (199, 31), (198, 31), (197, 32), (198, 32), (197, 31), (196, 30), (196, 29), (195, 30), (196, 31), (200, 31), (207, 26), (200, 32), (200, 33), (201, 33), (204, 31), (205, 32), (206, 32), (210, 31), (211, 32), (212, 32), (211, 33), (207, 35), (206, 34), (207, 34), (205, 33), (204, 32), (204, 33), (203, 34), (203, 33), (202, 33), (201, 34), (202, 34), (202, 35), (203, 36), (205, 36), (205, 35), (206, 36), (206, 38), (207, 39), (205, 38), (206, 37), (217, 41), (217, 39), (218, 40), (217, 38), (218, 38), (218, 37), (215, 34), (214, 35), (214, 33), (213, 32), (212, 33), (212, 31), (213, 31), (214, 32), (215, 33), (215, 32), (213, 34), (216, 34), (218, 35), (219, 35), (218, 34), (217, 33), (216, 33), (209, 35), (209, 28), (210, 28), (211, 28), (212, 29), (213, 29), (213, 28), (214, 27), (213, 27), (212, 27), (212, 28), (212, 30), (213, 30), (214, 30), (214, 29), (215, 29), (215, 28), (211, 27), (211, 26), (212, 26), (213, 25), (214, 24), (213, 23), (213, 22), (214, 21), (215, 21), (216, 22), (216, 23), (217, 23), (217, 24), (216, 25), (216, 26), (215, 26), (214, 25), (213, 26), (215, 27), (216, 28), (203, 37), (207, 37), (208, 45), (208, 46), (209, 47), (210, 46), (209, 45), (209, 46), (210, 45), (209, 44), (210, 44), (208, 47), (209, 48), (210, 47), (208, 48), (207, 48), (207, 49), (207, 46), (210, 49), (211, 48), (210, 48), (207, 51), (208, 52), (209, 52), (209, 51), (210, 50), (211, 49), (212, 50), (211, 51), (213, 50), (214, 49), (215, 49), (214, 48), (215, 47), (216, 48), (215, 48), (215, 50), (215, 51), (214, 51), (214, 50), (214, 52), (214, 53), (213, 52), (213, 51), (217, 49), (217, 50), (218, 50), (219, 49), (219, 48), (219, 47), (218, 49), (215, 52), (214, 47), (213, 46), (212, 45), (212, 46), (211, 47), (212, 49), (213, 49), (213, 48), (211, 50), (212, 51), (213, 53), (211, 55), (210, 56), (210, 53), (212, 52), (209, 54), (208, 54), (209, 53), (212, 55), (216, 54), (218, 54), (219, 52), (220, 53), (221, 53), (220, 54), (221, 54), (219, 54), (220, 55), (213, 47), (212, 48), (211, 45), (217, 43), (217, 42), (218, 41), (217, 45), (218, 44), (218, 43), (218, 42), (219, 41), (220, 41), (221, 40), (221, 39), (222, 41), (219, 42), (220, 40), (219, 40), (220, 39), (214, 31), (215, 30), (215, 31), (216, 31), (216, 32), (216, 30), (216, 29), (217, 29), (217, 30), (218, 39), (219, 38), (220, 37), (219, 39), (218, 36), (219, 36), (219, 37), (217, 32), (218, 31), (217, 31), (219, 31), (220, 30), (219, 32), (220, 33), (220, 32), (220, 31), (221, 31), (222, 31), (223, 31), (224, 31), (224, 32), (225, 32), (223, 33), (224, 34), (223, 35), (222, 35), (223, 36), (223, 37), (222, 36), (221, 37), (220, 38), (222, 38), (222, 39), (223, 40), (223, 42), (224, 41), (224, 42), (223, 41), (224, 40), (222, 40), (223, 39), (224, 39), (225, 39), (225, 40), (226, 39), (227, 38), (226, 38), (228, 39), (228, 40), (229, 39), (227, 39), (227, 37), (226, 36), (225, 37), (225, 36), (226, 35), (227, 34), (224, 36), (225, 35), (225, 34), (227, 35), (228, 36), (229, 37), (230, 38), (229, 38), (230, 39), (231, 40), (231, 37), (231, 36), (230, 35), (229, 35), (229, 36), (228, 37), (226, 37), (227, 36), (226, 34), (226, 33), (227, 33), (227, 32), (228, 31), (227, 31), (226, 30), (225, 30), (224, 30), (224, 29), (225, 31), (226, 31), (227, 30), (226, 29), (225, 28), (225, 29), (226, 28), (227, 27), (228, 26), (227, 26), (228, 27), (229, 26), (230, 25), (231, 24), (229, 24), (229, 23), (228, 22), (229, 22), (228, 23), (228, 24), (228, 25), (226, 26), (226, 25), (225, 26), (226, 27), (225, 27), (224, 26), (225, 25), (224, 27), (223, 26), (224, 25), (223, 25), (222, 26), (221, 26), (220, 26), (221, 27), (220, 28), (220, 29), (221, 29), (222, 29), (223, 29), (223, 30), (223, 32), (222, 32), (221, 33), (222, 33), (223, 34), (221, 34), (221, 35), (222, 34), (219, 33), (218, 32), (220, 35), (219, 34), (220, 36), (211, 46), (207, 36), (201, 42), (202, 43), (201, 44), (201, 45), (201, 46), (200, 46), (202, 47), (203, 47), (202, 48), (202, 46), (203, 45), (204, 46), (203, 48), (204, 48), (204, 50), (205, 51), (205, 52), (204, 51), (203, 50), (204, 49), (202, 49), (205, 50), (207, 45), (206, 45), (202, 38), (202, 36), (203, 35), (204, 34), (201, 40), (201, 41), (201, 43), (202, 42), (200, 42), (199, 41), (199, 40), (199, 42), (198, 41), (197, 40), (198, 39), (197, 39), (196, 38), (197, 37), (198, 36), (197, 36), (198, 37), (195, 37), (196, 37), (195, 38), (196, 39), (195, 39), (194, 39), (194, 38), (193, 38), (194, 37), (193, 36), (192, 36), (192, 35), (192, 34), (193, 33), (193, 32), (194, 31), (194, 30), (195, 29), (195, 32), (195, 31), (194, 29), (193, 29), (194, 28), (195, 28), (196, 28), (197, 25), (196, 25), (196, 24), (197, 24), (198, 25), (198, 24), (199, 25), (197, 27), (199, 24), (200, 24), (201, 23), (208, 23), (209, 23), (210, 23), (210, 22), (205, 20), (204, 20), (211, 16), (212, 15), (214, 18), (215, 19), (214, 20), (215, 20), (216, 19), (215, 18), (216, 21), (215, 22), (214, 23), (214, 22), (214, 19), (213, 19), (207, 12), (206, 13), (205, 13), (201, 15), (203, 16), (201, 16), (200, 17), (199, 17), (198, 17), (197, 16), (196, 16), (195, 15), (196, 15), (195, 16), (194, 14), (193, 14), (192, 15), (191, 15), (192, 16), (192, 17), (192, 18), (193, 18), (189, 15), (193, 15), (193, 16), (194, 17), (194, 16), (194, 15), (196, 17), (197, 15), (198, 15), (197, 14), (198, 14), (199, 13), (200, 12), (200, 14), (200, 15), (199, 15), (199, 14), (199, 16), (198, 18), (200, 18), (201, 17), (200, 16), (201, 14), (198, 16), (196, 14), (197, 13), (196, 12), (196, 11), (195, 11), (194, 12), (194, 13), (195, 13), (195, 12), (196, 13), (195, 14), (193, 13), (192, 13), (192, 12), (192, 11), (192, 10), (194, 11), (193, 10), (194, 8), (194, 6), (194, 7), (195, 6), (194, 5), (197, 7), (197, 6), (196, 5), (196, 6), (195, 10), (197, 10), (193, 12), (191, 9), (191, 10), (191, 11), (197, 12), (198, 12), (202, 14), (204, 13), (215, 8), (215, 6), (216, 5), (217, 6), (217, 5), (218, 6), (218, 5), (219, 5), (217, 7), (217, 8), (216, 9), (216, 10), (215, 9), (216, 8), (218, 9), (218, 8), (216, 7), (213, 8), (214, 9), (215, 10), (210, 10), (210, 11), (213, 10), (214, 11), (214, 13), (215, 12), (216, 13), (217, 13), (218, 14), (219, 13), (218, 12), (218, 11), (219, 10), (217, 9), (217, 10), (215, 11), (218, 10), (219, 11), (220, 11), (221, 12), (221, 13), (222, 13), (220, 13), (219, 14), (220, 12), (219, 12), (214, 10), (217, 11), (217, 12), (216, 11), (215, 14), (214, 15), (206, 12), (205, 12), (204, 11), (205, 11), (203, 11), (202, 11), (201, 11), (203, 10), (203, 9), (204, 8), (205, 9), (206, 10), (214, 14), (205, 8), (202, 6), (201, 6), (202, 5), (202, 8), (202, 10), (203, 5), (204, 12), (202, 12), (201, 13), (203, 13), (204, 15), (215, 15), (215, 16), (213, 18), (203, 20), (201, 18), (200, 19), (199, 20), (199, 21), (200, 20), (199, 19), (200, 21), (200, 22), (199, 23), (199, 22), (203, 19), (198, 23), (197, 23), (197, 22), (198, 22), (199, 18), (197, 18), (202, 13), (201, 12), (203, 12), (206, 11), (216, 12), (215, 13), (209, 6), (207, 6), (205, 7), (201, 10), (199, 10), (200, 9), (198, 11), (199, 12), (197, 11), (198, 13), (198, 10), (199, 11), (199, 7), (192, 14), (190, 14), (193, 17), (190, 20), (191, 21), (192, 22), (193, 23), (193, 24), (194, 24), (195, 25), (194, 26), (193, 27), (194, 27), (195, 26), (193, 28), (192, 27), (193, 26), (192, 26), (193, 25), (191, 25), (191, 24), (190, 23), (191, 23), (191, 22), (192, 21), (192, 23), (194, 23), (195, 22), (195, 21), (194, 21), (194, 20), (195, 20), (195, 19), (196, 18), (198, 19), (198, 20), (197, 19), (198, 21), (196, 20), (196, 19), (197, 17), (195, 18), (194, 18), (195, 17), (200, 13), (200, 8), (200, 6), (198, 5), (197, 5), (199, 5), (201, 5), (200, 5), (209, 24), (210, 24), (210, 25), (211, 25), (208, 25), (211, 23), (211, 22), (211, 24), (209, 27), (212, 24), (212, 20), (212, 22), (213, 21), (216, 24), (217, 22), (217, 21), (218, 22), (215, 24), (215, 23), (217, 18), (218, 17), (219, 17), (218, 16), (219, 18), (219, 19), (218, 20), (217, 19), (216, 20), (217, 20), (212, 21), (218, 24), (219, 24), (219, 25), (218, 25), (219, 26), (218, 27), (218, 28), (217, 26), (217, 27), (216, 27), (215, 25), (214, 26), (214, 28), (213, 24), (212, 25), (212, 23), (197, 20), (197, 21), (196, 22), (196, 23), (195, 23), (195, 24), (196, 26), (196, 27), (195, 27), (200, 23), (193, 21), (193, 22), (192, 24), (192, 25), (192, 28), (191, 28), (192, 29), (191, 30), (192, 30), (191, 29), (190, 30), (190, 28), (189, 27), (188, 26), (187, 26), (188, 27), (188, 28), (189, 28), (189, 29), (193, 30), (192, 31), (192, 33), (194, 32), (196, 32), (193, 31), (193, 34), (194, 33), (192, 37), (193, 37), (192, 39), (193, 39), (193, 40), (194, 40), (195, 40), (194, 41), (192, 41), (193, 41), (195, 41), (195, 42), (195, 43), (196, 42), (197, 41), (196, 40), (196, 41), (194, 42), (193, 43), (192, 43), (191, 43), (191, 44), (192, 45), (191, 45), (190, 44), (190, 45), (189, 44), (188, 45), (187, 45), (188, 46), (187, 44), (188, 44), (186, 45), (185, 44), (185, 45), (186, 46), (189, 47), (188, 48), (187, 47), (186, 47), (187, 46), (189, 46), (189, 45), (190, 47), (191, 47), (192, 47), (193, 46), (193, 47), (194, 48), (195, 49), (194, 49), (194, 50), (194, 51), (193, 50), (194, 47), (195, 48), (196, 47), (197, 47), (197, 48), (198, 47), (199, 47), (200, 48), (201, 48), (201, 49), (200, 50), (199, 50), (198, 49), (199, 49), (198, 48), (197, 46), (197, 45), (196, 46), (195, 45), (194, 44), (195, 44), (196, 43), (197, 43), (197, 44), (196, 44), (196, 45), (198, 43), (198, 46), (199, 48), (200, 45), (199, 46), (195, 46), (193, 48), (193, 49), (194, 46), (194, 45), (193, 45), (193, 44), (192, 44), (187, 43), (188, 43), (189, 43), (188, 42), (186, 42), (186, 43), (187, 48), (186, 49), (185, 50), (186, 50), (187, 50), (187, 51), (187, 52), (188, 52), (188, 51), (185, 49), (187, 49), (188, 50), (189, 49), (190, 48), (190, 50), (190, 51), (189, 52), (190, 52), (191, 53), (192, 52), (192, 53), (191, 54), (192, 54), (190, 54), (189, 53), (189, 51), (189, 50), (190, 53), (188, 53), (186, 51), (184, 50), (184, 49), (184, 48), (183, 48), (182, 49), (181, 50), (182, 51), (182, 50), (181, 51), (180, 52), (181, 53), (179, 52), (180, 51), (181, 52), (182, 53), (181, 54), (180, 55), (179, 55), (179, 56), (179, 57), (178, 57), (177, 57), (176, 56), (176, 55), (177, 56), (176, 57), (178, 56), (175, 57), (174, 57), (175, 58), (176, 58), (175, 56), (177, 59), (178, 60), (178, 59), (178, 58), (179, 59), (180, 58), (181, 57), (182, 56), (183, 56), (182, 57), (183, 58), (183, 57), (183, 55), (182, 55), (181, 58), (180, 57), (180, 56), (181, 56), (184, 56), (184, 57), (184, 58), (185, 57), (185, 56), (186, 57), (187, 56), (186, 55), (186, 56), (187, 57), (188, 56), (189, 55), (188, 55), (187, 55), (188, 54), (189, 54), (188, 57), (185, 55), (181, 55), (183, 52), (184, 52), (180, 54), (179, 58), (180, 59), (175, 55), (177, 55), (178, 55), (182, 58), (184, 59), (183, 60), (184, 60), (183, 61), (183, 62), (182, 63), (181, 63), (182, 64), (182, 62), (182, 61), (181, 60), (182, 60), (183, 59), (185, 59), (187, 58), (188, 59), (187, 60), (186, 60), (187, 59), (188, 60), (189, 61), (188, 62), (188, 61), (187, 62), (188, 63), (187, 64), (186, 64), (187, 65), (186, 65), (187, 63), (190, 60), (190, 61), (191, 60), (192, 61), (191, 62), (190, 63), (189, 64), (189, 63), (189, 62), (190, 62), (185, 66), (185, 67), (184, 67), (186, 66), (186, 67), (187, 66), (188, 67), (188, 68), (188, 69), (189, 68), (189, 69), (187, 68), (188, 66), (189, 66), (188, 65), (188, 64), (185, 68), (184, 68), (185, 69), (184, 69), (183, 68), (182, 68), (181, 69), (182, 70), (182, 71), (183, 71), (183, 70), (182, 69), (181, 70), (181, 71), (182, 72), (184, 71), (184, 70), (184, 72), (185, 71), (186, 70), (187, 70), (186, 71), (188, 71), (188, 70), (187, 71), (186, 69), (185, 70), (186, 72), (187, 73), (188, 72), (189, 72), (189, 73), (190, 73), (191, 73), (192, 74), (191, 75), (190, 74), (189, 74), (188, 73), (189, 71), (190, 71), (189, 70), (190, 69), (187, 69), (186, 68), (183, 69), (180, 69), (181, 68), (179, 68), (179, 69), (178, 70), (177, 69), (178, 68), (178, 69), (180, 68), (187, 72), (190, 70), (191, 72), (192, 72), (192, 73), (191, 74), (192, 76), (193, 76), (194, 75), (195, 74), (194, 73), (195, 73), (196, 72), (195, 71), (196, 71), (195, 72), (194, 72), (196, 73), (194, 76), (193, 77), (192, 75), (194, 78), (195, 78), (196, 77), (195, 76), (195, 77), (196, 76), (194, 79), (193, 78), (194, 77), (193, 75), (194, 74), (195, 75), (197, 76), (198, 77), (199, 77), (199, 76), (200, 78), (198, 76), (198, 75), (197, 77), (198, 78), (197, 75), (197, 74), (196, 74), (196, 75), (193, 74), (193, 73), (193, 72), (194, 71), (194, 70), (193, 71), (193, 70), (194, 69), (192, 71), (191, 76), (192, 77), (190, 76), (191, 77), (190, 77), (190, 75), (190, 72), (191, 71), (192, 70), (191, 70), (186, 73), (187, 74), (186, 74), (187, 75), (188, 75), (187, 76), (188, 77), (189, 77), (189, 76), (189, 78), (188, 78), (188, 79), (188, 80), (189, 81), (190, 80), (191, 79), (192, 80), (191, 80), (190, 79), (189, 79), (187, 81), (189, 80), (190, 81), (191, 81), (192, 82), (193, 82), (191, 83), (193, 83), (194, 82), (195, 81), (194, 81), (195, 83), (196, 83), (197, 83), (197, 82), (198, 81), (198, 83), (196, 82), (195, 82), (192, 81), (193, 81), (194, 80), (194, 83), (190, 78), (187, 78), (186, 78), (185, 79), (186, 79), (187, 79), (187, 80), (191, 78), (192, 78), (193, 79), (192, 79), (188, 74), (186, 76), (186, 75), (189, 75), (188, 76), (185, 76), (186, 77), (193, 80), (200, 76), (201, 77), (200, 77), (201, 76), (202, 77), (200, 75), (199, 74), (200, 73), (200, 72), (201, 74), (199, 72), (199, 71), (198, 72), (197, 71), (198, 71), (197, 70), (198, 70), (197, 69), (196, 70), (195, 70), (192, 69), (191, 68), (190, 67), (189, 67), (187, 67), (185, 64), (185, 63), (186, 63), (185, 62), (190, 64), (189, 65), (190, 66), (191, 65), (190, 65), (189, 60), (189, 59), (192, 59), (193, 59), (194, 58), (195, 57), (194, 57), (194, 56), (193, 56), (192, 55), (193, 57), (192, 56), (191, 57), (190, 56), (191, 55), (193, 55), (193, 58), (192, 57), (195, 56), (194, 55), (194, 54), (195, 55), (195, 53), (196, 52), (197, 51), (196, 50), (197, 49), (197, 50), (198, 50), (198, 51), (199, 51), (196, 49), (195, 51), (196, 51), (195, 50), (192, 49), (195, 47), (192, 48), (192, 46), (191, 46), (190, 42), (190, 43), (189, 42), (188, 41), (189, 40), (190, 39), (191, 40), (190, 41), (189, 41), (191, 42), (190, 40), (188, 40), (189, 39), (192, 42), (191, 41), (190, 46), (190, 49), (191, 48), (198, 45), (199, 45), (201, 38), (200, 37), (199, 36), (198, 38), (197, 38), (199, 37), (200, 36), (201, 37), (200, 43), (199, 43), (199, 44), (200, 44), (210, 35), (200, 47), (201, 47), (205, 48), (217, 44), (219, 43), (219, 44), (219, 45), (220, 44), (221, 45), (222, 44), (223, 45), (223, 44), (223, 43), (224, 43), (225, 44), (226, 43), (227, 44), (226, 45), (227, 46), (226, 46), (225, 45), (224, 46), (224, 45), (222, 42), (220, 42), (218, 46), (220, 48), (220, 49), (221, 49), (222, 48), (223, 48), (223, 49), (224, 50), (225, 50), (225, 49), (224, 49), (222, 49), (221, 48), (220, 47), (220, 46), (221, 47), (217, 55), (218, 56), (218, 57), (217, 58), (218, 58), (218, 59), (219, 60), (219, 59), (219, 58), (220, 57), (221, 56), (221, 57), (220, 56), (218, 55), (219, 56), (219, 57), (215, 53), (213, 56), (211, 58), (210, 57), (209, 56), (208, 55), (211, 57), (209, 58), (209, 59), (208, 60), (207, 59), (206, 59), (205, 58), (204, 57), (203, 56), (202, 57), (202, 56), (203, 57), (203, 58), (204, 59), (205, 59), (206, 60), (205, 60), (205, 61), (204, 60), (204, 61), (203, 60), (203, 61), (204, 58), (201, 56), (203, 59), (206, 62), (207, 63), (208, 63), (209, 64), (208, 65), (208, 66), (207, 67), (207, 66), (208, 68), (209, 69), (209, 68), (210, 67), (209, 67), (208, 67), (209, 65), (209, 66), (210, 66), (211, 66), (211, 67), (211, 68), (212, 68), (213, 69), (214, 69), (214, 68), (214, 67), (215, 67), (214, 66), (216, 66), (217, 67), (218, 68), (218, 67), (217, 66), (217, 65), (218, 65), (218, 64), (219, 64), (218, 63), (218, 66), (216, 67), (215, 66), (219, 65), (219, 63), (217, 64), (217, 63), (217, 62), (218, 62), (218, 61), (220, 62), (221, 61), (221, 62), (220, 63), (216, 65), (216, 64), (221, 63), (221, 64), (220, 65), (221, 65), (220, 66), (220, 67), (219, 68), (218, 69), (218, 70), (217, 68), (215, 68), (215, 69), (216, 69), (216, 68), (215, 70), (213, 68), (212, 70), (213, 71), (214, 72), (213, 72), (215, 72), (213, 73), (212, 72), (212, 71), (211, 71), (211, 70), (211, 69), (212, 69), (210, 68), (210, 70), (211, 72), (210, 72), (210, 71), (209, 71), (213, 70), (214, 70), (215, 71), (214, 71), (214, 73), (212, 73), (216, 72), (216, 70), (217, 71), (218, 71), (218, 72), (219, 73), (219, 74), (219, 75), (220, 74), (220, 75), (221, 74), (222, 73), (223, 72), (224, 72), (224, 73), (223, 74), (223, 73), (222, 75), (221, 76), (221, 75), (220, 76), (218, 75), (217, 75), (217, 74), (216, 73), (215, 74), (214, 74), (215, 73), (213, 67), (212, 67), (210, 64), (207, 64), (206, 65), (206, 64), (206, 63), (210, 65), (210, 63), (211, 63), (212, 63), (211, 64), (208, 64), (209, 62), (208, 62), (209, 61), (210, 61), (210, 60), (209, 60), (210, 62), (209, 63), (207, 65), (205, 63), (205, 64), (206, 66), (206, 67), (205, 68), (204, 69), (205, 69), (206, 70), (206, 71), (206, 72), (205, 71), (204, 71), (204, 72), (205, 72), (204, 73), (203, 72), (202, 72), (201, 71), (200, 70), (199, 70), (199, 69), (198, 73), (197, 73), (197, 72), (195, 69), (196, 69), (198, 69), (198, 68), (199, 68), (200, 68), (198, 67), (201, 68), (201, 69), (202, 68), (200, 69), (201, 70), (200, 71), (200, 67), (201, 66), (200, 65), (200, 64), (201, 65), (202, 64), (201, 64), (203, 64), (202, 65), (202, 66), (203, 65), (204, 64), (204, 62), (205, 62), (204, 63), (203, 62), (207, 62), (208, 61), (202, 60), (202, 59), (201, 60), (201, 58), (201, 59), (200, 60), (199, 59), (200, 61), (201, 62), (201, 61), (202, 61), (202, 62), (201, 63), (206, 61), (207, 60), (207, 61), (208, 59), (201, 57), (202, 58), (214, 59), (215, 60), (214, 60), (213, 61), (214, 61), (215, 61), (215, 62), (216, 62), (217, 61), (219, 62), (215, 64), (214, 63), (213, 62), (215, 63), (220, 61), (220, 60), (221, 60), (221, 59), (220, 59), (222, 61), (223, 60), (224, 60), (224, 61), (225, 62), (226, 63), (227, 64), (227, 65), (226, 64), (225, 63), (226, 65), (226, 66), (226, 67), (225, 67), (226, 68), (224, 66), (224, 65), (223, 65), (222, 65), (223, 66), (224, 67), (225, 68), (224, 68), (223, 69), (224, 69), (225, 70), (225, 69), (222, 69), (222, 70), (221, 70), (221, 69), (220, 68), (220, 69), (219, 69), (219, 70), (219, 72), (218, 73), (218, 74), (217, 76), (218, 76), (219, 77), (220, 78), (221, 78), (222, 77), (223, 77), (224, 78), (224, 79), (225, 80), (224, 80), (223, 81), (225, 81), (224, 82), (224, 81), (225, 83), (226, 82), (226, 81), (227, 81), (225, 82), (224, 84), (223, 85), (223, 84), (222, 84), (222, 83), (221, 84), (220, 85), (220, 84), (219, 83), (221, 83), (221, 82), (220, 83), (220, 82), (219, 82), (218, 83), (219, 85), (221, 85), (222, 86), (222, 85), (223, 86), (222, 87), (223, 87), (224, 86), (225, 86), (226, 86), (226, 87), (227, 87), (225, 85), (226, 85), (227, 88), (226, 89), (228, 87), (228, 86), (229, 87), (230, 88), (229, 89), (228, 88), (228, 89), (229, 88), (227, 89), (226, 90), (226, 91), (226, 92), (225, 91), (225, 92), (226, 93), (225, 94), (226, 94), (225, 93), (227, 94), (228, 94), (229, 93), (230, 92), (230, 93), (231, 94), (230, 94), (232, 94), (232, 95), (233, 95), (234, 94), (234, 95), (235, 96), (235, 95), (234, 96), (233, 96), (232, 96), (236, 97), (236, 96), (234, 93), (235, 94), (236, 95), (237, 95), (238, 95), (237, 96), (237, 97), (238, 97), (238, 98), (238, 99), (239, 98), (240, 98), (240, 97), (239, 97), (238, 96), (239, 96), (240, 96), (240, 95), (239, 99), (240, 99), (241, 98), (242, 99), (242, 98), (241, 99), (241, 100), (242, 100), (241, 101), (242, 101), (243, 99), (243, 98), (244, 97), (245, 97), (245, 96), (245, 95), (244, 95), (245, 94), (243, 96), (242, 96), (243, 95), (244, 94), (244, 93), (243, 94), (243, 93), (243, 92), (244, 92), (245, 91), (245, 90), (245, 89), (244, 88), (243, 87), (244, 86), (242, 87), (242, 86), (243, 86), (244, 87), (245, 87), (246, 88), (246, 87), (246, 86), (245, 88), (243, 88), (242, 88), (241, 87), (243, 85), (244, 85), (243, 84), (242, 85), (241, 85), (240, 84), (240, 83), (241, 84), (240, 85), (239, 85), (240, 86), (245, 85), (245, 86), (244, 89), (242, 89), (242, 90), (243, 89), (243, 90), (243, 91), (241, 90), (241, 89), (241, 88), (240, 89), (239, 88), (240, 88), (239, 86), (238, 86), (238, 85), (239, 87), (238, 88), (238, 87), (237, 86), (237, 85), (236, 84), (235, 83), (236, 82), (236, 83), (235, 82), (235, 81), (236, 81), (237, 82), (238, 83), (238, 84), (237, 84), (235, 84), (235, 85), (235, 86), (234, 85), (233, 85), (234, 84), (233, 84), (233, 83), (233, 82), (234, 82), (232, 84), (231, 85), (232, 86), (231, 87), (230, 87), (229, 86), (230, 85), (231, 84), (230, 84), (231, 83), (231, 82), (230, 81), (231, 81), (232, 81), (231, 80), (230, 79), (229, 79), (230, 78), (231, 79), (231, 78), (231, 77), (232, 78), (232, 79), (230, 80), (229, 80), (229, 81), (228, 82), (229, 82), (230, 82), (229, 78), (228, 77), (227, 78), (227, 79), (228, 78), (228, 79), (226, 80), (225, 79), (223, 78), (222, 78), (221, 77), (220, 77), (218, 77), (218, 78), (218, 79), (217, 79), (217, 80), (216, 79), (218, 80), (218, 81), (219, 81), (219, 80), (217, 78), (215, 78), (214, 78), (215, 77), (216, 76), (217, 77), (216, 78), (219, 78), (216, 81), (216, 80), (215, 80), (215, 79), (214, 79), (214, 80), (215, 81), (213, 79), (213, 78), (212, 77), (212, 76), (212, 75), (213, 76), (214, 77), (213, 80), (212, 80), (216, 77), (215, 76), (214, 76), (215, 75), (219, 76), (214, 75), (212, 74), (213, 75), (213, 74), (211, 76), (210, 75), (210, 74), (211, 74), (211, 73), (208, 70), (207, 71), (208, 72), (209, 73), (208, 73), (209, 72), (210, 73), (208, 69), (209, 70), (210, 69), (205, 66), (205, 65), (211, 60), (212, 61), (216, 63), (214, 62), (216, 61), (219, 66), (219, 67), (215, 65), (213, 64), (214, 64), (214, 65), (218, 60), (220, 58), (219, 61), (220, 64), (221, 66), (223, 67), (223, 68), (224, 70), (224, 71), (223, 70), (226, 69), (227, 69), (227, 68), (228, 68), (228, 70), (229, 69), (229, 70), (229, 71), (228, 71), (228, 72), (227, 73), (226, 73), (227, 74), (228, 75), (229, 74), (230, 73), (231, 73), (232, 74), (230, 74), (229, 75), (230, 76), (229, 76), (228, 81), (227, 80), (228, 80), (229, 83), (228, 83), (227, 83), (226, 83), (227, 82), (223, 82), (224, 83), (223, 83), (222, 82), (225, 84), (224, 85), (222, 81), (223, 80), (226, 78), (226, 79), (221, 81), (218, 82), (217, 84), (217, 83), (218, 84), (218, 85), (218, 86), (217, 85), (216, 86), (215, 87), (215, 88), (214, 87), (213, 88), (214, 88), (213, 87), (212, 86), (213, 85), (214, 84), (215, 85), (213, 83), (213, 82), (212, 82), (213, 81), (212, 81), (211, 82), (210, 83), (209, 83), (208, 82), (207, 83), (206, 82), (207, 82), (206, 81), (206, 80), (206, 79), (206, 78), (205, 79), (207, 79), (207, 77), (208, 76), (208, 75), (209, 75), (209, 76), (208, 74), (207, 73), (206, 73), (206, 74), (206, 75), (206, 76), (205, 76), (204, 75), (204, 76), (205, 77), (204, 78), (203, 79), (202, 80), (203, 80), (202, 79), (201, 81), (200, 80), (199, 81), (200, 81), (199, 82), (200, 82), (201, 80), (200, 79), (199, 78), (199, 79), (199, 80), (198, 82), (200, 83), (201, 84), (202, 83), (202, 82), (203, 83), (202, 81), (204, 79), (204, 77), (203, 76), (201, 78), (202, 76), (201, 75), (199, 73), (197, 68), (197, 67), (196, 66), (196, 67), (195, 67), (195, 68), (194, 66), (193, 66), (195, 65), (196, 64), (197, 65), (197, 64), (196, 63), (197, 63), (198, 64), (196, 62), (196, 61), (195, 62), (194, 63), (194, 64), (195, 63), (196, 65), (197, 66), (198, 66), (196, 68), (199, 67), (198, 74), (199, 75), (203, 77), (202, 78), (201, 79), (204, 80), (205, 80), (205, 78), (201, 82), (195, 84), (193, 84), (192, 83), (191, 82), (192, 85), (191, 85), (192, 84), (191, 84), (191, 86), (190, 86), (190, 85), (194, 84), (195, 79), (195, 80), (186, 82), (185, 83), (186, 83), (184, 83), (185, 82), (186, 80), (185, 80), (184, 80), (186, 81), (187, 82), (196, 79), (193, 69), (193, 68), (194, 67), (193, 67), (194, 65), (195, 64), (194, 68), (195, 66), (193, 64), (192, 64), (193, 63), (192, 63), (191, 64), (191, 63), (192, 65), (191, 66), (192, 68), (191, 67), (192, 67), (192, 66), (190, 68), (190, 59), (189, 58), (190, 57), (191, 56), (189, 56), (189, 57), (188, 58), (190, 58), (186, 58), (186, 59), (185, 60), (185, 58), (186, 61), (187, 61), (191, 58), (191, 59), (192, 60), (193, 60), (195, 59), (195, 60), (195, 58), (196, 56), (197, 57), (198, 57), (199, 57), (199, 56), (200, 56), (199, 55), (198, 56), (198, 55), (200, 55), (201, 54), (197, 55), (197, 56), (196, 57), (196, 58), (196, 60), (197, 60), (198, 61), (197, 62), (198, 63), (198, 62), (197, 61), (194, 61), (195, 61), (199, 62), (199, 61), (200, 62), (199, 63), (198, 65), (199, 66), (199, 65), (200, 66), (203, 66), (204, 66), (204, 67), (203, 67), (202, 67), (203, 68), (204, 68), (204, 70), (205, 70), (208, 71), (205, 67), (210, 59), (211, 59), (211, 62), (212, 62), (213, 63), (212, 64), (208, 58), (210, 58), (209, 57), (208, 57), (211, 61), (212, 47), (206, 51), (206, 52), (205, 53), (205, 54), (206, 54), (205, 55), (205, 56), (206, 56), (207, 55), (208, 56), (207, 57), (207, 58), (206, 55), (207, 56), (214, 58), (215, 57), (216, 57), (217, 56), (217, 57), (216, 56), (215, 58), (203, 46), (198, 44), (203, 49), (203, 52), (203, 53), (202, 53), (202, 54), (203, 54), (204, 55), (204, 54), (203, 55), (199, 60), (212, 65), (216, 59), (217, 60), (216, 60), (217, 59), (216, 58), (215, 59), (221, 55), (222, 55), (219, 55), (222, 57), (220, 51), (219, 50), (216, 47), (220, 50), (221, 52), (222, 51), (223, 52), (224, 51), (223, 50), (222, 50), (221, 50), (222, 47), (223, 47), (223, 46), (225, 46), (225, 47), (224, 47), (222, 46), (222, 45), (223, 38), (222, 37), (221, 38), (221, 41), (222, 43), (224, 44), (227, 45), (226, 44), (225, 43), (227, 43), (227, 42), (228, 44), (226, 42), (225, 42), (225, 41), (221, 42), (220, 43), (220, 45), (219, 46), (221, 44), (221, 43), (221, 46), (218, 48), (222, 53), (225, 51), (226, 50), (226, 48), (227, 49), (227, 48), (228, 48), (226, 49), (226, 47), (227, 47), (228, 46), (227, 50), (227, 51), (228, 50), (229, 48), (230, 49), (230, 48), (230, 47), (229, 50), (229, 51), (230, 50), (229, 49), (228, 51), (228, 52), (229, 53), (230, 52), (230, 51), (228, 49), (225, 48), (224, 48), (215, 46), (220, 52), (221, 51), (228, 47), (229, 47), (229, 45), (230, 46), (231, 45), (230, 45), (231, 46), (232, 46), (231, 47), (232, 45), (232, 47), (233, 46), (233, 47), (229, 44), (228, 43), (229, 42), (229, 43), (228, 42), (227, 41), (219, 51), (223, 51), (222, 52), (230, 42), (231, 43), (230, 44), (231, 44), (230, 43), (233, 45), (234, 46), (233, 44), (232, 43), (233, 43), (232, 44), (231, 42), (230, 41), (231, 41), (232, 40), (229, 46), (228, 45), (231, 51), (231, 53), (231, 52), (230, 53), (230, 54), (232, 51), (233, 52), (232, 52), (232, 53), (234, 52), (235, 52), (236, 53), (237, 52), (238, 52), (239, 51), (240, 50), (241, 50), (241, 51), (241, 52), (242, 53), (242, 52), (241, 53), (240, 52), (239, 53), (239, 52), (240, 51), (243, 52), (243, 53), (244, 52), (242, 54), (242, 55), (242, 56), (243, 56), (242, 57), (243, 58), (242, 58), (241, 58), (241, 57), (241, 55), (244, 55), (244, 56), (243, 57), (241, 56), (240, 57), (239, 57), (240, 56), (240, 55), (241, 54), (243, 55), (240, 54), (240, 53), (242, 51), (239, 50), (238, 50), (238, 51), (237, 51), (236, 51), (236, 50), (237, 50), (237, 49), (238, 49), (237, 48), (238, 47), (238, 46), (237, 46), (237, 47), (238, 48), (239, 48), (240, 48), (241, 47), (240, 46), (241, 48), (240, 47), (242, 47), (242, 48), (241, 49), (242, 49), (242, 46), (240, 49), (242, 50), (243, 49), (244, 48), (244, 47), (243, 48), (241, 46), (240, 45), (240, 44), (241, 44), (241, 43), (239, 43), (239, 42), (240, 41), (240, 43), (242, 42), (241, 41), (241, 42), (242, 41), (243, 41), (243, 42), (239, 44), (238, 45), (239, 46), (238, 44), (237, 45), (237, 44), (237, 43), (237, 42), (238, 43), (236, 43), (235, 44), (235, 45), (234, 44), (236, 45), (236, 44), (236, 42), (235, 43), (231, 48), (234, 43), (235, 42), (234, 45), (235, 46), (241, 45), (240, 42), (241, 40), (240, 39), (239, 39), (240, 38), (241, 38), (242, 39), (243, 40), (244, 40), (244, 39), (243, 39), (244, 41), (243, 43), (244, 42), (245, 42), (246, 42), (245, 41), (247, 43), (246, 44), (247, 45), (248, 45), (248, 44), (249, 43), (249, 44), (249, 45), (248, 46), (246, 46), (247, 47), (246, 48), (245, 49), (244, 49), (244, 50), (246, 50), (246, 51), (246, 52), (245, 51), (244, 51), (245, 50), (246, 49), (245, 48), (247, 50), (248, 50), (247, 48), (246, 45), (245, 46), (246, 47), (247, 46), (248, 47), (244, 45), (247, 49), (249, 50), (250, 49), (250, 50), (250, 51), (251, 50), (251, 49), (252, 50), (253, 49), (254, 50), (254, 49), (255, 49), (255, 51), (254, 51), (256, 51), (257, 50), (256, 52), (255, 50), (257, 51), (256, 50), (256, 49), (256, 48), (257, 48), (258, 49), (258, 48), (259, 48), (259, 49), (260, 48), (261, 49), (260, 50), (259, 50), (260, 49), (262, 48), (262, 50), (263, 51), (263, 52), (262, 52), (263, 53), (262, 53), (261, 53), (262, 54), (260, 52), (260, 51), (261, 51), (262, 51), (261, 52), (263, 54), (261, 54), (263, 55), (264, 54), (263, 50), (264, 51), (260, 53), (261, 50), (262, 49), (262, 47), (263, 46), (262, 45), (262, 46), (263, 45), (263, 44), (261, 46), (260, 46), (261, 45), (260, 44), (261, 43), (262, 43), (262, 44), (261, 44), (260, 43), (263, 42), (262, 42), (261, 42), (261, 41), (260, 41), (262, 41), (262, 40), (263, 40), (261, 40), (263, 41), (263, 39), (262, 39), (261, 38), (261, 39), (264, 38), (264, 37), (263, 36), (262, 35), (261, 35), (262, 36), (263, 37), (262, 38), (262, 37), (263, 38), (264, 39), (260, 40), (259, 41), (258, 40), (257, 40), (256, 39), (257, 38), (256, 38), (255, 38), (255, 39), (254, 39), (253, 39), (252, 38), (251, 39), (251, 38), (252, 39), (252, 37), (252, 36), (252, 35), (253, 37), (251, 36), (250, 35), (250, 36), (249, 36), (249, 37), (250, 38), (249, 39), (249, 38), (250, 39), (249, 40), (249, 41), (248, 41), (247, 41), (248, 40), (250, 40), (251, 41), (252, 41), (253, 41), (252, 42), (252, 43), (251, 42), (250, 41), (251, 43), (253, 43), (253, 44), (254, 44), (254, 45), (255, 45), (256, 44), (257, 44), (257, 43), (256, 43), (258, 43), (257, 42), (256, 42), (255, 42), (256, 41), (256, 40), (255, 40), (257, 41), (258, 41), (254, 40), (253, 40), (254, 38), (253, 38), (254, 36), (255, 35), (256, 35), (257, 34), (256, 33), (257, 33), (257, 32), (258, 33), (259, 32), (258, 31), (258, 30), (257, 30), (256, 30), (256, 31), (256, 32), (255, 32), (256, 34), (255, 33), (255, 34), (254, 33), (253, 33), (252, 33), (252, 34), (251, 33), (252, 32), (253, 32), (254, 32), (251, 34), (251, 37), (253, 36), (250, 37), (253, 35), (254, 37), (255, 37), (256, 36), (257, 36), (257, 37), (258, 39), (257, 39), (259, 39), (259, 42), (260, 39), (260, 38), (260, 37), (259, 36), (260, 35), (261, 36), (263, 34), (262, 34), (261, 34), (262, 33), (264, 35), (265, 35), (265, 34), (264, 33), (265, 33), (266, 32), (267, 32), (267, 33), (266, 33), (268, 33), (268, 34), (267, 34), (266, 34), (266, 36), (266, 35), (264, 34), (266, 37), (265, 36), (267, 37), (268, 38), (268, 39), (268, 40), (268, 37), (267, 38), (267, 39), (268, 41), (266, 39), (267, 40), (266, 41), (267, 42), (268, 42), (268, 43), (267, 44), (267, 45), (268, 45), (266, 46), (266, 45), (266, 44), (267, 43), (266, 42), (266, 43), (265, 43), (265, 42), (265, 41), (264, 40), (265, 40), (267, 41), (268, 44), (264, 43), (264, 44), (264, 41), (266, 40), (265, 39), (265, 38), (265, 37), (264, 36), (266, 38), (267, 36), (263, 33), (263, 35), (261, 33), (261, 32), (260, 31), (259, 31), (260, 32), (259, 30), (259, 29), (260, 29), (260, 28), (259, 28), (261, 28), (260, 27), (260, 26), (261, 27), (262, 29), (261, 29), (260, 30), (261, 30), (258, 32), (259, 34), (260, 34), (260, 33), (267, 35), (268, 32), (266, 31), (265, 32), (264, 32), (263, 32), (264, 31), (263, 30), (263, 29), (262, 30), (261, 31), (262, 32), (263, 31), (265, 31), (268, 35), (268, 36), (268, 31), (267, 30), (265, 30), (264, 30), (262, 31), (262, 27), (261, 26), (262, 26), (263, 25), (263, 24), (264, 25), (263, 26), (262, 25), (262, 24), (261, 25), (263, 23), (262, 23), (261, 24), (261, 23), (259, 25), (258, 26), (259, 27), (258, 28), (257, 27), (257, 28), (257, 29), (257, 31), (258, 29), (256, 27), (256, 28), (256, 29), (255, 28), (254, 27), (254, 28), (254, 26), (253, 27), (252, 26), (253, 25), (254, 25), (253, 24), (254, 23), (255, 23), (255, 22), (254, 22), (253, 22), (253, 21), (253, 20), (252, 19), (252, 18), (251, 20), (250, 19), (250, 21), (250, 22), (249, 22), (249, 21), (250, 20), (250, 18), (249, 19), (249, 20), (251, 22), (251, 21), (252, 22), (252, 20), (249, 18), (248, 19), (247, 18), (246, 18), (246, 17), (246, 16), (246, 15), (246, 14), (247, 13), (248, 14), (247, 15), (247, 14), (246, 13), (248, 12), (247, 12), (246, 12), (245, 11), (244, 12), (244, 11), (245, 12), (245, 13), (248, 15), (247, 16), (248, 16), (249, 16), (249, 17), (249, 15), (248, 13), (248, 11), (249, 11), (250, 11), (251, 10), (252, 9), (251, 9), (252, 10), (253, 9), (253, 10), (253, 11), (254, 11), (252, 11), (251, 12), (250, 12), (252, 12), (252, 8), (251, 7), (252, 7), (251, 8), (250, 7), (249, 7), (250, 6), (249, 8), (250, 8), (250, 10), (250, 9), (254, 10), (253, 8), (254, 8), (255, 8), (256, 7), (254, 9), (254, 7), (255, 9), (253, 7), (256, 10), (255, 11), (254, 12), (254, 13), (253, 12), (251, 13), (252, 14), (251, 14), (250, 15), (251, 15), (250, 16), (251, 16), (250, 17), (251, 18), (251, 17), (248, 20), (249, 23), (248, 23), (249, 24), (250, 23), (247, 23), (246, 24), (246, 23), (245, 24), (244, 24), (243, 25), (242, 24), (243, 23), (242, 23), (241, 24), (240, 23), (240, 24), (239, 25), (239, 26), (240, 26), (241, 25), (242, 25), (244, 25), (245, 25), (247, 22), (247, 24), (248, 24), (251, 24), (251, 25), (251, 27), (252, 25), (254, 24), (253, 23), (252, 23), (252, 24), (255, 26), (256, 26), (257, 25), (256, 24), (257, 23), (258, 22), (257, 24), (258, 23), (257, 22), (256, 21), (256, 22), (257, 21), (258, 21), (258, 20), (259, 19), (260, 20), (259, 20), (261, 20), (261, 21), (260, 21), (259, 21), (260, 22), (260, 23), (259, 22), (259, 23), (259, 24), (260, 25), (262, 28), (263, 28), (263, 27), (264, 28), (264, 29), (265, 28), (264, 27), (265, 27), (266, 27), (265, 29), (259, 35), (258, 34), (259, 33), (257, 35), (255, 36), (256, 37), (258, 38), (259, 38), (258, 37), (254, 35), (254, 34), (253, 34), (250, 33), (250, 34), (251, 35), (264, 26), (265, 25), (264, 24), (264, 23), (264, 22), (262, 22), (263, 22), (265, 21), (264, 21), (264, 20), (265, 22), (263, 19), (263, 21), (266, 23), (266, 24), (267, 25), (268, 26), (267, 26), (268, 27), (267, 28), (268, 28), (268, 29), (267, 29), (266, 28), (266, 29), (268, 30), (267, 27), (266, 30), (266, 25), (266, 26), (265, 24), (267, 24), (267, 23), (267, 22), (266, 22), (258, 27), (258, 24), (260, 24), (258, 25), (255, 25), (255, 27), (253, 26), (252, 27), (253, 28), (254, 29), (255, 29), (255, 30), (254, 31), (258, 36), (255, 31), (252, 31), (251, 30), (251, 31), (252, 30), (253, 30), (251, 32), (250, 31), (250, 32), (249, 33), (249, 34), (248, 38), (248, 37), (248, 39), (247, 42), (248, 42), (249, 42), (248, 43), (250, 43), (250, 42), (251, 44), (252, 45), (252, 44), (253, 45), (253, 46), (253, 47), (254, 47), (255, 47), (254, 48), (254, 46), (253, 48), (257, 52), (258, 51), (258, 50), (257, 49), (257, 47), (254, 52), (253, 50), (253, 52), (252, 53), (251, 54), (250, 55), (250, 53), (251, 52), (252, 51), (253, 51), (252, 52), (251, 51), (249, 49), (250, 48), (249, 48), (249, 51), (251, 48), (251, 47), (249, 47), (248, 49), (245, 45), (244, 44), (245, 43), (244, 46), (246, 43), (247, 44), (251, 40), (247, 36), (246, 36), (246, 37), (245, 36), (245, 37), (246, 38), (244, 36), (244, 35), (244, 34), (243, 36), (242, 37), (241, 36), (240, 37), (242, 40), (243, 38), (244, 38), (240, 40), (241, 39), (239, 41), (238, 42), (239, 47), (239, 49), (243, 50), (244, 53), (243, 54), (244, 54), (235, 51), (234, 51), (234, 50), (238, 53), (237, 53), (238, 54), (235, 50), (236, 49), (235, 49), (235, 48), (236, 48), (236, 47), (235, 47), (236, 46), (232, 42), (229, 41), (230, 40), (231, 39), (232, 41), (233, 42), (233, 41), (234, 48), (233, 49), (233, 50), (231, 50), (232, 50), (234, 49), (239, 45), (242, 44), (242, 43), (243, 45), (242, 45), (243, 46), (245, 44), (247, 40), (246, 40), (246, 39), (247, 39), (247, 38), (247, 37), (247, 35), (245, 35), (245, 34), (246, 33), (247, 34), (248, 33), (249, 30), (250, 30), (249, 29), (250, 29), (249, 28), (250, 28), (249, 27), (248, 28), (248, 29), (247, 28), (248, 27), (249, 26), (247, 26), (247, 25), (244, 23), (245, 23), (246, 25), (247, 27), (250, 27), (251, 28), (249, 32), (249, 31), (259, 40), (255, 41), (253, 31), (252, 29), (252, 28), (251, 26), (248, 25), (248, 22), (247, 21), (247, 20), (246, 19), (245, 19), (244, 19), (244, 18), (243, 19), (243, 18), (242, 17), (241, 18), (241, 19), (242, 20), (243, 20), (243, 21), (244, 21), (245, 20), (246, 21), (244, 22), (245, 22), (245, 21), (246, 20), (245, 18), (244, 20), (242, 19), (242, 18), (240, 18), (240, 17), (241, 17), (240, 16), (241, 15), (240, 14), (241, 14), (242, 14), (243, 13), (244, 14), (243, 15), (243, 14), (242, 13), (242, 12), (243, 12), (241, 13), (240, 13), (239, 14), (240, 15), (242, 15), (244, 13), (243, 11), (242, 11), (241, 10), (240, 10), (240, 9), (239, 9), (239, 10), (239, 11), (240, 12), (239, 13), (238, 13), (238, 12), (242, 16), (242, 10), (242, 9), (243, 9), (244, 9), (243, 10), (244, 10), (241, 8), (240, 8), (239, 8), (238, 9), (237, 8), (238, 8), (237, 9), (236, 9), (235, 10), (236, 11), (237, 12), (235, 11), (235, 12), (236, 12), (235, 13), (234, 13), (233, 14), (232, 15), (231, 15), (230, 14), (231, 13), (230, 13), (231, 14), (230, 15), (229, 14), (228, 14), (228, 15), (227, 15), (228, 16), (229, 15), (230, 16), (229, 16), (230, 17), (230, 18), (230, 19), (231, 18), (229, 17), (231, 19), (232, 20), (233, 19), (233, 20), (234, 19), (235, 19), (234, 18), (235, 17), (234, 16), (233, 17), (234, 17), (233, 16), (232, 13), (231, 17), (232, 17), (232, 18), (231, 16), (228, 18), (229, 19), (229, 18), (227, 18), (228, 19), (228, 20), (229, 21), (228, 21), (227, 20), (226, 21), (225, 20), (224, 19), (225, 19), (224, 20), (224, 21), (223, 20), (222, 19), (221, 18), (220, 17), (221, 17), (217, 16), (217, 15), (216, 15), (218, 18), (194, 25), (199, 30), (201, 32), (200, 35), (199, 35), (198, 34), (198, 35), (199, 34), (199, 33), (198, 33), (197, 34), (197, 33), (195, 33), (191, 31), (189, 31), (189, 32), (189, 34), (186, 34), (186, 31), (186, 36), (186, 33), (198, 42), (203, 51), (202, 52), (214, 46), (208, 53), (207, 54), (204, 56), (205, 57), (198, 60), (197, 59), (198, 59), (200, 59), (200, 63), (191, 69), (201, 73), (201, 72), (202, 75), (202, 74), (203, 75), (204, 74), (207, 76), (205, 75), (205, 81), (207, 78), (206, 77), (208, 78), (209, 79), (209, 78), (208, 79), (207, 80), (208, 80), (205, 74), (205, 73), (207, 72), (203, 71), (203, 69), (202, 70), (202, 71), (203, 73), (202, 73), (202, 69), (206, 68), (211, 75), (210, 76), (209, 77), (213, 77), (217, 81), (216, 75), (222, 76), (220, 79), (220, 80), (219, 84), (220, 86), (218, 87), (218, 88), (219, 89), (220, 88), (220, 89), (219, 90), (219, 91), (218, 90), (218, 89), (218, 91), (217, 91), (216, 92), (215, 91), (216, 91), (215, 93), (216, 94), (216, 93), (217, 93), (215, 94), (214, 93), (215, 92), (216, 90), (218, 92), (217, 92), (218, 93), (214, 95), (213, 94), (213, 93), (212, 94), (211, 95), (210, 96), (211, 96), (211, 97), (212, 96), (212, 95), (210, 95), (211, 94), (210, 94), (211, 93), (212, 93), (211, 92), (210, 92), (210, 91), (211, 90), (212, 89), (212, 91), (213, 91), (213, 90), (212, 90), (211, 91), (210, 90), (210, 89), (209, 88), (209, 90), (209, 89), (208, 89), (207, 90), (207, 89), (208, 88), (209, 87), (210, 88), (211, 89), (213, 89), (214, 89), (215, 90), (215, 89), (214, 91), (213, 92), (214, 92), (212, 92), (209, 91), (209, 92), (208, 91), (208, 92), (207, 91), (208, 90), (206, 91), (206, 90), (207, 92), (208, 93), (210, 93), (209, 93), (214, 90), (212, 87), (211, 88), (211, 87), (213, 86), (216, 89), (217, 90), (220, 91), (220, 90), (221, 90), (222, 89), (223, 88), (224, 88), (223, 89), (221, 88), (221, 86), (221, 87), (222, 88), (221, 89), (221, 91), (222, 92), (223, 92), (224, 91), (227, 93), (227, 92), (225, 95), (224, 96), (224, 95), (224, 94), (223, 94), (222, 95), (221, 95), (222, 94), (221, 93), (221, 94), (222, 93), (220, 93), (219, 92), (219, 93), (220, 94), (220, 95), (219, 94), (218, 94), (210, 87), (208, 87), (207, 87), (207, 88), (206, 87), (205, 86), (204, 87), (203, 86), (204, 86), (205, 87), (204, 88), (202, 85), (202, 84), (204, 85), (203, 87), (206, 86), (205, 85), (204, 84), (205, 84), (204, 83), (205, 83), (204, 82), (203, 84), (201, 85), (200, 84), (201, 83), (203, 78), (204, 81), (203, 81), (205, 82), (206, 83), (207, 84), (208, 85), (209, 84), (209, 85), (210, 85), (211, 84), (209, 82), (209, 81), (209, 80), (210, 80), (211, 80), (211, 81), (210, 81), (212, 83), (213, 84), (211, 83), (214, 83), (214, 85), (214, 86), (215, 83), (216, 83), (216, 82), (217, 82), (222, 80), (222, 79), (223, 79), (221, 79), (226, 77), (227, 77), (227, 76), (226, 76), (225, 76), (224, 77), (220, 87), (219, 88), (223, 90), (224, 89), (225, 89), (224, 90), (222, 90), (223, 91), (224, 92), (224, 93), (225, 96), (224, 97), (226, 95), (229, 94), (231, 93), (231, 95), (232, 93), (229, 92), (228, 93), (228, 92), (230, 91), (229, 91), (230, 90), (229, 90), (228, 91), (227, 91), (229, 95), (229, 96), (228, 95), (227, 96), (228, 96), (230, 95), (233, 94), (236, 94), (233, 93), (233, 92), (234, 91), (235, 90), (234, 90), (233, 91), (235, 91), (235, 92), (236, 91), (233, 89), (233, 90), (232, 90), (231, 90), (230, 89), (231, 88), (232, 87), (233, 88), (232, 88), (233, 86), (232, 85), (233, 87), (234, 87), (235, 87), (236, 88), (235, 88), (235, 89), (234, 89), (237, 88), (236, 87), (236, 86), (237, 87), (240, 87), (241, 86), (241, 91), (240, 92), (240, 93), (240, 94), (241, 95), (239, 94), (239, 95), (238, 94), (238, 93), (237, 94), (237, 93), (236, 98), (237, 99), (236, 99), (236, 100), (237, 100), (237, 101), (238, 102), (236, 102), (235, 101), (234, 101), (233, 102), (232, 103), (231, 103), (230, 104), (229, 103), (228, 103), (227, 103), (226, 103), (226, 102), (227, 102), (228, 102), (229, 102), (230, 103), (231, 104), (231, 105), (232, 105), (230, 105), (229, 106), (230, 106), (231, 106), (230, 107), (231, 107), (232, 107), (232, 106), (233, 106), (231, 108), (230, 109), (229, 110), (229, 109), (229, 108), (228, 108), (230, 110), (229, 111), (229, 112), (230, 112), (230, 113), (229, 113), (229, 114), (228, 113), (228, 112), (231, 110), (232, 110), (233, 110), (232, 109), (233, 108), (232, 108), (228, 105), (227, 106), (227, 107), (226, 106), (225, 107), (225, 108), (225, 106), (224, 107), (224, 106), (224, 105), (223, 105), (224, 108), (224, 104), (224, 103), (223, 102), (222, 103), (223, 103), (222, 104), (222, 105), (222, 106), (223, 106), (223, 104), (224, 102), (224, 101), (225, 101), (224, 100), (223, 101), (222, 102), (223, 100), (222, 101), (221, 102), (221, 100), (220, 101), (219, 101), (218, 101), (219, 100), (218, 100), (217, 100), (217, 99), (218, 99), (219, 98), (220, 97), (220, 98), (220, 96), (221, 97), (221, 96), (219, 96), (218, 97), (217, 96), (216, 96), (217, 95), (218, 95), (218, 96), (219, 95), (217, 94), (219, 97), (221, 98), (222, 97), (223, 97), (226, 96), (223, 96), (223, 98), (224, 98), (225, 99), (225, 100), (226, 99), (227, 99), (228, 99), (227, 98), (225, 98), (225, 97), (224, 99), (223, 99), (222, 100), (222, 99), (220, 99), (222, 98), (226, 100), (227, 100), (226, 101), (225, 102), (221, 101), (225, 103), (227, 101), (228, 101), (230, 101), (229, 100), (229, 101), (230, 100), (231, 100), (229, 99), (230, 99), (225, 104), (226, 105), (225, 105), (228, 104), (221, 104), (221, 103), (220, 104), (219, 103), (219, 104), (218, 104), (217, 104), (218, 105), (217, 106), (219, 106), (220, 105), (221, 105), (219, 105), (218, 106), (218, 103), (218, 102), (217, 102), (217, 101), (216, 101), (216, 99), (217, 98), (217, 97), (226, 98), (226, 97), (222, 96), (220, 102), (219, 102), (220, 103), (217, 105), (219, 107), (218, 108), (218, 107), (216, 104), (217, 103), (216, 103), (216, 102), (215, 101), (214, 102), (213, 102), (213, 103), (212, 103), (212, 102), (213, 101), (213, 100), (212, 101), (212, 100), (214, 100), (214, 99), (213, 98), (212, 98), (211, 99), (211, 100), (210, 100), (210, 99), (209, 98), (209, 97), (209, 95), (208, 96), (209, 96), (208, 95), (207, 96), (206, 95), (205, 94), (204, 94), (204, 93), (203, 93), (202, 94), (202, 93), (203, 92), (203, 94), (204, 92), (204, 91), (205, 91), (205, 90), (205, 89), (204, 89), (204, 90), (203, 91), (202, 92), (205, 92), (203, 90), (203, 89), (202, 89), (202, 90), (201, 91), (200, 90), (201, 90), (199, 91), (200, 92), (199, 89), (200, 89), (200, 88), (198, 90), (198, 91), (197, 90), (196, 90), (195, 90), (194, 89), (193, 89), (194, 90), (192, 90), (192, 91), (193, 92), (194, 92), (193, 91), (193, 90), (194, 91), (195, 88), (196, 87), (195, 87), (195, 86), (194, 85), (193, 85), (188, 81), (189, 82), (187, 84), (186, 84), (185, 81), (184, 79), (184, 78), (185, 78), (185, 77), (188, 82), (187, 77), (185, 73), (185, 72), (183, 72), (184, 73), (183, 73), (181, 72), (181, 73), (180, 70), (180, 71), (179, 71), (180, 72), (179, 73), (178, 73), (178, 74), (178, 75), (177, 74), (179, 75), (177, 75), (176, 75), (175, 76), (176, 76), (177, 76), (178, 77), (177, 77), (178, 76), (179, 74), (179, 76), (178, 78), (179, 77), (180, 77), (181, 77), (180, 76), (180, 75), (180, 74), (182, 73), (180, 73), (179, 70), (176, 70), (176, 69), (177, 70), (178, 71), (178, 72), (179, 72), (183, 66), (182, 65), (181, 66), (182, 67), (182, 66), (183, 67), (180, 66), (180, 65), (179, 64), (180, 63), (179, 62), (180, 62), (179, 61), (178, 61), (177, 61), (176, 60), (175, 61), (176, 62), (177, 63), (176, 64), (176, 63), (175, 63), (174, 64), (175, 64), (175, 62), (174, 63), (174, 62), (177, 62), (179, 60), (180, 60), (177, 58), (176, 59), (178, 54), (180, 53), (179, 51), (179, 50), (178, 51), (178, 50), (178, 49), (178, 48), (179, 48), (179, 49), (179, 47), (178, 47), (180, 48), (181, 49), (181, 48), (181, 47), (182, 46), (181, 46), (180, 47), (180, 49), (180, 50), (177, 47), (176, 47), (177, 46), (177, 49), (176, 50), (177, 50), (177, 51), (176, 52), (175, 53), (175, 54), (176, 54), (177, 54), (178, 53), (177, 52), (178, 52), (183, 50), (183, 51), (182, 48), (183, 49), (183, 47), (184, 46), (184, 45), (184, 43), (184, 42), (185, 41), (187, 42), (186, 44), (183, 44), (183, 43), (182, 43), (181, 44), (181, 45), (180, 46), (182, 45), (182, 44), (182, 47), (182, 52), (182, 54), (179, 53), (179, 54), (176, 53), (175, 52), (176, 51), (177, 53), (175, 59), (173, 58), (174, 56), (177, 60), (182, 59), (186, 54), (187, 54), (191, 52), (191, 51), (188, 49), (191, 50), (191, 49), (188, 47), (189, 48), (187, 41), (190, 38), (191, 38), (188, 32), (190, 32), (193, 35), (194, 34), (194, 35), (195, 34), (196, 34), (195, 35), (196, 35), (196, 33), (218, 23), (218, 21), (219, 22), (220, 21), (219, 21), (219, 20), (220, 19), (220, 18), (220, 16), (221, 19), (221, 20), (218, 19), (219, 16), (220, 20), (219, 23), (220, 27), (219, 27), (218, 26), (219, 28), (222, 30), (221, 30), (221, 32), (221, 28), (222, 28), (222, 27), (220, 25), (221, 15), (222, 14), (223, 15), (224, 16), (225, 15), (225, 14), (224, 15), (225, 16), (223, 16), (222, 16), (222, 15), (220, 14), (221, 14), (222, 11), (222, 10), (223, 10), (224, 10), (224, 9), (223, 11), (223, 12), (222, 12), (224, 12), (224, 13), (223, 13), (225, 13), (226, 14), (226, 13), (227, 14), (232, 14), (232, 16), (233, 13), (227, 13), (227, 17), (226, 15), (226, 16), (227, 12), (228, 11), (226, 12), (226, 11), (227, 11), (228, 10), (229, 10), (230, 11), (231, 11), (232, 12), (233, 12), (233, 15), (231, 12), (234, 14), (235, 14), (236, 15), (236, 14), (234, 12), (236, 13), (234, 15), (235, 15), (237, 16), (236, 17), (236, 16), (237, 17), (237, 18), (236, 19), (236, 20), (235, 20), (234, 21), (235, 21), (236, 21), (237, 21), (238, 20), (237, 20), (237, 22), (238, 23), (239, 23), (239, 24), (237, 24), (241, 23), (243, 26), (244, 27), (243, 27), (244, 28), (242, 26), (243, 28), (242, 28), (241, 29), (242, 29), (241, 27), (241, 28), (241, 30), (240, 29), (241, 31), (242, 32), (243, 32), (242, 33), (241, 34), (242, 34), (242, 35), (241, 35), (242, 36), (240, 36), (239, 35), (238, 34), (239, 34), (240, 35), (239, 33), (238, 33), (237, 33), (237, 34), (238, 35), (238, 36), (239, 37), (241, 37), (242, 38), (239, 40), (245, 40), (248, 32), (248, 34), (249, 35), (248, 35), (248, 36), (246, 41), (253, 42), (254, 43), (255, 46), (256, 45), (256, 47), (257, 46), (258, 45), (258, 46), (259, 45), (259, 44), (260, 45), (263, 47), (264, 48), (263, 49), (263, 48), (264, 50), (265, 50), (265, 51), (264, 52), (266, 51), (266, 50), (266, 52), (267, 50), (268, 49), (267, 48), (267, 49), (266, 49), (265, 49), (264, 49), (267, 52), (268, 52), (266, 53), (268, 51), (265, 52), (262, 55), (262, 56), (261, 57), (262, 58), (263, 59), (262, 57), (263, 56), (264, 55), (264, 56), (265, 56), (266, 55), (266, 56), (265, 55), (265, 54), (264, 53), (261, 56), (260, 56), (261, 55), (260, 54), (266, 54), (267, 55), (268, 56), (267, 56), (268, 55), (267, 54), (268, 53), (267, 53), (264, 57), (265, 58), (266, 59), (266, 60), (265, 59), (267, 59), (268, 58), (268, 59), (267, 58), (267, 60), (267, 61), (268, 61), (268, 60), (266, 58), (265, 60), (264, 60), (265, 61), (266, 62), (266, 63), (266, 64), (267, 64), (267, 65), (268, 65), (267, 63), (268, 64), (268, 63), (267, 62), (268, 62), (266, 65), (265, 65), (264, 64), (265, 64), (264, 63), (265, 62), (264, 62), (263, 63), (265, 63), (266, 61), (264, 65), (264, 66), (265, 66), (266, 67), (265, 67), (264, 67), (263, 67), (263, 66), (262, 67), (261, 67), (261, 66), (260, 66), (260, 65), (261, 64), (261, 65), (262, 65), (262, 66), (260, 67), (266, 66), (267, 67), (266, 68), (265, 68), (265, 69), (267, 68), (266, 69), (265, 70), (266, 71), (267, 70), (264, 68), (262, 68), (261, 68), (260, 68), (260, 69), (259, 69), (258, 70), (259, 67), (258, 66), (257, 67), (256, 67), (257, 68), (258, 69), (258, 68), (257, 69), (257, 70), (259, 70), (259, 71), (260, 71), (261, 70), (262, 71), (261, 72), (262, 73), (261, 73), (263, 73), (263, 74), (262, 74), (263, 72), (263, 71), (263, 70), (264, 70), (263, 69), (262, 70), (261, 69), (259, 68), (258, 71), (256, 71), (256, 70), (255, 69), (256, 69), (255, 68), (254, 69), (256, 68), (254, 68), (255, 66), (256, 66), (256, 65), (256, 64), (257, 63), (258, 64), (259, 64), (258, 63), (257, 64), (256, 63), (257, 62), (255, 64), (254, 65), (255, 67), (254, 66), (254, 64), (255, 63), (255, 62), (254, 61), (255, 61), (254, 60), (255, 60), (254, 59), (253, 60), (254, 62), (254, 63), (256, 62), (256, 61), (257, 60), (256, 60), (255, 59), (254, 58), (253, 58), (252, 59), (252, 60), (252, 61), (253, 61), (252, 62), (251, 60), (253, 65), (253, 64), (252, 63), (251, 62), (251, 61), (250, 61), (249, 60), (250, 60), (252, 58), (251, 58), (250, 57), (251, 57), (252, 57), (250, 58), (250, 59), (250, 56), (251, 56), (249, 56), (249, 55), (248, 55), (248, 56), (249, 54), (248, 53), (248, 52), (248, 51), (245, 52), (247, 51), (247, 52), (246, 53), (235, 53), (234, 54), (236, 54), (235, 55), (233, 53), (233, 51), (232, 54), (232, 55), (233, 55), (232, 56), (231, 55), (230, 55), (229, 54), (231, 54), (236, 52), (232, 49), (232, 48), (231, 49), (234, 53), (231, 56), (232, 57), (231, 57), (233, 57), (234, 56), (235, 57), (234, 55), (235, 54), (236, 55), (237, 56), (238, 56), (237, 57), (236, 58), (236, 59), (235, 59), (235, 58), (234, 58), (233, 59), (232, 58), (233, 56), (233, 54), (236, 56), (233, 58), (234, 57), (236, 60), (235, 61), (234, 61), (233, 62), (232, 62), (231, 63), (231, 62), (232, 61), (233, 60), (234, 60), (234, 59), (231, 60), (231, 61), (230, 62), (229, 62), (230, 61), (229, 61), (230, 63), (228, 62), (228, 61), (229, 60), (230, 59), (231, 59), (232, 59), (233, 61), (232, 60), (231, 58), (230, 58), (229, 57), (230, 57), (229, 56), (228, 55), (228, 56), (228, 57), (228, 58), (229, 59), (230, 60), (231, 64), (232, 65), (233, 66), (234, 66), (232, 67), (233, 67), (232, 68), (232, 69), (231, 69), (230, 69), (229, 68), (227, 72), (226, 72), (225, 71), (226, 71), (229, 72), (229, 73), (230, 72), (230, 75), (228, 76), (229, 77), (226, 75), (227, 71), (225, 72), (226, 70), (228, 69), (227, 70), (225, 74), (224, 74), (222, 72), (223, 71), (222, 71), (227, 67), (228, 67), (228, 66), (229, 67), (230, 67), (231, 68), (230, 68), (229, 66), (230, 66), (231, 67), (233, 69), (234, 69), (233, 70), (232, 70), (234, 70), (235, 69), (235, 68), (236, 70), (234, 68), (235, 67), (236, 68), (237, 68), (237, 67), (238, 68), (238, 69), (239, 68), (238, 67), (237, 69), (236, 69), (236, 67), (233, 71), (234, 71), (237, 70), (239, 69), (239, 67), (238, 70), (236, 71), (235, 71), (234, 72), (235, 72), (236, 73), (235, 73), (236, 72), (236, 74), (235, 74), (234, 74), (234, 75), (235, 76), (233, 74), (233, 75), (234, 76), (233, 76), (232, 76), (232, 75), (231, 76), (231, 75), (231, 74), (232, 73), (232, 72), (232, 71), (231, 71), (232, 66), (235, 66), (234, 65), (235, 64), (233, 64), (232, 63), (231, 72), (230, 71), (231, 70), (230, 70), (230, 77), (232, 77), (233, 77), (228, 73), (227, 75), (226, 74), (225, 75), (225, 78), (225, 88), (224, 87), (225, 90), (226, 84), (230, 83), (227, 84), (228, 84), (229, 84), (228, 85), (229, 85), (227, 85), (230, 86), (231, 86), (234, 86), (236, 85), (236, 80), (236, 79), (236, 78), (236, 77), (237, 77), (237, 76), (238, 75), (236, 76), (235, 77), (234, 78), (233, 79), (234, 80), (233, 80), (232, 80), (233, 81), (234, 79), (233, 78), (235, 78), (234, 77), (235, 80), (235, 79), (232, 82), (232, 83), (234, 92), (237, 92), (236, 93), (239, 93), (238, 92), (236, 92), (235, 93), (235, 97), (232, 92), (234, 88), (236, 89), (236, 90), (238, 89), (239, 84), (241, 83), (242, 84), (239, 92), (241, 93), (242, 92), (240, 90), (240, 91), (241, 92), (244, 90), (246, 89), (247, 85), (248, 85), (248, 84), (249, 85), (250, 86), (250, 87), (250, 88), (251, 89), (252, 88), (253, 88), (253, 89), (254, 89), (255, 89), (256, 89), (255, 90), (254, 90), (255, 88), (254, 88), (254, 87), (253, 86), (255, 86), (255, 87), (256, 86), (257, 86), (257, 85), (256, 85), (258, 85), (259, 84), (260, 85), (261, 84), (262, 84), (263, 83), (264, 82), (262, 82), (263, 81), (262, 80), (263, 80), (262, 79), (261, 79), (261, 80), (262, 81), (261, 81), (264, 81), (264, 80), (264, 79), (264, 78), (265, 79), (265, 78), (265, 77), (264, 76), (263, 75), (265, 76), (264, 77), (263, 77), (266, 76), (267, 76), (266, 77), (266, 75), (267, 74), (268, 75), (268, 73), (268, 72), (267, 71), (268, 71), (268, 70), (268, 69), (266, 72), (265, 71), (266, 70), (267, 69), (268, 68), (260, 72), (260, 70), (261, 71), (258, 72), (258, 67), (257, 66), (254, 67), (253, 68), (252, 67), (252, 66), (251, 66), (250, 66), (249, 67), (248, 68), (249, 68), (250, 68), (250, 69), (250, 67), (251, 68), (252, 69), (248, 66), (248, 65), (247, 66), (246, 65), (246, 64), (247, 63), (248, 62), (249, 62), (250, 63), (250, 62), (249, 61), (248, 61), (247, 61), (246, 62), (249, 63), (248, 63), (247, 62), (248, 60), (249, 59), (253, 59), (253, 62), (257, 61), (258, 60), (259, 61), (259, 60), (260, 59), (261, 58), (260, 55), (259, 56), (258, 56), (259, 57), (257, 57), (257, 58), (256, 59), (257, 59), (255, 58), (256, 58), (255, 57), (251, 63), (250, 64), (249, 65), (249, 64), (248, 64), (250, 65), (247, 65), (246, 66), (245, 66), (245, 65), (245, 64), (247, 64), (249, 66), (248, 67), (249, 69), (248, 70), (249, 71), (247, 71), (248, 71), (249, 72), (248, 72), (247, 72), (246, 71), (247, 73), (248, 73), (247, 74), (248, 74), (246, 74), (245, 74), (245, 75), (245, 73), (244, 72), (246, 73), (247, 75), (246, 75), (247, 76), (245, 76), (244, 75), (248, 75), (249, 74), (246, 72), (245, 71), (243, 71), (242, 71), (243, 72), (244, 71), (242, 70), (243, 70), (243, 69), (244, 69), (244, 68), (244, 67), (243, 66), (242, 67), (243, 67), (244, 65), (244, 64), (243, 64), (244, 66), (243, 68), (245, 67), (246, 68), (243, 65), (244, 63), (245, 63), (245, 62), (245, 61), (246, 60), (246, 61), (246, 67), (244, 70), (245, 72), (247, 70), (247, 69), (246, 69), (245, 69), (242, 69), (242, 72), (241, 73), (245, 70), (246, 70), (244, 73), (243, 73), (243, 74), (242, 73), (242, 74), (242, 75), (241, 74), (243, 75), (244, 76), (243, 76), (242, 76), (244, 74), (245, 77), (246, 76), (246, 77), (247, 78), (246, 78), (247, 77), (248, 77), (249, 77), (250, 77), (251, 76), (251, 77), (252, 77), (252, 75), (251, 74), (253, 76), (253, 75), (254, 76), (255, 77), (254, 78), (254, 79), (254, 80), (254, 81), (254, 82), (253, 81), (255, 82), (256, 82), (257, 82), (256, 83), (257, 83), (258, 82), (259, 82), (260, 82), (260, 83), (262, 85), (263, 85), (264, 86), (264, 87), (265, 86), (264, 85), (263, 84), (265, 85), (266, 85), (266, 86), (267, 87), (267, 86), (267, 85), (266, 84), (268, 86), (268, 85), (268, 84), (268, 83), (267, 84), (267, 83), (268, 82), (268, 81), (267, 82), (266, 82), (265, 82), (262, 83), (260, 80), (259, 79), (261, 82), (260, 81), (259, 81), (258, 83), (259, 83), (261, 83), (261, 85), (259, 85), (259, 86), (260, 84), (258, 86), (258, 84), (257, 84), (257, 81), (256, 84), (258, 81), (258, 80), (263, 82), (263, 79), (265, 81), (266, 81), (261, 78), (262, 77), (262, 76), (263, 76), (263, 78), (262, 78), (266, 78), (266, 79), (267, 80), (267, 79), (268, 80), (265, 83), (266, 83), (265, 84), (264, 83), (267, 78), (264, 75), (260, 74), (260, 73), (259, 65), (258, 65), (259, 63), (260, 64), (259, 66), (257, 65), (261, 63), (260, 62), (261, 60), (262, 60), (262, 61), (261, 62), (262, 63), (260, 63), (254, 57), (253, 57), (252, 56), (254, 56), (253, 56), (251, 55), (248, 57), (249, 57), (252, 54), (252, 55), (249, 58), (251, 59), (255, 56), (256, 56), (256, 57), (258, 58), (259, 59), (259, 58), (256, 55), (255, 55), (254, 55), (255, 54), (256, 53), (255, 53), (254, 53), (254, 54), (255, 52), (256, 54), (257, 53), (258, 52), (259, 51), (259, 54), (259, 52), (259, 53), (258, 54), (258, 53), (257, 54), (258, 55), (257, 55), (257, 56), (258, 57), (260, 58), (258, 59), (258, 61), (259, 55), (260, 57), (261, 59), (260, 60), (263, 57), (262, 59), (253, 63), (252, 64), (252, 65), (255, 65), (253, 66), (251, 65), (247, 67), (246, 63), (242, 65), (241, 66), (240, 65), (240, 66), (238, 66), (237, 65), (236, 64), (235, 65), (234, 64), (236, 66), (234, 67), (239, 66), (237, 66), (236, 65), (237, 72), (237, 73), (234, 73), (233, 68), (225, 73), (228, 74), (233, 72), (233, 73), (236, 75), (235, 75), (237, 75), (238, 76), (239, 75), (239, 76), (239, 77), (239, 78), (240, 77), (240, 76), (241, 76), (242, 77), (242, 78), (241, 78), (243, 77), (241, 75), (240, 74), (240, 75), (240, 73), (239, 73), (238, 73), (237, 71), (238, 72), (238, 74), (239, 74), (241, 72), (241, 77), (244, 77), (243, 78), (244, 78), (244, 79), (243, 79), (244, 80), (245, 79), (240, 78), (241, 79), (240, 79), (238, 77), (237, 79), (238, 78), (238, 79), (238, 80), (239, 81), (240, 80), (239, 79), (237, 78), (242, 79), (242, 80), (242, 81), (241, 81), (240, 82), (240, 81), (241, 80), (249, 70), (248, 69), (247, 68), (251, 70), (250, 70), (250, 71), (250, 72), (251, 71), (252, 70), (253, 71), (253, 70), (253, 69), (252, 68), (253, 67), (251, 67), (251, 69), (254, 71), (252, 71), (251, 72), (250, 73), (251, 73), (250, 74), (250, 75), (252, 78), (251, 79), (250, 79), (253, 78), (253, 77), (253, 74), (253, 73), (253, 72), (252, 73), (249, 75), (250, 76), (252, 72), (249, 76), (255, 79), (255, 80), (256, 79), (256, 80), (255, 81), (254, 83), (253, 83), (252, 84), (253, 85), (254, 85), (255, 85), (264, 84), (262, 86), (261, 86), (262, 87), (263, 86), (256, 81), (259, 80), (260, 79), (260, 78), (259, 77), (260, 77), (259, 78), (258, 78), (257, 78), (258, 77), (259, 76), (260, 76), (259, 75), (258, 74), (257, 73), (258, 73), (259, 73), (259, 74), (258, 75), (257, 76), (256, 77), (255, 78), (256, 78), (254, 77), (252, 79), (251, 80), (250, 80), (250, 78), (249, 78), (248, 76), (241, 69), (240, 68), (240, 67), (241, 65), (242, 64), (241, 63), (242, 62), (243, 62), (243, 63), (242, 63), (241, 64), (242, 66), (238, 65), (239, 65), (240, 64), (239, 64), (235, 70), (233, 65), (232, 64), (231, 65), (239, 71), (239, 72), (240, 71), (240, 72), (241, 70), (248, 78), (248, 79), (247, 79), (249, 80), (248, 80), (242, 68), (245, 60), (244, 61), (243, 61), (244, 62), (245, 68), (254, 74), (254, 72), (255, 71), (255, 72), (255, 70), (254, 70), (254, 73), (254, 75), (252, 74), (255, 76), (240, 69), (240, 63), (241, 68), (239, 70), (240, 70), (241, 71), (238, 71), (221, 73), (222, 74), (223, 75), (224, 75), (224, 76), (227, 66), (222, 68), (220, 70), (219, 71), (217, 70), (217, 69), (221, 68), (221, 71), (221, 72), (220, 73), (231, 66), (225, 65), (225, 64), (227, 63), (228, 63), (227, 62), (227, 61), (224, 64), (223, 64), (222, 64), (222, 63), (222, 62), (223, 62), (223, 59), (224, 59), (224, 58), (223, 58), (222, 58), (221, 58), (222, 59), (223, 57), (224, 57), (224, 56), (223, 56), (222, 60), (223, 61), (213, 65), (213, 66), (201, 55), (202, 55), (200, 53), (199, 52), (199, 53), (198, 53), (200, 52), (198, 52), (199, 54), (198, 54), (196, 55), (195, 54), (196, 54), (196, 53), (197, 52), (197, 54), (197, 53), (201, 53), (202, 51), (201, 52), (201, 51), (201, 50), (200, 49), (196, 48), (194, 43), (193, 42), (192, 40), (197, 42), (196, 36), (186, 48), (185, 46), (186, 40), (184, 40), (184, 38), (185, 33), (185, 32), (182, 38), (181, 39), (182, 39), (181, 40), (182, 41), (181, 42), (181, 43), (181, 41), (182, 42), (182, 40), (180, 40), (180, 41), (183, 40), (183, 42), (183, 41), (183, 46), (185, 47), (185, 48), (183, 45), (180, 45), (180, 43), (179, 43), (178, 44), (177, 43), (177, 42), (178, 41), (179, 40), (179, 41), (180, 42), (178, 40), (177, 40), (176, 41), (175, 41), (174, 36), (173, 36), (163, 38), (164, 39), (164, 40), (165, 40), (166, 40), (166, 38), (165, 37), (171, 40), (172, 41), (171, 41), (170, 42), (170, 41), (179, 18), (184, 30), (182, 31), (183, 31), (182, 19), (182, 32), (183, 32), (184, 31), (184, 32), (185, 31), (194, 36), (199, 32), (189, 37), (186, 41), (185, 39), (187, 39), (192, 38), (200, 54), (204, 53), (196, 59), (203, 63), (206, 58), (206, 57), (206, 53), (207, 52), (207, 53), (213, 57), (225, 59), (222, 66), (222, 67), (220, 71), (217, 73), (223, 76), (234, 83), (237, 83), (244, 84), (245, 84), (245, 83), (246, 82), (245, 82), (244, 83), (245, 81), (244, 82), (243, 81), (244, 81), (245, 80), (246, 79), (246, 81), (247, 81), (248, 81), (247, 82), (248, 83), (249, 83), (250, 82), (249, 84), (248, 82), (249, 81), (249, 79), (252, 81), (253, 80), (253, 79), (252, 76), (251, 78), (245, 78), (242, 82), (239, 80), (243, 80), (243, 82), (243, 83), (246, 80), (246, 83), (247, 83), (246, 85), (237, 74), (237, 80), (238, 81), (237, 81), (239, 82), (239, 83), (244, 91), (247, 89), (246, 90), (247, 86), (248, 86), (247, 84), (248, 87), (249, 87), (246, 84), (247, 88), (248, 88), (247, 87), (249, 86), (250, 83), (251, 82), (252, 83), (251, 84), (250, 85), (250, 84), (251, 85), (251, 83), (252, 82), (255, 83), (255, 84), (259, 87), (258, 87), (257, 88), (258, 88), (259, 88), (260, 87), (260, 88), (260, 89), (259, 89), (259, 90), (260, 91), (260, 90), (261, 92), (260, 92), (259, 91), (258, 90), (258, 91), (257, 90), (259, 92), (262, 92), (261, 93), (262, 93), (263, 94), (264, 95), (264, 96), (265, 95), (263, 97), (262, 97), (263, 98), (264, 98), (265, 97), (266, 96), (267, 97), (266, 97), (265, 96), (264, 94), (262, 94), (262, 95), (261, 94), (260, 94), (260, 95), (261, 96), (262, 96), (261, 95), (259, 94), (258, 94), (259, 95), (260, 93), (259, 93), (258, 92), (257, 93), (257, 92), (256, 93), (256, 92), (255, 93), (255, 94), (258, 93), (256, 91), (256, 90), (257, 89), (261, 88), (261, 87), (261, 77), (261, 74), (260, 75), (257, 74), (258, 76), (261, 75), (257, 77), (258, 79), (263, 87), (260, 86), (265, 80), (266, 80), (268, 79), (267, 81), (257, 79), (257, 80), (264, 73), (262, 75), (262, 72), (257, 72), (255, 75), (256, 75), (257, 75), (256, 74), (255, 74), (256, 73), (251, 75), (247, 80), (249, 82), (250, 81), (242, 83), (241, 82), (234, 81), (219, 79), (220, 81), (221, 80), (225, 77), (242, 91), (246, 91), (247, 91), (248, 91), (247, 90), (239, 89), (239, 90), (238, 90), (237, 89), (257, 71), (258, 62), (260, 61), (263, 60), (264, 59), (264, 58), (264, 61), (263, 61), (263, 62), (261, 61), (263, 58), (263, 64), (266, 57), (267, 57), (268, 57), (265, 57), (263, 68), (262, 64), (262, 62), (259, 62), (248, 59), (248, 58), (247, 58), (246, 58), (245, 58), (245, 59), (246, 57), (245, 57), (246, 56), (247, 56), (245, 55), (245, 56), (247, 55), (247, 54), (247, 53), (248, 54), (251, 53), (253, 53), (253, 54), (255, 73), (249, 73), (256, 72), (259, 72), (264, 69), (251, 64), (253, 55), (250, 54), (250, 52), (252, 49), (248, 48), (243, 47), (234, 42), (234, 41), (235, 41), (234, 40), (235, 40), (236, 39), (228, 35), (228, 34), (228, 33), (229, 33), (230, 33), (231, 34), (231, 35), (230, 36), (228, 38), (225, 38), (224, 37), (224, 38), (226, 40), (226, 41), (227, 52), (227, 53), (226, 54), (227, 55), (228, 54), (226, 56), (227, 56), (227, 58), (226, 57), (226, 55), (228, 53), (227, 54), (226, 53), (229, 52), (226, 51), (225, 52), (224, 52), (225, 53), (224, 53), (223, 54), (224, 54), (223, 55), (222, 56), (200, 58), (199, 58), (197, 58), (198, 58), (200, 57), (194, 53), (195, 52), (194, 52), (193, 52), (193, 53), (193, 54), (190, 55), (199, 64), (202, 63), (204, 65), (201, 67), (190, 83), (190, 82), (195, 85), (196, 84), (196, 78), (197, 79), (197, 80), (197, 78), (200, 74), (203, 70), (207, 70), (211, 65), (221, 67), (225, 66), (229, 65), (230, 64), (229, 63), (233, 63), (235, 60), (229, 58), (227, 57), (226, 59), (225, 57), (225, 56), (226, 58), (225, 60), (226, 61), (226, 60), (225, 58), (230, 56), (235, 56), (236, 57), (229, 55), (228, 59), (212, 66), (223, 63), (224, 63), (225, 61), (224, 62), (226, 62), (227, 59), (228, 60), (230, 65), (229, 64), (228, 64), (227, 60), (216, 71), (217, 72), (216, 74), (220, 72), (239, 63), (238, 63), (238, 62), (239, 61), (238, 60), (237, 59), (238, 58), (238, 57), (239, 58), (238, 59), (239, 59), (240, 60), (239, 60), (238, 61), (240, 59), (241, 60), (241, 59), (225, 55), (222, 54), (227, 40), (228, 32), (229, 32), (229, 34), (230, 37), (232, 35), (233, 35), (234, 36), (234, 35), (234, 34), (235, 35), (235, 36), (233, 34), (232, 33), (233, 32), (234, 31), (233, 30), (234, 30), (235, 29), (236, 30), (236, 31), (235, 30), (234, 32), (233, 31), (235, 31), (236, 29), (236, 28), (236, 27), (235, 28), (234, 29), (233, 28), (233, 27), (234, 26), (234, 25), (235, 24), (235, 25), (234, 24), (236, 23), (238, 24), (237, 25), (236, 25), (238, 25), (238, 26), (239, 27), (238, 27), (239, 22), (240, 22), (237, 23), (236, 24), (237, 26), (235, 23), (238, 21), (238, 22), (239, 21), (240, 25), (241, 26), (246, 22), (246, 26), (246, 27), (245, 27), (242, 22), (241, 22), (243, 24), (249, 25), (250, 24), (251, 23), (250, 25), (245, 26), (245, 28), (246, 28), (247, 29), (247, 30), (247, 31), (246, 32), (245, 31), (244, 30), (244, 29), (245, 30), (245, 29), (243, 30), (243, 29), (244, 31), (244, 32), (245, 32), (244, 33), (243, 34), (246, 30), (246, 29), (243, 31), (242, 30), (243, 33), (243, 35), (246, 34), (246, 35), (245, 39), (244, 43), (243, 44), (243, 51), (245, 53), (245, 54), (246, 59), (244, 58), (244, 59), (243, 60), (242, 59), (243, 59), (242, 60), (244, 57), (247, 60), (244, 60), (241, 67), (234, 63), (225, 54), (224, 55), (214, 81), (214, 82), (216, 84), (216, 85), (217, 86), (215, 84), (212, 78), (211, 77), (210, 78), (207, 68), (202, 50), (204, 52), (205, 31), (201, 36), (200, 34), (200, 41), (200, 40), (201, 35), (219, 29), (218, 29), (220, 34), (224, 35), (221, 36), (223, 28), (223, 27), (221, 25), (222, 24), (223, 23), (223, 22), (222, 22), (221, 23), (224, 22), (224, 23), (225, 23), (226, 22), (225, 21), (226, 20), (223, 19), (222, 18), (222, 17), (221, 16), (223, 17), (224, 17), (221, 21), (223, 18), (224, 18), (226, 17), (230, 20), (231, 21), (232, 21), (233, 22), (233, 21), (234, 20), (234, 22), (235, 22), (232, 23), (233, 23), (233, 24), (234, 23), (232, 22), (236, 22), (235, 18), (233, 18), (234, 11), (234, 10), (233, 10), (232, 11), (234, 9), (233, 11), (229, 13), (230, 12), (236, 10), (237, 10), (238, 10), (237, 13), (232, 10), (234, 8), (233, 8), (233, 9), (232, 9), (232, 8), (230, 10), (229, 9), (228, 8), (227, 7), (226, 6), (225, 6), (225, 5), (224, 5), (224, 6), (223, 7), (222, 8), (221, 9), (220, 10), (219, 9), (208, 5), (209, 5), (193, 8), (190, 9), (187, 8), (185, 8), (190, 10), (188, 9), (189, 14), (182, 11), (184, 13), (184, 8), (183, 7), (182, 7), (182, 6), (181, 6), (181, 5), (182, 5), (183, 6), (184, 5), (185, 5), (184, 6), (184, 7), (184, 9), (183, 10), (183, 8), (182, 8), (183, 9), (195, 7), (196, 21), (192, 7), (190, 7), (191, 8), (186, 5), (189, 5), (182, 9), (181, 8), (181, 7), (180, 6), (179, 6), (178, 7), (179, 5), (178, 5), (177, 7), (177, 6), (179, 7), (179, 8), (179, 9), (180, 8), (180, 7), (183, 5), (181, 9), (181, 11), (187, 20), (181, 10), (180, 9), (180, 10), (179, 11), (178, 10), (178, 9), (179, 10), (178, 11), (179, 12), (177, 8), (176, 7), (176, 8), (175, 7), (174, 7), (173, 6), (172, 5), (174, 6), (174, 5), (175, 5), (175, 6), (173, 5), (177, 9), (176, 9), (177, 10), (176, 11), (176, 10), (175, 11), (177, 11), (176, 12), (177, 12), (178, 13), (178, 12), (182, 10), (184, 20), (185, 20), (188, 20), (189, 20), (189, 22), (190, 24), (190, 25), (189, 26), (191, 26), (190, 26), (189, 24), (189, 23), (187, 24), (190, 27), (190, 29), (188, 22), (187, 30), (188, 30), (187, 31), (188, 31), (190, 31), (186, 32), (189, 30), (208, 29), (198, 40), (191, 27), (189, 21), (187, 21), (190, 21), (190, 22), (185, 42), (191, 39), (195, 36), (188, 29), (187, 29), (192, 50), (184, 41), (184, 44), (178, 32), (176, 39), (177, 38), (185, 22), (177, 13), (177, 14), (180, 11), (178, 14), (178, 17), (172, 15), (171, 15), (170, 14), (171, 14), (171, 13), (169, 15), (170, 15), (171, 16), (171, 17), (170, 17), (153, 23), (151, 22), (152, 22), (155, 18), (160, 37), (157, 35), (156, 36), (156, 34), (155, 35), (154, 33), (154, 32), (154, 31), (154, 30), (155, 32), (155, 33), (152, 34), (151, 33), (151, 32), (172, 14), (174, 14), (173, 14), (174, 13), (174, 12), (174, 11), (174, 10), (173, 9), (174, 9), (175, 9), (175, 10), (176, 13), (173, 16), (175, 12), (175, 13), (173, 13), (172, 12), (171, 11), (171, 12), (173, 12), (172, 13), (169, 13), (169, 12), (168, 13), (168, 14), (167, 14), (166, 15), (166, 13), (166, 14), (165, 15), (166, 16), (167, 16), (168, 16), (168, 17), (167, 15), (167, 12), (167, 13), (167, 17), (164, 16), (165, 17), (165, 16), (166, 12), (168, 12), (169, 11), (169, 14), (170, 16), (169, 16), (169, 18), (169, 17), (176, 14), (170, 13), (170, 12), (168, 10), (169, 9), (170, 9), (168, 9), (167, 10), (169, 8), (170, 7), (171, 7), (172, 7), (171, 8), (172, 6), (171, 5), (170, 6), (173, 7), (173, 8), (173, 10), (168, 18), (167, 18), (166, 18), (166, 17), (166, 19), (167, 19), (176, 26), (165, 38), (196, 7), (194, 19), (193, 20), (192, 20), (193, 19), (200, 11), (216, 14), (218, 13), (220, 15), (219, 15), (222, 21), (222, 20), (226, 19), (227, 19), (228, 17), (227, 16), (225, 17), (226, 18), (224, 14), (223, 14), (221, 11), (221, 10), (221, 8), (222, 7), (223, 8), (223, 9), (222, 9), (194, 22), (217, 25), (200, 39), (197, 35), (192, 51), (192, 58), (194, 59), (193, 61), (193, 51), (186, 52), (186, 53), (187, 53), (185, 54), (185, 52), (184, 53), (184, 54), (183, 53), (185, 51), (185, 53), (184, 55), (184, 51), (181, 64), (180, 64), (179, 63), (177, 71), (178, 67), (178, 66), (177, 65), (177, 66), (176, 66), (175, 66), (174, 65), (176, 61), (178, 62), (181, 61), (184, 61), (184, 62), (184, 64), (184, 63), (183, 63), (183, 65), (184, 65), (181, 67), (185, 74), (179, 67), (180, 67), (183, 74), (182, 74), (184, 74), (183, 75), (184, 75), (185, 75), (184, 77), (183, 78), (182, 77), (182, 78), (182, 79), (183, 80), (183, 81), (182, 80), (183, 79), (188, 83), (188, 84), (189, 84), (189, 83), (190, 84), (197, 81), (198, 79), (198, 80), (207, 74), (210, 77), (207, 75), (228, 65), (194, 60), (193, 62), (186, 62), (185, 61), (191, 61), (192, 62), (194, 62), (193, 65), (203, 74), (209, 74), (215, 82), (234, 62), (251, 81), (253, 82), (251, 86), (252, 86), (252, 85), (251, 88), (252, 90), (253, 91), (252, 91), (251, 90), (250, 91), (251, 92), (249, 90), (249, 91), (248, 92), (249, 89), (250, 89), (248, 89), (249, 88), (250, 90), (248, 90), (245, 92), (246, 93), (246, 92), (247, 93), (245, 93), (253, 90), (254, 91), (254, 92), (254, 93), (253, 93), (253, 94), (255, 92), (255, 91), (257, 91), (257, 94), (256, 95), (257, 95), (257, 96), (256, 97), (257, 97), (258, 96), (256, 94), (261, 91), (262, 90), (261, 90), (263, 90), (263, 91), (264, 91), (263, 92), (261, 89), (263, 89), (264, 88), (263, 88), (262, 89), (265, 91), (266, 92), (267, 93), (268, 92), (267, 92), (267, 91), (268, 91), (268, 90), (267, 90), (266, 90), (266, 91), (267, 89), (266, 89), (267, 88), (268, 87), (268, 89), (266, 87), (265, 88), (265, 87), (264, 89), (264, 92), (263, 93), (264, 93), (265, 93), (265, 92), (266, 93), (266, 94), (267, 94), (265, 90), (264, 90), (256, 88), (254, 86), (252, 87), (251, 87), (249, 92), (248, 93), (247, 92), (251, 91), (250, 93), (251, 93), (251, 94), (250, 94), (252, 89), (253, 87), (250, 92), (244, 96), (246, 98), (246, 97), (245, 98), (246, 99), (245, 100), (245, 101), (246, 101), (247, 100), (244, 99), (244, 98), (245, 99), (247, 98), (240, 100), (241, 97), (244, 100), (243, 100), (243, 101), (240, 101), (239, 102), (238, 101), (239, 100), (237, 102), (236, 101), (235, 100), (235, 98), (235, 99), (234, 100), (234, 99), (234, 98), (233, 98), (233, 99), (232, 98), (232, 99), (231, 99), (232, 97), (231, 97), (233, 100), (232, 100), (233, 101), (232, 102), (231, 101), (228, 100), (228, 97), (218, 98), (215, 96), (216, 97), (215, 97), (214, 96), (213, 97), (212, 97), (214, 98), (215, 99), (214, 101), (215, 102), (214, 103), (215, 104), (215, 105), (214, 106), (213, 107), (214, 108), (215, 107), (216, 108), (217, 108), (217, 107), (216, 107), (215, 108), (216, 106), (214, 107), (214, 109), (213, 109), (213, 108), (213, 110), (213, 111), (212, 110), (213, 112), (212, 113), (213, 113), (212, 114), (213, 114), (214, 114), (214, 113), (215, 114), (215, 113), (216, 113), (216, 112), (215, 112), (214, 111), (215, 110), (214, 112), (211, 113), (210, 113), (211, 114), (211, 115), (212, 116), (213, 115), (212, 115), (211, 116), (212, 117), (213, 116), (214, 115), (214, 116), (215, 115), (216, 114), (210, 114), (210, 115), (210, 116), (210, 117), (211, 118), (210, 119), (210, 118), (209, 117), (208, 116), (207, 115), (208, 115), (207, 114), (207, 116), (206, 117), (206, 118), (205, 118), (205, 117), (206, 116), (206, 115), (205, 115), (205, 116), (204, 115), (203, 116), (202, 115), (201, 114), (202, 113), (202, 114), (200, 113), (203, 113), (201, 113), (201, 112), (202, 112), (203, 111), (203, 112), (204, 112), (204, 111), (202, 110), (204, 113), (205, 114), (206, 113), (205, 113), (202, 111), (201, 111), (201, 110), (200, 110), (199, 111), (198, 112), (198, 111), (198, 110), (197, 109), (197, 110), (196, 111), (195, 112), (194, 112), (193, 112), (193, 111), (192, 111), (192, 112), (191, 111), (190, 112), (189, 112), (190, 113), (191, 114), (192, 114), (193, 114), (194, 113), (193, 113), (194, 114), (192, 113), (193, 110), (191, 110), (191, 112), (190, 111), (189, 111), (189, 110), (188, 110), (187, 109), (188, 109), (187, 110), (186, 110), (187, 108), (186, 109), (185, 108), (184, 107), (184, 106), (183, 106), (183, 105), (184, 105), (182, 107), (182, 108), (183, 107), (185, 106), (185, 105), (186, 104), (187, 105), (186, 106), (186, 107), (187, 107), (184, 104), (185, 103), (185, 104), (187, 104), (186, 105), (184, 103), (183, 102), (183, 103), (183, 104), (182, 103), (181, 104), (182, 104), (182, 105), (181, 106), (180, 105), (181, 105), (182, 106), (183, 108), (182, 109), (183, 110), (182, 111), (181, 110), (180, 110), (181, 109), (183, 109), (179, 106), (178, 107), (177, 106), (177, 105), (178, 104), (179, 104), (178, 105), (178, 106), (176, 105), (177, 107), (177, 108), (176, 108), (177, 109), (178, 110), (177, 111), (178, 111), (177, 110), (176, 110), (176, 111), (176, 112), (175, 113), (174, 114), (175, 114), (176, 113), (177, 112), (177, 113), (178, 114), (179, 114), (178, 115), (178, 116), (177, 117), (176, 116), (177, 115), (177, 116), (175, 116), (176, 115), (175, 115), (176, 114), (177, 114), (174, 116), (174, 117), (175, 117), (175, 118), (176, 119), (176, 118), (176, 117), (179, 117), (179, 115), (179, 116), (178, 113), (178, 112), (179, 113), (180, 112), (181, 113), (182, 114), (183, 113), (183, 112), (184, 113), (185, 112), (185, 113), (186, 113), (186, 112), (187, 112), (186, 111), (188, 111), (190, 110), (190, 114), (189, 113), (188, 113), (187, 114), (187, 113), (188, 112), (187, 111), (184, 114), (186, 114), (186, 115), (187, 115), (188, 116), (188, 115), (187, 116), (186, 117), (186, 116), (185, 116), (184, 117), (185, 115), (185, 117), (187, 118), (187, 117), (184, 116), (188, 117), (189, 117), (188, 118), (187, 119), (188, 119), (189, 120), (190, 120), (191, 120), (190, 121), (190, 119), (190, 118), (191, 117), (189, 118), (184, 118), (185, 118), (185, 119), (184, 119), (183, 119), (183, 118), (182, 119), (182, 120), (182, 118), (184, 120), (186, 119), (186, 118), (189, 116), (190, 117), (192, 118), (192, 117), (193, 118), (194, 119), (195, 120), (194, 121), (194, 120), (195, 121), (195, 122), (194, 123), (193, 122), (192, 122), (192, 121), (194, 124), (194, 125), (193, 125), (194, 126), (195, 127), (195, 128), (196, 127), (196, 126), (195, 126), (195, 129), (195, 130), (194, 130), (194, 129), (194, 131), (193, 130), (193, 129), (193, 128), (192, 129), (192, 130), (192, 131), (192, 132), (192, 133), (191, 133), (192, 134), (193, 132), (191, 132), (190, 131), (191, 130), (193, 131), (195, 131), (196, 132), (197, 132), (197, 131), (196, 131), (195, 132), (194, 132), (193, 133), (191, 131), (190, 132), (190, 133), (190, 134), (194, 128), (193, 127), (196, 128), (197, 126), (197, 127), (198, 128), (197, 128), (196, 129), (196, 130), (198, 125), (197, 125), (196, 125), (195, 125), (196, 124), (199, 124), (200, 123), (200, 124), (201, 122), (202, 121), (203, 122), (204, 122), (203, 121), (203, 120), (200, 121), (199, 120), (199, 119), (200, 119), (198, 120), (197, 120), (196, 121), (195, 119), (196, 119), (197, 121), (196, 118), (197, 117), (195, 118), (195, 123), (195, 124), (193, 123), (193, 124), (193, 121), (194, 122), (192, 123), (191, 123), (191, 124), (190, 125), (189, 124), (190, 123), (189, 122), (190, 122), (189, 123), (190, 124), (188, 122), (188, 123), (187, 122), (188, 121), (189, 121), (188, 120), (189, 119), (187, 121), (186, 120), (185, 120), (185, 121), (183, 120), (183, 117), (190, 116), (191, 116), (192, 116), (191, 115), (189, 115), (190, 115), (192, 115), (193, 116), (194, 115), (193, 115), (194, 116), (193, 117), (183, 114), (183, 115), (184, 115), (183, 116), (182, 117), (181, 117), (182, 116), (183, 121), (182, 121), (181, 122), (182, 123), (183, 122), (182, 122), (181, 123), (182, 124), (183, 123), (184, 122), (185, 122), (184, 121), (180, 116), (178, 117), (180, 113), (181, 114), (181, 115), (180, 114), (180, 115), (181, 116), (182, 115), (181, 112), (179, 112), (182, 112), (181, 111), (180, 111), (180, 109), (181, 108), (181, 107), (180, 104), (179, 103), (180, 103), (178, 103), (177, 102), (178, 102), (177, 101), (176, 102), (176, 101), (176, 100), (177, 100), (177, 99), (178, 99), (178, 100), (177, 98), (177, 97), (176, 98), (176, 97), (175, 96), (176, 96), (175, 97), (174, 98), (175, 99), (176, 95), (174, 96), (173, 96), (174, 95), (175, 95), (175, 94), (175, 93), (176, 93), (175, 92), (174, 91), (173, 91), (172, 91), (172, 92), (172, 90), (171, 90), (170, 90), (169, 91), (168, 90), (170, 91), (171, 91), (171, 92), (172, 93), (170, 92), (171, 93), (170, 94), (171, 94), (169, 93), (169, 94), (170, 95), (169, 96), (170, 97), (169, 98), (170, 98), (168, 95), (168, 94), (171, 89), (173, 90), (174, 89), (174, 88), (175, 87), (174, 87), (174, 86), (173, 87), (173, 86), (172, 86), (171, 87), (170, 88), (171, 88), (172, 87), (172, 88), (172, 89), (169, 92), (174, 90), (173, 89), (175, 89), (176, 90), (176, 89), (176, 88), (176, 87), (177, 87), (176, 86), (175, 86), (177, 85), (177, 84), (176, 83), (175, 82), (175, 83), (176, 82), (174, 83), (176, 81), (177, 80), (176, 79), (177, 78), (177, 79), (178, 79), (178, 80), (179, 80), (180, 80), (181, 79), (184, 82), (183, 82), (182, 81), (182, 82), (184, 81), (187, 83), (181, 59), (174, 55), (174, 54), (174, 53), (174, 52), (173, 53), (173, 52), (173, 51), (174, 50), (175, 51), (175, 50), (174, 49), (173, 50), (173, 49), (173, 48), (172, 50), (172, 49), (171, 50), (174, 51), (175, 49), (172, 51), (171, 51), (170, 52), (171, 52), (172, 53), (171, 53), (170, 53), (169, 53), (169, 54), (170, 54), (172, 54), (172, 52), (171, 54), (171, 55), (172, 55), (173, 55), (173, 54), (170, 50), (170, 49), (170, 51), (169, 49), (170, 48), (170, 47), (170, 46), (168, 48), (171, 48), (171, 47), (171, 49), (168, 54), (167, 54), (167, 55), (166, 54), (163, 45), (162, 44), (164, 45), (161, 43), (159, 42), (160, 44), (157, 46), (157, 43), (156, 46), (155, 47), (156, 47), (155, 46), (152, 44), (151, 44), (154, 48), (154, 44), (153, 44), (153, 43), (154, 41), (152, 40), (153, 42), (153, 41), (155, 41), (157, 39), (157, 38), (156, 39), (156, 38), (158, 37), (159, 38), (160, 39), (159, 40), (158, 39), (158, 38), (157, 41), (152, 39), (154, 42), (156, 37), (153, 34), (151, 34), (153, 28), (155, 17), (154, 18), (154, 17), (155, 16), (155, 15), (154, 16), (153, 16), (153, 14), (153, 12), (154, 13), (155, 14), (156, 15), (157, 14), (156, 14), (154, 14), (154, 15), (156, 16), (157, 15), (152, 17), (151, 17), (149, 24), (149, 23), (145, 25), (145, 33), (145, 34), (143, 35), (148, 38), (149, 37), (151, 39), (150, 39), (148, 37), (146, 38), (149, 36), (150, 34), (150, 35), (165, 39), (167, 41), (168, 41), (167, 40), (166, 39), (166, 42), (166, 41), (165, 42), (164, 42), (165, 43), (165, 44), (166, 49), (164, 55), (167, 51), (166, 53), (167, 52), (167, 53), (168, 53), (168, 52), (169, 52), (168, 51), (167, 48), (169, 46), (169, 45), (170, 45), (172, 46), (173, 45), (174, 44), (175, 45), (174, 43), (173, 43), (173, 44), (173, 46), (169, 44), (168, 44), (168, 43), (167, 44), (169, 51), (166, 44), (166, 43), (161, 54), (165, 56), (165, 57), (166, 58), (167, 59), (167, 60), (168, 60), (169, 60), (169, 61), (169, 62), (170, 62), (168, 61), (167, 61), (170, 60), (171, 59), (170, 59), (172, 59), (171, 58), (172, 60), (171, 61), (173, 61), (173, 62), (174, 61), (172, 63), (171, 63), (171, 62), (172, 62), (172, 61), (173, 60), (174, 59), (173, 59), (172, 58), (173, 57), (173, 56), (174, 58), (178, 63), (181, 62), (180, 61), (183, 64), (199, 83), (198, 84), (197, 85), (197, 84), (198, 85), (199, 85), (199, 84), (199, 86), (198, 87), (197, 88), (196, 88), (197, 87), (198, 89), (198, 88), (197, 89), (197, 91), (196, 92), (196, 93), (197, 94), (198, 94), (198, 95), (199, 96), (199, 95), (199, 94), (199, 93), (200, 94), (201, 95), (200, 96), (200, 97), (200, 98), (201, 97), (202, 96), (201, 96), (203, 95), (204, 96), (203, 96), (202, 97), (201, 98), (202, 99), (201, 99), (202, 100), (203, 101), (204, 100), (204, 99), (203, 99), (202, 98), (199, 98), (198, 99), (197, 98), (196, 98), (197, 97), (198, 98), (195, 99), (196, 99), (197, 99), (196, 97), (195, 97), (195, 96), (194, 98), (193, 99), (194, 100), (193, 100), (192, 101), (191, 100), (191, 101), (192, 102), (193, 102), (190, 100), (190, 99), (190, 98), (189, 98), (188, 98), (188, 97), (187, 96), (188, 96), (189, 96), (190, 95), (189, 94), (189, 95), (191, 94), (192, 93), (193, 93), (193, 94), (192, 92), (192, 89), (193, 88), (192, 87), (191, 88), (191, 89), (191, 87), (192, 86), (193, 86), (194, 86), (194, 87), (199, 87), (200, 87), (198, 86), (197, 86), (195, 89), (200, 91), (200, 93), (201, 94), (200, 95), (198, 92), (197, 92), (198, 93), (197, 93), (196, 94), (195, 93), (195, 94), (194, 93), (196, 89), (199, 88), (199, 90), (194, 88), (192, 88), (193, 87), (201, 92), (196, 95), (197, 96), (198, 97), (200, 99), (201, 100), (203, 98), (204, 97), (203, 97), (205, 98), (199, 97), (201, 93), (200, 86), (200, 85), (196, 86), (196, 85), (189, 85), (190, 88), (201, 88), (202, 87), (203, 85), (202, 86), (202, 88), (202, 91), (202, 95), (204, 95), (205, 96), (205, 95), (205, 97), (204, 98), (203, 100), (202, 102), (201, 101), (202, 101), (204, 101), (205, 101), (205, 100), (205, 99), (206, 100), (206, 99), (207, 98), (208, 99), (207, 99), (208, 97), (208, 98), (213, 95), (212, 88), (216, 88), (222, 91), (223, 93), (220, 92), (221, 92), (217, 88), (217, 87), (217, 89), (214, 94), (213, 96), (214, 97), (211, 101), (212, 99), (211, 98), (210, 98), (210, 97), (207, 97), (209, 99), (209, 100), (210, 101), (211, 102), (210, 102), (209, 101), (211, 103), (211, 104), (212, 104), (213, 104), (212, 105), (213, 105), (214, 105), (213, 106), (212, 106), (214, 104), (212, 109), (212, 111), (211, 112), (211, 111), (210, 112), (209, 111), (208, 110), (207, 109), (207, 108), (207, 110), (208, 111), (208, 109), (209, 110), (209, 112), (208, 113), (208, 114), (209, 114), (209, 113), (209, 115), (207, 113), (207, 112), (206, 112), (206, 114), (208, 112), (207, 111), (206, 111), (205, 111), (205, 110), (204, 110), (205, 109), (203, 110), (204, 109), (204, 108), (205, 108), (206, 109), (203, 109), (202, 108), (203, 107), (203, 106), (204, 107), (201, 109), (200, 109), (199, 109), (199, 110), (198, 109), (200, 111), (200, 114), (199, 113), (197, 111), (196, 112), (195, 113), (196, 113), (197, 114), (197, 113), (196, 114), (195, 115), (196, 116), (198, 117), (199, 117), (200, 117), (200, 118), (201, 120), (200, 120), (201, 119), (198, 121), (198, 122), (199, 123), (199, 122), (198, 123), (198, 124), (197, 123), (197, 124), (198, 126), (199, 127), (199, 126), (199, 125), (200, 125), (201, 125), (200, 126), (201, 126), (201, 127), (200, 128), (201, 129), (201, 128), (202, 127), (203, 127), (203, 128), (202, 128), (202, 126), (202, 125), (203, 125), (203, 126), (199, 128), (198, 129), (199, 129), (198, 130), (198, 131), (199, 132), (200, 131), (199, 130), (199, 131), (198, 132), (197, 130), (194, 133), (197, 129), (189, 132), (188, 131), (188, 132), (187, 131), (186, 131), (187, 132), (186, 132), (187, 133), (189, 131), (191, 134), (189, 134), (191, 129), (190, 129), (191, 128), (192, 127), (193, 126), (194, 134), (195, 133), (197, 133), (198, 133), (197, 134), (196, 134), (198, 134), (196, 133), (192, 128), (190, 130), (198, 127), (192, 126), (192, 125), (191, 125), (191, 127), (196, 123), (196, 122), (197, 122), (198, 119), (199, 121), (196, 120), (194, 118), (195, 117), (195, 116), (196, 115), (195, 114), (197, 115), (197, 116), (198, 116), (196, 117), (198, 114), (198, 113), (200, 112), (200, 115), (201, 115), (199, 114), (200, 116), (201, 117), (202, 117), (201, 118), (202, 116), (202, 118), (203, 117), (204, 116), (203, 115), (204, 114), (205, 112), (203, 118), (202, 119), (202, 123), (202, 122), (204, 121), (201, 123), (202, 120), (203, 119), (199, 116), (197, 119), (197, 118), (194, 117), (191, 118), (191, 119), (192, 119), (195, 111), (198, 115), (199, 118), (194, 127), (190, 126), (192, 124), (191, 122), (187, 120), (186, 122), (186, 121), (191, 126), (189, 125), (189, 126), (188, 127), (189, 128), (188, 128), (189, 129), (188, 129), (189, 130), (189, 133), (188, 133), (188, 134), (185, 131), (185, 132), (185, 133), (186, 133), (187, 134), (188, 130), (190, 127), (195, 134), (200, 133), (201, 132), (202, 131), (202, 132), (203, 133), (203, 132), (203, 131), (204, 132), (204, 131), (202, 130), (203, 129), (204, 130), (205, 130), (205, 129), (204, 128), (202, 129), (201, 130), (199, 133), (200, 132), (201, 131), (200, 129), (201, 124), (200, 122), (201, 121), (204, 118), (204, 119), (205, 120), (204, 120), (198, 118), (187, 130), (187, 129), (186, 130), (185, 130), (185, 129), (186, 128), (185, 127), (185, 126), (184, 127), (186, 127), (187, 126), (186, 126), (187, 127), (187, 125), (188, 125), (188, 124), (187, 123), (181, 119), (180, 122), (179, 122), (178, 123), (178, 122), (179, 123), (178, 124), (179, 124), (178, 121), (177, 121), (179, 121), (177, 120), (177, 122), (177, 123), (176, 124), (175, 125), (174, 124), (175, 124), (174, 126), (174, 125), (175, 127), (175, 128), (176, 129), (177, 130), (178, 130), (179, 130), (180, 131), (178, 131), (178, 132), (177, 132), (176, 131), (176, 132), (177, 131), (178, 133), (179, 134), (178, 134), (177, 133), (176, 133), (175, 132), (174, 133), (173, 133), (172, 132), (173, 132), (174, 131), (174, 132), (175, 131), (173, 130), (175, 130), (178, 129), (178, 128), (178, 127), (179, 126), (179, 127), (180, 127), (178, 126), (181, 128), (181, 129), (182, 130), (183, 129), (182, 128), (182, 127), (183, 126), (184, 126), (184, 125), (184, 128), (184, 129), (185, 128), (183, 127), (182, 126), (183, 125), (185, 125), (181, 127), (182, 129), (183, 130), (182, 131), (181, 132), (182, 132), (181, 131), (180, 130), (180, 129), (179, 129), (176, 134), (175, 133), (173, 134), (174, 134), (175, 134), (173, 131), (173, 129), (174, 130), (174, 129), (175, 129), (176, 130), (177, 129), (177, 128), (176, 128), (176, 127), (176, 126), (176, 125), (177, 126), (179, 128), (177, 125), (178, 125), (177, 124), (180, 124), (181, 125), (182, 125), (180, 125), (180, 123), (181, 124), (178, 120), (177, 119), (178, 119), (179, 120), (180, 119), (181, 120), (179, 119), (183, 124), (181, 126), (180, 126), (180, 128), (174, 128), (174, 127), (173, 127), (173, 126), (173, 125), (173, 123), (172, 123), (172, 124), (173, 122), (174, 121), (173, 121), (174, 122), (172, 122), (177, 127), (173, 128), (172, 127), (171, 126), (172, 125), (173, 124), (172, 126), (171, 128), (172, 129), (175, 126), (174, 123), (175, 123), (176, 123), (176, 121), (175, 122), (172, 121), (171, 121), (171, 122), (171, 123), (171, 127), (170, 125), (170, 126), (169, 127), (168, 127), (169, 128), (167, 127), (167, 128), (168, 126), (167, 126), (166, 127), (166, 128), (165, 129), (164, 129), (163, 129), (164, 130), (163, 130), (162, 129), (162, 131), (162, 130), (163, 131), (163, 132), (164, 131), (165, 132), (164, 133), (163, 133), (165, 131), (165, 130), (162, 132), (165, 133), (164, 132), (161, 133), (162, 133), (163, 134), (162, 134), (160, 134), (159, 133), (158, 134), (157, 134), (156, 133), (155, 134), (155, 133), (156, 134), (154, 133), (155, 132), (154, 131), (153, 131), (152, 131), (151, 132), (150, 132), (150, 131), (151, 131), (153, 130), (152, 129), (151, 130), (150, 129), (150, 128), (149, 127), (151, 128), (150, 130), (151, 129), (152, 128), (151, 127), (152, 127), (153, 127), (153, 129), (154, 130), (154, 129), (155, 130), (156, 130), (157, 130), (156, 131), (155, 131), (153, 132), (154, 132), (156, 132), (157, 133), (156, 129), (155, 129), (155, 128), (154, 128), (153, 128), (152, 130), (152, 132), (153, 133), (152, 134), (151, 134), (151, 133), (149, 133), (150, 133), (149, 134), (150, 134), (152, 133), (153, 134), (154, 127), (149, 128), (148, 129), (149, 130), (149, 131), (148, 132), (147, 131), (146, 130), (147, 133), (147, 132), (146, 133), (145, 134), (144, 133), (143, 134), (144, 134), (146, 134), (145, 133), (144, 132), (143, 131), (142, 132), (142, 131), (143, 132), (142, 133), (143, 133), (144, 131), (145, 132), (147, 134), (148, 133), (149, 132), (146, 132), (146, 131), (145, 131), (147, 130), (145, 130), (144, 129), (144, 130), (145, 129), (144, 128), (145, 128), (146, 127), (147, 127), (147, 128), (148, 127), (148, 128), (147, 129), (146, 126), (145, 127), (146, 128), (146, 129), (148, 130), (148, 131), (148, 134), (149, 129), (147, 126), (145, 126), (146, 125), (146, 124), (147, 123), (145, 124), (146, 123), (147, 124), (145, 123), (144, 124), (144, 122), (143, 121), (143, 122), (142, 121), (141, 121), (141, 120), (142, 119), (143, 119), (144, 120), (144, 119), (145, 118), (144, 118), (145, 119), (146, 119), (142, 118), (143, 117), (142, 116), (142, 117), (141, 117), (144, 117), (143, 118), (143, 120), (145, 121), (144, 121), (145, 122), (146, 122), (146, 121), (144, 123), (146, 120), (147, 119), (145, 120), (145, 117), (144, 116), (143, 116), (144, 115), (144, 114), (143, 113), (145, 116), (146, 117), (147, 116), (148, 117), (149, 117), (149, 118), (149, 119), (149, 120), (150, 121), (149, 121), (148, 121), (150, 122), (151, 122), (149, 122), (148, 122), (147, 122), (147, 121), (151, 121), (151, 120), (150, 120), (151, 119), (150, 119), (150, 118), (151, 117), (152, 116), (152, 117), (151, 118), (152, 118), (150, 116), (150, 115), (150, 114), (149, 116), (148, 115), (147, 115), (147, 117), (146, 116), (146, 115), (145, 115), (146, 118), (147, 120), (148, 123), (148, 124), (149, 125), (150, 125), (149, 123), (149, 124), (147, 125), (148, 125), (150, 124), (148, 120), (148, 119), (141, 122), (140, 121), (139, 122), (138, 122), (138, 123), (140, 122), (142, 122), (147, 118), (149, 126), (150, 127), (143, 130), (154, 134), (154, 126), (155, 127), (155, 126), (156, 127), (155, 125), (156, 124), (156, 123), (155, 124), (157, 123), (158, 124), (158, 125), (159, 126), (158, 126), (160, 126), (161, 126), (161, 125), (159, 125), (160, 125), (159, 124), (159, 123), (160, 122), (161, 122), (162, 122), (163, 121), (164, 122), (164, 121), (163, 122), (163, 123), (164, 123), (164, 124), (162, 123), (161, 123), (159, 122), (158, 121), (157, 121), (156, 122), (155, 123), (156, 126), (156, 125), (157, 126), (157, 127), (158, 127), (156, 128), (157, 128), (159, 127), (159, 128), (157, 132), (157, 131), (152, 126), (151, 126), (153, 126), (160, 124), (160, 123), (158, 123), (157, 125), (154, 125), (154, 124), (153, 125), (154, 123), (153, 124), (152, 124), (152, 125), (148, 126), (148, 118), (150, 117), (149, 115), (148, 116), (148, 114), (147, 113), (147, 112), (148, 111), (148, 112), (148, 113), (149, 112), (150, 111), (149, 110), (148, 109), (149, 109), (150, 109), (151, 110), (150, 110), (149, 111), (146, 113), (146, 112), (146, 114), (145, 114), (147, 114), (149, 114), (149, 113), (151, 115), (151, 114), (150, 113), (152, 115), (152, 113), (151, 112), (152, 112), (153, 113), (153, 112), (153, 111), (154, 110), (153, 109), (154, 108), (154, 109), (152, 109), (153, 110), (154, 111), (152, 111), (150, 112), (151, 113), (151, 111), (152, 110), (151, 109), (155, 107), (155, 106), (153, 107), (154, 107), (156, 107), (157, 107), (156, 108), (155, 109), (155, 108), (156, 106), (153, 108), (152, 107), (151, 107), (152, 108), (151, 116), (151, 123), (151, 125), (151, 124), (150, 123), (152, 122), (152, 121), (153, 122), (154, 121), (153, 120), (153, 121), (152, 120), (153, 119), (152, 119), (145, 125), (142, 134), (141, 134), (141, 133), (140, 133), (140, 134), (139, 133), (140, 132), (141, 132), (141, 131), (141, 130), (139, 131), (140, 131), (139, 130), (139, 129), (140, 129), (142, 129), (142, 130), (138, 131), (138, 130), (138, 129), (139, 128), (140, 127), (141, 126), (141, 127), (140, 128), (140, 125), (139, 126), (139, 127), (138, 126), (138, 127), (138, 128), (140, 130), (141, 129), (143, 129), (142, 128), (143, 128), (142, 127), (143, 126), (142, 125), (143, 125), (144, 126), (144, 127), (143, 123), (139, 120), (138, 120), (139, 119), (139, 118), (140, 119), (141, 118), (152, 114), (153, 117), (154, 116), (153, 115), (155, 116), (156, 116), (155, 117), (156, 115), (157, 116), (156, 117), (157, 118), (156, 119), (155, 119), (154, 119), (153, 118), (152, 123), (153, 123), (155, 122), (156, 121), (158, 120), (159, 120), (160, 121), (161, 124), (158, 128), (159, 129), (160, 128), (160, 129), (161, 128), (162, 127), (163, 128), (163, 127), (163, 126), (164, 127), (164, 126), (162, 128), (161, 127), (160, 127), (159, 130), (160, 130), (158, 129), (157, 129), (158, 130), (159, 131), (158, 131), (159, 132), (160, 132), (161, 131), (161, 132), (161, 130), (160, 131), (158, 132), (158, 133), (159, 134), (160, 133), (164, 128), (162, 126), (161, 134), (161, 129), (166, 130), (166, 129), (165, 127), (165, 128), (166, 126), (165, 126), (167, 129), (166, 131), (166, 132), (162, 125), (162, 124), (165, 121), (164, 120), (163, 124), (163, 125), (164, 125), (165, 120), (166, 119), (165, 118), (164, 118), (163, 117), (162, 116), (163, 115), (164, 116), (163, 116), (164, 115), (163, 114), (162, 113), (161, 113), (162, 114), (162, 115), (165, 115), (164, 114), (165, 114), (166, 113), (165, 113), (164, 113), (161, 114), (160, 115), (159, 114), (158, 115), (158, 116), (159, 117), (160, 118), (159, 119), (160, 120), (158, 118), (154, 117), (153, 116), (157, 117), (158, 117), (157, 114), (156, 113), (156, 114), (155, 115), (155, 118), (154, 118), (154, 122), (150, 126), (143, 127), (142, 126), (143, 124), (142, 124), (142, 123), (154, 120), (139, 134), (139, 132), (138, 132), (138, 134), (138, 133), (141, 128), (141, 125), (139, 125), (138, 125), (138, 124), (141, 124), (140, 126), (139, 124), (139, 123), (139, 121), (138, 121), (140, 124), (140, 123), (142, 120), (145, 112), (144, 112), (144, 113), (143, 114), (142, 113), (141, 114), (140, 115), (139, 114), (139, 113), (138, 114), (138, 113), (139, 112), (140, 112), (139, 111), (140, 110), (141, 109), (141, 108), (142, 107), (143, 108), (144, 109), (144, 110), (145, 110), (145, 109), (146, 109), (145, 108), (144, 107), (143, 106), (143, 107), (142, 106), (143, 105), (144, 105), (145, 104), (146, 103), (147, 104), (146, 104), (145, 103), (144, 104), (143, 104), (142, 104), (141, 104), (140, 104), (141, 105), (140, 106), (143, 103), (144, 102), (144, 103), (142, 105), (142, 103), (140, 105), (144, 101), (143, 101), (143, 102), (145, 102), (146, 101), (147, 101), (146, 100), (145, 101), (146, 102), (147, 103), (148, 104), (145, 100), (145, 99), (147, 99), (147, 102), (144, 106), (145, 107), (146, 108), (146, 106), (145, 106), (146, 107), (145, 105), (147, 106), (147, 105), (148, 102), (148, 100), (148, 101), (149, 102), (150, 103), (149, 103), (150, 104), (149, 104), (151, 103), (152, 103), (153, 103), (151, 104), (152, 105), (153, 105), (154, 106), (153, 104), (154, 104), (155, 103), (156, 102), (157, 103), (157, 102), (158, 102), (158, 104), (157, 105), (156, 105), (155, 105), (154, 103), (155, 102), (154, 102), (153, 102), (152, 101), (152, 100), (152, 99), (152, 98), (151, 97), (150, 96), (151, 96), (151, 98), (151, 99), (152, 97), (153, 97), (152, 96), (153, 95), (154, 95), (155, 96), (154, 97), (156, 96), (156, 97), (154, 98), (153, 96), (152, 95), (153, 94), (153, 93), (152, 94), (152, 92), (153, 92), (153, 91), (154, 92), (155, 92), (155, 93), (154, 94), (154, 93), (155, 94), (156, 95), (155, 95), (156, 94), (157, 95), (155, 97), (153, 98), (152, 102), (150, 102), (151, 101), (151, 102), (150, 101), (153, 101), (154, 101), (155, 101), (156, 100), (157, 99), (157, 101), (158, 100), (157, 100), (158, 101), (159, 102), (159, 101), (160, 100), (161, 101), (162, 101), (163, 102), (162, 102), (164, 101), (164, 100), (165, 101), (165, 102), (164, 103), (165, 103), (164, 102), (163, 101), (163, 103), (161, 100), (162, 99), (163, 98), (164, 97), (164, 98), (164, 99), (165, 100), (166, 100), (167, 100), (167, 99), (166, 98), (167, 98), (167, 97), (166, 97), (165, 97), (161, 102), (163, 100), (165, 99), (162, 97), (163, 97), (164, 96), (163, 96), (162, 96), (161, 97), (160, 97), (159, 97), (159, 98), (158, 99), (159, 99), (158, 98), (157, 98), (156, 98), (159, 100), (160, 99), (161, 98), (160, 98), (162, 98), (161, 99), (159, 96), (160, 95), (159, 95), (158, 94), (158, 93), (159, 94), (160, 94), (161, 94), (162, 94), (162, 93), (161, 95), (162, 95), (163, 95), (164, 95), (164, 94), (163, 94), (161, 93), (161, 92), (162, 92), (163, 92), (164, 91), (164, 92), (165, 92), (164, 93), (163, 93), (165, 94), (166, 95), (167, 94), (167, 93), (166, 94), (168, 93), (173, 92), (170, 89), (170, 87), (170, 86), (170, 85), (170, 84), (171, 84), (170, 83), (170, 82), (171, 82), (171, 83), (172, 83), (172, 84), (173, 85), (174, 85), (175, 85), (169, 90), (169, 89), (168, 88), (167, 87), (167, 86), (167, 85), (168, 85), (168, 86), (169, 86), (168, 87), (169, 87), (171, 86), (175, 90), (176, 91), (176, 92), (177, 92), (177, 91), (178, 90), (179, 91), (180, 92), (179, 93), (178, 93), (180, 94), (180, 93), (180, 95), (181, 96), (181, 95), (182, 95), (183, 94), (184, 93), (185, 92), (184, 92), (183, 93), (182, 93), (181, 93), (182, 92), (183, 92), (184, 91), (185, 91), (185, 90), (184, 89), (183, 88), (183, 89), (184, 90), (182, 87), (181, 88), (183, 86), (183, 85), (183, 84), (184, 84), (185, 85), (184, 85), (185, 86), (186, 87), (185, 87), (184, 86), (186, 85), (187, 85), (188, 86), (188, 87), (188, 85), (189, 86), (187, 87), (188, 88), (186, 86), (187, 86), (201, 86), (201, 87), (203, 88), (201, 89), (196, 91), (195, 92), (189, 88), (188, 89), (188, 90), (187, 90), (187, 89), (189, 91), (190, 92), (189, 92), (190, 91), (189, 90), (188, 93), (188, 92), (191, 91), (191, 90), (195, 91), (187, 88), (190, 89), (190, 90), (189, 89), (188, 91), (186, 90), (187, 91), (186, 92), (186, 91), (183, 90), (182, 90), (181, 90), (182, 89), (183, 87), (182, 86), (182, 85), (181, 85), (181, 84), (181, 83), (180, 82), (179, 81), (178, 81), (177, 81), (178, 82), (177, 82), (178, 83), (176, 85), (176, 84), (177, 86), (178, 86), (178, 87), (177, 88), (178, 88), (179, 88), (180, 89), (179, 87), (180, 86), (181, 87), (179, 86), (178, 85), (178, 84), (175, 88), (173, 88), (175, 84), (179, 85), (180, 84), (182, 88), (186, 88), (185, 88), (181, 89), (184, 88), (185, 89), (184, 87), (182, 91), (183, 91), (185, 94), (184, 94), (186, 89), (190, 87), (182, 83), (182, 84), (180, 88), (179, 89), (174, 84), (174, 82), (174, 81), (175, 80), (175, 81), (174, 79), (175, 78), (175, 77), (174, 78), (173, 79), (172, 79), (172, 80), (171, 81), (172, 82), (173, 82), (173, 81), (174, 80), (173, 80), (170, 80), (171, 79), (170, 81), (169, 80), (168, 80), (170, 79), (169, 81), (168, 82), (169, 83), (173, 83), (169, 82), (168, 81), (167, 80), (167, 79), (166, 78), (168, 78), (169, 77), (170, 77), (171, 78), (172, 77), (173, 78), (176, 78), (175, 79), (176, 80), (176, 77), (174, 77), (174, 76), (172, 78), (173, 77), (177, 73), (177, 72), (176, 72), (176, 73), (176, 71), (175, 71), (174, 72), (173, 71), (174, 71), (175, 73), (175, 72), (174, 73), (174, 74), (173, 75), (174, 75), (173, 74), (172, 73), (173, 73), (172, 74), (172, 76), (171, 75), (171, 76), (170, 75), (170, 74), (169, 74), (169, 73), (170, 72), (170, 71), (171, 72), (170, 73), (171, 73), (172, 72), (171, 71), (170, 70), (170, 69), (169, 69), (171, 70), (169, 70), (171, 68), (170, 67), (169, 66), (168, 67), (167, 68), (166, 68), (168, 69), (166, 67), (167, 66), (167, 65), (166, 65), (168, 65), (168, 66), (169, 65), (170, 65), (171, 66), (172, 65), (173, 64), (171, 64), (170, 66), (167, 64), (168, 64), (167, 67), (168, 68), (165, 68), (165, 67), (164, 66), (163, 66), (162, 67), (161, 66), (163, 68), (162, 68), (162, 69), (161, 68), (163, 65), (164, 64), (165, 63), (165, 61), (165, 60), (166, 60), (168, 62), (166, 62), (167, 63), (166, 66), (165, 65), (169, 64), (170, 64), (172, 64), (171, 65), (169, 63), (168, 63), (170, 63), (170, 61), (171, 60), (174, 60), (175, 60), (172, 57), (171, 56), (170, 55), (167, 43), (167, 42), (162, 41), (161, 41), (163, 40), (165, 41), (165, 48), (164, 52), (162, 60), (162, 61), (163, 62), (164, 63), (163, 63), (166, 61), (168, 59), (173, 65), (173, 66), (174, 66), (174, 67), (174, 68), (173, 68), (173, 67), (172, 68), (172, 69), (170, 68), (169, 68), (169, 67), (166, 63), (167, 62), (166, 64), (164, 65), (165, 64), (160, 61), (157, 60), (154, 65), (156, 66), (157, 66), (158, 66), (157, 67), (155, 69), (152, 67), (150, 67), (150, 68), (149, 68), (149, 71), (149, 72), (147, 74), (148, 75), (148, 76), (149, 75), (149, 74), (150, 75), (150, 76), (145, 73), (145, 74), (144, 74), (145, 75), (144, 75), (143, 75), (143, 78), (143, 79), (142, 77), (143, 76), (142, 78), (141, 78), (140, 78), (141, 77), (142, 79), (141, 79), (140, 77), (141, 76), (142, 75), (143, 74), (142, 73), (141, 73), (140, 72), (140, 73), (141, 72), (140, 71), (138, 74), (138, 73), (138, 75), (141, 74), (143, 65), (144, 63), (146, 62), (147, 61), (146, 60), (147, 58), (147, 62), (146, 63), (142, 60), (141, 59), (140, 60), (139, 60), (139, 59), (138, 59), (139, 61), (140, 61), (140, 62), (141, 63), (142, 62), (141, 62), (138, 61), (139, 62), (140, 63), (139, 63), (140, 64), (140, 59), (138, 60), (138, 57), (138, 58), (140, 58), (141, 58), (141, 61), (142, 63), (142, 61), (142, 64), (141, 64), (142, 66), (143, 62), (138, 62), (138, 63), (138, 64), (147, 64), (147, 69), (147, 70), (150, 74), (153, 75), (151, 77), (151, 74), (146, 73), (150, 69), (146, 70), (147, 76), (146, 74), (139, 79), (138, 79), (139, 80), (140, 79), (138, 81), (138, 82), (138, 83), (138, 84), (139, 85), (140, 84), (141, 85), (142, 85), (143, 85), (140, 86), (139, 86), (138, 85), (138, 86), (138, 87), (139, 87), (139, 88), (138, 88), (138, 89), (139, 90), (140, 89), (139, 91), (138, 92), (140, 90), (140, 91), (138, 90), (139, 89), (141, 91), (142, 90), (143, 89), (143, 90), (144, 90), (143, 91), (142, 92), (143, 93), (144, 93), (145, 94), (144, 94), (145, 93), (146, 94), (147, 94), (146, 93), (144, 95), (143, 94), (142, 93), (143, 92), (142, 91), (144, 89), (143, 88), (144, 88), (145, 87), (144, 87), (145, 88), (146, 89), (146, 90), (147, 90), (148, 90), (147, 89), (148, 88), (147, 91), (148, 92), (149, 91), (148, 91), (150, 90), (151, 90), (150, 91), (152, 89), (151, 89), (152, 90), (150, 88), (149, 87), (149, 88), (149, 89), (148, 89), (149, 92), (151, 91), (151, 88), (150, 89), (153, 90), (154, 89), (153, 88), (152, 88), (153, 89), (154, 90), (155, 91), (156, 90), (155, 90), (154, 91), (154, 87), (155, 87), (155, 86), (154, 86), (154, 85), (153, 85), (152, 86), (151, 86), (150, 84), (150, 83), (151, 82), (153, 84), (154, 84), (153, 86), (153, 87), (154, 88), (156, 87), (156, 86), (157, 88), (158, 87), (159, 87), (158, 88), (157, 86), (156, 88), (157, 89), (158, 90), (159, 90), (160, 89), (159, 89), (160, 90), (161, 89), (162, 88), (161, 87), (162, 86), (161, 86), (162, 85), (161, 85), (160, 85), (160, 84), (161, 84), (160, 83), (162, 84), (163, 86), (164, 85), (165, 86), (166, 85), (165, 85), (164, 84), (163, 83), (163, 85), (164, 86), (166, 86), (165, 84), (166, 84), (165, 87), (160, 86), (163, 84), (162, 87), (163, 87), (160, 87), (162, 83), (161, 82), (160, 82), (161, 83), (162, 82), (161, 81), (159, 82), (158, 82), (158, 83), (159, 83), (159, 84), (159, 86), (159, 85), (158, 84), (158, 86), (157, 87), (156, 89), (155, 89), (155, 85), (154, 83), (155, 82), (158, 89), (155, 88), (150, 87), (149, 90), (146, 91), (147, 92), (148, 93), (149, 94), (150, 93), (147, 93), (145, 92), (146, 92), (146, 95), (145, 96), (145, 95), (144, 92), (142, 94), (141, 94), (141, 93), (140, 94), (139, 93), (139, 92), (140, 93), (140, 92), (138, 91), (141, 89), (141, 90), (142, 89), (138, 93), (138, 94), (139, 95), (138, 96), (139, 96), (139, 97), (140, 97), (138, 95), (140, 96), (139, 94), (139, 98), (138, 99), (138, 100), (138, 98), (139, 99), (140, 99), (141, 98), (142, 98), (143, 99), (143, 100), (144, 100), (144, 99), (147, 100), (146, 99), (146, 98), (145, 98), (144, 98), (150, 100), (149, 101), (149, 100), (147, 98), (148, 99), (153, 106), (154, 105), (155, 104), (156, 104), (152, 104), (151, 105), (151, 106), (155, 111), (147, 110), (146, 110), (146, 111), (145, 111), (144, 108), (146, 105), (147, 107), (148, 107), (148, 106), (149, 107), (150, 106), (149, 108), (150, 107), (148, 108), (150, 108), (151, 108), (154, 113), (155, 114), (154, 114), (153, 114), (154, 112), (155, 112), (156, 112), (157, 112), (157, 113), (158, 111), (159, 112), (157, 111), (155, 113), (156, 111), (156, 110), (157, 109), (156, 109), (157, 108), (155, 110), (157, 110), (158, 110), (159, 109), (159, 110), (158, 109), (159, 108), (159, 107), (160, 107), (160, 108), (161, 109), (162, 110), (162, 109), (162, 108), (163, 108), (163, 107), (162, 107), (164, 107), (164, 106), (163, 106), (165, 108), (164, 109), (164, 108), (163, 105), (164, 104), (164, 105), (165, 107), (165, 109), (166, 109), (167, 109), (168, 108), (169, 109), (168, 109), (168, 110), (167, 110), (167, 111), (166, 112), (165, 111), (165, 110), (164, 110), (163, 109), (161, 110), (160, 109), (160, 110), (159, 111), (160, 111), (160, 112), (161, 111), (159, 113), (160, 113), (160, 114), (159, 116), (157, 115), (156, 118), (148, 110), (147, 111), (145, 113), (142, 115), (143, 115), (142, 114), (142, 112), (141, 115), (140, 114), (138, 112), (140, 111), (139, 110), (141, 112), (141, 113), (140, 113), (141, 111), (142, 110), (143, 110), (143, 109), (142, 111), (143, 112), (143, 111), (139, 116), (138, 116), (138, 115), (140, 116), (139, 117), (140, 118), (140, 117), (141, 116), (154, 115), (159, 118), (158, 122), (165, 125), (166, 124), (167, 125), (168, 124), (169, 123), (169, 122), (170, 122), (170, 121), (169, 121), (170, 123), (173, 120), (172, 120), (173, 119), (174, 119), (174, 118), (173, 118), (172, 119), (171, 119), (171, 118), (171, 120), (170, 119), (169, 118), (169, 119), (168, 119), (167, 119), (167, 118), (166, 117), (166, 118), (165, 116), (166, 116), (166, 115), (165, 117), (167, 115), (167, 114), (168, 116), (168, 115), (169, 115), (167, 116), (167, 117), (166, 114), (170, 116), (169, 116), (170, 117), (171, 116), (171, 115), (172, 116), (173, 115), (173, 117), (169, 120), (168, 120), (167, 120), (168, 121), (170, 120), (167, 121), (168, 122), (167, 123), (167, 122), (166, 122), (166, 123), (165, 123), (163, 120), (163, 119), (163, 118), (162, 119), (164, 119), (162, 117), (164, 117), (167, 112), (168, 111), (168, 112), (167, 113), (168, 113), (169, 111), (169, 112), (170, 113), (166, 110), (165, 106), (166, 107), (167, 106), (167, 105), (166, 105), (167, 104), (168, 104), (166, 106), (167, 107), (168, 106), (169, 105), (170, 105), (170, 106), (171, 106), (172, 106), (173, 105), (174, 105), (175, 106), (176, 107), (175, 108), (176, 109), (179, 111), (186, 108), (184, 108), (185, 107), (185, 111), (193, 119), (193, 120), (199, 115), (205, 121), (205, 122), (205, 123), (205, 124), (204, 123), (205, 125), (206, 126), (207, 127), (206, 128), (205, 127), (204, 127), (204, 126), (205, 126), (207, 125), (207, 126), (208, 128), (209, 129), (208, 129), (209, 128), (210, 127), (211, 128), (210, 128), (211, 127), (212, 127), (213, 126), (212, 125), (213, 124), (212, 123), (211, 124), (210, 125), (210, 126), (207, 128), (208, 127), (209, 127), (208, 130), (208, 131), (207, 130), (209, 132), (209, 133), (210, 134), (210, 133), (211, 133), (211, 134), (212, 134), (212, 133), (212, 132), (211, 131), (212, 131), (213, 131), (213, 132), (212, 130), (212, 129), (213, 130), (214, 131), (214, 130), (215, 131), (213, 134), (213, 133), (214, 134), (215, 134), (216, 133), (216, 132), (216, 131), (217, 131), (218, 130), (218, 129), (219, 129), (217, 130), (220, 129), (221, 128), (220, 128), (219, 127), (219, 128), (218, 127), (218, 128), (220, 126), (219, 126), (219, 125), (218, 124), (218, 125), (220, 124), (219, 124), (218, 123), (217, 123), (216, 122), (215, 121), (215, 122), (216, 123), (216, 124), (216, 125), (217, 126), (217, 125), (219, 123), (219, 122), (218, 122), (217, 122), (217, 121), (218, 120), (219, 121), (218, 121), (216, 120), (217, 120), (219, 119), (220, 118), (219, 117), (218, 118), (218, 119), (217, 118), (216, 119), (216, 118), (217, 119), (218, 117), (217, 117), (220, 116), (220, 117), (221, 117), (222, 116), (223, 115), (224, 116), (224, 115), (223, 116), (223, 114), (222, 115), (223, 117), (223, 118), (222, 118), (221, 118), (224, 114), (224, 117), (224, 118), (225, 117), (222, 117), (221, 115), (221, 114), (220, 115), (219, 116), (218, 115), (219, 114), (219, 115), (219, 118), (219, 120), (215, 119), (215, 120), (214, 119), (214, 118), (213, 119), (212, 118), (211, 119), (209, 118), (209, 119), (209, 120), (211, 117), (213, 117), (216, 115), (215, 111), (217, 113), (218, 112), (219, 111), (218, 111), (217, 111), (218, 110), (219, 109), (219, 110), (218, 109), (220, 111), (221, 112), (222, 111), (223, 110), (223, 109), (224, 110), (223, 111), (223, 112), (222, 113), (222, 114), (221, 116), (218, 114), (217, 115), (217, 114), (218, 113), (219, 113), (219, 112), (220, 113), (221, 113), (218, 116), (216, 117), (215, 118), (213, 120), (212, 120), (213, 121), (214, 122), (213, 123), (214, 123), (215, 124), (215, 123), (217, 124), (216, 126), (218, 126), (220, 123), (221, 124), (222, 125), (223, 124), (222, 124), (223, 123), (221, 125), (220, 125), (222, 123), (222, 126), (223, 125), (224, 124), (224, 125), (225, 126), (226, 126), (225, 127), (224, 127), (223, 128), (224, 128), (225, 128), (224, 126), (226, 128), (227, 128), (227, 127), (227, 129), (227, 130), (228, 131), (229, 130), (228, 130), (226, 131), (225, 132), (226, 132), (227, 133), (227, 134), (228, 134), (228, 133), (226, 133), (228, 132), (229, 131), (229, 132), (229, 133), (230, 132), (230, 133), (230, 134), (229, 134), (228, 129), (229, 128), (228, 128), (229, 127), (228, 126), (229, 125), (228, 125), (227, 126), (228, 127), (229, 126), (229, 124), (228, 124), (227, 123), (226, 122), (227, 124), (230, 124), (229, 123), (230, 122), (229, 122), (228, 122), (228, 121), (228, 123), (226, 123), (225, 123), (225, 124), (223, 126), (222, 127), (221, 127), (221, 126), (222, 128), (223, 129), (219, 130), (217, 129), (216, 130), (217, 132), (218, 131), (216, 129), (215, 128), (214, 127), (214, 128), (213, 128), (212, 128), (213, 127), (212, 126), (211, 129), (210, 129), (209, 130), (207, 129), (215, 127), (214, 129), (215, 130), (215, 126), (215, 125), (217, 128), (216, 127), (216, 128), (215, 129), (213, 129), (211, 130), (211, 132), (210, 131), (216, 134), (217, 134), (218, 133), (218, 132), (219, 131), (219, 132), (220, 133), (220, 134), (221, 134), (220, 132), (221, 131), (222, 131), (223, 132), (222, 133), (221, 133), (221, 132), (221, 130), (222, 130), (222, 129), (221, 129), (220, 127), (223, 130), (224, 129), (224, 130), (222, 132), (223, 134), (224, 134), (225, 134), (223, 133), (224, 133), (227, 132), (227, 131), (226, 127), (223, 127), (220, 130), (224, 123), (225, 122), (224, 122), (224, 121), (223, 121), (222, 120), (221, 119), (220, 119), (220, 120), (220, 121), (221, 122), (221, 123), (220, 122), (215, 116), (215, 117), (210, 120), (211, 120), (211, 121), (210, 122), (212, 122), (213, 122), (214, 121), (214, 120), (217, 116), (224, 119), (225, 118), (226, 117), (225, 115), (225, 116), (226, 116), (214, 124), (214, 126), (217, 127), (225, 125), (227, 125), (230, 126), (231, 127), (232, 126), (233, 126), (234, 126), (233, 127), (233, 128), (232, 127), (231, 128), (231, 129), (232, 130), (231, 130), (230, 130), (230, 129), (230, 128), (231, 131), (230, 131), (231, 132), (229, 129), (232, 128), (234, 129), (235, 128), (236, 127), (235, 127), (235, 126), (234, 125), (237, 128), (236, 128), (237, 127), (236, 126), (236, 125), (235, 125), (237, 126), (238, 125), (238, 124), (237, 125), (237, 124), (236, 124), (236, 123), (235, 123), (235, 122), (236, 121), (236, 120), (237, 120), (237, 119), (238, 118), (239, 117), (240, 116), (241, 116), (241, 117), (240, 117), (239, 116), (239, 118), (240, 119), (240, 120), (239, 121), (239, 122), (239, 123), (239, 124), (239, 125), (240, 123), (240, 124), (241, 125), (241, 124), (240, 125), (241, 123), (238, 126), (238, 127), (239, 128), (240, 127), (241, 127), (240, 128), (241, 129), (241, 128), (242, 127), (242, 128), (242, 126), (243, 127), (241, 126), (240, 126), (239, 126), (237, 129), (238, 128), (239, 127), (239, 129), (240, 129), (241, 130), (244, 126), (244, 125), (243, 125), (242, 125), (243, 126), (242, 124), (243, 124), (244, 124), (242, 123), (243, 123), (244, 123), (245, 124), (245, 123), (246, 124), (247, 124), (248, 123), (247, 122), (247, 123), (246, 123), (246, 122), (245, 121), (244, 121), (245, 120), (246, 120), (245, 119), (246, 119), (247, 119), (248, 118), (247, 118), (248, 117), (249, 117), (250, 116), (250, 117), (251, 116), (250, 118), (249, 118), (252, 116), (253, 116), (253, 117), (252, 118), (253, 118), (254, 117), (255, 117), (256, 117), (257, 118), (258, 119), (259, 118), (260, 117), (260, 118), (260, 119), (261, 120), (260, 121), (260, 122), (259, 121), (258, 120), (258, 121), (257, 120), (259, 119), (261, 118), (262, 118), (263, 117), (262, 117), (263, 116), (262, 116), (264, 116), (264, 115), (264, 114), (264, 113), (263, 114), (263, 115), (262, 115), (262, 114), (264, 117), (261, 116), (265, 116), (265, 115), (265, 117), (266, 118), (267, 117), (268, 116), (266, 117), (267, 118), (268, 118), (268, 119), (266, 119), (265, 118), (264, 118), (267, 119), (268, 117), (267, 116), (267, 115), (265, 114), (266, 114), (266, 115), (266, 116), (267, 114), (267, 113), (266, 113), (265, 112), (268, 112), (268, 111), (267, 111), (267, 112), (268, 110), (267, 110), (266, 109), (267, 108), (268, 108), (267, 109), (265, 108), (264, 108), (265, 109), (266, 108), (266, 107), (267, 106), (268, 105), (267, 104), (268, 103), (268, 104), (268, 106), (267, 105), (266, 105), (265, 106), (264, 105), (265, 105), (266, 104), (265, 103), (264, 103), (264, 104), (263, 104), (263, 103), (262, 103), (262, 102), (263, 102), (265, 104), (266, 103), (267, 103), (265, 107), (264, 102), (264, 101), (265, 100), (266, 99), (266, 98), (267, 98), (267, 96), (263, 96), (266, 95), (267, 95), (268, 96), (268, 95), (263, 95), (264, 97), (265, 99), (262, 99), (263, 99), (263, 100), (262, 98), (268, 97), (266, 88), (268, 88), (265, 89), (262, 91), (262, 88), (258, 95), (257, 98), (256, 98), (255, 99), (255, 98), (255, 97), (254, 98), (254, 99), (253, 97), (254, 97), (253, 99), (252, 99), (252, 100), (251, 99), (251, 100), (252, 101), (253, 100), (254, 100), (255, 100), (255, 101), (256, 101), (255, 102), (254, 103), (254, 104), (254, 105), (253, 103), (252, 102), (253, 102), (253, 105), (252, 106), (252, 107), (252, 108), (252, 109), (253, 109), (251, 108), (251, 109), (250, 107), (251, 106), (252, 105), (253, 104), (254, 106), (255, 105), (255, 106), (255, 104), (255, 103), (254, 102), (253, 101), (254, 101), (256, 102), (251, 105), (252, 104), (251, 104), (250, 108), (250, 109), (250, 110), (249, 110), (248, 111), (248, 110), (249, 111), (247, 110), (246, 109), (247, 109), (247, 111), (249, 109), (251, 110), (252, 110), (248, 108), (247, 107), (247, 106), (248, 106), (248, 107), (246, 106), (245, 105), (245, 106), (246, 105), (246, 104), (246, 103), (246, 102), (247, 101), (246, 100), (247, 99), (246, 96), (247, 95), (247, 97), (248, 96), (249, 96), (249, 95), (251, 95), (252, 96), (253, 96), (252, 95), (252, 92), (256, 87), (257, 87), (259, 96), (254, 84), (253, 84), (238, 64), (239, 62), (237, 61), (237, 63), (236, 62), (235, 63), (210, 86), (210, 84), (211, 86), (212, 85), (211, 85), (212, 84), (209, 86), (210, 82), (208, 84), (207, 86), (208, 86), (206, 84), (207, 85), (208, 83), (184, 66), (185, 65), (208, 77), (212, 79), (215, 86), (216, 95), (215, 95), (219, 86), (262, 69), (264, 72), (268, 67), (264, 74), (265, 74), (265, 73), (266, 74), (266, 73), (265, 72), (267, 75), (268, 77), (265, 75), (261, 76), (264, 71), (267, 66), (268, 66), (263, 65), (261, 48), (260, 47), (261, 47), (265, 48), (265, 47), (264, 47), (266, 48), (265, 53), (268, 50), (267, 51), (268, 54), (239, 91), (237, 90), (241, 94), (242, 97), (243, 97), (248, 98), (249, 97), (247, 96), (244, 101), (243, 102), (243, 103), (242, 102), (244, 104), (245, 103), (244, 102), (240, 102), (239, 101), (241, 102), (244, 103), (242, 103), (243, 104), (243, 105), (242, 105), (242, 104), (241, 103), (241, 106), (242, 106), (241, 107), (242, 108), (243, 108), (242, 107), (243, 107), (241, 105), (243, 109), (244, 108), (245, 108), (245, 109), (244, 109), (244, 110), (244, 107), (245, 110), (243, 110), (246, 107), (246, 108), (246, 110), (245, 107), (244, 105), (244, 106), (243, 106), (241, 104), (240, 104), (239, 103), (238, 104), (237, 105), (238, 105), (239, 104), (238, 103), (237, 103), (235, 102), (235, 103), (236, 103), (234, 104), (234, 105), (235, 104), (233, 105), (234, 107), (233, 107), (234, 108), (235, 109), (236, 110), (234, 109), (236, 108), (235, 108), (236, 107), (235, 106), (235, 105), (236, 104), (236, 105), (237, 106), (238, 106), (236, 106), (235, 107), (234, 106), (232, 104), (233, 103), (234, 103), (233, 104), (230, 102), (231, 102), (227, 104), (226, 108), (227, 108), (227, 109), (227, 110), (228, 110), (228, 111), (228, 109), (230, 108), (231, 109), (226, 107), (227, 105), (228, 107), (228, 106), (229, 105), (229, 104), (232, 101), (234, 102), (229, 107), (220, 106), (221, 106), (221, 107), (222, 108), (223, 107), (226, 104), (224, 109), (225, 110), (226, 109), (231, 98), (228, 98), (220, 107), (217, 109), (214, 110), (215, 109), (216, 110), (216, 111), (217, 110), (219, 108), (216, 109), (220, 110), (221, 108), (220, 108), (220, 100), (219, 99), (215, 98), (216, 98), (219, 87), (232, 89), (230, 96), (227, 90), (228, 90), (231, 89), (232, 91), (231, 91), (242, 61), (241, 61), (240, 62), (240, 61), (237, 62), (237, 60), (237, 64), (236, 63), (200, 51), (218, 30), (218, 33), (219, 30), (222, 25), (220, 24), (220, 23), (221, 24), (223, 24), (224, 24), (224, 28), (229, 25), (227, 25), (227, 24), (225, 33), (224, 33), (226, 32), (227, 29), (228, 29), (227, 28), (217, 28), (228, 30), (232, 37), (232, 36), (233, 37), (234, 38), (234, 37), (235, 38), (235, 39), (236, 40), (237, 40), (236, 41), (237, 41), (229, 40), (226, 52), (238, 82), (237, 91), (242, 93), (238, 91), (231, 96), (233, 97), (240, 103), (241, 96), (248, 97), (248, 99), (249, 99), (249, 100), (249, 101), (248, 100), (248, 101), (249, 98), (250, 98), (252, 98), (257, 99), (258, 98), (258, 97), (259, 97), (258, 89), (267, 77), (268, 76), (268, 78), (267, 73), (268, 74), (267, 72), (265, 94), (260, 97), (260, 98), (261, 97), (261, 98), (261, 99), (262, 100), (260, 99), (259, 100), (256, 99), (257, 100), (256, 100), (257, 101), (257, 102), (256, 103), (258, 101), (260, 101), (260, 100), (259, 99), (259, 98), (259, 101), (259, 102), (259, 103), (258, 103), (257, 103), (256, 104), (256, 105), (257, 104), (252, 103), (251, 102), (250, 101), (251, 103), (250, 103), (250, 102), (248, 102), (247, 103), (248, 104), (247, 102), (258, 100), (258, 99), (261, 101), (262, 101), (261, 102), (263, 106), (264, 106), (263, 107), (264, 107), (262, 105), (261, 105), (261, 104), (260, 103), (259, 104), (258, 104), (260, 104), (259, 105), (258, 105), (257, 106), (251, 101), (250, 100), (250, 99), (251, 98), (252, 97), (251, 97), (251, 96), (252, 94), (252, 93), (253, 92), (252, 80), (268, 93), (268, 94), (268, 98), (267, 99), (266, 100), (267, 100), (266, 101), (265, 102), (266, 102), (265, 101), (264, 99), (264, 100), (261, 100), (265, 98), (268, 99), (267, 102), (267, 101), (268, 101), (260, 96), (249, 103), (249, 102), (250, 104), (257, 105), (259, 106), (259, 107), (258, 107), (257, 108), (256, 108), (257, 107), (258, 108), (259, 109), (260, 109), (259, 108), (258, 109), (259, 110), (259, 111), (260, 112), (261, 112), (260, 113), (260, 114), (259, 114), (260, 115), (261, 115), (261, 117), (261, 119), (262, 120), (262, 121), (263, 119), (262, 119), (263, 118), (264, 119), (263, 120), (263, 121), (264, 121), (261, 114), (265, 119), (265, 113), (263, 113), (266, 112), (266, 111), (266, 110), (267, 107), (268, 109), (268, 113), (265, 111), (264, 110), (263, 111), (262, 110), (262, 109), (263, 110), (264, 111), (263, 109), (264, 109), (263, 112), (264, 112), (262, 104), (261, 106), (261, 107), (262, 107), (262, 108), (263, 108), (265, 110), (262, 111), (261, 108), (260, 107), (260, 105), (260, 106), (262, 106), (263, 105), (260, 108), (260, 110), (261, 110), (258, 110), (257, 110), (257, 111), (258, 111), (257, 109), (256, 106), (247, 108), (248, 109), (244, 111), (243, 112), (242, 111), (242, 112), (241, 112), (242, 109), (241, 108), (240, 107), (240, 108), (241, 109), (240, 109), (239, 109), (239, 110), (238, 111), (237, 111), (236, 111), (237, 112), (238, 112), (239, 112), (240, 111), (239, 111), (240, 110), (238, 109), (239, 108), (238, 110), (237, 109), (239, 107), (239, 106), (239, 105), (240, 105), (240, 106), (237, 98), (237, 107), (238, 107), (230, 98), (230, 111), (227, 111), (226, 111), (227, 112), (226, 113), (225, 113), (224, 112), (225, 111), (226, 110), (224, 111), (225, 112), (222, 109), (221, 110), (220, 109), (221, 109), (221, 111), (222, 112), (222, 110), (211, 110), (211, 109), (212, 108), (215, 106), (212, 107), (211, 106), (211, 107), (211, 108), (210, 109), (209, 109), (210, 110), (210, 111), (206, 110), (212, 112), (209, 116), (208, 117), (208, 118), (207, 118), (205, 119), (207, 117), (207, 119), (206, 119), (206, 120), (206, 121), (207, 120), (204, 117), (208, 120), (209, 121), (208, 121), (208, 119), (207, 121), (208, 122), (209, 122), (207, 122), (204, 124), (203, 123), (206, 107), (206, 108), (203, 108), (204, 106), (205, 106), (205, 105), (206, 106), (206, 105), (205, 104), (206, 104), (206, 103), (207, 104), (208, 105), (207, 105), (209, 106), (208, 106), (210, 105), (210, 108), (212, 119), (216, 121), (214, 117), (213, 118), (216, 116), (217, 112), (216, 100), (226, 112), (227, 113), (228, 114), (230, 114), (231, 114), (232, 113), (232, 114), (233, 113), (232, 112), (232, 111), (233, 112), (234, 113), (231, 111), (231, 112), (228, 115), (229, 115), (228, 116), (229, 116), (230, 117), (230, 118), (231, 117), (231, 118), (231, 119), (232, 118), (233, 119), (234, 118), (234, 119), (233, 118), (233, 117), (234, 116), (234, 115), (235, 115), (233, 116), (236, 116), (235, 117), (236, 118), (236, 119), (237, 117), (238, 116), (238, 117), (239, 115), (239, 119), (238, 120), (237, 121), (235, 121), (236, 122), (237, 122), (238, 123), (240, 121), (241, 121), (238, 121), (234, 128), (234, 127), (235, 129), (235, 130), (233, 129), (232, 125), (231, 125), (231, 124), (230, 123), (231, 122), (229, 121), (227, 122), (227, 121), (226, 120), (225, 120), (226, 121), (225, 121), (230, 121), (231, 123), (231, 126), (233, 124), (234, 124), (230, 125), (230, 127), (232, 129), (234, 130), (234, 131), (234, 132), (235, 133), (236, 133), (235, 134), (236, 134), (237, 133), (237, 132), (237, 131), (236, 132), (236, 131), (238, 134), (235, 132), (238, 135), (239, 135), (239, 134), (240, 134), (238, 136), (241, 133), (242, 134), (241, 134), (242, 133), (241, 132), (240, 132), (240, 133), (239, 133), (239, 132), (239, 131), (240, 130), (244, 127), (244, 128), (243, 129), (243, 128), (245, 126), (245, 127), (246, 127), (247, 128), (246, 129), (247, 130), (248, 130), (247, 129), (245, 130), (245, 131), (246, 131), (245, 129), (244, 129), (244, 122), (243, 121), (242, 121), (243, 120), (244, 120), (244, 118), (245, 117), (246, 116), (247, 115), (246, 121), (247, 121), (248, 120), (249, 121), (248, 122), (248, 121), (249, 120), (250, 119), (249, 122), (250, 123), (249, 124), (250, 124), (249, 125), (248, 126), (248, 127), (247, 127), (246, 130), (248, 131), (249, 131), (250, 130), (249, 132), (248, 133), (249, 134), (247, 133), (248, 134), (247, 132), (248, 129), (248, 128), (247, 131), (246, 132), (248, 132), (246, 133), (245, 133), (244, 132), (244, 133), (243, 132), (244, 131), (243, 133), (245, 132), (242, 132), (241, 131), (240, 131), (238, 133), (238, 132), (238, 137), (238, 138), (239, 139), (239, 140), (240, 139), (240, 138), (239, 138), (239, 137), (239, 136), (238, 139), (238, 140), (239, 141), (238, 142), (237, 141), (236, 140), (235, 140), (236, 139), (237, 139), (237, 140), (240, 140), (241, 139), (241, 140), (242, 140), (241, 141), (242, 142), (243, 143), (244, 143), (245, 142), (244, 141), (244, 140), (243, 141), (243, 140), (242, 139), (243, 138), (243, 139), (244, 138), (244, 139), (245, 140), (245, 141), (246, 142), (247, 142), (247, 141), (246, 141), (244, 142), (241, 138), (243, 142), (242, 143), (242, 144), (242, 141), (241, 143), (240, 142), (239, 143), (239, 142), (238, 143), (237, 142), (236, 141), (237, 143), (236, 143), (236, 142), (235, 144), (234, 145), (235, 146), (234, 146), (235, 145), (236, 144), (236, 145), (236, 146), (237, 147), (238, 148), (239, 149), (238, 149), (240, 148), (239, 148), (238, 147), (238, 146), (237, 145), (237, 146), (234, 147), (234, 148), (234, 149), (235, 150), (235, 148), (236, 149), (237, 150), (237, 149), (235, 149), (236, 150), (235, 151), (235, 152), (234, 153), (235, 153), (236, 153), (237, 152), (236, 151), (237, 148), (236, 148), (240, 147), (240, 149), (241, 148), (242, 148), (241, 147), (241, 149), (239, 147), (238, 145), (235, 147), (233, 148), (233, 147), (232, 147), (233, 146), (232, 146), (231, 146), (231, 148), (232, 148), (233, 145), (234, 144), (233, 144), (234, 143), (233, 142), (233, 141), (232, 142), (231, 142), (231, 141), (233, 140), (232, 139), (233, 138), (234, 138), (235, 138), (234, 139), (235, 139), (236, 138), (237, 138), (237, 134), (234, 133), (244, 134), (250, 133), (250, 132), (251, 131), (250, 131), (251, 130), (252, 129), (253, 128), (254, 127), (255, 127), (256, 128), (257, 129), (256, 127), (257, 128), (258, 128), (258, 129), (257, 127), (259, 127), (260, 126), (260, 127), (259, 128), (258, 127), (258, 126), (258, 125), (257, 124), (257, 123), (257, 125), (259, 126), (259, 125), (260, 124), (260, 123), (259, 123), (259, 124), (258, 123), (261, 123), (262, 122), (261, 121), (260, 120), (258, 118), (257, 119), (259, 117), (259, 116), (265, 120), (266, 120), (263, 122), (264, 123), (265, 124), (264, 124), (263, 124), (263, 125), (262, 124), (261, 124), (261, 125), (260, 125), (262, 125), (261, 126), (262, 127), (263, 127), (264, 128), (264, 126), (265, 125), (264, 125), (263, 126), (265, 127), (266, 127), (265, 126), (266, 128), (264, 127), (266, 126), (266, 129), (265, 130), (264, 129), (265, 128), (265, 129), (264, 130), (265, 131), (266, 131), (267, 132), (268, 132), (267, 131), (268, 130), (267, 130), (267, 129), (262, 123), (261, 122), (259, 120), (258, 117), (260, 116), (258, 115), (258, 116), (257, 117), (256, 119), (256, 116), (255, 118), (254, 118), (254, 119), (254, 120), (255, 121), (253, 120), (253, 119), (252, 119), (251, 120), (251, 121), (252, 120), (253, 121), (254, 122), (253, 123), (254, 121), (251, 119), (249, 116), (252, 121), (251, 122), (252, 123), (253, 122), (255, 120), (252, 117), (252, 115), (252, 114), (251, 113), (251, 114), (250, 115), (251, 117), (255, 116), (256, 118), (256, 120), (257, 121), (255, 119), (255, 122), (256, 123), (255, 115), (256, 114), (257, 115), (256, 115), (255, 114), (254, 114), (255, 113), (256, 113), (257, 113), (258, 114), (258, 113), (261, 113), (261, 111), (262, 112), (261, 109), (256, 109), (256, 107), (255, 107), (258, 112), (256, 112), (260, 111), (258, 106), (261, 103), (263, 101), (254, 95), (254, 96), (255, 95), (256, 96), (253, 95), (250, 95), (250, 96), (258, 102), (255, 96), (260, 102), (255, 108), (254, 109), (255, 110), (254, 107), (253, 107), (253, 106), (253, 108), (250, 111), (251, 111), (250, 112), (249, 112), (250, 106), (249, 106), (249, 108), (247, 112), (247, 113), (247, 114), (248, 114), (246, 115), (245, 115), (245, 114), (244, 115), (243, 114), (242, 113), (244, 112), (243, 113), (244, 113), (245, 112), (243, 111), (242, 110), (241, 111), (237, 110), (238, 113), (239, 113), (239, 114), (238, 115), (237, 116), (237, 118), (238, 119), (238, 122), (237, 123), (245, 128), (246, 128), (243, 130), (244, 130), (239, 130), (238, 141), (237, 144), (232, 144), (231, 143), (232, 143), (233, 143), (235, 142), (234, 141), (233, 139), (234, 140), (238, 130), (237, 130), (236, 129), (233, 130), (233, 131), (232, 132), (231, 133), (233, 133), (233, 134), (233, 132), (234, 134), (235, 131), (236, 130), (232, 131), (232, 133), (231, 134), (233, 125), (235, 124), (234, 123), (234, 122), (233, 122), (233, 123), (232, 124), (232, 123), (230, 120), (230, 119), (229, 118), (228, 118), (229, 119), (228, 120), (228, 119), (229, 120), (226, 125), (238, 131), (226, 124), (214, 125), (213, 125), (212, 124), (211, 125), (210, 123), (211, 123), (211, 122), (206, 124), (208, 126), (209, 125), (210, 124), (209, 124), (208, 123), (209, 123), (206, 122), (207, 123), (206, 123), (207, 124), (208, 124), (209, 126), (206, 129), (206, 127), (211, 126), (208, 125), (200, 127), (186, 129), (186, 123), (185, 124), (185, 123), (184, 123), (186, 124), (187, 124), (183, 128), (184, 130), (184, 132), (183, 133), (183, 132), (184, 131), (186, 125), (184, 124), (183, 131), (184, 133), (181, 130), (181, 133), (180, 132), (179, 132), (179, 131), (180, 133), (179, 133), (179, 125), (188, 126), (187, 128), (186, 134), (184, 134), (183, 134), (182, 133), (181, 134), (172, 128), (171, 125), (170, 124), (172, 130), (171, 129), (170, 129), (180, 134), (182, 134), (185, 134), (177, 134), (172, 131), (171, 131), (172, 133), (171, 134), (170, 134), (169, 134), (168, 133), (167, 134), (167, 133), (168, 134), (166, 133), (168, 132), (168, 131), (167, 131), (167, 132), (169, 130), (168, 130), (168, 129), (167, 130), (164, 134), (144, 125), (141, 119), (144, 111), (158, 114), (159, 115), (160, 116), (161, 115), (161, 112), (162, 111), (163, 111), (163, 110), (166, 108), (169, 108), (170, 109), (171, 109), (170, 108), (169, 107), (171, 107), (170, 107), (171, 105), (172, 104), (173, 107), (174, 106), (175, 107), (174, 107), (174, 108), (174, 109), (173, 109), (173, 110), (173, 111), (172, 111), (172, 112), (171, 113), (170, 114), (169, 114), (170, 115), (168, 114), (169, 117), (168, 118), (168, 117), (166, 120), (165, 119), (162, 121), (161, 120), (162, 120), (165, 122), (161, 121), (157, 122), (155, 121), (141, 123), (138, 119), (140, 120), (138, 118), (138, 117), (147, 108), (149, 105), (149, 106), (142, 108), (142, 109), (141, 110), (138, 111), (139, 115), (148, 105), (141, 106), (140, 107), (139, 106), (138, 107), (138, 106), (138, 105), (138, 104), (138, 103), (138, 102), (138, 101), (139, 102), (139, 104), (139, 103), (139, 101), (140, 100), (141, 101), (140, 101), (140, 102), (140, 103), (141, 107), (140, 109), (139, 109), (139, 105), (141, 102), (142, 101), (142, 100), (142, 99), (145, 97), (144, 97), (146, 97), (147, 97), (146, 96), (148, 98), (149, 97), (148, 96), (148, 97), (149, 99), (149, 98), (151, 95), (154, 96), (150, 95), (151, 94), (155, 98), (158, 97), (159, 103), (159, 104), (159, 105), (160, 105), (159, 106), (158, 105), (158, 106), (160, 106), (161, 105), (161, 106), (160, 104), (162, 104), (162, 103), (163, 104), (165, 104), (165, 105), (166, 104), (167, 103), (168, 102), (169, 103), (170, 103), (171, 102), (172, 102), (173, 103), (174, 104), (174, 103), (175, 104), (175, 103), (175, 101), (174, 99), (173, 98), (173, 99), (173, 100), (172, 101), (174, 101), (173, 101), (174, 102), (175, 102), (173, 104), (172, 105), (169, 106), (168, 105), (168, 107), (167, 108), (166, 102), (162, 106), (162, 105), (164, 111), (161, 108), (161, 107), (158, 107), (162, 112), (161, 117), (161, 118), (160, 119), (160, 117), (157, 119), (158, 113), (163, 112), (163, 113), (164, 112), (152, 106), (150, 105), (157, 106), (157, 104), (158, 103), (158, 108), (155, 120), (156, 120), (158, 112), (161, 116), (168, 123), (169, 124), (169, 125), (165, 134), (166, 134), (169, 133), (169, 132), (170, 133), (168, 128), (170, 128), (176, 120), (175, 120), (175, 121), (178, 118), (179, 118), (180, 117), (180, 118), (169, 126), (170, 127), (170, 130), (169, 129), (169, 131), (170, 132), (171, 130), (171, 132), (172, 134), (171, 124), (157, 124), (165, 124), (166, 121), (161, 119), (158, 119), (159, 121), (171, 133), (181, 121), (191, 121), (192, 120), (185, 114), (189, 114), (188, 114), (191, 113), (185, 110), (184, 110), (184, 111), (184, 112), (181, 118), (175, 119), (174, 120), (177, 118), (180, 121), (180, 120), (189, 127), (204, 125), (200, 130), (204, 129), (205, 128), (206, 130), (207, 131), (210, 130), (209, 131), (208, 132), (207, 132), (206, 133), (206, 132), (206, 131), (202, 124), (202, 133), (201, 134), (193, 134), (190, 128), (201, 133), (200, 134), (199, 134), (203, 134), (204, 133), (205, 132), (207, 134), (209, 134), (208, 133), (207, 133), (205, 134), (202, 134), (183, 111), (184, 109), (180, 108), (179, 107), (178, 108), (179, 108), (179, 109), (179, 110), (178, 109), (173, 106), (172, 107), (173, 108), (172, 109), (171, 108), (172, 110), (174, 112), (175, 111), (174, 110), (175, 110), (174, 111), (173, 112), (175, 109), (172, 113), (171, 112), (170, 112), (169, 113), (173, 116), (172, 115), (172, 117), (174, 115), (173, 113), (172, 108), (169, 110), (170, 111), (166, 111), (173, 114), (172, 114), (171, 114), (172, 118), (185, 109), (182, 113), (180, 106), (180, 107), (181, 102), (182, 102), (177, 104), (176, 106), (175, 112), (176, 104), (177, 103), (176, 99), (177, 95), (178, 94), (179, 94), (179, 95), (179, 92), (181, 92), (181, 94), (185, 95), (184, 95), (182, 96), (183, 95), (186, 96), (185, 97), (185, 98), (184, 97), (184, 96), (185, 96), (183, 96), (183, 97), (186, 95), (180, 85), (183, 83), (180, 83), (179, 83), (181, 82), (180, 79), (181, 80), (179, 82), (177, 83), (175, 75), (175, 74), (176, 74), (173, 72), (175, 70), (173, 70), (181, 76), (179, 78), (180, 81), (180, 78), (179, 79), (179, 84), (180, 90), (181, 91), (181, 86), (173, 76), (172, 67), (171, 67), (171, 69), (169, 71), (168, 72), (168, 73), (168, 74), (167, 75), (166, 75), (165, 76), (164, 76), (163, 76), (162, 77), (164, 75), (165, 75), (165, 74), (166, 74), (165, 73), (167, 73), (166, 73), (166, 72), (167, 72), (167, 71), (168, 70), (167, 74), (166, 76), (166, 77), (167, 77), (167, 76), (165, 78), (164, 78), (169, 79), (167, 81), (166, 81), (166, 80), (165, 80), (164, 79), (168, 79), (166, 82), (167, 82), (169, 84), (167, 84), (168, 83), (169, 78), (170, 78), (170, 76), (171, 77), (169, 75), (171, 74), (172, 75), (174, 70), (172, 71), (169, 72), (169, 76), (168, 77), (167, 78), (165, 77), (168, 76), (174, 69), (175, 69), (173, 69), (175, 68), (172, 70), (172, 66), (175, 65), (176, 65), (177, 67), (178, 64), (179, 65), (178, 65), (177, 68), (176, 67), (177, 64), (170, 58), (169, 59), (173, 63), (175, 67), (167, 69), (166, 69), (164, 68), (165, 70), (164, 67), (165, 66), (160, 62), (161, 62), (161, 59), (158, 58), (157, 59), (157, 58), (156, 63), (150, 60), (150, 59), (150, 58), (150, 57), (151, 59), (149, 58), (148, 59), (148, 58), (147, 57), (147, 56), (146, 57), (148, 57), (146, 56), (145, 56), (146, 58), (145, 55), (155, 49), (149, 59), (146, 35), (146, 39), (154, 43), (152, 43), (154, 45), (156, 42), (158, 43), (161, 45), (161, 42), (160, 41), (159, 41), (162, 38), (161, 38), (160, 40), (159, 39), (158, 40), (158, 41), (157, 37), (148, 35), (147, 34), (146, 31), (147, 30), (148, 30), (148, 29), (148, 28), (149, 28), (149, 29), (148, 32), (147, 33), (146, 33), (145, 32), (138, 20), (139, 20), (138, 24), (149, 32), (148, 33), (147, 32), (146, 32), (144, 22), (141, 20), (140, 20), (140, 19), (138, 16), (138, 18), (140, 18), (149, 20), (144, 14), (143, 13), (142, 12), (142, 11), (143, 10), (141, 10), (140, 8), (141, 8), (140, 9), (142, 8), (141, 9), (142, 9), (140, 7), (139, 8), (138, 8), (139, 7), (139, 6), (138, 5), (139, 5), (138, 6), (140, 5), (141, 6), (141, 7), (140, 6), (141, 5), (139, 11), (139, 12), (142, 10), (141, 12), (138, 9), (143, 9), (142, 7), (149, 14), (154, 12), (155, 12), (156, 13), (156, 12), (157, 12), (157, 11), (156, 11), (156, 10), (156, 9), (156, 8), (157, 9), (157, 8), (158, 7), (158, 8), (157, 7), (155, 9), (154, 8), (153, 8), (152, 7), (152, 8), (153, 9), (154, 7), (154, 6), (153, 7), (152, 6), (153, 6), (152, 5), (153, 5), (157, 5), (155, 10), (154, 10), (154, 11), (155, 13), (158, 14), (160, 14), (159, 13), (159, 12), (158, 12), (157, 13), (153, 13), (158, 13), (159, 11), (158, 11), (151, 5), (151, 6), (150, 6), (150, 5), (149, 5), (147, 5), (148, 5), (142, 5), (142, 6), (138, 11), (138, 7), (138, 10), (148, 18), (149, 17), (149, 19), (150, 20), (151, 19), (150, 16), (150, 8), (149, 7), (149, 8), (150, 7), (151, 7), (149, 6), (143, 12), (143, 15), (143, 14), (142, 13), (144, 13), (141, 13), (140, 14), (141, 15), (138, 13), (138, 12), (142, 14), (149, 18), (150, 18), (151, 18), (151, 16), (150, 19), (164, 38), (168, 40), (171, 42), (174, 45), (168, 56), (168, 55), (167, 56), (168, 57), (169, 55), (165, 58), (163, 57), (161, 55), (172, 47), (172, 48), (171, 46), (157, 42), (158, 42), (173, 47), (177, 48), (176, 49), (176, 48), (175, 46), (175, 44), (176, 43), (177, 44), (176, 44), (175, 43), (176, 45), (178, 43), (179, 42), (178, 42), (176, 42), (175, 42), (178, 45), (177, 45), (176, 40), (177, 39), (177, 41), (178, 39), (179, 44), (180, 44), (185, 43), (184, 47), (203, 82), (219, 53), (223, 53), (235, 62), (254, 94), (249, 93), (227, 86), (225, 87), (180, 5), (175, 8), (178, 6), (174, 8), (172, 8), (172, 9), (172, 10), (173, 11), (206, 69), (207, 69), (179, 46), (179, 45), (178, 46), (195, 5), (217, 14), (225, 10), (225, 9), (226, 8), (225, 8), (226, 7), (225, 7), (224, 8), (224, 11), (225, 11), (226, 10), (227, 9), (228, 9), (230, 9), (229, 11), (227, 10), (228, 12), (225, 12), (218, 15), (220, 22), (221, 22), (218, 7), (219, 6), (227, 22), (227, 23), (225, 22), (227, 21), (226, 23), (226, 24), (225, 24), (232, 38), (233, 39), (231, 38), (232, 39), (236, 36), (237, 37), (237, 38), (236, 37), (234, 39), (233, 38), (235, 37), (236, 35), (237, 35), (237, 36), (238, 38), (237, 39), (238, 41), (236, 38), (234, 47), (233, 48), (237, 54), (239, 56), (240, 58), (247, 59), (239, 54), (238, 55), (207, 81), (208, 81), (206, 85), (211, 78), (211, 79), (210, 79), (241, 62), (239, 55), (239, 36), (236, 33), (237, 32), (237, 30), (237, 31), (238, 32), (239, 31), (240, 31), (241, 32), (242, 31), (248, 31), (248, 30), (245, 33), (251, 29), (248, 26), (244, 26), (254, 21), (256, 23), (255, 24), (256, 25), (254, 20), (253, 19), (253, 18), (255, 21), (256, 20), (257, 20), (255, 20), (254, 19), (255, 18), (255, 19), (251, 19), (250, 14), (249, 14), (248, 17), (245, 17), (244, 17), (245, 16), (242, 21), (241, 21), (240, 20), (240, 19), (239, 20), (238, 19), (237, 19), (239, 19), (238, 18), (238, 17), (238, 16), (236, 18), (239, 18), (243, 17), (244, 16), (245, 15), (243, 16), (241, 12), (241, 11), (240, 11), (238, 11), (239, 12), (239, 15), (238, 15), (243, 8), (244, 7), (244, 6), (244, 5), (245, 6), (246, 6), (246, 5), (247, 6), (246, 7), (245, 7), (244, 8), (243, 5), (242, 6), (243, 6), (242, 7), (242, 8), (245, 10), (246, 9), (247, 9), (247, 8), (248, 8), (246, 8), (247, 10), (246, 10), (241, 16), (239, 16), (239, 17), (241, 9), (246, 11), (247, 7), (245, 8), (245, 9), (244, 15), (245, 14), (247, 17), (247, 19), (248, 21), (257, 19), (258, 19), (261, 22), (262, 21), (260, 19), (262, 20), (263, 20), (265, 23), (266, 20), (266, 19), (265, 18), (266, 18), (265, 19), (267, 20), (268, 19), (267, 18), (268, 18), (267, 17), (267, 19), (265, 20), (264, 19), (263, 18), (264, 18), (264, 17), (263, 17), (263, 16), (262, 15), (261, 15), (260, 16), (261, 16), (261, 17), (260, 14), (259, 14), (259, 13), (259, 12), (260, 12), (260, 13), (261, 14), (261, 13), (261, 12), (262, 13), (263, 12), (264, 11), (264, 10), (265, 9), (263, 11), (262, 12), (265, 12), (266, 11), (267, 12), (267, 11), (265, 13), (265, 14), (264, 13), (266, 13), (266, 12), (267, 14), (267, 13), (268, 12), (268, 11), (268, 10), (268, 13), (266, 14), (264, 14), (263, 14), (264, 12), (263, 15), (262, 16), (262, 18), (262, 17), (259, 16), (258, 15), (257, 14), (256, 13), (255, 13), (255, 12), (256, 11), (256, 12), (257, 13), (258, 12), (258, 13), (259, 15), (260, 15), (258, 14), (263, 13), (262, 14), (265, 15), (264, 15), (265, 16), (265, 17), (266, 17), (267, 16), (268, 17), (266, 16), (268, 15), (268, 16), (267, 15), (266, 15), (268, 14), (264, 16), (266, 21), (268, 21), (267, 10), (268, 9), (266, 10), (265, 10), (265, 11), (267, 9), (267, 8), (268, 7), (267, 6), (266, 5), (265, 5), (267, 5), (268, 6), (268, 8), (263, 10), (262, 9), (263, 9), (260, 11), (259, 11), (258, 11), (257, 11), (257, 10), (255, 10), (256, 9), (255, 7), (256, 6), (257, 5), (257, 6), (255, 5), (254, 5), (253, 5), (252, 6), (252, 13), (253, 13), (255, 14), (257, 12), (256, 14), (249, 5), (250, 5), (248, 5), (247, 5), (248, 6), (249, 6), (252, 5), (253, 6), (251, 6), (249, 9), (248, 9), (249, 10), (247, 11), (245, 5), (239, 7), (240, 7), (240, 6), (239, 6), (241, 5), (241, 6), (235, 9), (237, 11), (238, 14), (237, 14), (237, 15), (235, 16), (231, 10), (235, 8), (231, 9), (230, 8), (229, 8), (228, 7), (226, 5), (223, 5), (222, 5), (222, 6), (223, 6), (221, 6), (221, 7), (220, 9), (219, 8), (219, 7), (220, 6), (220, 7), (220, 5), (220, 8), (229, 20), (231, 22), (237, 27), (237, 28), (237, 29), (235, 32), (234, 33), (233, 33), (232, 32), (231, 32), (231, 33), (232, 34), (235, 34), (235, 33), (238, 31), (236, 34), (233, 36), (239, 32), (240, 33), (240, 34), (239, 38), (238, 40), (238, 39), (252, 40), (258, 35), (260, 36), (254, 30), (257, 26), (259, 26), (265, 26), (253, 29), (241, 33), (240, 32), (240, 30), (246, 31), (242, 27), (248, 18), (252, 21), (254, 18), (255, 17), (255, 16), (256, 16), (255, 15), (254, 15), (253, 16), (253, 15), (252, 15), (254, 16), (254, 17), (253, 14), (254, 14), (256, 15), (256, 17), (253, 17), (252, 17), (252, 16), (256, 19), (256, 18), (250, 44), (255, 43), (255, 44), (260, 42), (259, 43), (258, 44), (254, 41), (268, 25), (268, 24), (267, 31), (247, 33), (244, 37), (236, 32), (245, 38), (243, 37), (247, 32), (251, 46), (251, 45), (250, 46), (250, 47), (249, 46), (250, 45), (252, 47), (252, 48), (255, 48), (266, 47), (264, 45), (264, 46), (263, 43), (265, 46), (267, 46), (267, 47), (246, 54), (249, 52), (249, 53), (237, 58), (247, 57), (249, 94), (248, 94), (246, 94), (247, 94), (248, 95), (245, 104), (247, 105), (247, 104), (249, 107), (248, 105), (242, 95), (242, 94), (241, 110), (244, 114), (243, 116), (244, 116), (243, 115), (247, 116), (246, 117), (247, 117), (248, 116), (248, 113), (249, 113), (250, 114), (251, 115), (252, 113), (253, 114), (254, 113), (253, 113), (252, 112), (252, 111), (253, 110), (254, 111), (253, 111), (254, 110), (256, 110), (259, 112), (257, 114), (257, 112), (254, 108), (268, 100), (253, 98), (268, 102), (268, 107), (266, 106), (255, 112), (254, 112), (253, 112), (255, 109), (255, 111), (254, 115), (250, 120), (250, 121), (248, 119), (248, 115), (249, 115), (249, 114), (250, 113), (251, 112), (253, 115), (250, 105), (249, 104), (251, 107), (259, 113), (257, 116), (254, 116), (257, 122), (258, 122), (258, 124), (256, 121), (256, 122), (254, 123), (256, 126), (257, 126), (262, 126), (266, 130), (268, 129), (268, 128), (268, 127), (267, 127), (268, 126), (267, 128), (266, 133), (265, 132), (266, 132), (267, 133), (268, 134), (268, 133), (267, 134), (264, 131), (264, 132), (265, 133), (265, 134), (266, 134), (263, 131), (264, 134), (263, 130), (263, 129), (262, 128), (261, 127), (261, 128), (260, 128), (260, 129), (264, 120), (259, 115), (251, 118), (249, 119), (246, 118), (249, 123), (250, 122), (251, 123), (252, 122), (248, 125), (250, 126), (246, 125), (247, 125), (248, 124), (245, 122), (242, 120), (241, 119), (241, 122), (235, 120), (234, 121), (245, 125), (246, 114), (246, 113), (246, 112), (246, 111), (248, 112), (240, 113), (240, 114), (241, 113), (241, 114), (242, 114), (241, 115), (242, 115), (245, 118), (243, 118), (244, 117), (247, 120), (262, 113), (265, 122), (265, 123), (264, 122), (263, 123), (265, 121), (255, 126), (256, 125), (255, 125), (254, 126), (254, 125), (253, 124), (254, 124), (252, 124), (252, 125), (251, 126), (249, 126), (246, 126), (242, 129), (242, 130), (238, 129), (243, 122), (242, 122), (249, 130), (243, 131), (245, 134), (243, 134), (246, 134), (247, 134), (242, 131), (249, 127), (251, 125), (251, 124), (255, 123), (243, 119), (244, 119), (245, 116), (243, 117), (242, 116), (245, 111), (250, 125), (251, 127), (251, 128), (252, 128), (252, 127), (253, 126), (253, 125), (252, 126), (253, 127), (255, 124), (256, 124), (254, 128), (250, 128), (250, 127), (249, 129), (249, 128), (250, 129), (251, 132), (251, 133), (252, 132), (249, 133), (247, 126), (240, 141), (242, 138), (246, 140), (247, 140), (248, 142), (248, 141), (249, 140), (250, 141), (250, 142), (251, 141), (252, 141), (252, 142), (253, 142), (251, 140), (250, 140), (251, 139), (250, 139), (249, 139), (248, 140), (248, 139), (241, 144), (240, 143), (239, 144), (238, 144), (239, 145), (239, 146), (236, 147), (238, 150), (238, 151), (237, 151), (236, 152), (235, 143), (234, 142), (232, 145), (231, 145), (231, 147), (230, 147), (230, 144), (230, 142), (230, 146), (230, 148), (229, 148), (228, 148), (228, 147), (227, 147), (228, 146), (227, 145), (227, 144), (226, 144), (229, 146), (230, 145), (240, 146), (241, 146), (240, 145), (241, 145), (243, 144), (244, 144), (245, 144), (243, 145), (245, 145), (244, 145), (244, 146), (245, 143), (241, 142), (235, 141), (231, 139), (230, 140), (229, 141), (229, 142), (228, 142), (228, 141), (227, 142), (227, 143), (228, 143), (229, 144), (228, 144), (227, 146), (229, 140), (229, 139), (230, 138), (230, 139), (229, 138), (228, 139), (228, 140), (226, 141), (227, 140), (226, 140), (227, 139), (226, 138), (227, 141), (228, 138), (225, 139), (226, 139), (225, 140), (224, 140), (225, 141), (226, 142), (225, 142), (224, 142), (225, 143), (225, 144), (224, 143), (223, 143), (222, 142), (221, 142), (222, 141), (222, 140), (221, 139), (222, 138), (223, 138), (222, 139), (221, 138), (220, 139), (220, 138), (224, 138), (223, 139), (224, 139), (223, 140), (223, 141), (224, 141), (225, 138), (219, 138), (219, 139), (218, 138), (218, 139), (218, 140), (219, 141), (219, 140), (217, 139), (216, 140), (215, 140), (214, 141), (213, 142), (213, 141), (213, 140), (214, 140), (212, 141), (211, 142), (212, 142), (211, 143), (210, 142), (209, 143), (209, 142), (208, 142), (210, 143), (209, 144), (208, 141), (207, 141), (206, 141), (205, 140), (206, 139), (204, 141), (205, 141), (206, 140), (207, 139), (205, 138), (207, 142), (209, 141), (210, 141), (210, 140), (208, 140), (208, 139), (207, 140), (209, 139), (208, 138), (207, 138), (206, 142), (209, 140), (210, 139), (210, 138), (211, 139), (211, 138), (211, 140), (206, 138), (205, 139), (208, 143), (207, 143), (205, 142), (204, 142), (204, 138), (204, 139), (203, 140), (202, 141), (201, 140), (202, 140), (203, 141), (202, 142), (204, 140), (201, 142), (201, 141), (203, 143), (203, 144), (204, 143), (205, 144), (210, 144), (212, 143), (211, 144), (212, 144), (212, 145), (212, 146), (211, 145), (209, 145), (208, 145), (207, 145), (206, 144), (208, 146), (207, 146), (206, 147), (205, 147), (205, 146), (204, 146), (203, 146), (202, 145), (202, 144), (203, 142), (202, 139), (204, 144), (205, 143), (206, 145), (205, 145), (206, 146), (209, 146), (210, 146), (209, 147), (208, 147), (207, 148), (203, 145), (202, 146), (201, 146), (201, 145), (201, 144), (200, 145), (200, 146), (199, 145), (199, 144), (199, 146), (200, 147), (200, 148), (201, 149), (202, 149), (203, 150), (202, 151), (202, 152), (202, 153), (201, 152), (203, 151), (202, 150), (204, 150), (202, 148), (201, 147), (202, 147), (204, 145), (208, 144), (207, 144), (206, 143), (202, 143), (203, 148), (200, 150), (200, 149), (199, 150), (199, 149), (200, 151), (200, 152), (200, 153), (201, 153), (201, 154), (202, 154), (201, 155), (200, 155), (201, 156), (199, 154), (200, 154), (199, 156), (198, 155), (198, 156), (198, 157), (197, 156), (196, 155), (196, 154), (197, 155), (199, 157), (198, 158), (200, 156), (200, 157), (201, 157), (200, 158), (201, 158), (199, 158), (197, 158), (197, 157), (196, 156), (195, 155), (197, 154), (194, 155), (195, 156), (196, 157), (199, 159), (200, 160), (199, 161), (200, 161), (201, 162), (201, 161), (202, 161), (203, 161), (203, 162), (204, 161), (205, 160), (205, 162), (204, 162), (202, 162), (205, 163), (204, 163), (206, 164), (207, 163), (208, 163), (208, 162), (209, 162), (209, 163), (208, 164), (207, 164), (206, 163), (207, 162), (206, 162), (207, 165), (208, 165), (209, 166), (206, 161), (207, 161), (206, 160), (207, 160), (202, 163), (201, 164), (202, 164), (200, 163), (199, 162), (198, 163), (198, 162), (197, 163), (197, 162), (196, 162), (196, 163), (195, 163), (196, 164), (197, 164), (198, 164), (199, 165), (199, 164), (198, 165), (197, 165), (197, 166), (196, 167), (196, 166), (198, 166), (199, 163), (200, 162), (202, 160), (203, 159), (204, 159), (203, 158), (202, 159), (203, 160), (199, 160), (200, 159), (195, 158), (194, 158), (194, 157), (194, 156), (195, 157), (193, 158), (193, 157), (193, 156), (192, 156), (193, 155), (192, 158), (191, 157), (191, 156), (192, 157), (192, 155), (191, 154), (192, 153), (190, 154), (190, 157), (190, 156), (189, 155), (188, 155), (187, 156), (188, 157), (187, 157), (188, 156), (187, 155), (187, 154), (186, 153), (185, 154), (185, 153), (187, 152), (188, 151), (189, 150), (188, 150), (188, 149), (187, 150), (186, 150), (187, 149), (188, 148), (186, 149), (185, 149), (187, 148), (187, 147), (188, 146), (189, 147), (190, 148), (189, 148), (190, 149), (190, 147), (191, 146), (190, 145), (190, 146), (189, 149), (191, 149), (192, 150), (191, 150), (190, 150), (193, 149), (194, 149), (194, 148), (194, 147), (194, 146), (195, 146), (196, 146), (194, 145), (193, 146), (194, 144), (193, 143), (193, 142), (192, 142), (192, 141), (191, 141), (192, 140), (193, 140), (193, 139), (194, 141), (194, 142), (195, 143), (195, 145), (196, 144), (196, 145), (193, 145), (195, 148), (195, 149), (196, 147), (195, 144), (196, 143), (196, 142), (197, 143), (195, 141), (196, 140), (195, 140), (196, 141), (197, 139), (198, 138), (199, 139), (200, 139), (199, 138), (198, 139), (197, 140), (196, 138), (197, 138), (196, 139), (195, 139), (197, 141), (198, 140), (198, 141), (199, 140), (200, 140), (201, 139), (200, 141), (201, 143), (200, 143), (199, 143), (198, 143), (198, 142), (197, 142), (199, 141), (194, 140), (193, 141), (191, 140), (190, 140), (189, 139), (188, 139), (187, 139), (187, 140), (186, 140), (185, 140), (184, 140), (183, 141), (184, 141), (184, 142), (184, 143), (185, 141), (186, 142), (185, 143), (184, 144), (184, 145), (185, 145), (183, 143), (182, 143), (181, 143), (182, 144), (180, 142), (181, 142), (180, 141), (181, 141), (182, 142), (182, 141), (183, 142), (183, 144), (183, 145), (182, 146), (182, 145), (183, 146), (184, 146), (185, 147), (184, 147), (185, 148), (186, 147), (188, 147), (192, 149), (193, 148), (191, 151), (187, 146), (189, 146), (188, 145), (187, 145), (187, 144), (189, 145), (191, 147), (192, 147), (196, 149), (197, 150), (198, 149), (197, 148), (201, 148), (203, 149), (199, 148), (199, 147), (198, 145), (197, 146), (197, 147), (198, 148), (197, 149), (196, 148), (193, 147), (198, 150), (198, 144), (197, 144), (197, 145), (198, 146), (204, 149), (201, 150), (203, 152), (199, 151), (198, 151), (197, 151), (198, 152), (197, 153), (196, 153), (198, 153), (199, 153), (199, 152), (197, 152), (201, 151), (204, 151), (198, 147), (200, 144), (207, 147), (210, 147), (210, 145), (205, 148), (206, 148), (211, 141), (212, 140), (209, 138), (213, 143), (213, 144), (214, 143), (214, 144), (213, 145), (214, 145), (213, 147), (212, 148), (212, 147), (211, 146), (213, 146), (213, 148), (205, 149), (204, 148), (204, 147), (203, 147), (196, 150), (195, 147), (192, 148), (192, 145), (193, 144), (194, 143), (191, 145), (192, 144), (191, 142), (195, 138), (194, 138), (193, 138), (192, 138), (192, 139), (194, 139), (195, 142), (189, 140), (188, 141), (189, 142), (190, 143), (190, 139), (188, 140), (190, 142), (190, 141), (189, 141), (200, 138), (201, 138), (202, 138), (203, 139), (212, 139), (215, 143), (215, 144), (216, 143), (216, 145), (217, 145), (218, 144), (218, 145), (219, 143), (220, 143), (219, 144), (218, 143), (220, 145), (220, 144), (221, 143), (221, 141), (222, 143), (223, 142), (222, 144), (220, 142), (227, 138), (231, 140), (232, 140), (232, 141), (231, 138), (232, 138), (230, 141), (230, 143), (231, 144), (227, 148), (229, 147), (231, 149), (230, 150), (229, 149), (228, 150), (227, 151), (228, 151), (229, 150), (230, 151), (230, 152), (229, 151), (231, 152), (232, 153), (231, 154), (232, 154), (231, 153), (232, 151), (232, 152), (231, 151), (229, 152), (232, 149), (232, 150), (233, 149), (233, 150), (234, 151), (234, 150), (229, 145), (228, 145), (226, 147), (226, 148), (227, 149), (226, 146), (225, 146), (225, 147), (224, 148), (225, 148), (224, 146), (223, 145), (221, 144), (222, 145), (223, 144), (224, 144), (225, 145), (224, 145), (223, 146), (223, 147), (229, 143), (225, 149), (222, 146), (218, 142), (219, 142), (218, 141), (217, 142), (217, 143), (216, 144), (218, 146), (217, 146), (219, 145), (219, 146), (220, 146), (221, 145), (217, 144), (215, 146), (214, 146), (214, 147), (211, 147), (213, 139), (212, 138), (214, 142), (215, 142), (214, 148), (215, 149), (216, 148), (216, 149), (217, 150), (217, 149), (217, 147), (216, 147), (215, 145), (200, 142), (199, 142), (203, 153), (203, 154), (203, 155), (202, 155), (202, 157), (202, 156), (203, 156), (204, 154), (204, 155), (206, 150), (207, 151), (206, 152), (205, 152), (195, 150), (196, 152), (196, 151), (195, 151), (195, 152), (195, 154), (199, 155), (205, 150), (206, 149), (192, 143), (191, 143), (190, 144), (189, 143), (189, 144), (191, 144), (208, 149), (208, 150), (207, 150), (208, 151), (206, 151), (206, 153), (207, 154), (208, 153), (209, 152), (209, 153), (208, 154), (209, 154), (209, 155), (208, 156), (208, 157), (207, 158), (209, 157), (209, 156), (210, 156), (210, 154), (211, 155), (212, 155), (212, 156), (213, 156), (213, 154), (212, 154), (213, 153), (214, 153), (215, 154), (216, 153), (217, 153), (217, 154), (218, 154), (217, 155), (218, 156), (218, 155), (219, 155), (220, 155), (221, 155), (220, 156), (221, 156), (221, 157), (222, 156), (221, 154), (219, 157), (220, 157), (220, 158), (220, 159), (219, 159), (220, 160), (220, 161), (221, 160), (221, 159), (221, 162), (221, 163), (222, 163), (222, 162), (221, 164), (222, 165), (223, 166), (222, 166), (223, 165), (224, 166), (222, 167), (223, 168), (223, 169), (224, 169), (224, 168), (224, 167), (221, 165), (222, 164), (223, 163), (224, 164), (223, 164), (221, 167), (220, 162), (219, 161), (218, 161), (217, 161), (217, 162), (216, 163), (215, 162), (214, 163), (215, 163), (213, 163), (212, 162), (212, 161), (211, 162), (212, 163), (210, 162), (209, 161), (210, 163), (208, 161), (208, 160), (209, 160), (210, 161), (210, 160), (211, 161), (211, 160), (210, 159), (212, 160), (213, 160), (214, 161), (215, 160), (214, 159), (216, 160), (215, 161), (214, 160), (213, 161), (213, 162), (212, 164), (211, 163), (211, 158), (210, 158), (210, 157), (211, 159), (214, 162), (216, 161), (218, 162), (218, 163), (219, 162), (219, 164), (220, 163), (221, 161), (219, 163), (220, 164), (218, 164), (218, 165), (218, 166), (217, 166), (216, 165), (215, 166), (215, 165), (214, 167), (214, 166), (213, 166), (212, 167), (212, 168), (213, 167), (213, 165), (213, 164), (215, 159), (216, 158), (216, 157), (217, 158), (218, 158), (217, 157), (217, 156), (215, 158), (217, 159), (216, 159), (212, 159), (212, 158), (213, 157), (214, 156), (215, 155), (216, 154), (215, 153), (214, 155), (214, 154), (213, 152), (212, 153), (211, 154), (210, 155), (211, 157), (208, 155), (207, 155), (206, 155), (206, 154), (205, 153), (204, 152), (204, 157), (204, 158), (202, 158), (205, 158), (204, 156), (203, 157), (198, 154), (193, 159), (194, 154), (194, 153), (193, 153), (193, 154), (192, 154), (191, 155), (191, 153), (193, 152), (194, 151), (194, 150), (193, 151), (193, 150), (191, 152), (190, 152), (194, 152), (192, 152), (190, 153), (189, 152), (189, 151), (188, 144), (186, 148), (189, 153), (188, 153), (188, 154), (189, 154), (191, 158), (191, 159), (194, 159), (195, 159), (194, 160), (190, 158), (190, 159), (189, 160), (189, 161), (188, 160), (187, 161), (188, 161), (187, 160), (186, 160), (185, 159), (185, 158), (186, 158), (185, 160), (184, 160), (183, 160), (183, 161), (184, 159), (184, 158), (186, 159), (186, 161), (189, 162), (188, 163), (189, 163), (188, 164), (187, 165), (187, 166), (187, 167), (187, 168), (188, 167), (188, 169), (187, 170), (187, 171), (186, 172), (187, 173), (186, 174), (187, 175), (188, 176), (187, 174), (188, 175), (187, 176), (186, 175), (185, 175), (184, 175), (185, 174), (186, 173), (185, 173), (184, 172), (184, 173), (183, 172), (183, 171), (182, 172), (181, 171), (180, 170), (180, 169), (179, 169), (178, 168), (178, 167), (177, 167), (178, 166), (179, 167), (179, 166), (180, 167), (180, 168), (179, 165), (178, 164), (178, 163), (179, 164), (180, 164), (181, 165), (180, 166), (177, 166), (177, 165), (180, 165), (181, 166), (181, 167), (182, 168), (182, 167), (181, 168), (182, 169), (183, 168), (181, 170), (179, 170), (179, 168), (178, 169), (176, 168), (176, 165), (177, 164), (177, 163), (176, 162), (176, 161), (175, 161), (175, 160), (174, 159), (177, 160), (177, 161), (178, 161), (179, 162), (180, 163), (179, 163), (176, 163), (175, 164), (174, 164), (175, 163), (174, 162), (174, 161), (175, 162), (176, 159), (175, 159), (173, 160), (172, 159), (171, 158), (170, 158), (170, 159), (170, 160), (171, 161), (172, 160), (173, 161), (172, 162), (171, 162), (170, 162), (171, 163), (173, 162), (173, 163), (173, 164), (174, 163), (175, 165), (174, 165), (176, 164), (178, 162), (180, 162), (178, 160), (178, 159), (177, 158), (176, 157), (177, 157), (178, 158), (179, 158), (179, 157), (180, 157), (179, 159), (180, 159), (180, 160), (180, 161), (181, 159), (182, 159), (183, 158), (183, 159), (182, 161), (181, 162), (182, 163), (181, 164), (176, 160), (175, 158), (174, 158), (173, 159), (176, 158), (181, 158), (180, 158), (179, 160), (181, 160), (183, 162), (184, 163), (185, 162), (186, 162), (187, 162), (187, 163), (185, 161), (187, 159), (188, 158), (189, 157), (189, 158), (187, 158), (190, 160), (189, 159), (191, 161), (191, 162), (191, 163), (190, 162), (196, 158), (195, 161), (194, 161), (195, 162), (196, 161), (200, 165), (202, 165), (202, 166), (203, 166), (202, 167), (203, 168), (203, 169), (202, 169), (202, 170), (202, 171), (203, 172), (203, 173), (202, 173), (201, 173), (200, 174), (202, 174), (201, 174), (201, 175), (201, 172), (203, 170), (201, 169), (201, 168), (202, 168), (200, 169), (200, 170), (200, 171), (199, 170), (198, 169), (197, 168), (196, 169), (196, 170), (195, 170), (195, 168), (194, 168), (194, 169), (193, 170), (194, 170), (195, 169), (196, 168), (197, 167), (195, 167), (193, 168), (192, 167), (193, 166), (193, 165), (194, 164), (194, 163), (193, 163), (193, 162), (193, 164), (192, 165), (192, 164), (192, 166), (191, 166), (191, 167), (190, 167), (190, 166), (189, 166), (188, 166), (189, 167), (189, 168), (190, 168), (189, 169), (189, 170), (190, 169), (190, 170), (190, 171), (191, 170), (191, 171), (189, 172), (189, 173), (188, 173), (189, 174), (188, 177), (188, 178), (187, 179), (186, 179), (185, 179), (185, 178), (184, 179), (184, 178), (184, 177), (185, 177), (186, 178), (185, 180), (186, 181), (185, 181), (184, 181), (184, 180), (183, 180), (184, 182), (183, 182), (183, 183), (182, 184), (182, 183), (181, 182), (180, 181), (180, 182), (181, 181), (182, 180), (180, 180), (179, 179), (179, 180), (179, 181), (178, 180), (177, 180), (178, 179), (177, 178), (177, 179), (176, 177), (175, 178), (174, 178), (173, 179), (172, 178), (172, 179), (171, 180), (170, 181), (171, 181), (170, 180), (169, 180), (171, 182), (172, 181), (172, 182), (173, 181), (173, 182), (172, 183), (173, 183), (174, 184), (174, 181), (174, 182), (174, 183), (173, 184), (172, 184), (173, 185), (174, 186), (174, 185), (175, 186), (175, 185), (175, 187), (175, 181), (176, 182), (175, 183), (173, 180), (172, 180), (170, 179), (171, 178), (170, 178), (171, 177), (170, 177), (169, 177), (169, 178), (170, 176), (171, 175), (171, 176), (172, 175), (173, 175), (173, 176), (172, 176), (174, 175), (174, 176), (173, 177), (172, 177), (170, 175), (173, 174), (174, 174), (173, 173), (173, 172), (172, 173), (171, 172), (171, 171), (172, 170), (173, 169), (172, 168), (173, 167), (174, 168), (174, 167), (172, 166), (173, 165), (172, 165), (171, 166), (171, 167), (170, 165), (173, 166), (172, 167), (170, 167), (169, 168), (170, 166), (168, 167), (167, 168), (167, 169), (166, 168), (165, 168), (164, 169), (163, 168), (163, 169), (164, 170), (163, 171), (163, 170), (165, 169), (166, 167), (165, 166), (167, 167), (168, 166), (169, 166), (171, 165), (171, 164), (172, 164), (168, 165), (170, 168), (171, 168), (172, 169), (173, 170), (171, 170), (171, 169), (170, 169), (170, 170), (170, 171), (169, 171), (169, 172), (168, 170), (168, 168), (168, 169), (165, 167), (166, 166), (165, 165), (166, 164), (166, 163), (167, 162), (167, 161), (166, 162), (167, 163), (168, 163), (167, 164), (166, 161), (166, 160), (165, 159), (165, 161), (167, 160), (168, 159), (169, 160), (168, 160), (169, 161), (168, 161), (169, 159), (169, 158), (170, 157), (169, 156), (169, 157), (165, 160), (164, 160), (164, 162), (163, 162), (163, 163), (162, 163), (161, 163), (160, 164), (160, 163), (161, 162), (160, 162), (159, 163), (161, 165), (160, 165), (159, 165), (159, 164), (161, 166), (162, 166), (162, 165), (163, 165), (164, 166), (164, 165), (163, 166), (164, 167), (163, 167), (162, 168), (161, 167), (160, 168), (159, 168), (158, 167), (157, 166), (157, 167), (158, 166), (159, 166), (156, 166), (155, 167), (156, 167), (156, 165), (157, 164), (158, 163), (157, 163), (157, 162), (158, 161), (157, 160), (156, 161), (155, 161), (155, 162), (156, 162), (155, 160), (157, 159), (156, 159), (156, 158), (155, 158), (155, 159), (154, 160), (153, 160), (154, 161), (157, 157), (155, 157), (154, 158), (154, 157), (155, 156), (156, 155), (155, 154), (155, 155), (154, 156), (154, 155), (154, 154), (156, 157), (157, 158), (156, 160), (157, 161), (156, 164), (155, 166), (155, 165), (157, 168), (158, 169), (157, 170), (157, 169), (158, 170), (156, 170), (155, 169), (155, 170), (158, 168), (159, 169), (160, 167), (161, 168), (160, 166), (161, 169), (162, 169), (162, 170), (161, 170), (160, 169), (162, 167), (164, 168), (159, 167), (158, 164), (157, 165), (156, 163), (155, 163), (154, 164), (153, 165), (152, 164), (152, 163), (153, 163), (152, 162), (151, 162), (151, 163), (153, 162), (152, 161), (153, 161), (154, 162), (158, 162), (159, 162), (161, 164), (162, 162), (162, 161), (162, 160), (163, 159), (164, 158), (165, 158), (166, 158), (165, 157), (166, 157), (167, 157), (167, 158), (166, 159), (169, 162), (165, 163), (164, 164), (167, 165), (168, 162), (169, 163), (170, 163), (169, 164), (170, 164), (169, 167), (169, 169), (167, 170), (166, 170), (166, 169), (166, 165), (165, 164), (164, 163), (167, 166), (168, 164), (169, 165), (162, 171), (163, 172), (164, 173), (164, 172), (164, 171), (165, 171), (165, 172), (166, 171), (167, 171), (166, 172), (168, 171), (169, 170), (170, 172), (168, 172), (172, 171), (173, 171), (172, 172), (171, 160), (172, 161), (171, 159), (170, 161), (168, 157), (168, 156), (167, 155), (167, 154), (168, 154), (169, 155), (170, 156), (166, 156), (168, 158), (168, 155), (170, 155), (169, 154), (170, 154), (171, 156), (171, 155), (172, 156), (173, 155), (174, 155), (175, 155), (174, 154), (176, 156), (173, 154), (172, 153), (173, 152), (174, 151), (174, 150), (173, 150), (175, 151), (174, 152), (175, 150), (175, 149), (175, 148), (174, 148), (173, 148), (172, 148), (173, 149), (172, 150), (172, 151), (171, 150), (171, 149), (170, 149), (170, 148), (171, 147), (170, 146), (169, 147), (168, 148), (167, 147), (168, 147), (167, 149), (167, 148), (168, 149), (169, 149), (166, 147), (166, 148), (165, 149), (169, 150), (169, 151), (168, 151), (167, 152), (167, 151), (166, 152), (167, 153), (166, 155), (165, 156), (167, 156), (164, 155), (164, 154), (165, 155), (164, 156), (163, 157), (166, 154), (171, 153), (171, 154), (172, 155), (170, 153), (170, 152), (171, 151), (170, 150), (170, 151), (171, 152), (173, 153), (172, 152), (169, 152), (168, 152), (168, 153), (166, 151), (165, 151), (164, 151), (165, 152), (166, 153), (163, 156), (162, 156), (163, 155), (162, 155), (161, 156), (161, 157), (160, 158), (159, 159), (158, 159), (159, 158), (159, 157), (159, 156), (158, 155), (160, 157), (160, 156), (161, 155), (160, 154), (161, 154), (160, 155), (162, 158), (163, 158), (164, 157), (162, 154), (163, 154), (164, 153), (165, 153), (163, 153), (163, 152), (162, 153), (165, 154), (164, 159), (162, 157), (164, 150), (164, 149), (163, 148), (162, 147), (163, 147), (164, 148), (163, 146), (163, 145), (162, 145), (162, 144), (162, 143), (162, 142), (162, 141), (163, 142), (161, 142), (160, 142), (159, 141), (159, 140), (158, 142), (157, 143), (156, 142), (155, 143), (154, 142), (155, 141), (155, 140), (154, 141), (153, 141), (154, 140), (155, 142), (156, 141), (156, 139), (155, 139), (156, 140), (157, 140), (157, 141), (157, 142), (157, 139), (155, 138), (156, 138), (157, 138), (154, 138), (153, 139), (158, 138), (159, 138), (159, 139), (160, 140), (160, 139), (160, 138), (161, 139), (162, 140), (162, 139), (162, 138), (161, 138), (160, 141), (159, 142), (160, 143), (160, 144), (161, 144), (159, 145), (158, 144), (157, 144), (156, 143), (154, 143), (154, 144), (158, 140), (158, 139), (159, 143), (161, 143), (159, 144), (158, 145), (158, 146), (158, 147), (158, 148), (158, 149), (159, 150), (158, 150), (158, 151), (157, 150), (156, 151), (155, 152), (156, 152), (157, 152), (157, 151), (155, 153), (156, 149), (155, 148), (155, 149), (154, 150), (154, 149), (154, 148), (155, 150), (154, 151), (153, 151), (156, 148), (153, 149), (154, 147), (155, 147), (156, 146), (155, 145), (155, 144), (156, 144), (156, 145), (155, 146), (157, 147), (157, 146), (159, 146), (160, 147), (159, 148), (160, 145), (160, 146), (158, 143), (158, 141), (164, 141), (163, 141), (163, 143), (163, 144), (161, 141), (164, 142), (165, 143), (165, 142), (163, 140), (164, 140), (153, 142), (157, 145), (159, 147), (161, 148), (162, 149), (164, 147), (164, 146), (164, 145), (165, 145), (165, 144), (164, 144), (164, 143), (165, 141), (161, 140), (161, 145), (154, 139), (153, 140), (152, 142), (152, 140), (152, 139), (153, 138), (151, 138), (152, 138), (151, 141), (152, 141), (151, 142), (150, 143), (151, 144), (152, 145), (153, 145), (154, 145), (156, 147), (157, 148), (155, 151), (154, 152), (156, 153), (156, 150), (157, 149), (157, 153), (158, 153), (158, 152), (159, 152), (159, 151), (160, 151), (161, 150), (162, 151), (163, 150), (161, 151), (162, 150), (163, 149), (166, 150), (165, 150), (166, 149), (165, 148), (168, 150), (167, 150), (166, 146), (167, 146), (168, 145), (168, 146), (169, 146), (169, 148), (167, 145), (168, 144), (166, 145), (161, 146), (161, 147), (162, 146), (154, 146), (159, 149), (160, 148), (160, 149), (156, 154), (157, 155), (156, 156), (157, 156), (159, 161), (159, 160), (160, 159), (160, 160), (161, 160), (160, 161), (158, 160), (154, 163), (155, 164), (154, 165), (151, 164), (152, 165), (153, 166), (153, 164), (150, 162), (150, 161), (151, 160), (149, 161), (150, 160), (151, 161), (152, 160), (150, 163), (149, 162), (149, 163), (150, 164), (150, 165), (149, 165), (150, 166), (151, 167), (152, 166), (151, 165), (149, 166), (149, 167), (149, 168), (148, 167), (148, 166), (148, 165), (147, 165), (147, 164), (147, 163), (146, 164), (145, 163), (146, 163), (148, 163), (149, 164), (153, 167), (152, 167), (152, 168), (151, 168), (150, 168), (150, 167), (150, 169), (150, 170), (151, 169), (152, 170), (153, 171), (151, 171), (152, 171), (151, 172), (151, 173), (150, 172), (149, 173), (149, 172), (148, 171), (147, 172), (146, 171), (146, 172), (147, 170), (146, 169), (145, 170), (145, 169), (144, 169), (143, 169), (142, 170), (143, 170), (142, 171), (144, 170), (144, 171), (144, 172), (145, 172), (146, 173), (147, 173), (146, 174), (147, 174), (148, 172), (148, 173), (148, 174), (146, 175), (145, 175), (144, 176), (145, 176), (145, 174), (144, 175), (144, 174), (143, 174), (143, 176), (143, 177), (144, 177), (144, 178), (145, 178), (146, 178), (145, 179), (144, 179), (146, 176), (145, 173), (150, 173), (149, 174), (147, 171), (146, 170), (145, 171), (148, 170), (149, 170), (148, 169), (149, 169), (148, 168), (147, 168), (146, 168), (145, 167), (144, 168), (146, 167), (147, 167), (146, 166), (145, 165), (154, 167), (155, 168), (156, 168), (154, 166), (154, 168), (158, 165), (154, 170), (155, 171), (157, 171), (157, 172), (158, 173), (158, 174), (157, 174), (158, 175), (157, 176), (157, 177), (158, 178), (159, 179), (159, 178), (158, 177), (159, 177), (157, 178), (157, 179), (156, 180), (156, 179), (157, 180), (158, 180), (158, 179), (160, 179), (161, 180), (160, 178), (160, 177), (160, 176), (160, 175), (159, 175), (159, 173), (158, 172), (158, 171), (159, 172), (160, 172), (160, 173), (161, 174), (162, 173), (161, 173), (161, 172), (161, 171), (156, 175), (155, 176), (155, 177), (156, 178), (156, 176), (155, 175), (157, 175), (155, 178), (154, 178), (154, 177), (153, 176), (154, 176), (154, 175), (153, 174), (154, 173), (155, 172), (156, 172), (155, 173), (156, 171), (156, 169), (154, 171), (157, 173), (160, 174), (162, 174), (163, 173), (165, 170), (167, 172), (168, 173), (167, 173), (166, 173), (167, 174), (167, 175), (168, 174), (168, 175), (166, 174), (165, 173), (165, 174), (169, 173), (170, 173), (171, 173), (172, 174), (164, 174), (168, 176), (174, 160), (177, 162), (179, 161), (167, 159), (165, 162), (171, 157), (173, 156), (173, 157), (172, 157), (174, 157), (173, 158), (174, 156), (172, 158), (175, 157), (177, 156), (178, 157), (178, 156), (179, 156), (180, 155), (181, 154), (182, 155), (181, 155), (181, 156), (182, 156), (182, 157), (183, 156), (184, 155), (185, 155), (186, 156), (186, 157), (185, 157), (182, 158), (183, 157), (184, 157), (184, 156), (183, 154), (183, 153), (184, 152), (184, 151), (185, 151), (185, 150), (186, 146), (186, 145), (185, 146), (190, 151), (185, 144), (186, 144), (186, 141), (187, 141), (188, 142), (191, 138), (191, 139), (190, 138), (189, 138), (185, 139), (185, 138), (186, 138), (184, 138), (188, 143), (187, 142), (187, 143), (190, 155), (181, 146), (180, 145), (179, 144), (178, 144), (179, 143), (178, 143), (180, 144), (181, 144), (181, 145), (180, 146), (181, 147), (180, 147), (182, 147), (181, 148), (180, 143), (183, 140), (178, 142), (177, 143), (177, 144), (177, 145), (176, 144), (175, 145), (175, 144), (174, 144), (173, 143), (173, 144), (174, 143), (174, 142), (172, 142), (171, 143), (173, 141), (172, 141), (172, 140), (171, 140), (170, 141), (169, 141), (169, 140), (168, 140), (167, 140), (166, 141), (166, 144), (167, 143), (167, 139), (168, 139), (170, 140), (170, 139), (171, 141), (171, 139), (172, 138), (170, 138), (172, 139), (173, 138), (173, 139), (174, 140), (174, 141), (175, 142), (175, 143), (176, 145), (175, 146), (175, 147), (176, 148), (177, 147), (177, 146), (176, 146), (178, 145), (178, 146), (174, 146), (173, 147), (172, 147), (171, 148), (174, 147), (174, 149), (172, 149), (167, 144), (169, 144), (170, 145), (169, 145), (166, 143), (170, 144), (171, 144), (171, 146), (172, 146), (172, 145), (172, 144), (172, 143), (174, 145), (176, 143), (177, 142), (178, 141), (177, 141), (176, 140), (176, 139), (175, 138), (176, 138), (175, 139), (177, 138), (178, 139), (178, 138), (179, 139), (177, 139), (177, 140), (175, 140), (174, 139), (170, 142), (170, 143), (169, 142), (168, 142), (168, 141), (169, 139), (169, 138), (168, 138), (171, 138), (173, 140), (173, 142), (174, 138), (178, 140), (179, 141), (179, 140), (180, 139), (181, 140), (182, 140), (180, 140), (181, 139), (176, 141), (175, 141), (167, 142), (168, 143), (165, 146), (165, 147), (170, 147), (173, 146), (171, 145), (169, 143), (162, 148), (169, 153), (176, 149), (177, 150), (177, 151), (176, 151), (176, 150), (176, 147), (173, 151), (164, 152), (165, 140), (166, 140), (166, 139), (166, 138), (165, 138), (165, 139), (167, 141), (173, 145), (172, 154), (172, 163), (164, 161), (163, 161), (163, 160), (161, 161), (174, 153), (175, 154), (176, 155), (177, 155), (176, 154), (177, 154), (178, 155), (180, 156), (181, 157), (181, 163), (182, 164), (183, 164), (182, 162), (182, 160), (181, 161), (184, 161), (184, 162), (185, 163), (188, 162), (187, 164), (188, 165), (189, 164), (190, 165), (186, 164), (186, 163), (185, 164), (184, 165), (183, 165), (183, 166), (185, 166), (185, 165), (186, 165), (183, 163), (177, 159), (176, 166), (175, 166), (174, 166), (173, 168), (174, 169), (174, 170), (174, 171), (166, 142), (167, 138), (171, 142), (176, 142), (179, 145), (179, 146), (179, 147), (179, 148), (177, 148), (178, 149), (176, 152), (179, 150), (180, 150), (180, 149), (182, 149), (182, 148), (183, 148), (181, 149), (183, 150), (184, 149), (183, 149), (184, 148), (184, 150), (183, 151), (185, 152), (184, 153), (184, 154), (186, 155), (189, 156), (188, 159), (186, 154), (183, 152), (182, 151), (182, 150), (183, 155), (185, 156), (179, 154), (180, 153), (180, 152), (179, 153), (179, 152), (178, 152), (181, 153), (182, 152), (181, 151), (187, 153), (186, 152), (186, 151), (187, 151), (187, 138), (186, 139), (185, 142), (191, 148), (192, 151), (195, 153), (203, 138), (213, 138), (209, 148), (210, 149), (211, 149), (212, 150), (210, 150), (210, 151), (210, 152), (211, 152), (210, 153), (211, 153), (208, 152), (207, 152), (207, 153), (205, 154), (207, 149), (208, 148), (204, 153), (205, 156), (206, 157), (206, 159), (207, 159), (206, 158), (205, 159), (204, 160), (205, 161), (205, 155), (205, 157), (188, 152), (180, 151), (181, 150), (180, 148), (179, 149), (178, 150), (178, 147), (177, 149), (179, 142), (178, 148), (182, 139), (182, 138), (183, 147), (180, 138), (181, 138), (163, 139), (179, 151), (177, 152), (177, 153), (178, 153), (182, 154), (182, 153), (181, 152), (184, 164), (179, 155), (178, 154), (176, 153), (180, 154), (178, 165), (186, 167), (185, 168), (185, 169), (185, 170), (186, 171), (187, 172), (186, 170), (186, 169), (185, 171), (184, 170), (183, 170), (182, 170), (181, 169), (182, 166), (177, 169), (176, 167), (175, 167), (175, 168), (182, 165), (175, 152), (175, 153), (175, 156), (190, 163), (190, 164), (189, 165), (188, 168), (187, 169), (191, 169), (192, 169), (191, 168), (189, 171), (190, 172), (191, 172), (192, 168), (193, 167), (191, 165), (191, 164), (192, 163), (197, 160), (197, 161), (198, 161), (198, 160), (197, 159), (198, 159), (196, 159), (201, 159), (206, 156), (209, 149), (209, 150), (211, 151), (213, 151), (212, 151), (214, 152), (212, 152), (212, 149), (213, 149), (221, 146), (220, 140), (224, 147), (226, 143), (226, 145), (220, 141), (221, 140), (217, 140), (217, 138), (216, 138), (216, 139), (214, 139), (215, 139), (214, 138), (217, 141), (216, 141), (217, 148), (215, 148), (214, 149), (215, 150), (214, 150), (213, 150), (218, 149), (210, 148), (211, 148), (216, 146), (215, 147), (215, 141), (215, 138), (216, 142), (226, 149), (224, 150), (224, 149), (223, 148), (222, 149), (224, 151), (225, 151), (223, 152), (222, 152), (222, 153), (222, 154), (221, 153), (223, 151), (224, 152), (225, 150), (226, 150), (228, 149), (231, 150), (230, 153), (229, 154), (230, 155), (229, 156), (228, 155), (227, 154), (229, 155), (230, 156), (228, 157), (229, 157), (228, 156), (227, 156), (227, 155), (230, 157), (231, 157), (231, 158), (232, 158), (232, 159), (233, 158), (234, 159), (232, 157), (231, 156), (228, 153), (227, 153), (226, 154), (226, 153), (225, 154), (225, 152), (225, 153), (227, 152), (228, 154), (229, 153), (228, 152), (227, 150), (226, 151), (230, 154), (231, 155), (233, 152), (233, 154), (234, 155), (235, 156), (236, 156), (236, 157), (237, 157), (235, 157), (234, 156), (233, 156), (233, 157), (233, 159), (231, 160), (232, 161), (233, 162), (234, 163), (233, 164), (232, 165), (231, 166), (230, 166), (231, 165), (230, 165), (230, 164), (229, 163), (230, 162), (230, 161), (229, 161), (231, 161), (232, 160), (233, 161), (233, 160), (232, 162), (229, 162), (228, 163), (227, 163), (226, 164), (225, 164), (224, 165), (225, 165), (226, 165), (227, 166), (228, 166), (226, 166), (223, 162), (224, 163), (224, 162), (223, 161), (223, 160), (224, 161), (225, 161), (225, 162), (220, 168), (221, 168), (222, 169), (221, 169), (220, 169), (220, 170), (220, 171), (221, 171), (222, 172), (223, 172), (223, 171), (223, 173), (222, 173), (221, 172), (221, 173), (222, 174), (224, 173), (224, 174), (225, 174), (224, 175), (224, 176), (224, 177), (224, 178), (225, 178), (223, 178), (222, 178), (223, 179), (221, 177), (222, 177), (223, 176), (223, 175), (222, 175), (221, 175), (220, 176), (221, 174), (225, 173), (226, 173), (227, 172), (228, 171), (229, 171), (229, 172), (229, 173), (228, 172), (230, 173), (230, 174), (231, 174), (230, 175), (230, 176), (230, 177), (231, 177), (229, 175), (228, 174), (229, 174), (228, 173), (227, 174), (228, 175), (227, 176), (228, 176), (227, 175), (226, 174), (225, 175), (226, 175), (227, 177), (228, 177), (226, 177), (225, 176), (225, 177), (223, 177), (225, 179), (223, 180), (223, 181), (223, 182), (222, 182), (221, 181), (222, 181), (221, 182), (220, 181), (221, 180), (222, 180), (222, 183), (220, 183), (220, 182), (219, 182), (220, 180), (220, 179), (220, 178), (224, 179), (223, 174), (225, 172), (222, 171), (221, 170), (222, 168), (225, 169), (226, 169), (227, 169), (227, 170), (227, 168), (228, 167), (229, 166), (231, 164), (231, 163), (232, 164), (233, 163), (234, 162), (230, 163), (231, 162), (232, 163), (232, 166), (233, 166), (234, 166), (234, 165), (235, 165), (236, 165), (234, 164), (235, 164), (235, 163), (228, 162), (227, 162), (226, 162), (225, 163), (226, 163), (228, 164), (228, 165), (227, 164), (229, 164), (234, 158), (234, 157), (232, 155), (233, 155), (234, 154), (235, 154), (235, 158), (235, 159), (235, 160), (236, 161), (237, 162), (238, 162), (239, 162), (238, 163), (237, 163), (238, 164), (239, 165), (239, 166), (240, 165), (241, 165), (242, 165), (243, 164), (242, 164), (241, 164), (240, 163), (242, 163), (241, 163), (240, 164), (240, 166), (240, 167), (239, 168), (238, 168), (237, 168), (236, 167), (236, 166), (237, 165), (236, 164), (236, 163), (237, 161), (237, 160), (238, 160), (239, 161), (240, 161), (241, 161), (241, 162), (239, 164), (236, 162), (238, 161), (237, 159), (238, 159), (239, 160), (239, 159), (239, 158), (239, 157), (240, 157), (241, 157), (240, 158), (241, 158), (241, 159), (242, 159), (242, 160), (242, 161), (243, 161), (243, 160), (243, 159), (242, 158), (243, 157), (244, 156), (245, 156), (245, 157), (246, 157), (247, 156), (248, 157), (249, 156), (248, 155), (248, 156), (247, 155), (248, 154), (249, 153), (249, 152), (249, 151), (250, 151), (251, 152), (251, 151), (250, 150), (249, 150), (251, 149), (252, 149), (252, 150), (250, 152), (248, 149), (249, 149), (249, 148), (250, 148), (250, 149), (251, 148), (250, 147), (250, 146), (251, 147), (249, 147), (248, 147), (251, 150), (248, 151), (248, 148), (247, 148), (246, 148), (247, 149), (248, 150), (247, 151), (246, 152), (245, 152), (244, 151), (245, 150), (245, 151), (244, 152), (245, 153), (244, 150), (243, 151), (242, 150), (242, 149), (243, 150), (240, 150), (241, 150), (242, 151), (245, 149), (246, 150), (247, 152), (247, 150), (246, 149), (245, 147), (245, 148), (246, 147), (244, 147), (246, 146), (248, 146), (252, 146), (251, 146), (252, 145), (253, 144), (254, 143), (255, 142), (256, 141), (256, 142), (257, 141), (257, 140), (258, 141), (259, 140), (258, 139), (258, 140), (257, 139), (259, 141), (259, 142), (258, 142), (260, 143), (259, 143), (258, 144), (258, 143), (257, 144), (258, 145), (259, 144), (259, 145), (257, 145), (257, 146), (256, 145), (256, 144), (257, 143), (257, 142), (260, 141), (258, 138), (257, 138), (256, 138), (256, 139), (255, 138), (259, 138), (256, 140), (255, 141), (256, 146), (258, 147), (259, 148), (257, 147), (258, 148), (259, 149), (260, 149), (261, 148), (260, 148), (261, 149), (261, 150), (262, 149), (263, 148), (264, 147), (264, 148), (263, 147), (263, 146), (262, 147), (262, 150), (262, 148), (261, 147), (263, 149), (263, 150), (264, 150), (265, 151), (265, 150), (265, 149), (264, 149), (264, 151), (266, 151), (267, 152), (266, 152), (267, 151), (266, 150), (266, 148), (267, 148), (267, 149), (268, 150), (268, 151), (267, 150), (266, 149), (268, 149), (265, 148), (265, 147), (265, 146), (260, 150), (261, 146), (261, 145), (260, 144), (261, 144), (262, 144), (262, 145), (260, 145), (260, 140), (261, 140), (262, 139), (263, 138), (263, 139), (263, 140), (263, 141), (262, 140), (262, 141), (261, 141), (260, 142), (264, 141), (264, 142), (263, 142), (262, 142), (261, 143), (263, 143), (261, 142), (262, 143), (263, 144), (263, 145), (264, 144), (264, 143), (265, 145), (256, 148), (256, 147), (255, 147), (255, 146), (259, 147), (257, 148), (257, 149), (260, 146), (259, 146), (260, 147), (262, 146), (264, 145), (264, 146), (263, 151), (262, 152), (263, 152), (262, 153), (261, 152), (261, 153), (260, 151), (260, 152), (259, 152), (258, 153), (259, 151), (258, 151), (257, 151), (257, 150), (256, 150), (257, 152), (256, 151), (258, 150), (259, 150), (267, 153), (268, 153), (267, 154), (268, 154), (266, 154), (266, 153), (265, 154), (264, 153), (263, 153), (263, 154), (262, 154), (260, 154), (259, 154), (258, 154), (258, 155), (259, 155), (259, 156), (258, 157), (257, 156), (256, 157), (255, 157), (256, 158), (255, 159), (256, 160), (257, 160), (258, 159), (259, 160), (259, 158), (259, 159), (258, 160), (260, 159), (260, 158), (259, 157), (260, 155), (259, 153), (258, 156), (257, 157), (256, 155), (256, 156), (255, 155), (255, 154), (256, 153), (257, 153), (257, 154), (257, 155), (256, 152), (255, 151), (255, 149), (255, 148), (254, 149), (253, 149), (254, 147), (255, 145), (254, 145), (255, 144), (256, 143), (256, 149), (254, 146), (258, 146), (253, 147), (252, 147), (253, 148), (258, 149), (261, 151), (264, 152), (265, 153), (262, 151), (260, 153), (265, 152), (267, 147), (266, 147), (266, 146), (265, 144), (265, 143), (266, 145), (266, 144), (266, 143), (267, 142), (265, 142), (266, 142), (267, 144), (267, 143), (267, 145), (267, 146), (261, 139), (260, 138), (260, 139), (259, 139), (255, 140), (254, 139), (254, 140), (253, 140), (253, 141), (252, 140), (252, 143), (251, 142), (252, 139), (245, 139), (246, 139), (246, 143), (247, 144), (248, 143), (247, 143), (248, 144), (249, 143), (250, 144), (249, 142), (247, 145), (249, 144), (249, 145), (250, 145), (251, 145), (253, 146), (249, 146), (248, 145), (250, 143), (253, 145), (252, 148), (249, 141), (247, 138), (240, 144), (243, 148), (243, 147), (242, 147), (242, 145), (242, 146), (230, 149), (239, 151), (239, 150), (225, 129), (226, 129), (226, 130), (223, 131), (224, 132), (224, 131), (225, 130), (227, 120), (231, 121), (232, 120), (233, 120), (234, 120), (235, 119), (225, 131), (226, 134), (225, 133), (220, 131), (219, 133), (219, 134), (222, 134), (218, 134), (217, 133), (215, 133), (215, 132), (214, 132), (225, 119), (227, 116), (227, 114), (226, 114), (226, 115), (231, 113), (233, 115), (232, 117), (234, 117), (232, 119), (231, 120), (233, 121), (232, 122), (235, 118), (236, 117), (235, 116), (236, 115), (235, 114), (232, 116), (224, 120), (227, 119), (227, 118), (227, 117), (223, 119), (222, 119), (210, 121), (220, 112), (211, 105), (210, 104), (210, 103), (209, 105), (209, 104), (208, 104), (208, 103), (205, 107), (207, 107), (207, 106), (208, 107), (210, 107), (209, 107), (209, 108), (206, 125), (205, 131), (205, 133), (203, 124), (222, 122), (222, 121), (221, 120), (226, 119), (228, 117), (227, 115), (225, 114), (226, 118), (223, 113), (220, 114), (221, 121), (223, 120), (214, 133), (199, 112), (212, 121), (229, 117), (223, 122), (203, 114), (216, 105), (237, 104), (237, 108), (235, 111), (235, 110), (236, 109), (233, 109), (233, 111), (234, 112), (235, 112), (234, 110), (234, 111), (234, 114), (235, 113), (233, 114), (230, 116), (231, 116), (232, 121), (240, 122), (252, 130), (253, 129), (253, 131), (254, 132), (255, 132), (254, 133), (255, 133), (256, 132), (257, 131), (258, 132), (258, 133), (259, 134), (260, 134), (261, 134), (262, 133), (261, 132), (261, 131), (262, 132), (260, 133), (259, 132), (259, 131), (258, 131), (257, 130), (256, 130), (256, 131), (257, 133), (256, 133), (257, 134), (255, 130), (254, 129), (255, 131), (254, 131), (256, 129), (251, 129), (268, 120), (268, 121), (268, 122), (268, 123), (267, 124), (266, 124), (266, 123), (267, 122), (267, 123), (266, 125), (267, 125), (268, 124), (267, 120), (267, 121), (266, 122), (267, 126), (268, 125), (266, 121), (268, 114), (268, 115), (249, 105), (256, 111), (238, 108), (245, 102), (240, 112), (237, 114), (237, 113), (236, 112), (236, 113), (232, 115), (203, 105), (227, 97), (227, 95), (216, 87), (213, 99), (205, 88), (206, 89), (206, 92), (205, 93), (206, 93), (207, 94), (206, 94), (207, 93), (207, 95), (208, 94), (206, 97), (208, 100), (207, 100), (208, 101), (215, 103), (215, 100), (209, 94), (206, 88), (184, 76), (181, 75), (181, 74), (168, 75), (166, 71), (167, 70), (166, 70), (165, 69), (165, 71), (164, 69), (167, 58), (167, 57), (169, 56), (170, 57), (171, 57), (179, 66), (182, 76), (183, 76), (182, 75), (196, 80), (196, 81), (180, 91), (177, 93), (176, 94), (177, 94), (177, 96), (178, 97), (178, 96), (178, 98), (178, 95), (173, 95), (172, 95), (172, 96), (171, 97), (172, 94), (170, 93), (169, 95), (170, 96), (171, 96), (171, 95), (175, 98), (174, 97), (172, 99), (172, 98), (172, 97), (171, 99), (170, 100), (171, 100), (170, 99), (171, 98), (174, 100), (179, 102), (179, 101), (180, 101), (182, 101), (183, 100), (183, 101), (182, 100), (183, 99), (184, 98), (182, 98), (183, 98), (182, 99), (181, 103), (184, 102), (184, 101), (185, 101), (186, 100), (187, 99), (185, 99), (182, 97), (181, 97), (180, 96), (182, 94), (179, 96), (179, 98), (179, 99), (180, 98), (179, 97), (178, 101), (179, 100), (179, 105), (180, 102), (176, 103), (175, 100), (171, 110), (170, 110), (187, 100), (185, 100), (186, 102), (187, 102), (186, 103), (185, 102), (181, 101), (181, 100), (180, 99), (184, 100), (186, 101), (187, 103), (188, 103), (187, 101), (188, 102), (189, 101), (188, 101), (190, 102), (191, 103), (191, 102), (189, 102), (192, 104), (192, 105), (192, 106), (193, 105), (192, 103), (193, 103), (191, 104), (192, 107), (193, 107), (193, 108), (194, 108), (195, 109), (193, 109), (194, 111), (189, 109), (189, 108), (190, 107), (191, 108), (190, 108), (190, 109), (203, 130), (210, 132), (201, 116), (197, 112), (198, 108), (197, 108), (198, 107), (199, 107), (199, 108), (199, 106), (199, 105), (198, 104), (198, 103), (197, 103), (196, 102), (196, 103), (196, 104), (197, 104), (196, 105), (199, 104), (198, 105), (197, 105), (197, 106), (200, 104), (201, 103), (200, 102), (199, 101), (198, 101), (199, 100), (200, 100), (168, 71), (171, 80), (168, 84), (167, 83), (166, 79), (183, 77), (173, 84), (172, 85), (171, 85), (166, 87), (166, 83), (165, 82), (165, 81), (164, 80), (163, 81), (164, 82), (164, 83), (165, 83), (163, 82), (169, 88), (167, 88), (168, 89), (169, 85), (172, 81), (181, 78), (181, 81), (218, 45), (229, 29), (230, 28), (228, 28), (223, 21), (225, 18), (232, 24), (231, 23), (230, 22), (230, 21), (230, 24), (229, 27), (229, 28), (230, 29), (231, 29), (232, 28), (231, 28), (231, 27), (230, 27), (231, 30), (232, 27), (229, 30), (230, 31), (230, 32), (230, 30), (229, 31), (232, 30), (233, 29), (229, 12), (233, 7), (233, 6), (234, 6), (235, 5), (235, 6), (236, 7), (236, 6), (235, 7), (234, 5), (236, 5), (236, 4), (237, 7), (241, 7), (242, 5), (240, 5), (243, 7), (249, 13), (250, 13), (258, 10), (261, 19), (260, 18), (259, 17), (259, 18), (258, 17), (261, 18), (257, 18), (260, 17), (257, 16), (258, 16), (258, 18), (240, 21), (241, 20), (236, 26), (240, 27), (238, 28), (233, 26), (234, 27), (235, 27), (238, 30), (238, 29), (239, 30), (239, 29), (231, 31), (230, 34), (240, 28), (235, 26), (232, 29), (234, 28), (230, 23), (231, 20), (232, 19), (222, 23), (164, 70), (164, 71), (163, 70), (165, 72), (163, 74), (164, 74), (163, 73), (162, 72), (163, 71), (163, 72), (162, 71), (162, 70), (163, 69), (164, 72), (176, 68), (181, 65), (180, 87), (178, 91), (177, 90), (163, 67), (162, 66), (161, 67), (160, 66), (160, 65), (161, 64), (164, 73), (161, 70), (162, 75), (162, 74), (162, 73), (164, 77), (163, 77), (162, 76), (161, 76), (160, 77), (159, 77), (160, 76), (159, 78), (160, 75), (161, 75), (161, 77), (163, 78), (163, 80), (164, 81), (164, 88), (164, 87), (165, 88), (166, 89), (165, 90), (166, 91), (164, 90), (164, 89), (165, 89), (163, 90), (162, 89), (163, 89), (179, 90), (178, 89), (183, 54), (166, 59), (162, 62), (161, 61), (162, 63), (163, 64), (160, 58), (161, 56), (160, 55), (159, 57), (161, 57), (158, 60), (155, 67), (150, 33), (149, 33), (172, 40), (174, 41), (176, 46), (175, 47), (175, 48), (174, 48), (208, 102), (210, 106), (204, 105), (204, 104), (202, 105), (203, 104), (208, 108), (202, 106), (201, 106), (200, 107), (200, 108), (189, 106), (188, 107), (188, 105), (188, 104), (189, 104), (189, 105), (188, 106), (189, 103), (188, 100), (189, 100), (190, 101), (188, 99), (187, 98), (186, 97), (188, 95), (187, 94), (187, 95), (188, 94), (186, 98), (186, 99), (190, 97), (191, 96), (190, 96), (187, 97), (189, 99), (191, 99), (190, 104), (190, 105), (190, 103), (194, 102), (195, 101), (195, 102), (193, 104), (194, 105), (195, 104), (195, 103), (193, 101), (194, 101), (195, 100), (196, 101), (197, 102), (198, 102), (200, 101), (201, 102), (200, 103), (199, 102), (206, 98), (205, 102), (205, 103), (207, 103), (204, 103), (202, 104), (203, 103), (201, 104), (199, 103), (197, 107), (200, 106), (198, 106), (200, 105), (201, 105), (202, 103), (204, 102), (207, 101), (207, 102), (206, 102), (198, 100), (197, 100), (196, 100), (194, 97), (195, 98), (194, 99), (192, 98), (192, 99), (191, 98), (191, 97), (192, 96), (192, 95), (190, 93), (190, 94), (191, 95), (192, 97), (193, 98), (193, 97), (186, 94), (186, 93), (185, 93), (187, 93), (187, 92), (168, 91), (167, 91), (166, 90), (167, 89), (166, 88), (165, 91), (165, 93), (165, 96), (158, 96), (158, 95), (163, 99), (162, 100), (160, 101), (160, 102), (155, 100), (156, 99), (155, 99), (154, 100), (153, 100), (151, 100), (157, 120), (166, 103), (161, 104), (161, 103), (160, 103), (148, 103), (157, 96), (154, 99), (157, 97), (156, 101), (156, 103), (150, 98), (140, 108), (141, 103), (142, 102), (147, 109), (138, 108), (139, 108), (139, 107), (138, 110), (138, 109), (147, 96), (147, 95), (144, 96), (143, 95), (143, 96), (143, 97), (148, 95), (148, 94), (151, 92), (152, 87), (159, 88), (158, 91), (157, 90), (157, 91), (157, 92), (156, 93), (150, 92), (151, 87), (156, 85), (157, 85), (161, 88), (160, 88), (167, 90), (161, 90), (161, 91), (162, 90), (167, 92), (166, 92), (177, 89), (185, 84), (162, 78), (161, 78), (162, 79), (162, 80), (163, 79), (160, 78), (160, 79), (159, 79), (158, 78), (157, 77), (158, 77), (159, 76), (159, 80), (158, 80), (158, 81), (159, 81), (158, 79), (158, 85), (162, 91), (163, 91), (160, 93), (159, 93), (157, 93), (157, 94), (153, 99), (150, 99), (149, 96), (150, 97), (143, 98), (160, 96), (161, 96), (158, 92), (159, 92), (160, 91), (163, 88), (159, 91), (175, 91), (189, 87), (178, 92), (173, 97), (169, 99), (169, 97), (168, 96), (166, 96), (168, 98), (167, 95), (165, 98), (162, 118), (180, 100), (192, 94), (193, 95), (194, 94), (194, 95), (194, 96), (193, 96), (196, 96), (197, 95), (198, 96), (199, 99), (201, 108), (201, 107), (182, 110), (174, 113), (171, 111), (173, 102), (171, 101), (171, 103), (172, 103), (172, 100), (168, 97), (167, 96), (165, 95), (166, 99), (166, 101), (167, 102), (168, 103), (169, 102), (169, 101), (169, 100), (160, 92), (152, 93), (165, 112), (181, 99), (181, 98), (184, 99), (189, 97), (192, 100), (175, 105), (162, 81), (161, 80), (160, 80), (161, 79), (161, 74), (163, 75), (161, 73), (161, 72), (160, 72), (160, 73), (159, 73), (159, 72), (158, 73), (159, 74), (159, 75), (158, 76), (165, 79), (160, 74), (166, 93), (149, 86), (149, 93), (144, 91), (145, 90), (145, 91), (145, 89), (146, 87), (146, 86), (147, 86), (147, 87), (146, 88), (147, 88), (148, 87), (148, 86), (149, 85), (152, 91), (156, 91), (156, 92), (150, 94), (149, 95), (149, 84), (149, 82), (148, 82), (147, 82), (148, 83), (149, 83), (150, 82), (151, 93), (142, 95), (141, 95), (142, 96), (142, 97), (141, 97), (141, 96), (140, 98), (141, 99), (138, 97), (141, 100), (139, 100), (140, 95), (141, 92), (140, 87), (141, 88), (142, 88), (140, 88), (141, 86), (141, 87), (142, 87), (140, 85), (141, 83), (140, 82), (139, 81), (139, 82), (140, 81), (141, 81), (140, 80), (141, 80), (142, 80), (143, 81), (142, 81), (141, 82), (140, 83), (142, 82), (142, 83), (139, 83), (139, 84), (141, 84), (142, 84), (139, 77), (138, 76), (138, 77), (138, 78), (139, 78), (138, 80), (139, 76), (139, 75), (140, 76), (139, 74), (142, 74), (142, 72), (152, 74), (153, 73), (153, 74), (152, 75), (154, 74), (154, 73), (155, 72), (155, 71), (156, 73), (157, 72), (157, 73), (158, 75), (157, 76), (160, 81), (149, 80), (150, 81), (149, 81), (148, 85), (148, 84), (147, 83), (147, 84), (146, 83), (145, 84), (146, 84), (146, 85), (145, 86), (144, 84), (143, 84), (143, 83), (142, 86), (143, 82), (144, 82), (144, 83), (144, 85), (144, 81), (144, 86), (143, 87), (176, 122), (171, 117), (170, 118), (189, 107), (188, 108), (190, 106), (192, 109), (191, 109), (192, 110), (195, 110), (196, 110), (195, 108), (196, 107), (196, 109), (196, 108), (195, 107), (196, 106), (195, 105), (194, 104), (206, 101), (209, 102), (206, 96), (195, 95), (194, 103), (193, 106), (191, 105), (192, 108), (194, 110), (204, 134), (206, 134), (208, 134), (170, 131), (168, 125), (167, 124), (170, 104), (169, 104), (168, 99), (168, 100), (168, 92), (180, 97), (189, 93), (191, 93), (174, 92), (174, 93), (173, 93), (191, 92), (174, 46), (174, 47), (163, 56), (162, 58), (169, 58), (169, 57), (172, 56), (163, 39), (170, 56), (191, 106), (191, 107), (202, 109), (202, 107), (195, 106), (194, 106), (194, 107), (199, 92), (187, 106), (158, 74), (157, 74), (158, 72), (226, 88), (250, 97), (248, 103), (239, 120), (241, 120), (252, 131), (253, 130), (242, 118), (240, 118), (242, 119), (242, 117), (232, 134), (171, 104), (170, 102), (170, 101), (168, 101), (167, 101), (154, 72), (153, 72), (150, 70), (151, 69), (151, 68), (150, 66), (150, 65), (141, 60), (143, 58), (143, 59), (146, 61), (145, 63), (147, 59), (149, 64), (160, 63), (159, 63), (158, 69), (158, 68), (159, 67), (224, 113), (230, 115), (238, 114), (240, 115), (236, 114), (209, 103), (203, 102), (154, 69), (154, 70), (155, 70), (156, 68), (156, 69), (153, 69), (152, 71), (155, 73), (145, 79), (144, 80), (145, 80), (146, 80), (147, 79), (146, 79), (147, 80), (147, 81), (146, 82), (145, 83), (145, 85), (147, 85), (143, 86), (143, 80), (144, 79), (145, 81), (145, 82), (146, 81), (150, 80), (151, 79), (148, 80), (148, 68), (159, 58), (149, 65), (149, 66), (148, 65), (138, 45), (138, 40), (138, 39), (138, 38), (142, 16), (142, 15), (141, 14), (155, 11), (159, 6), (158, 9), (159, 9), (159, 8), (157, 10), (160, 12), (161, 13), (162, 13), (162, 14), (162, 15), (161, 14), (160, 13), (162, 16), (168, 15), (163, 13), (163, 11), (163, 10), (162, 10), (162, 11), (161, 11), (162, 12), (166, 11), (165, 19), (166, 20), (171, 10), (172, 11), (164, 20), (161, 12), (149, 34), (158, 36), (160, 10), (160, 11), (161, 10), (161, 9), (160, 9), (159, 7), (160, 7), (160, 8), (162, 7), (163, 6), (162, 5), (164, 5), (163, 9), (164, 9), (162, 9), (165, 7), (166, 6), (166, 5), (165, 6), (167, 7), (168, 7), (167, 6), (168, 6), (167, 5), (167, 8), (166, 9), (166, 10), (165, 10), (163, 14), (191, 19), (161, 5), (150, 17), (161, 71), (162, 65), (160, 67), (149, 67), (148, 66), (147, 63), (156, 70), (145, 78), (140, 75), (159, 68), (160, 68), (161, 69), (160, 69), (160, 70), (160, 71), (159, 69), (158, 70), (157, 69), (157, 71), (156, 71), (159, 71), (158, 71), (159, 70), (174, 94), (173, 94), (148, 81), (156, 72), (148, 63), (148, 61), (146, 64), (160, 6), (159, 10), (161, 8), (158, 10), (166, 125), (168, 8), (166, 7), (167, 11), (250, 134), (251, 134), (255, 129), (255, 134), (253, 132), (252, 133), (237, 115), (241, 118), (178, 8), (176, 6), (170, 11), (169, 10), (168, 11), (197, 101), (194, 109), (231, 115), (225, 109), (223, 108), (222, 107), (256, 76), (236, 61), (245, 47), (233, 40), (238, 37), (237, 55), (228, 41), (232, 31), (236, 8), (234, 7), (232, 7), (228, 13), (226, 9), (231, 8), (227, 8), (233, 5), (232, 6), (232, 5), (191, 20), (252, 46), (254, 42), (258, 42), (257, 45), (259, 47), (258, 47), (256, 46), (268, 48), (268, 46), (268, 47), (246, 55), (233, 25), (230, 26), (224, 7), (227, 6), (230, 7), (230, 6), (231, 6), (231, 7), (229, 7), (229, 6), (228, 5), (228, 6), (221, 5), (232, 26), (231, 25), (232, 25), (231, 26), (243, 22), (249, 12), (257, 9), (257, 8), (258, 9), (260, 10), (259, 9), (258, 8), (258, 7), (258, 6), (257, 7), (256, 5), (254, 6), (255, 6), (256, 8), (260, 8), (261, 9), (262, 10), (261, 11), (259, 10), (260, 9), (261, 10), (259, 8), (251, 11), (248, 10), (251, 5), (248, 7), (259, 7), (259, 6), (259, 5), (258, 5), (260, 7), (260, 6), (262, 8), (261, 7), (261, 6), (260, 5), (261, 5), (239, 5), (238, 6), (238, 7), (237, 6), (231, 5), (230, 5), (229, 5), (262, 11), (257, 15), (257, 17), (239, 28), (227, 5), (237, 5), (164, 6), (163, 5), (167, 9), (165, 5), (263, 8), (262, 7), (261, 8), (262, 6), (262, 5), (262, 19), (250, 26), (264, 9), (265, 8), (266, 8), (266, 7), (267, 7), (269, 5), (268, 5), (265, 7), (264, 6), (264, 5), (263, 5), (263, 7), (230, 97), (223, 95), (253, 133), (259, 122), (255, 128), (259, 130), (260, 131), (261, 130), (262, 131), (263, 132), (263, 133), (264, 133), (268, 131), (261, 133), (259, 133), (260, 132), (260, 130), (263, 134), (262, 134), (261, 129), (263, 128), (259, 129), (254, 130), (221, 99), (170, 5), (169, 6), (169, 5), (168, 5), (154, 71), (138, 71), (238, 5), (236, 3), (236, 2), (268, 23), (268, 22), (267, 21), (268, 20), (265, 45), (265, 44), (259, 46), (259, 37), (264, 42), (261, 37), (170, 10), (176, 5), (177, 5), (171, 6), (171, 9), (170, 8), (169, 7), (168, 58), (229, 97), (229, 98), (231, 92), (245, 113), (252, 134), (254, 134), (256, 134), (257, 132), (258, 134), (262, 129), (262, 130), (258, 130), (246, 95), (253, 134), (241, 151), (242, 152), (241, 152), (240, 152), (241, 153), (242, 153), (241, 154), (242, 154), (243, 153), (244, 153), (246, 151), (244, 148), (246, 144), (247, 146), (247, 147), (248, 152), (252, 152), (252, 153), (251, 153), (251, 154), (252, 154), (253, 153), (253, 154), (253, 152), (254, 153), (250, 154), (250, 155), (250, 156), (251, 155), (252, 156), (251, 157), (251, 156), (250, 157), (251, 158), (250, 158), (250, 159), (249, 160), (249, 158), (253, 156), (254, 157), (254, 158), (254, 159), (255, 160), (254, 161), (253, 161), (253, 162), (252, 163), (252, 162), (254, 160), (253, 159), (255, 158), (253, 160), (252, 161), (253, 158), (252, 159), (252, 160), (251, 159), (252, 158), (252, 157), (253, 157), (252, 155), (254, 155), (255, 156), (252, 151), (254, 151), (255, 152), (249, 155), (249, 154), (253, 155), (254, 154), (254, 152), (253, 151), (253, 150), (255, 150), (254, 150), (255, 153), (256, 154), (260, 157), (260, 156), (261, 157), (261, 158), (261, 159), (262, 158), (263, 159), (264, 159), (263, 160), (263, 161), (262, 161), (262, 162), (263, 162), (264, 163), (263, 164), (264, 164), (265, 164), (264, 165), (265, 166), (264, 166), (263, 166), (262, 167), (261, 168), (261, 167), (262, 168), (263, 167), (264, 168), (263, 169), (262, 166), (261, 165), (260, 166), (260, 165), (261, 166), (262, 164), (261, 164), (259, 166), (258, 166), (257, 166), (257, 165), (258, 164), (257, 164), (258, 165), (256, 165), (256, 166), (255, 166), (254, 166), (254, 167), (253, 167), (252, 166), (252, 165), (251, 166), (250, 165), (250, 164), (250, 163), (251, 163), (251, 162), (250, 162), (250, 161), (251, 160), (249, 162), (251, 161), (249, 159), (250, 160), (256, 159), (257, 158), (258, 161), (257, 161), (257, 159), (258, 158), (261, 156), (262, 157), (263, 158), (262, 159), (260, 160), (261, 161), (261, 162), (261, 163), (262, 163), (263, 163), (260, 162), (260, 161), (262, 160), (263, 165), (265, 165), (266, 165), (267, 166), (268, 167), (268, 168), (267, 168), (267, 167), (266, 168), (268, 169), (267, 170), (267, 169), (266, 169), (266, 167), (264, 167), (263, 168), (262, 169), (262, 170), (263, 171), (263, 170), (262, 171), (261, 171), (260, 171), (260, 170), (261, 169), (265, 167), (264, 170), (259, 171), (258, 170), (258, 169), (257, 169), (256, 168), (256, 167), (255, 167), (257, 167), (258, 168), (259, 167), (260, 168), (259, 168), (258, 167), (259, 165), (260, 167), (255, 169), (255, 170), (254, 170), (254, 169), (253, 168), (254, 168), (253, 171), (252, 172), (251, 173), (250, 174), (251, 175), (251, 176), (252, 176), (253, 175), (254, 176), (255, 177), (256, 177), (257, 176), (257, 175), (257, 174), (256, 176), (255, 176), (253, 177), (252, 178), (251, 179), (251, 180), (252, 180), (252, 181), (253, 180), (253, 179), (251, 181), (251, 182), (251, 183), (252, 182), (252, 183), (253, 183), (251, 184), (250, 184), (250, 183), (253, 182), (252, 179), (251, 177), (250, 178), (250, 179), (252, 175), (252, 174), (251, 174), (250, 173), (251, 172), (251, 171), (250, 171), (249, 171), (250, 170), (248, 172), (247, 172), (248, 173), (249, 173), (249, 174), (252, 173), (252, 171), (252, 170), (253, 169), (252, 168), (251, 169), (252, 169), (251, 168), (250, 167), (250, 166), (251, 167), (249, 165), (248, 165), (249, 166), (249, 167), (249, 168), (248, 166), (251, 165), (251, 164), (253, 163), (254, 162), (252, 164), (253, 165), (254, 165), (255, 165), (255, 168), (256, 169), (257, 168), (253, 170), (252, 167), (253, 166), (256, 170), (257, 170), (259, 170), (258, 171), (258, 172), (259, 169), (260, 169), (264, 169), (265, 168), (265, 169), (264, 171), (259, 172), (260, 172), (259, 173), (260, 174), (259, 174), (258, 173), (260, 175), (261, 174), (262, 175), (261, 175), (262, 174), (263, 174), (261, 176), (262, 176), (263, 176), (261, 177), (259, 176), (260, 177), (259, 177), (260, 176), (259, 175), (258, 175), (258, 177), (259, 178), (258, 178), (257, 178), (257, 177), (258, 176), (255, 175), (253, 174), (253, 173), (250, 172), (249, 172), (255, 171), (255, 172), (255, 173), (254, 173), (254, 174), (255, 174), (256, 178), (258, 174), (256, 174), (256, 173), (256, 172), (256, 171), (257, 172), (257, 171), (253, 172), (254, 171), (250, 168), (248, 168), (248, 169), (249, 170), (248, 171), (246, 173), (246, 171), (245, 170), (244, 169), (244, 168), (243, 169), (243, 168), (242, 169), (241, 168), (241, 169), (240, 170), (239, 171), (238, 170), (238, 171), (238, 172), (237, 173), (238, 174), (239, 173), (238, 173), (240, 173), (239, 174), (238, 175), (239, 176), (239, 175), (240, 176), (239, 177), (239, 178), (240, 178), (241, 179), (241, 180), (240, 179), (242, 179), (242, 180), (243, 178), (243, 177), (244, 177), (245, 176), (246, 177), (247, 178), (248, 179), (248, 178), (247, 179), (246, 180), (246, 181), (246, 182), (247, 181), (247, 180), (246, 179), (246, 178), (247, 177), (247, 176), (246, 176), (246, 175), (247, 175), (248, 176), (248, 175), (249, 175), (249, 176), (249, 177), (250, 177), (250, 176), (248, 177), (249, 178), (248, 174), (247, 174), (246, 174), (249, 179), (250, 175), (248, 167), (247, 166), (247, 167), (250, 169), (249, 169), (248, 170), (247, 169), (246, 168), (245, 169), (246, 170), (247, 170), (247, 171), (247, 173), (245, 172), (244, 171), (252, 177), (245, 173), (246, 172), (246, 169), (245, 171), (244, 170), (247, 168), (246, 167), (251, 170), (254, 172), (254, 164), (253, 164), (254, 163), (255, 161), (244, 173), (243, 173), (242, 174), (242, 175), (241, 176), (240, 175), (241, 174), (242, 173), (242, 172), (241, 173), (240, 174), (238, 176), (241, 178), (242, 178), (242, 177), (243, 179), (244, 179), (243, 180), (244, 180), (245, 179), (243, 181), (242, 182), (242, 183), (241, 182), (242, 181), (241, 181), (241, 177), (240, 177), (237, 175), (236, 175), (236, 176), (235, 175), (237, 176), (238, 177), (239, 179), (238, 178), (238, 179), (238, 180), (237, 179), (236, 180), (235, 180), (236, 179), (237, 180), (236, 178), (241, 175), (241, 172), (241, 171), (242, 170), (243, 170), (244, 172), (245, 168), (245, 167), (245, 166), (245, 165), (245, 164), (244, 165), (243, 165), (243, 163), (241, 166), (242, 166), (241, 167), (240, 168), (239, 169), (240, 169), (241, 170), (243, 167), (242, 167), (239, 167), (237, 167), (235, 168), (234, 167), (236, 168), (236, 169), (237, 169), (238, 169), (242, 168), (244, 166), (243, 166), (242, 162), (243, 162), (235, 162), (237, 164), (235, 166), (233, 167), (233, 168), (234, 168), (235, 169), (235, 170), (238, 167), (236, 170), (234, 161), (234, 160), (236, 160), (233, 165), (235, 161), (238, 158), (238, 157), (237, 156), (238, 155), (238, 156), (239, 155), (240, 155), (239, 156), (240, 154), (236, 158), (236, 159), (237, 158), (235, 155), (232, 156), (226, 155), (230, 158), (229, 158), (228, 158), (228, 159), (233, 151), (225, 155), (226, 156), (226, 157), (225, 157), (224, 156), (225, 156), (224, 157), (223, 156), (224, 155), (223, 154), (223, 155), (223, 157), (216, 152), (217, 152), (217, 151), (216, 150), (218, 150), (218, 147), (218, 148), (216, 151), (216, 155), (215, 156), (214, 157), (214, 158), (215, 157), (215, 152), (213, 159), (213, 155), (211, 150), (214, 151), (215, 151), (219, 147), (219, 148), (220, 147), (221, 147), (221, 148), (222, 148), (221, 150), (220, 151), (220, 150), (221, 149), (222, 150), (223, 150), (222, 151), (223, 149), (221, 151), (219, 149), (220, 149), (220, 148), (220, 152), (220, 153), (221, 152), (219, 153), (220, 154), (222, 155), (227, 157), (227, 158), (226, 159), (225, 159), (226, 160), (226, 158), (227, 159), (225, 160), (222, 160), (221, 158), (219, 156), (216, 156), (218, 157), (222, 157), (222, 158), (222, 159), (222, 161), (224, 159), (225, 158), (224, 153), (224, 154), (223, 153), (223, 158), (219, 158), (218, 159), (219, 160), (218, 160), (221, 166), (220, 166), (219, 167), (220, 165), (216, 162), (215, 164), (216, 164), (217, 165), (219, 166), (218, 168), (218, 169), (217, 170), (216, 170), (216, 169), (215, 170), (216, 171), (217, 172), (216, 172), (218, 172), (219, 173), (220, 172), (224, 172), (226, 176), (222, 176), (220, 174), (220, 175), (226, 172), (227, 173), (219, 175), (218, 173), (219, 171), (219, 170), (219, 169), (218, 170), (218, 167), (217, 168), (216, 167), (216, 166), (214, 165), (212, 165), (211, 166), (211, 165), (212, 166), (211, 167), (210, 166), (210, 167), (211, 164), (213, 158), (218, 152), (219, 152), (218, 153), (218, 151), (219, 150), (219, 151), (209, 158), (208, 158), (211, 156), (212, 157), (201, 160), (204, 164), (205, 164), (206, 165), (205, 166), (204, 167), (203, 167), (204, 166), (203, 165), (205, 151), (222, 147), (217, 160), (223, 159), (224, 158), (226, 152), (219, 154), (233, 153), (234, 152), (236, 154), (237, 154), (237, 153), (238, 154), (239, 154), (240, 153), (239, 153), (239, 152), (240, 151), (238, 153), (236, 155), (238, 152), (227, 167), (228, 168), (229, 169), (229, 170), (230, 171), (231, 171), (232, 172), (233, 172), (233, 171), (232, 170), (232, 169), (233, 170), (234, 171), (234, 172), (233, 173), (232, 173), (231, 172), (231, 173), (230, 172), (232, 171), (234, 170), (233, 169), (234, 169), (235, 171), (236, 172), (237, 171), (237, 172), (235, 172), (236, 171), (237, 170), (235, 173), (234, 173), (236, 173), (240, 171), (239, 170), (239, 172), (237, 174), (240, 172), (242, 176), (243, 174), (244, 181), (245, 182), (245, 181), (244, 182), (245, 183), (246, 184), (246, 185), (245, 186), (245, 187), (244, 188), (243, 188), (244, 189), (245, 189), (246, 189), (245, 188), (242, 188), (242, 189), (241, 189), (242, 190), (243, 190), (243, 191), (243, 189), (242, 187), (243, 186), (244, 187), (244, 185), (246, 186), (247, 187), (247, 186), (248, 187), (249, 188), (249, 189), (250, 188), (249, 187), (249, 186), (248, 185), (244, 186), (243, 184), (242, 185), (241, 185), (242, 184), (241, 184), (242, 186), (241, 186), (240, 186), (239, 187), (238, 187), (240, 188), (241, 188), (240, 185), (239, 186), (238, 188), (239, 185), (238, 184), (237, 185), (237, 184), (238, 185), (236, 185), (236, 184), (235, 184), (234, 185), (235, 185), (235, 186), (236, 187), (236, 188), (235, 189), (234, 188), (234, 187), (233, 188), (232, 187), (232, 188), (233, 187), (234, 186), (235, 188), (235, 187), (233, 186), (231, 188), (232, 186), (231, 187), (230, 186), (229, 186), (229, 187), (228, 187), (228, 186), (228, 188), (228, 185), (227, 186), (227, 187), (227, 188), (229, 188), (230, 187), (229, 184), (230, 184), (231, 183), (231, 182), (232, 183), (230, 182), (230, 181), (229, 182), (229, 181), (228, 181), (228, 182), (227, 182), (228, 183), (230, 183), (232, 182), (233, 181), (233, 182), (232, 181), (232, 180), (234, 182), (234, 183), (234, 184), (233, 185), (233, 184), (235, 183), (236, 182), (235, 182), (235, 181), (236, 181), (237, 182), (237, 181), (236, 183), (237, 186), (238, 183), (239, 183), (240, 184), (240, 183), (238, 186), (240, 187), (236, 186), (237, 188), (237, 189), (237, 190), (236, 190), (235, 191), (237, 191), (236, 191), (238, 182), (239, 184), (240, 182), (239, 182), (240, 181), (239, 180), (239, 181), (240, 180), (235, 178), (235, 177), (236, 177), (235, 179), (234, 178), (234, 179), (238, 181), (237, 187), (241, 183), (241, 187), (241, 190), (240, 191), (239, 190), (239, 189), (240, 190), (240, 189), (244, 184), (243, 183), (243, 185), (246, 188), (247, 188), (244, 190), (244, 191), (245, 191), (241, 191), (242, 192), (242, 193), (243, 192), (238, 191), (238, 192), (237, 193), (236, 193), (237, 192), (237, 194), (238, 193), (239, 193), (234, 190), (233, 191), (233, 190), (232, 189), (230, 188), (227, 185), (229, 189), (231, 186), (230, 185), (231, 185), (231, 184), (233, 183), (232, 184), (234, 180), (237, 178), (245, 178), (248, 181), (249, 181), (250, 182), (250, 181), (250, 180), (251, 178), (249, 180), (253, 176), (254, 177), (255, 178), (254, 178), (254, 175), (256, 175), (258, 179), (259, 179), (260, 179), (261, 180), (261, 179), (262, 179), (261, 178), (262, 178), (260, 180), (257, 179), (256, 179), (255, 180), (254, 181), (253, 181), (254, 180), (254, 179), (255, 181), (256, 180), (257, 181), (258, 180), (260, 178), (256, 181), (257, 180), (258, 181), (259, 181), (259, 180), (260, 182), (260, 183), (261, 184), (262, 184), (261, 185), (261, 183), (260, 184), (262, 183), (263, 183), (261, 182), (259, 182), (260, 185), (259, 184), (258, 184), (257, 185), (258, 186), (258, 185), (259, 186), (257, 186), (257, 183), (256, 184), (255, 185), (256, 185), (257, 187), (257, 188), (256, 189), (255, 189), (254, 188), (253, 188), (254, 187), (255, 188), (255, 190), (254, 189), (254, 190), (253, 190), (252, 189), (251, 190), (251, 191), (250, 191), (250, 192), (249, 192), (248, 191), (249, 191), (250, 190), (251, 189), (250, 189), (248, 190), (250, 187), (248, 189), (247, 189), (246, 187), (248, 188), (251, 187), (252, 186), (251, 186), (252, 187), (252, 188), (252, 190), (253, 189), (253, 187), (255, 187), (254, 186), (254, 185), (255, 186), (253, 186), (250, 186), (251, 188), (249, 185), (248, 184), (247, 184), (248, 183), (249, 184), (250, 185), (249, 182), (248, 180), (245, 175), (244, 175), (245, 177), (244, 176), (243, 176), (245, 174), (253, 178), (260, 173), (261, 173), (262, 173), (263, 172), (262, 172), (263, 173), (266, 170), (266, 164), (265, 163), (266, 163), (267, 164), (265, 162), (266, 162), (266, 161), (267, 162), (268, 163), (268, 164), (267, 165), (268, 165), (268, 166), (265, 170), (265, 171), (266, 171), (266, 166), (267, 163), (266, 172), (265, 172), (267, 171), (268, 170), (264, 172), (264, 173), (265, 173), (266, 173), (267, 173), (266, 174), (267, 174), (268, 173), (268, 172), (268, 171), (267, 172), (261, 170), (261, 172), (264, 174), (266, 175), (266, 176), (265, 176), (264, 177), (264, 178), (263, 177), (263, 179), (265, 179), (264, 180), (265, 181), (264, 182), (263, 182), (262, 182), (262, 181), (264, 183), (265, 184), (265, 182), (264, 181), (263, 180), (264, 179), (265, 180), (263, 178), (262, 177), (243, 182), (252, 184), (245, 184), (245, 180), (244, 178), (243, 175), (234, 174), (235, 174), (234, 175), (234, 176), (234, 177), (233, 176), (233, 177), (233, 178), (234, 181), (231, 180), (237, 177), (232, 178), (232, 179), (233, 179), (229, 185), (227, 189), (228, 190), (228, 191), (227, 192), (229, 190), (227, 191), (226, 186), (226, 187), (225, 187), (225, 186), (226, 185), (230, 189), (230, 190), (231, 189), (232, 190), (233, 189), (234, 189), (235, 190), (236, 189), (238, 189), (238, 190), (239, 188), (243, 187), (235, 167), (238, 166), (246, 166), (247, 165), (247, 164), (246, 165), (244, 164), (244, 163), (244, 162), (240, 162), (240, 160), (244, 161), (241, 160), (240, 159), (230, 160), (229, 159), (228, 160), (229, 160), (230, 159), (228, 161), (231, 159), (239, 163), (238, 165), (237, 166), (244, 174), (248, 182), (247, 182), (246, 183), (245, 185), (239, 191), (242, 191), (241, 192), (245, 190), (244, 192), (245, 192), (246, 190), (244, 183), (248, 186), (256, 187), (254, 184), (253, 185), (252, 185), (250, 193), (251, 193), (248, 193), (249, 193), (250, 194), (249, 194), (248, 194), (248, 195), (251, 195), (252, 194), (253, 193), (254, 192), (254, 191), (252, 192), (253, 195), (253, 196), (253, 197), (253, 198), (254, 199), (253, 200), (254, 200), (255, 200), (256, 200), (256, 199), (255, 199), (253, 199), (252, 200), (253, 201), (254, 202), (255, 201), (256, 201), (254, 201), (253, 202), (253, 203), (252, 204), (252, 205), (251, 206), (251, 205), (250, 205), (251, 204), (250, 203), (252, 206), (252, 207), (250, 206), (249, 204), (249, 203), (248, 202), (248, 203), (247, 203), (248, 201), (249, 200), (250, 200), (251, 201), (251, 200), (252, 201), (252, 202), (252, 203), (254, 203), (255, 203), (253, 204), (254, 204), (255, 202), (256, 202), (257, 202), (258, 201), (258, 200), (257, 200), (257, 199), (258, 199), (255, 204), (256, 204), (257, 204), (257, 203), (258, 203), (258, 202), (257, 201), (256, 203), (251, 203), (251, 202), (249, 201), (250, 199), (251, 199), (252, 199), (252, 198), (254, 197), (255, 197), (255, 198), (256, 198), (254, 198), (259, 200), (260, 201), (259, 201), (260, 200), (261, 200), (261, 199), (262, 200), (263, 201), (264, 200), (263, 200), (264, 199), (264, 198), (263, 198), (263, 197), (262, 197), (262, 199), (262, 198), (263, 199), (265, 199), (266, 199), (267, 198), (267, 197), (268, 197), (267, 196), (268, 199), (268, 200), (267, 199), (267, 200), (268, 198), (266, 201), (267, 201), (268, 202), (268, 201), (267, 202), (265, 200), (265, 201), (264, 201), (264, 202), (263, 202), (263, 203), (264, 203), (265, 202), (266, 200), (266, 202), (266, 203), (265, 203), (262, 202), (261, 203), (262, 204), (262, 203), (261, 204), (262, 205), (261, 205), (263, 204), (264, 205), (263, 206), (262, 206), (260, 204), (260, 203), (261, 202), (259, 203), (259, 204), (258, 204), (257, 205), (254, 205), (255, 206), (255, 207), (255, 208), (254, 207), (256, 208), (253, 208), (254, 209), (254, 210), (254, 208), (255, 209), (256, 209), (257, 207), (256, 206), (256, 207), (257, 206), (258, 205), (258, 206), (259, 206), (260, 206), (260, 207), (261, 207), (263, 205), (264, 206), (263, 207), (264, 208), (264, 207), (265, 208), (266, 208), (265, 207), (266, 207), (267, 207), (266, 206), (265, 205), (267, 208), (268, 208), (268, 207), (267, 206), (268, 206), (268, 209), (268, 210), (268, 211), (268, 212), (267, 212), (267, 213), (268, 214), (267, 215), (267, 216), (266, 215), (266, 216), (266, 217), (265, 216), (264, 215), (263, 214), (264, 213), (264, 214), (265, 214), (266, 213), (266, 211), (265, 211), (266, 210), (267, 211), (264, 210), (265, 212), (266, 212), (267, 210), (267, 209), (265, 210), (266, 209), (265, 209), (264, 211), (263, 212), (265, 213), (266, 214), (265, 215), (268, 213), (267, 214), (263, 211), (262, 211), (261, 211), (263, 210), (264, 212), (263, 213), (262, 212), (261, 210), (260, 211), (262, 210), (264, 209), (263, 208), (265, 206), (264, 204), (261, 206), (262, 207), (259, 207), (260, 208), (261, 208), (259, 208), (259, 209), (258, 208), (257, 209), (258, 207), (260, 205), (258, 209), (260, 209), (261, 209), (262, 209), (262, 208), (263, 209), (260, 210), (259, 210), (256, 210), (257, 210), (257, 211), (258, 210), (258, 211), (256, 211), (256, 212), (255, 211), (255, 210), (253, 209), (253, 210), (252, 209), (251, 210), (251, 211), (252, 210), (251, 209), (253, 211), (253, 212), (254, 212), (255, 213), (255, 214), (256, 213), (254, 213), (254, 215), (254, 216), (255, 215), (256, 216), (256, 217), (257, 217), (256, 218), (255, 219), (254, 218), (254, 219), (254, 220), (255, 221), (253, 218), (255, 220), (256, 220), (256, 219), (257, 220), (257, 219), (254, 221), (257, 218), (258, 218), (258, 216), (257, 215), (256, 215), (257, 214), (256, 214), (255, 216), (254, 217), (252, 217), (253, 217), (253, 216), (255, 217), (255, 218), (256, 221), (256, 222), (256, 223), (255, 223), (257, 224), (257, 225), (256, 226), (256, 227), (257, 226), (258, 227), (257, 227), (258, 226), (258, 225), (259, 224), (259, 225), (260, 225), (259, 226), (256, 225), (255, 226), (254, 227), (253, 226), (253, 227), (252, 227), (251, 227), (251, 226), (250, 225), (249, 225), (250, 224), (250, 226), (249, 226), (250, 227), (248, 226), (249, 227), (248, 227), (247, 227), (246, 226), (247, 225), (246, 224), (245, 223), (246, 222), (245, 221), (244, 220), (243, 220), (242, 220), (243, 221), (245, 219), (246, 218), (245, 218), (246, 217), (245, 216), (245, 215), (245, 214), (246, 214), (247, 215), (247, 214), (248, 215), (245, 213), (244, 214), (243, 213), (243, 212), (244, 213), (244, 215), (246, 216), (245, 217), (244, 218), (243, 217), (244, 216), (246, 215), (247, 218), (247, 217), (247, 216), (245, 212), (245, 211), (246, 210), (245, 209), (246, 208), (245, 208), (245, 207), (244, 207), (243, 206), (243, 207), (242, 207), (241, 208), (242, 209), (243, 209), (244, 210), (244, 211), (243, 211), (242, 210), (242, 211), (241, 210), (240, 211), (241, 211), (241, 209), (240, 210), (241, 212), (240, 212), (240, 213), (241, 213), (242, 213), (244, 212), (246, 212), (247, 213), (248, 214), (247, 219), (248, 219), (247, 220), (247, 221), (246, 221), (246, 220), (248, 222), (247, 222), (245, 220), (246, 219), (245, 222), (247, 223), (246, 223), (248, 223), (249, 223), (244, 219), (243, 219), (248, 217), (249, 218), (250, 217), (251, 217), (251, 216), (251, 218), (252, 218), (253, 219), (252, 219), (250, 219), (249, 217), (250, 216), (250, 215), (251, 214), (251, 215), (252, 214), (252, 215), (249, 216), (252, 220), (253, 221), (252, 221), (251, 221), (252, 222), (253, 223), (254, 224), (253, 224), (252, 225), (251, 224), (251, 223), (252, 224), (252, 223), (253, 222), (251, 225), (249, 228), (248, 229), (247, 228), (246, 229), (246, 228), (248, 225), (247, 226), (246, 227), (245, 230), (245, 229), (244, 230), (244, 231), (243, 231), (243, 232), (242, 233), (243, 234), (242, 234), (243, 235), (242, 235), (242, 236), (241, 235), (241, 236), (240, 235), (241, 234), (240, 233), (240, 234), (239, 235), (239, 234), (239, 233), (238, 232), (238, 231), (239, 230), (240, 229), (241, 228), (240, 228), (240, 227), (240, 226), (239, 225), (239, 227), (239, 229), (241, 229), (239, 228), (241, 227), (242, 228), (242, 227), (243, 226), (243, 228), (244, 229), (243, 229), (244, 228), (244, 227), (244, 225), (245, 225), (246, 225), (245, 226), (244, 224), (245, 224), (245, 228), (244, 226), (248, 224), (249, 224), (252, 226), (248, 228), (248, 230), (248, 231), (247, 230), (247, 229), (246, 230), (245, 227), (249, 230), (249, 231), (250, 232), (250, 233), (249, 233), (249, 232), (250, 231), (250, 230), (251, 230), (249, 229), (244, 223), (243, 223), (242, 224), (241, 224), (241, 223), (241, 222), (242, 222), (241, 221), (240, 220), (239, 221), (239, 222), (240, 221), (240, 222), (238, 222), (237, 223), (238, 223), (239, 223), (240, 223), (238, 221), (237, 220), (237, 219), (236, 220), (236, 219), (235, 219), (234, 218), (235, 218), (236, 218), (235, 221), (234, 221), (234, 222), (233, 221), (234, 223), (235, 223), (236, 224), (237, 224), (238, 224), (236, 225), (236, 226), (237, 226), (238, 225), (235, 225), (235, 224), (233, 222), (233, 223), (232, 224), (231, 224), (230, 225), (229, 226), (229, 225), (231, 226), (230, 227), (229, 227), (229, 228), (228, 229), (227, 230), (226, 229), (226, 228), (227, 228), (226, 230), (225, 230), (225, 231), (225, 232), (226, 232), (227, 232), (228, 233), (226, 231), (224, 232), (224, 231), (223, 231), (222, 231), (221, 232), (221, 233), (222, 233), (222, 234), (223, 234), (222, 235), (223, 233), (222, 232), (222, 230), (223, 230), (224, 230), (227, 229), (225, 229), (223, 229), (224, 233), (225, 234), (225, 233), (226, 233), (224, 234), (225, 235), (226, 234), (224, 235), (224, 236), (225, 236), (226, 235), (224, 237), (225, 237), (226, 237), (223, 232), (221, 230), (222, 229), (224, 229), (220, 230), (219, 231), (218, 231), (217, 231), (218, 230), (219, 230), (220, 231), (219, 232), (218, 233), (220, 233), (220, 234), (219, 234), (221, 235), (222, 236), (222, 237), (221, 237), (221, 236), (220, 237), (220, 238), (220, 236), (220, 235), (219, 235), (218, 234), (217, 235), (216, 236), (217, 237), (215, 235), (216, 235), (217, 234), (219, 237), (219, 236), (218, 235), (218, 237), (218, 238), (217, 238), (216, 237), (215, 236), (214, 234), (213, 234), (213, 235), (212, 236), (213, 236), (212, 234), (212, 235), (211, 234), (210, 233), (209, 233), (208, 232), (207, 232), (208, 231), (209, 231), (210, 230), (211, 230), (212, 229), (212, 228), (211, 227), (212, 227), (213, 229), (213, 227), (213, 228), (211, 228), (211, 229), (214, 228), (215, 229), (214, 229), (215, 227), (216, 227), (217, 226), (218, 225), (218, 226), (217, 227), (216, 228), (216, 230), (216, 232), (215, 232), (216, 233), (216, 234), (217, 233), (217, 232), (218, 229), (218, 228), (219, 227), (220, 227), (221, 227), (221, 226), (222, 226), (221, 225), (222, 224), (222, 225), (223, 224), (224, 225), (225, 226), (224, 226), (223, 227), (223, 225), (224, 224), (225, 225), (225, 224), (223, 226), (222, 227), (225, 223), (226, 224), (227, 225), (226, 225), (227, 226), (228, 226), (227, 223), (228, 223), (227, 222), (226, 223), (226, 222), (227, 224), (228, 225), (228, 224), (230, 224), (231, 225), (227, 227), (227, 231), (227, 234), (227, 233), (229, 233), (229, 234), (229, 235), (230, 235), (229, 236), (228, 237), (228, 236), (227, 236), (228, 238), (229, 238), (229, 237), (230, 237), (228, 235), (231, 236), (230, 236), (231, 237), (231, 238), (232, 237), (232, 238), (231, 239), (230, 238), (233, 238), (234, 237), (233, 236), (234, 236), (235, 235), (235, 234), (235, 233), (236, 232), (235, 231), (234, 232), (234, 233), (235, 232), (236, 231), (237, 230), (236, 230), (235, 230), (237, 233), (237, 232), (237, 231), (238, 233), (238, 234), (237, 234), (236, 234), (236, 233), (236, 235), (239, 232), (238, 229), (238, 230), (237, 229), (236, 228), (237, 227), (235, 226), (235, 227), (236, 227), (237, 225), (234, 226), (234, 225), (234, 224), (233, 224), (232, 223), (236, 222), (237, 221), (238, 220), (239, 220), (239, 219), (239, 218), (240, 219), (241, 220), (242, 221), (244, 221), (244, 222), (244, 217), (243, 216), (246, 213), (242, 214), (241, 215), (240, 214), (241, 214), (242, 215), (243, 215), (242, 216), (243, 218), (248, 216), (249, 215), (250, 214), (253, 215), (254, 214), (253, 214), (252, 213), (252, 216), (249, 214), (249, 213), (254, 222), (253, 220), (255, 222), (251, 219), (250, 220), (251, 220), (249, 222), (250, 221), (249, 220), (249, 219), (254, 223), (251, 222), (250, 223), (249, 221), (248, 218), (248, 220), (248, 221), (247, 224), (240, 224), (240, 225), (241, 225), (241, 226), (243, 227), (250, 222), (250, 228), (251, 229), (252, 228), (253, 229), (254, 228), (253, 228), (251, 228), (253, 225), (254, 225), (255, 224), (257, 221), (258, 220), (258, 221), (259, 220), (258, 219), (260, 219), (259, 219), (260, 220), (260, 221), (261, 222), (260, 223), (261, 223), (260, 224), (261, 225), (261, 224), (258, 224), (259, 223), (260, 222), (258, 222), (260, 226), (260, 227), (256, 224), (250, 218), (258, 212), (259, 211), (260, 212), (257, 212), (258, 213), (258, 214), (257, 213), (255, 212), (253, 213), (252, 212), (251, 212), (250, 212), (250, 211), (250, 210), (250, 208), (250, 209), (249, 210), (249, 209), (249, 208), (250, 207), (251, 208), (248, 209), (248, 210), (248, 208), (247, 208), (246, 209), (245, 206), (244, 206), (244, 205), (243, 208), (242, 208), (244, 208), (246, 205), (247, 206), (248, 206), (249, 206), (248, 205), (247, 205), (246, 204), (246, 207), (246, 206), (247, 207), (249, 205), (248, 204), (250, 204), (251, 207), (249, 207), (247, 209), (248, 207), (250, 202), (249, 202), (253, 205), (254, 206), (255, 205), (253, 206), (257, 208), (259, 213), (259, 212), (260, 213), (259, 214), (259, 215), (260, 214), (261, 214), (261, 213), (261, 212), (263, 215), (264, 216), (268, 216), (268, 215), (268, 217), (265, 217), (265, 218), (264, 219), (265, 219), (265, 220), (265, 221), (265, 222), (266, 222), (267, 222), (268, 222), (268, 223), (268, 224), (268, 225), (267, 226), (267, 227), (268, 226), (268, 228), (267, 228), (266, 229), (267, 230), (267, 231), (266, 230), (267, 229), (265, 229), (264, 230), (264, 231), (265, 232), (264, 233), (265, 234), (264, 235), (264, 236), (264, 237), (263, 236), (265, 238), (266, 239), (267, 240), (268, 241), (268, 240), (267, 239), (266, 240), (265, 241), (265, 240), (264, 239), (264, 238), (265, 237), (266, 236), (263, 237), (265, 235), (263, 232), (265, 230), (266, 231), (264, 229), (263, 229), (264, 228), (263, 228), (265, 227), (265, 226), (264, 227), (264, 226), (264, 225), (263, 225), (263, 226), (262, 226), (262, 225), (262, 227), (263, 227), (261, 227), (261, 226), (259, 227), (260, 228), (260, 229), (259, 229), (260, 230), (261, 230), (262, 229), (265, 228), (266, 228), (266, 227), (266, 226), (267, 225), (266, 224), (266, 225), (267, 224), (267, 223), (266, 221), (267, 220), (267, 221), (266, 220), (266, 219), (264, 218), (266, 223), (265, 223), (268, 232), (268, 231), (262, 228), (261, 228), (262, 224), (263, 224), (264, 224), (265, 225), (265, 224), (261, 229), (268, 227), (268, 229), (268, 230), (267, 232), (266, 232), (265, 231), (264, 223), (264, 222), (263, 221), (262, 220), (262, 221), (263, 220), (262, 219), (263, 219), (262, 218), (261, 219), (261, 220), (260, 218), (261, 218), (261, 217), (260, 217), (259, 217), (259, 216), (260, 215), (260, 216), (261, 215), (261, 216), (262, 214), (262, 213), (263, 216), (264, 217), (263, 217), (258, 217), (259, 218), (259, 222), (259, 221), (264, 220), (264, 221), (263, 223), (268, 221), (268, 219), (267, 219), (266, 218), (267, 217), (263, 231), (262, 230), (261, 231), (259, 231), (260, 231), (262, 231), (259, 228), (258, 228), (258, 223), (257, 222), (257, 223), (255, 225), (259, 230), (258, 231), (259, 232), (260, 233), (260, 232), (261, 233), (262, 232), (261, 232), (261, 234), (262, 235), (263, 235), (262, 234), (262, 233), (263, 234), (263, 233), (264, 232), (264, 234), (265, 236), (266, 237), (266, 238), (267, 237), (266, 241), (265, 239), (263, 238), (263, 239), (262, 238), (261, 239), (261, 240), (260, 239), (260, 240), (259, 239), (258, 240), (257, 240), (258, 239), (259, 238), (258, 238), (259, 237), (258, 236), (258, 235), (259, 235), (258, 234), (260, 236), (260, 235), (268, 237), (268, 236), (269, 236), (268, 238), (268, 239), (264, 240), (267, 241), (266, 242), (265, 242), (267, 238), (267, 235), (262, 239), (262, 240), (263, 240), (263, 241), (267, 236), (263, 242), (264, 241), (264, 243), (263, 243), (262, 243), (262, 244), (261, 243), (261, 244), (260, 245), (259, 245), (258, 244), (257, 245), (256, 245), (257, 246), (257, 247), (258, 247), (258, 246), (259, 248), (258, 249), (257, 248), (259, 249), (258, 250), (259, 247), (259, 250), (258, 248), (258, 251), (257, 251), (256, 248), (255, 248), (254, 249), (254, 248), (256, 249), (257, 250), (256, 251), (253, 250), (255, 250), (252, 249), (251, 248), (250, 247), (249, 248), (251, 247), (252, 246), (251, 245), (251, 244), (250, 243), (249, 242), (250, 241), (250, 242), (249, 243), (248, 243), (248, 242), (249, 241), (248, 244), (251, 242), (250, 240), (251, 241), (251, 240), (252, 240), (253, 239), (252, 238), (254, 239), (254, 238), (253, 238), (253, 237), (255, 240), (255, 241), (254, 240), (254, 236), (255, 236), (255, 235), (254, 234), (254, 235), (253, 235), (252, 236), (252, 237), (253, 234), (252, 235), (251, 236), (250, 235), (250, 234), (249, 235), (248, 235), (249, 234), (251, 234), (251, 235), (248, 236), (249, 236), (247, 235), (248, 234), (250, 236), (249, 237), (248, 238), (247, 237), (247, 238), (246, 239), (246, 238), (246, 236), (252, 234), (250, 237), (248, 237), (246, 237), (247, 236), (247, 233), (248, 233), (248, 239), (249, 238), (246, 235), (251, 232), (252, 229), (252, 230), (252, 231), (251, 231), (252, 232), (253, 233), (253, 231), (254, 230), (253, 232), (254, 233), (254, 232), (255, 231), (256, 230), (256, 229), (257, 228), (256, 228), (255, 227), (255, 229), (254, 231), (256, 231), (257, 231), (257, 230), (258, 229), (258, 232), (259, 233), (260, 234), (259, 236), (260, 237), (261, 238), (257, 239), (259, 240), (258, 241), (259, 241), (263, 230), (254, 226), (250, 229), (262, 222), (263, 222), (261, 221), (262, 223), (255, 228), (262, 215), (262, 217), (262, 216), (254, 211), (258, 215), (257, 216), (252, 211), (252, 208), (249, 211), (250, 213), (248, 213), (248, 212), (249, 212), (251, 213), (267, 218), (263, 218), (255, 230), (255, 232), (256, 232), (255, 233), (256, 233), (256, 234), (255, 234), (257, 234), (257, 233), (257, 235), (261, 236), (261, 235), (257, 236), (256, 235), (258, 237), (256, 237), (256, 238), (256, 239), (257, 238), (257, 237), (261, 237), (262, 236), (262, 237), (265, 233), (266, 233), (266, 234), (267, 234), (267, 233), (268, 233), (253, 240), (253, 241), (252, 241), (251, 243), (252, 245), (253, 244), (254, 245), (255, 244), (256, 243), (257, 244), (258, 245), (260, 248), (260, 249), (259, 246), (260, 244), (259, 244), (258, 243), (258, 242), (257, 243), (256, 242), (256, 241), (257, 242), (255, 242), (255, 243), (254, 244), (254, 243), (254, 241), (254, 242), (253, 243), (252, 243), (250, 244), (249, 244), (247, 243), (247, 244), (247, 245), (247, 246), (248, 247), (249, 247), (248, 248), (248, 249), (247, 248), (249, 250), (249, 251), (250, 250), (250, 249), (251, 250), (250, 251), (250, 252), (249, 252), (248, 253), (247, 253), (246, 253), (246, 252), (247, 251), (248, 252), (251, 249), (250, 248), (249, 246), (249, 245), (248, 245), (246, 243), (247, 242), (248, 241), (246, 245), (245, 246), (246, 247), (244, 246), (245, 247), (246, 246), (247, 247), (247, 249), (246, 250), (246, 251), (248, 251), (248, 250), (245, 254), (246, 255), (246, 254), (245, 255), (244, 255), (244, 256), (243, 255), (242, 256), (241, 255), (240, 255), (239, 256), (239, 255), (238, 257), (239, 257), (240, 256), (241, 254), (242, 254), (241, 253), (241, 252), (243, 254), (242, 253), (243, 253), (241, 256), (238, 255), (239, 254), (239, 253), (240, 254), (240, 253), (240, 251), (239, 250), (239, 249), (241, 250), (242, 251), (243, 250), (242, 250), (242, 249), (243, 249), (244, 250), (245, 250), (246, 249), (246, 248), (245, 249), (244, 248), (244, 249), (245, 248), (244, 247), (243, 248), (247, 250), (245, 245), (245, 244), (244, 243), (243, 244), (244, 244), (243, 243), (244, 242), (243, 242), (242, 243), (244, 245), (243, 246), (243, 245), (243, 241), (242, 242), (241, 241), (242, 240), (243, 239), (242, 238), (241, 239), (240, 239), (240, 238), (241, 240), (244, 238), (244, 239), (244, 240), (242, 241), (243, 240), (242, 239), (243, 238), (244, 237), (245, 240), (245, 241), (246, 240), (244, 241), (245, 243), (245, 242), (241, 242), (240, 243), (246, 242), (247, 241), (248, 240), (249, 239), (250, 239), (249, 240), (252, 242), (253, 245), (252, 244), (253, 246), (254, 247), (250, 238), (245, 238), (245, 239), (253, 236), (254, 237), (255, 237), (256, 236), (258, 233), (257, 232), (257, 229), (258, 230), (254, 229), (253, 230), (243, 224), (242, 225), (239, 224), (237, 222), (236, 221), (235, 220), (234, 220), (233, 219), (233, 220), (232, 222), (232, 221), (232, 225), (232, 226), (228, 227), (228, 228), (229, 229), (230, 229), (231, 228), (232, 228), (233, 229), (233, 230), (232, 227), (233, 227), (233, 228), (231, 223), (231, 222), (231, 221), (230, 220), (231, 220), (230, 221), (229, 219), (229, 221), (230, 222), (228, 222), (228, 221), (229, 220), (228, 220), (227, 220), (228, 219), (228, 218), (227, 219), (227, 218), (226, 218), (226, 219), (226, 220), (225, 221), (226, 221), (227, 221), (220, 224), (220, 225), (219, 224), (216, 226), (217, 225), (216, 225), (216, 224), (215, 226), (214, 226), (215, 225), (214, 225), (215, 224), (214, 227), (215, 228), (212, 226), (211, 225), (212, 225), (211, 224), (211, 223), (212, 223), (212, 222), (213, 222), (214, 223), (214, 224), (215, 222), (216, 221), (216, 220), (215, 220), (214, 220), (214, 219), (214, 221), (213, 219), (215, 219), (216, 218), (217, 217), (217, 216), (216, 217), (216, 216), (218, 216), (218, 217), (217, 215), (218, 214), (219, 213), (219, 214), (220, 213), (219, 212), (218, 211), (219, 210), (220, 211), (221, 210), (221, 209), (220, 210), (219, 211), (220, 208), (220, 209), (219, 209), (221, 208), (222, 209), (222, 208), (223, 207), (223, 206), (223, 205), (224, 206), (224, 205), (225, 206), (223, 204), (223, 203), (223, 202), (224, 202), (225, 202), (226, 203), (226, 204), (227, 203), (228, 204), (224, 201), (222, 204), (222, 203), (224, 203), (225, 203), (224, 204), (225, 204), (226, 205), (227, 204), (228, 205), (227, 206), (226, 206), (228, 206), (227, 205), (229, 204), (230, 204), (229, 203), (229, 202), (230, 202), (230, 203), (231, 204), (232, 203), (233, 203), (234, 202), (234, 203), (234, 204), (235, 203), (235, 202), (233, 201), (233, 204), (232, 204), (231, 203), (231, 205), (230, 206), (231, 207), (230, 207), (229, 207), (228, 208), (227, 207), (226, 207), (227, 208), (226, 209), (226, 210), (225, 209), (225, 210), (224, 210), (223, 209), (224, 209), (222, 210), (222, 211), (221, 211), (222, 212), (223, 213), (224, 212), (223, 212), (223, 211), (222, 214), (223, 215), (221, 213), (222, 213), (221, 215), (220, 214), (220, 212), (221, 212), (221, 214), (222, 216), (221, 217), (221, 216), (220, 215), (218, 215), (217, 214), (216, 215), (216, 214), (217, 213), (215, 214), (217, 212), (216, 213), (214, 214), (214, 215), (213, 216), (212, 216), (213, 215), (214, 216), (212, 214), (211, 214), (212, 215), (211, 215), (214, 217), (215, 217), (216, 212), (215, 213), (214, 212), (213, 211), (212, 212), (212, 213), (213, 213), (213, 214), (215, 211), (216, 210), (216, 211), (215, 212), (214, 211), (214, 210), (215, 210), (217, 211), (213, 210), (214, 209), (213, 209), (213, 208), (213, 207), (214, 206), (214, 207), (215, 208), (216, 208), (217, 208), (214, 208), (212, 210), (212, 211), (211, 212), (211, 211), (210, 210), (209, 209), (208, 208), (207, 207), (206, 208), (206, 209), (205, 208), (206, 207), (205, 209), (207, 206), (208, 206), (207, 205), (208, 207), (208, 205), (206, 205), (205, 206), (204, 206), (203, 205), (204, 205), (203, 204), (202, 203), (201, 204), (201, 205), (200, 206), (199, 206), (200, 207), (201, 207), (202, 207), (202, 208), (201, 208), (202, 209), (202, 206), (199, 207), (200, 205), (199, 205), (199, 204), (200, 203), (200, 202), (201, 206), (199, 208), (200, 208), (200, 209), (198, 205), (198, 206), (197, 205), (196, 206), (197, 207), (195, 206), (194, 206), (195, 205), (196, 205), (197, 206), (198, 207), (198, 208), (199, 210), (199, 209), (198, 209), (197, 210), (196, 209), (196, 208), (195, 210), (196, 211), (197, 212), (198, 212), (199, 212), (200, 211), (200, 210), (202, 205), (202, 204), (203, 203), (204, 204), (200, 204), (201, 203), (198, 203), (198, 204), (199, 202), (201, 202), (199, 203), (200, 201), (202, 202), (201, 201), (201, 200), (200, 200), (199, 199), (200, 199), (201, 198), (201, 197), (200, 196), (199, 196), (200, 195), (199, 194), (198, 194), (200, 194), (199, 195), (198, 195), (200, 197), (201, 196), (201, 195), (202, 195), (202, 196), (203, 196), (204, 195), (202, 197), (203, 197), (204, 197), (204, 198), (205, 197), (205, 198), (206, 197), (206, 198), (207, 198), (207, 199), (208, 199), (208, 200), (209, 199), (208, 198), (209, 197), (208, 197), (207, 196), (206, 196), (205, 199), (204, 199), (203, 199), (203, 200), (204, 201), (203, 202), (202, 201), (202, 200), (203, 201), (205, 202), (204, 202), (206, 202), (205, 201), (206, 200), (207, 201), (207, 200), (206, 201), (206, 203), (206, 204), (205, 204), (205, 205), (204, 207), (203, 206), (205, 203), (204, 203), (208, 201), (207, 202), (209, 200), (209, 201), (209, 202), (208, 203), (208, 202), (207, 203), (207, 204), (206, 206), (209, 205), (210, 206), (210, 205), (209, 204), (210, 203), (211, 203), (212, 203), (212, 202), (212, 201), (213, 203), (213, 204), (212, 204), (211, 205), (211, 204), (212, 205), (212, 206), (213, 206), (213, 205), (212, 207), (211, 207), (210, 207), (211, 206), (210, 208), (211, 208), (211, 209), (209, 206), (209, 208), (209, 207), (199, 201), (198, 202), (197, 202), (196, 203), (195, 204), (195, 203), (194, 204), (195, 207), (194, 208), (194, 207), (196, 207), (197, 203), (198, 201), (205, 200), (206, 199), (203, 198), (199, 197), (200, 198), (201, 194), (203, 195), (202, 194), (201, 193), (200, 193), (202, 192), (202, 198), (209, 198), (210, 200), (211, 200), (211, 199), (210, 198), (210, 199), (210, 197), (209, 196), (208, 195), (208, 196), (207, 195), (206, 194), (205, 193), (206, 193), (205, 194), (207, 192), (208, 192), (208, 193), (207, 193), (206, 192), (207, 194), (206, 195), (207, 191), (208, 190), (208, 191), (209, 191), (207, 190), (207, 189), (206, 189), (206, 190), (205, 188), (206, 187), (207, 188), (208, 189), (208, 188), (209, 187), (210, 188), (210, 189), (211, 190), (210, 191), (210, 192), (209, 192), (211, 191), (212, 191), (213, 191), (214, 192), (215, 192), (214, 191), (215, 191), (214, 193), (213, 194), (214, 194), (215, 195), (214, 196), (213, 195), (212, 195), (211, 196), (212, 196), (211, 195), (211, 197), (211, 194), (210, 193), (211, 192), (211, 193), (206, 191), (206, 188), (205, 189), (205, 190), (205, 192), (205, 191), (207, 187), (207, 186), (208, 187), (209, 186), (210, 186), (209, 185), (209, 188), (210, 187), (208, 186), (207, 185), (206, 186), (205, 187), (204, 186), (203, 185), (208, 185), (211, 185), (212, 186), (213, 186), (213, 185), (214, 187), (214, 186), (214, 185), (213, 187), (212, 187), (211, 186), (211, 188), (212, 189), (212, 190), (210, 194), (209, 194), (204, 191), (203, 191), (203, 190), (202, 189), (201, 188), (201, 189), (200, 188), (202, 190), (202, 191), (201, 190), (202, 188), (203, 187), (201, 187), (200, 186), (199, 186), (200, 185), (199, 187), (199, 188), (199, 189), (200, 189), (202, 187), (200, 187), (201, 186), (202, 186), (203, 186), (204, 185), (205, 184), (206, 184), (205, 183), (204, 182), (203, 183), (202, 184), (201, 184), (201, 185), (201, 183), (202, 182), (201, 181), (201, 182), (200, 181), (199, 180), (199, 181), (200, 180), (199, 179), (200, 182), (202, 183), (205, 186), (204, 184), (203, 184), (204, 183), (206, 185), (207, 183), (207, 184), (205, 185), (209, 189), (209, 193), (210, 195), (210, 196), (209, 195), (208, 194), (211, 189), (210, 185), (211, 187), (213, 189), (214, 188), (212, 188), (210, 190), (212, 192), (213, 192), (212, 193), (213, 193), (214, 190), (216, 192), (217, 192), (216, 191), (212, 194), (213, 196), (212, 197), (213, 197), (213, 198), (214, 197), (215, 196), (216, 197), (215, 197), (212, 198), (207, 197), (204, 196), (201, 192), (200, 191), (200, 192), (199, 191), (199, 193), (201, 191), (203, 192), (204, 192), (205, 196), (205, 195), (203, 194), (199, 200), (197, 200), (197, 201), (198, 200), (197, 199), (197, 198), (198, 199), (198, 198), (199, 198), (202, 199), (201, 199), (198, 197), (196, 197), (197, 197), (196, 198), (195, 198), (194, 199), (194, 200), (195, 201), (196, 200), (196, 199), (197, 196), (198, 196), (195, 199), (193, 199), (194, 198), (193, 197), (193, 196), (192, 196), (193, 195), (194, 196), (195, 196), (194, 197), (195, 197), (210, 201), (211, 202), (211, 198), (212, 199), (214, 198), (211, 201), (210, 202), (209, 203), (210, 204), (205, 207), (206, 210), (205, 210), (204, 209), (205, 211), (204, 210), (203, 209), (203, 208), (204, 208), (208, 209), (207, 210), (206, 211), (204, 211), (205, 212), (205, 213), (204, 213), (203, 213), (203, 212), (203, 211), (204, 212), (203, 210), (207, 208), (210, 209), (207, 209), (203, 207), (201, 210), (202, 211), (201, 211), (202, 212), (201, 213), (201, 214), (200, 214), (199, 213), (198, 214), (197, 214), (196, 213), (196, 214), (196, 215), (197, 213), (198, 215), (197, 215), (198, 216), (199, 216), (198, 217), (200, 217), (199, 218), (198, 218), (200, 218), (199, 219), (199, 217), (201, 217), (200, 220), (200, 221), (199, 221), (198, 221), (198, 222), (199, 222), (200, 222), (201, 222), (199, 220), (197, 221), (198, 220), (198, 219), (197, 219), (197, 218), (197, 220), (196, 220), (195, 220), (194, 219), (193, 220), (194, 220), (193, 221), (193, 218), (192, 217), (191, 216), (190, 215), (190, 214), (189, 214), (190, 213), (190, 212), (191, 211), (192, 212), (193, 213), (194, 214), (195, 215), (195, 213), (195, 214), (194, 213), (193, 214), (195, 212), (196, 212), (197, 216), (196, 217), (195, 218), (198, 211), (199, 211), (200, 212), (202, 210), (201, 209), (202, 213), (204, 214), (203, 214), (202, 214), (202, 215), (203, 216), (203, 215), (204, 216), (205, 216), (206, 216), (206, 217), (205, 217), (207, 216), (207, 215), (208, 214), (206, 215), (205, 214), (206, 213), (207, 214), (204, 218), (203, 217), (202, 217), (201, 218), (202, 218), (203, 219), (203, 218), (202, 219), (201, 220), (203, 220), (203, 221), (202, 222), (201, 221), (202, 220), (201, 219), (200, 219), (202, 216), (201, 215), (200, 213), (201, 212), (194, 212), (198, 213), (199, 214), (197, 211), (196, 216), (195, 217), (194, 217), (193, 217), (194, 216), (195, 216), (197, 217), (196, 218), (202, 221), (203, 222), (204, 222), (203, 223), (204, 220), (205, 220), (206, 220), (205, 221), (204, 221), (205, 219), (204, 219), (206, 222), (206, 223), (206, 224), (207, 225), (205, 223), (205, 222), (204, 223), (201, 216), (204, 217), (205, 215), (204, 215), (205, 218), (199, 223), (192, 218), (191, 217), (190, 217), (190, 216), (189, 217), (188, 218), (187, 217), (187, 216), (186, 217), (186, 218), (185, 219), (185, 220), (184, 219), (184, 220), (183, 221), (182, 222), (181, 221), (182, 220), (183, 222), (182, 221), (185, 218), (185, 217), (185, 216), (184, 216), (186, 216), (185, 215), (186, 214), (185, 213), (186, 213), (188, 216), (187, 215), (187, 214), (186, 215), (187, 218), (187, 219), (187, 220), (186, 220), (186, 221), (187, 221), (187, 222), (186, 222), (188, 223), (187, 223), (186, 223), (186, 224), (186, 225), (187, 226), (188, 225), (187, 224), (189, 225), (190, 225), (188, 224), (189, 224), (190, 224), (191, 224), (191, 225), (192, 225), (193, 226), (192, 227), (191, 227), (192, 228), (193, 228), (194, 227), (195, 226), (196, 225), (197, 226), (197, 225), (198, 225), (197, 224), (197, 223), (197, 222), (196, 223), (196, 222), (195, 224), (194, 224), (193, 224), (192, 224), (193, 223), (195, 225), (196, 227), (196, 228), (195, 228), (196, 229), (194, 229), (193, 230), (193, 229), (194, 228), (191, 226), (190, 226), (189, 226), (189, 227), (190, 227), (191, 228), (190, 229), (189, 228), (189, 229), (188, 228), (188, 227), (188, 226), (187, 227), (193, 225), (194, 225), (194, 226), (192, 226), (190, 228), (194, 230), (194, 231), (194, 232), (193, 231), (195, 229), (197, 229), (197, 228), (197, 227), (196, 224), (198, 226), (198, 224), (196, 226), (198, 228), (198, 229), (197, 230), (196, 230), (199, 228), (200, 227), (200, 228), (201, 228), (201, 229), (202, 229), (201, 230), (201, 231), (200, 230), (200, 231), (199, 230), (200, 232), (201, 232), (202, 232), (202, 233), (201, 234), (200, 234), (201, 235), (200, 236), (199, 235), (199, 234), (200, 235), (202, 235), (201, 236), (201, 237), (198, 233), (198, 234), (197, 234), (197, 235), (198, 235), (199, 236), (198, 236), (199, 237), (198, 237), (200, 238), (201, 239), (201, 238), (200, 239), (200, 240), (201, 240), (201, 241), (202, 241), (203, 242), (202, 243), (201, 242), (200, 243), (200, 242), (201, 243), (200, 244), (199, 243), (199, 241), (198, 242), (198, 243), (199, 242), (200, 241), (199, 240), (199, 239), (198, 239), (198, 240), (197, 239), (196, 240), (196, 241), (195, 240), (195, 241), (196, 242), (195, 242), (195, 243), (196, 244), (196, 243), (195, 244), (194, 243), (193, 244), (192, 244), (191, 245), (191, 246), (190, 245), (190, 244), (189, 244), (192, 246), (192, 247), (191, 247), (191, 248), (192, 248), (193, 247), (193, 245), (193, 246), (194, 244), (193, 243), (192, 243), (191, 244), (192, 245), (191, 243), (190, 243), (193, 249), (193, 250), (192, 250), (191, 249), (190, 250), (190, 251), (189, 252), (190, 253), (191, 254), (192, 255), (193, 256), (194, 256), (195, 256), (195, 255), (196, 256), (195, 257), (195, 258), (196, 259), (195, 259), (194, 260), (193, 260), (193, 259), (195, 260), (196, 255), (197, 256), (197, 255), (198, 256), (199, 257), (200, 257), (199, 256), (196, 257), (198, 255), (198, 254), (197, 258), (198, 259), (198, 260), (197, 259), (194, 257), (196, 260), (196, 258), (194, 255), (195, 254), (196, 253), (197, 252), (197, 251), (196, 251), (195, 252), (196, 252), (195, 251), (194, 252), (194, 253), (193, 254), (192, 253), (192, 254), (193, 253), (194, 254), (193, 255), (192, 256), (193, 257), (194, 258), (193, 258), (192, 258), (192, 257), (191, 258), (190, 257), (191, 257), (191, 255), (190, 256), (192, 259), (191, 260), (190, 259), (190, 258), (189, 259), (191, 259), (189, 258), (188, 257), (189, 256), (189, 257), (188, 259), (189, 260), (189, 261), (189, 262), (190, 261), (190, 262), (191, 262), (192, 261), (191, 261), (190, 260), (192, 260), (193, 261), (194, 261), (194, 262), (195, 262), (194, 263), (194, 264), (193, 265), (192, 266), (193, 266), (191, 265), (190, 265), (191, 264), (191, 263), (192, 262), (194, 259), (197, 261), (197, 260), (199, 258), (199, 259), (198, 258), (197, 257), (198, 261), (197, 262), (196, 261), (199, 260), (200, 261), (201, 261), (200, 262), (201, 263), (200, 263), (200, 264), (199, 264), (201, 264), (198, 264), (197, 264), (198, 263), (198, 262), (199, 263), (199, 262), (200, 260), (195, 261), (192, 264), (192, 265), (193, 264), (192, 263), (194, 265), (195, 266), (195, 267), (195, 268), (194, 267), (194, 266), (195, 265), (193, 263), (193, 262), (188, 260), (187, 261), (186, 260), (187, 260), (186, 261), (185, 261), (187, 262), (188, 263), (187, 263), (188, 262), (188, 261), (190, 263), (189, 263), (189, 264), (189, 265), (190, 266), (190, 267), (189, 268), (191, 266), (189, 267), (188, 266), (188, 267), (190, 268), (189, 266), (187, 267), (186, 268), (186, 267), (185, 268), (184, 268), (184, 267), (185, 266), (185, 267), (188, 268), (188, 265), (188, 264), (190, 264), (192, 267), (191, 267), (188, 258), (187, 257), (187, 259), (187, 266), (186, 265), (187, 264), (186, 263), (185, 262), (184, 261), (183, 260), (182, 259), (182, 258), (181, 258), (182, 260), (183, 259), (184, 259), (184, 260), (183, 258), (181, 257), (182, 257), (182, 256), (183, 256), (184, 255), (184, 254), (185, 255), (185, 254), (185, 256), (183, 255), (183, 254), (182, 253), (182, 252), (181, 251), (181, 254), (181, 255), (180, 254), (179, 254), (179, 253), (178, 254), (177, 255), (177, 256), (178, 257), (177, 258), (176, 259), (175, 258), (176, 257), (175, 259), (174, 257), (174, 258), (173, 256), (174, 255), (175, 256), (173, 259), (173, 260), (172, 261), (171, 261), (170, 260), (169, 259), (168, 260), (168, 261), (167, 261), (167, 262), (166, 263), (167, 263), (167, 264), (166, 264), (166, 265), (165, 266), (166, 266), (165, 265), (164, 266), (163, 266), (164, 267), (163, 268), (163, 267), (162, 267), (162, 266), (161, 266), (161, 267), (162, 268), (162, 265), (163, 264), (162, 263), (162, 262), (162, 264), (163, 263), (164, 263), (165, 264), (164, 264), (164, 262), (166, 267), (167, 268), (168, 267), (169, 268), (170, 267), (169, 267), (170, 266), (171, 267), (172, 267), (173, 268), (173, 267), (172, 266), (171, 265), (172, 265), (173, 265), (173, 266), (174, 267), (175, 266), (174, 265), (174, 266), (175, 267), (176, 268), (176, 266), (176, 267), (173, 264), (174, 263), (173, 262), (173, 261), (174, 260), (172, 258), (173, 258), (172, 257), (171, 257), (170, 256), (169, 256), (168, 255), (170, 257), (171, 258), (171, 259), (174, 256), (175, 255), (174, 254), (175, 253), (176, 252), (176, 251), (176, 250), (175, 249), (176, 249), (176, 248), (175, 248), (174, 248), (173, 247), (173, 248), (172, 248), (172, 249), (172, 247), (171, 249), (171, 250), (170, 250), (170, 251), (169, 252), (168, 252), (169, 253), (170, 253), (170, 254), (171, 254), (168, 254), (167, 255), (166, 254), (166, 256), (165, 255), (164, 254), (163, 255), (164, 255), (163, 254), (164, 253), (162, 255), (161, 254), (160, 254), (159, 254), (161, 253), (162, 252), (161, 251), (160, 250), (161, 250), (162, 250), (161, 249), (163, 249), (164, 248), (165, 249), (166, 248), (165, 247), (164, 247), (163, 246), (164, 245), (164, 246), (162, 245), (163, 245), (162, 246), (162, 247), (163, 247), (162, 244), (161, 243), (161, 244), (160, 244), (161, 245), (165, 244), (164, 243), (163, 242), (163, 243), (164, 242), (164, 241), (165, 242), (165, 243), (166, 242), (167, 241), (167, 240), (166, 240), (168, 241), (169, 242), (169, 243), (168, 242), (167, 243), (166, 243), (166, 244), (166, 245), (165, 245), (164, 244), (166, 246), (167, 245), (168, 246), (167, 244), (168, 244), (167, 246), (168, 245), (167, 242), (170, 244), (171, 243), (171, 242), (170, 242), (170, 243), (170, 241), (171, 241), (172, 241), (172, 243), (172, 242), (173, 241), (173, 240), (174, 239), (174, 238), (173, 238), (172, 238), (173, 239), (175, 238), (172, 239), (173, 242), (174, 243), (173, 244), (174, 244), (175, 244), (176, 244), (176, 243), (177, 243), (178, 243), (179, 243), (178, 244), (177, 245), (177, 246), (178, 246), (178, 247), (179, 246), (179, 245), (178, 245), (177, 242), (178, 241), (179, 240), (179, 241), (178, 240), (177, 240), (176, 240), (176, 241), (175, 241), (177, 239), (178, 242), (177, 241), (176, 242), (174, 242), (171, 244), (173, 243), (175, 245), (174, 245), (175, 243), (177, 244), (179, 247), (179, 248), (178, 249), (179, 250), (177, 248), (177, 249), (175, 251), (174, 252), (173, 253), (174, 253), (173, 254), (175, 252), (176, 253), (177, 253), (178, 253), (177, 254), (176, 254), (176, 255), (177, 252), (177, 251), (177, 250), (178, 248), (179, 249), (178, 250), (179, 251), (178, 251), (179, 252), (180, 253), (180, 255), (179, 256), (180, 256), (181, 256), (180, 257), (182, 255), (182, 254), (181, 253), (183, 252), (184, 251), (185, 251), (186, 252), (187, 253), (186, 250), (185, 252), (186, 253), (187, 254), (188, 254), (188, 253), (187, 252), (186, 255), (186, 254), (187, 255), (184, 256), (185, 257), (184, 257), (186, 257), (185, 258), (185, 259), (183, 257), (186, 256), (186, 258), (186, 259), (187, 258), (186, 262), (185, 263), (184, 262), (183, 262), (182, 262), (181, 262), (181, 261), (180, 262), (181, 263), (182, 263), (182, 264), (183, 263), (180, 260), (179, 261), (178, 261), (178, 260), (177, 261), (178, 262), (178, 263), (179, 262), (179, 263), (180, 264), (179, 265), (178, 266), (180, 265), (181, 265), (182, 265), (183, 266), (184, 265), (183, 267), (182, 266), (184, 266), (186, 266), (187, 268), (185, 265), (185, 264), (183, 268), (181, 267), (180, 268), (179, 267), (178, 267), (179, 268), (177, 267), (175, 265), (177, 266), (177, 265), (178, 264), (179, 264), (180, 261), (183, 264), (183, 265), (181, 266), (182, 267), (180, 267), (179, 266), (181, 268), (182, 268), (178, 268), (176, 265), (178, 265), (174, 264), (175, 264), (176, 264), (177, 263), (180, 266), (175, 263), (175, 262), (176, 262), (177, 262), (176, 263), (176, 261), (176, 260), (177, 260), (177, 259), (174, 262), (174, 261), (172, 260), (171, 262), (170, 262), (172, 263), (171, 260), (172, 262), (175, 261), (169, 260), (168, 259), (167, 259), (166, 259), (167, 260), (168, 262), (168, 265), (168, 263), (169, 264), (170, 265), (169, 265), (170, 263), (171, 264), (172, 264), (171, 266), (171, 263), (170, 264), (170, 259), (169, 258), (170, 258), (169, 257), (171, 255), (170, 255), (172, 255), (172, 256), (176, 256), (173, 255), (172, 254), (171, 253), (172, 252), (171, 252), (172, 253), (173, 251), (173, 250), (173, 249), (174, 249), (174, 247), (171, 246), (170, 245), (169, 246), (169, 245), (170, 246), (172, 246), (173, 246), (171, 245), (171, 247), (171, 248), (170, 249), (171, 251), (172, 251), (172, 250), (174, 251), (174, 250), (173, 245), (172, 245), (172, 244), (174, 246), (175, 246), (176, 246), (175, 247), (176, 247), (175, 250), (177, 247), (180, 248), (180, 249), (181, 249), (182, 248), (182, 249), (183, 248), (182, 247), (183, 247), (184, 247), (182, 246), (185, 247), (185, 246), (184, 246), (183, 246), (183, 245), (182, 245), (183, 244), (181, 246), (180, 245), (180, 246), (179, 244), (180, 244), (184, 248), (183, 249), (186, 248), (187, 247), (188, 247), (188, 248), (188, 249), (189, 250), (191, 250), (190, 249), (189, 248), (189, 249), (190, 247), (192, 249), (193, 251), (194, 250), (194, 249), (190, 248), (189, 247), (187, 249), (186, 249), (185, 248), (184, 249), (182, 250), (181, 250), (180, 251), (183, 251), (183, 250), (182, 251), (175, 254), (178, 258), (179, 259), (179, 260), (177, 264), (180, 263), (186, 264), (191, 268), (192, 268), (193, 268), (194, 268), (196, 265), (197, 263), (196, 264), (195, 264), (196, 263), (191, 256), (190, 254), (189, 253), (188, 255), (189, 254), (188, 252), (189, 251), (188, 251), (188, 250), (187, 250), (185, 249), (187, 248), (193, 248), (185, 250), (184, 250), (185, 253), (181, 247), (180, 247), (180, 250), (179, 257), (181, 260), (181, 259), (183, 253), (181, 252), (178, 255), (178, 256), (179, 255), (177, 257), (184, 253), (184, 252), (187, 256), (185, 260), (186, 251), (187, 251), (190, 252), (191, 251), (192, 252), (193, 252), (198, 257), (200, 259), (201, 260), (202, 260), (199, 261), (200, 258), (201, 259), (202, 258), (201, 257), (201, 256), (201, 255), (201, 254), (200, 254), (200, 253), (202, 255), (202, 256), (203, 254), (203, 255), (204, 256), (204, 254), (203, 253), (205, 253), (206, 254), (205, 254), (206, 253), (205, 252), (205, 251), (206, 252), (206, 250), (205, 250), (204, 250), (203, 250), (202, 251), (202, 252), (201, 253), (201, 252), (201, 250), (200, 249), (199, 250), (198, 251), (199, 252), (199, 251), (200, 251), (200, 252), (201, 251), (200, 250), (199, 248), (200, 248), (201, 247), (202, 246), (201, 246), (202, 245), (202, 244), (201, 244), (202, 242), (203, 244), (203, 246), (203, 247), (204, 247), (205, 248), (206, 247), (207, 248), (206, 248), (206, 249), (205, 249), (204, 249), (204, 248), (205, 247), (207, 247), (207, 246), (208, 247), (206, 246), (204, 246), (205, 246), (207, 245), (208, 246), (203, 245), (202, 247), (202, 248), (201, 249), (202, 250), (203, 248), (201, 245), (203, 243), (204, 242), (204, 244), (204, 243), (202, 240), (202, 239), (203, 238), (204, 239), (205, 239), (204, 238), (204, 237), (205, 238), (203, 237), (205, 237), (204, 236), (205, 236), (205, 235), (206, 236), (207, 237), (206, 237), (202, 236), (203, 235), (203, 236), (202, 237), (198, 238), (199, 238), (197, 242), (198, 241), (197, 240), (196, 239), (195, 239), (194, 240), (194, 241), (193, 242), (192, 241), (192, 242), (189, 243), (188, 244), (187, 243), (187, 242), (186, 243), (187, 244), (187, 245), (186, 245), (184, 245), (186, 247), (181, 248), (188, 256), (184, 263), (181, 264), (173, 263), (174, 268), (175, 268), (175, 260), (174, 259), (175, 257), (176, 258), (177, 268), (183, 261), (182, 261), (180, 258), (184, 258), (193, 267), (187, 265), (195, 263), (196, 262), (202, 259), (203, 259), (203, 260), (202, 261), (204, 261), (205, 262), (206, 263), (207, 262), (208, 262), (208, 261), (206, 261), (206, 260), (206, 262), (204, 262), (203, 262), (201, 262), (202, 263), (202, 264), (202, 262), (203, 261), (201, 265), (200, 266), (199, 267), (199, 266), (198, 267), (197, 267), (199, 268), (198, 268), (196, 266), (198, 265), (199, 265), (203, 263), (204, 264), (179, 258), (178, 259), (172, 268), (171, 268), (170, 268), (169, 261), (169, 262), (170, 261), (169, 263), (167, 265), (168, 266), (169, 266), (184, 264), (196, 267), (197, 266), (198, 266), (197, 265), (200, 265), (201, 266), (201, 267), (202, 267), (202, 266), (201, 268), (203, 266), (203, 265), (203, 264), (204, 265), (204, 266), (205, 267), (204, 267), (203, 268), (203, 267), (205, 265), (205, 264), (207, 263), (206, 264), (207, 264), (207, 265), (207, 266), (206, 267), (206, 266), (205, 266), (204, 268), (205, 268), (208, 267), (208, 266), (209, 266), (209, 267), (210, 267), (209, 268), (208, 268), (206, 265), (208, 264), (205, 263), (202, 265), (200, 267), (202, 268), (200, 268), (196, 268), (197, 268), (204, 263), (201, 258), (202, 257), (203, 257), (204, 257), (205, 258), (206, 258), (205, 259), (204, 260), (205, 261), (209, 263), (209, 264), (208, 265), (208, 263), (209, 262), (207, 261), (207, 260), (206, 259), (207, 259), (208, 258), (208, 257), (209, 259), (208, 260), (210, 263), (205, 260), (204, 259), (209, 260), (210, 259), (210, 260), (211, 261), (212, 262), (211, 262), (212, 261), (212, 263), (213, 264), (212, 264), (213, 263), (213, 262), (212, 260), (213, 259), (213, 260), (214, 261), (214, 262), (215, 263), (215, 264), (216, 265), (215, 266), (214, 266), (213, 265), (214, 264), (214, 265), (213, 267), (214, 268), (213, 268), (212, 268), (212, 267), (213, 266), (212, 265), (211, 266), (212, 266), (215, 267), (216, 267), (217, 266), (218, 266), (218, 265), (217, 264), (216, 263), (216, 262), (217, 261), (218, 261), (219, 261), (218, 262), (219, 262), (218, 260), (219, 260), (220, 260), (216, 260), (217, 260), (218, 259), (216, 259), (215, 259), (215, 258), (216, 258), (216, 257), (215, 257), (214, 257), (213, 258), (213, 257), (214, 258), (214, 260), (213, 261), (211, 263), (210, 262), (211, 264), (209, 261), (207, 258), (208, 259), (206, 257), (209, 257), (209, 256), (208, 255), (208, 256), (210, 257), (211, 257), (212, 257), (212, 256), (210, 258), (212, 258), (211, 256), (210, 256), (211, 255), (212, 254), (209, 255), (210, 254), (210, 255), (208, 254), (207, 256), (207, 257), (205, 257), (205, 256), (204, 258), (203, 258), (199, 255), (199, 254), (199, 253), (197, 253), (198, 250), (196, 250), (198, 252), (197, 250), (199, 249), (198, 248), (198, 249), (197, 249), (195, 249), (196, 249), (196, 248), (198, 253), (202, 249), (201, 248), (200, 247), (199, 247), (207, 249), (207, 250), (206, 251), (207, 252), (207, 254), (206, 255), (209, 258), (210, 265), (211, 265), (215, 265), (216, 264), (217, 265), (216, 266), (217, 263), (218, 263), (220, 262), (219, 263), (219, 264), (219, 265), (218, 264), (220, 265), (221, 264), (221, 265), (222, 264), (223, 263), (224, 264), (224, 265), (225, 264), (226, 265), (225, 265), (226, 266), (225, 266), (225, 267), (226, 267), (227, 267), (228, 268), (229, 267), (229, 266), (228, 265), (228, 266), (228, 267), (229, 268), (230, 267), (230, 268), (227, 268), (224, 266), (226, 263), (227, 264), (227, 266), (227, 265), (228, 264), (229, 263), (230, 263), (231, 264), (230, 265), (229, 264), (231, 266), (231, 265), (230, 264), (231, 263), (232, 262), (233, 263), (233, 264), (232, 263), (233, 261), (234, 262), (234, 263), (233, 265), (232, 264), (230, 266), (229, 265), (231, 262), (231, 261), (232, 260), (231, 259), (230, 260), (229, 259), (229, 260), (230, 259), (230, 258), (231, 260), (232, 261), (233, 260), (234, 261), (233, 259), (234, 258), (235, 259), (236, 259), (236, 260), (235, 258), (235, 257), (234, 257), (234, 259), (234, 260), (235, 260), (233, 262), (232, 265), (231, 267), (231, 268), (228, 263), (228, 262), (227, 261), (228, 260), (227, 260), (227, 259), (226, 260), (229, 258), (232, 259), (236, 261), (235, 261), (236, 262), (235, 263), (236, 263), (237, 261), (238, 261), (238, 262), (238, 263), (239, 261), (240, 262), (241, 262), (241, 261), (240, 261), (240, 260), (242, 260), (241, 260), (242, 259), (241, 259), (242, 261), (242, 262), (243, 261), (243, 260), (243, 259), (243, 263), (244, 264), (245, 265), (246, 266), (246, 264), (247, 263), (248, 263), (249, 263), (249, 264), (248, 265), (249, 266), (248, 266), (248, 264), (248, 262), (249, 262), (249, 261), (248, 261), (248, 260), (249, 259), (248, 258), (248, 259), (247, 259), (247, 260), (247, 261), (246, 262), (247, 262), (246, 259), (246, 260), (245, 260), (245, 259), (244, 258), (245, 257), (244, 257), (245, 256), (247, 254), (248, 255), (249, 254), (250, 253), (245, 253), (247, 252), (247, 256), (246, 256), (248, 254), (249, 255), (247, 255), (248, 256), (248, 257), (247, 257), (247, 258), (245, 261), (245, 262), (246, 258), (246, 257), (245, 258), (245, 252), (242, 245), (241, 246), (246, 241), (240, 241), (240, 240), (240, 242), (239, 242), (241, 243), (240, 244), (239, 241), (238, 240), (238, 241), (237, 241), (237, 239), (237, 240), (236, 239), (237, 238), (235, 238), (235, 237), (235, 236), (236, 236), (236, 237), (237, 235), (237, 236), (238, 235), (238, 236), (237, 237), (234, 234), (233, 235), (233, 237), (234, 238), (238, 237), (239, 237), (240, 236), (240, 237), (243, 236), (243, 237), (241, 238), (241, 237), (242, 237), (251, 238), (252, 239), (251, 239), (251, 233), (252, 233), (259, 234), (255, 239), (248, 232), (247, 232), (247, 231), (255, 238), (253, 242), (251, 237), (268, 220), (268, 218), (243, 222), (242, 223), (243, 214), (239, 213), (238, 212), (238, 211), (238, 210), (238, 209), (238, 208), (237, 209), (236, 208), (237, 207), (236, 207), (236, 206), (235, 205), (234, 205), (233, 206), (232, 205), (231, 202), (228, 202), (227, 202), (228, 203), (228, 201), (229, 201), (230, 201), (230, 200), (231, 201), (231, 200), (232, 200), (232, 199), (233, 199), (234, 200), (232, 202), (235, 200), (233, 200), (235, 199), (236, 198), (236, 197), (234, 199), (234, 198), (232, 201), (232, 198), (232, 197), (231, 198), (231, 197), (232, 196), (231, 195), (232, 195), (233, 194), (233, 193), (234, 192), (235, 192), (235, 193), (234, 193), (234, 191), (236, 192), (236, 195), (235, 194), (239, 192), (239, 194), (238, 194), (240, 193), (240, 192), (241, 194), (237, 195), (237, 196), (237, 197), (235, 197), (235, 196), (234, 195), (233, 195), (233, 196), (234, 197), (238, 197), (239, 196), (240, 195), (240, 196), (241, 195), (242, 195), (243, 195), (242, 194), (243, 193), (243, 194), (244, 195), (244, 194), (245, 193), (246, 193), (244, 193), (245, 194), (246, 194), (247, 194), (247, 195), (251, 192), (248, 192), (247, 190), (246, 192), (247, 191), (247, 192), (246, 191), (246, 195), (247, 196), (248, 196), (249, 197), (248, 197), (247, 197), (246, 198), (247, 198), (246, 199), (245, 200), (246, 201), (247, 202), (247, 201), (245, 210), (244, 209), (248, 211), (247, 212), (246, 211), (247, 211), (247, 210), (242, 226), (243, 225), (243, 230), (244, 232), (245, 231), (241, 233), (240, 232), (243, 233), (244, 236), (244, 235), (239, 240), (238, 239), (239, 239), (239, 238), (237, 242), (237, 243), (238, 242), (236, 241), (236, 242), (236, 238), (238, 238), (244, 234), (245, 235), (246, 234), (245, 236), (245, 237), (245, 234), (246, 233), (245, 233), (244, 233), (240, 231), (241, 231), (239, 231), (233, 231), (232, 232), (233, 232), (233, 233), (234, 235), (232, 234), (231, 235), (230, 234), (228, 234), (226, 236), (225, 238), (226, 239), (226, 238), (227, 237), (227, 235), (228, 232), (229, 232), (230, 233), (231, 232), (230, 231), (229, 231), (230, 230), (229, 230), (228, 231), (231, 229), (232, 229), (230, 228), (229, 222), (229, 223), (229, 224), (230, 223), (225, 227), (225, 228), (224, 227), (224, 228), (226, 226), (225, 222), (231, 227), (230, 226), (234, 228), (234, 229), (235, 229), (236, 229), (236, 223), (253, 207), (243, 210), (245, 204), (245, 205), (245, 203), (250, 201), (254, 196), (255, 195), (255, 194), (254, 195), (256, 194), (250, 195), (250, 196), (251, 196), (250, 197), (251, 197), (251, 198), (252, 197), (250, 198), (249, 198), (247, 193), (246, 196), (245, 195), (256, 188), (257, 189), (258, 190), (257, 191), (256, 192), (256, 191), (255, 192), (257, 192), (258, 191), (258, 192), (259, 192), (260, 193), (259, 194), (260, 195), (261, 193), (262, 193), (263, 192), (264, 192), (265, 192), (266, 193), (266, 194), (265, 194), (266, 195), (264, 195), (264, 194), (263, 193), (263, 194), (264, 193), (263, 191), (262, 191), (263, 190), (264, 190), (264, 191), (265, 191), (266, 191), (266, 192), (267, 191), (267, 192), (266, 190), (265, 189), (266, 188), (265, 187), (264, 186), (263, 187), (262, 186), (265, 185), (264, 184), (264, 185), (266, 184), (267, 183), (268, 183), (267, 184), (266, 183), (266, 185), (267, 185), (268, 184), (265, 186), (264, 187), (265, 188), (263, 186), (263, 185), (262, 187), (263, 188), (263, 189), (262, 190), (262, 192), (261, 191), (261, 190), (262, 189), (261, 188), (260, 187), (259, 188), (258, 189), (258, 188), (259, 189), (260, 189), (262, 188), (261, 189), (261, 186), (260, 186), (258, 183), (257, 184), (255, 183), (254, 182), (255, 184), (254, 183), (253, 184), (256, 186), (259, 190), (256, 193), (257, 193), (255, 193), (256, 195), (256, 196), (257, 197), (257, 198), (258, 198), (259, 197), (259, 198), (260, 199), (261, 198), (260, 197), (264, 197), (263, 196), (265, 197), (265, 198), (267, 203), (268, 204), (268, 205), (267, 204), (266, 204), (267, 205), (266, 205), (268, 203), (259, 205), (260, 202), (262, 201), (261, 201), (268, 196), (267, 195), (265, 196), (265, 195), (267, 194), (268, 194), (268, 193), (267, 189), (265, 193), (265, 190), (264, 189), (264, 188), (267, 188), (268, 187), (267, 187), (266, 187), (268, 186), (268, 188), (266, 186), (260, 188), (261, 192), (258, 193), (255, 191), (256, 190), (254, 193), (253, 191), (252, 191), (251, 185), (241, 193), (242, 196), (241, 197), (242, 198), (241, 198), (241, 199), (242, 200), (242, 199), (241, 200), (240, 199), (239, 199), (240, 200), (239, 200), (238, 201), (237, 202), (236, 203), (236, 202), (234, 201), (230, 198), (229, 197), (231, 199), (233, 198), (233, 197), (231, 196), (230, 195), (231, 194), (232, 194), (232, 193), (231, 193), (234, 194), (235, 195), (234, 196), (235, 198), (236, 200), (237, 200), (236, 201), (237, 201), (238, 202), (238, 203), (239, 203), (238, 204), (237, 204), (237, 203), (236, 204), (235, 204), (236, 205), (235, 206), (237, 206), (235, 208), (234, 206), (235, 207), (235, 209), (234, 207), (234, 208), (234, 209), (235, 210), (234, 211), (233, 210), (233, 209), (232, 208), (230, 208), (231, 206), (232, 206), (229, 208), (228, 207), (233, 207), (233, 208), (234, 210), (235, 211), (237, 208), (238, 207), (239, 208), (240, 209), (239, 210), (239, 211), (239, 212), (239, 209), (240, 207), (239, 206), (239, 207), (238, 206), (238, 205), (240, 202), (241, 201), (240, 201), (242, 202), (241, 202), (242, 203), (243, 203), (243, 202), (244, 203), (244, 202), (243, 201), (244, 200), (243, 199), (244, 201), (244, 204), (243, 204), (241, 204), (240, 203), (241, 203), (240, 204), (239, 204), (239, 201), (239, 202), (239, 205), (240, 206), (241, 206), (242, 205), (240, 205), (241, 205), (236, 209), (237, 210), (236, 210), (237, 211), (236, 212), (235, 212), (237, 213), (236, 214), (237, 215), (236, 215), (235, 214), (234, 214), (233, 213), (233, 212), (232, 214), (234, 213), (237, 214), (233, 214), (232, 212), (231, 212), (232, 213), (233, 215), (232, 216), (232, 217), (231, 217), (230, 216), (230, 215), (230, 214), (229, 214), (230, 213), (230, 212), (229, 211), (229, 212), (228, 213), (227, 214), (227, 215), (226, 215), (225, 214), (225, 215), (225, 216), (224, 216), (224, 215), (224, 217), (223, 216), (222, 215), (223, 208), (224, 207), (225, 208), (226, 211), (225, 211), (224, 211), (225, 212), (225, 213), (226, 212), (227, 213), (228, 214), (228, 212), (230, 211), (231, 210), (232, 209), (231, 209), (232, 210), (231, 211), (229, 210), (228, 210), (229, 209), (226, 213), (227, 212), (227, 211), (230, 210), (234, 212), (232, 211), (228, 209), (227, 210), (230, 209), (231, 208), (237, 205), (242, 201), (243, 200), (244, 199), (244, 198), (245, 197), (247, 199), (246, 200), (245, 201), (246, 202), (247, 204), (246, 203), (247, 200), (248, 199), (248, 200), (248, 198), (246, 197), (244, 196), (249, 190), (254, 194), (253, 194), (255, 196), (256, 197), (257, 196), (258, 195), (258, 196), (257, 195), (258, 197), (260, 196), (259, 196), (259, 199), (260, 198), (259, 202), (240, 198), (239, 198), (240, 197), (241, 196), (242, 197), (243, 198), (243, 197), (244, 197), (243, 196), (238, 199), (237, 199), (238, 200), (233, 202), (238, 198), (242, 204), (236, 213), (237, 212), (242, 206), (236, 199), (237, 198), (235, 201), (233, 192), (247, 183), (249, 183), (237, 183), (232, 192), (232, 191), (236, 194), (230, 197), (229, 196), (229, 195), (228, 194), (227, 195), (228, 195), (229, 194), (229, 193), (230, 194), (228, 192), (228, 189), (226, 190), (225, 191), (224, 190), (223, 189), (224, 188), (222, 189), (221, 188), (222, 187), (221, 187), (222, 188), (223, 190), (224, 191), (223, 192), (222, 191), (222, 190), (223, 191), (224, 192), (225, 193), (226, 194), (227, 193), (227, 194), (229, 198), (228, 198), (228, 199), (227, 200), (229, 199), (229, 200), (225, 205), (221, 207), (222, 207), (221, 206), (222, 206), (222, 205), (222, 202), (221, 202), (221, 204), (221, 205), (220, 207), (219, 206), (219, 207), (219, 208), (223, 210), (223, 214), (224, 214), (226, 216), (227, 217), (226, 217), (225, 217), (227, 216), (228, 217), (229, 217), (230, 217), (230, 218), (229, 216), (229, 218), (229, 215), (228, 216), (230, 219), (233, 217), (233, 216), (234, 215), (234, 216), (235, 215), (242, 212), (238, 213), (238, 214), (239, 214), (238, 216), (237, 217), (237, 218), (236, 216), (235, 216), (235, 217), (234, 219), (235, 222), (238, 219), (234, 217), (233, 218), (238, 218), (239, 217), (239, 216), (239, 215), (238, 217), (240, 217), (241, 219), (242, 218), (242, 217), (242, 219), (241, 218), (241, 217), (240, 216), (241, 216), (239, 226), (238, 227), (238, 226), (240, 230), (233, 226), (239, 236), (237, 244), (236, 244), (237, 245), (236, 246), (237, 246), (238, 245), (239, 246), (239, 245), (238, 246), (238, 247), (237, 247), (236, 247), (235, 246), (235, 247), (234, 248), (233, 247), (232, 248), (231, 248), (231, 247), (231, 249), (231, 250), (232, 249), (233, 250), (234, 250), (234, 251), (233, 251), (232, 252), (233, 252), (234, 252), (235, 253), (235, 254), (234, 253), (233, 253), (232, 253), (231, 252), (230, 251), (230, 252), (231, 251), (230, 250), (229, 250), (229, 249), (230, 248), (229, 247), (228, 247), (227, 248), (226, 247), (226, 248), (227, 246), (226, 246), (227, 247), (228, 245), (229, 244), (230, 245), (229, 246), (229, 245), (230, 243), (230, 244), (229, 243), (229, 242), (230, 242), (228, 241), (227, 240), (228, 239), (229, 240), (227, 239), (227, 238), (223, 235), (221, 234), (219, 233), (217, 236), (215, 234), (214, 233), (214, 232), (215, 231), (214, 235), (217, 230), (217, 229), (219, 228), (221, 229), (221, 231), (222, 228), (223, 223), (224, 223), (223, 222), (224, 222), (224, 221), (224, 220), (223, 221), (222, 222), (221, 222), (221, 223), (222, 221), (221, 220), (222, 220), (223, 220), (223, 219), (222, 218), (221, 218), (220, 218), (219, 219), (220, 220), (221, 221), (218, 220), (219, 221), (220, 221), (219, 222), (218, 223), (217, 224), (218, 224), (219, 225), (217, 223), (216, 223), (216, 222), (217, 222), (215, 223), (213, 226), (212, 230), (213, 231), (212, 232), (211, 232), (210, 232), (209, 232), (206, 233), (207, 234), (208, 233), (209, 234), (208, 230), (207, 230), (206, 231), (207, 231), (208, 234), (208, 235), (207, 235), (207, 236), (206, 238), (204, 240), (205, 241), (204, 241), (205, 242), (206, 243), (207, 242), (207, 243), (207, 244), (206, 244), (208, 245), (205, 245), (204, 245), (205, 244), (208, 243), (209, 242), (210, 243), (210, 242), (211, 243), (211, 244), (210, 244), (209, 244), (208, 244), (209, 245), (209, 246), (212, 243), (210, 245), (208, 241), (208, 240), (208, 239), (207, 238), (208, 237), (208, 236), (208, 238), (206, 239), (203, 240), (203, 239), (202, 238), (203, 241), (200, 245), (199, 246), (198, 246), (197, 245), (197, 244), (198, 244), (199, 245), (197, 246), (198, 245), (199, 244), (206, 241), (207, 240), (207, 239), (204, 235), (199, 233), (199, 232), (198, 231), (199, 231), (198, 232), (197, 232), (197, 231), (197, 233), (198, 230), (196, 233), (195, 234), (194, 235), (195, 235), (194, 234), (196, 235), (196, 234), (195, 233), (194, 233), (193, 233), (192, 234), (191, 235), (191, 236), (192, 235), (192, 236), (192, 237), (192, 238), (191, 239), (192, 240), (191, 238), (191, 237), (190, 237), (190, 236), (191, 234), (190, 235), (192, 233), (189, 235), (189, 236), (188, 235), (187, 236), (188, 236), (189, 237), (188, 238), (188, 239), (187, 238), (186, 237), (185, 238), (185, 239), (185, 240), (185, 241), (184, 241), (183, 242), (184, 242), (183, 240), (182, 240), (182, 241), (181, 242), (181, 241), (180, 240), (181, 240), (182, 239), (181, 238), (182, 238), (182, 237), (183, 236), (182, 235), (183, 234), (184, 235), (185, 236), (185, 237), (184, 236), (187, 237), (186, 238), (187, 239), (186, 240), (187, 240), (186, 239), (184, 237), (184, 238), (186, 235), (182, 236), (181, 237), (181, 236), (180, 236), (179, 235), (180, 234), (181, 235), (183, 237), (184, 239), (183, 238), (183, 239), (184, 240), (180, 239), (179, 238), (179, 242), (175, 242), (174, 241), (174, 240), (172, 240), (170, 240), (169, 240), (168, 240), (166, 241), (165, 240), (169, 241), (171, 239), (170, 239), (168, 239), (165, 239), (166, 239), (167, 238), (168, 243), (169, 244), (171, 240), (171, 238), (172, 237), (171, 237), (173, 236), (174, 235), (173, 235), (175, 236), (176, 235), (176, 234), (175, 233), (174, 232), (174, 231), (175, 230), (175, 232), (173, 231), (172, 230), (172, 229), (173, 229), (174, 230), (174, 229), (176, 229), (175, 231), (176, 232), (176, 233), (176, 231), (176, 230), (177, 229), (178, 229), (179, 230), (179, 231), (178, 230), (177, 230), (174, 234), (175, 234), (174, 233), (177, 232), (178, 233), (178, 232), (179, 232), (180, 232), (179, 233), (177, 233), (177, 231), (178, 231), (173, 232), (173, 230), (172, 231), (172, 232), (172, 233), (171, 234), (170, 235), (171, 235), (170, 234), (169, 235), (169, 236), (169, 237), (169, 238), (168, 238), (168, 237), (168, 236), (167, 236), (168, 235), (170, 236), (171, 236), (170, 238), (164, 240), (165, 241), (162, 242), (162, 241), (161, 242), (162, 243), (163, 244), (167, 239), (167, 237), (166, 237), (165, 236), (165, 235), (166, 234), (167, 234), (167, 233), (166, 235), (170, 237), (172, 236), (172, 235), (173, 237), (169, 239), (159, 243), (160, 243), (159, 242), (158, 242), (157, 242), (158, 241), (160, 241), (159, 241), (161, 241), (160, 240), (159, 240), (159, 239), (159, 238), (159, 237), (158, 236), (157, 237), (158, 237), (158, 238), (158, 239), (157, 238), (156, 239), (155, 240), (154, 241), (155, 242), (156, 242), (156, 243), (157, 243), (156, 241), (155, 241), (154, 240), (153, 240), (154, 239), (154, 242), (152, 241), (153, 239), (154, 238), (153, 237), (152, 236), (151, 237), (150, 237), (150, 238), (150, 239), (151, 238), (149, 239), (148, 240), (148, 239), (149, 238), (149, 236), (148, 237), (149, 237), (151, 236), (152, 237), (152, 238), (155, 239), (155, 237), (155, 238), (154, 237), (156, 236), (156, 237), (156, 238), (153, 238), (155, 236), (154, 235), (153, 234), (153, 235), (153, 236), (150, 236), (149, 235), (150, 235), (150, 234), (151, 235), (151, 234), (151, 233), (152, 234), (152, 233), (154, 234), (154, 233), (154, 232), (153, 232), (153, 233), (155, 232), (155, 233), (156, 232), (155, 234), (155, 235), (156, 235), (157, 236), (158, 235), (159, 235), (159, 236), (160, 236), (161, 235), (161, 236), (160, 237), (160, 238), (160, 239), (161, 240), (163, 241), (167, 247), (166, 238), (165, 238), (172, 234), (173, 234), (173, 233), (177, 236), (176, 236), (175, 235), (175, 237), (174, 237), (177, 235), (178, 235), (178, 236), (178, 237), (177, 238), (176, 239), (170, 247), (169, 248), (169, 249), (169, 250), (170, 252), (168, 253), (167, 253), (167, 254), (169, 255), (168, 256), (171, 256), (172, 259), (168, 264), (167, 266), (167, 267), (168, 268), (166, 261), (166, 260), (166, 258), (167, 257), (167, 256), (165, 256), (166, 255), (166, 253), (165, 253), (166, 252), (167, 251), (168, 250), (168, 251), (169, 251), (169, 254), (167, 252), (165, 251), (164, 251), (164, 250), (162, 248), (161, 246), (161, 247), (160, 248), (160, 249), (159, 250), (159, 251), (159, 252), (158, 252), (158, 251), (157, 251), (156, 252), (155, 253), (156, 253), (157, 253), (158, 250), (159, 249), (157, 250), (157, 249), (158, 249), (157, 248), (156, 248), (155, 248), (154, 247), (156, 249), (155, 250), (160, 251), (161, 252), (162, 251), (160, 252), (160, 253), (158, 253), (157, 254), (157, 252), (155, 252), (154, 253), (154, 254), (155, 255), (155, 254), (154, 255), (153, 254), (152, 253), (152, 255), (153, 255), (152, 256), (152, 257), (151, 256), (151, 257), (150, 257), (150, 256), (151, 255), (151, 254), (151, 253), (151, 252), (152, 251), (152, 252), (152, 254), (153, 253), (150, 252), (151, 251), (152, 250), (152, 249), (153, 250), (153, 249), (153, 248), (152, 247), (153, 246), (154, 246), (155, 247), (155, 246), (156, 246), (156, 247), (158, 247), (159, 247), (158, 248), (160, 247), (160, 245), (159, 245), (159, 244), (158, 243), (161, 239), (162, 240), (163, 240), (165, 248), (166, 247), (167, 249), (168, 249), (167, 250), (169, 247), (168, 247), (166, 249), (167, 248), (175, 239), (176, 237), (174, 236), (178, 239), (179, 239), (180, 238), (181, 239), (178, 238), (179, 236), (181, 234), (180, 235), (178, 234), (163, 239), (164, 239), (164, 237), (164, 238), (165, 234), (164, 234), (164, 235), (168, 234), (169, 234), (167, 235), (165, 246), (164, 249), (165, 250), (166, 250), (163, 248), (168, 248), (166, 251), (173, 252), (170, 248), (171, 233), (186, 246), (186, 244), (185, 245), (187, 246), (188, 246), (188, 245), (189, 246), (185, 244), (184, 244), (185, 243), (181, 244), (182, 243), (183, 243), (184, 243), (185, 242), (186, 242), (186, 241), (187, 241), (188, 243), (189, 245), (190, 246), (181, 245), (176, 245), (178, 252), (180, 252), (189, 255), (192, 251), (194, 248), (194, 247), (194, 246), (195, 247), (195, 246), (195, 248), (195, 250), (196, 246), (194, 251), (195, 253), (190, 255), (191, 252), (191, 253), (182, 244), (180, 243), (176, 238), (177, 237), (171, 232), (170, 231), (177, 234), (180, 233), (182, 233), (183, 232), (183, 231), (184, 230), (183, 229), (183, 228), (183, 230), (184, 231), (185, 230), (182, 229), (181, 228), (180, 229), (179, 229), (179, 228), (180, 227), (181, 227), (181, 226), (180, 226), (179, 226), (178, 227), (177, 226), (176, 225), (176, 227), (175, 226), (174, 225), (173, 224), (173, 223), (172, 224), (171, 224), (172, 225), (171, 225), (170, 226), (171, 226), (169, 225), (170, 224), (170, 225), (169, 224), (169, 223), (170, 222), (169, 221), (168, 221), (167, 221), (168, 220), (167, 220), (167, 219), (166, 220), (166, 219), (165, 218), (166, 217), (167, 218), (168, 217), (169, 216), (167, 216), (168, 215), (169, 214), (168, 216), (167, 215), (165, 217), (164, 216), (165, 215), (166, 215), (164, 214), (165, 213), (166, 214), (165, 214), (164, 212), (163, 212), (162, 213), (162, 212), (162, 211), (163, 210), (162, 210), (161, 212), (161, 211), (160, 212), (160, 213), (161, 214), (160, 214), (161, 215), (160, 216), (160, 215), (159, 215), (159, 217), (160, 217), (161, 217), (159, 216), (158, 217), (158, 218), (159, 219), (160, 219), (160, 218), (161, 218), (162, 218), (162, 217), (161, 216), (162, 216), (161, 219), (161, 220), (160, 220), (159, 221), (159, 220), (158, 220), (158, 219), (157, 219), (156, 220), (155, 220), (154, 221), (153, 222), (154, 222), (153, 223), (154, 224), (155, 225), (154, 225), (155, 224), (153, 226), (153, 227), (156, 225), (156, 226), (155, 226), (156, 227), (155, 228), (154, 227), (155, 227), (157, 228), (158, 228), (158, 227), (158, 226), (159, 228), (160, 228), (156, 228), (154, 226), (153, 225), (152, 224), (153, 224), (152, 226), (151, 227), (151, 226), (151, 225), (152, 225), (150, 226), (149, 226), (148, 225), (147, 224), (146, 225), (146, 226), (145, 226), (145, 225), (144, 225), (143, 224), (142, 225), (142, 226), (141, 226), (142, 227), (143, 226), (141, 225), (140, 226), (141, 224), (140, 223), (140, 224), (139, 225), (138, 225), (138, 226), (139, 226), (140, 225), (139, 224), (138, 227), (138, 228), (139, 229), (140, 228), (141, 229), (142, 228), (143, 229), (144, 229), (143, 228), (142, 229), (143, 230), (143, 231), (144, 231), (145, 232), (146, 232), (146, 233), (145, 233), (146, 234), (146, 235), (147, 236), (145, 235), (145, 234), (144, 235), (145, 236), (144, 236), (143, 235), (142, 234), (141, 234), (140, 233), (141, 233), (142, 232), (143, 233), (143, 234), (144, 233), (144, 232), (142, 231), (141, 230), (143, 232), (142, 233), (142, 235), (144, 230), (141, 232), (140, 232), (139, 232), (139, 233), (140, 234), (139, 231), (139, 230), (140, 231), (141, 231), (142, 230), (140, 230), (144, 234), (138, 234), (138, 235), (139, 235), (139, 236), (140, 237), (141, 237), (142, 238), (142, 237), (142, 236), (138, 237), (139, 237), (140, 238), (139, 238), (141, 236), (141, 238), (141, 239), (140, 239), (143, 237), (145, 231), (145, 230), (146, 230), (146, 231), (147, 234), (148, 235), (149, 234), (150, 233), (149, 232), (149, 233), (148, 234), (147, 233), (148, 232), (147, 232), (150, 232), (151, 232), (152, 232), (153, 231), (148, 233), (148, 231), (149, 230), (149, 231), (150, 231), (150, 230), (151, 229), (154, 230), (154, 231), (152, 235), (154, 236), (156, 240), (157, 239), (158, 240), (153, 241), (151, 241), (150, 240), (151, 240), (149, 240), (148, 241), (148, 242), (147, 242), (147, 241), (149, 242), (150, 241), (151, 242), (150, 243), (150, 242), (151, 239), (147, 238), (146, 237), (145, 238), (144, 237), (143, 236), (141, 235), (140, 235), (140, 236), (139, 234), (138, 236), (138, 238), (141, 240), (140, 241), (139, 241), (138, 240), (139, 240), (138, 241), (140, 242), (140, 243), (139, 243), (138, 244), (138, 243), (139, 244), (140, 245), (140, 244), (141, 243), (142, 244), (142, 245), (142, 246), (143, 247), (144, 246), (143, 246), (144, 245), (143, 244), (141, 244), (142, 243), (139, 245), (141, 245), (140, 246), (139, 246), (138, 245), (138, 246), (139, 247), (138, 247), (139, 248), (140, 249), (141, 249), (141, 250), (140, 250), (139, 249), (139, 251), (140, 251), (141, 252), (141, 251), (142, 250), (140, 252), (139, 252), (140, 248), (138, 248), (138, 249), (139, 250), (140, 253), (142, 252), (143, 252), (144, 252), (142, 251), (143, 251), (142, 253), (142, 254), (141, 255), (141, 256), (140, 257), (141, 257), (140, 258), (142, 258), (143, 257), (142, 257), (143, 256), (144, 256), (144, 257), (142, 256), (142, 255), (143, 255), (144, 255), (140, 255), (141, 254), (140, 254), (143, 253), (144, 254), (145, 254), (144, 253), (144, 250), (145, 251), (146, 251), (147, 251), (148, 252), (148, 253), (147, 254), (147, 253), (146, 254), (147, 255), (148, 254), (149, 255), (149, 256), (149, 257), (148, 257), (147, 258), (146, 257), (147, 256), (145, 257), (145, 255), (146, 256), (149, 258), (148, 258), (147, 259), (147, 260), (146, 259), (146, 260), (148, 261), (149, 260), (149, 259), (155, 256), (154, 256), (155, 257), (156, 258), (155, 258), (154, 258), (154, 257), (153, 256), (153, 257), (150, 254), (150, 253), (149, 253), (150, 255), (150, 258), (148, 259), (148, 262), (147, 261), (148, 260), (146, 261), (145, 258), (145, 256), (144, 258), (143, 258), (143, 259), (141, 259), (142, 259), (144, 259), (145, 260), (144, 261), (145, 261), (143, 262), (143, 263), (144, 263), (144, 262), (145, 263), (146, 263), (145, 262), (145, 264), (146, 262), (147, 263), (147, 262), (145, 259), (150, 259), (151, 259), (152, 259), (152, 260), (153, 260), (154, 259), (153, 258), (156, 259), (157, 260), (158, 259), (159, 260), (158, 260), (157, 261), (159, 259), (160, 258), (160, 259), (160, 260), (159, 261), (158, 261), (158, 258), (157, 259), (159, 258), (161, 258), (162, 259), (162, 260), (163, 259), (163, 260), (162, 261), (161, 259), (158, 257), (159, 256), (157, 257), (156, 257), (156, 256), (157, 256), (156, 255), (156, 254), (153, 252), (154, 252), (153, 251), (154, 250), (155, 249), (154, 248), (159, 248), (157, 247), (157, 246), (158, 245), (157, 245), (158, 244), (160, 246), (163, 238), (164, 236), (163, 236), (166, 236), (163, 235), (162, 234), (161, 233), (162, 232), (160, 232), (160, 231), (161, 230), (162, 231), (161, 231), (159, 230), (158, 229), (162, 230), (162, 229), (163, 230), (164, 231), (165, 231), (165, 230), (164, 230), (163, 231), (160, 230), (160, 229), (161, 229), (162, 228), (163, 227), (164, 228), (165, 227), (166, 227), (167, 227), (168, 226), (169, 227), (169, 228), (169, 229), (168, 227), (168, 228), (167, 228), (168, 229), (168, 230), (167, 231), (168, 231), (169, 232), (169, 231), (171, 230), (171, 231), (171, 229), (173, 228), (174, 228), (174, 227), (175, 227), (176, 228), (177, 228), (175, 228), (170, 229), (171, 228), (171, 227), (173, 227), (174, 226), (173, 225), (172, 226), (173, 226), (167, 226), (166, 226), (167, 225), (166, 228), (165, 226), (164, 226), (165, 228), (163, 228), (163, 229), (159, 231), (159, 232), (159, 233), (160, 234), (160, 235), (158, 234), (161, 234), (160, 233), (159, 234), (158, 233), (157, 232), (158, 231), (158, 230), (159, 229), (161, 232), (162, 233), (157, 235), (157, 234), (156, 234), (156, 233), (149, 241), (149, 243), (148, 244), (148, 245), (147, 245), (146, 244), (147, 243), (152, 242), (153, 242), (153, 243), (152, 244), (151, 243), (154, 243), (152, 243), (152, 240), (152, 239), (150, 244), (150, 245), (151, 245), (151, 244), (151, 246), (149, 244), (152, 246), (151, 247), (150, 248), (151, 248), (152, 248), (153, 247), (152, 245), (150, 247), (154, 245), (154, 244), (155, 245), (156, 244), (156, 245), (155, 244), (157, 244), (154, 249), (155, 251), (156, 251), (153, 244), (153, 245), (155, 243), (158, 246), (159, 246), (175, 240), (162, 249), (163, 250), (165, 252), (163, 253), (162, 253), (158, 254), (154, 251), (156, 250), (160, 242), (165, 237), (166, 233), (168, 232), (168, 233), (165, 233), (167, 232), (169, 233), (177, 227), (175, 229), (176, 226), (180, 228), (178, 228), (178, 226), (172, 228), (170, 232), (170, 233), (170, 230), (172, 227), (175, 225), (167, 230), (167, 229), (169, 230), (170, 228), (166, 231), (166, 232), (162, 235), (162, 236), (162, 237), (163, 237), (162, 239), (162, 238), (157, 233), (179, 237), (161, 255), (160, 256), (161, 257), (161, 256), (162, 256), (160, 257), (158, 256), (149, 262), (149, 263), (150, 264), (149, 264), (150, 263), (151, 264), (150, 262), (149, 261), (150, 260), (150, 261), (151, 261), (151, 263), (152, 264), (151, 265), (152, 265), (153, 266), (152, 266), (153, 264), (151, 262), (152, 262), (152, 261), (153, 261), (154, 260), (154, 261), (153, 259), (152, 258), (155, 259), (155, 260), (159, 257), (162, 258), (162, 257), (163, 258), (163, 257), (164, 259), (164, 258), (165, 259), (165, 260), (165, 258), (165, 257), (166, 257), (164, 257), (163, 256), (162, 254), (163, 252), (164, 252), (163, 251), (161, 248), (159, 253), (160, 255), (153, 262), (153, 263), (153, 265), (152, 267), (153, 267), (152, 268), (153, 268), (154, 268), (155, 268), (155, 267), (156, 267), (156, 268), (154, 267), (155, 266), (155, 265), (154, 266), (156, 266), (156, 265), (155, 264), (154, 265), (155, 263), (156, 262), (157, 263), (156, 263), (156, 264), (154, 262), (157, 265), (158, 264), (159, 264), (158, 265), (159, 266), (158, 263), (159, 263), (159, 262), (155, 262), (154, 263), (156, 261), (156, 260), (157, 262), (157, 264), (151, 266), (150, 265), (155, 261), (152, 263), (148, 263), (146, 264), (145, 265), (145, 266), (144, 267), (143, 268), (144, 268), (143, 267), (145, 267), (142, 268), (141, 267), (140, 267), (139, 266), (138, 266), (141, 266), (140, 265), (141, 264), (141, 265), (142, 266), (142, 267), (143, 266), (145, 268), (146, 267), (144, 266), (141, 268), (138, 267), (139, 268), (140, 268), (140, 266), (139, 265), (138, 264), (139, 264), (140, 263), (138, 265), (140, 264), (139, 263), (139, 267), (138, 263), (139, 262), (138, 262), (140, 262), (142, 265), (143, 265), (142, 264), (144, 265), (143, 264), (142, 263), (144, 264), (138, 268), (146, 268), (147, 267), (147, 266), (148, 267), (147, 268), (149, 266), (150, 267), (149, 268), (149, 267), (150, 266), (149, 265), (148, 264), (148, 265), (147, 265), (151, 260), (148, 256), (146, 255), (145, 253), (146, 252), (145, 250), (146, 249), (146, 250), (147, 252), (146, 253), (145, 252), (144, 251), (147, 250), (147, 249), (146, 248), (145, 248), (146, 247), (147, 246), (147, 247), (147, 244), (146, 245), (145, 244), (145, 243), (144, 242), (143, 243), (144, 243), (145, 242), (143, 242), (142, 242), (142, 241), (142, 240), (142, 239), (143, 239), (144, 238), (146, 239), (146, 238), (147, 237), (147, 239), (147, 240), (149, 247), (148, 247), (145, 246), (144, 247), (144, 248), (144, 249), (145, 249), (143, 245), (144, 244), (143, 249), (149, 251), (149, 252), (141, 253), (138, 252), (138, 251), (143, 254), (143, 260), (142, 261), (142, 260), (143, 261), (151, 258), (149, 254), (148, 255), (150, 251), (150, 250), (149, 250), (148, 251), (143, 250), (142, 249), (141, 248), (142, 248), (143, 248), (142, 247), (141, 246), (141, 247), (140, 247), (139, 253), (138, 253), (139, 255), (139, 254), (138, 254), (138, 255), (139, 256), (138, 257), (138, 250), (147, 248), (148, 248), (148, 246), (146, 246), (149, 245), (148, 243), (146, 243), (145, 241), (145, 240), (144, 241), (143, 240), (143, 238), (141, 228), (140, 229), (138, 233), (139, 239), (138, 232), (140, 240), (141, 241), (141, 242), (144, 239), (139, 242), (138, 242), (138, 239), (143, 241), (145, 245), (146, 241), (146, 242), (146, 240), (150, 246), (168, 257), (167, 258), (157, 258), (160, 261), (161, 262), (161, 263), (160, 262), (157, 255), (159, 255), (158, 255), (161, 261), (161, 264), (161, 265), (164, 268), (165, 268), (163, 262), (165, 263), (166, 262), (165, 262), (165, 261), (196, 254), (197, 254), (200, 255), (202, 254), (202, 253), (204, 255), (206, 256), (207, 255), (205, 255), (204, 252), (204, 251), (207, 251), (209, 254), (213, 255), (214, 255), (214, 256), (215, 255), (216, 254), (215, 254), (216, 255), (217, 256), (217, 255), (215, 253), (216, 253), (217, 254), (216, 256), (217, 259), (217, 258), (217, 257), (217, 253), (218, 254), (219, 255), (218, 253), (219, 254), (220, 255), (221, 255), (221, 256), (220, 254), (220, 253), (222, 256), (223, 255), (222, 254), (222, 255), (221, 254), (221, 253), (222, 257), (222, 258), (223, 257), (223, 256), (224, 255), (225, 255), (225, 254), (226, 254), (226, 255), (225, 256), (224, 256), (224, 257), (225, 257), (226, 257), (226, 256), (227, 254), (226, 253), (228, 254), (229, 253), (228, 253), (227, 253), (227, 252), (226, 251), (226, 250), (227, 251), (228, 252), (227, 250), (227, 249), (225, 252), (228, 251), (230, 247), (231, 243), (228, 244), (228, 243), (227, 244), (226, 244), (225, 243), (224, 243), (223, 244), (222, 243), (222, 242), (223, 241), (223, 243), (221, 242), (222, 241), (223, 242), (224, 242), (224, 241), (225, 242), (226, 241), (225, 241), (226, 242), (227, 243), (226, 243), (225, 244), (221, 241), (227, 245), (228, 246), (228, 248), (226, 249), (229, 252), (229, 251), (228, 255), (228, 256), (227, 257), (227, 256), (226, 258), (227, 258), (228, 257), (229, 257), (230, 257), (231, 257), (228, 258), (227, 255), (229, 255), (229, 254), (230, 253), (231, 246), (229, 248), (231, 245), (232, 246), (232, 247), (232, 245), (232, 244), (233, 245), (234, 246), (233, 243), (234, 243), (235, 243), (235, 242), (235, 241), (236, 240), (235, 240), (235, 239), (239, 243), (238, 244), (239, 244), (242, 244), (246, 244), (248, 246), (250, 245), (250, 246), (251, 246), (252, 247), (253, 248), (253, 247), (254, 250), (255, 249), (256, 247), (255, 246), (255, 245), (254, 246), (249, 249), (245, 251), (249, 253), (250, 254), (251, 253), (250, 256), (250, 257), (250, 258), (251, 257), (252, 256), (251, 256), (252, 257), (253, 257), (254, 257), (253, 258), (252, 258), (252, 255), (251, 258), (250, 259), (249, 257), (250, 255), (251, 254), (251, 252), (252, 251), (253, 251), (252, 250), (252, 253), (252, 254), (253, 254), (254, 253), (254, 255), (255, 255), (255, 254), (253, 256), (253, 252), (253, 255), (251, 251), (252, 248), (253, 249), (257, 249), (256, 250), (255, 251), (255, 252), (254, 252), (254, 251), (252, 252), (253, 253), (255, 253), (254, 254), (256, 252), (255, 247), (256, 246), (257, 241), (256, 240), (260, 238), (247, 234), (245, 232), (246, 231), (246, 232), (242, 229), (242, 230), (242, 231), (241, 230), (241, 232), (242, 232), (237, 228), (238, 228), (247, 240), (247, 239), (265, 204), (240, 218), (240, 215), (240, 208), (245, 202), (239, 197), (245, 198), (245, 199), (243, 205), (236, 217), (237, 216), (231, 216), (231, 218), (231, 219), (232, 219), (232, 220), (224, 219), (225, 218), (224, 218), (222, 217), (220, 217), (218, 218), (215, 218), (215, 215), (218, 213), (218, 212), (218, 210), (218, 209), (218, 208), (218, 207), (220, 205), (220, 204), (219, 203), (218, 204), (218, 205), (219, 205), (217, 207), (217, 209), (220, 206), (220, 203), (218, 206), (217, 205), (217, 206), (216, 205), (217, 204), (215, 205), (216, 204), (217, 203), (218, 202), (217, 202), (218, 203), (216, 202), (216, 201), (217, 200), (216, 199), (216, 200), (215, 201), (215, 202), (215, 203), (216, 203), (219, 204), (221, 203), (220, 202), (226, 202), (227, 201), (226, 200), (225, 200), (225, 201), (223, 200), (222, 201), (221, 201), (224, 200), (223, 201), (222, 200), (221, 200), (223, 199), (224, 199), (223, 198), (222, 198), (222, 197), (221, 197), (221, 196), (220, 197), (220, 196), (222, 195), (221, 195), (222, 196), (223, 195), (224, 196), (225, 195), (225, 196), (224, 197), (223, 197), (224, 198), (222, 199), (223, 196), (225, 197), (221, 198), (224, 195), (225, 194), (223, 194), (224, 194), (225, 198), (226, 199), (228, 200), (230, 199), (226, 201), (229, 206), (229, 205), (230, 205), (228, 211), (224, 208), (221, 199), (220, 199), (219, 198), (219, 196), (220, 195), (219, 194), (218, 195), (217, 195), (216, 196), (214, 195), (216, 195), (217, 196), (217, 197), (218, 198), (217, 198), (215, 200), (217, 199), (217, 201), (218, 200), (219, 201), (219, 200), (216, 207), (215, 207), (215, 209), (215, 206), (214, 205), (215, 204), (214, 204), (214, 203), (214, 202), (213, 201), (213, 212), (214, 213), (210, 213), (210, 211), (209, 211), (210, 212), (211, 213), (217, 210), (216, 209), (216, 206), (219, 195), (220, 194), (220, 193), (219, 192), (219, 193), (220, 192), (221, 192), (222, 193), (222, 192), (223, 193), (224, 193), (225, 192), (226, 192), (226, 191), (225, 190), (220, 191), (218, 192), (219, 191), (220, 190), (219, 190), (218, 190), (220, 189), (219, 189), (218, 188), (219, 187), (220, 188), (220, 186), (219, 185), (220, 184), (221, 185), (221, 186), (220, 187), (220, 185), (221, 184), (222, 185), (222, 186), (223, 186), (224, 185), (223, 185), (219, 188), (218, 187), (217, 188), (216, 188), (217, 187), (215, 189), (215, 184), (216, 183), (215, 182), (216, 181), (216, 180), (216, 179), (217, 181), (215, 178), (214, 177), (214, 178), (213, 178), (213, 179), (214, 179), (215, 180), (215, 181), (214, 181), (214, 182), (216, 182), (218, 182), (217, 182), (217, 183), (217, 184), (218, 184), (219, 184), (218, 185), (216, 184), (215, 183), (214, 183), (213, 183), (214, 184), (212, 184), (212, 183), (211, 183), (211, 184), (211, 182), (212, 182), (213, 182), (213, 181), (212, 181), (212, 180), (211, 181), (210, 182), (215, 185), (216, 186), (212, 185), (215, 187), (215, 186), (209, 184), (209, 183), (210, 184), (210, 183), (211, 180), (210, 179), (209, 179), (208, 180), (208, 181), (207, 182), (206, 182), (205, 182), (200, 184), (199, 185), (198, 185), (202, 185), (200, 183), (203, 188), (203, 189), (204, 187), (204, 193), (202, 193), (204, 190), (204, 189), (204, 188), (206, 183), (204, 181), (208, 184), (203, 193), (204, 194), (215, 198), (220, 201), (218, 199), (216, 198), (218, 196), (220, 198), (220, 200), (219, 199), (225, 199), (226, 193), (229, 191), (231, 191), (232, 185), (228, 184), (229, 183), (227, 181), (226, 182), (226, 181), (229, 180), (231, 181), (232, 176), (233, 175), (233, 174), (232, 174), (232, 167), (227, 165), (219, 165), (217, 167), (217, 169), (215, 169), (219, 172), (220, 173), (219, 174), (218, 174), (217, 173), (219, 176), (218, 175), (217, 175), (217, 176), (216, 175), (215, 175), (214, 176), (213, 177), (216, 176), (216, 177), (218, 176), (215, 174), (216, 173), (215, 173), (217, 171), (218, 171), (216, 168), (215, 167), (217, 174), (216, 174), (214, 175), (215, 176), (213, 176), (212, 177), (211, 177), (212, 176), (213, 175), (212, 178), (212, 179), (213, 180), (208, 183), (209, 182), (213, 184), (216, 185), (214, 180), (215, 179), (216, 178), (217, 178), (218, 179), (217, 179), (218, 181), (218, 180), (217, 180), (218, 183), (219, 183), (221, 183), (222, 184), (219, 181), (219, 180), (219, 178), (218, 177), (217, 177), (221, 176), (222, 179), (224, 181), (224, 182), (225, 183), (224, 184), (224, 186), (225, 185), (223, 187), (223, 188), (224, 187), (224, 183), (223, 184), (225, 184), (225, 189), (226, 188), (227, 190), (229, 192), (230, 192), (230, 193), (230, 196), (228, 196), (227, 196), (228, 197), (227, 197), (226, 198), (227, 198), (219, 197), (221, 191), (226, 197), (236, 196), (238, 195), (239, 195), (233, 205), (241, 207), (245, 196), (249, 195), (251, 194), (252, 196), (252, 195), (249, 199), (249, 196), (253, 192), (256, 183), (257, 182), (255, 182), (259, 183), (258, 182), (256, 182), (261, 187), (262, 185), (259, 187), (257, 190), (260, 190), (260, 192), (260, 191), (258, 194), (258, 187), (259, 193), (261, 194), (262, 194), (257, 194), (252, 193), (259, 191), (259, 195), (266, 189), (266, 197), (266, 198), (264, 196), (262, 195), (263, 195), (261, 196), (261, 197), (262, 196), (260, 194), (261, 195), (267, 193), (268, 192), (268, 195), (268, 191), (266, 196), (267, 190), (268, 190), (267, 186), (268, 189), (266, 235), (268, 234), (268, 235), (270, 236), (267, 242), (268, 242), (266, 243), (265, 244), (265, 245), (267, 243), (266, 244), (266, 245), (262, 242), (264, 242), (265, 243), (267, 245), (267, 246), (266, 247), (265, 247), (265, 248), (267, 247), (268, 248), (268, 246), (268, 247), (267, 248), (266, 249), (266, 250), (265, 250), (264, 251), (263, 252), (263, 253), (262, 252), (262, 251), (262, 250), (261, 250), (260, 251), (261, 252), (263, 251), (263, 250), (264, 249), (265, 249), (263, 248), (264, 248), (264, 247), (265, 246), (266, 246), (264, 245), (263, 244), (261, 242), (260, 243), (259, 243), (256, 244), (260, 242), (259, 242), (260, 241), (231, 213), (232, 215), (232, 218), (231, 215), (233, 211), (232, 207), (228, 215), (226, 214), (224, 213), (229, 213), (223, 217), (221, 219), (219, 216), (219, 217), (219, 218), (219, 215), (213, 202), (212, 200), (213, 200), (206, 214), (207, 213), (208, 215), (209, 215), (210, 214), (209, 214), (210, 215), (209, 213), (209, 212), (208, 213), (208, 212), (209, 216), (209, 210), (214, 201), (214, 199), (218, 194), (218, 193), (217, 193), (216, 193), (215, 193), (217, 190), (218, 189), (221, 189), (217, 191), (216, 190), (215, 190), (217, 194), (217, 189), (216, 187), (214, 189), (215, 188), (216, 189), (214, 173), (213, 173), (212, 174), (211, 174), (211, 173), (212, 175), (213, 174), (211, 176), (211, 175), (212, 173), (210, 174), (210, 175), (210, 176), (209, 177), (208, 178), (209, 180), (208, 179), (209, 181), (210, 180), (207, 180), (209, 190), (210, 181), (207, 179), (206, 179), (209, 178), (208, 177), (209, 176), (209, 175), (208, 176), (207, 176), (208, 175), (206, 177), (206, 176), (206, 175), (205, 175), (204, 175), (204, 174), (203, 176), (202, 175), (202, 176), (201, 177), (201, 178), (202, 177), (202, 178), (201, 179), (199, 182), (199, 183), (199, 184), (198, 184), (198, 183), (205, 180), (204, 179), (204, 180), (203, 179), (203, 178), (204, 178), (202, 179), (204, 177), (204, 176), (203, 175), (205, 176), (203, 174), (200, 173), (204, 173), (205, 173), (206, 172), (205, 171), (205, 172), (205, 174), (203, 171), (204, 170), (205, 170), (206, 169), (206, 170), (207, 170), (208, 171), (208, 170), (209, 169), (210, 170), (210, 169), (211, 169), (211, 170), (210, 171), (211, 172), (210, 172), (209, 172), (209, 173), (208, 172), (208, 173), (209, 174), (208, 174), (210, 173), (209, 170), (209, 171), (212, 169), (213, 168), (213, 169), (212, 170), (213, 171), (213, 170), (214, 169), (214, 171), (213, 172), (212, 171), (211, 171), (214, 172), (214, 174), (212, 172), (215, 172), (219, 168), (214, 170), (215, 171), (215, 177), (211, 178), (207, 181), (206, 181), (206, 180), (208, 182), (206, 178), (207, 178), (207, 177), (205, 177), (210, 178), (207, 175), (207, 174), (207, 173), (206, 174), (210, 177), (205, 181), (205, 179), (201, 180), (202, 181), (203, 180), (203, 181), (205, 178), (202, 180), (200, 177), (199, 178), (198, 179), (197, 179), (198, 180), (197, 178), (196, 178), (196, 179), (197, 180), (195, 178), (195, 179), (196, 180), (195, 181), (196, 182), (195, 183), (194, 182), (193, 183), (192, 182), (192, 183), (191, 183), (193, 184), (194, 185), (195, 186), (194, 187), (194, 186), (195, 185), (194, 184), (193, 185), (192, 184), (191, 184), (190, 184), (190, 183), (189, 184), (191, 185), (190, 186), (191, 187), (192, 186), (189, 183), (189, 182), (190, 182), (188, 183), (188, 184), (187, 183), (186, 183), (185, 182), (185, 183), (184, 183), (186, 184), (185, 184), (185, 185), (186, 186), (187, 187), (187, 188), (186, 189), (185, 189), (186, 188), (188, 187), (187, 189), (188, 188), (188, 189), (187, 190), (187, 191), (186, 192), (187, 193), (185, 192), (186, 193), (187, 192), (186, 191), (188, 191), (189, 190), (189, 186), (189, 185), (190, 185), (191, 186), (190, 187), (191, 188), (191, 189), (190, 188), (190, 189), (189, 189), (190, 190), (189, 188), (185, 186), (185, 187), (185, 188), (188, 190), (189, 191), (184, 187), (183, 186), (184, 186), (183, 187), (182, 188), (181, 188), (180, 189), (179, 190), (179, 189), (178, 188), (179, 187), (179, 188), (180, 190), (181, 190), (181, 191), (179, 191), (179, 192), (180, 192), (180, 191), (182, 189), (182, 190), (181, 192), (182, 191), (182, 192), (178, 190), (177, 190), (176, 189), (176, 190), (175, 190), (174, 190), (174, 189), (175, 189), (174, 191), (173, 190), (172, 189), (171, 188), (170, 187), (172, 188), (173, 187), (171, 189), (171, 190), (172, 190), (173, 191), (172, 192), (172, 191), (173, 189), (175, 188), (176, 187), (174, 188), (173, 188), (172, 187), (172, 186), (171, 185), (171, 184), (170, 183), (171, 183), (169, 183), (168, 183), (167, 182), (166, 183), (166, 184), (167, 185), (166, 186), (167, 186), (168, 186), (169, 187), (170, 186), (170, 185), (169, 186), (168, 185), (169, 185), (169, 188), (169, 189), (169, 190), (170, 190), (170, 191), (168, 191), (169, 191), (168, 190), (168, 189), (168, 188), (170, 189), (170, 188), (171, 187), (171, 186), (172, 185), (176, 188), (177, 189), (176, 191), (175, 191), (176, 192), (175, 192), (175, 193), (176, 194), (176, 195), (177, 195), (175, 196), (174, 197), (173, 197), (174, 198), (174, 199), (175, 197), (176, 196), (176, 197), (177, 196), (178, 195), (177, 194), (177, 193), (178, 194), (179, 193), (178, 193), (180, 193), (181, 194), (180, 194), (177, 192), (178, 192), (178, 191), (177, 191), (178, 189), (177, 188), (176, 186), (176, 185), (177, 185), (177, 187), (177, 186), (177, 184), (178, 185), (178, 186), (179, 186), (179, 185), (177, 183), (178, 184), (176, 193), (175, 194), (177, 197), (178, 197), (179, 197), (180, 197), (181, 198), (181, 199), (181, 200), (179, 196), (178, 196), (177, 198), (177, 199), (176, 198), (175, 199), (173, 199), (179, 195), (180, 195), (181, 196), (181, 195), (182, 194), (182, 195), (180, 196), (182, 196), (183, 195), (184, 195), (185, 196), (185, 195), (186, 195), (186, 196), (187, 197), (188, 198), (188, 197), (186, 198), (185, 197), (185, 198), (184, 198), (185, 199), (185, 200), (184, 201), (183, 200), (183, 201), (182, 200), (182, 198), (183, 197), (182, 197), (181, 197), (180, 199), (179, 199), (179, 198), (178, 199), (176, 199), (176, 200), (175, 200), (178, 198), (175, 195), (174, 195), (173, 194), (174, 193), (172, 195), (171, 194), (170, 194), (170, 193), (169, 192), (170, 192), (168, 192), (167, 192), (167, 193), (166, 192), (165, 193), (165, 192), (164, 191), (165, 190), (166, 189), (165, 188), (166, 188), (167, 187), (168, 187), (171, 191), (171, 192), (172, 193), (174, 194), (173, 193), (172, 194), (171, 193), (171, 195), (172, 196), (171, 196), (173, 196), (173, 198), (172, 197), (172, 198), (173, 195), (173, 192), (171, 197), (171, 199), (170, 198), (169, 199), (168, 199), (167, 198), (167, 197), (167, 196), (166, 196), (167, 195), (168, 196), (169, 196), (168, 195), (170, 196), (169, 195), (168, 194), (165, 196), (164, 197), (165, 197), (165, 198), (166, 198), (168, 198), (169, 197), (169, 198), (170, 199), (169, 200), (167, 200), (169, 194), (174, 192), (166, 185), (165, 185), (165, 184), (167, 183), (168, 182), (167, 181), (165, 182), (164, 182), (165, 183), (168, 184), (167, 184), (169, 182), (168, 181), (173, 186), (174, 187), (167, 190), (167, 189), (164, 187), (163, 186), (164, 185), (165, 186), (166, 187), (164, 184), (169, 184), (170, 184), (174, 180), (166, 182), (167, 180), (167, 179), (166, 180), (166, 179), (169, 181), (171, 179), (173, 178), (174, 177), (175, 176), (176, 175), (177, 176), (176, 176), (177, 177), (177, 175), (178, 175), (178, 174), (177, 174), (177, 173), (178, 173), (175, 177), (176, 178), (175, 179), (176, 179), (178, 177), (179, 178), (180, 179), (181, 180), (181, 183), (181, 184), (180, 183), (182, 185), (183, 185), (186, 187), (183, 184), (184, 185), (184, 188), (183, 188), (183, 189), (184, 190), (184, 189), (183, 190), (181, 189), (183, 192), (184, 191), (185, 190), (186, 190), (188, 192), (189, 192), (190, 192), (189, 193), (190, 191), (191, 191), (191, 192), (188, 193), (185, 193), (185, 194), (187, 194), (187, 195), (187, 196), (184, 197), (184, 196), (184, 193), (183, 194), (184, 194), (182, 193), (185, 191), (183, 193), (181, 193), (183, 191), (184, 192), (182, 187), (182, 186), (187, 185), (189, 181), (188, 181), (188, 182), (187, 182), (186, 182), (186, 180), (183, 181), (182, 181), (184, 184), (186, 185), (187, 184), (188, 186), (189, 187), (192, 189), (193, 190), (194, 189), (194, 190), (195, 189), (196, 189), (195, 190), (195, 191), (194, 191), (193, 188), (193, 187), (193, 186), (192, 185), (188, 185), (187, 181), (187, 180), (190, 181), (190, 180), (189, 180), (188, 180), (181, 187), (180, 187), (178, 187), (192, 187), (192, 188), (193, 189), (195, 188), (196, 188), (197, 190), (197, 189), (196, 190), (196, 191), (197, 192), (198, 191), (199, 192), (198, 192), (198, 193), (197, 193), (196, 192), (195, 187), (196, 186), (196, 185), (197, 187), (196, 184), (195, 184), (197, 184), (198, 186), (197, 188), (198, 189), (200, 190), (198, 188), (198, 190), (195, 192), (196, 187), (197, 186), (194, 188), (194, 183), (193, 182), (193, 181), (194, 180), (194, 181), (193, 180), (192, 181), (196, 181), (197, 182), (198, 182), (206, 173), (207, 171), (206, 171), (207, 172), (204, 172), (200, 175), (199, 174), (199, 175), (199, 176), (199, 177), (201, 176), (198, 176), (197, 176), (196, 176), (195, 175), (195, 176), (195, 177), (194, 177), (194, 178), (197, 181), (198, 187), (187, 186), (180, 198), (183, 196), (180, 188), (180, 186), (181, 186), (176, 184), (178, 182), (179, 183), (182, 182), (187, 178), (189, 179), (186, 176), (190, 173), (190, 174), (190, 175), (189, 176), (190, 177), (189, 178), (189, 175), (188, 174), (191, 174), (192, 174), (191, 176), (191, 177), (191, 178), (192, 178), (190, 178), (190, 179), (191, 182), (191, 180), (191, 179), (192, 180), (192, 179), (193, 178), (194, 179), (198, 178), (196, 177), (194, 176), (197, 177), (198, 177), (200, 176), (203, 177), (204, 171), (205, 169), (208, 169), (210, 168), (211, 168), (214, 168), (215, 168), (221, 178), (220, 177), (223, 170), (225, 170), (225, 171), (226, 170), (222, 170), (203, 182), (200, 179), (200, 178), (218, 178), (219, 177), (221, 179), (224, 180), (210, 165), (209, 165), (208, 166), (207, 166), (205, 165), (203, 164), (205, 167), (204, 168), (201, 171), (202, 172), (201, 170), (199, 169), (199, 168), (199, 167), (200, 166), (201, 167), (205, 168), (204, 169), (206, 168), (207, 168), (207, 169), (208, 168), (209, 168), (209, 164), (209, 159), (208, 159), (207, 157), (210, 164), (208, 167), (209, 167), (206, 167), (207, 167), (200, 172), (199, 172), (198, 173), (197, 173), (197, 174), (196, 174), (195, 173), (194, 173), (194, 172), (195, 171), (196, 172), (195, 172), (195, 174), (198, 174), (198, 175), (197, 175), (196, 175), (193, 175), (193, 176), (198, 181), (196, 183), (192, 193), (193, 192), (192, 191), (191, 193), (190, 193), (193, 193), (192, 192), (192, 190), (195, 182), (188, 179), (192, 177), (194, 175), (192, 176), (193, 177), (193, 179), (192, 175), (191, 175), (184, 174), (184, 176), (183, 179), (198, 172), (199, 173), (196, 173), (197, 172), (198, 170), (199, 171), (196, 165), (195, 164), (195, 166), (195, 165), (194, 165), (194, 166), (194, 167), (197, 170), (197, 171), (196, 171), (198, 168), (198, 167), (199, 166), (200, 167), (200, 164), (201, 166), (201, 165), (194, 162), (195, 160), (196, 160), (204, 165), (201, 163), (197, 169), (200, 168), (225, 180), (226, 180), (225, 181), (225, 182), (226, 183), (226, 184), (227, 184), (227, 183), (227, 180), (226, 179), (227, 179), (227, 178), (223, 183), (224, 171), (226, 171), (227, 171), (228, 170), (230, 170), (230, 169), (231, 168), (231, 170), (232, 168), (231, 167), (230, 167), (231, 169), (231, 176), (231, 178), (230, 178), (231, 179), (230, 180), (229, 179), (228, 180), (228, 179), (220, 167), (203, 163), (214, 164), (217, 163), (217, 164), (206, 166), (224, 170), (213, 190), (213, 188), (217, 185), (218, 186), (217, 186), (198, 171), (211, 179), (194, 174), (197, 185), (190, 194), (189, 194), (189, 195), (188, 196), (186, 197), (180, 200), (183, 199), (184, 199), (183, 198), (179, 194), (176, 183), (175, 184), (175, 182), (177, 182), (177, 181), (176, 181), (176, 180), (175, 180), (178, 178), (180, 177), (180, 178), (179, 177), (179, 176), (180, 175), (179, 175), (181, 177), (182, 176), (183, 175), (183, 176), (182, 175), (181, 174), (180, 176), (181, 179), (181, 185), (180, 184), (179, 184), (180, 185), (186, 177), (187, 177), (190, 176), (189, 177), (188, 172), (185, 176), (182, 174), (183, 174), (185, 172), (184, 171), (183, 173), (182, 171), (183, 169), (184, 168), (180, 171), (179, 171), (179, 172), (178, 171), (177, 170), (178, 170), (180, 173), (181, 173), (182, 177), (183, 177), (182, 173), (183, 178), (181, 172), (192, 170), (192, 171), (193, 169), (193, 171), (191, 173), (193, 174), (192, 173), (193, 173), (193, 172), (192, 172), (194, 171), (186, 166), (188, 170), (188, 171), (184, 169), (185, 167), (190, 161), (191, 160), (192, 159), (192, 160), (192, 161), (193, 160), (209, 151), (207, 156), (188, 138), (186, 143), (159, 154), (160, 153), (161, 152), (157, 154), (158, 154), (159, 153), (160, 152), (153, 156), (152, 157), (152, 156), (153, 155), (152, 154), (153, 154), (152, 155), (151, 153), (151, 154), (150, 154), (149, 153), (150, 153), (151, 155), (151, 156), (150, 156), (149, 157), (148, 157), (147, 156), (148, 155), (148, 156), (149, 155), (151, 157), (152, 158), (151, 158), (152, 159), (151, 159), (150, 157), (149, 158), (148, 158), (149, 159), (150, 158), (154, 153), (153, 152), (153, 150), (152, 151), (152, 150), (152, 149), (151, 149), (158, 157), (158, 158), (151, 166), (153, 169), (152, 169), (153, 168), (151, 170), (153, 172), (152, 173), (151, 174), (153, 173), (152, 174), (149, 171), (150, 174), (150, 171), (152, 172), (144, 173), (143, 173), (142, 173), (142, 172), (142, 169), (141, 169), (140, 168), (139, 169), (138, 168), (140, 169), (139, 170), (139, 171), (138, 170), (138, 171), (138, 169), (139, 168), (140, 167), (141, 166), (141, 167), (139, 167), (138, 166), (139, 165), (138, 164), (139, 164), (140, 163), (140, 162), (139, 163), (138, 162), (138, 163), (139, 162), (140, 164), (141, 163), (141, 164), (141, 165), (142, 166), (140, 165), (142, 165), (143, 165), (140, 166), (140, 171), (141, 172), (140, 170), (138, 172), (140, 172), (141, 173), (140, 174), (139, 174), (140, 175), (139, 175), (139, 173), (139, 172), (140, 173), (142, 174), (141, 175), (142, 176), (143, 175), (141, 171), (143, 171), (143, 172), (148, 175), (147, 175), (147, 176), (148, 177), (149, 177), (149, 176), (150, 177), (149, 178), (150, 176), (151, 178), (151, 179), (150, 179), (152, 178), (153, 178), (153, 177), (154, 179), (153, 179), (156, 177), (159, 176), (158, 176), (155, 179), (155, 180), (153, 175), (152, 176), (152, 177), (151, 176), (151, 177), (150, 175), (149, 175), (148, 176), (147, 169), (147, 166), (145, 166), (144, 167), (145, 168), (143, 168), (142, 167), (143, 166), (141, 161), (139, 161), (140, 160), (141, 160), (142, 161), (143, 161), (141, 159), (140, 158), (139, 157), (138, 156), (139, 155), (138, 158), (139, 158), (140, 159), (139, 159), (140, 157), (138, 159), (138, 160), (140, 161), (141, 162), (142, 162), (138, 165), (142, 164), (143, 163), (144, 163), (143, 164), (144, 164), (145, 164), (146, 165), (142, 163), (141, 157), (142, 157), (141, 158), (141, 156), (142, 156), (143, 156), (144, 157), (143, 157), (143, 158), (144, 155), (144, 156), (143, 155), (142, 154), (143, 153), (142, 153), (143, 154), (142, 155), (141, 155), (140, 155), (139, 156), (138, 157), (143, 162), (141, 168), (142, 168), (144, 165), (143, 160), (144, 159), (144, 160), (142, 159), (143, 159), (142, 160), (142, 158), (144, 158), (145, 154), (144, 153), (143, 152), (142, 151), (143, 150), (142, 150), (142, 149), (143, 149), (142, 148), (141, 148), (141, 147), (140, 148), (139, 148), (139, 149), (138, 148), (138, 149), (138, 150), (138, 147), (139, 146), (138, 146), (138, 145), (140, 145), (140, 144), (141, 144), (140, 143), (139, 144), (139, 145), (139, 142), (140, 142), (141, 142), (142, 142), (141, 141), (142, 141), (143, 142), (142, 143), (142, 144), (142, 145), (143, 146), (142, 146), (140, 149), (139, 150), (139, 151), (138, 151), (139, 152), (138, 153), (139, 153), (138, 152), (140, 152), (140, 153), (140, 154), (140, 156), (141, 154), (139, 154), (138, 155), (139, 160), (139, 166), (138, 167), (143, 167), (145, 177), (146, 177), (142, 175), (138, 173), (141, 170), (144, 161), (145, 162), (146, 162), (144, 162), (146, 161), (147, 160), (147, 161), (146, 160), (145, 161), (145, 157), (146, 157), (146, 156), (145, 158), (144, 166), (145, 156), (142, 178), (150, 178), (148, 178), (152, 179), (150, 180), (149, 180), (148, 179), (147, 179), (148, 180), (147, 180), (146, 180), (146, 179), (147, 181), (147, 182), (148, 182), (145, 180), (144, 180), (146, 181), (145, 181), (144, 181), (143, 181), (143, 180), (142, 181), (141, 180), (140, 180), (141, 181), (140, 179), (141, 179), (142, 179), (141, 178), (143, 178), (143, 179), (142, 180), (140, 182), (139, 183), (139, 182), (139, 181), (140, 181), (138, 182), (138, 183), (138, 184), (139, 185), (140, 185), (141, 184), (142, 184), (143, 184), (142, 185), (144, 184), (143, 183), (143, 182), (144, 182), (147, 183), (146, 184), (146, 183), (146, 182), (145, 182), (145, 183), (148, 181), (151, 175), (153, 170), (154, 169), (154, 159), (158, 156), (159, 155), (163, 151), (160, 150), (162, 152), (161, 153), (162, 159), (161, 159), (163, 164), (162, 164), (160, 170), (159, 171), (156, 174), (141, 174), (142, 177), (140, 178), (141, 176), (138, 174), (138, 175), (140, 176), (139, 177), (138, 176), (138, 178), (139, 178), (139, 179), (144, 183), (143, 185), (141, 185), (140, 184), (140, 183), (141, 183), (141, 182), (138, 180), (138, 179), (139, 180), (142, 182), (138, 181), (138, 177), (138, 154), (141, 153), (142, 152), (138, 161), (146, 158), (146, 159), (145, 160), (147, 159), (147, 158), (147, 157), (145, 159), (147, 155), (148, 159), (149, 156), (147, 162), (148, 162), (148, 164), (148, 161), (148, 160), (153, 180), (153, 181), (152, 181), (151, 182), (151, 181), (152, 180), (151, 180), (150, 181), (153, 182), (152, 183), (152, 184), (153, 183), (153, 184), (153, 185), (152, 186), (152, 185), (151, 184), (151, 183), (150, 183), (149, 182), (150, 182), (149, 181), (152, 182), (154, 184), (155, 185), (154, 185), (154, 183), (155, 184), (153, 186), (154, 186), (152, 187), (153, 187), (153, 188), (154, 188), (153, 189), (152, 190), (151, 191), (150, 191), (150, 190), (149, 191), (148, 192), (148, 193), (148, 194), (147, 195), (147, 196), (147, 197), (146, 197), (146, 196), (147, 198), (148, 198), (148, 196), (149, 195), (150, 195), (149, 196), (149, 197), (150, 196), (151, 197), (151, 196), (151, 195), (152, 194), (153, 193), (154, 193), (155, 192), (156, 191), (155, 193), (154, 194), (155, 195), (153, 194), (156, 193), (156, 194), (157, 195), (158, 195), (158, 194), (157, 193), (159, 195), (160, 195), (161, 196), (160, 197), (161, 198), (162, 199), (163, 198), (164, 198), (167, 199), (166, 199), (166, 197), (169, 193), (168, 193), (166, 190), (167, 191), (170, 197), (178, 183), (179, 182), (187, 198), (184, 200), (185, 201), (185, 202), (186, 202), (186, 203), (185, 204), (185, 203), (187, 202), (186, 204), (184, 205), (185, 205), (184, 204), (187, 203), (188, 203), (188, 204), (189, 203), (186, 201), (186, 200), (187, 199), (186, 199), (189, 198), (189, 197), (190, 197), (190, 199), (190, 200), (190, 198), (189, 199), (191, 198), (192, 199), (192, 198), (193, 198), (192, 200), (191, 200), (191, 201), (190, 201), (191, 202), (192, 203), (193, 202), (194, 203), (194, 202), (194, 201), (192, 202), (192, 201), (190, 202), (191, 203), (189, 201), (191, 199), (189, 200), (188, 200), (187, 201), (187, 200), (183, 202), (184, 202), (184, 203), (183, 203), (182, 203), (182, 204), (181, 205), (182, 205), (181, 206), (182, 206), (180, 206), (179, 207), (180, 208), (179, 208), (178, 208), (179, 209), (178, 210), (178, 211), (179, 211), (180, 212), (181, 213), (180, 210), (181, 211), (179, 210), (178, 209), (177, 208), (177, 207), (176, 208), (178, 207), (179, 206), (178, 206), (177, 205), (176, 206), (176, 207), (175, 208), (175, 207), (175, 206), (176, 205), (177, 204), (178, 205), (179, 204), (180, 205), (179, 205), (181, 204), (183, 204), (188, 201), (189, 202), (182, 201), (181, 201), (174, 196), (175, 198), (171, 198), (172, 199), (171, 200), (171, 201), (170, 200), (168, 200), (168, 197), (179, 200), (182, 199), (180, 201), (178, 200), (177, 200), (176, 201), (174, 200), (173, 200), (172, 201), (169, 201), (169, 202), (170, 201), (171, 202), (172, 203), (171, 204), (170, 205), (169, 205), (169, 206), (168, 207), (168, 208), (167, 208), (166, 209), (165, 209), (165, 210), (166, 211), (167, 210), (166, 210), (168, 210), (169, 211), (168, 211), (167, 211), (170, 211), (170, 210), (169, 210), (171, 212), (168, 209), (167, 212), (166, 213), (170, 217), (170, 218), (170, 219), (171, 220), (171, 221), (172, 222), (171, 223), (172, 221), (173, 220), (174, 221), (173, 222), (173, 221), (174, 222), (174, 219), (174, 218), (175, 218), (176, 219), (176, 220), (177, 221), (177, 219), (178, 219), (179, 219), (178, 218), (177, 218), (178, 217), (179, 216), (179, 215), (179, 214), (178, 213), (178, 212), (177, 211), (176, 210), (175, 210), (176, 211), (177, 209), (177, 210), (176, 212), (177, 212), (176, 209), (174, 207), (173, 207), (172, 206), (171, 205), (171, 206), (170, 203), (170, 202), (168, 201), (167, 201), (166, 201), (166, 200), (165, 200), (166, 202), (166, 203), (167, 203), (164, 199), (165, 199), (164, 200), (163, 201), (162, 202), (161, 203), (162, 203), (162, 204), (161, 205), (160, 206), (159, 205), (158, 205), (157, 206), (156, 206), (156, 205), (155, 204), (156, 204), (156, 203), (155, 202), (156, 202), (157, 203), (157, 204), (158, 203), (159, 204), (158, 204), (159, 203), (158, 202), (157, 201), (156, 200), (158, 200), (159, 199), (158, 198), (158, 197), (157, 196), (159, 198), (159, 196), (159, 197), (158, 196), (156, 195), (157, 194), (154, 195), (154, 196), (153, 196), (155, 194), (153, 195), (151, 194), (150, 193), (149, 192), (150, 192), (149, 193), (149, 194), (150, 194), (151, 193), (148, 195), (147, 194), (146, 195), (146, 194), (150, 198), (150, 197), (152, 197), (153, 197), (152, 195), (149, 198), (148, 199), (149, 199), (147, 200), (146, 201), (147, 201), (147, 202), (147, 203), (148, 204), (149, 205), (148, 205), (149, 206), (148, 207), (148, 206), (147, 205), (146, 203), (147, 204), (148, 203), (149, 202), (149, 203), (149, 201), (148, 200), (150, 199), (147, 199), (146, 198), (149, 200), (145, 197), (144, 197), (145, 196), (144, 196), (143, 197), (142, 197), (142, 196), (141, 197), (141, 198), (141, 199), (140, 200), (141, 201), (142, 200), (142, 199), (143, 199), (144, 200), (144, 201), (144, 202), (143, 202), (142, 201), (143, 200), (144, 203), (145, 203), (144, 204), (143, 205), (143, 204), (145, 205), (144, 206), (145, 207), (146, 207), (146, 206), (144, 205), (145, 204), (145, 202), (143, 201), (142, 202), (140, 202), (139, 201), (139, 200), (138, 199), (138, 198), (139, 197), (139, 198), (140, 199), (139, 202), (138, 202), (140, 203), (140, 201), (141, 200), (141, 203), (142, 204), (141, 205), (140, 204), (142, 206), (145, 206), (144, 207), (145, 208), (144, 208), (144, 209), (143, 208), (142, 208), (143, 207), (142, 209), (141, 208), (140, 207), (140, 206), (140, 205), (139, 206), (141, 207), (142, 207), (141, 206), (143, 206), (143, 209), (145, 209), (146, 208), (147, 207), (150, 207), (151, 207), (152, 208), (153, 208), (152, 207), (151, 208), (153, 207), (153, 206), (153, 205), (152, 206), (154, 204), (155, 203), (154, 205), (154, 207), (155, 208), (156, 208), (155, 209), (157, 209), (156, 210), (156, 209), (157, 207), (158, 206), (158, 207), (155, 210), (157, 208), (155, 211), (154, 212), (153, 213), (153, 214), (152, 213), (154, 214), (152, 212), (152, 215), (152, 214), (153, 215), (154, 215), (155, 215), (155, 216), (156, 217), (157, 217), (158, 216), (156, 216), (156, 215), (156, 214), (155, 214), (156, 213), (156, 212), (155, 212), (155, 213), (154, 213), (154, 216), (155, 218), (155, 219), (154, 218), (153, 219), (153, 220), (153, 221), (152, 221), (152, 222), (152, 219), (151, 220), (152, 220), (154, 220), (155, 221), (156, 222), (156, 221), (156, 219), (157, 218), (157, 216), (155, 217), (156, 218), (157, 220), (157, 221), (158, 222), (159, 223), (159, 222), (160, 223), (161, 222), (162, 222), (162, 221), (162, 219), (163, 216), (162, 215), (160, 221), (161, 221), (159, 218), (163, 219), (164, 220), (164, 219), (163, 220), (165, 219), (166, 218), (165, 216), (166, 216), (167, 217), (168, 218), (169, 217), (169, 218), (169, 219), (171, 219), (172, 218), (171, 218), (164, 217), (164, 215), (163, 217), (164, 218), (157, 215), (154, 219), (163, 218), (155, 222), (157, 222), (153, 218), (152, 218), (151, 218), (150, 218), (149, 219), (150, 219), (150, 220), (151, 221), (150, 222), (151, 222), (149, 221), (148, 222), (147, 223), (148, 224), (149, 225), (149, 227), (150, 227), (150, 228), (149, 229), (151, 228), (152, 227), (153, 228), (154, 228), (153, 229), (152, 228), (150, 229), (149, 228), (151, 224), (150, 224), (150, 225), (149, 224), (148, 223), (149, 222), (149, 223), (146, 223), (145, 222), (146, 221), (144, 222), (144, 223), (143, 223), (143, 222), (143, 221), (142, 220), (143, 219), (143, 220), (142, 219), (142, 218), (141, 220), (140, 220), (141, 219), (140, 218), (141, 218), (141, 217), (141, 216), (142, 215), (143, 216), (144, 217), (144, 216), (145, 218), (144, 218), (145, 217), (143, 217), (145, 216), (144, 215), (143, 214), (143, 215), (142, 216), (142, 217), (144, 213), (144, 214), (142, 214), (145, 214), (144, 212), (143, 211), (142, 210), (141, 210), (140, 209), (142, 211), (143, 213), (145, 213), (146, 212), (146, 211), (146, 210), (145, 211), (145, 210), (147, 212), (148, 212), (149, 213), (150, 213), (151, 214), (150, 214), (149, 214), (151, 212), (151, 213), (151, 211), (151, 210), (151, 209), (150, 210), (150, 208), (149, 208), (149, 207), (150, 205), (151, 204), (151, 203), (152, 202), (152, 203), (152, 204), (150, 203), (149, 204), (148, 201), (146, 199), (145, 198), (143, 198), (142, 198), (144, 199), (145, 200), (145, 199), (144, 198), (143, 196), (144, 195), (145, 194), (144, 194), (145, 195), (146, 202), (150, 200), (151, 200), (146, 200), (146, 204), (147, 206), (146, 205), (145, 201), (148, 202), (150, 201), (144, 193), (145, 193), (145, 192), (143, 195), (142, 195), (140, 198), (141, 202), (142, 203), (139, 203), (139, 204), (138, 203), (148, 197), (143, 194), (143, 193), (142, 194), (146, 193), (147, 192), (147, 193), (144, 192), (151, 190), (152, 191), (153, 190), (153, 191), (154, 192), (152, 196), (152, 198), (153, 199), (154, 200), (154, 199), (155, 199), (153, 200), (153, 201), (153, 202), (153, 203), (152, 205), (151, 205), (151, 206), (148, 209), (149, 209), (148, 208), (147, 208), (146, 209), (147, 209), (148, 210), (149, 211), (150, 211), (152, 210), (153, 210), (153, 209), (154, 208), (155, 207), (154, 206), (155, 206), (156, 207), (154, 209), (154, 210), (154, 211), (156, 211), (157, 211), (158, 210), (158, 211), (159, 212), (157, 212), (157, 210), (153, 211), (152, 211), (150, 209), (150, 206), (142, 205), (151, 201), (152, 201), (152, 200), (151, 199), (151, 202), (150, 202), (154, 203), (155, 205), (153, 204), (156, 201), (155, 200), (155, 201), (154, 201), (154, 202), (157, 205), (159, 206), (160, 205), (160, 203), (161, 206), (159, 207), (161, 204), (162, 205), (162, 206), (163, 204), (164, 203), (165, 202), (164, 201), (165, 201), (164, 196), (171, 203), (170, 204), (172, 204), (173, 205), (172, 205), (170, 206), (173, 203), (174, 203), (175, 204), (174, 205), (174, 204), (173, 204), (172, 202), (173, 202), (168, 206), (168, 205), (167, 205), (167, 206), (167, 207), (166, 207), (166, 206), (165, 207), (165, 206), (166, 208), (166, 205), (165, 204), (163, 200), (161, 199), (163, 197), (162, 198), (163, 199), (162, 200), (162, 201), (161, 201), (160, 200), (161, 200), (160, 202), (159, 208), (158, 208), (160, 207), (157, 200), (158, 201), (158, 199), (156, 199), (156, 198), (156, 197), (155, 197), (154, 197), (155, 198), (154, 198), (153, 198), (156, 192), (155, 191), (157, 191), (156, 190), (156, 189), (156, 188), (157, 189), (157, 192), (155, 196), (158, 193), (157, 197), (159, 200), (159, 201), (160, 201), (160, 199), (161, 197), (160, 198), (161, 202), (164, 204), (163, 203), (165, 205), (164, 206), (163, 206), (163, 207), (162, 208), (163, 208), (164, 207), (163, 205), (164, 205), (160, 204), (161, 207), (166, 204), (167, 202), (164, 202), (165, 208), (164, 209), (161, 210), (160, 209), (161, 209), (163, 211), (165, 212), (163, 215), (162, 220), (160, 222), (161, 223), (162, 224), (162, 225), (163, 224), (163, 223), (164, 222), (165, 221), (164, 221), (163, 222), (164, 223), (165, 222), (166, 222), (165, 223), (165, 224), (164, 225), (163, 225), (162, 226), (161, 227), (162, 227), (164, 227), (163, 226), (161, 226), (166, 225), (166, 221), (168, 219), (164, 213), (167, 214), (168, 213), (168, 214), (167, 213), (168, 212), (169, 212), (169, 215), (166, 212), (165, 211), (164, 210), (169, 213), (170, 212), (171, 216), (172, 217), (173, 219), (174, 220), (175, 221), (175, 220), (175, 219), (175, 217), (176, 216), (177, 217), (176, 217), (177, 216), (175, 216), (174, 217), (173, 218), (172, 220), (172, 219), (173, 217), (172, 216), (171, 215), (170, 214), (172, 214), (172, 213), (173, 212), (174, 211), (173, 210), (173, 209), (174, 210), (175, 209), (173, 211), (172, 210), (175, 211), (175, 212), (176, 213), (177, 214), (176, 215), (177, 215), (179, 212), (180, 213), (182, 214), (182, 213), (183, 213), (182, 212), (181, 212), (183, 212), (184, 211), (184, 210), (185, 210), (185, 211), (186, 211), (185, 212), (184, 213), (181, 214), (182, 215), (183, 215), (183, 216), (182, 216), (182, 217), (181, 216), (184, 218), (184, 217), (183, 217), (188, 217), (189, 219), (189, 220), (189, 221), (190, 221), (191, 220), (191, 221), (191, 222), (192, 221), (193, 222), (192, 223), (192, 222), (191, 223), (190, 223), (194, 222), (194, 221), (194, 218), (193, 215), (193, 216), (192, 215), (191, 214), (191, 215), (189, 218), (189, 215), (190, 218), (190, 219), (191, 218), (189, 216), (192, 216), (194, 215), (192, 214), (192, 219), (189, 213), (189, 211), (190, 210), (191, 210), (192, 211), (193, 210), (193, 209), (194, 210), (193, 207), (192, 206), (193, 205), (196, 204), (196, 202), (193, 201), (195, 200), (197, 204), (195, 208), (196, 210), (195, 209), (193, 208), (194, 209), (198, 210), (197, 209), (197, 208), (199, 215), (191, 213), (188, 214), (188, 213), (187, 213), (188, 212), (187, 212), (184, 214), (183, 214), (180, 214), (180, 215), (182, 218), (183, 218), (188, 219), (185, 214), (184, 215), (183, 211), (186, 212), (187, 211), (190, 211), (189, 212), (188, 211), (187, 210), (186, 210), (186, 209), (185, 209), (186, 208), (187, 207), (186, 207), (187, 206), (186, 205), (187, 204), (188, 202), (191, 197), (192, 197), (193, 194), (192, 195), (192, 194), (191, 194), (190, 195), (189, 196), (188, 195), (188, 194), (186, 194), (188, 199), (196, 196), (196, 195), (197, 195), (191, 195), (190, 196), (191, 196), (193, 200), (183, 205), (196, 201), (194, 205), (195, 211), (194, 211), (193, 212), (207, 212), (208, 211), (207, 211), (214, 200), (215, 199), (226, 195), (226, 196), (219, 202), (218, 197), (226, 189), (227, 199), (231, 214), (226, 227), (228, 230), (231, 230), (230, 232), (234, 227), (235, 228), (259, 185), (240, 194), (238, 196), (231, 192), (230, 191), (231, 190), (230, 179), (228, 193), (221, 190), (224, 189), (222, 194), (221, 194), (216, 194), (221, 193), (226, 208), (227, 209), (212, 209), (213, 199), (215, 194), (225, 207), (225, 188), (235, 213), (238, 215), (223, 228), (221, 224), (219, 223), (218, 222), (217, 221), (215, 221), (214, 222), (213, 221), (217, 220), (216, 219), (217, 219), (213, 220), (212, 219), (213, 218), (213, 217), (211, 216), (212, 217), (212, 218), (211, 218), (210, 219), (209, 219), (211, 219), (211, 217), (210, 216), (209, 217), (206, 212), (208, 210), (208, 217), (209, 218), (211, 210), (212, 208), (208, 204), (204, 200), (199, 190), (196, 193), (195, 193), (194, 192), (193, 191), (197, 183), (192, 162), (183, 167), (178, 151), (153, 146), (153, 143), (153, 144), (152, 143), (150, 144), (150, 145), (151, 146), (150, 142), (151, 143), (152, 144), (152, 146), (151, 145), (150, 147), (150, 146), (149, 146), (149, 145), (149, 147), (148, 147), (147, 147), (146, 147), (146, 148), (145, 149), (145, 150), (144, 151), (145, 152), (144, 152), (145, 151), (146, 150), (144, 149), (141, 152), (144, 150), (146, 151), (146, 152), (147, 152), (146, 153), (147, 154), (146, 154), (145, 153), (147, 153), (146, 155), (145, 155), (143, 151), (141, 151), (140, 151), (140, 150), (141, 149), (143, 148), (143, 147), (142, 147), (141, 146), (141, 145), (143, 144), (144, 143), (145, 142), (145, 143), (146, 142), (144, 142), (143, 143), (141, 143), (140, 146), (143, 145), (144, 148), (144, 147), (144, 145), (145, 144), (146, 144), (147, 144), (148, 145), (150, 141), (151, 140), (149, 142), (148, 143), (149, 141), (150, 140), (151, 139), (150, 138), (150, 139), (149, 139), (149, 138), (149, 140), (148, 139), (148, 140), (148, 141), (148, 142), (147, 141), (146, 140), (145, 139), (146, 139), (147, 140), (148, 138), (147, 142), (147, 139), (147, 138), (149, 143), (147, 143), (148, 144), (146, 145), (145, 145), (146, 146), (145, 147), (147, 148), (146, 149), (144, 144), (145, 141), (146, 141), (149, 144), (144, 141), (145, 140), (145, 138), (144, 139), (144, 140), (146, 138), (143, 139), (143, 140), (144, 146), (145, 146), (145, 148), (147, 149), (148, 148), (149, 148), (163, 138), (164, 139), (164, 138), (183, 138), (183, 139), (179, 138), (247, 153), (248, 153), (247, 154), (246, 154), (245, 155), (246, 155), (246, 158), (245, 159), (244, 160), (242, 157), (243, 158), (243, 156), (242, 155), (243, 155), (244, 154), (244, 157), (244, 158), (244, 159), (246, 156), (245, 163), (245, 162), (245, 161), (246, 162), (247, 163), (246, 164), (248, 164), (249, 163), (248, 163), (248, 162), (249, 164), (255, 163), (255, 162), (256, 161), (254, 156), (250, 153), (248, 158), (247, 157), (245, 158), (246, 159), (246, 160), (247, 160), (248, 159), (248, 160), (249, 157), (263, 157), (264, 156), (264, 155), (265, 155), (266, 156), (265, 156), (263, 156), (262, 155), (261, 154), (262, 156), (263, 155), (261, 155), (258, 152), (254, 144), (253, 143), (254, 142), (252, 144), (251, 143), (251, 144), (244, 149), (245, 146), (246, 153), (243, 152), (243, 149), (243, 146), (157, 70), (263, 6), (234, 97), (264, 7), (266, 6), (264, 8), (266, 9), (270, 5), (271, 5), (272, 6), (273, 5), (272, 5), (273, 6), (274, 7), (274, 8), (275, 8), (276, 8), (275, 9), (276, 10), (276, 9), (274, 9), (275, 10), (277, 8), (277, 7), (277, 6), (276, 6), (275, 7), (276, 7), (278, 9), (279, 10), (278, 10), (278, 11), (279, 11), (280, 10), (280, 11), (281, 12), (280, 12), (280, 13), (279, 12), (278, 12), (277, 11), (279, 13), (281, 11), (277, 13), (276, 14), (277, 14), (278, 15), (276, 13), (277, 12), (277, 10), (280, 14), (281, 14), (281, 13), (281, 15), (280, 9), (279, 9), (278, 8), (279, 7), (280, 6), (281, 7), (282, 7), (282, 6), (280, 8), (279, 8), (280, 7), (281, 5), (278, 7), (275, 6), (276, 11), (275, 12), (274, 12), (274, 13), (275, 13), (275, 11), (278, 13), (279, 14), (278, 14), (279, 15), (280, 15), (281, 16), (282, 14), (275, 14), (273, 13), (273, 14), (274, 15), (272, 13), (272, 14), (272, 15), (275, 15), (275, 16), (276, 15), (282, 16), (283, 16), (283, 17), (284, 16), (283, 15), (284, 17), (285, 18), (286, 19), (285, 19), (286, 20), (286, 21), (285, 21), (286, 22), (285, 23), (287, 21), (287, 22), (285, 20), (284, 20), (286, 18), (286, 17), (284, 19), (283, 18), (282, 19), (282, 18), (283, 19), (284, 18), (282, 17), (281, 17), (280, 16), (280, 17), (280, 18), (279, 18), (281, 18), (280, 19), (279, 19), (278, 18), (277, 17), (276, 17), (277, 18), (278, 17), (276, 19), (276, 20), (276, 21), (275, 21), (275, 22), (274, 20), (275, 20), (274, 19), (275, 18), (275, 17), (274, 16), (274, 17), (273, 17), (272, 18), (273, 18), (272, 17), (276, 16), (275, 19), (272, 19), (272, 20), (273, 21), (273, 19), (272, 16), (274, 18), (273, 20), (274, 21), (272, 21), (273, 15), (273, 16), (277, 16), (276, 18), (274, 22), (273, 22), (274, 23), (273, 23), (274, 24), (277, 19), (277, 20), (276, 22), (275, 23), (274, 25), (273, 24), (275, 24), (276, 25), (276, 26), (275, 27), (275, 26), (272, 25), (273, 25), (272, 26), (273, 26), (273, 27), (274, 27), (274, 28), (273, 28), (274, 26), (275, 25), (276, 23), (277, 24), (277, 25), (278, 24), (278, 23), (279, 22), (280, 23), (280, 22), (281, 22), (280, 24), (281, 24), (281, 25), (280, 25), (281, 23), (282, 22), (279, 23), (278, 22), (280, 21), (279, 20), (278, 19), (280, 20), (279, 24), (278, 21), (279, 21), (278, 20), (277, 21), (272, 22), (277, 22), (282, 23), (283, 24), (282, 25), (283, 25), (284, 25), (285, 25), (286, 24), (286, 23), (287, 23), (287, 24), (288, 24), (287, 25), (288, 26), (289, 27), (290, 27), (289, 26), (290, 26), (288, 27), (289, 28), (288, 28), (289, 29), (287, 28), (288, 29), (287, 29), (287, 30), (286, 30), (286, 31), (286, 29), (285, 30), (288, 30), (287, 26), (286, 27), (285, 27), (284, 28), (286, 26), (288, 25), (290, 29), (286, 25), (285, 24), (284, 24), (283, 26), (284, 23), (284, 22), (283, 21), (287, 18), (281, 20), (281, 19), (285, 17), (286, 16), (286, 15), (285, 15), (286, 14), (286, 13), (285, 12), (285, 11), (286, 12), (286, 10), (287, 11), (287, 12), (286, 11), (285, 13), (285, 14), (284, 15), (284, 14), (285, 16), (287, 14), (288, 15), (289, 15), (290, 14), (290, 15), (290, 16), (291, 16), (292, 16), (292, 17), (292, 15), (291, 14), (290, 13), (291, 13), (292, 14), (291, 15), (293, 13), (294, 12), (294, 13), (295, 14), (295, 13), (295, 11), (296, 14), (296, 13), (297, 14), (297, 15), (296, 16), (295, 15), (294, 15), (293, 16), (294, 14), (295, 16), (295, 17), (296, 17), (295, 18), (297, 16), (296, 15), (297, 13), (298, 13), (299, 13), (298, 12), (299, 14), (299, 15), (298, 14), (300, 14), (300, 13), (298, 15), (299, 12), (300, 12), (300, 11), (298, 11), (299, 10), (298, 10), (297, 11), (299, 9), (298, 9), (297, 10), (297, 9), (299, 11), (296, 10), (295, 9), (296, 9), (297, 8), (296, 8), (297, 7), (298, 7), (298, 8), (299, 6), (300, 6), (300, 5), (301, 7), (299, 7), (298, 6), (301, 6), (300, 7), (300, 8), (300, 9), (301, 8), (302, 7), (302, 8), (299, 5), (302, 6), (303, 6), (303, 7), (303, 8), (302, 9), (303, 9), (302, 10), (301, 9), (301, 5), (298, 5), (299, 8), (295, 8), (294, 9), (293, 9), (292, 8), (292, 7), (291, 9), (292, 10), (291, 10), (290, 11), (291, 12), (292, 13), (293, 14), (291, 17), (290, 17), (289, 16), (293, 15), (294, 17), (294, 16), (296, 18), (294, 19), (293, 19), (295, 20), (296, 21), (296, 22), (297, 22), (298, 23), (299, 24), (300, 24), (300, 25), (301, 26), (300, 27), (300, 26), (299, 26), (298, 26), (299, 27), (299, 28), (297, 26), (297, 27), (298, 27), (298, 25), (297, 24), (297, 23), (298, 24), (299, 25), (301, 25), (302, 25), (303, 25), (302, 24), (301, 24), (300, 23), (299, 23), (301, 23), (302, 23), (303, 24), (300, 28), (301, 27), (302, 26), (302, 27), (302, 28), (303, 29), (304, 29), (304, 28), (304, 27), (305, 26), (302, 30), (303, 30), (305, 28), (305, 29), (306, 28), (305, 27), (303, 28), (303, 27), (306, 25), (305, 25), (306, 26), (304, 26), (303, 26), (304, 25), (301, 29), (299, 29), (300, 30), (299, 30), (300, 31), (301, 30), (302, 31), (303, 32), (302, 33), (303, 34), (302, 35), (301, 34), (300, 34), (299, 33), (298, 32), (299, 31), (299, 32), (301, 31), (301, 32), (303, 31), (302, 32), (304, 31), (305, 31), (305, 32), (304, 33), (306, 33), (305, 33), (304, 34), (303, 33), (304, 32), (305, 30), (306, 31), (307, 32), (306, 32), (308, 33), (307, 33), (307, 31), (308, 31), (308, 30), (309, 30), (308, 29), (309, 29), (306, 30), (307, 30), (308, 32), (309, 32), (309, 31), (310, 30), (310, 31), (306, 29), (304, 30), (300, 29), (301, 28), (298, 28), (297, 29), (296, 30), (295, 31), (296, 31), (295, 30), (295, 32), (296, 33), (295, 33), (294, 32), (293, 33), (294, 33), (293, 32), (294, 31), (292, 34), (293, 34), (293, 30), (294, 29), (295, 29), (294, 30), (296, 29), (298, 29), (297, 30), (296, 26), (295, 26), (294, 25), (295, 24), (296, 25), (297, 25), (296, 24), (295, 25), (294, 26), (293, 27), (292, 27), (293, 28), (292, 28), (291, 27), (291, 26), (292, 26), (291, 25), (293, 26), (293, 25), (297, 28), (296, 27), (295, 27), (297, 21), (298, 22), (300, 22), (300, 21), (299, 20), (304, 24), (303, 23), (299, 21), (299, 22), (301, 21), (301, 22), (296, 23), (302, 29), (307, 25), (308, 26), (307, 27), (307, 26), (308, 25), (305, 24), (306, 23), (307, 24), (307, 23), (308, 23), (309, 22), (310, 23), (311, 22), (311, 21), (312, 20), (313, 20), (313, 19), (312, 18), (311, 17), (311, 18), (312, 19), (311, 20), (312, 21), (310, 21), (310, 20), (311, 24), (312, 24), (311, 25), (310, 26), (309, 26), (309, 27), (309, 25), (310, 25), (311, 26), (312, 26), (311, 27), (312, 27), (311, 28), (310, 29), (310, 28), (309, 28), (310, 27), (312, 29), (312, 30), (311, 30), (312, 31), (311, 32), (312, 33), (311, 34), (312, 34), (313, 33), (311, 33), (313, 30), (314, 29), (315, 28), (316, 28), (317, 27), (316, 27), (315, 26), (316, 25), (317, 24), (317, 23), (318, 23), (316, 24), (315, 25), (314, 26), (313, 27), (313, 26), (312, 25), (310, 24), (312, 23), (313, 28), (311, 31), (310, 32), (314, 33), (314, 32), (313, 31), (312, 32), (313, 32), (314, 31), (315, 31), (316, 31), (317, 30), (318, 29), (319, 29), (318, 28), (317, 28), (318, 27), (318, 26), (319, 26), (320, 26), (320, 25), (319, 27), (320, 27), (321, 27), (321, 28), (320, 29), (320, 30), (319, 30), (317, 29), (317, 26), (316, 29), (316, 26), (315, 27), (314, 28), (315, 29), (314, 30), (315, 33), (314, 34), (313, 34), (314, 35), (313, 35), (312, 35), (311, 36), (311, 35), (310, 35), (310, 36), (309, 36), (309, 37), (309, 38), (308, 38), (307, 39), (306, 40), (306, 41), (307, 42), (306, 43), (307, 44), (306, 44), (305, 44), (304, 43), (304, 42), (305, 42), (308, 42), (309, 41), (309, 40), (310, 39), (311, 38), (312, 39), (312, 38), (313, 38), (313, 37), (313, 36), (314, 36), (315, 35), (315, 36), (314, 37), (315, 37), (316, 37), (312, 36), (309, 35), (308, 35), (308, 36), (307, 35), (306, 34), (307, 34), (308, 34), (309, 34), (310, 34), (310, 33), (305, 34), (305, 35), (302, 34), (301, 35), (301, 36), (302, 36), (301, 37), (301, 38), (302, 38), (303, 36), (303, 37), (303, 38), (302, 37), (300, 37), (304, 38), (305, 38), (305, 37), (303, 39), (303, 40), (302, 41), (301, 42), (300, 42), (301, 43), (302, 42), (301, 41), (300, 41), (299, 41), (301, 40), (299, 40), (299, 39), (300, 38), (299, 36), (300, 35), (299, 35), (299, 34), (300, 33), (296, 28), (295, 23), (295, 28), (295, 22), (294, 21), (294, 22), (298, 21), (300, 32), (301, 33), (300, 36), (303, 35), (303, 22), (304, 22), (302, 21), (302, 22), (303, 21), (298, 20), (300, 20), (299, 19), (300, 18), (300, 17), (301, 18), (302, 19), (302, 20), (303, 19), (304, 20), (305, 19), (306, 19), (307, 20), (308, 20), (308, 21), (309, 21), (309, 20), (308, 22), (307, 22), (307, 21), (306, 21), (305, 21), (305, 22), (305, 23), (304, 21), (304, 23), (309, 23), (310, 22), (311, 23), (309, 24), (308, 27), (307, 28), (306, 27), (306, 35), (306, 36), (306, 37), (307, 36), (308, 37), (298, 31), (298, 30), (297, 31), (293, 29), (293, 31), (292, 32), (291, 32), (291, 31), (292, 31), (292, 30), (291, 29), (296, 32), (298, 33), (298, 34), (297, 34), (297, 33), (296, 34), (295, 34), (295, 35), (296, 35), (297, 36), (296, 36), (297, 37), (297, 38), (298, 39), (298, 40), (300, 40), (300, 39), (298, 38), (299, 37), (298, 37), (301, 39), (297, 39), (297, 40), (304, 35), (304, 36), (306, 38), (307, 37), (310, 37), (309, 39), (312, 28), (313, 29), (316, 30), (318, 30), (321, 25), (321, 26), (322, 27), (322, 28), (323, 27), (320, 28), (322, 26), (319, 25), (319, 24), (318, 24), (317, 25), (318, 25), (320, 24), (319, 28), (322, 29), (323, 26), (324, 27), (319, 31), (320, 31), (321, 32), (322, 31), (323, 30), (324, 29), (325, 30), (326, 30), (325, 29), (326, 28), (325, 27), (324, 26), (325, 26), (326, 25), (327, 25), (327, 26), (327, 24), (327, 23), (328, 24), (329, 23), (330, 22), (331, 23), (329, 22), (330, 21), (329, 20), (328, 21), (329, 21), (327, 20), (326, 20), (326, 21), (325, 22), (324, 21), (324, 23), (323, 24), (324, 24), (325, 23), (326, 24), (325, 25), (322, 25), (321, 23), (322, 22), (321, 21), (321, 22), (320, 23), (320, 22), (319, 23), (323, 25), (326, 27), (325, 28), (326, 29), (326, 31), (327, 28), (327, 29), (327, 31), (326, 32), (327, 32), (328, 31), (325, 32), (327, 30), (328, 32), (329, 32), (330, 32), (330, 33), (329, 33), (330, 31), (329, 30), (330, 30), (331, 31), (331, 30), (329, 31), (328, 30), (325, 31), (324, 30), (321, 31), (322, 32), (323, 32), (324, 31), (324, 33), (325, 33), (324, 32), (323, 31), (322, 30), (323, 29), (324, 28), (326, 33), (324, 22), (323, 23), (323, 22), (322, 24), (322, 23), (322, 21), (320, 21), (319, 20), (318, 20), (319, 21), (320, 20), (319, 19), (320, 19), (321, 20), (321, 19), (322, 19), (323, 19), (324, 18), (324, 19), (325, 18), (326, 19), (326, 18), (323, 17), (322, 18), (321, 24), (318, 22), (319, 22), (323, 21), (324, 25), (325, 24), (323, 28), (328, 29), (329, 29), (330, 29), (331, 32), (332, 30), (332, 31), (332, 32), (333, 32), (332, 33), (332, 34), (331, 34), (331, 35), (330, 35), (332, 35), (333, 34), (333, 33), (331, 33), (330, 34), (331, 36), (332, 36), (330, 36), (330, 37), (330, 38), (331, 39), (330, 39), (329, 40), (328, 41), (328, 42), (327, 42), (327, 43), (326, 44), (327, 45), (327, 46), (327, 47), (326, 48), (328, 46), (329, 45), (329, 46), (330, 46), (329, 47), (330, 47), (331, 48), (331, 47), (331, 46), (332, 47), (331, 49), (330, 48), (330, 49), (332, 49), (332, 48), (333, 49), (332, 50), (331, 50), (330, 50), (329, 49), (333, 50), (334, 49), (335, 48), (334, 48), (334, 50), (335, 51), (336, 52), (337, 52), (337, 51), (338, 50), (337, 50), (336, 50), (336, 49), (337, 49), (338, 48), (337, 48), (336, 47), (336, 46), (335, 45), (335, 47), (335, 46), (334, 45), (334, 44), (333, 45), (335, 43), (335, 44), (336, 43), (334, 43), (333, 43), (333, 44), (334, 42), (335, 41), (336, 44), (337, 43), (336, 42), (337, 45), (337, 44), (338, 44), (338, 43), (339, 44), (339, 43), (340, 43), (339, 42), (341, 42), (339, 45), (338, 45), (338, 42), (339, 41), (338, 41), (339, 40), (338, 40), (337, 40), (336, 39), (335, 40), (336, 38), (335, 39), (337, 39), (338, 39), (337, 38), (337, 37), (336, 37), (338, 37), (339, 37), (339, 38), (338, 38), (340, 37), (341, 38), (340, 38), (340, 36), (340, 35), (341, 36), (339, 36), (339, 39), (341, 37), (341, 34), (341, 33), (342, 33), (343, 32), (344, 33), (344, 34), (345, 35), (344, 35), (343, 36), (343, 37), (342, 36), (342, 35), (343, 34), (343, 33), (342, 34), (343, 35), (345, 34), (346, 35), (346, 36), (345, 36), (347, 36), (348, 36), (348, 35), (347, 35), (346, 34), (345, 33), (347, 37), (348, 37), (346, 37), (345, 37), (344, 38), (344, 39), (343, 39), (342, 38), (341, 39), (340, 40), (341, 40), (342, 39), (341, 35), (344, 32), (344, 31), (343, 31), (342, 31), (343, 30), (342, 32), (340, 33), (341, 32), (341, 31), (342, 30), (341, 30), (344, 36), (344, 37), (343, 38), (345, 40), (344, 41), (344, 40), (343, 40), (342, 41), (342, 40), (342, 42), (343, 43), (342, 44), (342, 45), (342, 46), (341, 46), (340, 46), (340, 45), (339, 46), (340, 44), (340, 41), (346, 39), (346, 38), (345, 38), (348, 38), (349, 38), (347, 38), (346, 40), (345, 41), (344, 42), (344, 43), (345, 42), (346, 41), (345, 39), (349, 39), (350, 38), (351, 38), (352, 38), (353, 37), (353, 36), (353, 35), (354, 34), (354, 33), (354, 32), (354, 31), (355, 31), (354, 30), (353, 31), (352, 32), (353, 32), (351, 32), (351, 33), (352, 34), (351, 35), (352, 36), (351, 36), (351, 34), (352, 35), (350, 34), (350, 33), (350, 32), (350, 31), (352, 33), (351, 31), (353, 33), (351, 30), (350, 30), (350, 29), (351, 28), (350, 27), (349, 27), (349, 28), (350, 28), (351, 27), (352, 27), (353, 28), (352, 29), (353, 29), (353, 30), (354, 29), (355, 30), (356, 31), (357, 32), (358, 33), (357, 34), (356, 35), (357, 36), (358, 35), (359, 35), (358, 36), (358, 37), (359, 36), (359, 37), (357, 37), (356, 38), (356, 37), (355, 37), (354, 37), (354, 38), (354, 36), (354, 35), (353, 34), (352, 37), (351, 37), (350, 37), (349, 36), (350, 35), (349, 33), (348, 32), (349, 32), (349, 30), (349, 31), (348, 33), (348, 29), (347, 28), (347, 29), (348, 30), (349, 29), (348, 28), (346, 29), (347, 30), (348, 31), (352, 31), (350, 36), (349, 35), (349, 34), (352, 39), (351, 39), (351, 40), (350, 39), (349, 37), (350, 40), (351, 41), (352, 41), (352, 40), (349, 41), (350, 42), (351, 43), (350, 43), (350, 44), (351, 45), (352, 45), (353, 45), (352, 46), (351, 46), (352, 44), (353, 43), (354, 43), (354, 42), (353, 41), (354, 40), (354, 39), (353, 39), (355, 40), (354, 41), (353, 42), (351, 42), (350, 41), (353, 38), (347, 33), (348, 34), (347, 32), (356, 39), (353, 40), (349, 40), (348, 40), (348, 41), (348, 42), (347, 43), (348, 44), (348, 43), (347, 42), (346, 42), (347, 44), (346, 44), (345, 43), (344, 44), (343, 44), (343, 45), (343, 42), (343, 41), (342, 37), (345, 44), (344, 45), (341, 45), (343, 46), (342, 47), (341, 47), (340, 47), (339, 47), (338, 47), (338, 49), (337, 47), (337, 46), (338, 46), (335, 52), (335, 53), (336, 54), (336, 55), (336, 56), (335, 55), (334, 56), (335, 57), (335, 58), (334, 57), (333, 58), (333, 57), (334, 58), (332, 58), (332, 59), (331, 57), (332, 57), (331, 58), (332, 56), (330, 56), (330, 57), (329, 58), (329, 57), (330, 58), (328, 57), (327, 56), (326, 57), (325, 57), (324, 58), (323, 58), (324, 59), (324, 60), (325, 59), (325, 61), (325, 62), (326, 62), (326, 63), (327, 64), (328, 64), (328, 63), (329, 62), (329, 61), (330, 62), (330, 61), (329, 63), (329, 64), (329, 65), (327, 63), (328, 62), (328, 61), (327, 60), (327, 62), (325, 60), (324, 61), (325, 63), (325, 64), (324, 64), (323, 63), (322, 64), (323, 65), (323, 64), (324, 63), (323, 62), (324, 62), (325, 65), (324, 66), (324, 67), (323, 67), (322, 66), (321, 65), (322, 63), (321, 63), (321, 64), (320, 63), (319, 63), (319, 62), (320, 64), (320, 62), (319, 61), (318, 60), (317, 59), (316, 58), (317, 57), (315, 58), (315, 57), (314, 58), (313, 57), (314, 56), (314, 57), (315, 56), (315, 55), (316, 55), (317, 56), (318, 56), (318, 55), (319, 55), (319, 54), (319, 53), (320, 52), (321, 51), (319, 52), (318, 52), (319, 51), (318, 51), (317, 51), (316, 51), (316, 52), (317, 52), (318, 53), (318, 54), (317, 53), (316, 53), (315, 54), (319, 56), (318, 57), (317, 58), (318, 58), (318, 59), (319, 58), (316, 57), (316, 56), (317, 55), (317, 54), (315, 52), (314, 53), (315, 51), (316, 54), (314, 55), (314, 54), (315, 53), (319, 57), (316, 59), (320, 57), (320, 56), (321, 55), (322, 55), (323, 55), (324, 54), (325, 54), (325, 53), (325, 52), (324, 53), (324, 52), (323, 52), (323, 51), (322, 52), (322, 53), (321, 52), (323, 53), (326, 52), (327, 53), (326, 53), (326, 51), (327, 51), (326, 50), (327, 49), (327, 50), (326, 49), (328, 49), (328, 50), (325, 48), (324, 48), (325, 47), (325, 49), (324, 47), (324, 46), (324, 45), (323, 46), (323, 45), (325, 44), (326, 43), (327, 44), (328, 45), (328, 47), (329, 48), (328, 48), (330, 45), (331, 45), (332, 45), (334, 46), (333, 47), (333, 46), (333, 48), (329, 51), (328, 51), (327, 52), (328, 53), (327, 54), (326, 54), (327, 55), (325, 51), (324, 50), (323, 49), (324, 49), (325, 50), (323, 54), (321, 53), (320, 54), (320, 53), (322, 51), (320, 51), (319, 50), (320, 50), (319, 49), (320, 48), (321, 48), (322, 49), (321, 49), (319, 48), (318, 48), (320, 47), (321, 47), (321, 46), (321, 45), (322, 45), (322, 47), (320, 49), (318, 49), (319, 47), (321, 50), (322, 48), (323, 48), (327, 48), (326, 47), (325, 55), (326, 55), (331, 51), (333, 51), (332, 51), (335, 49), (334, 47), (332, 46), (336, 45), (341, 43), (341, 41), (341, 44), (344, 47), (343, 48), (344, 48), (343, 47), (342, 43), (340, 42), (345, 45), (345, 46), (344, 46), (345, 47), (346, 46), (346, 43), (342, 48), (341, 48), (340, 49), (339, 50), (340, 51), (339, 49), (339, 48), (340, 50), (340, 52), (341, 53), (342, 52), (341, 51), (341, 52), (341, 50), (339, 51), (340, 48), (347, 40), (347, 41), (348, 45), (347, 45), (346, 47), (345, 48), (337, 41), (336, 40), (334, 39), (334, 40), (333, 39), (335, 38), (334, 38), (333, 37), (331, 37), (333, 36), (334, 32), (334, 31), (335, 30), (334, 29), (335, 31), (336, 30), (337, 31), (338, 30), (339, 30), (340, 31), (343, 29), (344, 30), (345, 29), (345, 30), (344, 29), (342, 29), (343, 28), (342, 28), (341, 29), (340, 28), (340, 27), (339, 28), (340, 29), (339, 27), (338, 27), (339, 26), (340, 25), (340, 26), (341, 26), (341, 25), (342, 26), (343, 26), (344, 27), (345, 27), (345, 26), (346, 28), (345, 28), (351, 26), (352, 28), (353, 27), (354, 27), (353, 26), (351, 29), (346, 33), (346, 32), (345, 31), (346, 31), (346, 30), (346, 27), (346, 26), (347, 26), (348, 26), (347, 25), (346, 25), (345, 24), (346, 24), (347, 24), (348, 23), (348, 24), (347, 23), (345, 23), (346, 23), (345, 22), (346, 21), (347, 21), (347, 22), (346, 22), (349, 23), (349, 22), (344, 26), (345, 25), (344, 28), (339, 31), (336, 31), (336, 32), (335, 32), (337, 30), (339, 29), (335, 33), (334, 30), (335, 29), (334, 33), (334, 34), (333, 35), (334, 35), (333, 30), (333, 31), (336, 28), (337, 28), (337, 29), (336, 29), (333, 29), (332, 28), (333, 27), (332, 27), (331, 27), (332, 26), (331, 26), (331, 25), (331, 24), (332, 24), (332, 23), (331, 22), (331, 21), (332, 21), (333, 21), (333, 22), (332, 20), (333, 20), (334, 19), (333, 19), (334, 18), (333, 18), (334, 20), (334, 22), (335, 22), (335, 21), (334, 21), (332, 22), (333, 23), (330, 24), (330, 25), (329, 26), (328, 27), (329, 28), (328, 26), (329, 25), (330, 26), (329, 24), (328, 23), (328, 22), (330, 23), (329, 27), (330, 27), (330, 28), (326, 26), (328, 25), (328, 28), (327, 27), (326, 23), (324, 20), (325, 19), (325, 21), (325, 20), (323, 20), (322, 20), (321, 18), (321, 17), (321, 16), (322, 17), (322, 16), (323, 15), (324, 14), (323, 14), (324, 16), (324, 17), (323, 18), (325, 17), (327, 18), (328, 19), (327, 19), (328, 20), (327, 17), (327, 16), (328, 15), (328, 16), (329, 16), (329, 17), (328, 18), (328, 17), (326, 17), (329, 19), (331, 20), (331, 19), (330, 19), (330, 18), (329, 18), (327, 21), (327, 22), (325, 16), (324, 15), (323, 16), (325, 15), (326, 16), (330, 20), (326, 22), (329, 15), (330, 16), (330, 17), (331, 16), (331, 17), (332, 17), (333, 17), (333, 16), (335, 17), (335, 16), (334, 17), (335, 18), (332, 19), (331, 18), (330, 15), (329, 14), (330, 13), (329, 13), (330, 12), (329, 12), (331, 12), (330, 11), (331, 10), (332, 9), (333, 9), (333, 8), (334, 7), (335, 6), (336, 5), (337, 5), (336, 6), (336, 7), (335, 5), (335, 7), (333, 7), (333, 6), (333, 5), (332, 6), (332, 5), (334, 6), (337, 7), (336, 8), (335, 8), (335, 9), (336, 9), (335, 10), (335, 11), (334, 10), (332, 10), (333, 11), (334, 12), (333, 12), (333, 13), (334, 11), (332, 12), (332, 13), (331, 14), (330, 14), (329, 11), (329, 10), (328, 14), (327, 13), (328, 13), (327, 12), (328, 11), (330, 10), (331, 11), (330, 9), (328, 9), (329, 9), (328, 8), (329, 7), (330, 7), (329, 6), (329, 5), (328, 6), (330, 5), (330, 6), (331, 6), (331, 7), (332, 7), (331, 5), (337, 9), (338, 10), (338, 11), (337, 11), (336, 11), (337, 10), (335, 12), (334, 13), (334, 14), (333, 15), (334, 15), (335, 15), (335, 14), (336, 13), (337, 13), (336, 12), (335, 13), (331, 13), (331, 28), (331, 29), (334, 26), (333, 26), (332, 25), (324, 34), (323, 35), (322, 34), (323, 33), (320, 33), (320, 32), (321, 33), (321, 30), (332, 29), (317, 22), (316, 21), (316, 20), (315, 20), (315, 21), (315, 22), (317, 21), (317, 20), (316, 19), (315, 19), (314, 18), (314, 19), (317, 19), (318, 19), (317, 18), (318, 17), (319, 18), (318, 18), (319, 17), (320, 18), (318, 21), (328, 33), (327, 33), (327, 34), (327, 35), (326, 36), (327, 37), (326, 38), (327, 39), (326, 39), (327, 38), (328, 37), (328, 36), (329, 36), (329, 35), (323, 34), (322, 33), (328, 34), (329, 34), (329, 38), (329, 37), (328, 38), (329, 39), (330, 40), (331, 40), (331, 41), (332, 41), (332, 42), (331, 43), (331, 42), (332, 43), (332, 44), (334, 41), (333, 41), (333, 42), (330, 42), (329, 41), (328, 40), (328, 39), (325, 38), (324, 39), (323, 39), (323, 40), (324, 41), (325, 42), (325, 43), (325, 45), (326, 45), (328, 44), (329, 43), (329, 44), (330, 44), (331, 44), (330, 43), (330, 41), (329, 42), (326, 42), (325, 41), (325, 40), (324, 40), (325, 39), (326, 40), (324, 42), (324, 43), (323, 44), (322, 43), (325, 46), (326, 46), (323, 43), (322, 42), (323, 42), (323, 41), (324, 44), (328, 43), (327, 41), (327, 36), (325, 37), (328, 35), (321, 35), (320, 34), (322, 35), (319, 32), (318, 31), (317, 31), (317, 32), (316, 33), (317, 33), (316, 32), (315, 32), (315, 30), (312, 37), (305, 36), (307, 40), (308, 41), (310, 40), (311, 41), (310, 42), (311, 43), (312, 44), (312, 43), (310, 44), (311, 45), (312, 46), (312, 47), (311, 47), (311, 46), (312, 45), (313, 45), (313, 44), (312, 42), (311, 42), (311, 39), (311, 40), (312, 41), (310, 43), (309, 44), (309, 43), (308, 43), (307, 43), (305, 43), (306, 45), (307, 46), (308, 46), (308, 45), (310, 45), (311, 44), (306, 42), (307, 41), (308, 40), (310, 41), (313, 41), (313, 43), (314, 44), (313, 42), (314, 45), (310, 46), (309, 46), (309, 45), (310, 47), (307, 45), (308, 44), (306, 46), (305, 47), (304, 48), (303, 47), (304, 47), (304, 46), (303, 46), (302, 46), (301, 46), (300, 46), (299, 46), (301, 47), (302, 45), (302, 44), (301, 44), (300, 43), (299, 38), (302, 39), (306, 22), (310, 19), (309, 18), (310, 17), (309, 16), (308, 15), (308, 14), (307, 14), (306, 14), (307, 13), (306, 12), (306, 11), (305, 12), (304, 11), (303, 10), (301, 11), (302, 12), (301, 13), (302, 13), (301, 14), (303, 12), (304, 12), (305, 13), (307, 15), (308, 13), (309, 14), (309, 15), (309, 17), (308, 16), (306, 16), (305, 15), (304, 15), (303, 16), (302, 15), (301, 15), (302, 16), (303, 17), (304, 18), (302, 18), (301, 19), (299, 17), (298, 16), (295, 12), (296, 11), (294, 10), (301, 10), (302, 5), (304, 6), (304, 7), (305, 7), (304, 8), (303, 11), (303, 13), (304, 13), (305, 10), (304, 10), (304, 9), (305, 8), (303, 5), (302, 11), (300, 10), (295, 10), (306, 6), (305, 5), (305, 9), (305, 11), (306, 9), (307, 10), (307, 9), (306, 10), (307, 11), (307, 12), (306, 15), (305, 14), (304, 16), (304, 17), (305, 18), (306, 18), (305, 17), (305, 20), (304, 19), (303, 18), (301, 17), (300, 19), (298, 19), (299, 18), (298, 18), (297, 18), (297, 19), (297, 20), (296, 20), (296, 19), (295, 19), (294, 18), (293, 18), (292, 18), (292, 19), (291, 18), (290, 18), (290, 19), (289, 19), (288, 19), (289, 18), (293, 17), (297, 17), (300, 15), (303, 14), (303, 15), (306, 13), (307, 8), (305, 6), (306, 5), (307, 5), (308, 6), (307, 7), (307, 6), (309, 7), (309, 6), (310, 7), (309, 5), (308, 7), (310, 8), (310, 9), (309, 9), (309, 10), (309, 11), (310, 12), (309, 12), (308, 11), (308, 12), (310, 15), (308, 17), (308, 18), (307, 16), (307, 17), (307, 18), (307, 19), (306, 17), (311, 19), (310, 18), (311, 16), (312, 17), (313, 18), (313, 17), (312, 22), (313, 22), (314, 21), (314, 20), (316, 22), (316, 23), (315, 24), (314, 24), (313, 23), (313, 25), (315, 23), (320, 17), (318, 16), (317, 16), (316, 17), (316, 16), (315, 15), (315, 14), (314, 14), (315, 13), (315, 12), (314, 15), (314, 13), (316, 12), (317, 11), (316, 10), (316, 11), (317, 12), (317, 13), (317, 14), (316, 14), (317, 15), (317, 17), (316, 18), (315, 17), (315, 16), (315, 18), (313, 21), (309, 19), (313, 24), (308, 24), (308, 19), (311, 29), (308, 28), (309, 33), (307, 29), (314, 27), (314, 22), (314, 23), (321, 29), (319, 34), (318, 34), (318, 35), (319, 35), (318, 33), (326, 34), (325, 34), (321, 34), (319, 33), (320, 35), (319, 36), (320, 37), (319, 38), (318, 37), (317, 37), (317, 38), (319, 37), (318, 38), (316, 36), (316, 35), (317, 36), (314, 38), (314, 39), (315, 38), (316, 38), (317, 34), (317, 35), (318, 36), (318, 32), (315, 34), (314, 25), (310, 16), (311, 15), (312, 16), (312, 15), (306, 20), (305, 16), (303, 20), (301, 20), (313, 14), (312, 14), (313, 15), (313, 16), (316, 15), (314, 16), (313, 13), (312, 13), (312, 12), (311, 12), (310, 11), (310, 10), (311, 9), (312, 9), (311, 10), (309, 13), (310, 14), (310, 13), (311, 13), (314, 17), (313, 12), (311, 14), (298, 36), (298, 35), (296, 40), (295, 40), (296, 39), (297, 41), (298, 41), (299, 42), (299, 44), (298, 45), (297, 46), (297, 47), (296, 47), (295, 47), (296, 48), (296, 46), (295, 46), (294, 47), (294, 46), (293, 45), (293, 44), (293, 43), (294, 42), (295, 43), (295, 42), (294, 41), (293, 40), (292, 40), (291, 39), (290, 38), (289, 37), (290, 37), (290, 36), (289, 35), (288, 34), (288, 33), (287, 33), (287, 34), (286, 35), (285, 35), (284, 35), (285, 34), (284, 33), (285, 32), (285, 33), (286, 32), (285, 31), (291, 38), (292, 39), (293, 41), (292, 42), (294, 44), (295, 44), (296, 42), (296, 43), (296, 44), (297, 44), (298, 43), (298, 42), (297, 43), (298, 44), (297, 45), (296, 45), (295, 45), (298, 46), (300, 47), (300, 48), (301, 49), (301, 50), (302, 49), (302, 50), (303, 50), (303, 51), (303, 49), (304, 52), (304, 51), (305, 51), (301, 51), (302, 51), (303, 52), (301, 52), (302, 52), (301, 53), (300, 53), (300, 54), (302, 53), (303, 53), (300, 51), (300, 52), (299, 50), (298, 50), (299, 49), (301, 48), (302, 48), (302, 47), (303, 45), (304, 45), (305, 45), (305, 46), (304, 49), (305, 50), (305, 49), (304, 50), (306, 48), (305, 48), (306, 50), (307, 49), (308, 48), (309, 47), (308, 47), (309, 48), (309, 49), (308, 50), (308, 51), (309, 51), (309, 52), (308, 53), (307, 53), (308, 52), (307, 51), (306, 51), (306, 52), (306, 49), (303, 48), (301, 45), (299, 45), (300, 45), (297, 42), (296, 41), (295, 41), (302, 43), (303, 43), (304, 44), (305, 41), (305, 40), (305, 39), (304, 40), (304, 39), (306, 39), (303, 41), (308, 39), (307, 38), (312, 40), (313, 46), (315, 44), (316, 45), (316, 44), (315, 45), (314, 46), (315, 43), (316, 43), (315, 42), (316, 41), (316, 42), (317, 41), (318, 41), (319, 41), (319, 42), (320, 41), (319, 40), (320, 39), (319, 39), (320, 38), (316, 39), (317, 39), (321, 37), (322, 38), (322, 36), (321, 36), (320, 36), (311, 37), (294, 27), (293, 35), (292, 35), (293, 36), (292, 37), (292, 38), (291, 40), (290, 41), (289, 41), (289, 40), (289, 39), (289, 38), (288, 38), (290, 39), (291, 36), (291, 37), (293, 37), (293, 38), (291, 34), (291, 35), (294, 35), (294, 34), (292, 29), (297, 35), (296, 37), (297, 32), (295, 36), (302, 40), (295, 39), (294, 40), (293, 39), (294, 38), (288, 36), (287, 37), (286, 38), (285, 39), (284, 38), (283, 37), (282, 37), (281, 37), (281, 36), (282, 35), (281, 34), (280, 34), (279, 33), (278, 33), (279, 35), (280, 35), (278, 32), (278, 31), (277, 33), (277, 32), (276, 32), (276, 33), (275, 34), (275, 33), (276, 35), (277, 34), (276, 34), (277, 35), (276, 36), (276, 37), (277, 37), (278, 36), (279, 36), (280, 36), (279, 37), (278, 34), (280, 37), (280, 38), (281, 38), (282, 39), (283, 39), (284, 39), (284, 40), (284, 41), (283, 41), (283, 42), (284, 42), (282, 41), (281, 42), (280, 43), (281, 43), (279, 42), (278, 41), (277, 42), (276, 42), (275, 41), (274, 42), (275, 42), (276, 41), (276, 43), (277, 44), (276, 45), (275, 44), (274, 45), (275, 46), (277, 46), (278, 46), (277, 47), (278, 48), (279, 48), (278, 47), (277, 45), (276, 44), (275, 43), (277, 43), (278, 45), (279, 46), (280, 48), (279, 49), (281, 48), (280, 49), (279, 50), (280, 51), (279, 52), (278, 52), (279, 51), (280, 50), (280, 52), (279, 53), (279, 54), (278, 54), (277, 52), (278, 53), (277, 53), (277, 51), (276, 52), (275, 52), (276, 53), (275, 54), (275, 55), (274, 56), (273, 56), (274, 57), (275, 57), (276, 58), (277, 58), (278, 57), (278, 56), (277, 56), (277, 57), (278, 58), (279, 59), (279, 57), (280, 56), (280, 55), (279, 56), (278, 55), (277, 54), (276, 55), (275, 56), (274, 54), (273, 54), (272, 55), (273, 55), (273, 57), (272, 57), (272, 58), (273, 59), (272, 59), (272, 60), (274, 60), (275, 59), (275, 58), (273, 58), (274, 55), (276, 59), (275, 60), (276, 61), (275, 61), (276, 60), (277, 62), (278, 63), (279, 63), (279, 64), (280, 64), (281, 63), (280, 63), (278, 64), (278, 65), (279, 66), (280, 66), (280, 67), (279, 67), (279, 68), (280, 68), (280, 69), (280, 70), (281, 69), (281, 71), (280, 72), (280, 73), (279, 71), (281, 72), (280, 71), (279, 72), (278, 72), (279, 73), (279, 74), (280, 74), (279, 75), (278, 76), (277, 76), (277, 77), (278, 78), (278, 79), (279, 78), (278, 80), (277, 80), (277, 79), (276, 80), (275, 79), (274, 79), (274, 78), (274, 80), (273, 79), (272, 79), (272, 78), (272, 80), (273, 80), (272, 81), (273, 81), (274, 82), (275, 81), (274, 81), (273, 82), (274, 83), (273, 84), (272, 83), (274, 85), (275, 84), (276, 84), (277, 84), (278, 85), (279, 86), (278, 86), (279, 85), (280, 84), (281, 84), (282, 84), (283, 84), (281, 85), (279, 84), (280, 83), (279, 83), (278, 83), (278, 82), (277, 83), (276, 82), (276, 83), (277, 85), (276, 86), (277, 86), (278, 84), (280, 82), (281, 83), (282, 83), (283, 82), (280, 85), (281, 82), (281, 81), (280, 81), (279, 82), (278, 87), (279, 88), (278, 89), (277, 90), (278, 91), (278, 90), (277, 89), (276, 88), (275, 87), (275, 86), (274, 87), (273, 88), (272, 89), (273, 89), (273, 90), (272, 91), (272, 92), (273, 92), (273, 93), (274, 94), (275, 94), (276, 95), (276, 94), (275, 95), (276, 96), (277, 95), (275, 96), (274, 95), (274, 97), (273, 98), (274, 99), (275, 98), (275, 97), (276, 93), (275, 92), (276, 99), (275, 99), (276, 98), (274, 98), (273, 99), (273, 100), (272, 101), (272, 100), (273, 101), (274, 101), (274, 100), (275, 101), (275, 102), (274, 102), (275, 103), (276, 104), (277, 103), (278, 102), (279, 102), (279, 101), (279, 100), (278, 101), (278, 100), (280, 102), (280, 101), (281, 102), (281, 103), (281, 104), (280, 103), (279, 103), (278, 103), (279, 104), (280, 104), (282, 104), (283, 105), (284, 105), (285, 104), (284, 104), (284, 103), (285, 102), (286, 101), (287, 102), (288, 103), (287, 104), (286, 105), (286, 104), (286, 103), (284, 101), (283, 102), (283, 103), (284, 102), (285, 101), (286, 100), (285, 99), (286, 99), (287, 99), (288, 98), (287, 97), (286, 97), (287, 98), (286, 98), (285, 98), (284, 99), (283, 99), (284, 98), (283, 97), (282, 97), (281, 97), (283, 98), (284, 97), (285, 96), (285, 95), (284, 96), (285, 97), (284, 95), (285, 94), (286, 93), (287, 94), (287, 93), (287, 92), (285, 92), (285, 91), (286, 90), (285, 90), (286, 89), (284, 89), (285, 89), (284, 91), (284, 92), (283, 92), (284, 93), (282, 91), (281, 92), (281, 93), (282, 94), (281, 95), (281, 94), (280, 94), (279, 93), (278, 92), (279, 92), (278, 93), (278, 94), (277, 93), (279, 94), (280, 93), (280, 92), (280, 91), (278, 95), (279, 95), (278, 96), (279, 97), (278, 98), (280, 98), (279, 96), (280, 96), (281, 96), (280, 95), (280, 97), (281, 98), (280, 99), (281, 101), (282, 101), (282, 102), (283, 100), (283, 101), (282, 100), (285, 100), (282, 98), (282, 96), (277, 100), (277, 101), (277, 102), (276, 103), (276, 105), (275, 104), (275, 105), (274, 104), (274, 105), (273, 105), (274, 106), (274, 107), (273, 106), (272, 107), (272, 108), (273, 108), (274, 108), (274, 109), (273, 110), (274, 111), (273, 111), (272, 110), (274, 112), (273, 112), (274, 113), (275, 112), (276, 112), (277, 111), (276, 111), (275, 113), (277, 113), (277, 114), (278, 115), (279, 116), (278, 117), (279, 115), (278, 114), (280, 116), (281, 117), (281, 116), (282, 117), (283, 117), (283, 118), (283, 119), (284, 120), (283, 120), (282, 121), (282, 122), (281, 122), (280, 121), (281, 120), (281, 121), (280, 120), (280, 119), (279, 119), (279, 118), (279, 117), (280, 118), (281, 118), (280, 117), (281, 115), (278, 116), (277, 115), (281, 114), (281, 113), (280, 113), (279, 114), (279, 113), (280, 114), (280, 115), (278, 113), (276, 113), (276, 114), (276, 115), (275, 115), (277, 116), (275, 114), (274, 115), (274, 116), (276, 116), (277, 117), (276, 118), (275, 119), (274, 118), (274, 119), (274, 120), (273, 119), (275, 120), (273, 120), (274, 121), (274, 122), (273, 121), (273, 118), (272, 121), (275, 123), (276, 124), (277, 124), (276, 125), (275, 125), (274, 124), (273, 123), (272, 124), (272, 125), (273, 124), (275, 121), (276, 119), (277, 119), (278, 119), (277, 120), (278, 120), (279, 121), (278, 122), (278, 123), (277, 122), (277, 121), (278, 118), (277, 118), (279, 120), (282, 120), (283, 121), (284, 121), (285, 120), (283, 122), (284, 123), (284, 122), (285, 121), (286, 120), (284, 119), (284, 118), (286, 121), (286, 122), (285, 122), (286, 123), (285, 123), (286, 124), (285, 124), (284, 124), (285, 118), (287, 122), (283, 124), (283, 123), (282, 119), (280, 122), (281, 123), (286, 119), (287, 119), (287, 118), (288, 117), (287, 116), (288, 116), (287, 117), (286, 118), (288, 118), (288, 119), (286, 117), (286, 116), (287, 115), (288, 115), (289, 117), (289, 116), (287, 114), (285, 116), (285, 115), (284, 115), (284, 116), (290, 116), (291, 116), (292, 115), (291, 115), (290, 114), (289, 114), (291, 117), (292, 117), (292, 116), (289, 115), (291, 114), (293, 116), (293, 117), (293, 118), (292, 119), (291, 119), (292, 118), (291, 118), (290, 118), (290, 119), (289, 119), (294, 118), (294, 117), (295, 118), (296, 118), (297, 118), (298, 119), (297, 119), (296, 120), (296, 121), (296, 122), (297, 122), (296, 123), (298, 122), (298, 123), (298, 124), (299, 123), (299, 122), (299, 121), (298, 121), (297, 121), (298, 120), (299, 120), (297, 120), (299, 119), (300, 118), (299, 118), (298, 117), (297, 116), (297, 117), (296, 117), (297, 124), (296, 125), (296, 124), (295, 124), (294, 125), (295, 125), (295, 126), (296, 126), (294, 124), (295, 123), (295, 122), (294, 121), (300, 120), (298, 118), (298, 116), (298, 115), (297, 114), (297, 113), (298, 112), (299, 111), (299, 110), (298, 111), (297, 111), (298, 110), (300, 112), (301, 112), (302, 111), (302, 113), (301, 113), (300, 113), (300, 114), (299, 115), (300, 116), (300, 117), (297, 123), (300, 124), (301, 124), (302, 123), (302, 122), (301, 122), (301, 123), (300, 125), (300, 123), (301, 121), (302, 124), (302, 125), (303, 125), (302, 126), (303, 127), (304, 126), (305, 125), (304, 125), (303, 124), (303, 123), (302, 121), (303, 122), (303, 121), (300, 122), (294, 122), (295, 121), (293, 122), (292, 121), (291, 121), (292, 120), (293, 119), (294, 120), (302, 120), (302, 119), (303, 120), (304, 119), (303, 118), (304, 118), (305, 119), (306, 119), (307, 119), (308, 119), (307, 118), (306, 118), (307, 117), (308, 116), (309, 115), (309, 116), (308, 117), (308, 118), (309, 119), (310, 118), (311, 117), (309, 117), (309, 118), (308, 120), (308, 121), (308, 122), (307, 121), (307, 122), (308, 123), (309, 123), (310, 122), (309, 122), (309, 124), (309, 125), (310, 126), (309, 127), (309, 128), (308, 129), (308, 128), (307, 128), (308, 126), (309, 126), (308, 124), (309, 120), (310, 119), (311, 120), (310, 120), (311, 118), (312, 118), (313, 119), (314, 118), (315, 117), (316, 116), (316, 117), (317, 118), (318, 119), (318, 120), (319, 121), (319, 122), (320, 121), (319, 120), (318, 122), (317, 122), (316, 121), (318, 123), (319, 124), (319, 123), (318, 124), (317, 124), (316, 124), (316, 123), (315, 122), (315, 123), (316, 122), (317, 123), (314, 123), (313, 124), (314, 124), (315, 124), (318, 121), (317, 121), (321, 122), (320, 122), (320, 123), (315, 121), (314, 121), (313, 122), (314, 122), (312, 125), (313, 126), (312, 127), (313, 127), (312, 128), (313, 128), (312, 126), (314, 128), (314, 127), (315, 126), (316, 127), (315, 128), (311, 129), (312, 130), (312, 131), (313, 131), (314, 131), (311, 130), (312, 129), (313, 129), (314, 130), (313, 130), (311, 128), (310, 128), (307, 125), (306, 126), (307, 126), (307, 127), (309, 129), (309, 130), (310, 130), (310, 131), (309, 131), (310, 132), (309, 133), (309, 132), (308, 132), (307, 132), (307, 133), (308, 133), (308, 134), (309, 134), (310, 133), (310, 134), (311, 133), (311, 132), (314, 129), (315, 130), (315, 131), (315, 129), (316, 130), (315, 125), (316, 126), (317, 125), (318, 125), (320, 125), (320, 124), (319, 125), (317, 126), (316, 129), (317, 129), (318, 126), (316, 125), (314, 126), (313, 123), (312, 124), (314, 125), (313, 125), (313, 121), (312, 120), (312, 119), (316, 120), (316, 119), (316, 118), (317, 117), (317, 116), (315, 116), (314, 116), (314, 117), (313, 118), (313, 116), (312, 117), (313, 117), (312, 115), (312, 114), (311, 115), (311, 114), (312, 113), (311, 113), (311, 116), (312, 116), (313, 115), (311, 119), (314, 120), (314, 119), (315, 119), (315, 118), (319, 118), (319, 117), (320, 119), (320, 120), (321, 120), (319, 119), (317, 120), (317, 119), (318, 118), (318, 116), (317, 115), (318, 115), (316, 115), (317, 114), (319, 114), (319, 115), (320, 114), (320, 115), (321, 115), (321, 116), (322, 116), (323, 117), (324, 116), (324, 115), (324, 114), (325, 115), (323, 118), (322, 118), (321, 117), (320, 117), (318, 117), (320, 118), (321, 118), (321, 119), (322, 121), (323, 121), (324, 120), (324, 121), (325, 122), (325, 123), (326, 122), (326, 121), (327, 122), (326, 123), (327, 123), (327, 124), (326, 124), (325, 124), (324, 125), (323, 124), (324, 123), (325, 119), (324, 118), (324, 117), (322, 117), (322, 115), (320, 116), (307, 120), (306, 121), (306, 122), (311, 121), (311, 122), (312, 121), (312, 123), (312, 122), (311, 123), (319, 126), (318, 127), (317, 128), (318, 129), (318, 130), (317, 131), (316, 128), (318, 131), (316, 132), (315, 133), (316, 133), (316, 134), (315, 132), (314, 132), (313, 132), (317, 134), (318, 134), (317, 133), (318, 132), (318, 133), (319, 133), (317, 132), (319, 134), (320, 134), (320, 133), (319, 132), (321, 133), (322, 132), (323, 131), (324, 130), (324, 131), (321, 132), (322, 133), (321, 131), (323, 132), (324, 133), (324, 132), (323, 133), (323, 134), (322, 131), (320, 132), (321, 130), (320, 130), (320, 129), (319, 130), (319, 129), (320, 128), (318, 128), (319, 127), (320, 126), (321, 126), (321, 127), (320, 127), (321, 128), (321, 129), (322, 128), (323, 127), (323, 128), (324, 129), (325, 130), (325, 129), (326, 128), (325, 128), (326, 127), (325, 127), (324, 127), (324, 128), (326, 126), (325, 125), (327, 127), (328, 127), (329, 126), (328, 125), (327, 126), (328, 128), (329, 129), (330, 129), (331, 128), (331, 127), (330, 126), (332, 129), (331, 129), (330, 128), (332, 128), (332, 127), (333, 126), (334, 125), (334, 127), (335, 127), (334, 126), (336, 128), (337, 129), (335, 128), (334, 128), (336, 127), (337, 128), (336, 129), (335, 130), (336, 131), (337, 132), (338, 133), (337, 134), (338, 134), (339, 133), (339, 132), (338, 132), (338, 131), (339, 131), (339, 134), (340, 133), (341, 134), (342, 133), (341, 132), (342, 131), (342, 130), (341, 130), (343, 129), (344, 130), (345, 131), (344, 131), (345, 130), (346, 129), (345, 129), (345, 128), (345, 127), (346, 127), (346, 128), (344, 128), (344, 127), (343, 126), (342, 125), (342, 126), (342, 127), (341, 126), (341, 127), (340, 128), (340, 129), (339, 128), (338, 127), (338, 126), (337, 127), (339, 127), (338, 128), (334, 129), (333, 128), (333, 127), (333, 130), (333, 129), (334, 131), (333, 131), (334, 132), (333, 132), (334, 133), (335, 133), (335, 132), (333, 133), (334, 134), (333, 134), (335, 134), (336, 133), (339, 130), (340, 130), (340, 132), (340, 131), (337, 133), (336, 134), (336, 132), (335, 131), (334, 130), (332, 131), (331, 132), (330, 133), (329, 134), (328, 133), (327, 134), (330, 134), (329, 133), (328, 132), (329, 132), (329, 131), (330, 131), (329, 130), (329, 128), (328, 129), (327, 130), (326, 130), (325, 131), (326, 125), (325, 126), (324, 126), (323, 126), (322, 126), (322, 127), (322, 125), (323, 125), (324, 124), (326, 131), (326, 129), (327, 128), (327, 129), (328, 130), (329, 127), (330, 127), (331, 126), (332, 130), (331, 133), (332, 132), (332, 134), (337, 131), (338, 130), (338, 129), (337, 130), (336, 130), (337, 126), (339, 125), (338, 124), (339, 124), (340, 126), (337, 123), (339, 123), (340, 123), (341, 124), (341, 123), (342, 123), (341, 122), (340, 121), (341, 120), (342, 121), (343, 122), (342, 122), (341, 121), (340, 124), (338, 125), (339, 126), (340, 125), (333, 125), (332, 124), (332, 123), (331, 122), (332, 121), (331, 120), (330, 119), (330, 118), (331, 117), (332, 118), (331, 119), (332, 120), (332, 119), (330, 121), (331, 121), (333, 119), (334, 119), (333, 120), (330, 120), (333, 122), (333, 121), (334, 120), (335, 121), (336, 120), (336, 119), (335, 119), (335, 120), (334, 121), (335, 122), (336, 123), (337, 122), (338, 122), (338, 121), (338, 120), (339, 119), (339, 120), (339, 121), (337, 120), (340, 118), (339, 117), (340, 116), (339, 116), (339, 115), (338, 116), (340, 115), (339, 114), (340, 114), (338, 117), (337, 118), (336, 117), (335, 116), (336, 116), (336, 115), (336, 114), (337, 113), (337, 114), (338, 114), (338, 113), (338, 112), (338, 111), (337, 112), (339, 112), (339, 113), (336, 113), (337, 111), (336, 112), (336, 111), (336, 110), (335, 111), (335, 109), (334, 112), (333, 111), (333, 110), (334, 109), (334, 108), (333, 107), (334, 107), (332, 107), (333, 108), (332, 108), (333, 109), (332, 110), (332, 109), (331, 110), (331, 111), (331, 112), (330, 113), (330, 114), (329, 115), (330, 115), (330, 116), (333, 118), (329, 119), (328, 119), (329, 118), (329, 117), (328, 118), (327, 118), (327, 117), (328, 116), (329, 116), (328, 115), (327, 116), (326, 115), (326, 116), (326, 117), (326, 114), (325, 114), (325, 116), (327, 115), (327, 114), (323, 114), (323, 113), (324, 112), (323, 111), (322, 111), (323, 110), (323, 109), (324, 110), (322, 110), (321, 111), (320, 112), (321, 113), (320, 113), (319, 113), (321, 114), (322, 114), (323, 115), (326, 113), (325, 112), (326, 112), (325, 113), (322, 113), (323, 112), (324, 111), (325, 111), (325, 110), (325, 109), (325, 108), (324, 108), (322, 109), (320, 111), (320, 110), (320, 109), (319, 110), (318, 109), (318, 108), (319, 109), (318, 111), (319, 111), (321, 109), (321, 108), (320, 108), (319, 108), (317, 109), (318, 110), (317, 112), (318, 113), (318, 112), (321, 110), (322, 108), (323, 108), (324, 109), (322, 107), (321, 106), (321, 107), (322, 106), (322, 105), (321, 105), (320, 105), (319, 104), (318, 103), (317, 103), (316, 104), (317, 105), (316, 106), (318, 105), (319, 105), (319, 106), (320, 107), (317, 106), (316, 105), (315, 106), (314, 105), (314, 106), (313, 105), (312, 104), (313, 103), (312, 102), (312, 101), (311, 100), (312, 99), (312, 98), (311, 99), (313, 99), (313, 97), (314, 98), (314, 97), (315, 96), (316, 95), (316, 94), (317, 93), (318, 93), (317, 94), (318, 94), (319, 93), (319, 94), (317, 92), (318, 91), (316, 91), (315, 91), (314, 91), (315, 90), (316, 90), (315, 92), (315, 93), (315, 94), (316, 93), (314, 94), (315, 95), (316, 92), (317, 91), (315, 89), (314, 89), (314, 88), (313, 87), (314, 86), (314, 87), (315, 87), (316, 87), (317, 88), (313, 85), (312, 84), (313, 83), (314, 82), (315, 82), (314, 81), (315, 81), (316, 81), (316, 83), (317, 83), (318, 84), (316, 84), (315, 84), (316, 85), (315, 85), (315, 86), (314, 85), (313, 84), (316, 86), (317, 85), (313, 81), (312, 80), (313, 79), (313, 80), (312, 79), (312, 81), (311, 80), (311, 79), (312, 78), (313, 78), (314, 78), (315, 79), (314, 79), (315, 78), (315, 77), (316, 78), (316, 79), (316, 80), (314, 80), (312, 82), (311, 81), (310, 80), (310, 79), (314, 83), (312, 85), (311, 85), (311, 84), (311, 83), (310, 84), (309, 85), (308, 85), (309, 84), (308, 83), (308, 82), (307, 82), (306, 81), (305, 82), (307, 81), (306, 80), (306, 79), (305, 80), (305, 79), (306, 78), (304, 80), (303, 80), (302, 80), (301, 80), (302, 79), (303, 79), (304, 78), (305, 78), (302, 78), (303, 78), (301, 77), (302, 77), (303, 76), (302, 76), (301, 76), (301, 75), (300, 78), (301, 78), (300, 79), (299, 79), (299, 78), (298, 77), (299, 76), (300, 77), (300, 76), (302, 75), (301, 74), (303, 75), (302, 74), (302, 73), (301, 73), (300, 74), (299, 73), (298, 72), (298, 71), (297, 70), (298, 69), (297, 69), (296, 68), (295, 67), (296, 66), (297, 65), (295, 65), (295, 64), (296, 64), (297, 63), (296, 63), (297, 64), (296, 65), (295, 66), (294, 68), (293, 69), (293, 68), (293, 70), (292, 68), (292, 67), (291, 66), (292, 65), (292, 66), (293, 65), (291, 67), (294, 69), (295, 70), (296, 70), (297, 68), (296, 67), (294, 66), (294, 67), (293, 66), (291, 65), (291, 69), (290, 70), (290, 71), (289, 71), (291, 72), (291, 71), (291, 70), (292, 71), (293, 71), (294, 72), (295, 71), (294, 70), (296, 72), (295, 72), (296, 71), (295, 69), (296, 69), (297, 67), (298, 68), (297, 66), (294, 65), (291, 64), (290, 65), (289, 64), (289, 65), (288, 64), (289, 63), (290, 64), (292, 64), (294, 64), (298, 65), (298, 66), (299, 65), (300, 64), (299, 63), (298, 64), (296, 62), (298, 62), (298, 63), (297, 62), (296, 61), (297, 61), (298, 61), (299, 62), (300, 63), (301, 63), (301, 64), (302, 65), (303, 64), (302, 63), (302, 64), (302, 66), (301, 66), (303, 66), (301, 67), (300, 68), (299, 68), (301, 68), (300, 69), (300, 67), (301, 69), (300, 70), (299, 69), (302, 70), (302, 69), (299, 67), (298, 67), (299, 66), (299, 70), (302, 67), (301, 70), (303, 69), (303, 68), (303, 67), (304, 66), (305, 67), (300, 65), (301, 65), (304, 69), (305, 69), (304, 70), (302, 68), (300, 66), (304, 67), (304, 68), (303, 70), (302, 71), (301, 71), (303, 71), (304, 71), (300, 71), (305, 70), (306, 69), (307, 68), (306, 67), (307, 66), (308, 67), (309, 68), (309, 67), (310, 68), (310, 69), (309, 70), (308, 71), (307, 71), (309, 72), (307, 72), (307, 73), (308, 74), (308, 75), (307, 76), (308, 76), (307, 77), (309, 75), (309, 74), (310, 73), (308, 73), (307, 74), (306, 73), (306, 74), (305, 75), (305, 76), (305, 77), (306, 77), (304, 75), (304, 74), (303, 73), (303, 74), (304, 73), (302, 72), (300, 75), (299, 74), (298, 73), (299, 75), (298, 75), (297, 75), (297, 74), (298, 74), (297, 73), (295, 73), (296, 73), (295, 74), (294, 75), (293, 76), (293, 75), (293, 74), (294, 76), (295, 75), (295, 76), (296, 76), (296, 77), (295, 77), (296, 78), (295, 78), (296, 79), (297, 80), (297, 79), (298, 78), (297, 78), (296, 80), (294, 78), (294, 77), (300, 72), (301, 72), (300, 73), (299, 71), (298, 70), (303, 72), (304, 76), (304, 77), (303, 77), (305, 74), (305, 73), (306, 72), (305, 72), (304, 72), (299, 64), (301, 62), (300, 61), (301, 61), (302, 60), (302, 61), (301, 59), (300, 60), (299, 60), (300, 59), (301, 60), (299, 61), (301, 58), (302, 59), (300, 62), (295, 63), (298, 79), (298, 80), (298, 81), (298, 82), (299, 81), (300, 81), (300, 80), (299, 80), (302, 81), (303, 82), (304, 82), (304, 81), (305, 81), (307, 79), (308, 79), (309, 79), (308, 80), (309, 83), (310, 83), (310, 86), (311, 82), (315, 83), (314, 84), (313, 86), (310, 87), (310, 85), (308, 86), (309, 86), (308, 84), (309, 82), (310, 82), (313, 82), (312, 83), (310, 81), (309, 81), (307, 84), (307, 85), (309, 80), (311, 78), (310, 78), (309, 77), (310, 76), (310, 77), (308, 78), (307, 78), (306, 76), (309, 78), (317, 82), (318, 82), (319, 82), (320, 81), (320, 82), (319, 81), (318, 81), (317, 81), (315, 80), (316, 77), (317, 76), (317, 77), (318, 78), (319, 78), (318, 79), (317, 78), (315, 76), (316, 76), (318, 77), (318, 76), (318, 75), (317, 74), (316, 74), (315, 74), (315, 73), (314, 74), (316, 72), (317, 72), (316, 71), (317, 73), (316, 73), (315, 72), (318, 74), (315, 75), (319, 75), (320, 75), (317, 75), (316, 75), (318, 71), (317, 71), (316, 70), (315, 71), (317, 70), (317, 69), (318, 69), (317, 68), (318, 68), (319, 69), (319, 70), (319, 72), (318, 73), (319, 73), (318, 72), (320, 74), (320, 73), (321, 74), (321, 73), (320, 72), (321, 75), (320, 76), (321, 76), (322, 75), (322, 76), (323, 76), (324, 76), (325, 77), (324, 78), (323, 79), (323, 80), (324, 80), (325, 81), (326, 80), (326, 79), (327, 79), (326, 81), (327, 80), (328, 79), (328, 78), (327, 78), (327, 77), (326, 76), (325, 75), (324, 74), (324, 73), (325, 72), (326, 72), (326, 73), (325, 73), (326, 74), (326, 77), (327, 76), (326, 75), (325, 74), (326, 78), (328, 77), (329, 77), (328, 76), (329, 75), (328, 74), (328, 73), (327, 73), (327, 72), (326, 71), (325, 70), (326, 70), (327, 71), (328, 72), (329, 71), (330, 72), (331, 72), (330, 71), (331, 70), (331, 71), (329, 72), (328, 71), (328, 70), (327, 70), (327, 69), (326, 69), (327, 68), (328, 69), (328, 68), (329, 67), (329, 69), (329, 68), (330, 69), (329, 70), (326, 68), (325, 68), (324, 68), (325, 69), (324, 70), (323, 69), (322, 70), (323, 70), (322, 71), (321, 72), (320, 71), (319, 71), (320, 70), (321, 69), (322, 68), (322, 67), (321, 66), (322, 62), (322, 61), (323, 60), (323, 61), (322, 60), (326, 64), (326, 60), (328, 59), (328, 60), (326, 61), (327, 61), (329, 60), (331, 60), (330, 60), (331, 61), (331, 62), (332, 62), (332, 61), (333, 62), (334, 62), (334, 63), (335, 63), (335, 64), (336, 65), (336, 66), (337, 67), (336, 68), (335, 67), (335, 65), (334, 66), (334, 67), (334, 68), (334, 69), (333, 70), (333, 69), (332, 70), (333, 68), (334, 70), (333, 71), (332, 69), (331, 69), (331, 68), (330, 67), (331, 66), (331, 65), (332, 64), (332, 65), (332, 66), (331, 64), (330, 65), (331, 67), (330, 68), (330, 70), (332, 71), (334, 71), (332, 72), (332, 73), (331, 74), (331, 73), (332, 74), (333, 75), (332, 76), (331, 77), (331, 76), (330, 75), (329, 74), (324, 71), (324, 72), (323, 74), (324, 75), (327, 74), (325, 76), (324, 77), (323, 75), (322, 74), (321, 71), (322, 73), (323, 73), (323, 72), (321, 70), (322, 69), (323, 68), (321, 68), (321, 67), (320, 66), (320, 67), (320, 68), (319, 67), (318, 66), (317, 66), (317, 65), (316, 65), (315, 64), (314, 63), (315, 62), (316, 62), (317, 63), (316, 63), (314, 61), (313, 62), (314, 60), (315, 60), (315, 59), (314, 59), (313, 59), (313, 60), (313, 58), (312, 58), (311, 57), (312, 56), (313, 56), (311, 56), (310, 57), (309, 57), (310, 56), (309, 55), (309, 56), (309, 54), (308, 54), (307, 52), (309, 53), (307, 55), (308, 55), (308, 56), (306, 54), (305, 53), (304, 53), (304, 54), (303, 54), (302, 54), (305, 54), (306, 53), (309, 50), (310, 49), (310, 48), (311, 49), (311, 48), (310, 50), (311, 50), (308, 49), (305, 52), (300, 49), (298, 48), (299, 47), (298, 47), (297, 48), (295, 48), (293, 46), (293, 47), (293, 48), (293, 49), (294, 48), (295, 49), (292, 46), (294, 45), (292, 49), (294, 49), (294, 39), (294, 36), (292, 36), (294, 37), (295, 38), (306, 24), (302, 17), (302, 14), (301, 12), (334, 23), (335, 24), (335, 23), (336, 23), (336, 22), (335, 20), (336, 19), (335, 19), (337, 19), (337, 18), (338, 19), (338, 18), (339, 18), (340, 18), (339, 17), (340, 17), (341, 18), (342, 17), (342, 19), (343, 18), (344, 19), (344, 20), (345, 20), (345, 19), (345, 18), (343, 20), (341, 19), (340, 20), (341, 21), (341, 20), (342, 21), (341, 22), (341, 23), (341, 24), (342, 24), (342, 25), (343, 24), (344, 23), (343, 22), (343, 23), (343, 25), (342, 27), (343, 27), (341, 27), (341, 28), (339, 32), (338, 33), (337, 32), (338, 32), (339, 33), (337, 33), (336, 34), (335, 34), (336, 33), (337, 34), (338, 35), (338, 34), (339, 34), (340, 34), (340, 32), (338, 26), (344, 24), (346, 20), (344, 21), (342, 20), (340, 22), (339, 23), (339, 20), (340, 21), (339, 21), (345, 17), (344, 18), (344, 17), (346, 18), (342, 23), (342, 22), (340, 24), (339, 24), (338, 24), (344, 25), (343, 19), (342, 18), (341, 17), (342, 16), (343, 16), (344, 15), (343, 14), (343, 15), (344, 14), (343, 13), (342, 12), (341, 12), (342, 11), (340, 11), (339, 10), (339, 11), (339, 12), (340, 13), (341, 13), (340, 12), (340, 9), (339, 9), (339, 8), (340, 8), (337, 12), (338, 12), (341, 8), (341, 9), (342, 10), (343, 10), (344, 11), (345, 10), (343, 11), (344, 10), (345, 11), (346, 11), (347, 11), (346, 12), (347, 10), (348, 11), (349, 12), (350, 12), (350, 13), (349, 14), (350, 14), (350, 15), (349, 16), (350, 16), (349, 15), (351, 13), (351, 12), (352, 12), (351, 11), (352, 13), (353, 12), (354, 13), (354, 12), (353, 13), (354, 14), (354, 11), (355, 12), (353, 10), (352, 11), (352, 10), (353, 9), (352, 8), (353, 8), (354, 7), (353, 7), (354, 8), (350, 11), (349, 13), (348, 13), (347, 14), (347, 15), (348, 15), (351, 14), (351, 16), (351, 17), (352, 16), (353, 15), (354, 16), (353, 16), (353, 17), (353, 18), (354, 19), (352, 18), (353, 19), (354, 20), (355, 21), (356, 22), (355, 22), (356, 21), (357, 21), (357, 22), (356, 23), (357, 23), (356, 24), (357, 24), (355, 23), (355, 24), (354, 24), (353, 24), (353, 23), (352, 24), (352, 23), (351, 22), (350, 23), (351, 23), (353, 25), (354, 25), (352, 25), (352, 26), (354, 23), (358, 25), (358, 26), (357, 27), (356, 26), (355, 26), (354, 28), (355, 28), (355, 27), (355, 29), (352, 30), (350, 26), (351, 25), (347, 27), (348, 25), (344, 22), (345, 21), (343, 21), (340, 19), (339, 16), (338, 17), (337, 17), (336, 18), (336, 17), (336, 25), (336, 26), (337, 26), (336, 27), (337, 27), (338, 28), (339, 25), (346, 19), (347, 18), (346, 17), (346, 16), (347, 17), (345, 15), (346, 14), (347, 16), (343, 17), (341, 16), (340, 15), (341, 14), (342, 14), (342, 13), (342, 15), (340, 10), (341, 10), (339, 7), (338, 8), (338, 7), (337, 6), (337, 8), (341, 7), (341, 6), (340, 7), (336, 10), (334, 9), (332, 8), (334, 8), (331, 8), (331, 9), (330, 8), (329, 8), (328, 10), (327, 10), (326, 9), (327, 9), (326, 10), (326, 11), (326, 12), (325, 13), (325, 12), (327, 11), (327, 8), (325, 8), (325, 9), (326, 8), (325, 7), (326, 7), (325, 6), (326, 6), (324, 6), (324, 5), (323, 5), (323, 6), (322, 5), (322, 6), (321, 7), (321, 6), (320, 5), (321, 5), (322, 7), (323, 8), (324, 9), (325, 10), (327, 7), (328, 5), (327, 6), (327, 5), (328, 7), (323, 7), (322, 8), (322, 9), (323, 9), (324, 10), (324, 11), (323, 10), (323, 11), (323, 12), (324, 13), (324, 12), (322, 13), (321, 13), (321, 14), (320, 14), (319, 14), (318, 14), (318, 13), (322, 14), (322, 15), (321, 15), (319, 13), (319, 12), (318, 12), (315, 10), (316, 9), (315, 9), (314, 9), (315, 11), (317, 10), (318, 10), (319, 9), (320, 8), (320, 9), (321, 9), (321, 8), (320, 7), (319, 6), (319, 7), (319, 8), (318, 7), (317, 6), (316, 6), (317, 7), (316, 7), (316, 8), (315, 8), (314, 8), (314, 10), (314, 11), (317, 9), (318, 8), (318, 9), (320, 15), (320, 16), (319, 15), (318, 11), (317, 8), (312, 11), (311, 11), (304, 14), (301, 16), (294, 24), (290, 28), (287, 27), (285, 28), (284, 27), (283, 27), (285, 29), (284, 30), (284, 31), (283, 29), (284, 29), (288, 31), (287, 32), (286, 33), (284, 32), (286, 34), (286, 36), (286, 37), (285, 36), (285, 38), (285, 37), (286, 39), (283, 38), (282, 38), (281, 39), (280, 40), (280, 39), (279, 38), (278, 39), (278, 40), (277, 41), (273, 43), (274, 43), (279, 40), (279, 39), (281, 40), (281, 41), (280, 42), (282, 42), (283, 43), (282, 44), (281, 44), (279, 44), (279, 45), (278, 44), (278, 43), (279, 47), (280, 46), (278, 49), (277, 48), (277, 49), (280, 47), (281, 45), (282, 46), (281, 46), (282, 45), (280, 41), (279, 41), (278, 38), (277, 38), (276, 38), (277, 31), (276, 31), (277, 30), (277, 29), (277, 28), (276, 27), (275, 28), (276, 29), (277, 27), (276, 28), (277, 26), (278, 27), (272, 27), (272, 24), (276, 24), (281, 21), (277, 23), (278, 16), (279, 16), (279, 17), (288, 18), (287, 17), (288, 16), (288, 17), (289, 17), (291, 11), (292, 11), (292, 12), (293, 12), (294, 11), (293, 11), (290, 12), (298, 17), (294, 20), (293, 21), (293, 22), (294, 23), (292, 21), (300, 16), (299, 16), (295, 21), (299, 43), (292, 41), (293, 42), (294, 43), (296, 38), (304, 37), (304, 5), (306, 7), (308, 5), (296, 12), (297, 6), (296, 7), (296, 6), (295, 5), (294, 6), (294, 5), (295, 6), (296, 5), (297, 5), (295, 7), (294, 7), (294, 8), (293, 10), (292, 20), (291, 21), (291, 22), (291, 20), (293, 24), (294, 28), (291, 33), (292, 24), (291, 24), (291, 19), (283, 13), (282, 12), (283, 12), (284, 11), (285, 10), (284, 12), (284, 13), (283, 14), (282, 15), (282, 13), (276, 12), (273, 9), (272, 8), (273, 7), (273, 8), (272, 9), (272, 7), (274, 6), (272, 10), (274, 10), (273, 10), (272, 11), (273, 12), (272, 12), (273, 11), (274, 11), (274, 14), (277, 15), (277, 9), (274, 5), (275, 5), (280, 5), (281, 6), (278, 6), (279, 6), (276, 5), (282, 11), (283, 11), (283, 10), (284, 10), (283, 9), (284, 9), (285, 8), (286, 9), (287, 10), (287, 9), (286, 8), (285, 9), (288, 12), (289, 11), (290, 10), (297, 12), (293, 20), (287, 15), (289, 13), (288, 13), (288, 14), (287, 13), (288, 11), (289, 10), (289, 9), (290, 9), (292, 9), (291, 8), (292, 6), (291, 7), (290, 7), (289, 6), (290, 6), (290, 8), (292, 25), (293, 23), (291, 28), (289, 30), (290, 30), (290, 31), (292, 33), (290, 34), (289, 34), (290, 35), (289, 36), (290, 40), (288, 42), (287, 42), (287, 43), (288, 43), (290, 42), (290, 43), (289, 42), (289, 43), (290, 44), (291, 43), (291, 44), (290, 45), (292, 43), (292, 44), (292, 45), (291, 41), (290, 32), (289, 32), (287, 31), (286, 28), (285, 22), (283, 22), (282, 21), (282, 24), (279, 26), (278, 26), (280, 26), (282, 20), (287, 19), (287, 20), (288, 21), (288, 22), (289, 21), (288, 20), (283, 23), (284, 26), (290, 22), (292, 23), (290, 20), (289, 8), (288, 10), (289, 12), (289, 14), (289, 20), (287, 16), (284, 8), (283, 8), (282, 8), (281, 8), (282, 10), (281, 9), (282, 9), (281, 10), (283, 7), (283, 6), (284, 7), (285, 7), (285, 6), (285, 5), (286, 6), (287, 5), (287, 4), (286, 3), (287, 2), (286, 5), (286, 4), (284, 6), (283, 5), (282, 5), (283, 20), (277, 5), (278, 5), (279, 5), (284, 5), (286, 7), (286, 2), (287, 3), (287, 6), (288, 5), (287, 7), (288, 6), (288, 7), (287, 8), (289, 5), (289, 7), (290, 5), (291, 6), (292, 5), (291, 5), (293, 5), (293, 6), (293, 7), (293, 8), (306, 8), (308, 10), (308, 9), (308, 8), (313, 11), (313, 9), (313, 10), (314, 12), (312, 8), (313, 7), (316, 13), (320, 13), (323, 13), (320, 12), (322, 12), (324, 8), (324, 7), (325, 5), (320, 10), (321, 10), (318, 6), (319, 5), (320, 6), (318, 5), (317, 5), (315, 6), (316, 5), (315, 7), (290, 21), (295, 37), (291, 30), (289, 31), (291, 23), (292, 22), (290, 23), (289, 23), (289, 24), (290, 24), (290, 25), (283, 28), (283, 30), (283, 31), (283, 32), (280, 27), (281, 26), (282, 27), (281, 28), (281, 29), (281, 30), (282, 31), (283, 33), (284, 34), (282, 33), (281, 32), (282, 32), (281, 31), (280, 31), (280, 30), (279, 31), (280, 29), (279, 30), (278, 29), (278, 30), (279, 29), (279, 28), (278, 28), (280, 28), (279, 27), (281, 27), (278, 25), (272, 23), (288, 8), (288, 9), (309, 8), (310, 5), (310, 6), (311, 5), (288, 32), (287, 35), (288, 37), (287, 36), (288, 35), (284, 36), (283, 34), (288, 39), (287, 38), (288, 40), (288, 41), (289, 44), (291, 45), (292, 47), (289, 45), (288, 44), (288, 45), (289, 46), (291, 46), (291, 42), (287, 46), (286, 47), (287, 47), (288, 46), (288, 47), (287, 45), (287, 44), (286, 43), (287, 40), (287, 41), (286, 41), (285, 42), (285, 43), (284, 43), (284, 44), (285, 44), (285, 45), (284, 46), (284, 45), (286, 46), (285, 48), (285, 47), (284, 48), (284, 49), (283, 48), (282, 48), (282, 47), (283, 47), (283, 46), (283, 45), (284, 47), (285, 46), (286, 45), (286, 48), (283, 49), (282, 49), (281, 49), (281, 47), (276, 48), (276, 47), (276, 46), (273, 45), (273, 44), (274, 44), (272, 45), (272, 44), (272, 46), (273, 47), (274, 46), (274, 47), (274, 48), (275, 47), (273, 46), (272, 47), (272, 48), (275, 48), (274, 49), (274, 50), (273, 50), (272, 49), (272, 50), (273, 51), (274, 51), (275, 51), (275, 50), (274, 52), (272, 51), (273, 52), (272, 52), (273, 53), (272, 53), (274, 53), (275, 53), (276, 51), (276, 54), (276, 50), (273, 48), (273, 49), (275, 49), (276, 49), (274, 41), (274, 40), (275, 40), (275, 45), (273, 41), (272, 41), (273, 42), (273, 40), (273, 39), (273, 38), (272, 37), (273, 37), (272, 39), (272, 38), (272, 40), (275, 39), (276, 39), (276, 40), (275, 38), (274, 39), (272, 43), (272, 42), (274, 38), (277, 39), (281, 35), (281, 33), (282, 34), (279, 32), (280, 32), (278, 35), (277, 36), (279, 34), (280, 44), (280, 45), (277, 40), (278, 42), (274, 37), (275, 37), (275, 36), (275, 35), (282, 36), (283, 36), (283, 35), (280, 33), (282, 30), (282, 29), (279, 25), (282, 26), (282, 28), (276, 30), (275, 31), (275, 32), (274, 32), (285, 26), (284, 21), (288, 23), (289, 25), (309, 42), (304, 41), (303, 42), (289, 33), (284, 37), (278, 37), (282, 43), (273, 36), (282, 40), (283, 40), (283, 44), (285, 41), (285, 40), (279, 43), (286, 44), (281, 50), (274, 36), (286, 40), (287, 39), (275, 29), (275, 30), (273, 29), (272, 28), (274, 29), (272, 29), (274, 31), (273, 31), (272, 32), (273, 33), (272, 34), (272, 35), (272, 36), (273, 34), (274, 33), (273, 32), (274, 34), (274, 30), (283, 50), (284, 50), (284, 51), (285, 51), (286, 50), (287, 51), (287, 52), (288, 51), (288, 50), (288, 49), (289, 49), (287, 49), (286, 49), (285, 49), (287, 48), (288, 48), (289, 48), (290, 47), (290, 46), (289, 47), (291, 48), (292, 48), (291, 49), (291, 47), (287, 50), (289, 50), (290, 50), (285, 50), (282, 50), (282, 51), (283, 51), (288, 52), (289, 52), (288, 53), (287, 54), (288, 55), (288, 56), (289, 57), (290, 57), (291, 58), (291, 57), (291, 56), (290, 56), (290, 58), (289, 59), (290, 60), (290, 59), (290, 61), (289, 62), (289, 61), (289, 60), (288, 62), (288, 61), (291, 59), (292, 56), (292, 55), (292, 54), (291, 55), (291, 54), (290, 54), (293, 53), (292, 53), (294, 54), (293, 55), (293, 57), (294, 57), (295, 56), (294, 55), (294, 56), (293, 52), (291, 53), (290, 52), (290, 53), (289, 54), (289, 53), (288, 54), (287, 55), (286, 53), (285, 52), (286, 51), (278, 51), (280, 53), (281, 54), (282, 55), (282, 56), (281, 55), (282, 54), (281, 53), (283, 53), (284, 52), (285, 53), (284, 53), (284, 54), (283, 52), (282, 52), (281, 52), (278, 50), (277, 50), (272, 56), (274, 59), (273, 60), (274, 58), (272, 54), (279, 55), (280, 54), (281, 56), (280, 58), (280, 59), (279, 58), (277, 59), (274, 61), (273, 61), (272, 61), (278, 60), (278, 59), (279, 60), (280, 60), (281, 59), (279, 61), (280, 62), (277, 63), (276, 62), (277, 61), (275, 62), (272, 62), (272, 63), (273, 64), (274, 65), (275, 65), (274, 66), (273, 67), (273, 65), (272, 65), (272, 66), (272, 67), (273, 66), (274, 67), (272, 68), (273, 68), (274, 68), (274, 69), (275, 67), (273, 69), (274, 70), (274, 71), (275, 71), (275, 72), (276, 71), (277, 72), (278, 71), (278, 70), (282, 71), (283, 72), (284, 72), (285, 73), (285, 72), (285, 71), (284, 70), (286, 71), (287, 70), (287, 71), (288, 70), (288, 69), (289, 69), (290, 69), (289, 68), (288, 67), (287, 67), (286, 67), (285, 67), (286, 66), (287, 68), (288, 68), (289, 67), (290, 67), (290, 66), (289, 66), (288, 66), (287, 65), (288, 63), (287, 62), (287, 60), (287, 61), (291, 61), (291, 62), (290, 63), (290, 62), (291, 60), (288, 59), (288, 60), (287, 59), (286, 59), (285, 59), (284, 59), (283, 59), (284, 58), (283, 58), (283, 57), (284, 57), (285, 57), (286, 58), (286, 57), (286, 56), (286, 55), (285, 55), (285, 56), (284, 55), (284, 56), (285, 54), (286, 54), (286, 52), (289, 51), (287, 53), (290, 55), (292, 60), (292, 59), (293, 60), (293, 59), (294, 60), (292, 61), (293, 62), (292, 62), (291, 63), (293, 63), (294, 62), (294, 63), (292, 63), (287, 63), (286, 62), (286, 61), (285, 60), (284, 61), (284, 60), (282, 58), (282, 57), (283, 55), (283, 54), (287, 57), (288, 57), (287, 56), (289, 55), (285, 58), (283, 56), (283, 60), (282, 60), (291, 52), (289, 56), (288, 58), (292, 57), (293, 56), (286, 60), (288, 65), (287, 64), (286, 63), (285, 62), (283, 61), (282, 61), (281, 60), (281, 61), (284, 62), (284, 63), (285, 64), (285, 63), (285, 65), (285, 66), (286, 65), (285, 61), (283, 62), (282, 63), (282, 62), (283, 63), (278, 61), (277, 60), (281, 58), (280, 61), (279, 62), (279, 65), (278, 66), (277, 66), (278, 67), (280, 65), (281, 66), (281, 65), (281, 62), (282, 64), (283, 65), (284, 64), (281, 64), (282, 65), (282, 66), (283, 67), (284, 67), (283, 66), (282, 67), (282, 68), (281, 67), (279, 69), (277, 64), (278, 62), (282, 59), (286, 64), (287, 58), (281, 51), (292, 51), (293, 51), (292, 52), (293, 54), (290, 51), (290, 49), (295, 55), (296, 56), (296, 55), (296, 54), (297, 55), (298, 55), (298, 56), (299, 55), (299, 54), (298, 53), (299, 53), (299, 51), (298, 51), (299, 52), (300, 50), (298, 49), (296, 49), (300, 44), (297, 49), (296, 50), (295, 50), (294, 50), (297, 50), (294, 51), (295, 51), (294, 52), (295, 52), (293, 50), (286, 42), (289, 22), (310, 38), (315, 39), (303, 44), (290, 33), (311, 8), (312, 7), (311, 7), (328, 12), (332, 11), (333, 10), (338, 6), (334, 5), (336, 14), (338, 13), (344, 13), (344, 12), (345, 13), (345, 12), (347, 12), (348, 12), (351, 15), (349, 11), (349, 10), (348, 9), (350, 10), (349, 9), (350, 9), (351, 9), (353, 6), (354, 6), (354, 5), (355, 7), (355, 6), (354, 9), (355, 8), (355, 9), (356, 9), (356, 8), (357, 9), (357, 8), (358, 9), (357, 7), (355, 10), (358, 10), (357, 10), (357, 11), (357, 12), (356, 13), (356, 12), (355, 13), (355, 14), (356, 14), (356, 15), (357, 15), (356, 16), (355, 17), (356, 17), (357, 17), (357, 18), (358, 17), (357, 16), (356, 18), (355, 18), (355, 19), (355, 16), (354, 15), (353, 14), (358, 16), (359, 16), (360, 17), (360, 18), (361, 18), (362, 19), (363, 18), (364, 17), (363, 17), (362, 18), (361, 17), (361, 19), (362, 17), (362, 16), (361, 16), (362, 15), (361, 14), (362, 13), (360, 13), (359, 12), (358, 13), (359, 13), (357, 13), (358, 14), (357, 14), (358, 15), (355, 20), (356, 20), (357, 20), (358, 19), (358, 20), (359, 19), (363, 16), (365, 18), (364, 19), (363, 20), (362, 20), (361, 20), (360, 19), (360, 20), (359, 18), (364, 16), (360, 21), (360, 22), (361, 21), (359, 20), (359, 21), (362, 21), (362, 22), (361, 23), (361, 24), (362, 23), (363, 23), (364, 23), (363, 24), (364, 24), (363, 25), (364, 26), (363, 27), (365, 26), (364, 25), (365, 25), (363, 26), (365, 27), (364, 27), (366, 26), (362, 25), (361, 25), (362, 24), (365, 22), (365, 23), (364, 21), (365, 21), (363, 22), (361, 22), (360, 23), (360, 24), (362, 26), (365, 24), (366, 25), (364, 28), (365, 29), (365, 30), (363, 29), (363, 30), (362, 30), (363, 31), (362, 31), (361, 30), (362, 29), (362, 28), (361, 29), (360, 28), (359, 28), (358, 27), (358, 28), (359, 29), (358, 30), (359, 31), (359, 30), (360, 31), (359, 32), (360, 30), (360, 29), (361, 31), (363, 28), (363, 32), (360, 32), (361, 32), (361, 33), (361, 34), (362, 35), (361, 35), (360, 36), (360, 35), (361, 36), (363, 36), (363, 37), (364, 37), (362, 38), (361, 38), (361, 39), (360, 39), (359, 38), (360, 37), (361, 37), (360, 34), (362, 33), (363, 33), (362, 32), (364, 31), (364, 30), (361, 28), (364, 29), (361, 27), (362, 27), (366, 30), (367, 29), (366, 29), (367, 30), (368, 29), (368, 30), (366, 28), (368, 28), (369, 27), (369, 28), (369, 29), (367, 28), (368, 27), (367, 26), (366, 27), (367, 27), (370, 26), (371, 25), (371, 26), (371, 27), (372, 27), (373, 26), (373, 27), (372, 26), (370, 28), (371, 29), (372, 28), (371, 28), (370, 27), (369, 26), (368, 25), (369, 25), (370, 25), (371, 24), (372, 23), (373, 22), (373, 24), (374, 25), (373, 25), (374, 24), (375, 25), (374, 26), (374, 27), (375, 27), (375, 28), (374, 29), (373, 30), (372, 30), (371, 31), (371, 32), (370, 33), (369, 34), (369, 33), (370, 32), (369, 31), (370, 30), (368, 31), (367, 32), (367, 33), (366, 32), (365, 31), (366, 31), (365, 32), (367, 31), (368, 32), (369, 30), (370, 29), (371, 30), (372, 31), (370, 31), (368, 34), (368, 35), (367, 36), (367, 37), (368, 36), (369, 35), (370, 35), (369, 36), (370, 37), (369, 38), (368, 37), (371, 38), (370, 38), (371, 37), (372, 36), (373, 35), (373, 34), (372, 34), (373, 33), (372, 32), (373, 32), (374, 33), (374, 32), (374, 31), (375, 32), (375, 33), (376, 32), (377, 33), (378, 33), (379, 32), (379, 33), (380, 33), (380, 34), (379, 34), (379, 35), (380, 35), (378, 36), (378, 37), (379, 36), (380, 36), (380, 37), (381, 37), (380, 38), (382, 37), (383, 36), (382, 35), (381, 34), (381, 35), (382, 34), (379, 37), (378, 38), (378, 39), (379, 40), (380, 41), (381, 41), (380, 40), (379, 39), (377, 37), (378, 35), (377, 39), (377, 38), (376, 39), (375, 39), (376, 38), (375, 37), (376, 36), (375, 35), (375, 36), (374, 37), (373, 38), (373, 37), (372, 38), (374, 36), (373, 36), (375, 38), (375, 40), (375, 41), (376, 40), (376, 41), (377, 42), (376, 43), (376, 42), (374, 39), (373, 39), (372, 39), (370, 36), (369, 37), (368, 33), (370, 34), (373, 31), (374, 34), (372, 33), (371, 33), (375, 30), (374, 30), (375, 29), (376, 31), (377, 31), (376, 30), (375, 31), (377, 30), (378, 31), (377, 32), (379, 31), (378, 30), (379, 30), (382, 36), (381, 38), (380, 39), (382, 40), (381, 42), (381, 43), (382, 43), (383, 43), (384, 43), (384, 44), (385, 43), (386, 44), (386, 45), (385, 45), (384, 46), (383, 47), (383, 48), (382, 48), (383, 49), (382, 50), (383, 51), (384, 52), (385, 52), (382, 49), (381, 48), (381, 49), (380, 48), (380, 49), (379, 48), (380, 47), (381, 46), (382, 47), (382, 46), (379, 50), (378, 51), (379, 51), (378, 50), (378, 52), (379, 53), (379, 52), (380, 51), (380, 50), (381, 52), (382, 51), (382, 52), (381, 51), (381, 50), (384, 48), (385, 49), (385, 50), (386, 49), (386, 50), (386, 51), (387, 50), (387, 51), (387, 49), (387, 48), (388, 50), (388, 51), (387, 52), (388, 52), (387, 53), (388, 53), (386, 52), (384, 49), (384, 47), (385, 47), (386, 47), (387, 47), (386, 46), (387, 46), (388, 47), (389, 48), (390, 48), (391, 47), (392, 47), (392, 46), (392, 45), (393, 45), (394, 46), (395, 46), (395, 45), (394, 45), (393, 44), (393, 46), (391, 46), (394, 44), (395, 43), (394, 42), (395, 42), (394, 43), (391, 44), (392, 44), (390, 47), (390, 49), (389, 47), (391, 50), (389, 50), (389, 49), (390, 50), (389, 51), (390, 51), (391, 52), (390, 52), (391, 51), (392, 51), (393, 50), (392, 52), (393, 53), (394, 52), (393, 52), (394, 51), (395, 50), (396, 49), (396, 50), (397, 50), (398, 49), (399, 50), (400, 51), (400, 52), (399, 52), (398, 52), (397, 51), (398, 50), (397, 52), (396, 51), (398, 53), (397, 54), (399, 54), (399, 53), (400, 53), (398, 54), (398, 55), (397, 55), (396, 54), (395, 55), (395, 56), (394, 56), (394, 55), (393, 56), (395, 57), (396, 58), (395, 59), (396, 60), (395, 61), (395, 60), (396, 61), (397, 60), (396, 59), (397, 59), (397, 61), (398, 61), (398, 60), (398, 62), (399, 61), (400, 61), (399, 60), (398, 63), (399, 62), (400, 63), (401, 64), (401, 63), (401, 62), (398, 59), (395, 58), (394, 60), (394, 59), (393, 59), (394, 58), (393, 57), (392, 58), (392, 59), (392, 57), (393, 58), (394, 57), (393, 60), (394, 61), (393, 61), (392, 61), (392, 62), (391, 61), (391, 62), (392, 63), (393, 62), (394, 62), (395, 62), (397, 58), (398, 57), (399, 58), (400, 59), (401, 60), (401, 59), (400, 60), (401, 58), (399, 59), (400, 58), (399, 57), (398, 56), (399, 56), (399, 55), (400, 56), (400, 55), (401, 56), (401, 55), (396, 55), (397, 56), (396, 56), (396, 57), (394, 63), (393, 64), (393, 65), (394, 64), (395, 64), (394, 65), (394, 66), (394, 67), (395, 66), (396, 65), (397, 65), (398, 66), (399, 67), (399, 66), (400, 66), (400, 67), (399, 65), (401, 67), (398, 67), (397, 68), (400, 68), (399, 68), (398, 69), (399, 70), (398, 70), (399, 71), (398, 71), (397, 72), (398, 72), (397, 73), (398, 74), (399, 73), (400, 74), (399, 75), (400, 76), (400, 75), (401, 76), (399, 76), (398, 77), (399, 77), (398, 75), (399, 74), (401, 73), (400, 72), (401, 74), (401, 75), (401, 77), (400, 78), (401, 78), (401, 79), (400, 80), (399, 80), (400, 81), (399, 82), (398, 81), (398, 82), (397, 82), (397, 80), (397, 81), (399, 81), (399, 79), (400, 79), (399, 78), (398, 78), (398, 79), (398, 80), (396, 80), (396, 81), (396, 82), (395, 83), (394, 83), (393, 82), (394, 81), (393, 81), (392, 81), (391, 81), (390, 80), (391, 79), (390, 79), (390, 81), (389, 81), (390, 82), (390, 83), (391, 84), (391, 85), (392, 85), (391, 86), (392, 86), (390, 86), (389, 86), (388, 86), (387, 85), (387, 86), (388, 87), (388, 88), (387, 87), (386, 87), (385, 86), (386, 86), (390, 87), (392, 87), (393, 86), (394, 86), (394, 85), (395, 86), (393, 85), (394, 84), (393, 84), (391, 83), (392, 84), (395, 85), (396, 84), (397, 85), (396, 85), (397, 86), (396, 86), (394, 87), (396, 87), (395, 87), (392, 83), (393, 83), (394, 82), (395, 84), (394, 88), (393, 88), (393, 89), (392, 89), (393, 90), (394, 89), (394, 90), (395, 89), (395, 88), (393, 87), (392, 90), (391, 89), (391, 88), (391, 82), (390, 84), (389, 84), (389, 85), (389, 87), (391, 87), (390, 88), (389, 88), (388, 85), (388, 84), (385, 88), (384, 87), (385, 85), (386, 84), (385, 84), (385, 83), (385, 82), (386, 82), (387, 81), (388, 81), (389, 82), (388, 82), (387, 82), (386, 83), (389, 83), (388, 83), (387, 84), (386, 85), (390, 89), (387, 88), (385, 87), (384, 86), (384, 85), (384, 84), (383, 86), (382, 87), (381, 87), (380, 86), (382, 86), (382, 85), (381, 85), (380, 84), (380, 85), (379, 85), (379, 86), (378, 85), (379, 84), (378, 84), (379, 83), (378, 83), (379, 82), (379, 87), (378, 87), (377, 86), (380, 82), (379, 81), (380, 81), (381, 82), (382, 81), (382, 80), (381, 81), (380, 83), (381, 84), (378, 86), (377, 85), (377, 87), (376, 84), (377, 84), (375, 83), (376, 82), (376, 85), (375, 85), (374, 86), (374, 87), (373, 86), (373, 85), (372, 85), (372, 84), (372, 83), (371, 84), (373, 83), (373, 82), (370, 83), (369, 83), (370, 84), (371, 85), (370, 86), (370, 85), (370, 82), (371, 82), (371, 83), (369, 82), (372, 81), (371, 80), (370, 81), (368, 84), (367, 84), (366, 85), (367, 85), (366, 84), (366, 83), (367, 82), (368, 83), (368, 85), (367, 83), (366, 82), (365, 83), (364, 82), (364, 81), (365, 80), (365, 81), (365, 82), (364, 83), (366, 80), (366, 79), (366, 78), (365, 79), (363, 81), (362, 81), (361, 81), (362, 80), (361, 80), (362, 82), (362, 83), (361, 83), (360, 84), (359, 85), (358, 84), (357, 84), (356, 84), (357, 85), (358, 85), (359, 86), (358, 86), (358, 87), (360, 85), (359, 84), (359, 83), (358, 83), (357, 82), (358, 82), (359, 82), (359, 81), (360, 81), (360, 82), (360, 83), (361, 84), (362, 85), (361, 85), (361, 86), (361, 87), (361, 88), (362, 89), (363, 89), (364, 90), (365, 89), (366, 89), (366, 88), (365, 91), (366, 90), (367, 90), (368, 90), (369, 90), (369, 89), (370, 88), (371, 89), (372, 88), (371, 87), (371, 86), (372, 87), (373, 87), (372, 86), (376, 86), (375, 86), (373, 84), (372, 82), (370, 80), (371, 81), (372, 80), (373, 81), (372, 79), (372, 78), (371, 78), (370, 77), (369, 76), (368, 75), (369, 77), (371, 77), (370, 78), (369, 79), (370, 79), (369, 80), (368, 81), (369, 81), (374, 81), (374, 84), (374, 85), (374, 83), (375, 81), (374, 80), (373, 79), (374, 78), (375, 77), (375, 78), (375, 76), (376, 75), (377, 76), (376, 76), (377, 75), (376, 74), (375, 74), (375, 73), (374, 72), (374, 73), (374, 75), (373, 74), (373, 73), (373, 72), (373, 71), (372, 70), (373, 69), (374, 70), (373, 70), (371, 69), (372, 69), (372, 68), (373, 68), (372, 67), (373, 66), (373, 65), (372, 65), (371, 64), (370, 65), (369, 64), (369, 65), (368, 66), (368, 67), (369, 67), (368, 65), (367, 65), (366, 65), (365, 65), (364, 64), (365, 64), (364, 65), (363, 64), (363, 63), (363, 62), (362, 63), (364, 63), (362, 64), (362, 65), (361, 64), (361, 63), (363, 65), (362, 66), (363, 67), (364, 66), (364, 67), (363, 68), (362, 67), (364, 68), (361, 65), (362, 62), (362, 61), (362, 60), (361, 60), (362, 59), (363, 58), (362, 58), (363, 59), (364, 60), (365, 60), (364, 61), (363, 60), (363, 61), (361, 62), (360, 64), (360, 62), (360, 61), (359, 61), (360, 60), (361, 59), (361, 61), (360, 65), (360, 66), (359, 67), (358, 66), (358, 67), (357, 68), (358, 68), (357, 66), (358, 65), (357, 65), (361, 66), (363, 66), (360, 67), (361, 67), (366, 61), (367, 62), (368, 63), (369, 63), (369, 62), (370, 62), (370, 61), (371, 60), (372, 60), (373, 61), (372, 59), (373, 58), (373, 59), (372, 58), (372, 57), (372, 56), (371, 57), (371, 58), (370, 59), (370, 60), (369, 60), (369, 61), (368, 62), (367, 61), (368, 60), (368, 59), (367, 59), (367, 58), (369, 59), (372, 61), (371, 61), (374, 61), (375, 61), (375, 62), (375, 63), (376, 63), (377, 64), (378, 64), (379, 63), (380, 62), (379, 62), (380, 63), (380, 64), (377, 63), (378, 62), (379, 64), (381, 64), (382, 65), (383, 64), (382, 66), (381, 65), (380, 66), (381, 67), (380, 67), (379, 66), (378, 66), (377, 66), (379, 65), (382, 64), (383, 65), (381, 66), (379, 67), (382, 67), (378, 67), (377, 67), (376, 68), (377, 68), (378, 68), (378, 69), (377, 69), (376, 69), (378, 70), (378, 71), (379, 70), (380, 71), (381, 71), (381, 72), (380, 72), (379, 69), (379, 71), (378, 72), (377, 71), (376, 71), (375, 70), (374, 71), (375, 72), (376, 72), (377, 73), (376, 73), (375, 71), (372, 71), (376, 70), (377, 72), (377, 74), (375, 75), (374, 74), (372, 72), (378, 74), (378, 73), (378, 76), (378, 77), (379, 77), (379, 76), (377, 77), (376, 77), (376, 78), (377, 78), (378, 78), (379, 78), (380, 77), (380, 78), (380, 79), (374, 79), (375, 80), (376, 79), (377, 80), (376, 81), (377, 82), (377, 83), (376, 83), (376, 87), (377, 88), (375, 87), (376, 88), (375, 88), (376, 89), (377, 90), (377, 91), (376, 91), (376, 90), (375, 91), (377, 92), (378, 92), (377, 93), (378, 93), (376, 94), (377, 95), (378, 96), (379, 97), (380, 97), (379, 96), (378, 95), (377, 94), (378, 94), (379, 95), (380, 95), (381, 94), (380, 93), (379, 92), (380, 92), (379, 93), (380, 94), (381, 96), (381, 97), (380, 96), (381, 98), (382, 97), (383, 97), (382, 96), (383, 95), (384, 95), (382, 95), (382, 94), (381, 93), (381, 95), (377, 96), (376, 95), (375, 94), (375, 93), (374, 93), (374, 95), (375, 95), (374, 96), (373, 94), (374, 94), (374, 92), (374, 91), (375, 92), (373, 92), (374, 90), (373, 89), (373, 90), (372, 95), (372, 96), (373, 96), (373, 93), (372, 93), (372, 92), (373, 91), (375, 89), (375, 90), (377, 89), (378, 88), (379, 89), (379, 88), (378, 89), (378, 90), (378, 91), (376, 92), (376, 93), (376, 96), (371, 91), (372, 91), (380, 89), (374, 88), (378, 81), (378, 82), (381, 83), (377, 81), (375, 82), (375, 79), (373, 78), (378, 79), (378, 80), (382, 83), (382, 82), (376, 80), (379, 79), (380, 80), (381, 80), (379, 80), (383, 82), (383, 83), (383, 87), (381, 86), (380, 87), (382, 84), (383, 81), (384, 81), (385, 80), (384, 79), (384, 78), (385, 78), (384, 77), (384, 76), (385, 75), (386, 76), (385, 77), (386, 78), (387, 77), (387, 78), (388, 77), (389, 76), (387, 79), (388, 79), (389, 80), (387, 83), (390, 85), (389, 89), (391, 90), (390, 90), (389, 91), (390, 91), (391, 91), (392, 92), (392, 91), (393, 92), (392, 93), (391, 93), (393, 91), (392, 88), (394, 91), (395, 90), (396, 89), (397, 88), (398, 88), (399, 87), (400, 87), (401, 88), (400, 89), (401, 87), (400, 88), (399, 88), (400, 90), (399, 90), (400, 91), (401, 89), (401, 90), (398, 89), (399, 86), (398, 87), (397, 87), (396, 83), (397, 83), (398, 83), (399, 84), (399, 85), (401, 86), (400, 84), (397, 90), (396, 90), (395, 91), (396, 92), (397, 92), (398, 93), (399, 93), (398, 94), (397, 95), (396, 96), (398, 96), (399, 95), (400, 94), (401, 93), (400, 93), (401, 94), (400, 92), (401, 91), (401, 92), (399, 89), (400, 86), (400, 85), (398, 85), (398, 84), (395, 82), (396, 88), (397, 89), (397, 84), (398, 86), (395, 92), (395, 93), (395, 94), (394, 95), (396, 93), (397, 93), (397, 94), (398, 95), (399, 94), (399, 96), (400, 95), (401, 95), (400, 97), (401, 96), (400, 98), (401, 98), (400, 96), (399, 97), (397, 97), (397, 96), (396, 95), (396, 94), (401, 97), (396, 97), (397, 98), (396, 99), (395, 99), (394, 99), (393, 98), (393, 97), (392, 97), (392, 98), (391, 97), (392, 96), (393, 96), (391, 96), (391, 95), (390, 95), (390, 94), (389, 94), (389, 95), (390, 96), (393, 95), (392, 94), (392, 95), (391, 94), (393, 94), (394, 93), (394, 94), (393, 93), (391, 92), (390, 92), (389, 90), (388, 92), (389, 93), (388, 94), (388, 95), (388, 93), (387, 92), (386, 93), (385, 93), (386, 94), (387, 93), (387, 94), (387, 95), (389, 96), (390, 97), (394, 96), (395, 97), (395, 95), (395, 96), (394, 98), (394, 97), (395, 98), (391, 98), (392, 99), (391, 100), (391, 101), (392, 102), (393, 101), (393, 100), (392, 101), (392, 100), (393, 99), (394, 100), (395, 101), (396, 101), (396, 102), (397, 102), (398, 103), (397, 103), (399, 104), (398, 104), (399, 103), (397, 104), (398, 105), (397, 105), (398, 106), (399, 107), (400, 107), (399, 106), (399, 102), (400, 102), (400, 101), (399, 101), (397, 101), (398, 100), (399, 99), (400, 99), (400, 100), (401, 99), (399, 100), (397, 99), (398, 99), (399, 98), (398, 97), (399, 92), (398, 92), (398, 91), (399, 83), (400, 82), (401, 82), (401, 81), (400, 83), (401, 83), (397, 79), (396, 78), (395, 77), (394, 78), (393, 77), (393, 78), (394, 79), (393, 79), (395, 79), (395, 78), (395, 76), (394, 75), (394, 74), (393, 75), (393, 76), (394, 77), (394, 76), (392, 77), (392, 78), (394, 80), (395, 80), (396, 79), (397, 78), (396, 77), (395, 75), (396, 76), (393, 74), (392, 74), (393, 73), (391, 74), (392, 73), (392, 72), (392, 71), (391, 72), (391, 71), (392, 70), (393, 71), (391, 69), (392, 68), (391, 67), (393, 68), (391, 68), (392, 69), (391, 70), (390, 70), (391, 75), (391, 76), (390, 74), (389, 74), (389, 73), (390, 73), (390, 75), (389, 75), (392, 76), (391, 77), (391, 78), (390, 77), (389, 78), (389, 79), (388, 80), (386, 81), (385, 81), (383, 80), (384, 80), (384, 82), (386, 80), (384, 83), (383, 84), (383, 85), (384, 88), (386, 89), (372, 89), (381, 92), (382, 92), (383, 91), (384, 91), (383, 90), (384, 90), (385, 90), (385, 91), (384, 92), (386, 91), (387, 91), (387, 90), (388, 89), (386, 88), (385, 89), (386, 92), (385, 92), (384, 93), (384, 94), (383, 93), (383, 92), (385, 94), (387, 96), (398, 98), (394, 92), (388, 97), (388, 98), (387, 97), (388, 99), (387, 98), (389, 97), (390, 98), (388, 96), (386, 97), (385, 96), (385, 95), (383, 94), (386, 96), (389, 98), (390, 99), (391, 99), (389, 99), (389, 100), (390, 100), (390, 102), (390, 101), (389, 102), (389, 103), (390, 103), (390, 104), (389, 105), (388, 106), (389, 106), (388, 105), (390, 107), (390, 108), (389, 109), (389, 107), (390, 106), (388, 107), (387, 107), (387, 106), (386, 106), (385, 106), (386, 105), (387, 105), (386, 107), (387, 104), (389, 104), (388, 104), (388, 103), (385, 107), (384, 107), (384, 106), (385, 108), (384, 109), (383, 109), (383, 110), (384, 110), (385, 110), (386, 109), (387, 110), (386, 111), (385, 112), (385, 111), (386, 110), (387, 111), (386, 112), (386, 113), (384, 111), (384, 112), (386, 108), (387, 109), (388, 108), (387, 108), (388, 112), (389, 112), (390, 111), (391, 110), (390, 112), (390, 113), (391, 113), (390, 114), (389, 115), (389, 114), (388, 114), (388, 115), (387, 115), (386, 116), (385, 115), (385, 114), (384, 114), (384, 115), (384, 116), (385, 116), (386, 115), (384, 117), (385, 117), (383, 116), (383, 118), (384, 119), (385, 119), (386, 120), (386, 119), (387, 119), (388, 118), (389, 118), (388, 119), (388, 120), (389, 121), (390, 122), (388, 122), (388, 121), (387, 121), (387, 120), (386, 121), (383, 120), (382, 120), (381, 121), (381, 122), (382, 121), (382, 122), (381, 123), (381, 120), (380, 122), (380, 123), (379, 124), (379, 125), (378, 123), (379, 122), (379, 121), (380, 121), (380, 125), (380, 124), (381, 124), (382, 124), (381, 125), (382, 126), (383, 125), (379, 123), (378, 124), (377, 123), (376, 123), (376, 122), (377, 121), (378, 121), (377, 120), (378, 120), (378, 119), (379, 120), (376, 120), (375, 119), (376, 121), (375, 120), (379, 118), (379, 119), (377, 122), (377, 124), (378, 125), (375, 123), (375, 122), (376, 119), (376, 118), (374, 120), (375, 121), (374, 122), (375, 118), (374, 119), (375, 124), (376, 124), (380, 126), (381, 127), (380, 127), (382, 127), (381, 128), (380, 129), (380, 130), (381, 129), (382, 129), (382, 130), (383, 131), (383, 130), (383, 129), (382, 128), (381, 126), (379, 127), (379, 128), (379, 130), (378, 130), (378, 131), (379, 131), (380, 131), (379, 132), (380, 132), (381, 133), (380, 133), (381, 132), (381, 131), (382, 132), (379, 129), (378, 129), (377, 128), (377, 129), (376, 129), (375, 128), (376, 127), (377, 127), (376, 126), (375, 127), (376, 128), (374, 127), (373, 127), (373, 128), (372, 128), (371, 127), (372, 127), (372, 129), (373, 129), (374, 130), (375, 129), (374, 128), (376, 130), (374, 126), (375, 126), (374, 125), (375, 130), (376, 131), (377, 131), (375, 132), (375, 131), (374, 132), (373, 133), (374, 133), (375, 133), (374, 129), (372, 126), (371, 126), (371, 125), (377, 130), (376, 132), (377, 132), (378, 133), (378, 134), (377, 134), (379, 133), (378, 128), (374, 131), (376, 134), (378, 132), (377, 133), (376, 133), (373, 132), (373, 126), (373, 124), (374, 123), (373, 121), (372, 122), (373, 123), (378, 122), (377, 119), (377, 118), (378, 118), (377, 117), (375, 117), (374, 117), (376, 116), (376, 117), (377, 116), (378, 117), (378, 116), (379, 116), (380, 116), (381, 117), (382, 118), (381, 118), (382, 117), (383, 117), (382, 119), (384, 120), (385, 118), (386, 117), (387, 118), (388, 117), (389, 117), (390, 117), (391, 118), (390, 118), (389, 119), (384, 118), (383, 119), (386, 118), (386, 114), (387, 114), (389, 113), (388, 113), (389, 116), (385, 120), (390, 120), (390, 119), (390, 116), (391, 116), (392, 115), (392, 114), (393, 113), (393, 112), (392, 112), (394, 114), (394, 115), (395, 115), (394, 116), (393, 114), (393, 115), (395, 116), (395, 117), (393, 117), (393, 116), (391, 114), (391, 115), (390, 115), (394, 113), (395, 113), (396, 112), (395, 111), (395, 110), (394, 110), (396, 110), (396, 109), (395, 108), (395, 109), (397, 110), (398, 111), (397, 111), (398, 112), (398, 113), (399, 113), (397, 113), (396, 113), (394, 112), (392, 116), (392, 117), (394, 118), (395, 118), (396, 119), (397, 118), (397, 119), (397, 120), (396, 121), (396, 122), (395, 121), (395, 120), (396, 120), (394, 120), (393, 121), (393, 119), (392, 120), (392, 122), (391, 121), (391, 120), (389, 122), (389, 123), (388, 124), (388, 123), (390, 121), (391, 122), (391, 123), (390, 123), (391, 124), (390, 125), (389, 126), (388, 126), (387, 127), (389, 125), (389, 124), (388, 125), (390, 124), (391, 125), (387, 122), (386, 123), (386, 122), (385, 122), (385, 123), (384, 121), (387, 116), (388, 116), (387, 117), (383, 115), (382, 114), (383, 114), (384, 113), (383, 112), (385, 113), (387, 112), (388, 111), (389, 111), (388, 110), (389, 110), (390, 110), (392, 109), (393, 109), (392, 110), (391, 109), (391, 108), (392, 108), (393, 108), (392, 107), (390, 109), (391, 107), (393, 107), (394, 107), (395, 106), (396, 107), (396, 106), (394, 105), (395, 105), (394, 104), (394, 103), (395, 104), (394, 102), (393, 102), (393, 103), (392, 104), (392, 103), (394, 101), (388, 100), (385, 98), (384, 98), (384, 96), (383, 96), (385, 99), (384, 100), (383, 101), (382, 101), (381, 100), (380, 101), (379, 100), (378, 101), (378, 100), (379, 101), (377, 100), (378, 99), (377, 99), (379, 98), (373, 88), (375, 84), (379, 91), (380, 90), (379, 94), (371, 94), (370, 94), (369, 93), (368, 94), (367, 93), (366, 93), (369, 92), (370, 91), (372, 90), (371, 88), (369, 84), (368, 82), (366, 81), (364, 79), (363, 80), (364, 80), (363, 79), (363, 78), (363, 77), (362, 78), (361, 79), (360, 79), (359, 80), (358, 81), (357, 86), (356, 86), (355, 87), (354, 86), (353, 86), (354, 85), (354, 84), (353, 84), (355, 83), (356, 83), (357, 83), (356, 82), (355, 82), (357, 87), (356, 87), (355, 86), (359, 88), (360, 88), (361, 89), (360, 87), (359, 87), (358, 88), (357, 89), (357, 88), (356, 88), (355, 89), (354, 89), (353, 90), (352, 90), (352, 89), (351, 90), (351, 89), (352, 91), (353, 92), (354, 91), (354, 90), (355, 91), (352, 88), (353, 87), (353, 88), (353, 89), (353, 91), (355, 92), (356, 91), (357, 92), (357, 91), (356, 90), (356, 89), (357, 90), (358, 90), (359, 90), (360, 90), (359, 91), (360, 91), (360, 92), (359, 93), (360, 93), (359, 92), (358, 89), (354, 87), (355, 85), (355, 84), (353, 85), (354, 83), (354, 82), (354, 81), (353, 80), (352, 80), (351, 79), (353, 79), (352, 81), (353, 81), (352, 82), (354, 80), (352, 79), (353, 78), (352, 77), (353, 77), (354, 76), (355, 76), (356, 76), (357, 75), (356, 77), (357, 78), (357, 79), (358, 80), (357, 80), (356, 81), (353, 83), (352, 83), (351, 82), (351, 83), (350, 84), (350, 83), (349, 83), (348, 83), (347, 84), (347, 83), (348, 84), (347, 85), (346, 84), (345, 85), (344, 85), (343, 84), (342, 83), (343, 82), (344, 83), (345, 82), (346, 82), (347, 82), (346, 81), (347, 81), (346, 80), (345, 81), (344, 82), (344, 81), (343, 81), (344, 80), (343, 79), (344, 79), (343, 78), (344, 78), (343, 77), (342, 76), (341, 77), (342, 78), (345, 77), (345, 78), (344, 77), (345, 76), (346, 76), (347, 75), (346, 75), (347, 76), (347, 77), (346, 78), (347, 78), (348, 74), (347, 74), (347, 73), (346, 72), (348, 73), (349, 73), (350, 74), (350, 73), (349, 74), (350, 75), (351, 75), (349, 76), (348, 75), (345, 74), (346, 73), (345, 73), (344, 72), (343, 71), (343, 70), (342, 69), (341, 69), (340, 70), (341, 70), (340, 69), (341, 71), (342, 70), (343, 69), (344, 68), (343, 68), (344, 67), (345, 67), (346, 67), (345, 68), (343, 67), (342, 68), (342, 71), (343, 72), (342, 72), (342, 73), (343, 74), (344, 73), (346, 74), (345, 75), (350, 72), (349, 72), (348, 71), (349, 70), (350, 70), (351, 69), (352, 68), (351, 70), (350, 71), (349, 71), (348, 70), (347, 69), (346, 70), (347, 71), (345, 69), (344, 69), (342, 67), (343, 66), (342, 65), (341, 66), (342, 66), (341, 67), (341, 65), (340, 66), (340, 65), (343, 64), (344, 64), (343, 63), (342, 62), (342, 61), (341, 62), (341, 63), (341, 61), (342, 60), (341, 59), (341, 60), (340, 60), (339, 59), (339, 58), (340, 59), (340, 61), (340, 62), (339, 61), (338, 59), (337, 59), (336, 59), (337, 58), (337, 60), (336, 58), (338, 58), (338, 57), (339, 57), (337, 57), (337, 56), (338, 62), (337, 63), (338, 64), (339, 64), (339, 63), (339, 62), (337, 62), (337, 61), (336, 60), (338, 61), (338, 63), (339, 65), (340, 63), (340, 64), (341, 64), (342, 63), (343, 62), (339, 60), (338, 65), (339, 66), (338, 67), (339, 68), (338, 68), (339, 67), (337, 68), (338, 69), (335, 69), (336, 69), (337, 69), (335, 70), (335, 66), (336, 64), (337, 65), (337, 64), (336, 63), (337, 66), (336, 67), (338, 66), (335, 68), (333, 67), (333, 65), (332, 63), (326, 65), (325, 58), (326, 58), (327, 57), (326, 59), (327, 59), (327, 58), (325, 56), (324, 55), (322, 50), (324, 51), (323, 50), (328, 52), (330, 51), (329, 50), (322, 46), (319, 46), (318, 46), (320, 46), (321, 44), (320, 44), (320, 43), (320, 42), (320, 40), (321, 40), (321, 41), (322, 40), (318, 42), (317, 40), (318, 40), (316, 40), (315, 41), (314, 41), (315, 40), (317, 42), (319, 43), (319, 44), (318, 45), (317, 44), (317, 43), (318, 43), (318, 39), (322, 41), (321, 39), (318, 44), (323, 36), (316, 34), (299, 48), (296, 51), (294, 53), (295, 53), (296, 52), (296, 53), (296, 57), (297, 56), (297, 57), (298, 57), (299, 56), (300, 55), (300, 56), (300, 57), (301, 57), (302, 57), (303, 58), (304, 58), (304, 59), (304, 57), (305, 57), (305, 58), (303, 59), (304, 60), (303, 61), (295, 62), (295, 61), (295, 60), (294, 61), (295, 59), (295, 58), (296, 58), (297, 58), (299, 57), (299, 58), (300, 58), (295, 57), (294, 58), (294, 59), (293, 61), (281, 57), (277, 55), (276, 56), (273, 62), (273, 63), (274, 63), (274, 64), (272, 69), (275, 64), (276, 64), (275, 63), (274, 62), (276, 63), (276, 65), (277, 67), (291, 50), (290, 48), (292, 50), (291, 51), (297, 51), (298, 52), (301, 54), (302, 55), (303, 56), (303, 55), (302, 56), (304, 56), (305, 56), (306, 55), (305, 55), (304, 55), (307, 54), (310, 54), (311, 54), (310, 55), (310, 53), (308, 57), (309, 58), (307, 56), (306, 56), (306, 57), (312, 54), (311, 52), (310, 51), (311, 53), (313, 54), (319, 60), (320, 61), (321, 61), (321, 60), (322, 59), (318, 61), (317, 60), (317, 61), (316, 61), (315, 61), (313, 55), (311, 55), (311, 58), (314, 62), (315, 63), (316, 64), (315, 65), (314, 66), (313, 67), (312, 67), (313, 66), (312, 66), (311, 67), (311, 66), (310, 67), (311, 70), (312, 69), (311, 69), (311, 68), (312, 68), (314, 67), (313, 68), (312, 70), (313, 69), (314, 68), (315, 67), (315, 68), (314, 69), (313, 70), (315, 69), (314, 70), (315, 70), (313, 71), (313, 72), (313, 73), (312, 73), (311, 73), (312, 74), (311, 74), (311, 72), (312, 72), (314, 72), (316, 69), (316, 68), (317, 67), (316, 66), (314, 64), (313, 63), (315, 66), (318, 67), (317, 64), (318, 63), (318, 62), (318, 64), (318, 65), (316, 67), (314, 65), (313, 65), (312, 64), (311, 65), (312, 65), (311, 64), (313, 64), (311, 63), (311, 62), (312, 63), (310, 63), (309, 64), (310, 64), (310, 65), (309, 65), (310, 66), (309, 66), (308, 65), (307, 64), (309, 63), (308, 62), (307, 62), (308, 61), (308, 64), (307, 63), (307, 65), (306, 66), (307, 67), (306, 68), (297, 71), (303, 63), (304, 62), (304, 63), (305, 63), (305, 62), (305, 64), (306, 64), (305, 66), (305, 65), (304, 65), (303, 65), (306, 65), (303, 62), (302, 62), (304, 61), (306, 62), (306, 63), (304, 64), (292, 58), (289, 58), (282, 53), (307, 50), (312, 49), (313, 48), (313, 49), (312, 50), (313, 51), (313, 52), (312, 53), (312, 52), (313, 53), (313, 61), (312, 62), (310, 62), (309, 61), (307, 60), (307, 61), (305, 61), (302, 58), (303, 57), (308, 58), (310, 58), (307, 58), (307, 59), (308, 59), (310, 59), (309, 59), (301, 55), (301, 56), (297, 52), (297, 54), (298, 54), (298, 58), (299, 59), (298, 60), (298, 59), (296, 59), (297, 59), (296, 60), (293, 64), (303, 60), (286, 68), (285, 68), (286, 69), (285, 70), (285, 69), (284, 69), (286, 72), (287, 72), (288, 72), (287, 73), (288, 73), (287, 74), (286, 74), (285, 75), (284, 75), (284, 74), (285, 74), (286, 75), (288, 74), (288, 75), (287, 75), (287, 76), (287, 77), (288, 76), (288, 77), (289, 76), (289, 77), (290, 78), (290, 76), (291, 75), (291, 76), (290, 77), (291, 77), (292, 77), (291, 78), (290, 79), (289, 78), (293, 78), (296, 74), (299, 72), (311, 76), (312, 77), (311, 77), (312, 76), (313, 77), (314, 77), (313, 76), (314, 76), (318, 80), (318, 83), (319, 84), (317, 79), (317, 80), (314, 75), (314, 73), (310, 74), (310, 72), (312, 75), (308, 77), (309, 76), (310, 75), (306, 75), (307, 75), (308, 72), (296, 75), (298, 76), (297, 76), (295, 80), (294, 79), (295, 79), (293, 77), (292, 76), (292, 78), (292, 79), (292, 75), (294, 74), (297, 72), (294, 71), (306, 61), (306, 60), (306, 59), (305, 59), (305, 60), (310, 52), (311, 51), (312, 57), (317, 62), (319, 65), (319, 68), (320, 69), (313, 74), (311, 75), (309, 73), (309, 62), (308, 60), (308, 63), (297, 60), (293, 58), (280, 57), (276, 57), (295, 54), (297, 53), (307, 69), (308, 69), (312, 71), (311, 71), (310, 71), (309, 71), (308, 70), (310, 70), (307, 70), (306, 70), (308, 68), (309, 69), (314, 71), (319, 74), (322, 72), (327, 75), (328, 75), (329, 73), (330, 73), (333, 72), (333, 73), (334, 72), (328, 67), (327, 67), (328, 66), (318, 70), (312, 61), (311, 60), (310, 61), (310, 60), (309, 60), (311, 61), (312, 60), (316, 60), (312, 59), (312, 55), (313, 50), (314, 51), (314, 52), (315, 50), (316, 50), (318, 50), (317, 50), (321, 54), (322, 54), (322, 56), (323, 56), (324, 57), (328, 58), (329, 59), (330, 59), (329, 56), (331, 56), (331, 55), (329, 55), (328, 54), (326, 56), (328, 56), (328, 55), (330, 52), (329, 52), (331, 52), (332, 53), (331, 54), (330, 55), (329, 54), (330, 53), (336, 48), (335, 50), (334, 51), (323, 57), (324, 56), (320, 55), (321, 56), (322, 57), (321, 57), (329, 53), (330, 54), (332, 54), (332, 55), (333, 56), (333, 59), (333, 60), (333, 61), (334, 61), (335, 62), (338, 60), (340, 58), (340, 56), (338, 56), (335, 60), (336, 61), (336, 62), (341, 68), (340, 67), (344, 65), (344, 66), (343, 65), (342, 64), (344, 63), (345, 64), (344, 62), (345, 62), (346, 61), (344, 61), (343, 61), (345, 60), (344, 59), (344, 60), (345, 59), (346, 58), (346, 60), (347, 61), (345, 61), (342, 59), (343, 58), (344, 57), (343, 56), (342, 55), (341, 54), (340, 53), (342, 51), (342, 50), (341, 49), (342, 49), (340, 54), (340, 55), (339, 56), (339, 55), (341, 57), (342, 57), (341, 56), (339, 54), (338, 53), (337, 54), (337, 55), (338, 54), (336, 53), (338, 52), (337, 53), (336, 51), (335, 54), (333, 55), (334, 55), (335, 56), (336, 57), (340, 57), (338, 55), (335, 59), (339, 69), (338, 70), (337, 70), (339, 70), (336, 70), (337, 71), (340, 68), (339, 71), (340, 72), (339, 72), (340, 71), (334, 64), (334, 65), (329, 66), (327, 66), (328, 65), (330, 66), (332, 67), (326, 66), (326, 67), (327, 65), (333, 64), (323, 59), (322, 58), (331, 59), (332, 60), (333, 63), (330, 64), (331, 63), (325, 67), (333, 66), (343, 60), (345, 58), (345, 57), (342, 58), (343, 55), (342, 56), (341, 55), (339, 53), (334, 52), (333, 52), (334, 54), (333, 53), (332, 52), (331, 53), (333, 54), (334, 53), (338, 51), (339, 52), (343, 53), (344, 54), (345, 55), (344, 55), (345, 56), (346, 56), (344, 56), (343, 57), (344, 58), (343, 59), (346, 57), (347, 58), (348, 57), (343, 54), (344, 53), (344, 52), (343, 52), (344, 51), (343, 51), (342, 53), (342, 54), (341, 58), (345, 53), (346, 53), (345, 52), (345, 54), (346, 54), (346, 52), (347, 51), (346, 55), (347, 55), (347, 56), (348, 55), (348, 56), (349, 55), (350, 54), (351, 53), (350, 53), (349, 54), (349, 53), (348, 54), (348, 53), (347, 54), (347, 53), (347, 57), (349, 57), (348, 58), (349, 58), (344, 50), (345, 51), (346, 62), (347, 62), (348, 61), (347, 63), (346, 63), (345, 63), (348, 62), (348, 63), (346, 64), (346, 65), (347, 66), (347, 64), (348, 64), (349, 63), (350, 63), (349, 62), (349, 60), (348, 60), (347, 60), (349, 59), (348, 59), (350, 59), (349, 61), (350, 60), (348, 52), (350, 52), (351, 54), (351, 55), (352, 53), (352, 52), (352, 51), (351, 52), (350, 51), (349, 51), (349, 56), (346, 59), (347, 59), (351, 58), (351, 60), (350, 61), (351, 62), (351, 63), (352, 62), (353, 63), (354, 62), (355, 62), (355, 63), (356, 62), (357, 62), (358, 62), (357, 61), (356, 61), (358, 63), (357, 64), (359, 65), (359, 66), (361, 68), (362, 68), (361, 69), (360, 68), (360, 69), (360, 70), (362, 69), (367, 60), (366, 60), (365, 59), (365, 58), (364, 59), (361, 58), (362, 57), (363, 56), (364, 57), (360, 59), (360, 58), (359, 57), (360, 57), (363, 57), (359, 58), (358, 57), (357, 56), (357, 55), (358, 56), (358, 55), (359, 55), (360, 54), (361, 54), (360, 53), (360, 52), (359, 53), (361, 52), (362, 51), (361, 50), (362, 49), (361, 49), (362, 48), (361, 47), (360, 47), (360, 48), (359, 48), (359, 49), (360, 50), (362, 52), (362, 50), (363, 50), (360, 51), (359, 51), (358, 52), (357, 53), (357, 54), (356, 53), (357, 52), (356, 52), (358, 53), (359, 54), (360, 55), (359, 52), (358, 54), (356, 54), (355, 54), (355, 53), (354, 52), (354, 54), (353, 55), (354, 55), (353, 54), (353, 51), (353, 50), (352, 49), (353, 52), (354, 51), (354, 53), (354, 50), (353, 49), (353, 48), (355, 50), (355, 51), (356, 51), (354, 49), (355, 49), (356, 49), (357, 48), (356, 48), (356, 47), (357, 47), (358, 48), (358, 49), (357, 49), (360, 49), (359, 46), (358, 45), (359, 44), (360, 45), (358, 43), (357, 42), (356, 41), (358, 42), (357, 43), (359, 41), (360, 41), (361, 42), (362, 43), (361, 44), (362, 45), (361, 45), (361, 46), (362, 47), (357, 44), (358, 46), (359, 50), (359, 47), (360, 46), (361, 48), (362, 46), (363, 45), (360, 44), (360, 43), (361, 43), (362, 44), (363, 42), (364, 43), (365, 44), (365, 43), (364, 42), (365, 41), (364, 41), (362, 41), (362, 40), (362, 39), (360, 38), (361, 40), (361, 41), (360, 40), (359, 39), (359, 40), (359, 42), (358, 41), (357, 41), (355, 39), (355, 41), (356, 40), (355, 43), (352, 42), (356, 36), (355, 35), (356, 34), (355, 33), (356, 33), (357, 35), (358, 34), (357, 33), (359, 34), (360, 33), (358, 29), (357, 29), (358, 31), (358, 32), (355, 34), (355, 32), (356, 32), (356, 28), (359, 26), (360, 27), (360, 26), (361, 26), (364, 33), (363, 34), (364, 32), (365, 33), (365, 28), (368, 26), (370, 24), (370, 23), (371, 22), (370, 21), (370, 22), (371, 23), (372, 22), (373, 21), (373, 23), (374, 23), (375, 24), (376, 23), (377, 22), (378, 22), (379, 22), (377, 21), (376, 22), (377, 23), (377, 24), (378, 24), (379, 23), (379, 21), (380, 20), (381, 21), (381, 22), (382, 22), (382, 23), (381, 23), (380, 22), (378, 21), (378, 23), (377, 25), (376, 24), (376, 25), (379, 24), (380, 23), (382, 24), (381, 24), (381, 25), (380, 24), (380, 25), (380, 26), (379, 26), (379, 27), (379, 25), (378, 26), (378, 27), (378, 28), (379, 29), (380, 30), (380, 31), (377, 29), (378, 29), (379, 28), (380, 28), (381, 29), (381, 28), (380, 29), (381, 31), (378, 32), (380, 32), (381, 32), (382, 33), (383, 32), (382, 31), (383, 31), (384, 32), (385, 33), (386, 32), (385, 32), (386, 31), (387, 31), (387, 30), (387, 29), (386, 28), (385, 27), (385, 28), (384, 28), (385, 29), (384, 27), (384, 30), (385, 31), (385, 30), (384, 29), (383, 30), (384, 31), (382, 32), (381, 33), (381, 36), (378, 34), (377, 35), (376, 34), (377, 34), (376, 35), (381, 30), (382, 28), (382, 27), (382, 29), (380, 27), (381, 26), (382, 25), (378, 25), (379, 20), (380, 21), (381, 19), (380, 19), (379, 19), (380, 18), (381, 18), (382, 18), (383, 19), (383, 20), (382, 19), (383, 18), (382, 17), (384, 19), (383, 17), (384, 16), (383, 15), (383, 14), (384, 15), (385, 14), (386, 15), (386, 14), (387, 14), (388, 15), (389, 16), (389, 17), (390, 16), (391, 16), (392, 15), (392, 14), (393, 15), (393, 16), (394, 16), (394, 15), (394, 14), (395, 14), (395, 15), (396, 15), (393, 14), (392, 13), (391, 14), (391, 15), (392, 16), (390, 17), (391, 18), (391, 19), (392, 20), (393, 19), (391, 20), (392, 21), (391, 22), (390, 23), (391, 24), (392, 24), (392, 25), (391, 25), (390, 26), (390, 25), (390, 24), (392, 26), (393, 26), (394, 25), (393, 25), (394, 26), (395, 26), (394, 24), (393, 23), (392, 23), (393, 24), (395, 25), (395, 27), (396, 26), (396, 28), (396, 29), (395, 29), (395, 28), (394, 28), (393, 29), (393, 30), (394, 29), (394, 30), (393, 31), (393, 32), (392, 32), (394, 33), (393, 33), (394, 32), (395, 31), (392, 28), (392, 29), (392, 31), (391, 30), (390, 31), (391, 31), (390, 30), (390, 29), (391, 33), (391, 34), (392, 35), (390, 34), (390, 33), (391, 32), (389, 33), (389, 34), (389, 31), (389, 30), (388, 30), (388, 31), (392, 34), (393, 35), (394, 35), (395, 36), (394, 36), (394, 34), (395, 33), (395, 32), (394, 31), (393, 28), (392, 30), (396, 33), (397, 32), (396, 31), (396, 32), (398, 32), (398, 31), (399, 30), (399, 29), (399, 28), (398, 27), (399, 27), (400, 26), (400, 27), (400, 28), (398, 26), (397, 27), (398, 28), (397, 29), (396, 30), (395, 30), (397, 31), (398, 33), (399, 32), (399, 31), (400, 30), (401, 30), (400, 31), (401, 32), (400, 33), (401, 34), (401, 35), (400, 36), (399, 37), (398, 37), (397, 36), (398, 38), (397, 38), (397, 39), (396, 38), (396, 39), (397, 40), (396, 40), (397, 41), (398, 42), (397, 43), (398, 44), (399, 45), (399, 43), (400, 42), (401, 42), (399, 41), (400, 40), (401, 39), (401, 41), (400, 41), (399, 39), (399, 38), (399, 40), (398, 39), (398, 40), (396, 41), (395, 41), (396, 43), (397, 42), (398, 43), (399, 44), (400, 44), (400, 43), (400, 45), (398, 46), (397, 46), (397, 45), (396, 45), (398, 45), (397, 44), (396, 42), (395, 44), (393, 43), (392, 43), (390, 45), (390, 43), (389, 43), (391, 42), (391, 41), (390, 42), (389, 42), (390, 41), (391, 40), (390, 40), (389, 40), (388, 39), (387, 40), (386, 41), (385, 41), (385, 40), (386, 42), (387, 43), (385, 44), (384, 42), (383, 41), (382, 41), (380, 42), (379, 42), (383, 42), (382, 44), (381, 44), (383, 44), (382, 45), (379, 49), (378, 48), (377, 47), (378, 47), (378, 46), (378, 45), (379, 46), (377, 45), (376, 45), (375, 46), (375, 45), (376, 44), (375, 44), (374, 45), (373, 46), (373, 45), (373, 47), (372, 46), (371, 46), (371, 47), (370, 48), (372, 45), (370, 46), (372, 47), (372, 48), (371, 49), (370, 49), (372, 49), (371, 48), (370, 45), (370, 44), (371, 43), (371, 44), (371, 45), (370, 47), (369, 47), (372, 44), (369, 46), (369, 45), (368, 46), (372, 42), (373, 41), (373, 43), (372, 43), (374, 44), (375, 42), (377, 40), (378, 41), (378, 42), (378, 43), (378, 44), (377, 44), (379, 44), (380, 43), (382, 42), (380, 44), (381, 45), (380, 46), (379, 47), (377, 43), (375, 43), (374, 42), (374, 41), (374, 43), (373, 44), (374, 46), (374, 47), (373, 48), (374, 48), (375, 47), (375, 48), (376, 46), (376, 47), (377, 46), (375, 49), (376, 49), (377, 50), (377, 49), (376, 50), (376, 48), (377, 48), (381, 47), (380, 45), (379, 43), (381, 40), (383, 39), (384, 39), (384, 38), (383, 38), (384, 40), (386, 40), (385, 39), (384, 41), (383, 45), (383, 46), (384, 45), (387, 45), (388, 46), (389, 46), (391, 48), (391, 45), (393, 47), (393, 48), (394, 49), (395, 49), (394, 48), (390, 44), (388, 43), (391, 43), (393, 42), (393, 41), (394, 40), (394, 39), (393, 40), (392, 42), (389, 44), (388, 42), (388, 41), (387, 42), (383, 40), (381, 39), (379, 41), (385, 46), (385, 48), (383, 50), (384, 51), (384, 53), (385, 54), (384, 54), (383, 53), (381, 53), (382, 53), (383, 52), (384, 50), (386, 53), (387, 54), (388, 55), (388, 56), (389, 55), (388, 54), (389, 53), (389, 54), (390, 53), (388, 49), (388, 48), (391, 49), (390, 46), (392, 48), (394, 47), (393, 49), (392, 49), (392, 50), (391, 53), (392, 54), (393, 54), (393, 55), (394, 54), (392, 55), (391, 56), (392, 56), (391, 58), (390, 58), (390, 59), (389, 59), (389, 60), (390, 60), (391, 57), (389, 57), (388, 57), (387, 57), (387, 56), (386, 56), (385, 55), (384, 56), (385, 56), (386, 57), (386, 55), (385, 53), (387, 55), (389, 56), (389, 41), (389, 45), (385, 51), (386, 54), (383, 55), (382, 54), (380, 53), (380, 52), (380, 54), (379, 54), (378, 53), (378, 55), (377, 56), (378, 56), (378, 57), (377, 55), (377, 57), (376, 57), (375, 58), (374, 58), (374, 57), (376, 58), (377, 58), (378, 58), (377, 59), (376, 60), (376, 59), (375, 59), (375, 57), (374, 56), (373, 57), (371, 56), (370, 57), (369, 56), (368, 57), (368, 56), (368, 55), (367, 56), (367, 55), (366, 54), (366, 55), (367, 54), (367, 53), (366, 56), (365, 56), (364, 55), (364, 56), (362, 55), (361, 55), (360, 56), (359, 56), (361, 56), (362, 54), (363, 55), (362, 56), (361, 57), (359, 59), (358, 59), (357, 60), (357, 59), (356, 60), (355, 60), (355, 61), (354, 63), (354, 61), (354, 60), (353, 59), (353, 60), (353, 62), (352, 63), (352, 64), (353, 64), (351, 65), (352, 66), (353, 66), (352, 65), (351, 66), (350, 67), (349, 68), (348, 69), (346, 69), (347, 70), (345, 70), (347, 68), (345, 72), (343, 73), (342, 74), (341, 75), (341, 76), (340, 76), (340, 75), (341, 74), (342, 77), (343, 76), (344, 75), (342, 75), (343, 75), (341, 73), (341, 72), (340, 73), (339, 74), (338, 73), (339, 73), (340, 74), (344, 71), (345, 71), (347, 72), (349, 69), (351, 71), (352, 72), (351, 72), (351, 73), (351, 74), (352, 73), (352, 74), (351, 76), (352, 75), (353, 75), (353, 74), (352, 76), (353, 76), (354, 77), (355, 78), (355, 77), (354, 78), (355, 79), (356, 79), (356, 80), (357, 81), (361, 82), (363, 82), (363, 83), (363, 84), (364, 85), (363, 85), (362, 84), (363, 86), (362, 86), (360, 86), (356, 85), (360, 80), (355, 90), (355, 88), (354, 88), (350, 88), (349, 89), (350, 90), (351, 91), (352, 92), (351, 92), (349, 90), (348, 90), (350, 91), (350, 92), (349, 91), (348, 92), (347, 91), (347, 90), (346, 90), (346, 91), (345, 91), (345, 92), (344, 93), (343, 94), (344, 95), (345, 95), (346, 96), (346, 95), (347, 94), (347, 93), (346, 92), (345, 90), (344, 90), (344, 91), (343, 91), (346, 89), (344, 89), (344, 88), (343, 88), (342, 88), (343, 89), (344, 92), (343, 92), (343, 93), (343, 95), (342, 96), (343, 96), (342, 97), (341, 96), (341, 95), (342, 95), (344, 94), (345, 94), (346, 93), (346, 94), (348, 93), (348, 95), (347, 95), (342, 94), (340, 95), (340, 94), (339, 95), (339, 94), (340, 93), (341, 93), (341, 94), (340, 96), (344, 96), (343, 97), (342, 98), (341, 98), (340, 97), (341, 97), (338, 94), (337, 94), (337, 95), (338, 95), (338, 96), (339, 96), (343, 98), (342, 99), (343, 100), (344, 97), (345, 96), (346, 97), (346, 98), (347, 99), (348, 99), (349, 98), (350, 97), (350, 96), (351, 95), (351, 96), (352, 95), (353, 95), (354, 94), (354, 93), (353, 94), (353, 93), (353, 96), (354, 97), (355, 96), (355, 95), (356, 94), (356, 95), (356, 96), (357, 96), (357, 97), (358, 97), (359, 96), (360, 96), (360, 95), (359, 95), (360, 94), (359, 94), (361, 93), (361, 94), (362, 94), (361, 96), (361, 95), (362, 95), (358, 95), (358, 96), (359, 97), (362, 96), (358, 93), (358, 92), (358, 94), (357, 94), (357, 95), (355, 97), (356, 97), (358, 98), (359, 98), (362, 92), (362, 93), (361, 92), (362, 91), (361, 90), (358, 91), (361, 91), (362, 90), (363, 90), (364, 91), (363, 88), (363, 91), (364, 89), (365, 88), (365, 87), (366, 87), (366, 86), (365, 85), (364, 84), (363, 87), (362, 87), (362, 88), (364, 88), (364, 87), (367, 89), (368, 89), (370, 87), (369, 86), (368, 87), (367, 87), (367, 88), (369, 87), (368, 86), (369, 85), (369, 91), (369, 88), (368, 88), (367, 81), (367, 80), (368, 79), (367, 78), (366, 77), (365, 78), (365, 77), (364, 78), (364, 77), (364, 76), (363, 76), (364, 75), (364, 74), (363, 75), (361, 78), (362, 79), (362, 77), (361, 77), (360, 76), (359, 77), (361, 76), (362, 76), (362, 75), (362, 74), (362, 73), (363, 74), (361, 75), (360, 75), (359, 75), (359, 74), (358, 74), (357, 74), (358, 75), (356, 75), (357, 76), (358, 76), (358, 77), (355, 80), (354, 79), (350, 76), (349, 77), (348, 78), (347, 79), (346, 79), (345, 79), (343, 80), (342, 81), (341, 82), (340, 82), (341, 83), (341, 81), (342, 82), (341, 84), (342, 84), (341, 85), (340, 85), (341, 86), (340, 86), (339, 85), (340, 84), (338, 84), (337, 84), (336, 83), (337, 85), (336, 86), (336, 85), (335, 85), (334, 86), (333, 86), (332, 87), (331, 86), (332, 86), (330, 86), (330, 87), (331, 87), (330, 88), (329, 89), (330, 90), (331, 90), (330, 89), (329, 90), (330, 91), (331, 92), (330, 92), (329, 91), (328, 90), (327, 90), (326, 91), (326, 92), (325, 93), (324, 93), (323, 92), (324, 94), (325, 92), (325, 91), (325, 90), (324, 91), (323, 91), (324, 92), (326, 89), (325, 89), (324, 90), (324, 89), (323, 89), (323, 90), (322, 91), (322, 92), (325, 88), (324, 87), (325, 87), (324, 88), (323, 88), (322, 88), (321, 87), (320, 86), (320, 87), (321, 88), (326, 93), (327, 93), (327, 92), (328, 91), (331, 91), (328, 92), (328, 93), (329, 92), (327, 91), (326, 88), (327, 87), (328, 87), (327, 86), (328, 86), (329, 87), (329, 88), (328, 88), (328, 89), (327, 88), (327, 85), (328, 85), (326, 85), (327, 84), (326, 84), (326, 83), (325, 85), (324, 84), (324, 85), (325, 86), (325, 83), (325, 84), (326, 90), (327, 89), (322, 93), (321, 94), (322, 94), (323, 94), (324, 95), (325, 95), (325, 94), (327, 94), (327, 95), (327, 96), (326, 95), (328, 94), (328, 95), (329, 94), (329, 93), (326, 94), (322, 90), (326, 87), (329, 85), (328, 84), (327, 83), (327, 82), (327, 81), (328, 82), (329, 81), (330, 82), (331, 81), (332, 80), (333, 80), (333, 79), (331, 80), (333, 81), (334, 81), (334, 80), (335, 79), (336, 78), (337, 77), (338, 76), (339, 75), (339, 77), (340, 77), (340, 78), (341, 78), (345, 80), (344, 76), (346, 77), (340, 79), (341, 79), (342, 79), (348, 85), (346, 85), (347, 86), (348, 86), (349, 85), (349, 84), (351, 81), (350, 82), (351, 80), (352, 78), (351, 77), (350, 78), (349, 79), (350, 79), (350, 80), (348, 80), (349, 80), (350, 81), (349, 82), (350, 77), (349, 78), (351, 84), (350, 85), (351, 85), (353, 82), (352, 93), (351, 94), (350, 94), (349, 97), (348, 98), (350, 99), (349, 100), (348, 100), (349, 99), (350, 100), (350, 101), (349, 101), (348, 102), (347, 103), (347, 102), (346, 101), (347, 100), (346, 99), (345, 100), (344, 100), (343, 99), (342, 100), (341, 99), (340, 100), (341, 100), (340, 99), (339, 98), (339, 97), (340, 98), (339, 99), (339, 100), (338, 99), (341, 101), (342, 101), (339, 101), (339, 102), (338, 102), (338, 101), (340, 101), (338, 103), (338, 104), (338, 105), (337, 106), (337, 107), (336, 106), (337, 105), (339, 105), (340, 104), (341, 103), (340, 103), (341, 104), (342, 103), (341, 102), (342, 104), (343, 103), (343, 104), (344, 103), (345, 104), (344, 104), (343, 105), (344, 102), (345, 103), (345, 102), (344, 101), (345, 101), (343, 101), (345, 99), (347, 97), (345, 97), (342, 93), (345, 87), (344, 87), (345, 86), (344, 84), (343, 85), (342, 86), (342, 87), (343, 87), (344, 86), (346, 86), (346, 87), (347, 87), (346, 88), (342, 89), (341, 90), (340, 89), (339, 88), (340, 88), (341, 87), (341, 88), (345, 88), (345, 89), (347, 89), (347, 88), (348, 89), (349, 88), (348, 88), (349, 87), (349, 86), (348, 87), (350, 86), (351, 86), (352, 86), (354, 92), (349, 92), (348, 91), (347, 92), (343, 90), (342, 90), (342, 91), (342, 92), (341, 92), (340, 92), (340, 91), (341, 91), (339, 93), (340, 90), (339, 89), (339, 87), (338, 88), (337, 89), (338, 89), (337, 88), (336, 88), (335, 87), (333, 85), (332, 84), (331, 84), (331, 83), (331, 82), (332, 79), (331, 79), (333, 78), (332, 81), (332, 82), (332, 78), (333, 77), (334, 78), (335, 78), (334, 77), (332, 77), (333, 76), (331, 78), (331, 75), (332, 75), (330, 76), (330, 74), (330, 77), (329, 76), (329, 78), (330, 78), (330, 79), (329, 80), (334, 82), (335, 83), (336, 82), (336, 84), (337, 83), (335, 82), (335, 80), (334, 79), (336, 79), (325, 66), (324, 65), (334, 60), (334, 59), (335, 61), (332, 68), (324, 69), (330, 63), (335, 71), (335, 72), (334, 73), (335, 74), (336, 74), (337, 73), (336, 72), (337, 72), (338, 72), (338, 71), (335, 73), (336, 71), (334, 74), (336, 73), (337, 74), (338, 75), (337, 75), (336, 75), (323, 66), (322, 65), (321, 62), (320, 60), (320, 59), (319, 59), (320, 58), (317, 49), (314, 49), (314, 50), (315, 49), (315, 48), (314, 48), (314, 47), (307, 47), (306, 47), (307, 48), (287, 66), (283, 70), (284, 66), (284, 65), (287, 69), (289, 70), (290, 68), (289, 75), (290, 75), (291, 80), (292, 80), (292, 81), (293, 82), (291, 81), (290, 82), (291, 83), (290, 84), (291, 84), (292, 83), (290, 83), (291, 85), (290, 86), (289, 86), (290, 85), (288, 87), (287, 88), (288, 89), (289, 87), (288, 86), (288, 85), (287, 86), (288, 88), (290, 87), (289, 88), (289, 89), (287, 87), (286, 86), (286, 88), (285, 88), (285, 87), (286, 87), (287, 89), (284, 87), (285, 86), (284, 86), (285, 85), (286, 84), (287, 84), (289, 84), (289, 83), (289, 85), (293, 84), (293, 83), (292, 84), (292, 85), (288, 84), (287, 85), (286, 85), (292, 82), (293, 81), (294, 82), (295, 83), (295, 82), (294, 83), (291, 79), (289, 80), (288, 80), (287, 80), (286, 81), (285, 80), (284, 81), (285, 81), (285, 82), (285, 83), (286, 82), (285, 84), (284, 83), (284, 82), (287, 81), (288, 81), (290, 81), (291, 82), (293, 80), (288, 83), (289, 82), (288, 82), (289, 81), (287, 79), (287, 78), (286, 78), (286, 80), (286, 79), (288, 78), (288, 79), (290, 80), (294, 84), (294, 85), (296, 82), (297, 83), (297, 82), (296, 81), (295, 81), (294, 81), (291, 88), (292, 89), (291, 90), (292, 90), (293, 89), (294, 88), (294, 87), (293, 88), (294, 89), (295, 87), (295, 86), (296, 85), (297, 86), (298, 87), (297, 84), (296, 83), (296, 84), (297, 85), (298, 83), (298, 84), (298, 85), (299, 84), (300, 83), (300, 84), (301, 85), (300, 85), (301, 82), (302, 82), (301, 83), (302, 83), (303, 83), (304, 84), (303, 84), (302, 84), (301, 84), (302, 85), (301, 86), (304, 83), (299, 83), (296, 86), (294, 86), (293, 86), (293, 87), (295, 88), (296, 89), (295, 90), (296, 90), (297, 90), (298, 91), (298, 90), (297, 91), (297, 92), (296, 91), (295, 89), (297, 89), (296, 88), (298, 89), (298, 88), (297, 88), (296, 87), (297, 87), (299, 86), (292, 88), (291, 86), (294, 80), (297, 81), (299, 82), (299, 85), (298, 86), (302, 86), (303, 85), (305, 84), (306, 83), (305, 83), (307, 86), (308, 87), (309, 88), (309, 87), (311, 86), (306, 82), (303, 81), (301, 79), (299, 77), (307, 80), (307, 83), (306, 84), (308, 81), (305, 85), (306, 85), (312, 86), (313, 88), (316, 88), (316, 89), (315, 88), (317, 84), (319, 83), (321, 81), (321, 80), (321, 79), (320, 78), (320, 79), (319, 79), (306, 71), (305, 71), (292, 69), (293, 67), (291, 68), (291, 73), (291, 74), (289, 74), (290, 74), (290, 73), (290, 72), (289, 73), (289, 72), (285, 76), (283, 76), (284, 77), (282, 77), (283, 77), (283, 78), (284, 79), (285, 78), (285, 79), (284, 80), (283, 79), (283, 80), (286, 83), (287, 82), (287, 83), (284, 85), (282, 85), (283, 86), (286, 77), (285, 77), (284, 76), (283, 75), (282, 76), (281, 76), (281, 77), (282, 78), (280, 76), (284, 78), (286, 73), (281, 78), (281, 79), (280, 80), (280, 79), (281, 80), (279, 79), (279, 81), (278, 81), (277, 82), (276, 85), (277, 87), (278, 88), (279, 89), (280, 88), (281, 89), (281, 90), (279, 91), (281, 99), (281, 100), (282, 103), (284, 100), (283, 96), (283, 95), (283, 94), (284, 94), (283, 93), (285, 93), (282, 95), (282, 93), (286, 94), (284, 90), (283, 90), (282, 90), (281, 91), (280, 90), (279, 90), (280, 89), (282, 92), (283, 91), (279, 98), (277, 92), (276, 92), (275, 93), (277, 94), (274, 92), (274, 91), (273, 91), (274, 93), (274, 89), (275, 89), (276, 89), (275, 88), (275, 90), (276, 87), (277, 88), (273, 94), (273, 95), (274, 90), (275, 91), (276, 90), (272, 93), (276, 91), (274, 96), (272, 90), (272, 94), (272, 96), (273, 97), (273, 96), (272, 95), (272, 97), (274, 88), (272, 88), (273, 86), (274, 86), (273, 87), (272, 86), (272, 87), (272, 98), (272, 99), (277, 91), (279, 87), (280, 87), (281, 88), (282, 88), (281, 87), (282, 86), (282, 82), (283, 83), (280, 86), (281, 86), (282, 87), (275, 83), (273, 85), (275, 85), (275, 82), (282, 81), (283, 81), (277, 81), (283, 87), (284, 84), (282, 80), (284, 88), (283, 88), (280, 100), (279, 99), (282, 105), (281, 105), (281, 106), (282, 106), (282, 107), (283, 108), (283, 107), (284, 107), (284, 108), (283, 104), (278, 104), (279, 105), (280, 106), (281, 107), (279, 106), (278, 106), (278, 107), (277, 107), (277, 106), (277, 105), (276, 106), (278, 105), (277, 104), (274, 103), (273, 102), (272, 103), (272, 102), (275, 100), (276, 101), (276, 102), (276, 100), (279, 107), (280, 107), (280, 108), (281, 109), (280, 109), (281, 110), (279, 109), (280, 110), (281, 111), (278, 110), (275, 116), (276, 117), (275, 117), (275, 118), (276, 121), (275, 122), (276, 123), (274, 123), (275, 124), (277, 123), (276, 122), (276, 120), (273, 122), (272, 120), (272, 119), (274, 117), (272, 117), (273, 116), (273, 114), (272, 114), (272, 113), (275, 111), (274, 110), (275, 109), (276, 110), (277, 109), (277, 110), (278, 111), (279, 112), (280, 112), (281, 112), (279, 111), (278, 112), (277, 112), (275, 110), (282, 111), (283, 110), (282, 109), (283, 109), (284, 109), (284, 110), (282, 108), (284, 106), (283, 106), (285, 109), (286, 110), (286, 111), (286, 112), (286, 113), (286, 115), (285, 114), (286, 114), (290, 117), (290, 115), (292, 114), (293, 114), (293, 113), (294, 114), (293, 112), (294, 111), (295, 111), (295, 110), (294, 110), (294, 112), (289, 118), (291, 120), (290, 120), (295, 117), (294, 119), (295, 119), (300, 119), (301, 120), (296, 119), (294, 116), (293, 115), (294, 115), (293, 111), (292, 110), (291, 109), (290, 108), (291, 108), (290, 107), (289, 108), (289, 109), (289, 110), (288, 110), (289, 111), (288, 111), (288, 112), (287, 113), (288, 114), (288, 113), (287, 112), (285, 113), (284, 114), (283, 115), (284, 117), (283, 116), (282, 114), (282, 115), (273, 125), (272, 123), (279, 122), (278, 121), (280, 123), (279, 123), (277, 125), (277, 126), (277, 127), (278, 127), (279, 127), (280, 126), (279, 125), (278, 125), (272, 126), (273, 126), (274, 126), (274, 127), (275, 126), (274, 125), (272, 127), (273, 127), (273, 128), (272, 122), (274, 114), (273, 115), (272, 116), (273, 117), (272, 118), (275, 127), (276, 126), (281, 119), (282, 116), (285, 119), (281, 124), (281, 125), (280, 124), (282, 125), (282, 126), (278, 128), (279, 126), (278, 126), (277, 128), (276, 128), (278, 129), (276, 129), (276, 127), (278, 124), (284, 125), (285, 125), (283, 126), (283, 127), (283, 128), (282, 129), (282, 130), (283, 130), (284, 129), (285, 129), (285, 130), (286, 131), (285, 132), (285, 131), (286, 130), (284, 130), (283, 131), (284, 131), (287, 129), (286, 128), (282, 132), (281, 132), (282, 133), (283, 134), (283, 133), (284, 134), (283, 132), (283, 129), (282, 131), (285, 133), (286, 132), (287, 133), (288, 132), (288, 133), (289, 133), (287, 132), (288, 134), (287, 134), (286, 133), (287, 131), (287, 130), (288, 130), (288, 131), (289, 130), (288, 129), (289, 134), (289, 132), (290, 133), (290, 132), (291, 131), (292, 132), (293, 133), (294, 134), (293, 134), (295, 134), (294, 133), (294, 132), (293, 132), (294, 131), (295, 130), (294, 129), (295, 128), (296, 128), (296, 127), (295, 127), (294, 126), (293, 124), (294, 123), (293, 123), (293, 121), (292, 123), (292, 124), (293, 125), (293, 127), (292, 127), (291, 127), (291, 126), (291, 128), (290, 127), (289, 128), (289, 129), (289, 127), (290, 128), (290, 126), (289, 126), (290, 125), (289, 124), (291, 124), (290, 123), (289, 122), (289, 121), (288, 121), (288, 122), (287, 123), (287, 124), (284, 126), (286, 125), (282, 127), (281, 127), (281, 126), (280, 127), (274, 129), (272, 129), (272, 128), (275, 128), (274, 128), (272, 130), (273, 129), (273, 130), (274, 131), (274, 132), (275, 131), (275, 130), (276, 130), (276, 131), (274, 130), (272, 131), (275, 129), (277, 130), (277, 129), (278, 130), (279, 129), (280, 130), (281, 129), (281, 130), (280, 131), (281, 131), (280, 132), (279, 131), (280, 133), (281, 133), (282, 134), (279, 133), (280, 134), (279, 132), (278, 131), (279, 128), (282, 128), (280, 128), (279, 130), (280, 129), (277, 131), (277, 132), (278, 132), (275, 132), (274, 133), (273, 133), (272, 134), (275, 134), (276, 134), (277, 134), (277, 133), (276, 132), (276, 133), (278, 134), (273, 134), (272, 133), (272, 132), (273, 131), (273, 132), (283, 114), (280, 111), (279, 110), (278, 108), (279, 108), (282, 110), (282, 112), (282, 113), (281, 108), (278, 109), (274, 134), (275, 133), (281, 128), (284, 127), (285, 128), (285, 127), (286, 129), (288, 128), (293, 128), (294, 128), (292, 128), (292, 126), (294, 127), (293, 126), (293, 129), (292, 129), (291, 123), (291, 125), (290, 124), (289, 123), (290, 122), (288, 120), (287, 120), (290, 121), (287, 121), (285, 117), (289, 113), (290, 112), (291, 113), (291, 112), (290, 113), (289, 120), (284, 113), (285, 111), (287, 111), (287, 110), (285, 112), (284, 112), (283, 112), (283, 111), (280, 105), (279, 124), (272, 115), (279, 134), (278, 133), (280, 125), (282, 118), (283, 113), (285, 110), (284, 111), (277, 108), (276, 109), (272, 112), (272, 111), (273, 113), (272, 109), (275, 108), (276, 108), (276, 107), (275, 107), (275, 106), (293, 120), (295, 116), (296, 115), (297, 115), (298, 114), (296, 116), (295, 114), (294, 113), (298, 125), (297, 125), (292, 125), (293, 130), (295, 131), (295, 132), (292, 134), (291, 134), (291, 133), (290, 134), (291, 132), (292, 133), (292, 131), (296, 134), (297, 133), (296, 132), (295, 133), (296, 133), (298, 133), (298, 132), (299, 133), (297, 132), (296, 131), (297, 131), (297, 130), (298, 131), (299, 130), (300, 129), (301, 129), (302, 130), (302, 131), (303, 131), (302, 132), (304, 131), (305, 132), (306, 133), (306, 132), (306, 131), (305, 131), (305, 130), (306, 130), (306, 129), (304, 130), (303, 129), (300, 130), (300, 131), (299, 132), (301, 131), (297, 129), (298, 129), (299, 131), (298, 130), (300, 132), (296, 130), (295, 129), (294, 130), (293, 131), (292, 130), (291, 130), (291, 129), (288, 127), (288, 126), (287, 125), (287, 126), (286, 126), (285, 126), (288, 125), (290, 130), (290, 129), (289, 131), (287, 128), (290, 131), (285, 134), (284, 128), (287, 127), (286, 127), (288, 124), (289, 125), (291, 122), (283, 125), (296, 129), (300, 133), (299, 134), (298, 134), (297, 134), (300, 134), (301, 133), (301, 134), (301, 132), (302, 133), (301, 130), (302, 129), (303, 130), (304, 132), (305, 133), (306, 134), (305, 134), (304, 133), (304, 129), (304, 128), (303, 128), (304, 127), (305, 127), (305, 126), (306, 125), (307, 124), (307, 123), (306, 123), (306, 120), (304, 121), (305, 122), (300, 121), (295, 120), (297, 126), (299, 125), (298, 126), (297, 127), (298, 127), (299, 127), (300, 126), (301, 127), (301, 126), (300, 127), (302, 127), (303, 126), (300, 128), (301, 128), (304, 124), (301, 119), (304, 122), (304, 123), (305, 123), (305, 121), (305, 118), (304, 120), (302, 117), (301, 116), (301, 118), (302, 118), (303, 117), (303, 119), (299, 124), (301, 125), (299, 126), (299, 128), (299, 129), (303, 132), (303, 133), (304, 134), (303, 134), (286, 134), (284, 133), (284, 132), (282, 123), (277, 99), (272, 105), (273, 104), (282, 99), (286, 102), (287, 103), (277, 97), (277, 98), (276, 97), (278, 97), (277, 96), (281, 75), (282, 74), (283, 74), (282, 75), (283, 73), (284, 73), (284, 71), (282, 73), (282, 72), (283, 71), (286, 76), (281, 74), (281, 73), (283, 64), (294, 73), (297, 77), (288, 71), (286, 70), (283, 69), (284, 68), (283, 68), (292, 72), (295, 68), (276, 66), (276, 68), (275, 68), (276, 69), (275, 69), (277, 68), (278, 68), (282, 70), (281, 70), (282, 69), (281, 68), (278, 69), (277, 69), (277, 70), (279, 70), (275, 66), (273, 35), (274, 35), (273, 30), (278, 75), (277, 75), (276, 77), (275, 76), (275, 77), (275, 78), (276, 78), (274, 77), (274, 76), (276, 76), (276, 75), (275, 74), (275, 73), (274, 73), (276, 74), (277, 74), (274, 75), (273, 75), (274, 74), (273, 74), (273, 76), (273, 77), (272, 82), (273, 83), (274, 84), (272, 84), (272, 77), (272, 76), (279, 80), (277, 78), (276, 79), (276, 81), (275, 80), (272, 85), (282, 89), (283, 89), (283, 85), (292, 86), (291, 87), (291, 89), (291, 91), (290, 91), (289, 90), (289, 79), (292, 73), (292, 74), (282, 79), (278, 77), (279, 76), (300, 86), (303, 87), (302, 87), (303, 88), (304, 87), (305, 88), (304, 89), (303, 90), (305, 87), (306, 87), (306, 89), (306, 88), (305, 90), (306, 90), (305, 91), (304, 90), (302, 89), (304, 91), (303, 92), (303, 93), (304, 93), (305, 92), (305, 93), (304, 92), (304, 94), (305, 95), (305, 94), (306, 94), (306, 93), (307, 92), (308, 91), (309, 90), (309, 89), (308, 88), (307, 89), (308, 89), (310, 89), (310, 88), (308, 90), (311, 89), (311, 90), (312, 89), (312, 88), (313, 89), (310, 90), (309, 91), (310, 92), (311, 91), (312, 91), (313, 92), (314, 90), (312, 87), (311, 88), (313, 90), (312, 90), (313, 91), (314, 92), (313, 93), (317, 90), (314, 95), (313, 95), (312, 95), (311, 94), (311, 93), (310, 94), (311, 95), (312, 94), (314, 93), (317, 89), (318, 88), (319, 89), (320, 90), (319, 91), (319, 90), (320, 89), (320, 88), (321, 89), (322, 89), (321, 91), (321, 92), (321, 93), (320, 93), (320, 92), (318, 92), (316, 96), (316, 97), (317, 97), (317, 98), (316, 99), (316, 98), (315, 99), (316, 100), (315, 100), (314, 99), (312, 100), (314, 100), (313, 98), (310, 100), (309, 101), (309, 100), (309, 99), (310, 99), (311, 101), (309, 98), (310, 97), (311, 98), (311, 97), (310, 98), (308, 99), (307, 99), (306, 99), (307, 98), (308, 98), (308, 101), (310, 101), (309, 102), (310, 102), (310, 103), (309, 103), (311, 102), (311, 103), (310, 104), (311, 105), (311, 104), (312, 103), (313, 102), (313, 104), (314, 104), (310, 105), (309, 105), (309, 104), (308, 102), (308, 103), (307, 102), (307, 101), (307, 103), (307, 104), (306, 103), (306, 102), (307, 100), (306, 101), (305, 102), (306, 100), (308, 100), (306, 98), (305, 97), (306, 96), (307, 95), (308, 94), (307, 93), (306, 92), (306, 91), (307, 90), (307, 91), (303, 94), (304, 95), (302, 94), (301, 95), (301, 94), (300, 94), (300, 95), (299, 94), (299, 93), (298, 94), (297, 95), (296, 94), (296, 95), (297, 94), (298, 93), (298, 92), (299, 91), (299, 90), (299, 89), (300, 89), (301, 89), (300, 90), (301, 88), (301, 90), (300, 88), (299, 87), (301, 87), (295, 85), (300, 87), (300, 82), (299, 88), (301, 91), (301, 92), (300, 91), (302, 91), (302, 90), (304, 88), (303, 89), (303, 91), (308, 92), (307, 94), (305, 89), (305, 86), (304, 86), (300, 92), (303, 96), (304, 96), (304, 97), (305, 98), (306, 97), (305, 96), (302, 95), (302, 97), (301, 96), (300, 96), (301, 97), (302, 96), (303, 97), (304, 98), (303, 95), (308, 97), (307, 97), (307, 96), (309, 96), (309, 95), (310, 95), (311, 96), (309, 97), (305, 99), (299, 96), (298, 96), (297, 96), (296, 97), (295, 97), (294, 97), (294, 98), (295, 98), (294, 99), (295, 96), (294, 96), (294, 95), (293, 97), (292, 96), (291, 95), (291, 96), (292, 94), (292, 93), (293, 93), (294, 92), (295, 91), (293, 90), (293, 91), (292, 92), (290, 92), (291, 92), (294, 90), (290, 88), (280, 77), (280, 75), (280, 78), (279, 77), (276, 73), (278, 74), (273, 78), (275, 75), (285, 103), (285, 106), (285, 105), (278, 99), (288, 102), (287, 101), (287, 100), (288, 101), (288, 99), (288, 97), (289, 97), (289, 98), (289, 99), (288, 96), (289, 95), (290, 94), (289, 93), (289, 94), (288, 94), (288, 93), (286, 95), (288, 95), (290, 95), (288, 92), (287, 91), (287, 90), (288, 90), (290, 89), (286, 91), (289, 91), (288, 91), (292, 91), (289, 92), (290, 93), (287, 95), (291, 93), (293, 92), (293, 94), (294, 93), (295, 92), (294, 91), (303, 86), (307, 88), (310, 91), (311, 92), (310, 93), (309, 93), (308, 93), (309, 92), (310, 96), (308, 95), (293, 79), (291, 94), (290, 96), (289, 96), (292, 95), (291, 97), (290, 97), (291, 98), (290, 98), (287, 96), (295, 84), (304, 79), (301, 81), (306, 58), (305, 68), (272, 75), (272, 74), (273, 73), (274, 72), (273, 72), (272, 73), (272, 72), (272, 71), (272, 70), (273, 71), (273, 70), (319, 66), (320, 65), (347, 65), (348, 65), (349, 65), (350, 65), (349, 66), (348, 66), (345, 65), (346, 66), (347, 67), (348, 67), (349, 64), (350, 64), (351, 64), (350, 66), (351, 67), (352, 67), (353, 67), (354, 68), (353, 69), (352, 70), (354, 67), (355, 66), (354, 65), (355, 65), (356, 64), (355, 64), (356, 65), (357, 63), (354, 64), (353, 65), (349, 67), (346, 68), (353, 61), (356, 63), (358, 60), (358, 61), (357, 58), (358, 58), (355, 52), (355, 55), (353, 53), (356, 56), (356, 55), (355, 56), (353, 56), (353, 57), (354, 56), (352, 55), (354, 57), (352, 57), (352, 59), (352, 58), (353, 58), (354, 58), (351, 57), (351, 56), (352, 54), (347, 52), (343, 49), (344, 49), (345, 49), (346, 48), (347, 49), (346, 50), (346, 51), (345, 50), (349, 52), (351, 50), (350, 50), (349, 49), (350, 49), (350, 48), (349, 48), (349, 47), (348, 47), (348, 48), (347, 47), (347, 46), (349, 46), (348, 46), (349, 44), (351, 44), (352, 43), (353, 44), (349, 42), (348, 39), (347, 34), (347, 39), (335, 37), (335, 36), (335, 35), (337, 36), (339, 35), (338, 29), (338, 25), (338, 22), (338, 21), (339, 22), (340, 23), (338, 23), (344, 16), (345, 16), (346, 15), (347, 13), (346, 13), (338, 16), (338, 15), (338, 14), (339, 14), (341, 15), (337, 14), (338, 5), (338, 9), (332, 14), (327, 14), (326, 13), (325, 11), (326, 5), (314, 6), (313, 5), (313, 6), (314, 7), (314, 5), (312, 5), (312, 6), (311, 6), (313, 8), (312, 10), (318, 15), (321, 12), (322, 11), (321, 11), (320, 11), (322, 10), (319, 10), (319, 11), (319, 16), (326, 14), (325, 14), (326, 15), (307, 57), (312, 48), (315, 46), (316, 46), (317, 46), (317, 45), (314, 40), (313, 39), (313, 40), (272, 30), (272, 31), (286, 92), (292, 87), (304, 85), (316, 82), (319, 64), (311, 59), (323, 47), (337, 42), (336, 36), (336, 35), (338, 36), (337, 35), (334, 37), (333, 38), (332, 38), (332, 39), (332, 40), (326, 41), (327, 40), (331, 38), (332, 37), (334, 36), (333, 40), (335, 42), (321, 43), (321, 42), (326, 35), (325, 36), (325, 35), (324, 36), (324, 35), (343, 50), (348, 50), (347, 50), (346, 49), (350, 57), (350, 58), (351, 59), (350, 62), (352, 61), (352, 60), (351, 61), (355, 59), (355, 58), (356, 59), (357, 57), (358, 64), (359, 64), (354, 66), (355, 67), (356, 67), (346, 71), (348, 72), (350, 69), (348, 68), (356, 58), (356, 57), (348, 51), (349, 50), (325, 78), (323, 78), (322, 79), (321, 77), (321, 78), (322, 78), (322, 80), (322, 81), (320, 80), (319, 77), (319, 76), (320, 77), (324, 79), (325, 80), (324, 81), (326, 86), (323, 87), (318, 90), (321, 90), (320, 91), (319, 92), (317, 87), (318, 87), (313, 75), (319, 80), (320, 83), (321, 83), (322, 82), (323, 81), (324, 82), (323, 82), (325, 82), (324, 83), (323, 83), (328, 83), (329, 82), (329, 83), (328, 81), (328, 80), (330, 81), (336, 80), (337, 80), (337, 81), (338, 82), (337, 82), (338, 83), (339, 83), (339, 82), (338, 81), (336, 81), (335, 81), (334, 83), (333, 82), (333, 83), (325, 71), (321, 58), (321, 59), (318, 47), (347, 48), (348, 49), (346, 45), (350, 47), (323, 77), (317, 86), (311, 87), (309, 94), (312, 93), (307, 87), (306, 86), (301, 93), (302, 93), (302, 98), (301, 98), (300, 98), (300, 97), (299, 98), (298, 99), (300, 99), (299, 97), (298, 97), (299, 100), (300, 100), (300, 101), (301, 100), (302, 100), (301, 101), (302, 101), (303, 101), (301, 102), (303, 100), (302, 99), (303, 98), (298, 95), (299, 95), (299, 92), (302, 92), (306, 95), (296, 92), (302, 88), (300, 93), (297, 97), (297, 98), (297, 99), (298, 98), (293, 95), (296, 98), (295, 99), (303, 99), (301, 99), (304, 99), (305, 100), (305, 101), (305, 104), (304, 103), (304, 104), (303, 104), (305, 103), (303, 103), (303, 105), (304, 105), (306, 104), (307, 105), (308, 106), (308, 105), (310, 106), (311, 106), (311, 107), (310, 107), (311, 108), (312, 108), (312, 107), (313, 109), (314, 110), (315, 111), (315, 110), (313, 110), (312, 111), (312, 109), (309, 108), (310, 108), (309, 107), (310, 109), (309, 109), (309, 110), (310, 110), (309, 111), (308, 110), (308, 109), (308, 108), (307, 108), (307, 107), (306, 108), (306, 107), (305, 108), (304, 109), (305, 109), (306, 110), (307, 111), (307, 112), (308, 112), (309, 113), (310, 113), (313, 114), (313, 113), (310, 115), (310, 114), (312, 112), (313, 112), (310, 116), (309, 114), (308, 113), (308, 115), (314, 113), (314, 115), (314, 114), (314, 112), (315, 112), (316, 112), (315, 113), (316, 114), (319, 116), (318, 114), (315, 115), (325, 117), (328, 114), (324, 113), (322, 112), (326, 111), (326, 110), (327, 112), (328, 113), (329, 114), (326, 118), (325, 118), (324, 119), (324, 122), (325, 121), (323, 123), (322, 124), (322, 123), (321, 121), (323, 119), (328, 120), (327, 119), (326, 119), (323, 120), (325, 120), (326, 120), (327, 120), (327, 121), (328, 121), (328, 123), (328, 124), (329, 124), (330, 125), (330, 124), (329, 125), (331, 124), (332, 125), (333, 124), (334, 124), (334, 123), (335, 123), (335, 124), (333, 123), (332, 122), (331, 123), (327, 131), (326, 132), (327, 133), (328, 134), (330, 132), (331, 131), (332, 133), (341, 131), (341, 129), (339, 129), (342, 128), (343, 127), (340, 127), (343, 125), (342, 124), (343, 123), (340, 122), (340, 120), (339, 122), (338, 123), (336, 122), (329, 122), (328, 122), (328, 117), (330, 122), (330, 123), (334, 122), (336, 121), (337, 119), (338, 119), (338, 118), (337, 121), (336, 118), (340, 119), (342, 120), (343, 120), (343, 121), (344, 120), (344, 122), (343, 124), (344, 126), (341, 125), (343, 128), (344, 129), (342, 132), (343, 132), (343, 133), (344, 133), (344, 132), (345, 133), (345, 134), (346, 132), (346, 131), (347, 128), (347, 129), (348, 129), (347, 130), (346, 133), (347, 132), (348, 132), (348, 131), (349, 131), (349, 130), (350, 129), (351, 128), (352, 127), (353, 127), (353, 126), (352, 126), (351, 126), (351, 125), (350, 125), (352, 124), (353, 125), (353, 124), (352, 125), (349, 125), (348, 125), (347, 126), (347, 125), (348, 126), (347, 127), (346, 130), (345, 132), (343, 131), (346, 126), (345, 125), (344, 124), (345, 123), (345, 124), (345, 126), (344, 125), (346, 125), (348, 124), (348, 123), (347, 123), (349, 123), (350, 123), (349, 124), (350, 122), (351, 123), (351, 122), (352, 123), (353, 123), (351, 121), (352, 121), (353, 121), (352, 122), (353, 122), (354, 120), (353, 119), (353, 118), (352, 118), (353, 120), (352, 119), (354, 117), (355, 116), (355, 115), (356, 116), (355, 117), (353, 117), (353, 116), (354, 115), (354, 114), (355, 113), (355, 114), (356, 114), (355, 112), (354, 112), (353, 111), (353, 110), (352, 109), (351, 110), (351, 111), (350, 111), (350, 110), (352, 111), (353, 109), (354, 109), (355, 110), (355, 109), (356, 110), (357, 109), (358, 108), (358, 109), (356, 108), (356, 109), (359, 109), (356, 111), (357, 111), (356, 112), (354, 110), (354, 111), (353, 112), (356, 113), (357, 114), (355, 111), (358, 112), (357, 112), (358, 111), (359, 112), (359, 111), (358, 110), (357, 107), (358, 107), (358, 106), (357, 105), (356, 106), (355, 105), (354, 106), (355, 106), (354, 105), (355, 104), (356, 104), (356, 103), (354, 104), (355, 103), (355, 102), (355, 101), (355, 100), (354, 100), (354, 101), (354, 102), (356, 102), (356, 101), (357, 101), (357, 102), (357, 103), (358, 102), (359, 101), (356, 100), (356, 99), (355, 98), (356, 98), (357, 98), (357, 93), (356, 93), (355, 94), (354, 96), (354, 95), (352, 96), (353, 97), (353, 98), (352, 99), (351, 98), (349, 96), (348, 97), (347, 98), (347, 96), (348, 96), (352, 97), (352, 98), (354, 99), (354, 98), (353, 99), (352, 100), (352, 101), (351, 101), (351, 100), (350, 102), (351, 103), (345, 98), (344, 99), (342, 102), (340, 102), (344, 105), (343, 106), (344, 106), (343, 107), (342, 106), (342, 105), (341, 106), (341, 105), (342, 107), (344, 107), (343, 108), (342, 108), (341, 109), (340, 110), (341, 110), (340, 109), (340, 108), (340, 111), (339, 110), (338, 109), (337, 110), (340, 113), (341, 112), (341, 111), (340, 112), (341, 113), (340, 117), (341, 117), (342, 112), (342, 113), (343, 114), (344, 113), (345, 114), (344, 115), (345, 116), (345, 115), (344, 116), (344, 117), (343, 117), (342, 118), (342, 116), (341, 116), (335, 113), (335, 112), (334, 111), (334, 110), (330, 109), (330, 108), (329, 108), (329, 109), (328, 107), (327, 107), (328, 108), (329, 107), (327, 108), (327, 109), (326, 109), (326, 107), (326, 108), (327, 111), (328, 111), (327, 110), (328, 110), (329, 111), (329, 112), (328, 112), (327, 113), (322, 119), (322, 120), (315, 120), (317, 127), (321, 125), (321, 124), (321, 123), (323, 122), (327, 125), (328, 126), (327, 132), (326, 133), (326, 134), (325, 134), (324, 134), (322, 134), (321, 134), (316, 131), (317, 130), (315, 134), (314, 133), (313, 134), (314, 134), (312, 134), (311, 134), (308, 130), (311, 131), (312, 132), (313, 133), (308, 131), (302, 134), (307, 134), (312, 133), (307, 129), (308, 127), (310, 129), (308, 125), (310, 125), (306, 127), (306, 128), (307, 130), (307, 131), (325, 133), (325, 132), (323, 130), (322, 130), (320, 131), (323, 129), (319, 128), (322, 129), (315, 127), (310, 127), (311, 127), (319, 131), (311, 126), (311, 124), (305, 128), (298, 128), (292, 122), (292, 113), (295, 113), (295, 112), (295, 109), (296, 110), (296, 111), (296, 109), (295, 108), (295, 107), (294, 106), (293, 105), (295, 106), (294, 105), (296, 107), (296, 108), (297, 108), (297, 107), (298, 107), (299, 108), (299, 109), (298, 109), (300, 109), (300, 107), (301, 106), (300, 106), (299, 107), (298, 108), (297, 109), (296, 112), (297, 110), (292, 112), (291, 111), (292, 111), (290, 111), (291, 110), (293, 109), (294, 109), (293, 110), (296, 113), (299, 114), (300, 111), (301, 111), (302, 114), (303, 114), (304, 115), (303, 116), (304, 116), (303, 115), (302, 115), (301, 115), (305, 116), (303, 113), (304, 112), (304, 111), (303, 110), (300, 115), (299, 113), (300, 110), (299, 112), (301, 114), (292, 109), (293, 108), (294, 108), (292, 107), (291, 106), (291, 105), (292, 106), (292, 105), (293, 106), (294, 104), (295, 105), (295, 104), (296, 106), (297, 106), (298, 105), (299, 106), (300, 108), (301, 109), (302, 109), (302, 108), (301, 107), (302, 107), (303, 108), (306, 109), (307, 110), (308, 111), (309, 112), (308, 114), (310, 112), (311, 112), (310, 117), (307, 115), (306, 117), (305, 117), (306, 116), (305, 115), (305, 114), (304, 114), (305, 113), (306, 112), (305, 112), (304, 113), (302, 116), (301, 117), (299, 117), (299, 116), (304, 117), (307, 116), (306, 115), (305, 124), (306, 124), (295, 115), (296, 114), (289, 112), (290, 110), (285, 108), (285, 107), (287, 105), (286, 106), (286, 107), (287, 108), (288, 109), (288, 108), (287, 109), (286, 109), (286, 108), (288, 123), (287, 106), (288, 107), (289, 107), (288, 100), (289, 101), (289, 102), (289, 103), (290, 102), (290, 103), (288, 104), (289, 104), (289, 105), (290, 105), (290, 104), (288, 105), (291, 103), (291, 104), (290, 106), (289, 106), (287, 107), (282, 124), (281, 134), (302, 112), (303, 111), (303, 112), (302, 110), (301, 110), (301, 108), (300, 105), (301, 105), (302, 104), (301, 104), (302, 105), (302, 103), (301, 103), (302, 102), (303, 102), (304, 100), (304, 102), (300, 104), (299, 103), (299, 104), (298, 106), (299, 105), (303, 109), (303, 107), (302, 106), (303, 106), (304, 107), (305, 107), (305, 106), (304, 108), (304, 110), (305, 110), (305, 111), (306, 113), (307, 114), (306, 111), (306, 114), (311, 111), (310, 111), (312, 110), (314, 111), (313, 111), (314, 109), (316, 111), (316, 110), (313, 120), (311, 110), (315, 114), (307, 113), (311, 109), (313, 108), (314, 108), (315, 109), (316, 109), (308, 107), (309, 106), (313, 94), (317, 95), (319, 95), (320, 94), (322, 95), (323, 95), (324, 96), (323, 96), (322, 96), (321, 96), (321, 97), (322, 97), (320, 96), (320, 95), (321, 95), (323, 93), (318, 95), (319, 96), (319, 97), (318, 98), (318, 97), (317, 96), (315, 97), (314, 96), (315, 98), (312, 96), (308, 96), (296, 99), (298, 100), (297, 100), (299, 101), (300, 102), (299, 99), (305, 105), (306, 105), (314, 103), (314, 102), (315, 103), (316, 103), (318, 104), (317, 102), (316, 101), (316, 102), (313, 101), (320, 97), (318, 99), (317, 99), (317, 101), (317, 104), (315, 104), (318, 102), (319, 101), (320, 100), (320, 101), (319, 100), (320, 99), (319, 98), (315, 102), (315, 105), (313, 100), (317, 100), (318, 101), (318, 100), (319, 99), (318, 96), (318, 89), (308, 104), (307, 109), (316, 113), (317, 111), (317, 108), (316, 108), (323, 116), (324, 107), (328, 109), (320, 106), (319, 112), (317, 113), (321, 112), (322, 122), (298, 113), (297, 112), (312, 92), (312, 97), (299, 102), (297, 101), (298, 102), (297, 102), (296, 103), (296, 104), (295, 103), (293, 104), (291, 107), (293, 107), (273, 103), (290, 109), (292, 108), (288, 106), (294, 103), (296, 105), (297, 104), (300, 103), (298, 103), (297, 93), (296, 96), (293, 99), (295, 100), (294, 101), (293, 100), (292, 100), (293, 101), (292, 99), (291, 99), (292, 97), (292, 98), (293, 96), (295, 95), (294, 94), (296, 93), (295, 94), (286, 96), (289, 100), (291, 101), (291, 102), (290, 101), (292, 103), (294, 107), (292, 104), (292, 101), (294, 100), (292, 102), (293, 102), (293, 103), (290, 100), (291, 100), (290, 99), (298, 104), (297, 103), (295, 102), (293, 98), (273, 109), (273, 107), (272, 106), (294, 102), (305, 120), (302, 128), (297, 128), (305, 129), (310, 124), (310, 123), (310, 121), (309, 121), (311, 125), (329, 123), (331, 125), (344, 134), (343, 134), (342, 134), (348, 130), (348, 128), (349, 127), (350, 128), (349, 128), (349, 129), (350, 131), (351, 130), (350, 130), (351, 131), (352, 132), (351, 132), (352, 131), (352, 130), (353, 131), (351, 129), (348, 127), (343, 130), (342, 129), (341, 128), (341, 133), (340, 134), (346, 134), (347, 134), (348, 134), (347, 133), (348, 133), (337, 124), (336, 124), (336, 125), (335, 126), (335, 125), (336, 126), (335, 129), (331, 130), (331, 134), (330, 130), (329, 121), (329, 120), (331, 118), (332, 117), (331, 116), (331, 115), (332, 115), (333, 116), (331, 114), (337, 125), (339, 118), (337, 117), (335, 118), (332, 126), (334, 118), (335, 117), (334, 117), (333, 117), (333, 115), (334, 115), (334, 114), (335, 115), (337, 116), (333, 114), (333, 113), (333, 112), (332, 112), (335, 108), (336, 108), (337, 108), (338, 107), (336, 107), (338, 108), (336, 109), (335, 110), (338, 115), (337, 115), (332, 111), (332, 114), (332, 113), (331, 113), (330, 112), (329, 113), (313, 107), (304, 106), (296, 100), (296, 101), (295, 101), (298, 101), (296, 102), (297, 105), (308, 66), (315, 47), (313, 47), (312, 51), (317, 48), (317, 47), (316, 48), (316, 47), (316, 49), (323, 37), (324, 37), (323, 38), (324, 38), (326, 37), (322, 39), (320, 45), (322, 44), (321, 38), (322, 37), (319, 45), (272, 64), (323, 71), (322, 77), (333, 74), (334, 75), (335, 75), (337, 76), (338, 77), (339, 76), (347, 80), (348, 81), (348, 79), (348, 77), (349, 75), (348, 76), (342, 80), (340, 81), (341, 80), (339, 80), (338, 80), (337, 79), (338, 78), (339, 78), (334, 84), (334, 85), (331, 85), (330, 84), (332, 83), (332, 85), (330, 85), (330, 83), (329, 84), (326, 82), (325, 79), (320, 84), (321, 84), (320, 85), (319, 86), (319, 87), (320, 98), (321, 99), (322, 99), (322, 100), (321, 100), (322, 101), (321, 101), (320, 102), (320, 103), (320, 104), (319, 107), (318, 106), (318, 107), (317, 107), (321, 103), (319, 103), (321, 104), (330, 111), (337, 103), (336, 104), (335, 104), (335, 103), (335, 102), (334, 101), (333, 101), (333, 102), (334, 103), (333, 104), (332, 103), (334, 105), (333, 106), (336, 105), (338, 106), (337, 104), (335, 105), (336, 103), (334, 104), (331, 103), (332, 104), (332, 102), (333, 103), (333, 105), (332, 106), (331, 106), (331, 105), (331, 108), (332, 105), (336, 102), (334, 102), (335, 101), (336, 101), (335, 100), (335, 99), (334, 100), (334, 99), (334, 98), (333, 98), (332, 98), (332, 97), (331, 98), (330, 98), (331, 97), (331, 96), (331, 95), (331, 94), (330, 95), (329, 96), (329, 95), (330, 96), (332, 96), (333, 95), (333, 96), (334, 95), (335, 96), (334, 97), (334, 96), (333, 94), (332, 93), (332, 92), (332, 91), (333, 92), (333, 93), (334, 94), (335, 93), (336, 92), (336, 91), (335, 90), (336, 89), (336, 87), (338, 87), (337, 86), (335, 86), (339, 90), (339, 91), (339, 92), (338, 93), (337, 96), (337, 97), (337, 98), (336, 99), (336, 98), (335, 98), (336, 97), (335, 97), (333, 99), (333, 97), (332, 99), (331, 100), (333, 100), (330, 100), (329, 100), (330, 101), (329, 102), (329, 103), (328, 104), (327, 104), (326, 105), (325, 105), (326, 104), (326, 103), (325, 102), (325, 103), (325, 101), (324, 102), (323, 103), (323, 104), (322, 103), (323, 102), (324, 103), (324, 101), (326, 100), (322, 102), (323, 101), (327, 103), (326, 102), (324, 104), (325, 104), (325, 106), (324, 105), (321, 98), (319, 102), (321, 102), (322, 104), (323, 105), (323, 107), (316, 107), (315, 108), (315, 107), (313, 106), (312, 105), (307, 106), (306, 106), (328, 131), (272, 104), (332, 116), (330, 117), (334, 116), (334, 113), (330, 110), (329, 110), (335, 106), (335, 107), (339, 108), (338, 110), (339, 109), (340, 107), (339, 107), (339, 106), (337, 109), (334, 106), (330, 105), (329, 105), (328, 106), (329, 104), (330, 104), (329, 106), (330, 106), (328, 105), (327, 106), (330, 107), (339, 111), (341, 108), (340, 105), (344, 108), (340, 106), (339, 104), (337, 102), (337, 101), (338, 100), (339, 103), (337, 100), (337, 99), (336, 100), (332, 100), (338, 98), (331, 101), (331, 102), (330, 103), (330, 102), (330, 99), (332, 101), (332, 95), (335, 95), (338, 97), (336, 95), (336, 94), (337, 93), (336, 96), (332, 90), (332, 89), (333, 91), (334, 91), (335, 91), (337, 92), (336, 93), (338, 92), (331, 99), (329, 98), (329, 99), (329, 101), (328, 103), (328, 100), (327, 101), (328, 102), (328, 101), (327, 100), (325, 100), (327, 99), (328, 99), (326, 101), (327, 102), (328, 98), (327, 98), (326, 99), (324, 106), (325, 107), (331, 107), (331, 104), (327, 105), (326, 106), (341, 115), (342, 115), (342, 114), (341, 114), (343, 113), (343, 112), (344, 111), (344, 110), (343, 109), (345, 107), (345, 106), (346, 107), (346, 106), (346, 108), (345, 108), (346, 103), (347, 104), (348, 103), (348, 104), (348, 105), (346, 104), (346, 102), (347, 101), (349, 104), (350, 105), (351, 106), (351, 105), (349, 106), (351, 104), (352, 105), (353, 105), (352, 104), (352, 106), (351, 107), (353, 107), (352, 108), (352, 107), (350, 107), (351, 108), (351, 109), (353, 108), (354, 108), (355, 108), (352, 110), (349, 111), (348, 110), (349, 110), (350, 109), (350, 108), (353, 106), (350, 106), (350, 104), (349, 105), (348, 106), (347, 106), (347, 107), (348, 108), (349, 107), (349, 108), (348, 107), (349, 102), (348, 101), (350, 98), (343, 102), (344, 98), (348, 94), (349, 93), (349, 95), (351, 99), (352, 102), (353, 103), (353, 102), (353, 101), (353, 100), (352, 103), (354, 103), (353, 104), (354, 107), (355, 107), (356, 107), (359, 106), (359, 105), (358, 104), (360, 105), (361, 106), (362, 107), (361, 107), (360, 107), (359, 107), (357, 106), (358, 105), (357, 104), (356, 105), (359, 104), (359, 108), (360, 104), (361, 104), (362, 104), (363, 104), (362, 105), (361, 105), (358, 103), (359, 103), (359, 102), (358, 101), (358, 100), (360, 106), (360, 108), (360, 109), (361, 109), (361, 110), (362, 110), (362, 109), (363, 110), (363, 109), (363, 108), (364, 109), (365, 110), (366, 109), (367, 108), (368, 108), (368, 107), (367, 106), (369, 107), (368, 106), (367, 107), (369, 108), (369, 109), (369, 110), (368, 110), (367, 110), (366, 111), (366, 110), (365, 111), (367, 109), (370, 108), (371, 108), (367, 105), (368, 105), (366, 106), (365, 106), (364, 105), (363, 106), (362, 106), (364, 107), (360, 103), (363, 105), (360, 102), (360, 101), (359, 100), (360, 100), (361, 101), (361, 100), (362, 100), (362, 101), (361, 102), (361, 103), (362, 102), (363, 101), (360, 99), (361, 99), (362, 98), (363, 98), (364, 99), (364, 100), (365, 99), (365, 98), (366, 97), (365, 96), (365, 97), (364, 97), (363, 96), (364, 95), (364, 96), (363, 95), (364, 94), (364, 93), (363, 93), (362, 97), (362, 99), (357, 100), (355, 93), (352, 94), (350, 93), (351, 93), (356, 92), (359, 89), (367, 86), (365, 84), (371, 79), (368, 80), (373, 77), (374, 76), (374, 77), (373, 76), (373, 75), (372, 75), (385, 79), (382, 79), (381, 79), (382, 78), (383, 79), (383, 78), (383, 76), (383, 77), (386, 79), (388, 78), (386, 77), (385, 76), (384, 74), (385, 74), (386, 74), (385, 73), (383, 74), (382, 75), (383, 75), (382, 74), (381, 74), (381, 75), (382, 73), (381, 73), (380, 73), (383, 72), (384, 72), (384, 71), (385, 71), (385, 70), (384, 69), (385, 69), (386, 70), (384, 70), (386, 68), (385, 67), (385, 66), (386, 67), (386, 69), (385, 68), (384, 67), (384, 66), (383, 67), (383, 68), (382, 69), (381, 70), (380, 70), (379, 68), (380, 69), (376, 67), (377, 65), (378, 65), (380, 65), (378, 61), (377, 61), (376, 61), (377, 62), (379, 61), (380, 61), (382, 63), (381, 63), (383, 63), (384, 62), (385, 61), (386, 60), (386, 59), (386, 58), (385, 57), (387, 59), (388, 58), (387, 58), (390, 57), (390, 56), (390, 55), (391, 59), (389, 61), (391, 60), (397, 57), (398, 58), (397, 63), (397, 62), (396, 62), (393, 63), (392, 60), (389, 58), (391, 55), (390, 54), (388, 60), (387, 61), (386, 61), (386, 62), (387, 63), (387, 62), (385, 63), (384, 64), (378, 60), (377, 60), (378, 59), (379, 60), (379, 59), (379, 58), (380, 57), (381, 57), (380, 58), (381, 59), (380, 59), (378, 63), (380, 60), (381, 58), (382, 59), (381, 60), (379, 57), (380, 56), (381, 55), (381, 56), (382, 60), (383, 60), (382, 57), (382, 58), (383, 58), (384, 59), (385, 58), (384, 58), (383, 57), (384, 57), (383, 56), (391, 54), (389, 52), (388, 59), (382, 38), (382, 39), (384, 37), (385, 36), (386, 37), (386, 38), (387, 37), (387, 36), (386, 36), (386, 35), (387, 35), (388, 36), (387, 38), (388, 40), (387, 41), (388, 44), (387, 44), (388, 45), (385, 42), (386, 43), (392, 41), (391, 39), (392, 40), (392, 39), (392, 38), (389, 39), (387, 39), (396, 47), (396, 46), (397, 47), (397, 48), (398, 48), (399, 49), (400, 49), (398, 51), (396, 52), (395, 51), (397, 49), (396, 48), (396, 44), (395, 39), (394, 38), (395, 38), (395, 37), (396, 37), (396, 36), (397, 37), (398, 41), (399, 42), (388, 38), (389, 38), (390, 37), (390, 36), (390, 35), (389, 36), (388, 37), (389, 37), (385, 34), (386, 34), (387, 33), (388, 32), (388, 33), (387, 32), (384, 33), (384, 34), (383, 33), (386, 30), (386, 33), (385, 35), (384, 36), (383, 35), (383, 34), (382, 30), (384, 35), (379, 38), (378, 40), (386, 39), (385, 38), (385, 37), (388, 34), (387, 34), (388, 35), (389, 35), (389, 32), (390, 32), (397, 33), (398, 34), (399, 33), (397, 28), (396, 27), (393, 27), (395, 24), (396, 25), (394, 23), (392, 22), (393, 21), (393, 22), (394, 21), (394, 22), (394, 20), (392, 19), (390, 18), (391, 17), (392, 18), (392, 17), (389, 15), (388, 16), (390, 15), (385, 16), (386, 16), (387, 17), (387, 16), (386, 17), (385, 17), (387, 15), (388, 14), (387, 13), (388, 13), (388, 12), (387, 11), (387, 12), (385, 15), (385, 13), (385, 12), (384, 13), (383, 12), (384, 12), (384, 11), (383, 11), (382, 10), (381, 10), (381, 9), (380, 9), (382, 9), (383, 10), (385, 11), (385, 10), (386, 12), (382, 13), (382, 14), (381, 15), (382, 15), (382, 16), (381, 14), (380, 13), (381, 13), (379, 13), (380, 14), (378, 14), (377, 14), (377, 15), (378, 15), (376, 13), (375, 14), (376, 15), (375, 16), (376, 16), (376, 17), (376, 18), (375, 18), (374, 19), (374, 20), (374, 21), (375, 22), (376, 21), (375, 20), (375, 21), (375, 19), (376, 20), (377, 19), (377, 20), (375, 23), (372, 25), (372, 24), (375, 26), (376, 26), (377, 26), (383, 27), (383, 26), (385, 26), (386, 27), (384, 25), (382, 26), (382, 20), (384, 20), (385, 19), (385, 18), (386, 19), (387, 18), (387, 19), (386, 20), (385, 21), (385, 20), (386, 21), (387, 20), (388, 19), (387, 21), (386, 22), (387, 23), (386, 24), (385, 24), (384, 23), (384, 24), (383, 24), (384, 22), (384, 21), (385, 22), (386, 23), (386, 18), (384, 18), (384, 17), (389, 12), (390, 11), (391, 12), (390, 13), (390, 12), (391, 13), (392, 12), (389, 13), (386, 13), (389, 14), (390, 14), (389, 11), (388, 11), (383, 16), (381, 17), (381, 16), (380, 16), (379, 17), (378, 16), (378, 17), (377, 18), (378, 19), (378, 18), (378, 20), (379, 18), (377, 17), (379, 16), (380, 17), (380, 15), (377, 16), (379, 15), (380, 12), (380, 11), (380, 10), (381, 11), (381, 8), (382, 8), (381, 7), (382, 7), (380, 7), (380, 8), (379, 11), (379, 10), (379, 12), (378, 12), (378, 13), (376, 14), (377, 13), (375, 12), (376, 11), (375, 10), (374, 9), (375, 8), (376, 9), (376, 10), (377, 10), (375, 9), (374, 10), (375, 11), (374, 11), (373, 9), (373, 10), (376, 12), (377, 12), (375, 13), (374, 12), (374, 13), (373, 12), (372, 11), (373, 11), (372, 9), (371, 8), (370, 7), (370, 8), (370, 6), (370, 5), (369, 6), (371, 5), (372, 5), (372, 6), (373, 5), (373, 6), (373, 7), (374, 8), (376, 7), (377, 6), (378, 7), (378, 8), (379, 9), (379, 6), (378, 6), (379, 5), (380, 6), (380, 5), (381, 6), (381, 5), (382, 6), (382, 5), (383, 5), (384, 5), (383, 6), (384, 7), (385, 8), (385, 9), (384, 6), (385, 6), (385, 7), (386, 7), (386, 8), (386, 9), (384, 10), (384, 9), (386, 10), (386, 11), (388, 10), (387, 9), (387, 8), (388, 9), (388, 8), (389, 8), (387, 10), (383, 13), (384, 14), (384, 8), (383, 9), (382, 11), (382, 12), (383, 8), (383, 7), (389, 10), (386, 6), (385, 5), (387, 7), (387, 6), (388, 6), (389, 6), (390, 7), (391, 8), (391, 9), (390, 5), (390, 6), (388, 5), (389, 5), (388, 7), (387, 5), (386, 5), (379, 14), (388, 20), (388, 17), (388, 18), (389, 18), (387, 22), (385, 23), (383, 22), (383, 23), (383, 21), (389, 19), (393, 13), (391, 11), (390, 10), (389, 9), (389, 7), (378, 5), (377, 8), (377, 9), (376, 8), (378, 9), (378, 10), (379, 8), (375, 7), (377, 11), (378, 11), (372, 10), (373, 8), (371, 10), (370, 10), (370, 11), (369, 10), (370, 9), (371, 9), (369, 9), (371, 11), (372, 12), (370, 12), (369, 13), (369, 14), (368, 12), (368, 11), (367, 12), (366, 12), (366, 11), (365, 11), (365, 12), (364, 13), (365, 13), (364, 14), (363, 13), (364, 12), (363, 14), (363, 15), (363, 12), (362, 14), (362, 12), (362, 11), (361, 12), (360, 11), (361, 13), (360, 12), (359, 11), (358, 12), (358, 11), (359, 10), (358, 8), (358, 7), (359, 6), (359, 7), (360, 8), (360, 7), (357, 6), (358, 6), (358, 5), (359, 8), (359, 5), (356, 6), (357, 5), (360, 5), (360, 6), (354, 10), (355, 11), (353, 11), (352, 14), (352, 15), (355, 15), (353, 20), (352, 20), (351, 20), (352, 21), (350, 21), (350, 22), (349, 21), (348, 21), (351, 21), (349, 20), (348, 19), (348, 20), (349, 19), (350, 19), (352, 19), (354, 18), (354, 17), (357, 19), (356, 19), (352, 17), (356, 10), (356, 11), (352, 9), (348, 14), (349, 17), (348, 18), (348, 17), (348, 16), (345, 14), (350, 17), (350, 18), (351, 18), (340, 16), (337, 20), (336, 20), (339, 19), (337, 21), (338, 20), (332, 18), (332, 16), (332, 15), (315, 5), (293, 73), (293, 85), (290, 90), (295, 93), (304, 101), (314, 101), (313, 96), (292, 70), (314, 43), (334, 76), (335, 77), (335, 76), (336, 76), (337, 78), (340, 80), (340, 83), (342, 85), (339, 81), (335, 84), (338, 74), (336, 77), (338, 79), (339, 79), (343, 83), (338, 85), (339, 86), (340, 87), (337, 87), (333, 84), (335, 88), (336, 90), (337, 91), (334, 90), (335, 89), (334, 88), (337, 90), (338, 90), (338, 91), (341, 89), (343, 86), (349, 94), (350, 95), (350, 87), (348, 82), (346, 83), (345, 83), (349, 81), (352, 85), (352, 84), (359, 79), (360, 78), (358, 79), (357, 77), (354, 75), (354, 74), (355, 74), (356, 74), (355, 75), (356, 78), (351, 78), (355, 81), (353, 73), (353, 72), (354, 72), (354, 73), (355, 73), (356, 73), (356, 72), (356, 71), (355, 71), (354, 71), (355, 72), (357, 70), (358, 69), (356, 69), (357, 69), (358, 71), (357, 71), (358, 70), (359, 70), (356, 70), (353, 71), (352, 71), (354, 70), (355, 70), (355, 69), (356, 68), (353, 68), (355, 68), (352, 87), (365, 86), (364, 86), (360, 89), (351, 97), (357, 99), (350, 103), (349, 103), (346, 105), (347, 105), (345, 105), (344, 109), (343, 110), (342, 110), (342, 111), (343, 115), (344, 114), (327, 97), (328, 97), (328, 96), (326, 98), (326, 97), (326, 96), (323, 97), (323, 98), (324, 99), (324, 100), (323, 100), (323, 99), (324, 98), (325, 99), (322, 98), (341, 119), (342, 119), (341, 118), (335, 114), (329, 97), (325, 98), (325, 97), (325, 96), (324, 97), (323, 106), (312, 106), (331, 109), (335, 92), (334, 92), (342, 109), (346, 122), (345, 122), (345, 121), (345, 120), (345, 119), (344, 118), (343, 116), (343, 118), (346, 113), (345, 113), (347, 113), (346, 112), (345, 112), (346, 111), (345, 110), (346, 109), (347, 108), (348, 109), (349, 109), (357, 110), (357, 108), (359, 113), (358, 114), (357, 115), (356, 115), (358, 113), (359, 114), (360, 112), (360, 111), (359, 110), (360, 110), (364, 111), (364, 110), (367, 112), (368, 113), (369, 113), (370, 114), (371, 115), (370, 115), (370, 113), (369, 114), (370, 112), (369, 115), (371, 114), (369, 112), (368, 111), (366, 108), (365, 107), (364, 108), (364, 106), (365, 105), (365, 104), (364, 103), (363, 102), (362, 103), (363, 100), (364, 101), (365, 100), (364, 98), (366, 99), (367, 99), (367, 100), (366, 98), (367, 98), (368, 97), (367, 97), (368, 96), (369, 98), (369, 97), (370, 97), (368, 98), (365, 95), (366, 96), (367, 96), (368, 95), (369, 95), (370, 96), (371, 96), (372, 94), (371, 95), (371, 93), (370, 93), (374, 97), (373, 97), (374, 98), (374, 99), (373, 99), (375, 97), (372, 97), (371, 97), (371, 98), (370, 99), (371, 99), (369, 99), (368, 99), (370, 100), (371, 101), (370, 101), (370, 102), (371, 103), (369, 102), (369, 103), (369, 104), (368, 104), (367, 104), (369, 106), (370, 106), (370, 107), (371, 107), (372, 107), (373, 108), (372, 109), (371, 109), (372, 108), (371, 110), (371, 111), (370, 110), (370, 109), (373, 110), (373, 109), (374, 110), (375, 109), (375, 110), (374, 111), (374, 112), (375, 111), (373, 112), (373, 111), (373, 113), (372, 113), (374, 113), (374, 114), (374, 115), (375, 115), (375, 116), (376, 115), (374, 116), (373, 115), (372, 114), (371, 113), (372, 115), (373, 120), (372, 119), (371, 118), (370, 119), (371, 119), (372, 117), (371, 116), (370, 116), (369, 116), (368, 116), (367, 117), (367, 116), (366, 117), (366, 116), (366, 115), (367, 114), (366, 113), (367, 113), (368, 114), (368, 115), (367, 111), (368, 109), (366, 112), (365, 112), (364, 113), (363, 114), (371, 112), (372, 111), (370, 111), (372, 112), (374, 109), (376, 109), (376, 110), (377, 111), (377, 112), (377, 110), (378, 110), (378, 111), (378, 112), (376, 112), (376, 113), (377, 113), (375, 113), (369, 111), (371, 105), (372, 104), (372, 105), (373, 105), (373, 104), (371, 106), (370, 105), (369, 105), (370, 104), (364, 104), (365, 103), (361, 112), (357, 113), (361, 108), (363, 107), (364, 102), (365, 102), (365, 101), (366, 101), (367, 102), (366, 102), (363, 103), (362, 108), (366, 105), (366, 107), (365, 108), (366, 104), (367, 103), (366, 103), (368, 101), (369, 101), (370, 98), (371, 100), (372, 101), (373, 100), (374, 100), (373, 101), (374, 101), (375, 101), (375, 100), (376, 100), (377, 101), (380, 98), (381, 99), (382, 100), (383, 100), (384, 99), (385, 100), (386, 100), (386, 101), (387, 102), (386, 102), (386, 103), (391, 105), (392, 105), (391, 103), (391, 102), (395, 100), (387, 99), (386, 99), (386, 98), (385, 97), (378, 97), (377, 98), (376, 101), (377, 102), (376, 102), (376, 103), (377, 103), (376, 104), (377, 104), (378, 105), (377, 105), (375, 105), (375, 104), (374, 104), (374, 105), (373, 106), (374, 106), (373, 107), (372, 106), (376, 105), (378, 104), (377, 106), (378, 107), (378, 106), (379, 106), (377, 107), (376, 106), (379, 107), (379, 105), (379, 104), (374, 103), (374, 102), (373, 103), (372, 102), (373, 102), (371, 102), (371, 104), (368, 103), (368, 102), (367, 101), (366, 100), (363, 99), (363, 97), (361, 98), (360, 97), (360, 98), (358, 99), (363, 94), (363, 92), (364, 92), (365, 93), (366, 92), (366, 94), (367, 94), (368, 93), (370, 92), (368, 92), (367, 92), (367, 91), (366, 91), (369, 94), (373, 95), (375, 96), (370, 95), (373, 98), (372, 99), (372, 98), (369, 96), (355, 99), (365, 94), (361, 97), (351, 87), (345, 84), (357, 72), (358, 72), (359, 72), (360, 73), (360, 74), (361, 74), (360, 77), (359, 73), (360, 72), (358, 73), (361, 73), (359, 78), (351, 88), (350, 89), (338, 86), (339, 84), (344, 70), (351, 49), (352, 50), (354, 48), (354, 47), (353, 47), (352, 47), (349, 45), (350, 45), (351, 47), (351, 48), (353, 46), (354, 46), (354, 45), (355, 45), (356, 44), (355, 44), (354, 44), (356, 43), (356, 42), (357, 45), (357, 46), (355, 47), (355, 46), (349, 43), (350, 46), (356, 46), (358, 44), (359, 43), (363, 44), (363, 43), (363, 40), (363, 41), (365, 42), (366, 41), (366, 42), (364, 40), (363, 39), (364, 38), (365, 39), (366, 40), (365, 40), (362, 42), (364, 46), (364, 44), (364, 45), (365, 46), (365, 47), (366, 47), (367, 48), (368, 49), (367, 49), (367, 47), (367, 46), (365, 48), (365, 49), (364, 48), (363, 49), (363, 46), (358, 47), (355, 48), (356, 45), (352, 48), (351, 51), (354, 59), (355, 57), (359, 60), (360, 63), (368, 61), (365, 57), (364, 54), (363, 53), (363, 51), (364, 50), (365, 51), (366, 50), (366, 49), (364, 49), (365, 50), (366, 48), (367, 50), (366, 46), (368, 48), (368, 47), (368, 45), (366, 51), (367, 52), (366, 53), (366, 52), (367, 51), (368, 52), (368, 53), (369, 53), (368, 54), (369, 57), (369, 58), (370, 58), (370, 56), (367, 57), (368, 58), (366, 59), (364, 58), (365, 61), (364, 62), (359, 63), (359, 62), (363, 54), (361, 53), (362, 53), (363, 52), (364, 53), (364, 51), (363, 48), (358, 51), (357, 51), (356, 50), (352, 56), (350, 56), (356, 66), (357, 67), (359, 68), (359, 69), (361, 70), (361, 71), (365, 67), (365, 68), (365, 66), (365, 63), (365, 69), (366, 67), (366, 66), (367, 66), (367, 67), (366, 64), (367, 63), (366, 63), (367, 64), (365, 62), (366, 62), (370, 63), (370, 64), (371, 65), (371, 66), (370, 66), (368, 64), (372, 64), (373, 64), (373, 63), (372, 62), (371, 63), (372, 63), (372, 66), (374, 65), (375, 65), (375, 66), (376, 65), (375, 69), (376, 66), (376, 64), (376, 62), (383, 59), (384, 60), (385, 59), (388, 62), (389, 63), (390, 64), (391, 64), (391, 63), (390, 62), (390, 61), (387, 60), (385, 60), (384, 61), (385, 62), (386, 63), (384, 55), (383, 66), (383, 62), (385, 64), (384, 65), (385, 65), (386, 65), (386, 66), (387, 67), (387, 68), (388, 67), (388, 66), (388, 65), (387, 66), (382, 62), (383, 61), (382, 61), (382, 56), (383, 54), (384, 63), (382, 55), (379, 55), (378, 54), (376, 56), (381, 62), (381, 61), (379, 56), (374, 59), (373, 56), (375, 55), (376, 54), (375, 54), (374, 53), (375, 53), (376, 52), (376, 53), (376, 51), (375, 52), (374, 51), (373, 52), (373, 50), (373, 51), (372, 52), (372, 51), (371, 51), (370, 51), (369, 52), (369, 51), (370, 54), (371, 54), (370, 55), (371, 55), (371, 59), (371, 50), (371, 52), (370, 53), (369, 54), (369, 55), (372, 54), (373, 54), (373, 53), (374, 54), (377, 54), (380, 55), (375, 60), (388, 61), (387, 69), (388, 69), (388, 68), (386, 71), (387, 70), (388, 70), (389, 70), (388, 71), (387, 72), (386, 73), (385, 72), (387, 71), (389, 71), (388, 72), (389, 67), (389, 68), (390, 68), (390, 69), (393, 72), (394, 71), (390, 71), (394, 73), (392, 79), (393, 80), (391, 80), (388, 101), (389, 101), (393, 104), (396, 105), (395, 107), (394, 106), (393, 105), (393, 106), (386, 104), (385, 104), (384, 103), (384, 104), (385, 105), (384, 105), (383, 104), (382, 103), (381, 104), (382, 105), (383, 106), (382, 106), (381, 105), (382, 104), (383, 103), (384, 102), (385, 103), (381, 106), (382, 107), (381, 108), (380, 109), (381, 109), (384, 108), (383, 107), (380, 104), (380, 105), (383, 105), (382, 111), (385, 109), (383, 108), (382, 108), (382, 109), (389, 108), (391, 106), (391, 111), (393, 111), (394, 109), (393, 110), (397, 106), (397, 107), (398, 108), (399, 108), (398, 109), (399, 110), (399, 111), (400, 111), (399, 112), (398, 110), (397, 112), (395, 112), (394, 111), (395, 114), (392, 118), (391, 119), (392, 119), (394, 119), (393, 118), (394, 117), (391, 117), (395, 119), (396, 117), (397, 121), (389, 120), (387, 113), (385, 121), (383, 122), (382, 123), (383, 121), (380, 119), (380, 118), (380, 117), (379, 117), (381, 116), (381, 115), (380, 115), (380, 114), (379, 113), (380, 112), (379, 112), (379, 111), (375, 112), (375, 114), (376, 114), (374, 108), (375, 108), (376, 108), (376, 107), (375, 106), (374, 107), (378, 108), (379, 103), (380, 106), (381, 107), (380, 108), (380, 110), (379, 109), (379, 110), (379, 108), (381, 110), (382, 110), (381, 111), (380, 111), (383, 111), (388, 109), (387, 103), (387, 101), (387, 100), (385, 101), (388, 102), (385, 102), (390, 105), (391, 104), (396, 100), (396, 103), (395, 102), (397, 100), (395, 103), (398, 101), (386, 95), (390, 93), (389, 92), (399, 91), (396, 91), (397, 91), (382, 116), (381, 119), (396, 118), (396, 116), (396, 115), (394, 122), (395, 123), (395, 122), (396, 123), (395, 124), (396, 124), (397, 124), (398, 124), (397, 123), (397, 122), (398, 121), (394, 121), (394, 123), (393, 124), (393, 125), (394, 125), (395, 125), (394, 124), (393, 122), (393, 123), (392, 123), (391, 112), (382, 115), (381, 114), (381, 113), (380, 113), (376, 111), (380, 120), (373, 117), (372, 116), (373, 114), (375, 107), (375, 103), (375, 102), (376, 99), (375, 99), (369, 100), (366, 95), (365, 92), (365, 90), (370, 89), (367, 95), (359, 99), (351, 102), (361, 111), (362, 112), (363, 111), (364, 112), (365, 113), (364, 114), (364, 115), (365, 115), (365, 114), (366, 114), (362, 115), (362, 116), (362, 117), (361, 118), (362, 118), (363, 119), (363, 120), (364, 120), (364, 121), (365, 120), (365, 121), (365, 122), (364, 119), (363, 118), (363, 117), (361, 116), (361, 117), (360, 117), (360, 118), (359, 116), (359, 117), (358, 117), (359, 118), (358, 118), (358, 119), (357, 118), (357, 119), (356, 120), (356, 121), (356, 122), (357, 123), (356, 123), (357, 122), (358, 123), (357, 124), (356, 125), (355, 126), (356, 127), (355, 128), (356, 128), (357, 129), (356, 130), (356, 129), (357, 130), (358, 130), (357, 131), (358, 132), (359, 133), (358, 131), (359, 131), (360, 132), (361, 131), (360, 131), (361, 130), (361, 129), (361, 128), (362, 128), (363, 127), (364, 127), (364, 126), (363, 126), (362, 125), (365, 127), (365, 126), (365, 125), (361, 126), (360, 126), (359, 126), (360, 125), (359, 124), (361, 125), (362, 126), (363, 125), (362, 124), (361, 123), (360, 124), (359, 123), (359, 122), (360, 123), (356, 124), (354, 126), (355, 125), (354, 124), (355, 124), (355, 123), (355, 122), (354, 122), (352, 120), (350, 124), (350, 126), (351, 124), (349, 126), (350, 120), (351, 120), (354, 121), (355, 121), (354, 123), (357, 121), (358, 120), (357, 120), (355, 120), (359, 119), (359, 120), (358, 121), (360, 119), (361, 120), (362, 119), (363, 115), (364, 116), (363, 113), (362, 113), (362, 111), (372, 100), (368, 100), (376, 97), (375, 98), (376, 98), (383, 98), (383, 99), (382, 98), (384, 97), (383, 102), (384, 101), (394, 108), (399, 109), (400, 109), (400, 110), (401, 111), (401, 112), (400, 113), (398, 114), (397, 114), (398, 115), (396, 108), (397, 109), (400, 112), (400, 114), (400, 115), (399, 115), (399, 116), (399, 114), (401, 113), (401, 114), (401, 115), (396, 111), (397, 116), (397, 115), (398, 116), (398, 117), (396, 114), (398, 118), (399, 122), (398, 123), (399, 123), (400, 124), (399, 124), (400, 125), (399, 125), (399, 126), (398, 125), (397, 126), (396, 126), (396, 127), (396, 128), (395, 129), (395, 130), (394, 131), (395, 132), (395, 133), (396, 134), (394, 134), (393, 133), (392, 132), (392, 133), (393, 132), (392, 131), (391, 131), (391, 130), (390, 131), (390, 132), (389, 133), (390, 134), (391, 133), (391, 132), (393, 131), (392, 130), (393, 129), (394, 130), (396, 131), (395, 131), (396, 129), (397, 127), (397, 128), (395, 128), (395, 127), (394, 126), (393, 120), (400, 123), (401, 123), (401, 122), (401, 124), (397, 125), (396, 125), (393, 126), (392, 127), (392, 128), (391, 129), (390, 129), (392, 129), (391, 128), (393, 128), (394, 129), (394, 128), (394, 127), (390, 130), (389, 129), (388, 128), (387, 129), (388, 130), (386, 130), (387, 131), (388, 132), (388, 133), (389, 134), (390, 128), (391, 127), (390, 127), (389, 127), (388, 127), (392, 125), (392, 126), (393, 127), (398, 126), (397, 131), (397, 130), (397, 129), (395, 126), (393, 130), (396, 130), (398, 128), (399, 129), (398, 130), (398, 129), (398, 131), (399, 130), (400, 129), (400, 128), (401, 130), (400, 131), (396, 132), (397, 132), (392, 121), (398, 122), (394, 132), (394, 133), (393, 134), (392, 134), (400, 126), (400, 127), (400, 121), (400, 120), (399, 119), (399, 118), (400, 118), (399, 117), (400, 116), (401, 117), (397, 108), (398, 107), (400, 108), (401, 108), (400, 104), (400, 103), (401, 104), (400, 105), (399, 105), (401, 109), (401, 110), (401, 116), (400, 117), (401, 118), (400, 119), (398, 119), (399, 120), (383, 113), (382, 113), (382, 112), (397, 117), (401, 120), (401, 119), (381, 112), (378, 103), (380, 103), (381, 102), (380, 102), (381, 101), (382, 102), (381, 103), (378, 102), (379, 102), (378, 98), (379, 99), (380, 100), (378, 114), (379, 115), (377, 109), (378, 113), (377, 114), (378, 115), (379, 114), (380, 107), (382, 93), (387, 89), (386, 75), (387, 76), (384, 75), (382, 77), (381, 78), (381, 76), (383, 73), (392, 80), (395, 81), (398, 90), (401, 101), (396, 98), (380, 91), (379, 90), (370, 90), (371, 90), (374, 89), (373, 80), (368, 76), (369, 75), (368, 74), (367, 75), (369, 74), (367, 74), (367, 73), (367, 72), (367, 71), (367, 70), (366, 69), (366, 68), (367, 68), (368, 68), (367, 69), (366, 70), (365, 70), (364, 71), (365, 71), (365, 72), (364, 73), (363, 73), (365, 75), (367, 79), (368, 78), (369, 78), (374, 82), (377, 79), (381, 77), (382, 76), (384, 73), (383, 69), (382, 68), (381, 69), (382, 70), (380, 68), (381, 68), (386, 48), (393, 51), (392, 53), (395, 54), (396, 53), (394, 53), (394, 50), (399, 51), (395, 53), (400, 57), (401, 54), (400, 54), (395, 52), (386, 64), (387, 64), (389, 66), (390, 67), (389, 69), (390, 72), (389, 72), (391, 73), (394, 72), (395, 71), (394, 70), (395, 70), (395, 72), (390, 66), (391, 66), (391, 65), (392, 65), (395, 63), (401, 57), (396, 64), (397, 64), (398, 64), (399, 63), (400, 64), (399, 64), (401, 61), (400, 65), (401, 66), (401, 65), (396, 63), (395, 65), (392, 64), (390, 65), (389, 62), (390, 63), (389, 64), (388, 64), (388, 63), (389, 65), (393, 66), (393, 67), (392, 67), (392, 66), (393, 70), (393, 69), (394, 68), (394, 69), (396, 72), (396, 71), (395, 73), (395, 69), (395, 68), (396, 68), (397, 67), (396, 67), (396, 66), (381, 54), (377, 53), (374, 52), (375, 64), (374, 67), (374, 68), (375, 67), (377, 70), (384, 68), (383, 70), (387, 65), (375, 56), (376, 55), (373, 60), (373, 62), (374, 62), (344, 74), (351, 68), (351, 112), (350, 112), (350, 113), (349, 112), (352, 112), (353, 113), (354, 113), (357, 116), (349, 113), (350, 114), (351, 114), (352, 114), (352, 115), (351, 116), (350, 116), (351, 117), (351, 118), (350, 117), (349, 117), (348, 118), (350, 118), (351, 119), (358, 124), (360, 121), (360, 120), (359, 121), (361, 121), (358, 116), (358, 115), (354, 116), (354, 118), (355, 119), (360, 122), (360, 116), (361, 119), (364, 118), (364, 117), (365, 117), (367, 118), (368, 117), (368, 118), (368, 119), (367, 119), (369, 120), (369, 119), (371, 117), (370, 117), (369, 117), (369, 118), (370, 118), (368, 120), (367, 120), (372, 120), (373, 119), (374, 121), (371, 121), (372, 121), (373, 122), (377, 115), (369, 121), (370, 120), (371, 120), (370, 121), (373, 118), (374, 118), (372, 118), (371, 123), (371, 122), (372, 123), (370, 123), (370, 124), (370, 125), (369, 125), (370, 126), (369, 126), (368, 125), (367, 124), (366, 125), (368, 124), (367, 125), (366, 126), (364, 128), (363, 129), (362, 130), (363, 130), (362, 129), (364, 129), (364, 130), (365, 128), (364, 125), (364, 124), (365, 123), (366, 124), (368, 123), (369, 124), (372, 124), (371, 124), (372, 125), (368, 126), (369, 127), (370, 128), (368, 127), (369, 128), (369, 129), (368, 128), (368, 129), (369, 130), (368, 131), (368, 130), (369, 131), (370, 132), (371, 131), (370, 131), (371, 132), (369, 133), (370, 133), (369, 132), (368, 132), (368, 134), (367, 133), (366, 132), (366, 133), (365, 132), (365, 133), (366, 134), (365, 134), (364, 133), (363, 132), (362, 133), (362, 132), (361, 133), (361, 134), (363, 133), (364, 131), (365, 131), (367, 134), (367, 132), (368, 133), (367, 131), (366, 131), (366, 130), (366, 129), (365, 129), (365, 130), (363, 131), (362, 131), (361, 132), (363, 128), (363, 134), (369, 134), (370, 134), (371, 133), (372, 130), (373, 130), (371, 128), (370, 127), (371, 129), (370, 129), (367, 128), (366, 128), (372, 132), (372, 133), (375, 134), (374, 134), (373, 134), (371, 130), (370, 130), (367, 130), (367, 129), (371, 134), (373, 131), (372, 134), (372, 131), (380, 128), (367, 115), (365, 118), (365, 116), (368, 112), (365, 109), (363, 112), (366, 118), (365, 119), (366, 120), (366, 121), (368, 121), (368, 122), (369, 123), (370, 122), (369, 122), (392, 113), (392, 106), (390, 126), (391, 126), (398, 127), (399, 131), (401, 131), (400, 132), (399, 132), (400, 133), (399, 133), (400, 134), (401, 134), (401, 133), (401, 132), (399, 134), (400, 130), (401, 129), (401, 128), (399, 128), (399, 127), (395, 134), (396, 133), (397, 134), (397, 133), (398, 133), (398, 132), (387, 126), (386, 127), (387, 128), (386, 128), (389, 128), (388, 129), (389, 130), (392, 124), (398, 134), (401, 127), (399, 121), (398, 120), (386, 124), (387, 123), (387, 124), (400, 122), (401, 121), (401, 126), (385, 124), (384, 125), (383, 126), (384, 126), (385, 126), (386, 125), (385, 125), (385, 128), (384, 129), (385, 129), (386, 129), (384, 130), (384, 131), (382, 131), (381, 130), (380, 134), (378, 127), (378, 126), (379, 126), (383, 123), (383, 124), (387, 125), (384, 122), (378, 109), (373, 116), (374, 124), (373, 125), (367, 127), (366, 127), (360, 130), (359, 129), (358, 129), (358, 128), (360, 129), (360, 128), (359, 130), (360, 127), (361, 127), (359, 127), (358, 127), (362, 127), (367, 126), (366, 123), (365, 124), (375, 125), (376, 125), (377, 125), (362, 134), (364, 132), (359, 134), (360, 133), (359, 132), (356, 131), (355, 131), (355, 132), (354, 132), (354, 133), (354, 134), (355, 133), (356, 132), (355, 130), (358, 133), (357, 132), (357, 133), (358, 134), (360, 134), (357, 134), (356, 133), (356, 134), (354, 130), (355, 129), (357, 126), (358, 126), (357, 125), (358, 125), (359, 125), (358, 122), (366, 119), (363, 121), (364, 122), (363, 122), (363, 123), (363, 124), (362, 122), (364, 123), (367, 122), (377, 126), (392, 111), (396, 104), (401, 107), (398, 102), (400, 106), (401, 105), (401, 103), (401, 100), (401, 102), (382, 91), (383, 89), (384, 89), (383, 88), (382, 89), (382, 90), (386, 90), (388, 90), (388, 91), (377, 108), (345, 93), (335, 94), (346, 100), (348, 112), (348, 111), (347, 110), (345, 109), (343, 111), (344, 112), (345, 111), (356, 117), (352, 116), (353, 115), (353, 114), (360, 113), (351, 115), (350, 115), (352, 117), (351, 113), (349, 114), (349, 116), (347, 119), (348, 119), (349, 120), (349, 121), (350, 121), (349, 122), (348, 121), (347, 122), (348, 122), (355, 127), (357, 127), (356, 126), (359, 128), (357, 128), (355, 134), (354, 131), (353, 133), (353, 134), (362, 121), (366, 122), (362, 120), (367, 121), (382, 125), (379, 134), (381, 134), (382, 133), (383, 132), (383, 133), (382, 134), (383, 134), (384, 132), (384, 134), (384, 133), (385, 132), (385, 133), (385, 134), (386, 133), (386, 132), (386, 131), (385, 130), (385, 127), (386, 126), (384, 124), (384, 123), (401, 106), (362, 123), (361, 122), (361, 113), (363, 116), (361, 114), (360, 114), (360, 115), (357, 117), (355, 118), (359, 115), (350, 119), (349, 119), (349, 118), (348, 117), (348, 115), (347, 115), (348, 114), (352, 113), (347, 117), (348, 116), (349, 115), (348, 113), (354, 119), (348, 120), (347, 121), (347, 120), (346, 120), (344, 119), (344, 121), (344, 123), (346, 124), (346, 123), (346, 121), (346, 119), (347, 118), (343, 119), (345, 118), (346, 117), (347, 116), (347, 114), (346, 115), (346, 116), (342, 117), (345, 117), (346, 118), (346, 114), (356, 118), (356, 119), (354, 129), (364, 134), (383, 128), (383, 127), (392, 82), (377, 97), (381, 90), (368, 91), (329, 86), (329, 79), (330, 80), (345, 66), (350, 68), (352, 69), (354, 69), (359, 71), (360, 71), (361, 72), (362, 72), (362, 70), (362, 71), (363, 71), (364, 70), (364, 69), (363, 70), (363, 69), (366, 71), (366, 72), (365, 73), (368, 70), (368, 72), (369, 71), (368, 71), (369, 72), (369, 73), (370, 72), (366, 76), (366, 75), (366, 74), (370, 73), (371, 74), (372, 74), (374, 69), (371, 68), (370, 67), (371, 67), (373, 67), (371, 71), (372, 76), (371, 75), (372, 73), (371, 70), (370, 69), (369, 69), (369, 70), (370, 68), (370, 70), (370, 71), (369, 68), (369, 66), (368, 69), (368, 73), (368, 77), (370, 75), (370, 76), (372, 77), (371, 76), (380, 88), (381, 88), (381, 89), (390, 76), (388, 74), (389, 77), (388, 76), (387, 75), (387, 73), (388, 73), (386, 72), (387, 80), (383, 71), (380, 74), (380, 75), (380, 76), (379, 75), (378, 75), (382, 71), (379, 73), (379, 72), (387, 74), (395, 67), (397, 66), (396, 69), (397, 70), (397, 69), (397, 71), (396, 70), (398, 65), (398, 68), (392, 75), (388, 75), (382, 72), (399, 69), (400, 73), (398, 76), (397, 75), (397, 76), (396, 75), (397, 74), (390, 78), (397, 77), (400, 77), (401, 80), (401, 84), (401, 85), (375, 68), (371, 62), (366, 57), (366, 58), (365, 55), (371, 53), (370, 52), (370, 50), (369, 50), (374, 50), (373, 49), (373, 42), (373, 40), (372, 40), (374, 38), (372, 37), (371, 36), (371, 34), (372, 35), (374, 35), (376, 37), (394, 37), (393, 37), (393, 38), (392, 37), (393, 36), (391, 36), (391, 37), (390, 38), (391, 38), (376, 33), (375, 34), (369, 32), (366, 33), (365, 34), (364, 35), (356, 29), (357, 28), (356, 27), (357, 30), (359, 27), (357, 26), (357, 25), (355, 25), (351, 24), (350, 25), (354, 26), (356, 25), (356, 30), (359, 33), (357, 31), (362, 36), (363, 35), (358, 40), (358, 39), (357, 38), (357, 39), (357, 40), (355, 42), (336, 16), (336, 15), (342, 7), (343, 8), (344, 8), (343, 7), (344, 6), (345, 6), (344, 7), (345, 7), (346, 6), (347, 7), (348, 8), (347, 9), (348, 10), (351, 10), (353, 5), (356, 7), (360, 9), (361, 9), (362, 8), (362, 10), (363, 11), (363, 10), (363, 9), (364, 10), (364, 9), (365, 10), (365, 9), (366, 10), (367, 10), (368, 9), (369, 8), (368, 10), (364, 11), (367, 11), (369, 11), (371, 13), (371, 12), (372, 13), (373, 13), (372, 8), (374, 7), (374, 14), (376, 19), (382, 21), (393, 12), (393, 11), (390, 20), (389, 20), (390, 19), (389, 21), (390, 22), (391, 23), (388, 21), (389, 22), (390, 21), (391, 21), (388, 23), (389, 24), (388, 25), (387, 25), (388, 24), (388, 22), (395, 21), (396, 20), (395, 20), (381, 12), (390, 8), (391, 7), (390, 9), (393, 10), (394, 10), (394, 11), (395, 11), (396, 12), (395, 13), (397, 12), (398, 11), (398, 12), (397, 11), (398, 10), (399, 9), (400, 9), (399, 10), (399, 11), (400, 12), (401, 11), (401, 12), (401, 10), (401, 9), (400, 10), (400, 8), (399, 7), (398, 9), (400, 11), (398, 13), (401, 7), (401, 8), (400, 7), (399, 8), (397, 9), (396, 9), (396, 10), (395, 10), (394, 9), (395, 8), (397, 10), (399, 12), (399, 13), (399, 14), (400, 15), (400, 14), (401, 13), (397, 8), (397, 7), (398, 8), (398, 7), (399, 6), (399, 5), (398, 6), (400, 6), (401, 5), (401, 6), (400, 5), (398, 5), (397, 5), (396, 6), (395, 6), (395, 7), (396, 8), (396, 7), (395, 12), (396, 11), (399, 16), (398, 15), (400, 17), (400, 18), (400, 19), (399, 20), (400, 20), (399, 19), (398, 18), (398, 17), (399, 18), (401, 19), (401, 20), (400, 21), (401, 22), (400, 22), (399, 23), (399, 21), (401, 18), (399, 17), (398, 19), (398, 20), (397, 19), (396, 18), (397, 20), (398, 16), (397, 16), (397, 14), (398, 14), (397, 13), (400, 13), (399, 15), (396, 13), (396, 14), (397, 15), (397, 17), (396, 16), (395, 16), (394, 17), (394, 18), (395, 17), (396, 17), (400, 16), (401, 14), (398, 21), (397, 22), (396, 19), (395, 19), (395, 18), (396, 21), (396, 22), (397, 21), (398, 22), (399, 22), (400, 23), (401, 23), (399, 24), (400, 25), (400, 24), (399, 25), (398, 25), (399, 26), (401, 26), (401, 25), (401, 24), (401, 27), (401, 28), (398, 29), (398, 30), (400, 29), (401, 29), (401, 31), (401, 33), (400, 32), (391, 29), (389, 28), (388, 27), (387, 27), (386, 26), (386, 25), (385, 25), (386, 29), (387, 28), (392, 36), (393, 34), (391, 35), (395, 34), (396, 34), (397, 34), (397, 30), (394, 27), (395, 23), (396, 24), (395, 22), (396, 23), (397, 23), (398, 23), (398, 24), (397, 24), (397, 26), (389, 29), (388, 28), (389, 26), (388, 26), (387, 26), (389, 27), (387, 24), (389, 23), (391, 26), (391, 27), (392, 27), (390, 27), (389, 25), (388, 29), (390, 39), (395, 48), (397, 53), (399, 46), (401, 40), (400, 39), (401, 43), (401, 44), (401, 45), (395, 47), (374, 55), (377, 52), (401, 68), (401, 69), (400, 69), (399, 72), (401, 72), (398, 73), (400, 71), (400, 70), (401, 70), (400, 62), (401, 71), (395, 74), (382, 88), (381, 91), (374, 60), (367, 76), (372, 103), (370, 103), (380, 99), (382, 99), (372, 110), (371, 92), (387, 130), (384, 128), (387, 133), (387, 134), (386, 134), (385, 131), (367, 123), (362, 114), (361, 115), (365, 74), (359, 76), (358, 78), (363, 72), (366, 73), (367, 77), (365, 76), (371, 73), (370, 74), (334, 87), (333, 87), (332, 88), (333, 88), (331, 88), (331, 89), (333, 89), (330, 93), (330, 94), (330, 97), (332, 94), (331, 93), (322, 87), (321, 86), (321, 85), (322, 85), (323, 85), (322, 86), (323, 86), (322, 83), (324, 86), (323, 84), (322, 84), (319, 88), (334, 89), (315, 101), (318, 86), (318, 85), (319, 85), (276, 72), (277, 71), (278, 73), (293, 72), (347, 124), (347, 131), (361, 124), (390, 133), (388, 134), (388, 131), (389, 131), (389, 132), (387, 132), (384, 127), (401, 125), (334, 93), (333, 90), (321, 82), (314, 42), (355, 36), (362, 37), (364, 36), (362, 34), (349, 25), (348, 22), (347, 19), (351, 19), (349, 18), (349, 24), (349, 26), (350, 20), (347, 20), (350, 8), (351, 7), (350, 7), (351, 6), (350, 6), (349, 6), (348, 7), (347, 8), (346, 8), (345, 8), (345, 9), (344, 9), (341, 11), (342, 9), (343, 12), (353, 21), (352, 22), (350, 24), (346, 9), (346, 10), (346, 7), (346, 5), (345, 5), (344, 5), (348, 6), (348, 5), (347, 5), (349, 7), (347, 6), (349, 5), (349, 8), (340, 14), (339, 13), (337, 22), (337, 23), (336, 21), (337, 24), (337, 25), (336, 24), (337, 16), (345, 32), (353, 22), (354, 22), (354, 21), (355, 38), (359, 45), (348, 27), (337, 15), (339, 15), (342, 8), (339, 5), (339, 6), (340, 5), (340, 6), (341, 5), (342, 5), (343, 9), (336, 41), (340, 39), (396, 73), (401, 52), (375, 51), (369, 49), (371, 42), (370, 42), (369, 41), (368, 41), (368, 40), (367, 41), (367, 42), (368, 43), (369, 43), (370, 43), (370, 41), (367, 40), (367, 39), (367, 38), (368, 39), (369, 40), (369, 44), (368, 44), (367, 44), (367, 43), (366, 44), (366, 43), (368, 38), (366, 36), (366, 35), (365, 36), (367, 35), (366, 37), (365, 38), (364, 39), (366, 38), (365, 37), (363, 38), (366, 34), (364, 34), (366, 39), (350, 55), (353, 70), (357, 73), (272, 33), (352, 128), (351, 127), (352, 129), (353, 129), (353, 130), (350, 127), (354, 125), (379, 74), (374, 63), (375, 50), (377, 51), (373, 55), (369, 48), (368, 51), (372, 50), (368, 42), (363, 47), (364, 47), (365, 52), (361, 51), (372, 55), (374, 66), (395, 40), (379, 45), (378, 49), (377, 36), (383, 37), (376, 29), (376, 27), (377, 27), (381, 27), (393, 20), (401, 21), (397, 18), (394, 19), (401, 17), (393, 18), (396, 5), (395, 5), (397, 6), (394, 12), (392, 11), (391, 10), (392, 9), (373, 14), (376, 5), (375, 6), (374, 5), (372, 7), (371, 6), (371, 7), (369, 5), (368, 6), (368, 7), (369, 7), (368, 8), (367, 7), (368, 5), (367, 6), (366, 6), (366, 5), (367, 5), (365, 7), (364, 7), (364, 8), (362, 9), (367, 9), (367, 8), (366, 8), (366, 7), (365, 6), (365, 5), (363, 8), (363, 7), (362, 6), (361, 6), (361, 7), (362, 7), (361, 8), (361, 10), (366, 13), (365, 14), (366, 15), (367, 16), (367, 17), (368, 15), (369, 15), (370, 14), (368, 16), (368, 17), (369, 16), (369, 17), (370, 18), (371, 17), (372, 17), (371, 16), (370, 15), (371, 15), (371, 14), (370, 16), (370, 17), (368, 18), (368, 19), (367, 18), (369, 20), (369, 21), (369, 19), (371, 19), (371, 20), (372, 20), (371, 21), (372, 21), (368, 22), (367, 21), (367, 22), (366, 21), (368, 21), (366, 22), (367, 23), (368, 23), (363, 21), (364, 20), (364, 18), (363, 19), (360, 16), (359, 14), (360, 15), (361, 15), (359, 15), (365, 20), (366, 19), (366, 17), (369, 12), (370, 13), (369, 18), (369, 23), (369, 22), (366, 23), (364, 22), (366, 24), (367, 25), (372, 29), (368, 24), (369, 24), (370, 20), (370, 19), (372, 19), (373, 19), (372, 18), (371, 18), (372, 16), (373, 16), (374, 17), (374, 18), (373, 17), (374, 16), (375, 15), (372, 14), (367, 15), (366, 16), (365, 15), (365, 17), (365, 16), (366, 18), (373, 18), (373, 20), (377, 41), (395, 35), (392, 33), (401, 46), (401, 47), (400, 48), (400, 47), (401, 38), (400, 38), (400, 37), (401, 36), (401, 37), (396, 35), (398, 35), (397, 35), (394, 7), (393, 8), (392, 8), (395, 9), (394, 8), (393, 9), (392, 10), (393, 7), (392, 7), (394, 6), (393, 6), (393, 5), (394, 5), (391, 5), (392, 6), (391, 6), (392, 5), (373, 15), (372, 15), (375, 17), (374, 28), (373, 29), (377, 28), (376, 28), (374, 49), (372, 53), (374, 64), (365, 54), (365, 53), (364, 52), (365, 45), (367, 45), (360, 42), (359, 25), (360, 25), (365, 19), (366, 20), (367, 19), (368, 20), (367, 20), (374, 15), (393, 17), (368, 13), (364, 15), (358, 23), (359, 23), (359, 22), (358, 21), (358, 22), (359, 24), (358, 18), (359, 17), (360, 14), (347, 31), (352, 5), (355, 5), (356, 5), (352, 7), (374, 22), (373, 28), (383, 28), (383, 29), (384, 26), (393, 39), (391, 28), (398, 36), (390, 28), (371, 35), (371, 40), (370, 40), (371, 41), (365, 35), (367, 34), (366, 9), (365, 8), (376, 6), (377, 5), (377, 7), (375, 5), (381, 20), (379, 7), (374, 6), (360, 10), (359, 9), (352, 6), (350, 5), (351, 5), (361, 11), (364, 6), (363, 6), (364, 5), (363, 5), (362, 5), (361, 5), (368, 14), (367, 14), (367, 13), (397, 25), (401, 16), (401, 15), (333, 14), (327, 15), (331, 15), (334, 16), (333, 24), (334, 25), (333, 25), (334, 28), (335, 27), (335, 28), (335, 26), (334, 27), (333, 28), (338, 31), (369, 42), (396, 74), (275, 70), (276, 70), (276, 67), (277, 65), (277, 73), (334, 24), (335, 25), (358, 50), (369, 39), (358, 24), (358, 38), (314, 107), (357, 50), (366, 14), (366, 45), (347, 111), (347, 112), (346, 110), (347, 109), (350, 132), (349, 133), (350, 133), (349, 134), (350, 134), (351, 133), (352, 134), (353, 132), (351, 134), (349, 132), (353, 128), (352, 133), (354, 128), (391, 134), (354, 127), (364, 72), (341, 107), (383, 25), (394, 13), (399, 34), (399, 36), (399, 35), (400, 34), (400, 35), (401, 53), (400, 50), (401, 50), (399, 48), (401, 49), (401, 48), (400, 46), (399, 47), (398, 47), (394, 41), (371, 72), (374, 40), (372, 41), (351, 8), (370, 39), (317, 110), (367, 24), (342, 6), (343, 6), (340, 30), (343, 5), (265, 6), (238, 100), (224, 160), (227, 160), (227, 161), (229, 165), (226, 161), (195, 180), (194, 195), (194, 194), (195, 195), (196, 194), (197, 194), (195, 194), (195, 202), (230, 168), (257, 163), (257, 162), (255, 164), (258, 163), (258, 162), (259, 162), (260, 163), (260, 164), (259, 161), (256, 164), (247, 158), (247, 161), (247, 162), (246, 163), (244, 167), (243, 172), (243, 171), (242, 171), (245, 160), (255, 139), (254, 141), (255, 143), (268, 142), (267, 141), (264, 140), (264, 138), (265, 139), (264, 139), (265, 138), (265, 140), (266, 139), (267, 139), (267, 138), (268, 139), (268, 140), (266, 140), (266, 138), (268, 141), (268, 143), (267, 140), (266, 141), (268, 138), (265, 141), (262, 138), (253, 139), (251, 138), (250, 138), (248, 138), (249, 138), (247, 139), (254, 148), (246, 145), (264, 158), (265, 159), (266, 159), (266, 158), (267, 157), (268, 157), (268, 156), (266, 157), (267, 158), (264, 157), (265, 157), (264, 154), (268, 152), (261, 138), (268, 147), (268, 145), (268, 146), (268, 144), (267, 155), (268, 155), (266, 155), (247, 159), (243, 154), (242, 156), (245, 154), (244, 155), (235, 176), (232, 177), (236, 174), (229, 178), (229, 177), (228, 178), (226, 178), (225, 168), (226, 168), (223, 167), (219, 179), (193, 161), (184, 166), (161, 149), (153, 147), (154, 172), (141, 150), (139, 147), (146, 143), (144, 138), (143, 138), (140, 147), (139, 184), (139, 186), (138, 186), (138, 185), (142, 183), (144, 186), (145, 187), (145, 185), (145, 184), (143, 186), (142, 186), (140, 177), (139, 176), (147, 177), (141, 177), (149, 179), (147, 178), (148, 184), (147, 184), (148, 183), (146, 185), (146, 186), (147, 186), (147, 185), (148, 185), (149, 185), (150, 186), (151, 185), (152, 175), (154, 174), (153, 159), (149, 160), (150, 159), (153, 157), (153, 153), (147, 150), (148, 150), (148, 149), (147, 151), (150, 155), (149, 154), (144, 154), (148, 154), (148, 153), (148, 152), (148, 151), (137, 155), (138, 144), (138, 143), (138, 141), (139, 141), (139, 140), (140, 140), (141, 139), (140, 139), (141, 138), (140, 138), (139, 138), (138, 138), (139, 139), (141, 140), (140, 141), (138, 142), (138, 140), (138, 139), (142, 139), (139, 143), (142, 138), (142, 140), (148, 146), (147, 146), (147, 145), (161, 177), (162, 178), (163, 177), (162, 176), (162, 177), (163, 176), (162, 175), (161, 175), (159, 170), (160, 171), (159, 174), (161, 179), (164, 178), (163, 178), (164, 179), (165, 178), (165, 179), (164, 180), (165, 181), (164, 181), (163, 181), (164, 183), (163, 182), (163, 183), (162, 183), (163, 184), (162, 184), (163, 185), (164, 186), (165, 187), (166, 193), (166, 191), (164, 192), (164, 193), (163, 192), (162, 193), (163, 194), (162, 195), (161, 194), (160, 194), (160, 193), (161, 193), (160, 192), (160, 191), (159, 192), (159, 191), (159, 190), (159, 189), (158, 189), (158, 190), (158, 188), (159, 188), (160, 189), (160, 188), (161, 189), (162, 188), (163, 189), (164, 189), (163, 190), (162, 190), (161, 188), (158, 187), (158, 186), (159, 187), (160, 186), (160, 185), (160, 184), (161, 184), (161, 183), (160, 183), (159, 184), (159, 185), (159, 186), (160, 187), (161, 186), (162, 187), (161, 190), (158, 185), (159, 183), (161, 182), (162, 182), (163, 180), (163, 179), (162, 179), (161, 178), (160, 181), (159, 180), (158, 181), (159, 181), (159, 182), (158, 183), (160, 182), (160, 180), (162, 180), (162, 181), (162, 185), (161, 185), (162, 186), (161, 187), (163, 187), (164, 188), (162, 189), (162, 191), (161, 191), (163, 191), (167, 188), (165, 189), (163, 188), (160, 190), (158, 192), (162, 192), (163, 193), (165, 191), (165, 194), (165, 195), (166, 194), (166, 181), (170, 182), (191, 181), (182, 179), (181, 178), (181, 175), (179, 174), (179, 173), (180, 172), (181, 176), (182, 178), (191, 190), (194, 193), (218, 191), (233, 225), (220, 226), (220, 229), (220, 228), (221, 228), (220, 239), (220, 240), (220, 241), (219, 240), (218, 241), (217, 240), (217, 241), (216, 242), (217, 243), (216, 244), (217, 245), (217, 244), (216, 243), (217, 242), (218, 242), (219, 241), (219, 242), (219, 243), (218, 243), (220, 244), (220, 245), (221, 245), (221, 246), (221, 247), (220, 246), (219, 246), (218, 246), (218, 245), (216, 245), (218, 244), (220, 247), (221, 248), (222, 249), (223, 248), (224, 248), (225, 248), (225, 249), (224, 249), (225, 250), (224, 250), (223, 249), (222, 250), (221, 249), (221, 250), (220, 249), (220, 250), (221, 251), (222, 252), (221, 252), (222, 253), (223, 251), (222, 248), (219, 248), (219, 247), (219, 245), (219, 249), (218, 249), (219, 250), (219, 251), (219, 252), (223, 250), (224, 251), (220, 251), (218, 251), (219, 253), (218, 252), (216, 252), (217, 251), (218, 250), (217, 249), (217, 250), (220, 252), (216, 250), (215, 249), (214, 250), (213, 249), (212, 249), (212, 250), (211, 251), (210, 251), (209, 251), (208, 251), (208, 250), (209, 250), (208, 249), (208, 248), (209, 249), (210, 250), (208, 252), (208, 253), (209, 253), (209, 252), (204, 253), (207, 253), (210, 252), (210, 253), (211, 252), (212, 251), (211, 250), (210, 249), (203, 252), (203, 251), (210, 261), (210, 264), (210, 266), (211, 267), (210, 268), (211, 268), (211, 260), (211, 259), (212, 259), (211, 258), (214, 259), (214, 263), (215, 262), (215, 260), (215, 261), (214, 267), (209, 265), (207, 267), (211, 249), (211, 248), (210, 248), (210, 247), (209, 248), (211, 246), (212, 245), (211, 245), (212, 244), (213, 243), (214, 244), (215, 245), (215, 246), (214, 247), (215, 248), (214, 249), (214, 251), (215, 250), (215, 251), (215, 252), (217, 252), (216, 251), (213, 251), (213, 250), (216, 248), (216, 249), (214, 252), (217, 248), (218, 247), (217, 247), (216, 247), (214, 248), (213, 247), (213, 248), (212, 248), (203, 249), (206, 245), (209, 239), (210, 238), (211, 238), (210, 237), (210, 239), (212, 237), (211, 237), (210, 236), (211, 236), (210, 234), (210, 235), (209, 236), (211, 235), (211, 231), (213, 232), (212, 231), (214, 231), (210, 231), (209, 237), (205, 240), (209, 247), (203, 256), (200, 256), (195, 245), (196, 245), (197, 241), (197, 243), (197, 237), (196, 237), (195, 238), (194, 237), (195, 236), (195, 237), (194, 236), (193, 236), (188, 237), (193, 239), (193, 240), (194, 239), (194, 238), (193, 237), (193, 234), (193, 235), (196, 232), (195, 232), (192, 232), (191, 231), (192, 230), (192, 229), (191, 229), (190, 230), (191, 230), (188, 229), (188, 230), (189, 231), (190, 231), (189, 230), (187, 230), (188, 231), (187, 232), (188, 233), (189, 233), (190, 234), (190, 233), (191, 233), (189, 232), (187, 231), (187, 229), (186, 228), (186, 229), (186, 230), (185, 229), (186, 231), (185, 231), (186, 232), (191, 232), (189, 234), (188, 234), (187, 234), (187, 235), (193, 227), (187, 225), (185, 222), (185, 221), (184, 221), (183, 220), (184, 222), (186, 219), (191, 219), (190, 220), (190, 222), (195, 223), (195, 221), (196, 221), (192, 220), (193, 219), (195, 230), (195, 227), (194, 223), (192, 231), (193, 232), (190, 232), (188, 232), (187, 233), (186, 234), (186, 233), (185, 232), (184, 229), (182, 230), (181, 230), (181, 229), (184, 227), (185, 228), (187, 228), (196, 231), (195, 231), (198, 227), (199, 227), (200, 226), (200, 225), (200, 224), (199, 224), (201, 225), (202, 225), (203, 225), (203, 224), (204, 224), (205, 224), (207, 223), (208, 224), (208, 225), (207, 226), (208, 227), (206, 226), (205, 227), (204, 227), (203, 226), (201, 224), (201, 223), (202, 224), (202, 223), (200, 223), (199, 225), (199, 226), (201, 233), (200, 233), (202, 234), (200, 237), (205, 243), (206, 242), (208, 242), (209, 243), (210, 241), (210, 246), (211, 247), (212, 246), (213, 244), (215, 243), (214, 242), (213, 241), (214, 241), (213, 240), (214, 240), (212, 242), (212, 241), (211, 240), (211, 239), (210, 240), (209, 240), (209, 241), (211, 242), (211, 241), (212, 247), (213, 246), (214, 246), (214, 245), (213, 245), (216, 246), (217, 246), (215, 247), (218, 248), (219, 244), (220, 243), (222, 240), (222, 239), (223, 238), (223, 237), (232, 236), (232, 235), (256, 205), (268, 185), (266, 182), (268, 182), (267, 181), (267, 182), (263, 184), (261, 181), (260, 181), (262, 165), (264, 162), (261, 160), (264, 160), (268, 162), (265, 161), (265, 160), (266, 160), (267, 161), (267, 160), (268, 160), (264, 161), (257, 173), (262, 180), (263, 181), (266, 180), (265, 183), (266, 181), (267, 180), (268, 179), (268, 178), (266, 179), (267, 179), (265, 178), (266, 177), (267, 176), (267, 177), (268, 180), (268, 177), (268, 176), (268, 175), (267, 175), (265, 175), (264, 175), (265, 174), (264, 176), (265, 177), (268, 161), (268, 159), (267, 159), (259, 164), (256, 163), (259, 163), (263, 175), (256, 162), (265, 158), (247, 185), (229, 176), (231, 175), (267, 178), (266, 178), (268, 174), (268, 181), (267, 156), (268, 158), (255, 179), (248, 161), (241, 155), (237, 155), (240, 156), (254, 138), (252, 138), (246, 138), (245, 138), (268, 148), (253, 138), (236, 211), (180, 174), (175, 175), (178, 176), (176, 174), (178, 172), (176, 170), (175, 171), (176, 171), (177, 171), (177, 168), (176, 169), (175, 169), (176, 173), (175, 174), (170, 174), (169, 174), (169, 175), (169, 176), (175, 170), (184, 139), (186, 168), (183, 206), (187, 205), (188, 205), (189, 206), (188, 206), (189, 205), (186, 206), (185, 206), (185, 207), (184, 206), (183, 207), (182, 202), (181, 203), (181, 202), (178, 201), (178, 202), (177, 202), (177, 201), (175, 201), (176, 202), (174, 201), (192, 146), (175, 173), (175, 172), (174, 179), (168, 178), (168, 179), (168, 180), (178, 181), (233, 180), (219, 186), (218, 201), (225, 219), (180, 202), (180, 203), (179, 203), (179, 201), (174, 173), (161, 181), (158, 182), (157, 181), (156, 181), (157, 182), (157, 185), (164, 190), (167, 194), (172, 200), (180, 204), (180, 207), (181, 208), (181, 209), (182, 210), (182, 211), (184, 212), (188, 209), (187, 208), (185, 208), (184, 208), (188, 207), (190, 204), (190, 203), (193, 206), (192, 207), (192, 208), (192, 209), (191, 209), (191, 208), (191, 207), (190, 206), (190, 207), (190, 208), (192, 210), (193, 211), (200, 216), (192, 204), (189, 204), (174, 172), (176, 172), (171, 174), (152, 153), (151, 152), (152, 152), (153, 158), (161, 158), (151, 150), (150, 149), (149, 149), (150, 152), (151, 151), (149, 152), (143, 141), (153, 148), (152, 147), (151, 147), (151, 148), (150, 148), (150, 150), (149, 150), (149, 151), (136, 155), (135, 155), (134, 154), (134, 153), (133, 154), (133, 153), (134, 155), (133, 155), (132, 154), (132, 155), (134, 152), (133, 152), (133, 156), (134, 156), (134, 151), (133, 150), (133, 151), (132, 152), (132, 151), (132, 156), (131, 157), (132, 158), (131, 158), (131, 159), (132, 160), (133, 159), (133, 158), (134, 158), (134, 159), (133, 157), (130, 157), (130, 156), (130, 155), (129, 157), (130, 158), (130, 159), (131, 160), (132, 161), (132, 162), (131, 162), (130, 161), (132, 163), (132, 164), (131, 165), (130, 164), (130, 165), (130, 163), (131, 164), (131, 163), (132, 165), (133, 164), (134, 164), (134, 165), (133, 166), (132, 166), (133, 167), (132, 168), (131, 168), (133, 169), (134, 168), (134, 167), (134, 166), (134, 169), (133, 168), (133, 165), (130, 166), (131, 166), (130, 167), (131, 167), (129, 166), (134, 163), (129, 165), (129, 164), (128, 165), (128, 166), (128, 167), (128, 168), (129, 169), (129, 170), (130, 169), (131, 169), (130, 168), (134, 170), (133, 170), (134, 171), (133, 171), (134, 172), (132, 172), (132, 170), (132, 167), (129, 163), (128, 164), (127, 164), (127, 163), (126, 163), (127, 162), (127, 161), (128, 161), (128, 163), (126, 162), (126, 161), (127, 160), (128, 160), (125, 163), (124, 163), (123, 163), (122, 164), (121, 163), (122, 163), (125, 164), (125, 165), (124, 165), (126, 164), (127, 167), (127, 166), (127, 165), (124, 162), (123, 164), (123, 165), (123, 166), (122, 167), (121, 166), (120, 165), (119, 164), (120, 164), (120, 166), (119, 165), (118, 166), (119, 167), (119, 166), (118, 167), (117, 166), (116, 166), (115, 166), (116, 167), (117, 167), (118, 168), (120, 168), (119, 168), (117, 168), (118, 169), (117, 170), (117, 169), (116, 170), (118, 171), (117, 171), (118, 170), (119, 170), (119, 171), (117, 172), (118, 173), (119, 173), (118, 172), (120, 172), (121, 172), (122, 171), (123, 170), (123, 169), (124, 169), (124, 170), (125, 169), (125, 170), (124, 168), (124, 167), (125, 166), (126, 165), (124, 164), (124, 166), (126, 166), (125, 167), (125, 168), (126, 168), (127, 168), (125, 171), (126, 169), (126, 170), (127, 171), (123, 168), (126, 167), (123, 167), (122, 166), (121, 165), (121, 164), (121, 162), (120, 163), (121, 167), (122, 165), (125, 162), (125, 161), (127, 170), (128, 170), (127, 169), (126, 172), (125, 172), (124, 172), (123, 172), (124, 171), (126, 171), (125, 173), (126, 173), (125, 174), (124, 174), (123, 173), (124, 173), (126, 174), (126, 175), (124, 175), (124, 176), (125, 176), (124, 177), (125, 175), (123, 175), (123, 174), (122, 175), (121, 176), (121, 175), (120, 176), (121, 177), (122, 177), (123, 177), (124, 178), (122, 176), (120, 175), (120, 174), (120, 178), (119, 177), (118, 176), (119, 176), (118, 175), (119, 175), (120, 177), (121, 174), (119, 174), (118, 174), (119, 172), (120, 171), (120, 170), (119, 169), (120, 169), (118, 165), (118, 164), (116, 168), (116, 169), (115, 169), (114, 168), (113, 168), (115, 167), (114, 167), (114, 169), (112, 167), (113, 167), (112, 166), (113, 166), (111, 167), (111, 166), (112, 165), (114, 166), (113, 165), (112, 164), (113, 163), (113, 164), (114, 164), (115, 168), (114, 165), (114, 163), (115, 163), (116, 163), (116, 162), (115, 162), (114, 161), (113, 160), (112, 159), (112, 160), (111, 160), (110, 159), (109, 160), (109, 161), (108, 161), (108, 160), (109, 159), (108, 158), (108, 157), (107, 157), (106, 156), (106, 157), (107, 158), (106, 158), (105, 158), (105, 157), (105, 156), (104, 155), (103, 154), (103, 155), (103, 156), (104, 157), (104, 156), (104, 154), (104, 153), (105, 154), (105, 153), (106, 154), (107, 153), (107, 155), (107, 156), (108, 155), (108, 156), (109, 158), (109, 157), (109, 156), (107, 159), (106, 160), (106, 155), (108, 154), (107, 154), (109, 155), (110, 155), (109, 154), (110, 154), (111, 155), (112, 155), (111, 154), (110, 153), (109, 152), (109, 153), (113, 155), (112, 156), (111, 156), (108, 153), (106, 152), (106, 153), (105, 152), (104, 151), (105, 151), (103, 151), (102, 151), (101, 150), (100, 150), (99, 151), (98, 152), (99, 152), (100, 152), (100, 151), (101, 151), (101, 152), (102, 153), (103, 152), (104, 152), (103, 153), (106, 151), (107, 152), (105, 155), (102, 154), (101, 154), (102, 155), (101, 153), (102, 152), (100, 153), (100, 154), (104, 150), (103, 149), (107, 151), (108, 152), (109, 151), (110, 150), (109, 149), (109, 150), (110, 149), (111, 152), (110, 152), (108, 150), (108, 151), (106, 150), (105, 149), (104, 149), (104, 148), (103, 150), (102, 149), (101, 149), (100, 149), (99, 150), (98, 151), (97, 152), (96, 151), (95, 150), (94, 150), (94, 149), (93, 150), (93, 149), (93, 148), (93, 147), (92, 147), (92, 146), (91, 146), (90, 147), (89, 146), (90, 145), (91, 144), (90, 144), (90, 143), (91, 143), (90, 142), (92, 143), (93, 142), (93, 141), (94, 141), (95, 141), (95, 140), (94, 139), (93, 138), (92, 139), (93, 139), (93, 140), (94, 140), (92, 138), (91, 139), (91, 140), (90, 140), (90, 139), (89, 138), (90, 138), (91, 138), (95, 139), (95, 138), (94, 138), (96, 139), (97, 138), (96, 138), (89, 139), (88, 139), (87, 139), (87, 138), (86, 140), (85, 140), (84, 139), (85, 138), (86, 139), (85, 139), (87, 140), (86, 141), (85, 142), (86, 142), (87, 141), (84, 141), (84, 140), (85, 141), (84, 142), (83, 143), (82, 142), (81, 141), (82, 140), (83, 139), (82, 138), (81, 139), (82, 139), (83, 140), (83, 142), (83, 141), (81, 138), (80, 139), (79, 139), (81, 140), (84, 138), (80, 138), (82, 141), (82, 143), (81, 142), (81, 143), (80, 142), (79, 141), (79, 140), (80, 141), (78, 141), (79, 138), (78, 138), (77, 138), (78, 137), (78, 136), (77, 139), (76, 140), (77, 140), (76, 139), (76, 141), (77, 142), (76, 142), (76, 143), (75, 143), (74, 144), (73, 145), (72, 145), (72, 146), (75, 145), (75, 144), (76, 144), (77, 145), (76, 146), (78, 144), (79, 143), (78, 143), (79, 144), (80, 143), (80, 145), (80, 144), (79, 145), (78, 146), (77, 147), (77, 148), (76, 149), (77, 149), (78, 148), (79, 148), (79, 147), (78, 147), (79, 149), (80, 150), (81, 150), (80, 151), (81, 152), (80, 153), (79, 153), (80, 152), (81, 151), (80, 154), (82, 152), (82, 151), (83, 152), (83, 153), (84, 153), (84, 154), (84, 155), (85, 155), (84, 156), (83, 156), (85, 154), (86, 155), (87, 155), (86, 154), (86, 153), (86, 152), (87, 151), (86, 150), (87, 149), (86, 149), (85, 149), (87, 148), (86, 147), (86, 148), (88, 149), (87, 150), (89, 150), (88, 150), (89, 149), (88, 152), (89, 151), (90, 152), (90, 149), (89, 148), (88, 148), (88, 147), (90, 146), (89, 145), (89, 144), (89, 143), (89, 142), (89, 141), (88, 141), (88, 142), (88, 143), (88, 140), (87, 142), (90, 141), (91, 145), (93, 145), (94, 144), (95, 143), (94, 142), (96, 142), (95, 142), (96, 141), (97, 140), (98, 139), (98, 138), (99, 139), (100, 139), (99, 140), (99, 141), (98, 141), (99, 142), (98, 142), (100, 143), (101, 143), (100, 144), (100, 145), (101, 145), (100, 146), (99, 146), (98, 147), (99, 148), (99, 147), (100, 147), (100, 148), (99, 153), (99, 154), (98, 154), (98, 153), (97, 154), (99, 155), (97, 151), (96, 152), (95, 153), (96, 153), (97, 153), (97, 155), (95, 151), (93, 151), (95, 152), (94, 151), (92, 151), (92, 152), (91, 152), (90, 151), (90, 150), (88, 151), (85, 151), (84, 152), (85, 152), (86, 151), (85, 150), (84, 150), (87, 152), (88, 153), (89, 154), (88, 154), (89, 153), (87, 153), (89, 147), (88, 146), (84, 149), (84, 151), (85, 153), (89, 152), (90, 153), (91, 151), (91, 150), (91, 149), (92, 150), (92, 149), (90, 154), (89, 155), (88, 156), (89, 156), (88, 155), (87, 156), (87, 154), (86, 157), (86, 156), (89, 157), (89, 158), (88, 157), (89, 159), (88, 160), (88, 161), (89, 162), (88, 162), (89, 161), (89, 163), (87, 163), (87, 162), (86, 163), (85, 164), (84, 164), (83, 165), (84, 165), (83, 166), (83, 167), (84, 167), (85, 168), (86, 168), (87, 167), (86, 169), (87, 168), (87, 166), (88, 167), (88, 166), (88, 165), (88, 164), (87, 165), (89, 164), (90, 165), (90, 164), (90, 163), (91, 162), (90, 162), (91, 164), (92, 164), (91, 163), (92, 163), (93, 164), (94, 163), (93, 163), (94, 162), (95, 163), (95, 162), (93, 161), (94, 161), (95, 161), (92, 160), (93, 160), (92, 161), (92, 159), (91, 159), (91, 160), (92, 162), (94, 164), (93, 162), (95, 160), (96, 160), (96, 159), (95, 159), (95, 158), (94, 157), (94, 158), (93, 158), (92, 158), (93, 157), (94, 156), (95, 156), (96, 157), (96, 158), (95, 157), (94, 159), (97, 160), (98, 159), (99, 159), (97, 159), (98, 158), (99, 158), (100, 158), (101, 159), (102, 160), (102, 161), (101, 162), (101, 161), (102, 162), (100, 163), (99, 163), (98, 163), (98, 164), (97, 165), (96, 164), (96, 165), (95, 165), (94, 166), (93, 167), (92, 166), (91, 165), (91, 167), (90, 166), (92, 165), (91, 166), (92, 167), (91, 168), (90, 167), (89, 166), (87, 164), (89, 165), (88, 163), (87, 161), (87, 160), (88, 159), (90, 161), (90, 160), (89, 160), (90, 159), (90, 158), (88, 158), (85, 156), (87, 157), (90, 155), (91, 154), (91, 155), (92, 155), (92, 156), (93, 156), (96, 156), (97, 157), (97, 156), (98, 157), (99, 156), (98, 156), (95, 155), (95, 154), (94, 153), (94, 152), (93, 153), (93, 154), (93, 152), (94, 148), (94, 147), (94, 146), (93, 146), (92, 148), (91, 147), (92, 145), (92, 144), (93, 144), (92, 142), (97, 142), (97, 141), (94, 143), (94, 145), (95, 144), (96, 145), (97, 146), (96, 147), (97, 147), (97, 148), (96, 146), (95, 145), (96, 144), (96, 143), (97, 143), (98, 143), (100, 140), (101, 138), (98, 140), (97, 139), (99, 138), (96, 140), (100, 142), (100, 141), (100, 138), (101, 139), (92, 140), (91, 141), (93, 143), (88, 145), (88, 144), (87, 144), (87, 143), (86, 144), (84, 143), (84, 144), (83, 145), (82, 146), (82, 145), (82, 147), (83, 147), (84, 146), (84, 147), (84, 148), (85, 147), (85, 148), (83, 148), (83, 146), (81, 146), (81, 147), (82, 148), (80, 146), (81, 145), (81, 144), (86, 138), (89, 140), (91, 142), (80, 140), (85, 143), (83, 144), (82, 144), (86, 143), (85, 144), (86, 145), (87, 145), (87, 146), (91, 148), (95, 148), (96, 148), (96, 149), (95, 149), (95, 146), (96, 150), (96, 154), (92, 153), (91, 153), (93, 165), (91, 161), (93, 159), (94, 160), (93, 155), (92, 154), (94, 155), (94, 154), (98, 155), (97, 150), (97, 149), (99, 157), (100, 157), (100, 159), (101, 160), (100, 160), (99, 160), (98, 160), (100, 161), (103, 161), (104, 161), (104, 160), (103, 159), (104, 159), (103, 158), (102, 157), (103, 157), (101, 156), (100, 156), (100, 155), (97, 158), (101, 158), (102, 158), (102, 159), (97, 161), (98, 161), (99, 162), (99, 164), (97, 163), (98, 165), (99, 165), (100, 166), (99, 167), (97, 166), (98, 166), (99, 166), (100, 167), (101, 166), (101, 167), (102, 167), (103, 168), (103, 169), (102, 170), (102, 171), (101, 171), (100, 170), (99, 171), (99, 172), (99, 173), (98, 173), (97, 174), (96, 174), (96, 175), (97, 175), (98, 176), (99, 176), (100, 175), (98, 175), (100, 176), (99, 174), (99, 175), (100, 177), (101, 176), (101, 175), (102, 176), (101, 177), (102, 175), (102, 174), (101, 173), (101, 172), (100, 171), (101, 170), (102, 172), (103, 172), (104, 172), (104, 173), (104, 174), (103, 174), (103, 175), (102, 177), (99, 178), (99, 179), (98, 178), (98, 177), (97, 178), (97, 179), (96, 180), (95, 179), (95, 178), (95, 177), (96, 177), (94, 178), (94, 177), (93, 176), (92, 175), (93, 174), (91, 174), (90, 173), (90, 172), (89, 173), (89, 172), (91, 173), (91, 172), (90, 171), (89, 171), (88, 172), (88, 173), (87, 172), (87, 173), (86, 174), (86, 175), (85, 174), (85, 175), (87, 175), (88, 174), (88, 175), (88, 176), (89, 177), (90, 178), (91, 177), (91, 176), (92, 177), (90, 177), (90, 176), (91, 175), (92, 174), (92, 173), (90, 170), (91, 170), (86, 173), (88, 170), (88, 171), (87, 170), (89, 169), (89, 170), (87, 171), (86, 167), (88, 168), (87, 169), (84, 168), (86, 170), (88, 169), (92, 169), (92, 170), (91, 171), (91, 169), (90, 168), (90, 169), (89, 168), (86, 164), (85, 163), (84, 162), (84, 161), (83, 161), (82, 162), (81, 161), (82, 161), (83, 160), (82, 160), (85, 161), (85, 160), (85, 159), (86, 159), (87, 159), (87, 158), (86, 158), (86, 160), (85, 162), (86, 165), (86, 166), (96, 155), (98, 149), (98, 148), (95, 147), (90, 148), (87, 147), (85, 146), (86, 146), (85, 145), (84, 145), (82, 149), (82, 150), (83, 149), (81, 148), (80, 147), (79, 146), (78, 145), (77, 146), (76, 148), (78, 149), (79, 150), (80, 149), (83, 150), (83, 151), (90, 157), (91, 157), (92, 157), (94, 165), (95, 166), (95, 167), (96, 168), (96, 169), (97, 168), (95, 170), (96, 171), (97, 171), (96, 170), (96, 172), (96, 173), (97, 172), (98, 172), (95, 173), (94, 173), (93, 173), (94, 174), (100, 179), (101, 178), (102, 178), (102, 179), (101, 180), (103, 178), (104, 179), (103, 179), (102, 180), (101, 181), (102, 182), (100, 180), (101, 182), (102, 183), (103, 181), (104, 181), (105, 181), (105, 182), (104, 182), (104, 180), (105, 179), (106, 180), (106, 179), (107, 178), (107, 177), (108, 176), (107, 175), (108, 174), (109, 174), (109, 173), (108, 173), (107, 172), (106, 173), (105, 172), (105, 171), (104, 170), (103, 171), (102, 169), (101, 168), (102, 166), (103, 167), (103, 166), (102, 165), (103, 164), (102, 164), (103, 163), (104, 164), (105, 164), (106, 163), (107, 164), (108, 163), (108, 164), (109, 163), (110, 162), (110, 161), (109, 162), (108, 162), (107, 162), (106, 161), (105, 162), (104, 163), (105, 163), (103, 165), (101, 165), (101, 163), (102, 163), (104, 169), (105, 170), (105, 169), (105, 168), (106, 167), (107, 167), (107, 168), (106, 166), (105, 165), (106, 164), (105, 167), (107, 166), (106, 165), (104, 162), (105, 161), (106, 162), (107, 160), (104, 158), (103, 160), (96, 161), (96, 162), (96, 163), (97, 162), (99, 161), (98, 162), (100, 162), (103, 162), (91, 158), (95, 164), (93, 166), (94, 167), (93, 168), (86, 171), (85, 170), (84, 171), (84, 170), (83, 169), (83, 168), (84, 169), (82, 170), (82, 171), (81, 171), (80, 170), (81, 170), (80, 171), (81, 169), (82, 168), (82, 169), (81, 167), (81, 166), (81, 165), (82, 164), (82, 165), (82, 166), (82, 167), (83, 168), (84, 167), (83, 166), (83, 167), (84, 166), (85, 167), (86, 167), (87, 168), (88, 167), (89, 168), (88, 168), (87, 167), (89, 167), (90, 166), (90, 165), (91, 165), (92, 165), (92, 164), (89, 166), (88, 165), (87, 165), (88, 164), (89, 164), (90, 164), (89, 165), (88, 163), (87, 164), (86, 164), (85, 165), (86, 166), (88, 166), (87, 166), (85, 166), (84, 165), (83, 165), (85, 168), (84, 168), (82, 168), (83, 169), (86, 168), (87, 169), (88, 169), (87, 170), (88, 171), (89, 170), (90, 169), (89, 169), (91, 169), (90, 168), (90, 167), (90, 170), (91, 171), (92, 172), (91, 172), (90, 172), (89, 171), (92, 170), (91, 170), (92, 169), (93, 170), (93, 169), (93, 168), (94, 167), (93, 166), (93, 167), (92, 168), (92, 167), (95, 168), (94, 169), (94, 168), (94, 170), (93, 171), (94, 166), (94, 165), (95, 165), (95, 166), (96, 167), (97, 166), (96, 165), (97, 164), (98, 164), (97, 165), (98, 163), (99, 162), (100, 161), (100, 162), (101, 162), (100, 160), (99, 161), (101, 161), (102, 161), (102, 162), (101, 163), (100, 163), (99, 163), (98, 162), (101, 164), (101, 165), (100, 165), (102, 163), (100, 164), (99, 164), (99, 165), (98, 165), (97, 163), (103, 161), (103, 160), (103, 159), (104, 160), (104, 161), (105, 160), (105, 161), (106, 162), (106, 163), (107, 163), (105, 162), (104, 163), (105, 163), (104, 164), (104, 165), (104, 166), (105, 167), (104, 168), (103, 169), (103, 170), (104, 170), (104, 167), (103, 164), (102, 165), (105, 165), (106, 164), (105, 164), (107, 162), (106, 161), (106, 160), (104, 162), (103, 165), (103, 166), (105, 166), (106, 165), (106, 166), (107, 167), (106, 168), (105, 168), (105, 169), (104, 169), (102, 170), (103, 168), (102, 168), (101, 167), (102, 167), (100, 166), (102, 166), (103, 167), (101, 166), (101, 168), (102, 169), (102, 171), (104, 171), (105, 171), (104, 172), (103, 173), (104, 173), (105, 173), (105, 174), (105, 175), (106, 176), (105, 176), (106, 175), (104, 174), (106, 174), (106, 173), (107, 174), (108, 173), (108, 172), (109, 172), (108, 174), (108, 175), (107, 176), (109, 175), (109, 176), (108, 177), (108, 176), (106, 177), (107, 177), (107, 178), (106, 178), (107, 175), (107, 172), (108, 171), (109, 170), (109, 171), (110, 172), (109, 173), (109, 174), (107, 171), (106, 172), (105, 170), (103, 172), (103, 171), (106, 171), (107, 170), (108, 170), (109, 169), (108, 168), (109, 168), (106, 170), (106, 169), (107, 169), (108, 169), (107, 168), (110, 169), (110, 171), (111, 170), (112, 170), (111, 171), (110, 173), (110, 175), (110, 176), (111, 176), (111, 175), (112, 175), (113, 175), (114, 174), (113, 174), (112, 174), (110, 174), (103, 175), (104, 176), (104, 175), (104, 177), (107, 173), (105, 172), (109, 177), (109, 178), (110, 179), (111, 178), (110, 177), (111, 177), (105, 178), (104, 179), (105, 179), (105, 180), (105, 181), (104, 180), (103, 179), (103, 178), (103, 177), (102, 178), (101, 178), (100, 178), (101, 179), (101, 180), (100, 181), (99, 180), (100, 179), (100, 177), (101, 177), (100, 176), (100, 175), (101, 176), (102, 176), (102, 177), (99, 177), (101, 175), (104, 178), (102, 180), (102, 181), (101, 181), (102, 179), (103, 180), (100, 180), (99, 179), (104, 182), (103, 181), (103, 182), (102, 182), (102, 183), (103, 183), (101, 182), (101, 183), (104, 181), (100, 183), (99, 182), (100, 182), (99, 183), (99, 184), (99, 185), (100, 184), (101, 185), (101, 184), (98, 184), (100, 185), (99, 186), (98, 186), (100, 186), (98, 185), (98, 183), (103, 176), (102, 175), (99, 176), (98, 177), (98, 178), (104, 183), (105, 183), (106, 182), (107, 183), (106, 184), (106, 185), (105, 184), (104, 185), (103, 185), (104, 184), (106, 181), (106, 179), (107, 179), (108, 179), (107, 180), (108, 180), (109, 181), (108, 182), (108, 183), (107, 184), (106, 183), (105, 186), (104, 187), (104, 186), (105, 185), (106, 186), (103, 187), (105, 187), (105, 188), (104, 189), (104, 188), (102, 188), (103, 188), (105, 190), (106, 189), (107, 190), (108, 190), (107, 191), (106, 192), (105, 193), (105, 194), (104, 193), (104, 194), (106, 193), (107, 192), (108, 192), (105, 192), (104, 195), (105, 195), (105, 196), (104, 197), (103, 198), (104, 199), (105, 199), (104, 200), (103, 200), (102, 200), (101, 200), (100, 200), (100, 201), (101, 202), (102, 201), (102, 202), (102, 203), (103, 202), (102, 204), (103, 203), (101, 201), (101, 199), (100, 199), (100, 198), (99, 197), (99, 198), (100, 197), (99, 199), (98, 200), (97, 200), (97, 199), (99, 200), (99, 201), (99, 202), (99, 203), (98, 203), (97, 203), (98, 202), (98, 204), (98, 205), (99, 204), (99, 205), (97, 205), (97, 206), (96, 206), (97, 207), (96, 208), (95, 208), (94, 208), (95, 209), (95, 210), (94, 210), (95, 211), (96, 211), (97, 211), (98, 210), (97, 210), (96, 210), (95, 212), (96, 212), (97, 212), (96, 213), (94, 211), (96, 207), (93, 208), (93, 209), (95, 207), (97, 208), (96, 209), (94, 207), (94, 209), (92, 207), (91, 206), (90, 207), (90, 208), (91, 209), (91, 208), (90, 209), (89, 208), (88, 209), (89, 209), (90, 210), (88, 208), (88, 207), (88, 206), (89, 207), (91, 207), (90, 206), (89, 206), (89, 205), (88, 204), (89, 204), (87, 205), (88, 205), (89, 211), (90, 211), (91, 211), (91, 212), (91, 213), (92, 211), (93, 211), (92, 210), (91, 210), (92, 209), (92, 208), (89, 210), (88, 210), (95, 206), (97, 209), (98, 211), (97, 213), (97, 214), (97, 215), (98, 214), (99, 215), (100, 214), (99, 214), (98, 215), (98, 213), (98, 216), (97, 216), (98, 217), (99, 218), (99, 219), (100, 218), (100, 219), (98, 219), (98, 220), (101, 217), (102, 217), (103, 217), (102, 218), (101, 218), (101, 219), (101, 220), (102, 220), (103, 219), (102, 219), (103, 218), (102, 216), (103, 215), (104, 216), (104, 215), (105, 214), (106, 215), (106, 216), (107, 216), (106, 214), (107, 215), (108, 215), (108, 216), (107, 217), (106, 217), (105, 218), (105, 217), (105, 216), (104, 217), (103, 216), (105, 219), (106, 220), (106, 221), (105, 220), (105, 221), (106, 222), (106, 223), (107, 224), (106, 225), (106, 226), (107, 227), (108, 226), (109, 227), (108, 227), (109, 228), (107, 226), (108, 225), (109, 225), (108, 224), (107, 225), (105, 226), (104, 226), (103, 225), (103, 227), (103, 228), (104, 228), (105, 228), (104, 227), (104, 229), (104, 230), (103, 231), (102, 232), (102, 233), (101, 233), (102, 234), (100, 234), (99, 233), (98, 234), (97, 234), (98, 235), (99, 234), (99, 235), (99, 236), (98, 237), (98, 238), (97, 238), (97, 237), (96, 236), (95, 235), (97, 235), (98, 232), (97, 232), (98, 231), (97, 233), (96, 233), (96, 232), (95, 233), (94, 232), (93, 232), (92, 232), (92, 233), (91, 232), (90, 233), (89, 233), (88, 233), (87, 234), (86, 233), (87, 233), (86, 234), (85, 233), (85, 234), (84, 235), (83, 236), (84, 236), (85, 235), (86, 236), (86, 237), (87, 237), (88, 237), (88, 236), (89, 236), (89, 235), (90, 234), (91, 235), (92, 235), (91, 234), (91, 233), (90, 235), (87, 238), (86, 238), (85, 239), (87, 239), (88, 238), (89, 239), (89, 240), (88, 239), (90, 241), (91, 241), (91, 242), (90, 242), (92, 240), (93, 241), (93, 240), (94, 239), (95, 239), (96, 240), (97, 239), (97, 240), (98, 239), (96, 239), (95, 238), (95, 240), (94, 240), (93, 239), (98, 240), (96, 238), (99, 240), (100, 241), (101, 240), (101, 241), (100, 242), (101, 243), (102, 242), (103, 241), (102, 241), (103, 240), (103, 239), (104, 239), (104, 240), (105, 241), (106, 242), (105, 243), (106, 243), (106, 244), (106, 245), (106, 246), (107, 246), (105, 245), (107, 247), (106, 247), (107, 245), (105, 246), (104, 247), (108, 245), (109, 245), (109, 246), (108, 246), (109, 247), (108, 247), (109, 248), (108, 248), (106, 248), (105, 248), (107, 249), (106, 250), (105, 251), (104, 250), (103, 251), (102, 252), (101, 251), (101, 252), (100, 251), (100, 252), (100, 250), (101, 250), (100, 249), (99, 250), (99, 253), (100, 254), (101, 255), (102, 254), (102, 253), (103, 254), (104, 253), (104, 252), (105, 249), (106, 251), (105, 250), (105, 252), (106, 253), (105, 254), (105, 255), (104, 255), (104, 256), (103, 256), (102, 255), (101, 256), (100, 257), (99, 256), (98, 255), (99, 255), (100, 256), (102, 257), (102, 258), (101, 257), (99, 257), (100, 258), (101, 258), (101, 259), (100, 259), (100, 260), (99, 259), (98, 260), (98, 261), (98, 262), (99, 261), (97, 260), (96, 261), (97, 259), (96, 258), (95, 257), (94, 256), (93, 257), (92, 258), (91, 257), (92, 257), (93, 258), (93, 259), (92, 259), (92, 260), (91, 258), (91, 259), (90, 260), (89, 259), (90, 259), (91, 260), (91, 261), (92, 262), (91, 263), (92, 261), (93, 262), (92, 263), (93, 264), (94, 265), (93, 266), (95, 264), (96, 265), (97, 266), (98, 267), (98, 268), (99, 268), (99, 267), (97, 267), (97, 268), (98, 266), (97, 265), (97, 264), (96, 263), (95, 262), (94, 261), (94, 262), (95, 261), (95, 260), (94, 259), (94, 260), (93, 260), (94, 257), (93, 256), (89, 261), (89, 260), (88, 260), (87, 260), (86, 259), (86, 260), (86, 261), (87, 262), (88, 261), (86, 258), (85, 258), (86, 257), (85, 259), (84, 259), (84, 258), (83, 258), (82, 259), (81, 259), (80, 258), (79, 257), (80, 257), (81, 257), (80, 256), (81, 256), (82, 257), (80, 255), (79, 256), (81, 258), (83, 257), (82, 256), (79, 255), (80, 254), (79, 253), (78, 253), (77, 252), (76, 251), (77, 251), (76, 252), (75, 253), (75, 252), (75, 254), (74, 253), (73, 253), (72, 253), (73, 252), (74, 254), (73, 254), (74, 255), (74, 256), (75, 257), (76, 257), (76, 256), (77, 257), (78, 256), (79, 254), (78, 254), (78, 252), (79, 251), (80, 250), (81, 249), (82, 250), (81, 250), (81, 251), (81, 248), (80, 249), (79, 249), (79, 248), (78, 248), (77, 247), (77, 246), (76, 245), (76, 244), (77, 243), (77, 242), (78, 243), (79, 244), (78, 244), (80, 243), (79, 243), (77, 245), (79, 242), (80, 242), (81, 241), (81, 240), (82, 240), (83, 239), (82, 238), (82, 237), (81, 238), (82, 239), (83, 241), (82, 242), (83, 243), (82, 244), (82, 245), (83, 244), (81, 244), (81, 245), (80, 244), (80, 245), (79, 245), (79, 246), (78, 247), (78, 245), (80, 246), (79, 247), (78, 246), (77, 248), (80, 247), (80, 248), (78, 249), (77, 249), (79, 250), (80, 251), (78, 251), (77, 253), (76, 253), (75, 255), (75, 256), (74, 252), (78, 250), (77, 250), (77, 254), (76, 254), (76, 255), (77, 255), (79, 252), (80, 252), (76, 248), (75, 249), (74, 250), (74, 249), (73, 249), (75, 250), (76, 249), (76, 250), (75, 248), (74, 247), (75, 246), (74, 246), (73, 247), (74, 248), (75, 247), (74, 245), (74, 244), (75, 244), (75, 245), (76, 246), (77, 244), (78, 242), (78, 241), (79, 241), (79, 240), (78, 240), (78, 239), (77, 238), (77, 237), (77, 236), (76, 237), (76, 235), (78, 236), (79, 235), (79, 234), (80, 233), (79, 232), (79, 233), (78, 233), (77, 234), (78, 235), (77, 235), (78, 237), (79, 238), (80, 239), (77, 241), (76, 242), (76, 241), (75, 240), (74, 239), (73, 239), (72, 240), (72, 241), (73, 242), (71, 240), (71, 241), (70, 241), (71, 242), (71, 243), (73, 241), (74, 240), (74, 241), (74, 242), (75, 241), (76, 240), (77, 239), (76, 239), (77, 240), (75, 239), (74, 238), (75, 238), (74, 237), (75, 236), (74, 235), (75, 235), (73, 236), (76, 238), (78, 238), (79, 237), (80, 237), (80, 238), (81, 237), (81, 236), (82, 236), (80, 236), (81, 235), (80, 234), (80, 235), (81, 234), (83, 238), (83, 237), (82, 235), (82, 234), (82, 233), (83, 232), (82, 232), (83, 231), (82, 231), (83, 233), (84, 232), (84, 233), (84, 234), (85, 232), (86, 235), (87, 236), (87, 235), (87, 232), (86, 231), (86, 232), (88, 235), (84, 237), (85, 238), (89, 238), (89, 237), (90, 236), (91, 236), (89, 234), (89, 232), (89, 231), (88, 232), (90, 231), (91, 231), (90, 230), (92, 231), (93, 231), (94, 231), (93, 230), (93, 233), (93, 229), (92, 229), (93, 228), (92, 227), (91, 226), (92, 225), (92, 226), (93, 226), (93, 227), (94, 226), (93, 225), (91, 224), (92, 224), (91, 225), (91, 227), (90, 226), (91, 228), (90, 229), (90, 227), (90, 225), (89, 225), (89, 224), (92, 228), (91, 229), (89, 227), (89, 226), (88, 226), (87, 226), (87, 225), (86, 225), (85, 226), (84, 227), (84, 226), (83, 227), (82, 228), (82, 227), (81, 227), (80, 228), (79, 229), (80, 230), (81, 231), (80, 232), (81, 233), (78, 234), (78, 232), (77, 232), (77, 231), (78, 231), (79, 231), (79, 230), (80, 231), (81, 232), (76, 236), (75, 237), (74, 236), (72, 237), (73, 237), (76, 234), (76, 233), (77, 233), (76, 232), (73, 238), (79, 236), (80, 241), (82, 241), (83, 240), (84, 239), (86, 239), (90, 238), (91, 239), (91, 240), (90, 239), (90, 237), (88, 241), (91, 238), (91, 237), (92, 237), (93, 236), (94, 236), (93, 237), (88, 234), (92, 230), (91, 230), (89, 230), (88, 229), (89, 228), (88, 227), (87, 228), (86, 228), (87, 227), (86, 226), (85, 225), (86, 224), (87, 223), (87, 222), (88, 223), (87, 224), (89, 223), (88, 222), (87, 221), (88, 224), (93, 224), (92, 223), (93, 222), (94, 221), (94, 222), (93, 223), (94, 228), (94, 229), (94, 230), (94, 234), (93, 235), (87, 240), (87, 241), (89, 241), (88, 242), (87, 242), (86, 242), (87, 243), (85, 243), (86, 243), (86, 244), (87, 244), (88, 243), (89, 243), (88, 244), (89, 242), (90, 240), (88, 240), (85, 245), (84, 246), (84, 247), (83, 247), (83, 246), (82, 247), (82, 246), (83, 245), (84, 245), (84, 244), (83, 248), (82, 248), (83, 249), (83, 250), (83, 251), (82, 249), (84, 249), (82, 251), (82, 252), (81, 247), (85, 246), (86, 247), (87, 245), (86, 246), (86, 245), (85, 244), (86, 241), (92, 236), (93, 234), (94, 233), (95, 234), (95, 231), (95, 232), (90, 228), (88, 228), (96, 231), (97, 231), (96, 234), (96, 235), (99, 232), (100, 232), (97, 236), (98, 236), (96, 230), (98, 230), (97, 229), (97, 230), (95, 237), (94, 237), (92, 234), (94, 235), (95, 236), (96, 237), (97, 241), (96, 241), (97, 242), (98, 243), (97, 243), (98, 242), (96, 243), (95, 243), (95, 244), (96, 244), (97, 244), (98, 244), (97, 245), (96, 245), (97, 246), (94, 245), (95, 245), (94, 246), (95, 246), (95, 247), (96, 248), (94, 247), (93, 245), (93, 244), (94, 244), (93, 243), (94, 243), (96, 247), (92, 244), (91, 244), (90, 243), (91, 243), (92, 242), (96, 242), (95, 242), (94, 238), (93, 238), (92, 238), (88, 231), (90, 232), (95, 229), (98, 233), (100, 233), (99, 231), (100, 230), (100, 231), (101, 229), (102, 228), (104, 225), (105, 224), (106, 224), (104, 223), (105, 222), (104, 221), (103, 221), (103, 222), (103, 223), (104, 224), (103, 224), (102, 224), (103, 226), (102, 226), (105, 227), (102, 227), (105, 229), (103, 229), (102, 230), (101, 231), (102, 231), (103, 232), (101, 230), (102, 229), (106, 228), (107, 229), (108, 230), (108, 231), (107, 232), (107, 231), (108, 233), (109, 232), (110, 231), (109, 231), (110, 232), (110, 230), (111, 231), (112, 230), (113, 230), (114, 229), (115, 229), (115, 230), (116, 230), (116, 229), (114, 230), (114, 228), (115, 228), (114, 227), (116, 228), (117, 228), (116, 227), (115, 227), (116, 226), (117, 227), (118, 227), (118, 229), (119, 228), (119, 230), (120, 231), (119, 231), (119, 229), (118, 228), (117, 229), (118, 230), (113, 229), (114, 231), (114, 232), (113, 232), (113, 233), (114, 234), (113, 234), (114, 233), (113, 231), (112, 233), (113, 235), (114, 235), (115, 236), (115, 235), (116, 234), (115, 233), (115, 232), (111, 230), (110, 229), (108, 232), (109, 233), (109, 230), (110, 233), (111, 232), (110, 234), (109, 235), (108, 234), (107, 235), (107, 236), (108, 236), (108, 235), (107, 234), (106, 233), (105, 233), (105, 232), (104, 234), (103, 233), (102, 235), (104, 232), (105, 234), (104, 233), (106, 234), (103, 230), (101, 228), (100, 228), (101, 227), (102, 223), (101, 223), (105, 223), (107, 222), (107, 223), (108, 223), (107, 221), (107, 220), (107, 219), (108, 218), (107, 218), (107, 214), (109, 216), (109, 215), (109, 214), (110, 214), (110, 213), (111, 214), (111, 213), (110, 212), (111, 211), (111, 212), (112, 213), (113, 214), (113, 215), (112, 214), (111, 215), (110, 216), (110, 215), (112, 215), (113, 216), (114, 215), (114, 216), (114, 214), (115, 215), (114, 217), (115, 217), (114, 218), (113, 219), (114, 219), (115, 219), (112, 219), (112, 220), (111, 220), (110, 219), (111, 219), (113, 218), (114, 213), (113, 212), (112, 211), (112, 210), (113, 209), (114, 210), (114, 209), (115, 210), (113, 210), (113, 213), (112, 212), (110, 211), (110, 210), (111, 209), (112, 208), (112, 207), (111, 207), (112, 206), (111, 208), (110, 209), (112, 209), (114, 208), (115, 209), (114, 211), (116, 210), (116, 209), (117, 210), (118, 209), (118, 208), (117, 209), (119, 208), (118, 207), (117, 206), (118, 206), (117, 205), (116, 206), (115, 206), (116, 207), (116, 208), (115, 208), (113, 208), (113, 207), (111, 210), (109, 213), (108, 212), (107, 213), (108, 213), (108, 214), (107, 212), (109, 211), (109, 212), (108, 211), (107, 211), (108, 210), (109, 209), (108, 208), (108, 209), (107, 208), (106, 207), (109, 208), (107, 207), (108, 206), (108, 207), (107, 206), (106, 205), (107, 204), (108, 204), (109, 204), (109, 205), (110, 204), (111, 204), (110, 205), (112, 205), (111, 205), (113, 205), (112, 204), (112, 203), (113, 203), (111, 202), (111, 201), (111, 200), (112, 199), (113, 198), (114, 199), (114, 200), (115, 200), (114, 201), (116, 199), (115, 199), (116, 198), (117, 197), (117, 198), (116, 197), (117, 196), (118, 198), (118, 199), (117, 200), (118, 201), (118, 202), (118, 200), (117, 199), (116, 200), (115, 198), (118, 196), (117, 195), (116, 196), (115, 196), (116, 195), (112, 197), (112, 198), (111, 197), (112, 196), (111, 195), (111, 194), (112, 193), (113, 194), (113, 193), (113, 195), (112, 194), (112, 192), (113, 192), (114, 193), (114, 192), (115, 191), (116, 192), (116, 193), (117, 193), (118, 192), (117, 192), (118, 191), (117, 190), (116, 191), (116, 189), (115, 190), (114, 189), (115, 192), (118, 194), (116, 194), (115, 195), (114, 196), (114, 197), (115, 197), (113, 197), (117, 194), (117, 191), (115, 193), (118, 190), (114, 194), (112, 195), (114, 195), (115, 194), (111, 196), (110, 198), (110, 197), (109, 198), (108, 199), (108, 198), (109, 197), (110, 199), (109, 199), (110, 200), (109, 201), (109, 200), (108, 200), (107, 199), (106, 199), (107, 198), (108, 197), (108, 196), (109, 195), (109, 194), (108, 193), (109, 193), (110, 193), (110, 194), (111, 193), (110, 192), (108, 194), (108, 195), (113, 199), (113, 201), (112, 202), (112, 201), (112, 200), (111, 199), (114, 198), (113, 200), (110, 201), (107, 197), (106, 197), (106, 195), (106, 196), (107, 194), (110, 195), (111, 198), (107, 196), (107, 195), (109, 196), (109, 192), (109, 191), (107, 189), (106, 190), (105, 189), (107, 193), (108, 191), (110, 190), (111, 191), (110, 191), (111, 192), (114, 191), (114, 190), (118, 193), (119, 193), (119, 192), (119, 191), (113, 196), (114, 202), (115, 202), (114, 203), (114, 204), (115, 205), (115, 203), (115, 204), (115, 201), (111, 206), (110, 206), (113, 204), (111, 203), (110, 202), (108, 201), (109, 202), (107, 201), (106, 200), (106, 194), (104, 192), (103, 191), (103, 190), (103, 189), (102, 190), (102, 189), (101, 189), (100, 190), (101, 191), (102, 191), (102, 192), (100, 191), (99, 190), (98, 191), (98, 192), (97, 192), (96, 193), (95, 194), (94, 194), (94, 195), (93, 194), (93, 195), (94, 196), (95, 195), (94, 193), (95, 192), (93, 193), (93, 192), (92, 191), (93, 190), (93, 191), (94, 190), (94, 191), (94, 192), (92, 192), (91, 193), (90, 193), (89, 193), (89, 192), (90, 192), (91, 191), (90, 191), (91, 192), (92, 193), (90, 194), (89, 191), (88, 190), (89, 190), (88, 191), (88, 189), (87, 189), (87, 188), (86, 187), (88, 188), (89, 187), (90, 186), (91, 187), (91, 188), (91, 189), (92, 189), (93, 188), (94, 188), (94, 189), (95, 190), (96, 191), (91, 194), (91, 195), (92, 195), (92, 196), (93, 196), (94, 197), (94, 198), (95, 197), (93, 197), (93, 198), (92, 199), (91, 200), (90, 199), (90, 200), (89, 199), (88, 198), (88, 199), (89, 198), (88, 200), (88, 201), (87, 201), (88, 202), (89, 203), (90, 204), (90, 205), (91, 205), (91, 204), (92, 203), (92, 202), (93, 203), (94, 203), (94, 202), (94, 204), (94, 205), (94, 206), (98, 208), (96, 205), (96, 204), (97, 204), (98, 206), (95, 205), (93, 204), (97, 202), (98, 207), (99, 207), (100, 208), (100, 209), (101, 208), (102, 208), (103, 209), (101, 207), (102, 206), (101, 205), (101, 204), (101, 203), (100, 203), (100, 202), (100, 204), (101, 206), (100, 205), (100, 206), (102, 207), (102, 205), (102, 209), (103, 208), (101, 209), (100, 207), (102, 210), (101, 211), (101, 212), (102, 212), (100, 212), (99, 211), (98, 212), (93, 207), (93, 210), (90, 213), (90, 214), (90, 215), (89, 215), (91, 214), (91, 215), (89, 213), (89, 216), (89, 217), (90, 218), (91, 218), (91, 219), (90, 220), (92, 220), (92, 219), (91, 221), (90, 219), (91, 217), (90, 217), (90, 216), (92, 216), (91, 216), (92, 215), (89, 214), (88, 214), (92, 218), (92, 217), (93, 216), (94, 216), (95, 217), (94, 218), (93, 219), (93, 220), (94, 220), (90, 221), (89, 222), (88, 221), (89, 220), (92, 221), (93, 221), (89, 221), (88, 220), (87, 220), (86, 220), (88, 219), (88, 218), (87, 218), (86, 217), (85, 217), (84, 218), (83, 217), (85, 219), (85, 218), (84, 219), (86, 216), (85, 215), (84, 214), (83, 214), (83, 213), (83, 212), (84, 212), (83, 211), (85, 213), (86, 212), (82, 213), (82, 210), (82, 212), (82, 211), (81, 212), (80, 212), (80, 213), (81, 213), (80, 214), (79, 214), (79, 213), (81, 211), (78, 213), (78, 214), (79, 212), (78, 211), (79, 210), (80, 209), (81, 210), (82, 209), (83, 210), (84, 211), (85, 210), (85, 211), (86, 210), (87, 209), (85, 209), (86, 209), (88, 211), (87, 211), (86, 211), (87, 212), (87, 210), (84, 213), (83, 215), (82, 215), (83, 216), (82, 216), (82, 217), (82, 218), (83, 219), (86, 219), (86, 218), (88, 217), (87, 219), (87, 216), (87, 217), (84, 220), (83, 221), (83, 220), (83, 218), (82, 220), (83, 222), (82, 221), (82, 222), (81, 221), (81, 222), (80, 222), (80, 221), (79, 221), (80, 223), (79, 223), (81, 223), (80, 224), (81, 224), (81, 225), (80, 226), (80, 225), (79, 225), (78, 226), (77, 227), (77, 226), (76, 227), (77, 228), (76, 229), (77, 230), (76, 228), (75, 227), (75, 228), (75, 229), (76, 230), (75, 230), (76, 231), (78, 230), (80, 229), (78, 229), (79, 228), (77, 229), (74, 230), (74, 229), (73, 229), (73, 230), (75, 231), (74, 227), (73, 228), (74, 228), (76, 226), (75, 225), (74, 226), (74, 225), (76, 224), (76, 225), (73, 225), (72, 225), (71, 225), (70, 226), (71, 227), (72, 228), (71, 229), (72, 229), (79, 227), (78, 228), (75, 224), (77, 224), (76, 223), (77, 223), (78, 224), (79, 224), (82, 224), (83, 223), (83, 224), (84, 223), (85, 224), (84, 224), (82, 223), (79, 222), (78, 223), (82, 226), (83, 226), (83, 228), (80, 227), (77, 222), (78, 222), (78, 221), (75, 223), (74, 224), (76, 222), (77, 225), (79, 226), (78, 227), (82, 225), (84, 225), (83, 225), (84, 228), (85, 229), (84, 230), (85, 230), (84, 229), (84, 231), (83, 230), (82, 230), (83, 229), (85, 231), (83, 235), (83, 234), (79, 239), (72, 239), (71, 239), (72, 242), (74, 243), (75, 242), (75, 243), (73, 243), (70, 242), (70, 243), (69, 242), (69, 241), (69, 243), (72, 243), (73, 244), (73, 245), (73, 246), (72, 246), (73, 248), (72, 247), (71, 247), (70, 246), (70, 247), (70, 248), (69, 249), (69, 250), (68, 250), (67, 251), (66, 252), (65, 252), (66, 253), (67, 254), (66, 255), (65, 256), (64, 257), (63, 256), (63, 255), (62, 256), (61, 255), (61, 256), (62, 255), (61, 254), (60, 255), (60, 254), (59, 255), (58, 256), (59, 257), (59, 258), (60, 259), (59, 260), (58, 260), (57, 259), (57, 258), (56, 258), (58, 259), (58, 258), (59, 259), (57, 257), (58, 257), (60, 256), (60, 257), (58, 254), (59, 254), (60, 253), (58, 253), (59, 252), (61, 253), (61, 252), (61, 251), (60, 250), (61, 249), (60, 251), (59, 251), (60, 252), (59, 250), (60, 249), (60, 248), (61, 247), (60, 246), (61, 246), (60, 247), (61, 248), (62, 248), (62, 247), (63, 247), (64, 248), (63, 248), (63, 246), (62, 245), (61, 244), (61, 245), (60, 245), (62, 244), (63, 244), (63, 245), (64, 245), (59, 245), (59, 247), (58, 248), (59, 246), (65, 244), (65, 243), (64, 244), (63, 243), (64, 243), (62, 242), (61, 243), (60, 242), (59, 242), (58, 241), (57, 240), (58, 239), (59, 240), (60, 240), (61, 240), (60, 241), (59, 241), (58, 240), (57, 241), (56, 240), (55, 241), (55, 242), (54, 241), (55, 240), (56, 239), (54, 242), (53, 241), (53, 242), (52, 243), (53, 243), (53, 244), (52, 245), (53, 246), (52, 244), (53, 245), (51, 243), (51, 242), (50, 241), (51, 240), (50, 239), (49, 238), (50, 238), (51, 237), (52, 238), (52, 237), (53, 236), (54, 236), (53, 237), (52, 236), (53, 235), (55, 236), (55, 237), (55, 238), (54, 239), (55, 239), (54, 238), (50, 236), (50, 235), (49, 235), (48, 236), (47, 237), (47, 238), (47, 239), (46, 238), (46, 239), (45, 237), (44, 238), (43, 239), (43, 240), (42, 240), (42, 241), (41, 240), (43, 238), (44, 239), (45, 239), (44, 240), (43, 241), (45, 240), (46, 240), (47, 240), (48, 241), (48, 240), (48, 239), (48, 238), (48, 237), (47, 236), (46, 235), (45, 235), (46, 234), (47, 234), (45, 234), (44, 234), (45, 233), (46, 232), (44, 232), (44, 233), (46, 236), (48, 235), (49, 237), (51, 238), (49, 239), (51, 239), (51, 241), (52, 240), (52, 241), (52, 239), (50, 237), (51, 236), (50, 234), (49, 233), (48, 234), (49, 234), (50, 233), (50, 232), (51, 234), (51, 235), (49, 236), (52, 235), (53, 234), (52, 234), (54, 237), (53, 238), (53, 239), (53, 240), (52, 242), (54, 245), (54, 246), (54, 247), (55, 246), (55, 244), (56, 244), (56, 245), (55, 245), (54, 244), (54, 243), (50, 244), (50, 245), (49, 246), (50, 246), (50, 247), (51, 247), (51, 246), (51, 244), (49, 245), (49, 244), (48, 245), (47, 246), (48, 247), (47, 247), (46, 248), (47, 248), (46, 247), (45, 247), (46, 246), (45, 248), (44, 247), (43, 247), (44, 248), (48, 246), (51, 245), (49, 243), (48, 242), (50, 243), (50, 242), (50, 240), (49, 240), (49, 242), (48, 243), (47, 243), (46, 243), (45, 243), (44, 242), (45, 241), (45, 242), (46, 241), (47, 241), (49, 241), (52, 246), (51, 248), (52, 248), (52, 247), (53, 248), (53, 249), (53, 250), (54, 251), (54, 252), (54, 253), (55, 254), (56, 254), (57, 253), (58, 252), (58, 251), (57, 252), (57, 251), (59, 253), (59, 256), (60, 258), (56, 259), (55, 258), (54, 259), (53, 259), (53, 260), (54, 261), (53, 262), (52, 261), (53, 261), (54, 262), (55, 261), (52, 262), (52, 263), (53, 263), (54, 263), (53, 264), (54, 264), (55, 264), (56, 265), (56, 264), (57, 265), (58, 264), (57, 264), (58, 265), (58, 263), (58, 262), (57, 263), (56, 263), (57, 262), (57, 261), (57, 260), (58, 261), (60, 260), (59, 262), (59, 261), (60, 261), (62, 251), (62, 252), (56, 253), (56, 255), (55, 255), (55, 253), (55, 256), (54, 257), (54, 256), (55, 257), (53, 258), (52, 259), (52, 260), (51, 261), (51, 260), (55, 260), (54, 260), (54, 258), (53, 257), (55, 259), (56, 261), (56, 262), (55, 262), (56, 260), (55, 263), (61, 260), (61, 259), (62, 259), (63, 260), (63, 261), (62, 260), (64, 260), (65, 261), (65, 260), (66, 259), (67, 258), (66, 257), (66, 258), (67, 257), (68, 258), (68, 259), (69, 260), (69, 259), (68, 260), (67, 259), (65, 257), (63, 257), (64, 258), (65, 258), (66, 256), (65, 254), (66, 254), (65, 253), (65, 251), (64, 251), (64, 250), (63, 250), (61, 250), (62, 249), (63, 249), (62, 250), (58, 250), (59, 249), (59, 248), (64, 249), (65, 249), (66, 250), (67, 249), (66, 249), (65, 248), (65, 247), (64, 247), (64, 246), (65, 245), (66, 245), (61, 242), (62, 241), (63, 241), (60, 244), (60, 243), (59, 244), (58, 243), (57, 244), (58, 244), (57, 243), (56, 243), (55, 243), (57, 242), (58, 242), (59, 243), (62, 246), (65, 246), (65, 250), (66, 248), (67, 247), (66, 246), (65, 242), (66, 243), (66, 244), (67, 244), (67, 243), (67, 245), (68, 245), (68, 246), (68, 247), (69, 247), (71, 246), (71, 245), (70, 245), (69, 245), (69, 246), (67, 248), (68, 249), (70, 251), (69, 252), (71, 252), (70, 252), (69, 251), (68, 251), (68, 252), (67, 252), (66, 251), (67, 250), (69, 253), (70, 253), (71, 254), (72, 255), (72, 256), (73, 257), (74, 258), (73, 256), (73, 255), (72, 257), (77, 256), (78, 255), (79, 258), (80, 259), (80, 260), (79, 261), (80, 262), (81, 261), (82, 261), (81, 260), (80, 261), (79, 263), (79, 260), (79, 262), (78, 261), (77, 262), (77, 261), (77, 263), (78, 263), (77, 264), (76, 263), (76, 264), (77, 265), (78, 264), (76, 260), (76, 259), (75, 259), (74, 260), (75, 261), (74, 261), (73, 260), (72, 260), (71, 261), (70, 260), (70, 261), (70, 262), (71, 260), (71, 259), (69, 262), (68, 261), (68, 262), (69, 261), (71, 263), (70, 263), (71, 264), (72, 265), (71, 266), (70, 267), (71, 268), (72, 267), (73, 267), (72, 266), (73, 266), (73, 264), (72, 263), (70, 264), (71, 265), (72, 264), (73, 263), (74, 264), (75, 263), (79, 265), (79, 266), (78, 267), (77, 266), (77, 267), (76, 268), (75, 267), (76, 266), (75, 265), (75, 266), (76, 265), (76, 262), (74, 262), (74, 263), (75, 262), (76, 261), (73, 262), (73, 261), (74, 259), (75, 260), (79, 264), (80, 263), (80, 264), (80, 265), (78, 265), (77, 268), (76, 267), (73, 265), (74, 265), (78, 266), (78, 268), (79, 267), (80, 267), (79, 268), (80, 268), (81, 268), (80, 266), (81, 266), (82, 265), (83, 264), (84, 264), (84, 265), (84, 266), (85, 266), (85, 267), (86, 265), (87, 266), (88, 267), (87, 267), (88, 266), (88, 265), (87, 265), (86, 267), (83, 266), (83, 265), (82, 264), (83, 263), (85, 265), (86, 266), (85, 264), (82, 267), (83, 267), (82, 268), (83, 268), (81, 267), (81, 265), (82, 266), (86, 263), (87, 263), (86, 262), (87, 261), (88, 258), (89, 257), (88, 256), (87, 256), (89, 255), (88, 255), (87, 255), (88, 257), (87, 257), (87, 258), (86, 256), (87, 254), (86, 254), (87, 253), (88, 253), (88, 254), (89, 254), (90, 255), (90, 256), (89, 256), (90, 253), (91, 254), (92, 253), (91, 252), (92, 252), (90, 252), (89, 252), (91, 253), (92, 254), (93, 253), (94, 253), (95, 253), (95, 254), (93, 252), (92, 251), (91, 251), (89, 251), (89, 250), (88, 251), (88, 252), (89, 253), (90, 254), (92, 255), (95, 255), (96, 255), (97, 256), (97, 254), (97, 253), (98, 252), (98, 251), (98, 250), (99, 251), (99, 252), (99, 249), (98, 248), (99, 247), (98, 246), (98, 245), (96, 246), (97, 247), (99, 242), (99, 246), (100, 246), (98, 247), (99, 248), (98, 249), (97, 249), (95, 248), (95, 249), (96, 250), (96, 249), (97, 248), (97, 250), (100, 247), (100, 253), (101, 254), (102, 256), (99, 260), (100, 261), (101, 260), (99, 258), (102, 259), (101, 261), (102, 262), (102, 263), (103, 262), (103, 261), (102, 261), (103, 260), (104, 260), (105, 259), (106, 258), (105, 258), (106, 259), (107, 258), (108, 259), (108, 260), (107, 260), (107, 259), (105, 257), (104, 258), (103, 257), (103, 258), (104, 259), (106, 260), (107, 261), (106, 261), (105, 260), (104, 261), (105, 261), (104, 257), (106, 257), (107, 257), (108, 258), (109, 258), (109, 257), (109, 259), (109, 256), (108, 256), (107, 255), (106, 254), (107, 253), (107, 254), (108, 253), (109, 252), (110, 253), (111, 253), (110, 254), (111, 254), (112, 253), (112, 252), (113, 251), (114, 251), (115, 251), (115, 252), (115, 253), (116, 252), (116, 253), (117, 252), (115, 254), (114, 254), (114, 255), (115, 255), (116, 254), (116, 255), (113, 255), (113, 254), (114, 253), (112, 254), (113, 253), (112, 255), (114, 252), (117, 255), (117, 256), (116, 256), (116, 257), (113, 256), (114, 256), (114, 257), (112, 257), (114, 258), (115, 259), (114, 259), (115, 260), (115, 258), (116, 259), (117, 260), (116, 260), (117, 259), (118, 258), (118, 257), (119, 256), (118, 256), (119, 255), (118, 255), (119, 257), (117, 258), (114, 260), (114, 261), (113, 261), (113, 260), (113, 259), (112, 259), (113, 258), (112, 258), (111, 259), (111, 260), (110, 261), (109, 262), (109, 261), (108, 261), (109, 260), (108, 262), (107, 262), (107, 263), (108, 263), (107, 264), (106, 264), (110, 260), (110, 262), (110, 258), (111, 258), (110, 257), (111, 256), (111, 257), (112, 256), (111, 255), (113, 257), (112, 260), (110, 259), (108, 257), (105, 256), (106, 255), (104, 254), (103, 253), (101, 253), (97, 251), (94, 248), (93, 248), (93, 249), (94, 250), (94, 251), (93, 251), (87, 252), (88, 250), (90, 250), (91, 250), (93, 250), (92, 249), (92, 248), (91, 249), (94, 249), (94, 242), (92, 243), (90, 244), (91, 245), (90, 245), (93, 242), (92, 245), (92, 246), (93, 246), (92, 241), (89, 244), (89, 245), (91, 247), (92, 247), (91, 246), (93, 247), (98, 241), (99, 238), (100, 239), (101, 242), (99, 241), (101, 244), (102, 245), (103, 246), (104, 246), (105, 247), (104, 248), (103, 247), (102, 246), (103, 245), (103, 248), (106, 249), (107, 248), (108, 249), (109, 249), (110, 250), (108, 250), (107, 244), (108, 244), (105, 244), (104, 245), (104, 244), (104, 243), (103, 249), (104, 249), (102, 249), (101, 249), (101, 248), (100, 248), (96, 252), (96, 253), (95, 252), (95, 251), (95, 250), (101, 247), (101, 246), (102, 247), (107, 250), (109, 250), (110, 249), (110, 247), (111, 248), (112, 247), (111, 247), (112, 248), (111, 249), (111, 251), (112, 250), (113, 250), (114, 249), (115, 248), (115, 247), (115, 246), (115, 245), (114, 244), (115, 243), (116, 242), (117, 243), (116, 244), (117, 245), (117, 246), (118, 246), (119, 246), (120, 247), (120, 246), (119, 247), (119, 248), (121, 247), (120, 248), (121, 249), (122, 250), (123, 249), (122, 249), (121, 250), (120, 251), (119, 252), (119, 250), (118, 250), (119, 251), (117, 249), (117, 248), (118, 248), (116, 250), (117, 251), (117, 250), (118, 252), (120, 252), (120, 249), (122, 251), (123, 251), (122, 252), (121, 252), (121, 253), (120, 253), (120, 254), (121, 254), (122, 255), (122, 254), (119, 253), (119, 258), (120, 257), (120, 256), (121, 257), (122, 258), (123, 258), (121, 258), (122, 257), (123, 256), (122, 253), (123, 253), (124, 253), (124, 252), (125, 253), (124, 254), (123, 255), (125, 255), (126, 256), (126, 257), (125, 257), (124, 258), (125, 258), (127, 258), (126, 258), (125, 259), (124, 260), (125, 260), (124, 261), (123, 261), (123, 262), (124, 263), (123, 263), (124, 264), (124, 262), (125, 263), (125, 262), (126, 261), (126, 260), (125, 261), (124, 259), (123, 260), (126, 262), (126, 263), (127, 262), (128, 262), (128, 261), (129, 262), (129, 263), (130, 262), (130, 261), (131, 261), (132, 260), (132, 259), (133, 258), (134, 259), (134, 260), (134, 257), (134, 258), (133, 259), (133, 256), (134, 255), (134, 254), (133, 255), (134, 256), (133, 257), (132, 257), (132, 256), (132, 255), (133, 254), (133, 260), (132, 258), (131, 258), (130, 259), (131, 260), (132, 261), (131, 259), (131, 256), (131, 257), (133, 261), (130, 260), (132, 262), (134, 261), (133, 263), (134, 262), (133, 262), (134, 264), (133, 265), (133, 266), (134, 265), (133, 264), (134, 263), (131, 263), (130, 264), (130, 265), (129, 266), (128, 265), (128, 264), (129, 264), (128, 263), (127, 263), (127, 261), (125, 264), (123, 264), (122, 265), (123, 266), (122, 266), (121, 265), (120, 266), (119, 266), (120, 265), (121, 264), (120, 263), (120, 262), (119, 263), (118, 264), (117, 265), (116, 265), (116, 264), (115, 263), (116, 263), (114, 263), (113, 264), (113, 263), (112, 263), (112, 262), (112, 264), (111, 265), (111, 266), (110, 265), (110, 264), (110, 263), (111, 264), (111, 263), (109, 263), (108, 264), (109, 264), (109, 265), (108, 265), (109, 266), (110, 266), (112, 266), (113, 267), (112, 267), (113, 266), (111, 267), (110, 267), (109, 267), (108, 266), (110, 268), (109, 268), (111, 268), (114, 268), (114, 267), (115, 268), (112, 265), (113, 268), (114, 266), (116, 268), (116, 267), (115, 267), (112, 268), (113, 265), (114, 262), (115, 261), (118, 261), (118, 262), (117, 261), (118, 259), (119, 259), (118, 260), (121, 256), (122, 256), (121, 255), (120, 255), (119, 254), (118, 254), (119, 249), (121, 248), (123, 250), (123, 252), (125, 252), (126, 252), (127, 252), (126, 251), (127, 251), (128, 251), (129, 250), (128, 249), (129, 248), (130, 247), (129, 247), (130, 248), (128, 250), (127, 250), (126, 249), (127, 248), (126, 248), (126, 250), (125, 249), (124, 250), (124, 248), (125, 248), (123, 248), (122, 248), (116, 246), (116, 245), (113, 245), (113, 244), (113, 243), (113, 242), (114, 242), (114, 243), (115, 242), (116, 241), (115, 240), (114, 241), (114, 240), (115, 241), (112, 241), (111, 240), (110, 240), (110, 241), (111, 242), (110, 243), (111, 244), (110, 245), (110, 246), (112, 246), (112, 245), (112, 243), (111, 243), (110, 242), (109, 242), (109, 239), (109, 238), (110, 239), (111, 239), (112, 240), (113, 241), (113, 240), (115, 239), (114, 238), (114, 237), (116, 235), (117, 235), (115, 234), (114, 236), (112, 236), (113, 237), (111, 235), (110, 236), (109, 236), (109, 237), (110, 237), (110, 238), (109, 240), (108, 240), (109, 241), (109, 243), (108, 243), (110, 244), (111, 246), (112, 242), (111, 241), (112, 244), (108, 242), (107, 241), (107, 240), (106, 240), (106, 241), (105, 240), (105, 239), (104, 241), (102, 239), (102, 240), (103, 242), (102, 238), (101, 238), (99, 243), (91, 248), (90, 248), (89, 248), (92, 250), (102, 248), (99, 245), (99, 244), (100, 245), (100, 244), (89, 246), (90, 246), (88, 245), (87, 246), (84, 243), (83, 242), (84, 242), (85, 241), (84, 240), (84, 241), (80, 240), (81, 242), (76, 243), (76, 247), (81, 246), (72, 249), (72, 248), (72, 244), (72, 245), (71, 248), (70, 249), (69, 248), (62, 243), (58, 247), (58, 246), (57, 246), (56, 246), (58, 249), (57, 250), (56, 249), (55, 249), (54, 248), (53, 247), (49, 247), (50, 248), (49, 249), (49, 248), (48, 249), (48, 250), (49, 250), (49, 251), (50, 250), (48, 252), (49, 252), (50, 252), (51, 253), (52, 253), (53, 252), (55, 252), (55, 251), (56, 250), (61, 257), (61, 258), (63, 258), (63, 259), (62, 258), (64, 261), (65, 262), (66, 261), (66, 260), (67, 261), (68, 257), (69, 257), (68, 256), (67, 255), (69, 258), (69, 256), (68, 255), (69, 254), (70, 250), (71, 253), (72, 254), (72, 252), (71, 251), (74, 251), (75, 251), (78, 257), (75, 258), (74, 257), (71, 255), (75, 226), (73, 227), (72, 226), (71, 226), (70, 225), (72, 227), (71, 228), (70, 227), (69, 226), (71, 224), (70, 224), (70, 223), (69, 224), (72, 224), (73, 224), (74, 222), (73, 222), (74, 221), (73, 221), (72, 220), (72, 221), (72, 222), (74, 220), (73, 220), (75, 220), (75, 219), (76, 219), (75, 218), (74, 218), (75, 217), (74, 217), (74, 219), (76, 221), (76, 220), (75, 221), (77, 219), (76, 218), (75, 216), (76, 216), (76, 217), (77, 215), (77, 214), (77, 213), (76, 213), (75, 212), (75, 213), (76, 212), (76, 211), (77, 212), (80, 215), (81, 215), (81, 216), (81, 217), (81, 218), (80, 217), (79, 218), (78, 217), (78, 216), (78, 215), (76, 215), (76, 214), (75, 214), (74, 214), (74, 215), (73, 216), (72, 216), (72, 215), (73, 215), (71, 215), (71, 214), (70, 214), (70, 215), (69, 215), (70, 216), (68, 214), (69, 213), (68, 212), (68, 211), (69, 212), (70, 211), (71, 212), (71, 211), (69, 210), (70, 210), (69, 211), (68, 210), (69, 209), (70, 209), (71, 210), (68, 213), (67, 212), (67, 214), (66, 215), (69, 214), (70, 212), (72, 212), (72, 210), (72, 209), (73, 210), (74, 210), (73, 209), (74, 209), (75, 208), (75, 207), (76, 207), (74, 206), (73, 205), (72, 205), (72, 206), (72, 207), (73, 208), (72, 208), (73, 207), (73, 206), (71, 204), (72, 203), (71, 203), (71, 202), (70, 202), (70, 201), (70, 200), (69, 200), (69, 199), (68, 201), (69, 201), (67, 200), (66, 199), (67, 199), (66, 198), (67, 198), (68, 200), (71, 201), (72, 200), (73, 201), (71, 200), (73, 199), (74, 200), (74, 201), (73, 200), (75, 200), (75, 199), (76, 199), (76, 201), (77, 201), (75, 202), (75, 203), (76, 202), (74, 202), (75, 201), (76, 200), (74, 198), (72, 199), (73, 198), (74, 197), (74, 199), (75, 197), (74, 196), (73, 196), (72, 195), (73, 195), (75, 196), (74, 195), (75, 195), (76, 196), (76, 195), (76, 194), (77, 195), (78, 195), (77, 194), (77, 193), (76, 193), (76, 192), (77, 191), (76, 190), (75, 190), (74, 190), (75, 191), (77, 190), (75, 189), (74, 189), (73, 189), (72, 188), (72, 187), (73, 187), (74, 187), (74, 188), (73, 188), (72, 189), (73, 190), (76, 191), (75, 192), (75, 193), (74, 192), (78, 192), (77, 192), (78, 191), (78, 193), (78, 194), (76, 189), (77, 189), (78, 189), (79, 190), (79, 189), (79, 188), (80, 187), (81, 186), (82, 186), (83, 185), (84, 185), (85, 185), (86, 186), (87, 186), (88, 187), (89, 186), (90, 185), (90, 184), (89, 184), (88, 183), (89, 183), (89, 182), (90, 183), (90, 182), (91, 182), (87, 183), (87, 184), (86, 184), (86, 183), (86, 182), (86, 181), (85, 181), (85, 182), (85, 183), (84, 184), (84, 186), (83, 186), (82, 185), (83, 184), (82, 184), (83, 183), (85, 184), (84, 183), (83, 182), (84, 182), (84, 181), (82, 183), (81, 184), (81, 185), (80, 186), (79, 185), (79, 186), (78, 186), (78, 187), (77, 188), (79, 187), (80, 188), (78, 190), (78, 188), (80, 189), (80, 185), (81, 187), (80, 184), (80, 183), (78, 185), (77, 185), (76, 184), (76, 183), (75, 184), (77, 184), (78, 184), (79, 184), (81, 183), (80, 182), (79, 183), (79, 182), (78, 182), (79, 181), (80, 181), (81, 180), (81, 181), (82, 182), (83, 181), (82, 181), (84, 180), (83, 180), (83, 179), (82, 178), (83, 177), (84, 176), (83, 175), (83, 174), (82, 174), (83, 176), (82, 175), (81, 175), (81, 174), (81, 173), (80, 172), (79, 172), (80, 173), (79, 174), (78, 174), (79, 173), (80, 175), (79, 176), (78, 177), (77, 178), (78, 176), (79, 177), (79, 178), (78, 178), (79, 179), (78, 179), (79, 175), (78, 175), (78, 173), (79, 171), (80, 171), (81, 172), (78, 171), (77, 172), (77, 173), (78, 172), (77, 174), (77, 175), (76, 174), (76, 176), (76, 175), (80, 174), (77, 176), (77, 177), (76, 177), (75, 178), (74, 179), (73, 178), (73, 177), (72, 178), (75, 180), (74, 181), (75, 181), (74, 182), (73, 181), (74, 180), (73, 180), (76, 179), (77, 180), (75, 175), (74, 174), (75, 174), (76, 173), (76, 172), (74, 173), (73, 174), (72, 174), (72, 175), (71, 176), (70, 175), (69, 176), (68, 175), (70, 176), (69, 175), (71, 174), (70, 174), (71, 175), (72, 176), (73, 175), (73, 173), (72, 172), (72, 171), (71, 172), (70, 172), (70, 171), (69, 172), (70, 170), (71, 171), (71, 170), (68, 173), (67, 174), (67, 173), (68, 174), (67, 175), (67, 176), (67, 177), (66, 178), (65, 177), (64, 178), (65, 178), (66, 177), (64, 176), (65, 176), (66, 175), (66, 174), (67, 178), (66, 179), (67, 179), (68, 179), (69, 179), (69, 178), (69, 177), (68, 177), (68, 178), (67, 180), (66, 181), (66, 182), (67, 181), (68, 181), (68, 180), (65, 180), (64, 179), (65, 181), (64, 180), (63, 179), (62, 178), (61, 179), (61, 180), (60, 178), (60, 179), (61, 178), (62, 179), (63, 180), (63, 177), (62, 176), (63, 175), (64, 174), (63, 174), (64, 173), (65, 174), (70, 179), (70, 178), (71, 177), (66, 176), (68, 176), (64, 175), (62, 175), (62, 174), (63, 176), (61, 174), (61, 175), (60, 174), (59, 173), (58, 172), (59, 171), (60, 172), (59, 174), (59, 175), (58, 175), (58, 174), (60, 173), (59, 172), (58, 171), (57, 171), (57, 173), (57, 172), (56, 172), (56, 173), (55, 171), (55, 172), (55, 173), (54, 174), (53, 174), (53, 175), (54, 175), (55, 176), (55, 177), (54, 177), (53, 177), (52, 178), (53, 176), (52, 176), (51, 176), (50, 177), (49, 176), (48, 175), (47, 176), (46, 175), (46, 174), (45, 175), (44, 174), (43, 174), (44, 175), (44, 176), (43, 175), (43, 173), (43, 172), (43, 171), (44, 170), (44, 171), (45, 170), (46, 170), (47, 171), (48, 172), (48, 171), (49, 172), (50, 171), (50, 170), (49, 170), (49, 171), (48, 173), (47, 172), (48, 170), (49, 169), (51, 171), (51, 172), (51, 173), (52, 174), (54, 176), (55, 175), (55, 174), (54, 173), (53, 173), (52, 173), (52, 175), (51, 175), (51, 174), (50, 173), (50, 172), (49, 173), (50, 174), (50, 175), (49, 174), (48, 174), (47, 175), (48, 176), (48, 177), (49, 178), (49, 175), (47, 170), (47, 169), (48, 169), (49, 168), (48, 168), (47, 168), (46, 168), (45, 167), (45, 168), (44, 169), (44, 168), (43, 169), (43, 168), (43, 167), (43, 166), (42, 166), (44, 167), (45, 169), (44, 166), (45, 171), (49, 167), (49, 166), (50, 165), (50, 166), (51, 167), (52, 167), (52, 168), (51, 169), (51, 168), (50, 168), (50, 167), (51, 166), (53, 166), (53, 165), (52, 165), (54, 166), (55, 166), (54, 165), (55, 164), (56, 165), (56, 166), (57, 167), (58, 166), (57, 165), (56, 164), (56, 163), (57, 162), (58, 163), (59, 162), (60, 163), (59, 164), (60, 164), (59, 163), (60, 162), (61, 161), (62, 162), (63, 162), (64, 162), (63, 163), (63, 164), (64, 164), (65, 163), (64, 163), (63, 161), (62, 161), (61, 160), (60, 159), (60, 160), (59, 160), (61, 159), (62, 160), (63, 159), (63, 160), (64, 159), (65, 160), (66, 161), (65, 162), (66, 163), (66, 162), (67, 163), (68, 162), (69, 162), (69, 163), (70, 163), (68, 163), (67, 162), (68, 164), (67, 165), (67, 164), (68, 165), (69, 165), (69, 166), (70, 167), (70, 166), (68, 166), (67, 167), (70, 165), (69, 167), (68, 167), (67, 168), (66, 168), (65, 169), (64, 168), (64, 169), (63, 169), (62, 168), (61, 167), (60, 167), (60, 168), (61, 169), (61, 170), (61, 171), (60, 171), (62, 172), (62, 173), (63, 173), (63, 172), (63, 171), (64, 171), (65, 170), (65, 171), (64, 172), (65, 172), (66, 172), (67, 172), (67, 171), (68, 171), (68, 172), (69, 173), (70, 177), (71, 178), (72, 177), (73, 176), (74, 176), (75, 177), (74, 178), (73, 179), (75, 179), (76, 180), (77, 179), (78, 180), (78, 181), (79, 180), (80, 180), (77, 182), (78, 183), (77, 181), (76, 185), (76, 186), (76, 187), (77, 186), (77, 187), (75, 186), (75, 187), (76, 188), (75, 188), (71, 187), (70, 186), (70, 187), (69, 188), (69, 189), (68, 188), (67, 189), (68, 189), (67, 190), (66, 191), (65, 190), (64, 191), (63, 191), (65, 189), (65, 188), (64, 188), (65, 187), (66, 186), (65, 185), (66, 184), (65, 184), (64, 184), (64, 183), (63, 182), (64, 181), (63, 178), (64, 177), (65, 175), (65, 173), (61, 173), (57, 170), (56, 169), (56, 168), (57, 168), (58, 167), (59, 168), (60, 169), (60, 170), (57, 169), (58, 168), (59, 169), (59, 167), (59, 166), (59, 165), (61, 164), (61, 165), (60, 165), (60, 166), (58, 164), (56, 162), (57, 163), (55, 163), (55, 165), (57, 164), (58, 165), (58, 162), (59, 161), (58, 161), (62, 164), (57, 166), (58, 169), (56, 170), (54, 171), (55, 170), (51, 177), (52, 179), (53, 180), (52, 180), (53, 181), (54, 180), (53, 179), (53, 178), (54, 178), (52, 177), (50, 178), (49, 179), (50, 180), (50, 181), (49, 180), (50, 179), (51, 179), (49, 181), (50, 182), (51, 181), (52, 181), (51, 180), (51, 178), (50, 176), (46, 176), (47, 174), (46, 173), (45, 174), (49, 177), (47, 173), (46, 172), (46, 171), (47, 167), (46, 167), (46, 166), (45, 165), (45, 166), (46, 165), (45, 164), (46, 163), (44, 163), (44, 162), (45, 161), (46, 161), (46, 162), (45, 162), (44, 161), (43, 162), (42, 162), (41, 162), (42, 163), (43, 164), (43, 165), (42, 165), (44, 164), (43, 163), (43, 160), (43, 159), (43, 158), (42, 158), (41, 159), (41, 158), (42, 159), (44, 160), (45, 160), (45, 159), (46, 159), (46, 160), (47, 160), (44, 159), (42, 157), (41, 156), (40, 156), (41, 155), (41, 157), (40, 157), (43, 157), (44, 157), (45, 156), (44, 155), (44, 156), (45, 154), (44, 154), (43, 154), (42, 153), (41, 153), (43, 153), (44, 153), (45, 152), (46, 152), (46, 153), (46, 154), (47, 154), (48, 153), (49, 154), (48, 154), (48, 155), (48, 156), (47, 155), (50, 154), (50, 155), (51, 156), (52, 157), (53, 157), (52, 156), (53, 155), (52, 155), (51, 155), (52, 154), (52, 153), (50, 156), (49, 155), (47, 156), (46, 155), (43, 156), (41, 160), (40, 159), (39, 158), (40, 158), (42, 160), (41, 161), (42, 161), (43, 161), (42, 164), (41, 165), (41, 164), (40, 163), (39, 164), (39, 165), (39, 166), (40, 166), (40, 164), (40, 162), (39, 163), (41, 163), (40, 165), (38, 165), (37, 165), (37, 166), (36, 166), (36, 165), (35, 166), (35, 165), (34, 167), (33, 167), (34, 166), (34, 165), (34, 164), (33, 165), (32, 164), (33, 164), (32, 165), (31, 165), (30, 165), (29, 164), (29, 165), (28, 165), (28, 164), (27, 163), (28, 163), (29, 166), (30, 166), (28, 166), (27, 164), (30, 164), (27, 166), (27, 165), (28, 167), (29, 168), (28, 168), (29, 167), (30, 163), (30, 162), (31, 161), (32, 161), (31, 162), (32, 162), (31, 163), (31, 166), (30, 168), (30, 169), (29, 170), (30, 171), (28, 169), (29, 169), (27, 169), (27, 170), (26, 169), (26, 168), (27, 168), (25, 168), (26, 167), (26, 166), (27, 167), (31, 169), (30, 167), (26, 165), (26, 164), (25, 164), (26, 163), (27, 162), (26, 162), (25, 162), (25, 163), (24, 163), (23, 163), (22, 163), (22, 162), (21, 163), (20, 164), (21, 164), (19, 165), (20, 166), (21, 166), (20, 167), (19, 166), (20, 165), (21, 162), (23, 164), (22, 165), (23, 166), (22, 166), (21, 167), (22, 167), (21, 168), (23, 167), (24, 166), (25, 166), (24, 167), (24, 168), (25, 169), (25, 167), (25, 165), (29, 163), (29, 162), (28, 161), (27, 160), (26, 160), (25, 161), (25, 160), (24, 159), (24, 158), (23, 160), (22, 160), (21, 161), (20, 161), (19, 162), (18, 163), (18, 164), (17, 163), (16, 162), (15, 162), (15, 161), (16, 161), (17, 162), (18, 161), (19, 161), (19, 163), (21, 165), (22, 164), (23, 165), (19, 164), (17, 164), (17, 165), (16, 165), (16, 164), (16, 163), (15, 163), (16, 160), (17, 161), (18, 162), (15, 164), (15, 165), (16, 166), (15, 167), (14, 168), (16, 167), (16, 168), (16, 169), (15, 170), (14, 169), (13, 169), (15, 169), (16, 170), (17, 171), (16, 171), (16, 172), (16, 173), (15, 174), (14, 173), (14, 172), (14, 171), (14, 170), (13, 170), (15, 173), (16, 174), (15, 175), (14, 176), (14, 177), (13, 177), (12, 176), (12, 178), (11, 178), (10, 178), (10, 177), (9, 179), (9, 180), (10, 180), (10, 179), (12, 177), (13, 176), (14, 178), (15, 177), (15, 178), (16, 177), (17, 178), (17, 179), (18, 179), (19, 180), (19, 181), (18, 182), (17, 183), (17, 184), (16, 183), (16, 184), (15, 185), (16, 186), (14, 185), (15, 186), (15, 187), (14, 188), (14, 189), (13, 189), (14, 190), (12, 188), (12, 190), (11, 190), (11, 189), (10, 188), (11, 187), (12, 186), (11, 185), (10, 185), (9, 185), (10, 186), (10, 184), (9, 186), (10, 187), (8, 184), (8, 185), (7, 183), (8, 183), (7, 182), (8, 182), (6, 182), (5, 181), (5, 182), (6, 183), (5, 183), (6, 181), (7, 181), (6, 180), (7, 179), (6, 179), (5, 178), (5, 177), (6, 177), (5, 176), (5, 175), (6, 175), (6, 174), (6, 178), (5, 179), (5, 180), (8, 180), (9, 182), (10, 183), (9, 183), (7, 180), (8, 178), (9, 177), (8, 177), (9, 176), (8, 176), (7, 177), (7, 178), (6, 176), (6, 184), (5, 185), (6, 185), (7, 185), (8, 186), (9, 184), (7, 184), (9, 181), (10, 182), (10, 181), (11, 182), (11, 181), (12, 182), (13, 181), (14, 181), (15, 180), (15, 181), (16, 182), (15, 182), (14, 183), (14, 182), (13, 180), (12, 181), (12, 180), (13, 182), (15, 184), (16, 185), (16, 187), (17, 187), (18, 188), (19, 188), (18, 189), (17, 189), (16, 188), (16, 189), (17, 188), (16, 190), (17, 191), (17, 192), (17, 193), (16, 192), (15, 191), (14, 192), (18, 193), (19, 194), (19, 195), (18, 195), (17, 196), (16, 197), (17, 198), (16, 198), (17, 197), (18, 199), (18, 200), (19, 201), (18, 201), (17, 200), (16, 200), (15, 199), (14, 200), (13, 199), (14, 198), (13, 198), (14, 197), (14, 196), (13, 197), (12, 198), (11, 197), (12, 197), (11, 196), (10, 197), (11, 198), (12, 200), (11, 200), (11, 199), (10, 199), (9, 199), (9, 200), (10, 200), (11, 201), (10, 202), (10, 201), (9, 203), (10, 204), (11, 204), (12, 205), (13, 206), (14, 206), (15, 206), (16, 205), (17, 206), (15, 204), (14, 205), (13, 207), (12, 208), (13, 205), (12, 207), (11, 207), (12, 206), (13, 208), (13, 209), (13, 210), (13, 211), (14, 210), (15, 211), (15, 210), (14, 209), (15, 208), (14, 207), (14, 208), (11, 205), (12, 204), (13, 204), (14, 204), (10, 205), (8, 203), (8, 202), (8, 201), (11, 202), (10, 203), (11, 203), (15, 205), (16, 204), (16, 203), (11, 206), (10, 206), (9, 207), (8, 208), (7, 208), (6, 209), (6, 210), (5, 209), (5, 208), (6, 208), (7, 209), (6, 211), (7, 211), (8, 210), (7, 210), (8, 211), (8, 209), (9, 209), (5, 207), (6, 207), (7, 207), (6, 206), (5, 206), (5, 205), (6, 205), (7, 206), (8, 205), (8, 206), (8, 207), (7, 204), (7, 203), (6, 203), (6, 202), (7, 202), (6, 201), (5, 201), (5, 202), (6, 200), (6, 199), (7, 200), (7, 199), (8, 199), (5, 198), (6, 198), (7, 197), (8, 197), (8, 196), (8, 195), (7, 196), (6, 197), (5, 197), (7, 201), (5, 203), (6, 204), (7, 205), (9, 210), (10, 209), (7, 212), (10, 208), (11, 208), (11, 209), (10, 210), (9, 211), (11, 210), (10, 211), (11, 211), (12, 212), (12, 211), (12, 213), (11, 212), (9, 212), (10, 212), (8, 212), (8, 213), (7, 213), (6, 214), (7, 215), (7, 214), (5, 213), (5, 212), (5, 214), (5, 215), (6, 215), (6, 216), (7, 216), (6, 217), (8, 216), (8, 215), (8, 214), (9, 215), (9, 216), (10, 214), (9, 213), (10, 213), (9, 214), (11, 214), (13, 212), (13, 214), (14, 213), (12, 214), (11, 213), (12, 210), (9, 208), (9, 206), (9, 202), (9, 201), (9, 204), (5, 204), (9, 205), (10, 207), (12, 202), (13, 203), (13, 202), (14, 203), (12, 203), (15, 203), (12, 201), (13, 200), (14, 199), (15, 200), (16, 201), (15, 201), (14, 202), (13, 201), (14, 201), (17, 203), (17, 205), (18, 205), (19, 206), (18, 207), (19, 208), (20, 208), (19, 209), (18, 208), (17, 208), (16, 207), (9, 217), (9, 218), (8, 218), (8, 219), (9, 220), (8, 220), (9, 219), (10, 220), (9, 221), (10, 218), (11, 219), (11, 218), (12, 218), (12, 219), (13, 220), (14, 220), (14, 219), (15, 218), (16, 219), (17, 220), (18, 221), (19, 221), (20, 222), (19, 222), (20, 221), (21, 221), (20, 220), (19, 220), (18, 219), (18, 220), (17, 221), (16, 220), (17, 218), (17, 219), (19, 218), (20, 218), (20, 219), (21, 220), (19, 219), (21, 219), (22, 220), (23, 221), (22, 222), (23, 222), (22, 221), (19, 223), (19, 224), (18, 223), (19, 217), (18, 217), (20, 216), (20, 217), (21, 216), (20, 215), (21, 214), (20, 213), (21, 212), (22, 213), (20, 212), (21, 213), (22, 214), (21, 215), (22, 216), (21, 217), (22, 218), (22, 219), (22, 223), (24, 221), (23, 220), (21, 222), (23, 219), (23, 218), (24, 218), (25, 217), (24, 216), (23, 215), (24, 215), (22, 215), (23, 216), (23, 217), (22, 217), (22, 211), (23, 212), (24, 213), (23, 213), (19, 216), (18, 216), (25, 216), (25, 214), (26, 215), (27, 214), (25, 215), (24, 217), (21, 218), (19, 211), (19, 210), (20, 207), (18, 209), (17, 207), (16, 206), (19, 207), (20, 209), (20, 210), (21, 209), (21, 207), (21, 208), (22, 209), (23, 210), (21, 210), (22, 212), (24, 212), (25, 212), (25, 213), (23, 214), (24, 214), (26, 211), (27, 212), (27, 211), (28, 212), (29, 212), (28, 211), (29, 211), (30, 210), (29, 209), (28, 210), (28, 209), (29, 208), (30, 207), (31, 208), (32, 209), (31, 209), (32, 210), (33, 211), (34, 210), (32, 211), (31, 211), (29, 210), (28, 213), (30, 212), (30, 211), (32, 212), (31, 212), (30, 2